In [1]:
import os
import pandas as pd, numpy as np
from sklearn.preprocessing import LabelEncoder
from IPython.display import display, Math, Latex
import torch
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from datasets import Dataset
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import average_precision_score

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

VER=1
#model_name = "jhu-clsp/ettin-encoder-1b"
model_name = "/kaggle/input/gemma2-9b-it-cv945"
EPOCHS = 3

tokenizer = AutoTokenizer.from_pretrained(model_name)
MAX_LEN = 512

DIR = f"ver_{VER}"
os.makedirs(DIR, exist_ok=True)

le = LabelEncoder()
train = pd.read_csv('/kaggle/input/map-charting-student-math-misunderstandings/train.csv')
train.Misconception = train.Misconception.fillna('NA')
train['target'] = train.Category+":"+train.Misconception
train['label'] = le.fit_transform(train['target'])
n_classes = len(le.classes_)
print(f"Train shape: {train.shape} with {n_classes} target classes")
print(train.head())

idx = train.apply(lambda row: row.Category.split('_')[0],axis=1)=='True'
correct = train.loc[idx].copy()
correct['c'] = correct.groupby(['QuestionId','MC_Answer']).MC_Answer.transform('count')
correct = correct.sort_values('c',ascending=False)
correct = correct.drop_duplicates(['QuestionId'])
correct = correct[['QuestionId','MC_Answer']]
correct['is_correct'] = 1

train = train.merge(correct, on=['QuestionId','MC_Answer'], how='left')
train.is_correct = train.is_correct.fillna(0)

tmp = train.groupby(['QuestionId','MC_Answer']).size().reset_index(name='count')
tmp['rank'] = tmp.groupby('QuestionId')['count'].rank(method='dense', ascending=False).astype(int) - 1
tmp = tmp.drop('count',axis=1)
tmp = tmp.sort_values(['QuestionId','rank'])

Q = tmp.QuestionId.unique()
for q in Q:
    question = train.loc[train.QuestionId==q].iloc[0].QuestionText
    choices = tmp.loc[tmp.QuestionId==q].MC_Answer.values
    labels="ABCD"
    choice_str = " ".join([f"({labels[i]}) {choice}" for i, choice in enumerate(choices)])
    
    print()
    display(Latex(f"QuestionId {q}: {question}") )
    display(Latex(f"MC Answers: {choice_str}"))

def format_input(row):
    x = "Yes"
    if not row['is_correct']:
        x = "No"
    return (
        f"Question: {row['QuestionText']}\n"
        f"Answer: {row['MC_Answer']}\n"
        f"Correct? {x}\n"
        f"Student Explanation: {row['StudentExplanation']}"
    )

train['text'] = train.apply(format_input,axis=1)
print("Example prompt for our LLM:")
print()
print( train.text.values[0] )

lengths = [len(tokenizer.encode(t, truncation=False)) for t in train["text"]]
L = (np.array(lengths)>MAX_LEN).sum()
print(f"There are {L} train sample(s) with more than {MAX_LEN} tokens")
np.sort( lengths )

# Split into train and validation sets
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset
COLS = ['text','label']
train_ds = Dataset.from_pandas(train_df[COLS])
val_ds = Dataset.from_pandas(val_df[COLS])

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)

# Set format for PyTorch
columns = ['input_ids', 'attention_mask', 'label']
train_ds.set_format(type='torch', columns=columns)
val_ds.set_format(type='torch', columns=columns)

model = AutoModelForSequenceClassification.from_pretrained(
    "/kaggle/input/gemma2-9b-it-bf16",
    num_labels=n_classes,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

from peft import PeftModel
model = PeftModel.from_pretrained(model, model_name)

#Training arguments
training_args = TrainingArguments(
    output_dir = f"./{DIR}",
    do_train=True,
    do_eval=True,
    eval_strategy="steps",
    save_strategy="steps", #no for no saving 
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=1e-5,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=200,
    eval_steps=200,
    save_total_limit=1,
    metric_for_best_model="map@3",
    greater_is_better=True,
    load_best_model_at_end=True,
    report_to="none",
    bf16=False,          
    fp16=True,
)

def compute_map3(eval_pred):
    logits, labels = eval_pred
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
    
    top3 = np.argsort(-probs, axis=1)[:, :3]  # Top 3 predictions
    match = (top3 == labels[:, None])

    # Compute MAP@3 manually
    map3 = 0
    for i in range(len(labels)):
        if match[i, 0]:
            map3 += 1.0
        elif match[i, 1]:
            map3 += 1.0 / 2
        elif match[i, 2]:
            map3 += 1.0 / 3
    return {"map@3": map3 / len(labels)}

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_map3,
)

#trainer.train()

test = pd.read_csv('/kaggle/input/map-charting-student-math-misunderstandings/test.csv')
print( test.shape)

test = test.merge(correct, on=['QuestionId','MC_Answer'], how='left')
test.is_correct = test.is_correct.fillna(0)

test['text'] = test.apply(format_input,axis=1)

ds_test = Dataset.from_pandas(test[['text']])
ds_test = ds_test.map(tokenize, batched=True)

predictions = trainer.predict(ds_test)
probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=1).numpy()

# Get top 3 predicted class indices
top3 = np.argsort(-probs, axis=1)[:, :3]   # shape: [num_samples, 3]

# Decode numeric class indices to original string labels
flat_top3 = top3.flatten()
decoded_labels = le.inverse_transform(flat_top3)
top3_labels = decoded_labels.reshape(top3.shape)

# Join 3 labels per row with space
joined_preds = [" ".join(row) for row in top3_labels]

# Save submission
sub = pd.DataFrame({
    "row_id": test.row_id.values,
    "Category:Misconception": joined_preds
})
sub.to_csv("submission_gemma.csv", index=False)
print(sub.head())

##Release memory
import torch
import gc

del top3_labels, flat_top3, decoded_labels, top3, test, ds_test
del training_args, train_ds, val_ds, model, trainer, predictions, probs
# Delete any other lingering references
for obj in list(globals().keys()):
    if isinstance(globals()[obj], torch.nn.Module) or isinstance(globals()[obj], torch.Tensor):
        del globals()[obj]

# Dọn sạch autograd
torch.cuda.empty_cache()
gc.collect()

# Nếu dùng nhiều GPU, làm thêm bước này để clear hết:
torch.cuda.ipc_collect()

# In ra kiểm tra
print("Memory allocated:", torch.cuda.memory_allocated())
print("Memory reserved:", torch.cuda.memory_reserved())

#Ettin encoder
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

VER=1
#model_name = "jhu-clsp/ettin-encoder-1b"
model_name = "/kaggle/input/ettin-encoder-1b-cv943"
EPOCHS = 3

DIR = f"ver_{VER}"
os.makedirs(DIR, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)
MAX_LEN = 512


# Split into train and validation sets
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset
COLS = ['text','label']
train_ds = Dataset.from_pandas(train_df[COLS])
val_ds = Dataset.from_pandas(val_df[COLS])

#Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)

# Set format for PyTorch
columns = ['input_ids', 'attention_mask', 'label']
train_ds.set_format(type='torch', columns=columns)
val_ds.set_format(type='torch', columns=columns)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=n_classes,
    reference_compile=False,
)

training_args = TrainingArguments(
    output_dir = f"./{DIR}",
    do_train=True,
    do_eval=True,
    eval_strategy="steps",
    save_strategy="steps", #no for no saving 
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=1e-5,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=200,
    eval_steps=200,
    save_total_limit=1,
    metric_for_best_model="map@3",
    greater_is_better=True,
    load_best_model_at_end=True,
    report_to="none",
    bf16=False,           
    fp16=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_map3,
)

test = pd.read_csv('/kaggle/input/map-charting-student-math-misunderstandings/test.csv')
print( test.shape )

test = test.merge(correct, on=['QuestionId','MC_Answer'], how='left')
test.is_correct = test.is_correct.fillna(0)

test['text'] = test.apply(format_input,axis=1)

ds_test = Dataset.from_pandas(test[['text']])
ds_test = ds_test.map(tokenize, batched=True)

predictions = trainer.predict(ds_test)
probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=1).numpy()

#Get top 3 predicted class indices
top3 = np.argsort(-probs, axis=1)[:, :]   # shape: [num_samples, 3]

# Decode numeric class indices to original string labels
flat_top3 = top3.flatten()
decoded_labels = le.inverse_transform(flat_top3)
top3_labels = decoded_labels.reshape(top3.shape)

# Join 3 labels per row with space
joined_preds = ["|".join(row) for row in top3_labels]

# Save submission
sub = pd.DataFrame({
    "row_id": test.row_id.values,
    "Category:Misconception": joined_preds
})
sub.to_csv("submission_ettin.csv", index=False)

#Release memory
del top3_labels, flat_top3, decoded_labels, top3, test, ds_test
del training_args, train_ds, val_ds, model, trainer, predictions, probs
# Delete any other lingering references
for obj in list(globals().keys()):
    if isinstance(globals()[obj], torch.nn.Module) or isinstance(globals()[obj], torch.Tensor):
        del globals()[obj]

# Dọn sạch autograd
torch.cuda.empty_cache()
gc.collect()

# Nếu dùng nhiều GPU, làm thêm bước này để clear hết:
torch.cuda.ipc_collect()

# In ra kiểm tra
print("Memory allocated:", torch.cuda.memory_allocated())
print("Memory reserved:", torch.cuda.memory_reserved())

#Modern bert
VER=1
model_name = "/kaggle/input/modernbert-large-cv938"
EPOCHS = 3

DIR = f"ver_{VER}"
os.makedirs(DIR, exist_ok=True)

tokenizer = AutoTokenizer.from_pretrained(model_name)
MAX_LEN = 512

# Split into train and validation sets
train_df, val_df = train_test_split(train, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset
COLS = ['text','label']
train_ds = Dataset.from_pandas(train_df[COLS])
val_ds = Dataset.from_pandas(val_df[COLS])

# Tokenization function
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

train_ds = train_ds.map(tokenize, batched=True)
val_ds = val_ds.map(tokenize, batched=True)

# Set format for PyTorch
columns = ['input_ids', 'attention_mask', 'label']
train_ds.set_format(type='torch', columns=columns)
val_ds.set_format(type='torch', columns=columns)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=n_classes,
    reference_compile=False,
)

training_args = TrainingArguments(
    output_dir = f"./{DIR}",
    do_train=True,
    do_eval=True,
    eval_strategy="steps",
    save_strategy="steps", #no for no saving 
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=1e-5,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=200,
    eval_steps=200,
    save_total_limit=1,
    metric_for_best_model="map@3",
    greater_is_better=True,
    load_best_model_at_end=True,
    report_to="none",
    bf16=False,           
    fp16=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_map3,
)

test = pd.read_csv('/kaggle/input/map-charting-student-math-misunderstandings/test.csv')
print( test.shape )

test = test.merge(correct, on=['QuestionId','MC_Answer'], how='left')
test.is_correct = test.is_correct.fillna(0)

test['text'] = test.apply(format_input,axis=1)

ds_test = Dataset.from_pandas(test[['text']])
ds_test = ds_test.map(tokenize, batched=True)

predictions = trainer.predict(ds_test)
probs = torch.nn.functional.softmax(torch.tensor(predictions.predictions), dim=1).numpy()

# Get top 3 predicted class indices
top3 = np.argsort(-probs, axis=1)[:, :]   # shape: [num_samples, 3]

# Decode numeric class indices to original string labels
flat_top3 = top3.flatten()
decoded_labels = le.inverse_transform(flat_top3)
top3_labels = decoded_labels.reshape(top3.shape)

# Join 3 labels per row with space
joined_preds = ["|".join(row) for row in top3_labels]

# Save submission
sub = pd.DataFrame({
    "row_id": test.row_id.values,
    "Category:Misconception": joined_preds
})
sub.to_csv("submission_modern.csv", index=False)

#Release memory
del top3_labels, flat_top3, decoded_labels, top3, test, ds_test
del training_args, train_ds, val_ds, model, trainer, predictions, probs
# Delete any other lingering references
for obj in list(globals().keys()):
    if isinstance(globals()[obj], torch.nn.Module) or isinstance(globals()[obj], torch.Tensor):
        del globals()[obj]

# Dọn sạch autograd
torch.cuda.empty_cache()
gc.collect()

# Nếu dùng nhiều GPU, làm thêm bước này để clear hết:
torch.cuda.ipc_collect()

# In ra kiểm tra
print("Memory allocated:", torch.cuda.memory_allocated())
print("Memory reserved:", torch.cuda.memory_reserved())

import numpy as np
import pandas as pd
from sklearn import *
import xgboost as xgb
from nltk.corpus import stopwords
es = set(stopwords.words('english'))
from tqdm import tqdm
tqdm.pandas(desc="Mapping Progress")

p = '/kaggle/input/map-charting-student-math-misunderstandings/'
train = pd.read_csv(p+'train.csv')
test = pd.read_csv(p+'test.csv')
sub = pd.read_csv(p+'sample_submission.csv')

import transformers, torch
import gc, os, logging
import math

os.environ['OMP_NUM_THREADS'] = '2'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
MODEL_PATH = "/kaggle/input/gemma-3/transformers/gemma-3-1b-it/1"
DEVICE = torch.device('cuda')

class PerplexityCalculator:
    def __init__(self,):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_PATH)
        self.model = transformers.AutoModelForCausalLM.from_pretrained(
            MODEL_PATH, device_map="auto",
            torch_dtype=torch.float16,)
        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')
        self.model.eval()

    def get_perplexity(self, text: str) -> float:
        with torch.no_grad():
            text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"
            model_inputs = self.tokenizer(text_with_special, return_tensors='pt', add_special_tokens=False,)
            model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}
            logits = self.model(**model_inputs, use_cache=False)['logits']
            shift_logits = logits[..., :-1, :].contiguous()
            shift_labels = model_inputs['input_ids'][..., 1:].contiguous()
            loss = self.loss_fct(
                shift_logits.view(-1, shift_logits.size(-1)),
                shift_labels.view(-1))
            sequence_loss = loss.sum() / len(loss)
            loss_list = sequence_loss.cpu().item()
        return math.exp(loss_list)
        
scorer = PerplexityCalculator()

c = ['row_id', 'QuestionId', 'QuestionText', 'MC_Answer', 'StudentExplanation']
dfall = pd.concat([train[c], test[c]], axis=0)
qi = dfall.QuestionId.unique()
dfall['char_len'] = dfall['StudentExplanation'].map(len)
dfall['word_len'] = dfall['StudentExplanation'].map(lambda x: len(str(x).split(' ')))
gqid = dfall.groupby(['QuestionId']).agg(
    achar_len=('char_len', 'mean'), 
    mnchar_len=('char_len', 'min'), 
    mxchar_len=('char_len', 'max'), 
    aword_len=('word_len', 'mean'),
    mnword_len=('word_len', 'min'), 
    mxword_len=('word_len', 'max'), 
)
#https://www.kaggle.com/code/cdeotte/deberta-starter-cv-0-930
gqida = dfall.groupby(['QuestionId','MC_Answer']).size().reset_index(name='count')
gqida['arank'] = gqida.groupby('QuestionId')['count'].rank(method='dense', ascending=False).astype(int) - 1
gqida = gqida.drop('count',axis=1)
gqida = gqida.sort_values(['QuestionId','arank'])

#Takes about 40min for 36,696 records
cached_perplexity = [82.92766733936132, 33.31005738633129, 46.066137010589394, 50.29817277977263, 48.27689708476307, 43.19069160819034, 29.33866439394716, 35.45837151398557, 23.073202403172917, 66.37447035867858, 35.7364749685521, 34.908640417532865, 46.88305220915621, 32.159346537604755, 45.351947753528236, 45.351947753528236, 36.016759615546256, 25.94189877662824, 26.401940203271806, 32.92198346312245, 47.34313736903344, 22.893645317537526, 20.64232851707703, 23.80562446936611, 26.76538304079522, 29.22428378123494, 30.26997357328364, 30.26997357328364, 30.26997357328364, 32.72964525738904, 32.72964525738904, 30.927341318986464, 32.411575418595035, 34.367432111764096, 36.299242562272255, 38.71588573814723, 38.71588573814723, 38.26483200699776, 25.58963789966816, 25.58963789966816, 33.11545195869231, 34.908640417532865, 29.167260806600495, 48.27689708476307, 42.27266353442275, 45.44061233480379, 35.87634357692165, 72.04880596519355, 42.93836112094585, 37.01518097066168, 35.59715165620933, 40.1011996370117, 50.6926662381911, 56.33128135834348, 38.94340253332466, 38.867415430562446, 36.08717359043372, 22.67116267165818, 22.938403163153815, 36.942956250241515, 39.7114923241556, 43.61454191906839, 34.84052601883783, 35.806340977539655, 42.10785803778799, 51.99626426796157, 57.21836996148928, 52.50652775191775, 49.809370301716946, 64.33234259239626, 47.15856396830611, 51.39048944047805, 53.85677052285177, 50.6926662381911, 54.067559962798995, 36.65546388385301, 36.228414794400415, 28.771203256660744, 33.96703939987107, 33.90076227087091, 33.90076227087091, 27.940524471445343, 43.02230705927991, 22.75989509352673, 55.6750020519895, 38.115651564714454, 39.556671887793456, 35.7364749685521, 40.25815129472595, 46.15619785585283, 30.447857334592324, 28.65903496526287, 37.524723159601, 30.093129052017023, 37.37842812826559, 29.56877039069624, 42.93836112094585, 47.06654728734682, 30.034410770075546, 40.81233544283598, 40.81233544283598, 28.60311491479713, 47.06654728734682, 26.87013978811538, 42.93836112094585, 34.908640417532865, 30.388446900124176, 37.67159077349981, 36.37020880080945, 37.8190332109868, 32.159346537604755, 35.25121511458854, 40.17959882934157, 34.300373728940066, 32.411575418595035, 31.660762082694095, 31.660762082694095, 40.97207040739876, 38.56494680401705, 38.56494680401705, 38.56494680401705, 34.16664924473404, 43.444504934395866, 50.79177210350426, 50.79177210350426, 29.9173176993003, 44.12865633779556, 26.713157919740368, 38.94340253332466, 40.41571724385996, 28.771203256660744, 42.10785803778799, 31.53732846903916, 42.77096031537661, 27.293282919182133, 39.01953819342525, 39.78912962559818, 43.61454191906839, 38.56494680401705, 31.10908815096766, 33.31005738633129, 39.095822701298815, 35.52769382137767, 44.64883097460398, 33.90076227087091, 33.31005738633129, 34.97688798216538, 34.97688798216538, 32.66578255521719, 31.72265993608251, 38.26483200699776, 35.25121511458854, 34.16664924473404, 45.263456176258764, 42.10785803778799, 34.09998263323346, 30.447857334592324, 58.57526450222688, 58.57526450222688, 35.38918446958933, 35.52769382137767, 29.74253347911242, 37.524723159601, 51.1901369093998, 33.96703939987107, 30.806767041912167, 34.501942437892716, 31.291903031786116, 29.453492679719986, 27.83159468972004, 40.73270164287187, 40.17959882934157, 40.17959882934157, 35.7364749685521, 44.82358155812517, 37.01518097066168, 31.41437607743883, 43.61454191906839, 34.09998263323346, 34.09998263323346, 34.09998263323346, 34.09998263323346, 34.09998263323346, 35.25121511458854, 36.87087245606033, 39.78912962559818, 35.04526897307925, 26.35042423874788, 39.7114923241556, 42.52108200006278, 42.52108200006278, 31.90908051733889, 31.90908051733889, 27.186876493132687, 27.507346790200362, 31.72265993608251, 22.938403163153815, 26.453556883236853, 33.31005738633129, 30.987805310145493, 39.40245503911115, 31.169907463288272, 33.63694444585419, 35.66674528344124, 35.25121511458854, 30.806767041912167, 28.883810562338418, 39.32557222442699, 29.74253347911242, 42.68750480955783, 30.56702687878971, 33.83461446305701, 32.66578255521719, 34.367432111764096, 33.702705802144536, 31.048387510438282, 27.77728915635237, 31.475792237910618, 29.453492679719986, 29.453492679719986, 31.9714638416083, 28.547303976590324, 38.71588573814723, 25.39049775763493, 41.698649825700315, 37.9670527218579, 31.41437607743883, 36.08717359043372, 30.866995306309306, 34.63697922255266, 39.40245503911115, 43.444504934395866, 37.45150421083523, 45.17513726542669, 24.657426302639607, 39.172256347947744, 34.77254452624438, 32.53843073806492, 37.305494633128845, 33.57131140436464, 32.79363281328884, 29.858942463791767, 31.784678801853513, 39.172256347947744, 43.61454191906839, 39.86691871057298, 35.38918446958933, 26.09434828302453, 31.10908815096766, 39.7114923241556, 39.94485987582193, 44.12865633779556, 50.89107172411317, 53.646802873050504, 31.90908051733889, 37.08754689283638, 41.53608219643496, 47.250760544967605, 30.034410770075546, 30.987805310145493, 29.167260806600495, 38.56494680401705, 27.668995768810042, 29.800681131145613, 28.715064341305684, 38.71588573814723, 31.35307975333966, 31.35307975333966, 26.35042423874788, 31.35307975333966, 35.45837151398557, 35.45837151398557, 63.3349620664254, 60.199030648454944, 35.59715165620933, 47.15856396830611, 48.84596987082467, 39.634006510082365, 37.74523999888851, 36.58394105020859, 45.086990684122, 26.557093177483523, 43.3597351955915, 41.617286632367644, 24.086237637543274, 42.27266353442275, 33.57131140436464, 25.63966650134361, 32.53843073806492, 29.110349096271467, 27.293282919182133, 27.028044172168716, 50.79177210350426, 47.71445419747699, 33.4404292647954, 61.626609520407456, 26.609013186724894, 40.33685733281881, 46.246434773093995, 48.94146538270724, 31.846818916590372, 35.87634357692165, 43.19069160819034, 43.02230705927991, 49.9067492938035, 53.75168417471723, 13.72392451825968, 42.190180314697045, 26.557093177483523, 61.38635013907805, 47.71445419747699, 54.38529201854992, 63.08804244970428, 63.08804244970428, 37.08754689283638, 34.4346215959791, 39.095822701298815, 48.08868329990042, 43.61454191906839, 31.291903031786116, 41.943695056893915, 38.79157659527073, 32.79363281328884, 35.45837151398557, 38.26483200699776, 43.275130861102994, 42.02569638966092, 39.40245503911115, 27.133828946090556, 35.52769382137767, 34.84052601883783, 41.132430556936306, 35.66674528344124, 38.33964103892552, 49.22907356066477, 68.7494509271083, 43.10641711474324, 37.01518097066168, 37.67159077349981, 37.45150421083523, 49.133016862271724, 42.10785803778799, 61.38635013907805, 56.11166650505244, 32.85774546700983, 26.87013978811538, 35.59715165620933, 39.556671887793456, 39.479488162280596, 26.713157919740368, 39.94485987582193, 23.666546242140512, 40.81233544283598, 73.18340956555208, 28.15966472087403, 55.6750020519895, 37.08754689283638, 29.22428378123494, 31.169907463288272, 43.87084603329354, 32.85774546700983, 21.717643615002626, 34.16664924473404, 22.98324851186175, 31.9714638416083, 78.82163381626928, 68.21443845541918, 38.94340253332466, 41.698649825700315, 41.698649825700315, 35.66674528344124, 76.99573263552006, 35.806340977539655, 40.33685733281881, 34.16664924473404, 25.2915095879081, 32.60204446315635, 44.12865633779556, 30.034410770075546, 32.9863470466753, 32.92198346312245, 56.11166650505244, 37.160054292819765, 43.5294404008871, 41.780172086809024, 29.33866439394716, 29.33866439394716, 49.22907356066477, 63.83170433047907, 49.809370301716946, 40.65322322588273, 50.5937537501147, 39.7114923241556, 36.016759615546256, 27.83159468972004, 30.806767041912167, 49.133016862271724, 47.52843316962603, 23.80562446936611, 40.892124929554, 38.33964103892552, 38.33964103892552, 34.908640417532865, 32.47494113727571, 39.40245503911115, 34.908640417532865, 24.039240115615065, 33.4404292647954, 43.87084603329354, 50.6926662381911, 40.81233544283598, 34.908640417532865, 34.908640417532865, 34.908640417532865, 34.908640417532865, 39.40245503911115, 41.29341833835342, 38.64034257045414, 33.90076227087091, 35.38918446958933, 42.93836112094585, 40.49473132867618, 36.299242562272255, 47.99485175252902, 36.44131378072682, 41.698649825700315, 26.975306541751994, 32.60204446315635, 27.507346790200362, 31.660762082694095, 39.634006510082365, 65.09067188950593, 33.63694444585419, 33.57131140436464, 58.34690118774256, 53.542126217761066, 51.99626426796157, 67.41971617548266, 59.73055807527518, 61.38635013907805, 55.89290784893326, 100.81440478091984, 52.9183415530491, 33.50580642730562, 25.992616060150944, 64.58413226217957, 69.28865955361954, 44.04255166984753, 63.582848099201826, 48.27689708476307, 34.4346215959791, 37.67159077349981, 37.08754689283638, 51.49095957359226, 35.25121511458854, 38.41459632502361, 54.173263858709824, 49.32531805312982, 39.32557222442699, 36.016759615546256, 41.617286632367644, 34.77254452624438, 38.56494680401705, 40.17959882934157, 38.115651564714454, 38.94340253332466, 61.867809250367884, 64.08153455750669, 52.71203249107963, 32.22221914035047, 44.21492934326177, 44.21492934326177, 53.43765380953904, 66.37447035867858, 41.617286632367644, 45.44061233480379, 54.27917440928786, 40.49473132867618, 59.265731184583274, 56.11166650505244, 46.33684810653952, 49.32531805312982, 46.51820540231633, 50.6926662381911, 58.80452160545304, 27.561124577174567, 28.940279382223952, 36.87087245606033, 33.57131140436464, 34.03344610288473, 38.115651564714454, 37.37842812826559, 35.38918446958933, 44.387981683814154, 67.1568716116849, 41.05217218133816, 27.886006392229614, 30.329152388418176, 36.299242562272255, 41.53608219643496, 38.71588573814723, 26.35042423874788, 40.65322322588273, 36.58394105020859, 40.022953418667065, 41.2128458403547, 50.89107172411317, 50.89107172411317, 50.29817277977263, 25.1437504597293, 42.10785803778799, 39.248839424943355, 34.23344619169951, 52.9183415530491, 37.45150421083523, 51.79355004464666, 23.992334296139543, 26.24769367033534, 40.33685733281881, 46.427438201089, 46.427438201089, 51.591626129191596, 24.753932740556596, 45.88654216244616, 30.62678644311483, 33.702705802144536, 43.78524440968234, 54.27917440928786, 31.660762082694095, 44.82358155812517, 32.72964525738904, 57.89284200037607, 27.77728915635237, 28.491601937740562, 28.491601937740562, 31.846818916590372, 26.66103470132838, 38.56494680401705, 42.190180314697045, 32.9863470466753, 47.34313736903344, 26.66103470132838, 36.72712654703974, 55.24173574865985, 41.2128458403547, 30.866995306309306, 30.927341318986464, 30.26997357328364, 27.83159468972004, 45.529450258314064, 53.02179868836547, 96.19777706766972, 34.70469568042362, 56.55175575986736, 36.65546388385301, 59.73055807527518, 34.97688798216538, 35.52769382137767, 37.01518097066168, 54.811841150452466, 37.89297069129353, 47.62135285300038, 36.798929313140206, 27.080884906424416, 36.51255777326842, 25.1437504597293, 38.48969815122384, 34.4346215959791, 32.85774546700983, 37.37842812826559, 37.67159077349981, 66.63425269055388, 59.73055807527518, 33.702705802144536, 47.15856396830611, 50.396507397123145, 49.037147591523336, 51.99626426796157, 33.76859572409541, 34.77254452624438, 62.10995300937982, 62.10995300937982, 24.18050850623376, 30.866995306309306, 53.75168417471723, 53.75168417471723, 51.290215346585704, 50.89107172411317, 40.1011996370117, 46.7915731119465, 69.55984769951485, 53.02179868836547, 49.42175070681059, 49.71218131735948, 44.21492934326177, 46.066137010589394, 41.132430556936306, 49.71218131735948, 59.03467599561125, 37.89297069129353, 50.79177210350426, 33.63694444585419, 33.4404292647954, 27.080884906424416, 37.89297069129353, 33.31005738633129, 40.81233544283598, 86.90744918987926, 81.641993024771, 93.2380826167318, 50.6926662381911, 28.15966472087403, 33.83461446305701, 32.72964525738904, 15.551243837871848, 49.51837188956857, 64.33234259239626, 30.74665629604239, 59.03467599561125, 69.55984769951485, 47.34313736903344, 55.89290784893326, 28.715064341305684, 28.715064341305684, 38.0412795852805, 25.992616060150944, 17.45061689198871, 30.866995306309306, 16.329425896569767, 28.65903496526287, 28.491601937740562, 43.5294404008871, 34.03344610288473, 35.59715165620933, 37.8190332109868, 42.27266353442275, 31.291903031786116, 26.35042423874788, 55.02636863114642, 38.0412795852805, 24.657426302639607, 31.90908051733889, 28.15966472087403, 32.22221914035047, 56.9952966774873, 44.99901609609206, 42.3553080116137, 56.773093073800744, 48.27689708476307, 33.245062173048, 33.1801937796528, 30.56702687878971, 42.93836112094585, 58.80452160545304, 58.80452160545304, 55.457945789238345, 43.5294404008871, 43.95661501095434, 33.702705802144536, 57.21836996148928, 54.38529201854992, 24.133327041205547, 57.21836996148928, 70.10541234668786, 28.325147094540903, 34.367432111764096, 52.19977189231385, 48.182698291098816, 67.9484955630781, 38.33964103892552, 65.34542957624015, 34.300373728940066, 63.83170433047907, 60.434642992688794, 28.940279382223952, 32.53843073806492, 45.44061233480379, 42.190180314697045, 35.59715165620933, 29.33866439394716, 32.79363281328884, 41.943695056893915, 37.8190332109868, 38.64034257045414, 47.250760544967605, 49.22907356066477, 53.333385249853684, 55.457945789238345, 42.68750480955783, 29.74253347911242, 27.507346790200362, 44.47476167904283, 34.03344610288473, 36.942956250241515, 43.02230705927991, 57.89284200037607, 55.02636863114642, 59.49769069799599, 34.4346215959791, 38.64034257045414, 37.160054292819765, 39.86691871057298, 26.14536361045317, 54.598150033144236, 53.646802873050504, 63.83170433047907, 30.447857334592324, 37.524723159601, 24.513371642677637, 30.56702687878971, 52.19977189231385, 45.44061233480379, 38.26483200699776, 25.63966650134361, 32.60204446315635, 65.09067188950593, 38.26483200699776, 48.84596987082467, 26.713157919740368, 37.160054292819765, 50.79177210350426, 88.62154503481175, 52.81508628524973, 63.3349620664254, 51.1901369093998, 36.44131378072682, 36.942956250241515, 36.65546388385301, 36.228414794400415, 31.10908815096766, 36.798929313140206, 24.086237637543274, 49.51837188956857, 64.83690740886694, 45.263456176258764, 48.75066069158903, 78.51433738825796, 37.01518097066168, 51.99626426796157, 30.806767041912167, 23.4365524707797, 36.228414794400415, 29.33866439394716, 26.975306541751994, 21.464625403032095, 22.145985754016134, 34.97688798216538, 50.1020787877761, 54.811841150452466, 32.47494113727571, 31.41437607743883, 27.240027749911206, 37.74523999888851, 31.846818916590372, 50.990565478815945, 44.12865633779556, 38.64034257045414, 51.1901369093998, 29.281418237700255, 29.56877039069624, 41.617286632367644, 48.27689708476307, 55.02636863114642, 38.0412795852805, 30.093129052017023, 42.10785803778799, 43.19069160819034, 29.511075247281195, 25.24216031657971, 77.59961765106212, 32.28521466114651, 72.33079701870139, 48.56059987746486, 29.684499285876434, 38.0412795852805, 31.41437607743883, 32.9863470466753, 28.940279382223952, 29.626578330054702, 28.214717811314713, 38.48969815122384, 45.263456176258764, 40.1011996370117, 29.626578330054702, 24.133327041205547, 38.48969815122384, 21.33922385967176, 20.64232851707703, 24.80232751057381, 26.35042423874788, 26.609013186724894, 41.943695056893915, 34.84052601883783, 29.9173176993003, 43.87084603329354, 27.507346790200362, 44.04255166984753, 46.70027251047952, 42.27266353442275, 61.38635013907805, 27.293282919182133, 47.52843316962603, 47.52843316962603, 39.095822701298815, 38.79157659527073, 37.8190332109868, 51.09025374715126, 47.06654728734682, 47.62135285300038, 59.03467599561125, 40.97207040739876, 44.73612093895834, 37.08754689283638, 63.08804244970428, 59.265731184583274, 34.84052601883783, 27.293282919182133, 33.702705802144536, 56.11166650505244, 37.74523999888851, 27.668995768810042, 31.9714638416083, 22.938403163153815, 42.10785803778799, 39.86691871057298, 35.38918446958933, 25.539706914745466, 29.396022468352484, 40.1011996370117, 38.26483200699776, 37.8190332109868, 36.798929313140206, 39.095822701298815, 31.35307975333966, 33.63694444585419, 35.59715165620933, 36.228414794400415, 36.15772522700737, 40.1011996370117, 38.26483200699776, 31.784678801853513, 28.827451925481085, 27.4536739354601, 37.232703447205715, 34.4346215959791, 36.798929313140206, 49.32531805312982, 28.827451925481085, 28.214717811314713, 23.118311288923124, 42.02569638966092, 41.698649825700315, 37.74523999888851, 43.02230705927991, 41.37414835829223, 50.5937537501147, 33.1801937796528, 47.71445419747699, 39.556671887793456, 48.46584751755162, 48.27689708476307, 59.73055807527518, 25.58963789966816, 39.479488162280596, 32.22221914035047, 40.65322322588273, 32.72964525738904, 37.305494633128845, 38.41459632502361, 42.3553080116137, 43.95661501095434, 33.63694444585419, 39.94485987582193, 46.60915005647103, 46.60915005647103, 63.3349620664254, 49.32531805312982, 29.56877039069624, 78.82163381626928, 29.9173176993003, 40.57389988868238, 36.58394105020859, 50.29817277977263, 26.453556883236853, 41.132430556936306, 40.33685733281881, 37.89297069129353, 41.29341833835342, 36.228414794400415, 47.52843316962603, 42.52108200006278, 40.25815129472595, 26.453556883236853, 32.66578255521719, 32.9863470466753, 21.887977204970085, 39.78912962559818, 42.68750480955783, 35.11378365112735, 32.79363281328884, 37.232703447205715, 30.034410770075546, 33.96703939987107, 29.626578330054702, 31.598985005566618, 35.806340977539655, 67.1568716116849, 43.78524440968234, 41.132430556936306, 35.25121511458854, 60.199030648454944, 55.89290784893326, 36.299242562272255, 39.095822701298815, 26.2990087931472, 46.15619785585283, 28.2698785323965, 23.852165264858517, 42.77096031537661, 19.353835657005096, 35.18243227767251, 45.529450258314064, 26.09434828302453, 33.90076227087091, 45.7076474882609, 50.6926662381911, 36.228414794400415, 34.70469568042362, 27.028044172168716, 30.329152388418176, 33.11545195869231, 57.442316329639176, 32.411575418595035, 43.3597351955915, 26.87013978811538, 26.817710263716034, 26.24769367033534, 33.31005738633129, 39.7114923241556, 27.40010580820802, 45.97625189374871, 34.70469568042362, 32.66578255521719, 45.44061233480379, 56.33128135834348, 33.31005738633129, 31.230845679407448, 39.32557222442699, 32.72964525738904, 38.56494680401705, 47.15856396830611, 17.89937193856263, 29.33866439394716, 28.996858600549206, 46.7915731119465, 24.609314266596865, 26.2990087931472, 35.52769382137767, 33.11545195869231, 35.04526897307925, 36.72712654703974, 49.037147591523336, 29.33866439394716, 35.59715165620933, 33.96703939987107, 19.429584428039178, 45.79700747446617, 39.172256347947744, 33.245062173048, 35.87634357692165, 30.62678644311483, 50.20003003478527, 47.90120329104557, 31.784678801853513, 55.02636863114642, 44.30137101535166, 33.76859572409541, 35.45837151398557, 53.22932014095188, 39.40245503911115, 37.305494633128845, 24.275148340672132, 28.60311491479713, 24.417803063601138, 19.543763832601815, 69.83209724580306, 43.275130861102994, 41.53608219643496, 49.51837188956857, 51.894808174781225, 51.79355004464666, 44.21492934326177, 44.82358155812517, 35.04526897307925, 38.115651564714454, 56.773093073800744, 25.740017318702005, 35.806340977539655, 37.01518097066168, 39.634006510082365, 28.883810562338418, 33.1801937796528, 51.290215346585704, 41.698649825700315, 30.329152388418176, 34.70469568042362, 34.70469568042362, 42.10785803778799, 56.773093073800744, 52.097918711212095, 33.76859572409541, 27.186876493132687, 35.52769382137767, 41.861853726676614, 39.556671887793456, 37.5980852538711, 37.5980852538711, 33.63694444585419, 39.248839424943355, 49.9067492938035, 28.827451925481085, 25.992616060150944, 45.263456176258764, 24.948080241282657, 36.58394105020859, 27.133828946090556, 22.059646737841184, 34.84052601883783, 26.401940203271806, 46.88305220915621, 46.33684810653952, 28.1047190510635, 33.702705802144536, 25.992616060150944, 46.51820540231633, 51.1901369093998, 24.513371642677637, 44.21492934326177, 33.4404292647954, 45.88654216244616, 36.016759615546256, 28.60311491479713, 38.115651564714454, 30.151962130172098, 29.684499285876434, 20.44172438427552, 25.740017318702005, 39.01953819342525, 54.173263858709824, 31.598985005566618, 50.396507397123145, 60.67117749756472, 30.806767041912167, 35.94648303373688, 42.02569638966092, 56.9952966774873, 36.44131378072682, 37.89297069129353, 81.641993024771, 34.97688798216538, 38.56494680401705, 35.45837151398557, 35.52769382137767, 31.9714638416083, 35.04526897307925, 34.97688798216538, 30.447857334592324, 42.3553080116137, 49.51837188956857, 54.38529201854992, 41.861853726676614, 35.32013242426123, 38.26483200699776, 35.59715165620933, 57.66713919909174, 44.47476167904283, 38.94340253332466, 38.41459632502361, 50.29817277977263, 34.84052601883783, 85.56007407666469, 35.806340977539655, 34.4346215959791, 24.9968545761854, 85.89494673838159, 30.26997357328364, 31.784678801853513, 33.375179667439795, 49.9067492938035, 50.396507397123145, 44.47476167904283, 44.21492934326177, 53.542126217761066, 44.82358155812517, 48.56059987746486, 36.87087245606033, 36.87087245606033, 36.08717359043372, 37.45150421083523, 29.281418237700255, 34.908640417532865, 26.453556883236853, 24.322607071104724, 37.5980852538711, 35.04526897307925, 32.28521466114651, 28.65903496526287, 29.453492679719986, 31.048387510438282, 44.82358155812517, 30.806767041912167, 32.85774546700983, 30.866995306309306, 39.01953819342525, 35.94648303373688, 30.987805310145493, 47.34313736903344, 39.556671887793456, 37.305494633128845, 28.715064341305684, 43.275130861102994, 23.52828054362271, 23.52828054362271, 29.9173176993003, 34.09998263323346, 39.172256347947744, 59.03467599561125, 30.26997357328364, 39.248839424943355, 32.53843073806492, 30.151962130172098, 32.09659661306934, 22.494734284035275, 24.18050850623376, 48.37128004023317, 48.37128004023317, 44.82358155812517, 52.301824199806006, 49.133016862271724, 40.25815129472595, 38.64034257045414, 59.265731184583274, 66.11570082213345, 29.511075247281195, 60.67117749756472, 54.38529201854992, 35.806340977539655, 35.32013242426123, 36.87087245606033, 36.51255777326842, 59.964336869699544, 32.60204446315635, 25.489873356103875, 38.33964103892552, 31.660762082694095, 30.74665629604239, 31.90908051733889, 35.18243227767251, 32.60204446315635, 42.93836112094585, 40.41571724385996, 67.9484955630781, 49.809370301716946, 79.75075829332496, 79.75075829332496, 44.91121316574036, 30.507383918455513, 35.806340977539655, 36.51255777326842, 43.87084603329354, 59.03467599561125, 31.72265993608251, 46.427438201089, 26.76538304079522, 32.348333340301785, 41.861853726676614, 54.173263858709824, 52.609179777419676, 24.80232751057381, 59.73055807527518, 51.692489491288626, 36.65546388385301, 43.78524440968234, 46.15619785585283, 37.08754689283638, 81.32370105824417, 37.74523999888851, 27.133828946090556, 33.96703939987107, 45.88654216244616, 43.10641711474324, 40.81233544283598, 48.84596987082467, 28.547303976590324, 41.2128458403547, 48.37128004023317, 40.25815129472595, 27.561124577174567, 41.698649825700315, 44.04255166984753, 40.65322322588273, 24.80232751057381, 46.7915731119465, 34.4346215959791, 32.159346537604755, 33.90076227087091, 28.380523708580828, 37.37842812826559, 37.08754689283638, 53.43765380953904, 50.1020787877761, 32.348333340301785, 35.45837151398557, 39.78912962559818, 36.299242562272255, 34.70469568042362, 31.846818916590372, 37.305494633128845, 35.7364749685521, 36.228414794400415, 31.475792237910618, 31.90908051733889, 31.230845679407448, 32.79363281328884, 42.854578979512624, 33.90076227087091, 27.186876493132687, 27.186876493132687, 41.132430556936306, 39.01953819342525, 34.97688798216538, 35.25121511458854, 39.556671887793456, 30.927341318986464, 33.4404292647954, 25.63966650134361, 65.34542957624015, 29.22428378123494, 47.15856396830611, 38.79157659527073, 38.64034257045414, 45.88654216244616, 29.9173176993003, 29.453492679719986, 34.63697922255266, 34.77254452624438, 63.582848099201826, 48.08868329990042, 40.892124929554, 42.93836112094585, 58.34690118774256, 33.1801937796528, 45.529450258314064, 26.817710263716034, 34.908640417532865, 33.4404292647954, 31.72265993608251, 27.293282919182133, 24.086237637543274, 28.65903496526287, 29.22428378123494, 30.56702687878971, 21.297586362924953, 41.05217218133816, 30.987805310145493, 37.37842812826559, 27.240027749911206, 34.09998263323346, 38.41459632502361, 50.396507397123145, 34.56939489431364, 52.50652775191775, 38.0412795852805, 36.942956250241515, 39.095822701298815, 37.524723159601, 30.388446900124176, 36.58394105020859, 28.60311491479713, 23.574279122424027, 37.67159077349981, 27.186876493132687, 22.145985754016134, 34.56939489431364, 53.85677052285177, 42.3553080116137, 49.71218131735948, 37.232703447205715, 34.09998263323346, 36.08717359043372, 40.41571724385996, 68.48142221881122, 23.20879380099744, 26.14536361045317, 26.87013978811538, 34.501942437892716, 51.290215346585704, 39.94485987582193, 25.34095533871963, 45.44061233480379, 32.53843073806492, 34.97688798216538, 24.609314266596865, 48.56059987746486, 62.84208548133934, 41.45503620813183, 28.883810562338418, 26.76538304079522, 31.598985005566618, 31.846818916590372, 26.505274475545082, 24.80232751057381, 32.28521466114651, 40.33685733281881, 36.228414794400415, 28.15966472087403, 36.58394105020859, 51.79355004464666, 38.19016894386643, 35.32013242426123, 36.72712654703974, 43.19069160819034, 51.290215346585704, 40.25815129472595, 18.90550453688997, 22.938403163153815, 50.79177210350426, 55.02636863114642, 41.37414835829223, 26.196478674560588, 43.02230705927991, 42.52108200006278, 35.04526897307925, 48.56059987746486, 45.88654216244616, 32.60204446315635, 26.817710263716034, 31.475792237910618, 23.299630451611073, 30.987805310145493, 37.524723159601, 38.56494680401705, 19.620255961497417, 21.173160702117308, 48.65553748142455, 58.34690118774256, 25.63966650134361, 25.1437504597293, 25.840760898054555, 26.557093177483523, 36.016759615546256, 30.62678644311483, 51.692489491288626, 26.14536361045317, 27.028044172168716, 32.159346537604755, 37.74523999888851, 37.8190332109868, 39.40245503911115, 28.65903496526287, 24.18050850623376, 29.626578330054702, 27.72308958496738, 31.660762082694095, 35.04526897307925, 67.1568716116849, 56.55175575986736, 38.41459632502361, 58.119428177447986, 26.557093177483523, 39.479488162280596, 34.97688798216538, 31.72265993608251, 52.9183415530491, 21.887977204970085, 20.68268496272443, 26.713157919740368, 44.91121316574036, 44.91121316574036, 46.15619785585283, 36.228414794400415, 43.275130861102994, 24.370158584949497, 25.94189877662824, 49.32531805312982, 52.40407602298735, 27.995149135337257, 37.524723159601, 29.626578330054702, 43.5294404008871, 44.21492934326177, 35.52769382137767, 44.04255166984753, 39.78912962559818, 29.975807060355134, 19.89034374461794, 34.84052601883783, 46.88305220915621, 46.88305220915621, 26.505274475545082, 41.29341833835342, 29.74253347911242, 36.08717359043372, 21.760102337787384, 43.10641711474324, 42.10785803778799, 28.436008585761353, 34.97688798216538, 33.57131140436464, 29.56877039069624, 39.78912962559818, 31.9714638416083, 25.63966650134361, 44.91121316574036, 32.47494113727571, 35.52769382137767, 49.809370301716946, 30.62678644311483, 42.854578979512624, 30.507383918455513, 34.84052601883783, 34.501942437892716, 29.053548433146837, 22.40703562619394, 25.094689498803355, 32.09659661306934, 35.59715165620933, 61.14702744030778, 28.771203256660744, 21.802644068801417, 22.189281900071105, 46.15619785585283, 46.15619785585283, 34.56939489431364, 20.8042283062039, 20.203571081597193, 34.300373728940066, 26.817710263716034, 34.300373728940066, 33.31005738633129, 27.40010580820802, 21.256030110048442, 28.60311491479713, 23.620367629992042, 42.93836112094585, 28.940279382223952, 20.36202965350754, 35.7364749685521, 47.71445419747699, 38.0412795852805, 31.598985005566618, 34.4346215959791, 33.96703939987107, 52.71203249107963, 31.475792237910618, 15.765318566620541, 36.299242562272255, 21.887977204970085, 24.039240115615065, 44.73612093895834, 24.513371642677637, 26.609013186724894, 37.67159077349981, 67.1568716116849, 49.037147591523336, 32.28521466114651, 37.08754689283638, 42.68750480955783, 41.943695056893915, 46.33684810653952, 53.646802873050504, 39.94485987582193, 33.96703939987107, 34.56939489431364, 38.79157659527073, 46.7915731119465, 46.7915731119465, 40.57389988868238, 39.7114923241556, 41.45503620813183, 30.388446900124176, 37.524723159601, 28.1047190510635, 28.325147094540903, 23.52828054362271, 48.94146538270724, 33.31005738633129, 37.9670527218579, 53.542126217761066, 63.08804244970428, 45.61846186294896, 36.15772522700737, 43.10641711474324, 54.598150033144236, 39.095822701298815, 45.17513726542669, 35.66674528344124, 33.31005738633129, 34.4346215959791, 31.048387510438282, 33.4404292647954, 28.491601937740562, 30.507383918455513, 37.08754689283638, 27.4536739354601, 42.68750480955783, 20.322298876557472, 32.159346537604755, 84.89424029774196, 26.2990087931472, 41.37414835829223, 52.301824199806006, 71.76791429061606, 63.83170433047907, 57.66713919909174, 47.250760544967605, 57.66713919909174, 72.04880596519355, 71.20941195067294, 56.773093073800744, 71.20941195067294, 64.33234259239626, 38.48969815122384, 46.60915005647103, 55.6750020519895, 24.18050850623376, 28.1047190510635, 35.45837151398557, 43.78524440968234, 39.32557222442699, 38.41459632502361, 34.16664924473404, 31.598985005566618, 24.561296107667808, 39.86691871057298, 32.411575418595035, 33.702705802144536, 38.19016894386643, 47.99485175252902, 42.52108200006278, 35.32013242426123, 41.45503620813183, 62.10995300937982, 49.32531805312982, 38.41459632502361, 52.301824199806006, 20.56185170491249, 47.15856396830611, 54.27917440928786, 39.479488162280596, 41.698649825700315, 53.85677052285177, 48.27689708476307, 42.190180314697045, 36.72712654703974, 39.248839424943355, 46.88305220915621, 52.301824199806006, 24.9968545761854, 23.028181534732802, 41.2128458403547, 38.79157659527073, 57.66713919909174, 31.10908815096766, 30.388446900124176, 44.21492934326177, 50.1020787877761, 45.7076474882609, 58.57526450222688, 38.48969815122384, 67.68358948381629, 113.34868209247811, 54.811841150452466, 44.12865633779556, 52.19977189231385, 58.119428177447986, 39.172256347947744, 39.172256347947744, 39.095822701298815, 47.435694792893365, 36.016759615546256, 49.61518196998445, 70.37979717263202, 74.33588057277046, 67.41971617548266, 61.626609520407456, 47.250760544967605, 63.08804244970428, 53.646802873050504, 54.598150033144236, 47.62135285300038, 47.62135285300038, 32.92198346312245, 52.301824199806006, 56.33128135834348, 58.34690118774256, 33.31005738633129, 40.41571724385996, 36.72712654703974, 46.70027251047952, 40.57389988868238, 40.57389988868238, 47.435694792893365, 41.943695056893915, 37.01518097066168, 44.387981683814154, 44.99901609609206, 41.780172086809024, 40.81233544283598, 36.87087245606033, 36.87087245606033, 36.87087245606033, 36.87087245606033, 37.8190332109868, 37.160054292819765, 40.81233544283598, 37.524723159601, 42.190180314697045, 32.033969127372174, 34.97688798216538, 34.501942437892716, 36.87087245606033, 29.626578330054702, 27.561124577174567, 38.0412795852805, 64.83690740886694, 38.41459632502361, 28.1047190510635, 47.15856396830611, 35.7364749685521, 50.49503426195374, 22.189281900071105, 50.990565478815945, 57.89284200037607, 75.21212841006654, 33.96703939987107, 33.375179667439795, 56.55175575986736, 66.37447035867858, 61.14702744030778, 64.83690740886694, 51.09025374715126, 52.40407602298735, 65.34542957624015, 35.04526897307925, 52.50652775191775, 52.097918711212095, 38.94340253332466, 43.02230705927991, 47.99485175252902, 46.33684810653952, 46.97471015107385, 62.597087408320306, 35.45837151398557, 40.57389988868238, 37.89297069129353, 36.37020880080945, 41.2128458403547, 36.15772522700737, 38.26483200699776, 45.61846186294896, 47.15856396830611, 38.41459632502361, 48.56059987746486, 51.49095957359226, 54.811841150452466, 54.811841150452466, 50.5937537501147, 37.5980852538711, 33.63694444585419, 48.56059987746486, 58.80452160545304, 63.83170433047907, 46.427438201089, 48.94146538270724, 30.62678644311483, 38.0412795852805, 62.3530444922685, 57.442316329639176, 39.32557222442699, 37.08754689283638, 43.699809813576366, 61.38635013907805, 54.38529201854992, 60.908637772317434, 55.89290784893326, 42.190180314697045, 49.22907356066477, 33.4404292647954, 37.67159077349981, 48.94146538270724, 33.11545195869231, 37.232703447205715, 34.300373728940066, 32.033969127372174, 35.25121511458854, 25.740017318702005, 27.4536739354601, 39.172256347947744, 44.82358155812517, 28.883810562338418, 40.17959882934157, 45.61846186294896, 66.63425269055388, 41.45503620813183, 27.668995768810042, 40.41571724385996, 37.89297069129353, 98.09510821132314, 42.68750480955783, 32.92198346312245, 38.867415430562446, 28.325147094540903, 29.396022468352484, 33.76859572409541, 36.72712654703974, 39.32557222442699, 44.387981683814154, 41.861853726676614, 42.3553080116137, 35.38918446958933, 46.97471015107385, 37.08754689283638, 31.72265993608251, 33.63694444585419, 49.32531805312982, 57.442316329639176, 48.75066069158903, 48.75066069158903, 37.524723159601, 45.79700747446617, 47.99485175252902, 25.539706914745466, 50.89107172411317, 31.9714638416083, 47.435694792893365, 44.73612093895834, 33.50580642730562, 26.453556883236853, 31.784678801853513, 31.72265993608251, 42.68750480955783, 31.598985005566618, 27.507346790200362, 23.16350836406023, 28.1047190510635, 39.86691871057298, 34.300373728940066, 41.943695056893915, 41.943695056893915, 34.4346215959791, 56.33128135834348, 50.6926662381911, 43.3597351955915, 39.172256347947744, 47.71445419747699, 37.89297069129353, 37.89297069129353, 49.42175070681059, 33.375179667439795, 33.375179667439795, 66.63425269055388, 53.43765380953904, 43.275130861102994, 43.275130861102994, 40.1011996370117, 40.25815129472595, 37.305494633128845, 57.21836996148928, 52.097918711212095, 35.04526897307925, 38.115651564714454, 59.964336869699544, 75.50650035756513, 46.51820540231633, 44.12865633779556, 50.1020787877761, 59.49769069799599, 34.908640417532865, 68.21443845541918, 70.10541234668786, 37.9670527218579, 48.37128004023317, 56.773093073800744, 50.29817277977263, 53.333385249853684, 52.40407602298735, 36.016759615546256, 116.49082330627213, 66.63425269055388, 28.827451925481085, 106.06610162605153, 37.160054292819765, 40.41571724385996, 35.45837151398557, 90.36944839098756, 58.34690118774256, 32.47494113727571, 40.49473132867618, 42.27266353442275, 60.908637772317434, 50.004318665090636, 62.10995300937982, 59.49769069799599, 46.51820540231633, 45.97625189374871, 33.50580642730562, 48.56059987746486, 38.48969815122384, 42.93836112094585, 45.97625189374871, 46.427438201089, 39.94485987582193, 48.27689708476307, 52.9183415530491, 44.21492934326177, 59.265731184583274, 49.42175070681059, 59.964336869699544, 55.6750020519895, 46.51820540231633, 53.96206231832688, 66.37447035867858, 32.28521466114651, 61.14702744030778, 35.59715165620933, 61.867809250367884, 61.867809250367884, 47.15856396830611, 31.475792237910618, 40.65322322588273, 57.66713919909174, 31.9714638416083, 24.133327041205547, 31.660762082694095, 58.119428177447986, 35.66674528344124, 58.119428177447986, 71.76791429061606, 33.57131140436464, 42.77096031537661, 31.90908051733889, 35.87634357692165, 45.88654216244616, 28.547303976590324, 56.11166650505244, 40.41571724385996, 58.119428177447986, 36.72712654703974, 56.11166650505244, 62.597087408320306, 36.15772522700737, 47.15856396830611, 37.37842812826559, 62.597087408320306, 60.434642992688794, 34.03344610288473, 41.53608219643496, 41.53608219643496, 73.46984133199462, 32.9863470466753, 28.65903496526287, 34.908640417532865, 34.501942437892716, 35.25121511458854, 43.444504934395866, 34.908640417532865, 35.32013242426123, 42.27266353442275, 59.964336869699544, 37.8190332109868, 39.01953819342525, 25.689792910615846, 38.64034257045414, 44.99901609609206, 51.290215346585704, 50.6926662381911, 45.79700747446617, 24.417803063601138, 46.15619785585283, 44.47476167904283, 63.3349620664254, 33.11545195869231, 47.99485175252902, 62.10995300937982, 43.95661501095434, 36.299242562272255, 47.99485175252902, 54.811841150452466, 31.598985005566618, 43.444504934395866, 39.94485987582193, 33.702705802144536, 44.12865633779556, 43.87084603329354, 40.73270164287187, 32.47494113727571, 43.3597351955915, 29.9173176993003, 45.086990684122, 39.94485987582193, 49.22907356066477, 53.333385249853684, 36.37020880080945, 41.617286632367644, 35.7364749685521, 43.5294404008871, 55.02636863114642, 60.908637772317434, 71.20941195067294, 42.190180314697045, 30.26997357328364, 37.232703447205715, 55.457945789238345, 49.9067492938035, 59.03467599561125, 58.119428177447986, 34.4346215959791, 39.095822701298815, 36.228414794400415, 44.73612093895834, 37.305494633128845, 55.02636863114642, 57.21836996148928, 55.6750020519895, 23.299630451611073, 51.09025374715126, 54.067559962798995, 31.35307975333966, 41.2128458403547, 29.74253347911242, 51.591626129191596, 36.87087245606033, 33.1801937796528, 26.557093177483523, 28.715064341305684, 44.82358155812517, 34.77254452624438, 42.93836112094585, 39.94485987582193, 36.65546388385301, 29.396022468352484, 49.51837188956857, 51.99626426796157, 58.34690118774256, 53.12545808585662, 32.79363281328884, 45.88654216244616, 46.70027251047952, 22.494734284035275, 53.96206231832688, 65.8579401324037, 22.450842133517945, 32.79363281328884, 25.34095533871963, 52.097918711212095, 28.2698785323965, 48.08868329990042, 40.57389988868238, 36.299242562272255, 33.245062173048, 52.40407602298735, 26.09434828302453, 30.987805310145493, 42.93836112094585, 55.89290784893326, 49.9067492938035, 41.29341833835342, 27.61500750152876, 24.9968545761854, 51.290215346585704, 28.940279382223952, 34.4346215959791, 44.04255166984753, 31.048387510438282, 76.99573263552006, 50.990565478815945, 48.65553748142455, 44.12865633779556, 52.81508628524973, 76.39654709045266, 52.609179777419676, 67.68358948381629, 23.4365524707797, 31.35307975333966, 32.28521466114651, 57.21836996148928, 45.44061233480379, 40.81233544283598, 33.375179667439795, 39.172256347947744, 36.15772522700737, 35.66674528344124, 36.15772522700737, 48.84596987082467, 48.08868329990042, 27.940524471445343, 49.9067492938035, 43.10641711474324, 56.11166650505244, 58.80452160545304, 36.798929313140206, 55.6750020519895, 67.68358948381629, 57.21836996148928, 57.442316329639176, 31.846818916590372, 38.79157659527073, 46.51820540231633, 36.51255777326842, 37.5980852538711, 42.10785803778799, 37.8190332109868, 35.04526897307925, 40.33685733281881, 51.99626426796157, 46.70027251047952, 32.47494113727571, 39.556671887793456, 53.646802873050504, 52.9183415530491, 38.33964103892552, 34.56939489431364, 51.894808174781225, 42.93836112094585, 30.56702687878971, 61.38635013907805, 61.38635013907805, 59.49769069799599, 63.83170433047907, 64.58413226217957, 51.49095957359226, 48.37128004023317, 47.06654728734682, 67.1568716116849, 42.3553080116137, 49.61518196998445, 27.4536739354601, 45.44061233480379, 57.442316329639176, 42.68750480955783, 42.52108200006278, 37.74523999888851, 36.37020880080945, 42.10785803778799, 38.48969815122384, 42.10785803778799, 42.52108200006278, 55.457945789238345, 69.83209724580306, 62.3530444922685, 83.57807709540792, 25.34095533871963, 32.92198346312245, 55.02636863114642, 41.617286632367644, 42.52108200006278, 42.02569638966092, 46.427438201089, 42.68750480955783, 56.33128135834348, 47.52843316962603, 54.067559962798995, 50.396507397123145, 44.99901609609206, 37.67159077349981, 75.50650035756513, 53.542126217761066, 74.33588057277046, 39.479488162280596, 39.479488162280596, 50.396507397123145, 73.18340956555208, 51.49095957359226, 30.034410770075546, 45.351947753528236, 31.291903031786116, 45.61846186294896, 57.21836996148928, 31.846818916590372, 55.6750020519895, 35.94648303373688, 67.68358948381629, 33.4404292647954, 47.62135285300038, 42.27266353442275, 44.30137101535166, 47.34313736903344, 57.442316329639176, 42.93836112094585, 33.11545195869231, 35.32013242426123, 35.45837151398557, 29.453492679719986, 41.45503620813183, 36.87087245606033, 38.41459632502361, 43.19069160819034, 62.3530444922685, 34.56939489431364, 35.32013242426123, 36.15772522700737, 35.25121511458854, 64.33234259239626, 37.232703447205715, 26.66103470132838, 24.275148340672132, 33.11545195869231, 37.37842812826559, 36.299242562272255, 59.73055807527518, 72.33079701870139, 37.232703447205715, 58.119428177447986, 52.71203249107963, 38.94340253332466, 41.132430556936306, 35.38918446958933, 31.291903031786116, 56.773093073800744, 56.773093073800744, 40.41571724385996, 34.300373728940066, 56.33128135834348, 47.250760544967605, 32.53843073806492, 63.3349620664254, 56.11166650505244, 27.561124577174567, 53.43765380953904, 40.49473132867618, 39.172256347947744, 74.62682298320091, 39.556671887793456, 45.7076474882609, 43.3597351955915, 42.02569638966092, 40.33685733281881, 36.798929313140206, 39.479488162280596, 36.37020880080945, 30.806767041912167, 21.973644325590612, 48.56059987746486, 36.58394105020859, 44.82358155812517, 47.99485175252902, 83.25223705241488, 71.48811770889668, 51.79355004464666, 34.70469568042362, 39.172256347947744, 41.45503620813183, 48.27689708476307, 49.809370301716946, 38.71588573814723, 66.63425269055388, 38.64034257045414, 44.21492934326177, 44.21492934326177, 38.94340253332466, 34.63697922255266, 37.37842812826559, 36.15772522700737, 33.90076227087091, 31.72265993608251, 39.94485987582193, 39.94485987582193, 33.11545195869231, 42.10785803778799, 34.4346215959791, 32.348333340301785, 26.2990087931472, 45.88654216244616, 44.21492934326177, 55.6750020519895, 37.37842812826559, 30.329152388418176, 34.367432111764096, 26.76538304079522, 31.53732846903916, 37.8190332109868, 33.245062173048, 36.65546388385301, 41.943695056893915, 35.806340977539655, 42.77096031537661, 36.51255777326842, 36.016759615546256, 44.04255166984753, 47.15856396830611, 38.71588573814723, 27.028044172168716, 26.453556883236853, 39.78912962559818, 44.99901609609206, 33.375179667439795, 33.11545195869231, 33.245062173048, 34.56939489431364, 43.19069160819034, 30.806767041912167, 28.2698785323965, 39.78912962559818, 33.50580642730562, 20.602050815688813, 27.346642204097876, 29.053548433146837, 28.214717811314713, 32.033969127372174, 37.37842812826559, 53.85677052285177, 26.817710263716034, 23.52828054362271, 34.16664924473404, 32.033969127372174, 50.29817277977263, 40.97207040739876, 51.591626129191596, 75.50650035756513, 42.68750480955783, 30.507383918455513, 35.59715165620933, 19.353835657005096, 45.17513726542669, 32.09659661306934, 22.016603538363892, 26.35042423874788, 31.35307975333966, 33.1801937796528, 28.491601937740562, 35.87634357692165, 27.77728915635237, 38.41459632502361, 36.08717359043372, 41.780172086809024, 27.668995768810042, 52.19977189231385, 24.322607071104724, 24.657426302639607, 35.94648303373688, 26.505274475545082, 28.1047190510635, 39.172256347947744, 50.89107172411317, 55.457945789238345, 37.37842812826559, 41.861853726676614, 42.27266353442275, 44.82358155812517, 40.17959882934157, 48.27689708476307, 40.17959882934157, 28.60311491479713, 46.51820540231633, 52.19977189231385, 62.84208548133934, 38.115651564714454, 43.444504934395866, 35.25121511458854, 41.37414835829223, 41.45503620813183, 49.133016862271724, 39.7114923241556, 41.29341833835342, 47.90120329104557, 55.457945789238345, 58.34690118774256, 34.367432111764096, 38.48969815122384, 40.49473132867618, 75.50650035756513, 34.501942437892716, 36.942956250241515, 41.617286632367644, 30.806767041912167, 41.2128458403547, 34.501942437892716, 36.58394105020859, 44.91121316574036, 40.97207040739876, 38.33964103892552, 46.246434773093995, 34.09998263323346, 54.598150033144236, 50.29817277977263, 28.996858600549206, 47.52843316962603, 47.62135285300038, 41.132430556936306, 34.09998263323346, 52.19977189231385, 44.73612093895834, 40.65322322588273, 59.03467599561125, 43.10641711474324, 40.25815129472595, 62.597087408320306, 62.597087408320306, 54.173263858709824, 46.7915731119465, 43.3597351955915, 37.305494633128845, 39.78912962559818, 31.9714638416083, 40.65322322588273, 32.159346537604755, 30.62678644311483, 75.21212841006654, 39.32557222442699, 61.14702744030778, 56.33128135834348, 35.18243227767251, 61.867809250367884, 48.84596987082467, 62.3530444922685, 69.55984769951485, 70.93179276322707, 69.01852867010916, 75.21212841006654, 57.89284200037607, 66.11570082213345, 74.04607243930377, 61.38635013907805, 34.09998263323346, 22.059646737841184, 26.975306541751994, 50.990565478815945, 84.563269020706, 40.49473132867618, 28.2698785323965, 42.02569638966092, 33.57131140436464, 42.68750480955783, 46.066137010589394, 37.9670527218579, 32.033969127372174, 46.066137010589394, 46.7915731119465, 62.597087408320306, 51.99626426796157, 51.99626426796157, 50.004318665090636, 46.427438201089, 54.38529201854992, 38.867415430562446, 38.48969815122384, 47.99485175252902, 78.51433738825796, 38.94340253332466, 44.91121316574036, 33.4404292647954, 37.5980852538711, 45.263456176258764, 51.99626426796157, 49.32531805312982, 43.19069160819034, 43.19069160819034, 45.17513726542669, 42.854578979512624, 33.90076227087091, 51.39048944047805, 51.39048944047805, 30.56702687878971, 47.15856396830611, 53.75168417471723, 26.505274475545082, 46.427438201089, 53.85677052285177, 54.38529201854992, 45.97625189374871, 29.626578330054702, 34.16664924473404, 31.784678801853513, 41.05217218133816, 26.609013186724894, 47.52843316962603, 47.52843316962603, 30.26997357328364, 25.790339917193062, 50.396507397123145, 51.79355004464666, 30.034410770075546, 30.034410770075546, 41.132430556936306, 37.45150421083523, 29.56877039069624, 29.56877039069624, 41.617286632367644, 43.275130861102994, 48.46584751755162, 45.086990684122, 39.556671887793456, 44.82358155812517, 41.37414835829223, 53.02179868836547, 50.1020787877761, 39.32557222442699, 44.387981683814154, 61.626609520407456, 47.99485175252902, 44.99901609609206, 56.55175575986736, 41.943695056893915, 41.943695056893915, 39.32557222442699, 39.32557222442699, 40.41571724385996, 38.19016894386643, 45.351947753528236, 46.246434773093995, 47.52843316962603, 44.5617113320772, 37.232703447205715, 37.232703447205715, 42.190180314697045, 46.33684810653952, 65.8579401324037, 101.20898120494374, 46.427438201089, 75.21212841006654, 25.58963789966816, 29.9173176993003, 42.10785803778799, 52.40407602298735, 43.5294404008871, 44.99901609609206, 35.32013242426123, 46.51820540231633, 42.93836112094585, 46.427438201089, 32.66578255521719, 36.37020880080945, 48.65553748142455, 43.02230705927991, 37.232703447205715, 43.87084603329354, 46.246434773093995, 56.33128135834348, 70.93179276322707, 49.51837188956857, 47.06654728734682, 50.5937537501147, 42.3553080116137, 42.438114061533675, 69.28865955361954, 74.62682298320091, 44.82358155812517, 68.7494509271083, 56.11166650505244, 55.02636863114642, 46.51820540231633, 37.45150421083523, 29.511075247281195, 33.96703939987107, 59.964336869699544, 55.6750020519895, 49.42175070681059, 60.434642992688794, 50.990565478815945, 51.49095957359226, 35.32013242426123, 31.660762082694095, 54.811841150452466, 46.066137010589394, 43.444504934395866, 47.99485175252902, 43.78524440968234, 39.78912962559818, 47.250760544967605, 34.4346215959791, 53.12545808585662, 43.95661501095434, 43.87084603329354, 22.059646737841184, 41.37414835829223, 38.26483200699776, 38.26483200699776, 40.49473132867618, 42.3553080116137, 38.115651564714454, 47.435694792893365, 60.908637772317434, 38.41459632502361, 40.81233544283598, 41.37414835829223, 43.699809813576366, 59.265731184583274, 31.72265993608251, 32.159346537604755, 40.022953418667065, 47.71445419747699, 24.370158584949497, 44.12865633779556, 43.02230705927991, 39.01953819342525, 40.1011996370117, 35.806340977539655, 48.94146538270724, 40.33685733281881, 43.699809813576366, 52.71203249107963, 60.199030648454944, 66.11570082213345, 31.35307975333966, 41.617286632367644, 33.83461446305701, 42.52108200006278, 28.771203256660744, 31.90908051733889, 30.74665629604239, 60.199030648454944, 36.72712654703974, 40.57389988868238, 42.3553080116137, 55.457945789238345, 33.05083646319603, 38.26483200699776, 47.71445419747699, 42.52108200006278, 39.78912962559818, 35.66674528344124, 36.44131378072682, 41.37414835829223, 44.04255166984753, 51.692489491288626, 56.55175575986736, 38.33964103892552, 51.79355004464666, 52.097918711212095, 60.67117749756472, 41.861853726676614, 53.85677052285177, 40.022953418667065, 34.84052601883783, 43.275130861102994, 63.582848099201826, 55.24173574865985, 34.70469568042362, 35.806340977539655, 26.505274475545082, 22.494734284035275, 42.52108200006278, 35.32013242426123, 27.995149135337257, 34.23344619169951, 44.5617113320772, 29.110349096271467, 51.894808174781225, 32.09659661306934, 43.19069160819034, 34.501942437892716, 31.598985005566618, 35.11378365112735, 36.942956250241515, 33.4404292647954, 27.940524471445343, 33.702705802144536, 32.79363281328884, 50.6926662381911, 33.76859572409541, 44.21492934326177, 49.037147591523336, 42.3553080116137, 60.199030648454944, 62.84208548133934, 39.7114923241556, 46.066137010589394, 46.7915731119465, 37.305494633128845, 42.190180314697045, 34.70469568042362, 47.99485175252902, 32.9863470466753, 39.479488162280596, 41.861853726676614, 36.72712654703974, 51.49095957359226, 34.16664924473404, 42.27266353442275, 64.33234259239626, 57.442316329639176, 41.2128458403547, 51.09025374715126, 57.66713919909174, 40.1011996370117, 38.867415430562446, 50.20003003478527, 46.70027251047952, 37.45150421083523, 51.290215346585704, 51.290215346585704, 37.37842812826559, 35.25121511458854, 35.59715165620933, 51.290215346585704, 46.70027251047952, 31.291903031786116, 62.597087408320306, 62.597087408320306, 65.8579401324037, 62.84208548133934, 52.097918711212095, 51.692489491288626, 51.692489491288626, 43.10641711474324, 50.20003003478527, 43.87084603329354, 44.30137101535166, 38.33964103892552, 69.28865955361954, 69.28865955361954, 74.91890411002962, 35.04526897307925, 47.34313736903344, 70.65525591042105, 44.30137101535166, 55.89290784893326, 34.908640417532865, 56.773093073800744, 56.773093073800744, 53.85677052285177, 51.1901369093998, 41.05217218133816, 42.68750480955783, 58.34690118774256, 42.93836112094585, 36.942956250241515, 45.529450258314064, 28.380523708580828, 42.10785803778799, 36.37020880080945, 50.79177210350426, 50.79177210350426, 73.46984133199462, 39.86691871057298, 53.646802873050504, 52.9183415530491, 46.60915005647103, 46.88305220915621, 51.290215346585704, 66.37447035867858, 47.52843316962603, 48.65553748142455, 51.1901369093998, 48.27689708476307, 36.08717359043372, 34.97688798216538, 52.19977189231385, 62.10995300937982, 51.692489491288626, 66.11570082213345, 45.263456176258764, 41.698649825700315, 46.427438201089, 55.89290784893326, 43.10641711474324, 62.3530444922685, 49.133016862271724, 48.37128004023317, 42.02569638966092, 66.11570082213345, 44.04255166984753, 43.87084603329354, 63.83170433047907, 77.29708541258049, 43.19069160819034, 40.73270164287187, 44.387981683814154, 48.37128004023317, 53.333385249853684, 49.42175070681059, 44.5617113320772, 48.75066069158903, 25.94189877662824, 46.427438201089, 42.93836112094585, 35.52769382137767, 39.40245503911115, 40.81233544283598, 45.263456176258764, 41.2128458403547, 54.173263858709824, 41.617286632367644, 41.2128458403547, 43.95661501095434, 50.49503426195374, 25.740017318702005, 50.396507397123145, 47.62135285300038, 48.94146538270724, 35.66674528344124, 51.79355004464666, 45.79700747446617, 52.097918711212095, 59.03467599561125, 42.52108200006278, 49.037147591523336, 42.02569638966092, 45.97625189374871, 32.09659661306934, 45.7076474882609, 49.037147591523336, 35.66674528344124, 49.32531805312982, 42.3553080116137, 41.698649825700315, 54.598150033144236, 57.66713919909174, 48.46584751755162, 39.095822701298815, 46.7915731119465, 59.964336869699544, 83.25223705241488, 38.33964103892552, 114.68480048885151, 107.31637526839133, 58.34690118774256, 36.44131378072682, 34.63697922255266, 39.479488162280596, 30.151962130172098, 39.78912962559818, 41.05217218133816, 51.894808174781225, 38.867415430562446, 38.867415430562446, 34.84052601883783, 40.41571724385996, 38.26483200699776, 40.65322322588273, 40.97207040739876, 62.3530444922685, 37.74523999888851, 42.68750480955783, 34.77254452624438, 32.033969127372174, 38.71588573814723, 44.387981683814154, 47.06654728734682, 41.05217218133816, 31.10908815096766, 34.908640417532865, 47.807737558209425, 47.34313736903344, 37.45150421083523, 48.182698291098816, 40.65322322588273, 47.15856396830611, 46.88305220915621, 32.9863470466753, 29.858942463791767, 29.858942463791767, 37.74523999888851, 46.97471015107385, 36.44131378072682, 42.68750480955783, 36.798929313140206, 54.38529201854992, 54.38529201854992, 36.58394105020859, 37.74523999888851, 38.115651564714454, 56.9952966774873, 50.49503426195374, 67.41971617548266, 41.29341833835342, 34.908640417532865, 36.016759615546256, 40.97207040739876, 44.47476167904283, 63.08804244970428, 62.3530444922685, 40.892124929554, 47.06654728734682, 58.34690118774256, 58.34690118774256, 44.04255166984753, 57.66713919909174, 50.990565478815945, 50.1020787877761, 41.698649825700315, 59.73055807527518, 57.442316329639176, 90.7231444115574, 49.133016862271724, 50.396507397123145, 39.32557222442699, 42.10785803778799, 30.507383918455513, 36.798929313140206, 62.3530444922685, 43.78524440968234, 33.63694444585419, 46.51820540231633, 42.10785803778799, 24.948080241282657, 58.34690118774256, 47.06654728734682, 37.5980852538711, 86.56862915519338, 49.42175070681059, 38.94340253332466, 36.87087245606033, 29.396022468352484, 42.52108200006278, 46.7915731119465, 57.21836996148928, 46.70027251047952, 55.457945789238345, 38.48969815122384, 31.048387510438282, 23.390822626823073, 29.684499285876434, 58.57526450222688, 58.57526450222688, 39.248839424943355, 43.78524440968234, 40.892124929554, 38.48969815122384, 35.11378365112735, 53.22932014095188, 63.08804244970428, 44.04255166984753, 69.55984769951485, 61.867809250367884, 45.263456176258764, 53.333385249853684, 60.199030648454944, 56.33128135834348, 43.3597351955915, 49.22907356066477, 58.80452160545304, 82.28231911219243, 91.79256012938836, 60.67117749756472, 65.60118435636834, 50.29817277977263, 38.19016894386643, 41.943695056893915, 59.964336869699544, 52.301824199806006, 58.57526450222688, 34.501942437892716, 62.3530444922685, 75.21212841006654, 65.8579401324037, 56.11166650505244, 45.88654216244616, 50.004318665090636, 61.867809250367884, 41.698649825700315, 43.275130861102994, 56.9952966774873, 33.4404292647954, 55.24173574865985, 58.80452160545304, 36.299242562272255, 47.71445419747699, 60.67117749756472, 42.93836112094585, 51.39048944047805, 44.99901609609206, 102.80279074731456, 55.02636863114642, 50.6926662381911, 48.65553748142455, 42.77096031537661, 42.93836112094585, 42.77096031537661, 42.77096031537661, 53.333385249853684, 50.5937537501147, 33.31005738633129, 56.9952966774873, 66.63425269055388, 66.11570082213345, 51.290215346585704, 64.33234259239626, 37.524723159601, 34.84052601883783, 50.20003003478527, 56.33128135834348, 35.11378365112735, 48.94146538270724, 44.73612093895834, 48.75066069158903, 46.33684810653952, 35.11378365112735, 31.72265993608251, 28.15966472087403, 40.33685733281881, 34.23344619169951, 38.71588573814723, 41.617286632367644, 36.299242562272255, 32.09659661306934, 28.491601937740562, 33.375179667439795, 35.59715165620933, 32.60204446315635, 29.396022468352484, 38.115651564714454, 38.79157659527073, 36.87087245606033, 36.87087245606033, 45.086990684122, 62.10995300937982, 48.75066069158903, 41.37414835829223, 35.87634357692165, 32.79363281328884, 29.053548433146837, 27.080884906424416, 31.10908815096766, 31.10908815096766, 44.387981683814154, 44.387981683814154, 30.62678644311483, 29.33866439394716, 36.016759615546256, 42.68750480955783, 32.60204446315635, 41.617286632367644, 32.47494113727571, 33.1801937796528, 33.1801937796528, 33.1801937796528, 33.1801937796528, 33.1801937796528, 33.1801937796528, 33.1801937796528, 33.1801937796528, 32.09659661306934, 37.08754689283638, 37.08754689283638, 32.72964525738904, 41.617286632367644, 41.617286632367644, 34.367432111764096, 31.660762082694095, 34.908640417532865, 42.27266353442275, 42.02569638966092, 40.57389988868238, 38.48969815122384, 76.39654709045266, 27.995149135337257, 38.33964103892552, 37.305494633128845, 27.4536739354601, 29.800681131145613, 45.086990684122, 36.58394105020859, 56.773093073800744, 53.85677052285177, 36.15772522700737, 33.96703939987107, 38.33964103892552, 38.41459632502361, 34.16664924473404, 34.16664924473404, 40.33685733281881, 48.182698291098816, 45.529450258314064, 23.4365524707797, 42.68750480955783, 37.160054292819765, 49.9067492938035, 49.9067492938035, 44.30137101535166, 42.93836112094585, 33.05083646319603, 24.275148340672132, 37.37842812826559, 38.26483200699776, 35.7364749685521, 34.97688798216538, 30.151962130172098, 30.26997357328364, 42.27266353442275, 43.10641711474324, 36.299242562272255, 33.96703939987107, 34.908640417532865, 34.03344610288473, 29.396022468352484, 34.908640417532865, 34.84052601883783, 30.388446900124176, 44.21492934326177, 42.52108200006278, 33.63694444585419, 30.866995306309306, 36.51255777326842, 28.771203256660744, 33.4404292647954, 49.51837188956857, 56.11166650505244, 29.22428378123494, 28.325147094540903, 33.50580642730562, 23.482371718117374, 43.10641711474324, 42.3553080116137, 43.10641711474324, 33.1801937796528, 33.4404292647954, 42.854578979512624, 42.854578979512624, 37.74523999888851, 44.5617113320772, 37.45150421083523, 30.151962130172098, 52.9183415530491, 48.75066069158903, 33.90076227087091, 44.21492934326177, 22.98324851186175, 40.49473132867618, 39.248839424943355, 41.2128458403547, 31.41437607743883, 40.892124929554, 31.169907463288272, 50.004318665090636, 70.93179276322707, 60.908637772317434, 24.322607071104724, 51.1901369093998, 51.1901369093998, 55.02636863114642, 36.016759615546256, 45.086990684122, 50.29817277977263, 51.99626426796157, 59.49769069799599, 54.811841150452466, 54.811841150452466, 69.83209724580306, 41.943695056893915, 34.501942437892716, 36.72712654703974, 32.66578255521719, 32.66578255521719, 45.263456176258764, 36.87087245606033, 34.23344619169951, 49.22907356066477, 43.444504934395866, 33.63694444585419, 33.63694444585419, 33.63694444585419, 41.29341833835342, 31.475792237910618, 32.60204446315635, 37.89297069129353, 32.033969127372174, 35.38918446958933, 35.66674528344124, 29.684499285876434, 31.10908815096766, 36.299242562272255, 39.32557222442699, 37.305494633128845, 47.34313736903344, 34.501942437892716, 38.48969815122384, 30.447857334592324, 26.975306541751994, 29.626578330054702, 42.52108200006278, 37.01518097066168, 49.42175070681059, 42.68750480955783, 41.617286632367644, 48.08868329990042, 40.33685733281881, 49.51837188956857, 40.022953418667065, 37.08754689283638, 64.08153455750669, 53.75168417471723, 50.5937537501147, 44.82358155812517, 37.37842812826559, 34.77254452624438, 39.7114923241556, 41.943695056893915, 35.45837151398557, 46.97471015107385, 33.96703939987107, 32.60204446315635, 31.784678801853513, 49.42175070681059, 42.52108200006278, 44.04255166984753, 36.299242562272255, 47.06654728734682, 55.24173574865985, 38.26483200699776, 42.52108200006278, 35.25121511458854, 43.02230705927991, 37.74523999888851, 45.263456176258764, 30.388446900124176, 38.71588573814723, 22.98324851186175, 27.72308958496738, 49.9067492938035, 42.52108200006278, 35.04526897307925, 35.04526897307925, 40.57389988868238, 40.57389988868238, 31.660762082694095, 40.022953418667065, 39.479488162280596, 69.83209724580306, 43.95661501095434, 45.7076474882609, 35.66674528344124, 38.115651564714454, 43.19069160819034, 46.7915731119465, 36.58394105020859, 42.68750480955783, 37.9670527218579, 39.01953819342525, 36.37020880080945, 39.40245503911115, 39.40245503911115, 36.016759615546256, 44.30137101535166, 45.79700747446617, 35.52769382137767, 39.634006510082365, 37.305494633128845, 55.24173574865985, 55.24173574865985, 40.65322322588273, 37.67159077349981, 50.990565478815945, 38.0412795852805, 41.2128458403547, 37.8190332109868, 35.7364749685521, 44.99901609609206, 37.160054292819765, 37.160054292819765, 32.53843073806492, 35.32013242426123, 38.26483200699776, 32.28521466114651, 33.76859572409541, 33.76859572409541, 39.172256347947744, 39.172256347947744, 33.702705802144536, 43.275130861102994, 40.892124929554, 48.65553748142455, 44.387981683814154, 49.22907356066477, 42.3553080116137, 46.246434773093995, 46.246434773093995, 44.04255166984753, 33.83461446305701, 32.85774546700983, 32.85774546700983, 45.263456176258764, 37.5980852538711, 34.77254452624438, 34.77254452624438, 38.115651564714454, 28.15966472087403, 34.16664924473404, 45.263456176258764, 41.943695056893915, 41.05217218133816, 36.228414794400415, 33.702705802144536, 38.94340253332466, 59.73055807527518, 47.99485175252902, 38.26483200699776, 34.367432111764096, 40.892124929554, 40.892124929554, 39.248839424943355, 47.435694792893365, 34.367432111764096, 44.91121316574036, 50.49503426195374, 53.85677052285177, 41.861853726676614, 41.861853726676614, 55.24173574865985, 70.93179276322707, 33.11545195869231, 35.38918446958933, 46.427438201089, 45.086990684122, 47.99485175252902, 40.892124929554, 51.894808174781225, 48.94146538270724, 44.82358155812517, 33.245062173048, 66.37447035867858, 32.92198346312245, 38.56494680401705, 44.47476167904283, 44.91121316574036, 41.29341833835342, 29.684499285876434, 41.37414835829223, 27.507346790200362, 34.16664924473404, 37.89297069129353, 26.09434828302453, 58.57526450222688, 47.90120329104557, 39.172256347947744, 51.99626426796157, 44.12865633779556, 37.524723159601, 39.01953819342525, 59.73055807527518, 40.17959882934157, 39.248839424943355, 37.5980852538711, 43.10641711474324, 43.87084603329354, 49.42175070681059, 45.97625189374871, 61.14702744030778, 44.82358155812517, 57.89284200037607, 53.96206231832688, 43.78524440968234, 35.11378365112735, 42.02569638966092, 50.396507397123145, 52.19977189231385, 53.646802873050504, 44.47476167904283, 51.1901369093998, 39.94485987582193, 41.53608219643496, 43.444504934395866, 38.71588573814723, 45.263456176258764, 48.65553748142455, 56.33128135834348, 26.453556883236853, 37.67159077349981, 38.0412795852805, 36.15772522700737, 54.173263858709824, 36.798929313140206, 36.798929313140206, 45.61846186294896, 41.53608219643496, 42.52108200006278, 53.75168417471723, 70.93179276322707, 45.351947753528236, 38.26483200699776, 33.63694444585419, 61.867809250367884, 43.19069160819034, 51.591626129191596, 33.375179667439795, 41.861853726676614, 36.65546388385301, 31.53732846903916, 51.79355004464666, 71.20941195067294, 72.33079701870139, 61.38635013907805, 34.501942437892716, 34.300373728940066, 67.41971617548266, 59.964336869699544, 39.095822701298815, 54.811841150452466, 63.3349620664254, 44.99901609609206, 36.798929313140206, 46.15619785585283, 42.10785803778799, 55.457945789238345, 67.9484955630781, 47.250760544967605, 49.9067492938035, 48.75066069158903, 53.646802873050504, 45.17513726542669, 47.52843316962603, 46.15619785585283, 49.809370301716946, 58.119428177447986, 51.692489491288626, 56.11166650505244, 49.42175070681059, 59.964336869699544, 81.32370105824417, 36.228414794400415, 46.33684810653952, 76.39654709045266, 38.33964103892552, 46.066137010589394, 44.47476167904283, 44.47476167904283, 33.90076227087091, 36.798929313140206, 33.90076227087091, 46.60915005647103, 33.96703939987107, 40.73270164287187, 28.715064341305684, 39.7114923241556, 31.10908815096766, 50.89107172411317, 43.87084603329354, 39.556671887793456, 42.93836112094585, 41.2128458403547, 44.99901609609206, 30.806767041912167, 46.7915731119465, 33.63694444585419, 47.99485175252902, 45.7076474882609, 40.49473132867618, 37.89297069129353, 35.7364749685521, 51.39048944047805, 45.97625189374871, 45.97625189374871, 59.964336869699544, 31.10908815096766, 32.9863470466753, 36.51255777326842, 33.31005738633129, 40.17959882934157, 48.56059987746486, 50.89107172411317, 48.37128004023317, 90.01713130052181, 53.43765380953904, 57.442316329639176, 48.182698291098816, 51.99626426796157, 34.56939489431364, 38.41459632502361, 52.301824199806006, 45.7076474882609, 46.246434773093995, 38.867415430562446, 44.64883097460398, 42.77096031537661, 39.634006510082365, 61.14702744030778, 62.3530444922685, 60.908637772317434, 30.987805310145493, 45.79700747446617, 51.09025374715126, 30.151962130172098, 43.699809813576366, 45.97625189374871, 47.62135285300038, 50.1020787877761, 61.626609520407456, 33.96703939987107, 30.034410770075546, 47.71445419747699, 48.46584751755162, 23.390822626823073, 26.24769367033534, 34.77254452624438, 35.11378365112735, 36.15772522700737, 36.72712654703974, 32.09659661306934, 42.854578979512624, 39.479488162280596, 43.10641711474324, 43.10641711474324, 50.49503426195374, 44.73612093895834, 42.10785803778799, 36.798929313140206, 43.87084603329354, 33.50580642730562, 31.475792237910618, 23.345182011801924, 19.929230034775262, 39.248839424943355, 39.634006510082365, 46.427438201089, 40.022953418667065, 34.70469568042362, 17.416566918622852, 32.79363281328884, 47.435694792893365, 34.84052601883783, 28.049880592281973, 35.7364749685521, 50.990565478815945, 39.78912962559818, 41.37414835829223, 35.04526897307925, 43.444504934395866, 52.9183415530491, 76.69555472159655, 40.25815129472595, 37.08754689283638, 38.41459632502361, 32.47494113727571, 27.293282919182133, 40.17959882934157, 41.45503620813183, 39.01953819342525, 35.45837151398557, 43.78524440968234, 39.86691871057298, 36.798929313140206, 35.94648303373688, 40.25815129472595, 45.529450258314064, 35.87634357692165, 40.892124929554, 40.49473132867618, 56.11166650505244, 34.300373728940066, 42.93836112094585, 45.61846186294896, 38.0412795852805, 38.19016894386643, 31.846818916590372, 33.4404292647954, 34.09998263323346, 28.940279382223952, 37.74523999888851, 37.67159077349981, 34.4346215959791, 33.4404292647954, 39.172256347947744, 29.053548433146837, 25.045724266551254, 27.293282919182133, 29.9173176993003, 34.908640417532865, 41.53608219643496, 38.867415430562446, 29.396022468352484, 35.32013242426123, 39.479488162280596, 40.25815129472595, 73.18340956555208, 76.99573263552006, 38.79157659527073, 36.37020880080945, 61.867809250367884, 35.7364749685521, 37.37842812826559, 81.00664999449546, 33.76859572409541, 35.94648303373688, 32.22221914035047, 27.028044172168716, 35.87634357692165, 30.806767041912167, 31.169907463288272, 43.78524440968234, 33.245062173048, 22.80439145296227, 27.240027749911206, 39.40245503911115, 39.01953819342525, 31.598985005566618, 39.01953819342525, 33.11545195869231, 32.09659661306934, 32.09659661306934, 28.380523708580828, 38.41459632502361, 24.657426302639607, 29.22428378123494, 50.5937537501147, 45.7076474882609, 43.61454191906839, 21.675267738479917, 24.370158584949497, 31.41437607743883, 37.45150421083523, 39.40245503911115, 34.97688798216538, 43.78524440968234, 45.79700747446617, 43.444504934395866, 49.133016862271724, 57.66713919909174, 21.42274322042474, 22.319678873017494, 50.20003003478527, 48.84596987082467, 68.21443845541918, 48.46584751755162, 38.94340253332466, 44.30137101535166, 31.846818916590372, 29.858942463791767, 69.28865955361954, 37.8190332109868, 41.2128458403547, 26.66103470132838, 28.65903496526287, 40.81233544283598, 30.093129052017023, 37.8190332109868, 37.8190332109868, 34.4346215959791, 30.68666283939559, 37.67159077349981, 41.132430556936306, 41.132430556936306, 34.97688798216538, 25.992616060150944, 30.507383918455513, 25.790339917193062, 26.401940203271806, 33.96703939987107, 36.58394105020859, 34.03344610288473, 41.45503620813183, 46.066137010589394, 25.94189877662824, 51.09025374715126, 44.12865633779556, 44.99901609609206, 42.60421214369869, 39.556671887793456, 35.87634357692165, 33.31005738633129, 28.380523708580828, 31.784678801853513, 28.380523708580828, 35.25121511458854, 30.447857334592324, 26.76538304079522, 31.048387510438282, 31.291903031786116, 35.52769382137767, 44.99901609609206, 45.44061233480379, 42.854578979512624, 26.505274475545082, 35.11378365112735, 35.11378365112735, 32.85774546700983, 32.85774546700983, 39.556671887793456, 39.556671887793456, 37.08754689283638, 34.63697922255266, 31.35307975333966, 26.505274475545082, 37.160054292819765, 46.70027251047952, 29.167260806600495, 32.348333340301785, 42.52108200006278, 43.699809813576366, 43.87084603329354, 47.90120329104557, 29.56877039069624, 32.53843073806492, 37.232703447205715, 51.79355004464666, 34.4346215959791, 37.9670527218579, 46.427438201089, 46.427438201089, 32.47494113727571, 47.435694792893365, 42.190180314697045, 45.529450258314064, 40.81233544283598, 49.32531805312982, 43.61454191906839, 41.53608219643496, 42.52108200006278, 37.08754689283638, 37.67159077349981, 42.68750480955783, 30.74665629604239, 43.87084603329354, 46.33684810653952, 28.996858600549206, 46.97471015107385, 30.56702687878971, 24.948080241282657, 47.06654728734682, 20.602050815688813, 22.538712245181248, 40.97207040739876, 36.942956250241515, 39.94485987582193, 34.97688798216538, 42.02569638966092, 39.172256347947744, 47.807737558209425, 27.40010580820802, 28.547303976590324, 30.151962130172098, 26.922671813996086, 21.717643615002626, 38.33964103892552, 56.11166650505244, 55.6750020519895, 34.501942437892716, 25.045724266551254, 35.87634357692165, 35.18243227767251, 36.798929313140206, 28.996858600549206, 29.9173176993003, 28.996858600549206, 39.94485987582193, 47.435694792893365, 27.72308958496738, 26.453556883236853, 22.715485556228362, 27.346642204097876, 32.53843073806492, 24.70563239932894, 44.5617113320772, 37.232703447205715, 43.699809813576366, 36.37020880080945, 47.435694792893365, 51.591626129191596, 52.81508628524973, 40.49473132867618, 43.3597351955915, 48.65553748142455, 44.12865633779556, 56.33128135834348, 35.38918446958933, 27.561124577174567, 34.4346215959791, 38.115651564714454, 26.713157919740368, 45.7076474882609, 33.4404292647954, 23.482371718117374, 42.02569638966092, 45.17513726542669, 35.66674528344124, 32.92198346312245, 31.10908815096766, 29.975807060355134, 45.7076474882609, 42.27266353442275, 26.975306541751994, 28.60311491479713, 30.447857334592324, 33.83461446305701, 53.43765380953904, 26.2990087931472, 23.71281513502692, 30.034410770075546, 55.24173574865985, 24.22778221261098, 31.10908815096766, 26.09434828302453, 32.60204446315635, 42.854578979512624, 41.53608219643496, 41.37414835829223, 57.66713919909174, 42.68750480955783, 41.698649825700315, 41.698649825700315, 59.73055807527518, 45.263456176258764, 43.5294404008871, 31.10908815096766, 41.943695056893915, 41.780172086809024, 34.77254452624438, 28.827451925481085, 27.61500750152876, 47.99485175252902, 48.84596987082467, 26.453556883236853, 33.11545195869231, 42.190180314697045, 24.753932740556596, 32.033969127372174, 35.94648303373688, 40.892124929554, 31.41437607743883, 40.33685733281881, 27.61500750152876, 35.87634357692165, 67.41971617548266, 44.99901609609206, 34.97688798216538, 40.1011996370117, 44.99901609609206, 39.01953819342525, 48.08868329990042, 53.646802873050504, 20.4816886421048, 37.89297069129353, 56.33128135834348, 26.505274475545082, 38.19016894386643, 48.65553748142455, 34.300373728940066, 36.37020880080945, 37.5980852538711, 31.53732846903916, 37.8190332109868, 40.892124929554, 34.16664924473404, 29.22428378123494, 21.760102337787384, 23.52828054362271, 35.59715165620933, 34.84052601883783, 37.232703447205715, 27.77728915635237, 32.9863470466753, 29.56877039069624, 26.87013978811538, 29.511075247281195, 24.465540688808968, 36.87087245606033, 27.77728915635237, 39.556671887793456, 39.40245503911115, 37.305494633128845, 30.21091022897123, 38.26483200699776, 34.77254452624438, 35.11378365112735, 30.987805310145493, 32.66578255521719, 47.06654728734682, 39.634006510082365, 38.33964103892552, 36.299242562272255, 38.867415430562446, 46.15619785585283, 25.63966650134361, 26.043432497666576, 32.47494113727571, 22.538712245181248, 43.444504934395866, 31.53732846903916, 41.132430556936306, 33.50580642730562, 38.33964103892552, 34.16664924473404, 33.76859572409541, 42.854578979512624, 42.190180314697045, 34.70469568042362, 41.617286632367644, 52.097918711212095, 34.300373728940066, 26.14536361045317, 35.04526897307925, 42.10785803778799, 35.25121511458854, 40.73270164287187, 31.53732846903916, 31.53732846903916, 51.591626129191596, 33.11545195869231, 37.45150421083523, 34.63697922255266, 42.52108200006278, 33.4404292647954, 35.25121511458854, 38.115651564714454, 33.96703939987107, 32.9863470466753, 28.715064341305684, 37.9670527218579, 29.511075247281195, 31.291903031786116, 30.74665629604239, 33.96703939987107, 48.37128004023317, 27.293282919182133, 23.25416777248505, 29.22428378123494, 25.24216031657971, 33.96703939987107, 27.186876493132687, 32.28521466114651, 24.22778221261098, 40.33685733281881, 35.52769382137767, 46.33684810653952, 40.17959882934157, 35.87634357692165, 34.84052601883783, 35.25121511458854, 30.987805310145493, 20.967397891114594, 35.32013242426123, 42.68750480955783, 40.49473132867618, 40.57389988868238, 33.96703939987107, 24.086237637543274, 38.41459632502361, 40.25815129472595, 25.39049775763493, 41.780172086809024, 30.034410770075546, 48.37128004023317, 39.40245503911115, 39.40245503911115, 40.33685733281881, 37.232703447205715, 37.232703447205715, 39.479488162280596, 32.66578255521719, 32.66578255521719, 32.66578255521719, 32.66578255521719, 32.66578255521719, 46.427438201089, 33.1801937796528, 35.11378365112735, 44.30137101535166, 43.3597351955915, 34.300373728940066, 33.76859572409541, 34.63697922255266, 34.63697922255266, 35.11378365112735, 37.01518097066168, 44.91121316574036, 33.1801937796528, 29.167260806600495, 25.689792910615846, 30.62678644311483, 35.66674528344124, 39.556671887793456, 36.44131378072682, 22.450842133517945, 26.2990087931472, 19.053782246709744, 22.938403163153815, 22.10277408821932, 39.556671887793456, 30.26997357328364, 30.034410770075546, 40.49473132867618, 28.049880592281973, 31.169907463288272, 31.230845679407448, 31.230845679407448, 40.41571724385996, 40.73270164287187, 28.883810562338418, 22.84897480427519, 36.228414794400415, 29.56877039069624, 27.40010580820802, 18.61240185635776, 19.581972547389178, 40.73270164287187, 24.370158584949497, 25.539706914745466, 31.10908815096766, 20.521731031400577, 32.47494113727571, 30.388446900124176, 31.784678801853513, 30.987805310145493, 39.479488162280596, 31.291903031786116, 33.57131140436464, 26.043432497666576, 23.52828054362271, 37.01518097066168, 26.043432497666576, 32.033969127372174, 35.38918446958933, 34.908640417532865, 32.09659661306934, 41.37414835829223, 36.51255777326842, 27.72308958496738, 44.47476167904283, 41.53608219643496, 24.513371642677637, 34.56939489431364, 29.110349096271467, 21.008389858307645, 29.684499285876434, 34.77254452624438, 37.67159077349981, 53.75168417471723, 50.20003003478527, 35.87634357692165, 40.33685733281881, 39.556671887793456, 36.37020880080945, 33.4404292647954, 33.1801937796528, 40.57389988868238, 39.94485987582193, 35.11378365112735, 31.72265993608251, 40.17959882934157, 32.60204446315635, 38.867415430562446, 43.61454191906839, 45.351947753528236, 33.63694444585419, 46.88305220915621, 50.79177210350426, 38.56494680401705, 38.48969815122384, 45.79700747446617, 46.97471015107385, 49.32531805312982, 35.7364749685521, 37.232703447205715, 40.892124929554, 27.72308958496738, 35.52769382137767, 46.88305220915621, 47.99485175252902, 42.68750480955783, 43.61454191906839, 42.3553080116137, 52.50652775191775, 34.97688798216538, 27.186876493132687, 46.60915005647103, 25.63966650134361, 37.01518097066168, 26.557093177483523, 58.57526450222688, 24.133327041205547, 50.004318665090636, 40.57389988868238, 35.52769382137767, 46.33684810653952, 44.47476167904283, 40.97207040739876, 38.115651564714454, 46.70027251047952, 45.88654216244616, 43.444504934395866, 43.10641711474324, 44.5617113320772, 39.248839424943355, 38.26483200699776, 28.883810562338418, 42.3553080116137, 42.3553080116137, 55.02636863114642, 47.15856396830611, 37.74523999888851, 32.92198346312245, 31.72265993608251, 29.9173176993003, 28.491601937740562, 52.19977189231385, 39.479488162280596, 59.964336869699544, 49.32531805312982, 34.23344619169951, 51.09025374715126, 49.037147591523336, 32.92198346312245, 29.396022468352484, 63.582848099201826, 48.94146538270724, 35.18243227767251, 39.94485987582193, 36.72712654703974, 27.77728915635237, 44.12865633779556, 38.0412795852805, 62.84208548133934, 27.080884906424416, 39.40245503911115, 54.811841150452466, 33.57131140436464, 35.52769382137767, 37.5980852538711, 40.33685733281881, 29.167260806600495, 30.806767041912167, 46.427438201089, 56.11166650505244, 53.542126217761066, 76.39654709045266, 50.79177210350426, 50.004318665090636, 56.773093073800744, 52.609179777419676, 36.15772522700737, 38.94340253332466, 54.598150033144236, 40.33685733281881, 35.18243227767251, 35.806340977539655, 44.73612093895834, 44.387981683814154, 44.387981683814154, 44.73612093895834, 49.42175070681059, 47.52843316962603, 48.56059987746486, 48.56059987746486, 42.190180314697045, 29.800681131145613, 57.21836996148928, 43.5294404008871, 37.01518097066168, 42.52108200006278, 32.92198346312245, 37.45150421083523, 46.427438201089, 35.52769382137767, 39.479488162280596, 36.87087245606033, 36.87087245606033, 34.4346215959791, 34.4346215959791, 27.561124577174567, 26.09434828302453, 75.80202444429054, 84.89424029774196, 34.23344619169951, 31.846818916590372, 44.47476167904283, 34.23344619169951, 42.10785803778799, 46.066137010589394, 31.35307975333966, 39.556671887793456, 28.771203256660744, 32.348333340301785, 38.79157659527073, 58.57526450222688, 40.25815129472595, 36.72712654703974, 37.08754689283638, 36.44131378072682, 34.300373728940066, 37.9670527218579, 39.095822701298815, 50.20003003478527, 51.99626426796157, 61.38635013907805, 62.3530444922685, 57.89284200037607, 56.33128135834348, 41.2128458403547, 42.52108200006278, 51.894808174781225, 40.25815129472595, 50.29817277977263, 44.47476167904283, 46.97471015107385, 41.617286632367644, 36.72712654703974, 34.56939489431364, 39.40245503911115, 68.21443845541918, 42.68750480955783, 42.77096031537661, 36.299242562272255, 32.22221914035047, 38.94340253332466, 61.38635013907805, 46.97471015107385, 54.811841150452466, 38.0412795852805, 49.71218131735948, 53.85677052285177, 53.85677052285177, 40.57389988868238, 34.03344610288473, 30.987805310145493, 35.66674528344124, 31.41437607743883, 32.60204446315635, 40.65322322588273, 38.94340253332466, 42.10785803778799, 64.08153455750669, 48.08868329990042, 54.598150033144236, 41.45503620813183, 42.3553080116137, 47.15856396830611, 52.609179777419676, 36.08717359043372, 34.84052601883783, 57.89284200037607, 44.04255166984753, 29.281418237700255, 35.25121511458854, 47.435694792893365, 37.67159077349981, 34.56939489431364, 38.19016894386643, 48.65553748142455, 45.97625189374871, 31.291903031786116, 43.10641711474324, 39.32557222442699, 48.182698291098816, 65.34542957624015, 40.41571724385996, 35.87634357692165, 40.57389988868238, 45.44061233480379, 46.60915005647103, 37.524723159601, 46.33684810653952, 37.232703447205715, 42.52108200006278, 38.867415430562446, 38.115651564714454, 39.7114923241556, 36.37020880080945, 56.9952966774873, 48.08868329990042, 45.97625189374871, 39.01953819342525, 54.067559962798995, 46.60915005647103, 46.60915005647103, 38.0412795852805, 38.0412795852805, 34.70469568042362, 39.556671887793456, 59.03467599561125, 31.10908815096766, 34.501942437892716, 31.169907463288272, 31.169907463288272, 42.190180314697045, 25.1437504597293, 27.133828946090556, 24.22778221261098, 41.943695056893915, 40.57389988868238, 47.250760544967605, 48.65553748142455, 44.91121316574036, 36.37020880080945, 45.97625189374871, 30.987805310145493, 33.31005738633129, 38.115651564714454, 37.89297069129353, 37.74523999888851, 45.44061233480379, 42.93836112094585, 56.11166650505244, 33.83461446305701, 59.964336869699544, 52.097918711212095, 43.87084603329354, 50.20003003478527, 50.49503426195374, 41.780172086809024, 43.19069160819034, 36.942956250241515, 34.908640417532865, 43.87084603329354, 43.444504934395866, 44.387981683814154, 50.20003003478527, 43.10641711474324, 51.39048944047805, 44.82358155812517, 47.71445419747699, 50.396507397123145, 45.61846186294896, 46.51820540231633, 50.5937537501147, 53.85677052285177, 39.01953819342525, 40.97207040739876, 42.02569638966092, 46.246434773093995, 53.96206231832688, 56.773093073800744, 51.39048944047805, 56.11166650505244, 41.698649825700315, 40.41571724385996, 43.699809813576366, 49.133016862271724, 55.89290784893326, 38.867415430562446, 35.7364749685521, 37.67159077349981, 40.1011996370117, 44.12865633779556, 39.479488162280596, 41.780172086809024, 45.79700747446617, 43.78524440968234, 65.09067188950593, 37.89297069129353, 48.46584751755162, 46.15619785585283, 42.93836112094585, 60.199030648454944, 44.73612093895834, 46.427438201089, 53.43765380953904, 44.5617113320772, 53.85677052285177, 38.71588573814723, 38.41459632502361, 46.15619785585283, 46.15619785585283, 32.66578255521719, 45.97625189374871, 40.49473132867618, 51.99626426796157, 51.99626426796157, 34.501942437892716, 58.80452160545304, 37.67159077349981, 40.33685733281881, 43.699809813576366, 53.02179868836547, 39.40245503911115, 63.08804244970428, 51.290215346585704, 50.5937537501147, 49.51837188956857, 59.03467599561125, 62.84208548133934, 46.246434773093995, 37.5980852538711, 46.33684810653952, 46.33684810653952, 38.71588573814723, 36.72712654703974, 36.72712654703974, 46.066137010589394, 65.09067188950593, 49.32531805312982, 49.32531805312982, 45.88654216244616, 41.617286632367644, 50.004318665090636, 63.08804244970428, 50.004318665090636, 50.5937537501147, 58.80452160545304, 34.23344619169951, 52.097918711212095, 61.867809250367884, 31.41437607743883, 43.87084603329354, 58.34690118774256, 54.598150033144236, 53.43765380953904, 41.2128458403547, 46.60915005647103, 60.434642992688794, 53.333385249853684, 60.67117749756472, 60.67117749756472, 48.46584751755162, 43.444504934395866, 42.93836112094585, 31.291903031786116, 51.290215346585704, 47.62135285300038, 54.27917440928786, 55.457945789238345, 88.62154503481175, 57.66713919909174, 28.996858600549206, 62.597087408320306, 39.94485987582193, 28.380523708580828, 47.807737558209425, 50.89107172411317, 46.7915731119465, 37.08754689283638, 49.42175070681059, 53.43765380953904, 42.93836112094585, 36.942956250241515, 36.228414794400415, 29.858942463791767, 28.491601937740562, 31.048387510438282, 39.78912962559818, 40.892124929554, 44.73612093895834, 41.780172086809024, 47.435694792893365, 90.7231444115574, 53.646802873050504, 38.94340253332466, 54.173263858709824, 56.55175575986736, 45.97625189374871, 41.2128458403547, 30.987805310145493, 45.79700747446617, 52.50652775191775, 52.40407602298735, 47.34313736903344, 45.263456176258764, 54.811841150452466, 49.61518196998445, 49.61518196998445, 45.263456176258764, 29.74253347911242, 41.29341833835342, 27.240027749911206, 44.91121316574036, 32.411575418595035, 34.367432111764096, 54.811841150452466, 29.626578330054702, 33.245062173048, 32.53843073806492, 37.160054292819765, 39.32557222442699, 43.275130861102994, 50.89107172411317, 40.17959882934157, 31.660762082694095, 34.77254452624438, 20.763634702888442, 58.57526450222688, 44.64883097460398, 43.3597351955915, 40.17959882934157, 53.85677052285177, 50.990565478815945, 35.25121511458854, 35.25121511458854, 62.3530444922685, 64.58413226217957, 45.97625189374871, 33.11545195869231, 82.28231911219243, 44.30137101535166, 60.67117749756472, 59.964336869699544, 44.99901609609206, 35.59715165620933, 47.62135285300038, 44.12865633779556, 43.444504934395866, 41.45503620813183, 32.60204446315635, 38.94340253332466, 35.806340977539655, 48.65553748142455, 57.21836996148928, 36.37020880080945, 35.25121511458854, 39.556671887793456, 39.7114923241556, 40.73270164287187, 40.65322322588273, 43.275130861102994, 62.10995300937982, 28.547303976590324, 28.827451925481085, 27.77728915635237, 31.10908815096766, 25.689792910615846, 40.25815129472595, 34.70469568042362, 36.58394105020859, 35.59715165620933, 32.60204446315635, 41.943695056893915, 26.196478674560588, 24.513371642677637, 29.858942463791767, 25.840760898054555, 43.19069160819034, 32.159346537604755, 34.09998263323346, 26.196478674560588, 31.475792237910618, 29.684499285876434, 36.15772522700737, 27.4536739354601, 28.60311491479713, 45.17513726542669, 49.71218131735948, 49.037147591523336, 41.617286632367644, 37.8190332109868, 37.9670527218579, 41.780172086809024, 32.159346537604755, 33.4404292647954, 33.4404292647954, 38.94340253332466, 32.22221914035047, 33.4404292647954, 29.33866439394716, 41.2128458403547, 40.81233544283598, 42.77096031537661, 61.867809250367884, 47.52843316962603, 45.79700747446617, 44.47476167904283, 47.06654728734682, 39.78912962559818, 40.73270164287187, 37.9670527218579, 22.938403163153815, 48.65553748142455, 32.85774546700983, 42.93836112094585, 26.09434828302453, 26.87013978811538, 45.17513726542669, 41.29341833835342, 39.01953819342525, 33.96703939987107, 31.90908051733889, 35.11378365112735, 43.61454191906839, 35.25121511458854, 32.60204446315635, 25.689792910615846, 38.19016894386643, 37.305494633128845, 36.44131378072682, 38.41459632502361, 39.172256347947744, 41.2128458403547, 38.56494680401705, 37.9670527218579, 26.196478674560588, 34.84052601883783, 40.73270164287187, 32.22221914035047, 28.65903496526287, 27.346642204097876, 49.51837188956857, 28.214717811314713, 28.380523708580828, 34.16664924473404, 50.20003003478527, 41.29341833835342, 29.281418237700255, 30.388446900124176, 41.780172086809024, 35.806340977539655, 50.004318665090636, 33.4404292647954, 40.17959882934157, 44.21492934326177, 44.64883097460398, 35.806340977539655, 24.322607071104724, 40.57389988868238, 45.351947753528236, 32.22221914035047, 26.2990087931472, 34.97688798216538, 25.63966650134361, 34.70469568042362, 28.60311491479713, 28.715064341305684, 30.093129052017023, 38.115651564714454, 44.387981683814154, 29.281418237700255, 25.24216031657971, 39.172256347947744, 46.7915731119465, 41.37414835829223, 41.29341833835342, 55.24173574865985, 34.70469568042362, 40.41571724385996, 34.501942437892716, 44.99901609609206, 39.40245503911115, 34.03344610288473, 41.53608219643496, 52.609179777419676, 39.556671887793456, 27.83159468972004, 35.87634357692165, 38.0412795852805, 34.23344619169951, 41.37414835829223, 42.10785803778799, 27.240027749911206, 23.75917448515314, 42.3553080116137, 32.53843073806492, 51.39048944047805, 49.61518196998445, 49.22907356066477, 43.275130861102994, 49.9067492938035, 37.8190332109868, 32.47494113727571, 37.9670527218579, 70.93179276322707, 42.10785803778799, 67.1568716116849, 31.72265993608251, 45.97625189374871, 42.52108200006278, 37.160054292819765, 49.71218131735948, 34.4346215959791, 29.22428378123494, 46.15619785585283, 41.943695056893915, 53.96206231832688, 43.275130861102994, 49.22907356066477, 41.37414835829223, 37.9670527218579, 46.70027251047952, 30.62678644311483, 52.40407602298735, 52.301824199806006, 52.19977189231385, 25.1437504597293, 41.617286632367644, 35.7364749685521, 48.65553748142455, 43.5294404008871, 35.45837151398557, 47.99485175252902, 42.68750480955783, 45.529450258314064, 35.25121511458854, 38.26483200699776, 48.65553748142455, 42.27266353442275, 25.790339917193062, 29.22428378123494, 58.119428177447986, 41.698649825700315, 41.698649825700315, 41.617286632367644, 35.38918446958933, 24.753932740556596, 27.72308958496738, 27.77728915635237, 40.33685733281881, 43.87084603329354, 28.883810562338418, 34.367432111764096, 32.60204446315635, 57.66713919909174, 27.293282919182133, 36.58394105020859, 51.290215346585704, 26.87013978811538, 27.886006392229614, 33.50580642730562, 29.167260806600495, 32.47494113727571, 25.790339917193062, 44.04255166984753, 30.56702687878971, 51.1901369093998, 40.97207040739876, 41.45503620813183, 40.81233544283598, 37.8190332109868, 34.63697922255266, 34.63697922255266, 77.59961765106212, 48.94146538270724, 39.32557222442699, 36.798929313140206, 42.438114061533675, 45.61846186294896, 36.08717359043372, 51.39048944047805, 47.90120329104557, 35.38918446958933, 29.396022468352484, 29.56877039069624, 34.56939489431364, 44.82358155812517, 46.15619785585283, 30.388446900124176, 32.411575418595035, 40.33685733281881, 40.49473132867618, 40.41571724385996, 50.004318665090636, 35.11378365112735, 49.51837188956857, 33.4404292647954, 34.97688798216538, 42.52108200006278, 37.9670527218579, 32.60204446315635, 46.51820540231633, 57.89284200037607, 41.132430556936306, 50.20003003478527, 39.32557222442699, 35.32013242426123, 46.246434773093995, 37.305494633128845, 35.11378365112735, 39.172256347947744, 34.09998263323346, 34.23344619169951, 27.77728915635237, 55.24173574865985, 46.246434773093995, 25.992616060150944, 39.095822701298815, 27.080884906424416, 36.228414794400415, 34.16664924473404, 30.987805310145493, 38.0412795852805, 40.33685733281881, 36.016759615546256, 29.22428378123494, 32.47494113727571, 42.27266353442275, 39.248839424943355, 41.2128458403547, 44.30137101535166, 34.300373728940066, 30.927341318986464, 36.51255777326842, 34.03344610288473, 34.03344610288473, 26.713157919740368, 42.438114061533675, 37.08754689283638, 32.47494113727571, 30.26997357328364, 44.21492934326177, 49.037147591523336, 39.095822701298815, 39.095822701298815, 39.095822701298815, 39.095822701298815, 39.095822701298815, 39.095822701298815, 39.095822701298815, 41.698649825700315, 43.19069160819034, 34.70469568042362, 34.70469568042362, 34.70469568042362, 44.12865633779556, 38.0412795852805, 37.45150421083523, 31.169907463288272, 42.60421214369869, 42.60421214369869, 37.08754689283638, 34.367432111764096, 35.38918446958933, 27.995149135337257, 31.846818916590372, 23.073202403172917, 33.57131140436464, 18.868615759264884, 27.240027749911206, 34.23344619169951, 25.34095533871963, 34.70469568042362, 34.97688798216538, 41.29341833835342, 36.08717359043372, 36.016759615546256, 31.72265993608251, 42.190180314697045, 36.72712654703974, 37.67159077349981, 33.11545195869231, 34.300373728940066, 35.806340977539655, 37.160054292819765, 39.172256347947744, 29.9173176993003, 30.034410770075546, 31.41437607743883, 47.99485175252902, 34.16664924473404, 30.68666283939559, 33.90076227087091, 33.57131140436464, 29.975807060355134, 41.780172086809024, 30.21091022897123, 31.169907463288272, 32.411575418595035, 33.245062173048, 31.846818916590372, 26.713157919740368, 42.52108200006278, 26.401940203271806, 41.861853726676614, 32.348333340301785, 47.807737558209425, 31.90908051733889, 40.49473132867618, 37.8190332109868, 37.8190332109868, 37.232703447205715, 44.91121316574036, 49.133016862271724, 59.265731184583274, 20.085536923187668, 47.15856396830611, 57.89284200037607, 47.34313736903344, 31.35307975333966, 40.65322322588273, 32.28521466114651, 39.86691871057298, 49.133016862271724, 49.133016862271724, 29.56877039069624, 27.72308958496738, 29.453492679719986, 34.300373728940066, 37.45150421083523, 38.64034257045414, 30.62678644311483, 47.62135285300038, 58.80452160545304, 41.861853726676614, 38.94340253332466, 33.4404292647954, 34.23344619169951, 52.097918711212095, 52.9183415530491, 36.72712654703974, 43.78524440968234, 40.97207040739876, 33.702705802144536, 42.27266353442275, 36.228414794400415, 40.17959882934157, 45.263456176258764, 35.59715165620933, 36.016759615546256, 44.5617113320772, 40.65322322588273, 42.68750480955783, 46.70027251047952, 50.49503426195374, 43.10641711474324, 33.375179667439795, 53.22932014095188, 44.47476167904283, 31.53732846903916, 42.854578979512624, 48.75066069158903, 48.75066069158903, 36.228414794400415, 47.435694792893365, 41.53608219643496, 48.75066069158903, 43.3597351955915, 42.3553080116137, 42.52108200006278, 45.263456176258764, 38.0412795852805, 44.387981683814154, 29.9173176993003, 24.850816893992036, 44.73612093895834, 57.21836996148928, 47.99485175252902, 46.33684810653952, 55.457945789238345, 55.457945789238345, 57.89284200037607, 45.17513726542669, 46.246434773093995, 35.94648303373688, 34.23344619169951, 43.61454191906839, 40.97207040739876, 33.4404292647954, 32.60204446315635, 34.70469568042362, 29.684499285876434, 48.56059987746486, 51.99626426796157, 35.32013242426123, 56.55175575986736, 45.44061233480379, 33.702705802144536, 43.275130861102994, 37.232703447205715, 52.9183415530491, 47.90120329104557, 64.08153455750669, 72.04880596519355, 46.33684810653952, 61.14702744030778, 66.63425269055388, 31.53732846903916, 48.46584751755162, 42.02569638966092, 48.46584751755162, 55.6750020519895, 53.542126217761066, 54.173263858709824, 64.58413226217957, 57.89284200037607, 47.71445419747699, 49.037147591523336, 52.097918711212095, 37.524723159601, 39.7114923241556, 39.78912962559818, 28.547303976590324, 30.507383918455513, 29.684499285876434, 39.634006510082365, 65.09067188950593, 35.11378365112735, 32.92198346312245, 43.87084603329354, 60.434642992688794, 41.698649825700315, 39.78912962559818, 45.97625189374871, 40.57389988868238, 35.38918446958933, 56.773093073800744, 42.60421214369869, 40.41571724385996, 19.505629671380603, 37.45150421083523, 18.28808948244362, 29.9173176993003, 26.14536361045317, 30.034410770075546, 12.206311145618837, 17.079700020438295, 29.800681131145613, 29.800681131145613, 85.89494673838159, 109.86130976812342, 30.388446900124176, 29.56877039069624, 37.160054292819765, 33.1801937796528, 48.182698291098816, 46.427438201089, 46.427438201089, 29.684499285876434, 31.230845679407448, 40.41571724385996, 24.80232751057381, 20.967397891114594, 49.037147591523336, 54.27917440928786, 24.850816893992036, 22.582776184718522, 51.591626129191596, 45.79700747446617, 38.19016894386643, 49.71218131735948, 209.29610089202527, 34.16664924473404, 39.86691871057298, 34.77254452624438, 62.3530444922685, 57.66713919909174, 51.49095957359226, 61.14702744030778, 25.94189877662824, 23.80562446936611, 136.19157033683516, 174.19167092744075, 63.83170433047907, 46.15619785585283, 33.05083646319603, 47.34313736903344, 33.1801937796528, 28.380523708580828, 49.51837188956857, 43.10641711474324, 43.10641711474324, 32.66578255521719, 38.0412795852805, 30.927341318986464, 31.475792237910618, 38.94340253332466, 37.232703447205715, 54.49161709130268, 50.1020787877761, 48.65553748142455, 71.76791429061606, 70.65525591042105, 41.45503620813183, 49.71218131735948, 33.63694444585419, 54.811841150452466, 55.457945789238345, 71.48811770889668, 41.45503620813183, 34.97688798216538, 62.3530444922685, 90.01713130052181, 79.43983955226133, 110.7229626998625, 62.10995300937982, 51.09025374715126, 79.75075829332496, 60.67117749756472, 59.265731184583274, 80.69083499570341, 101.20898120494374, 119.72006788311512, 117.40447215712238, 61.626609520407456, 51.692489491288626, 113.34868209247811, 98.86447768050158, 142.1710983175922, 59.265731184583274, 106.89798837141365, 116.49082330627213, 134.60489048229144, 78.2082389954826, 35.66674528344124, 35.66674528344124, 70.65525591042105, 95.82273747708687, 101.20898120494374, 74.04607243930377, 74.04607243930377, 74.04607243930377, 74.04607243930377, 74.04607243930377, 72.33079701870139, 61.14702744030778, 73.18340956555208, 73.18340956555208, 67.9484955630781, 105.24068867470233, 68.7494509271083, 55.02636863114642, 97.71267162761835, 68.21443845541918, 64.83690740886694, 39.32557222442699, 104.01459751366006, 40.25815129472595, 49.037147591523336, 64.83690740886694, 61.14702744030778, 75.50650035756513, 75.50650035756513, 75.50650035756513, 82.60436300370014, 128.94358717585138, 77.29708541258049, 70.65525591042105, 83.90519244027122, 71.20941195067294, 73.46984133199462, 76.39654709045266, 81.9615307508321, 51.1901369093998, 77.90333396724647, 127.94014021950323, 104.83039411284926, 73.18340956555208, 49.71218131735948, 53.02179868836547, 73.46984133199462, 97.71267162761835, 126.94450214932196, 83.90519244027122, 88.27604237373367, 88.27604237373367, 81.9615307508321, 45.79700747446617, 38.48969815122384, 27.83159468972004, 50.396507397123145, 110.29129477712178, 47.06654728734682, 71.76791429061606, 202.06587336346374, 108.15806803146498, 33.05083646319603, 33.05083646319603, 45.086990684122, 83.90519244027122, 105.65258908406659, 60.67117749756472, 74.91890411002962, 65.8579401324037, 40.41571724385996, 87.93188669987823, 31.475792237910618, 49.51837188956857, 83.57807709540792, 87.24759532868651, 39.479488162280596, 56.9952966774873, 68.7494509271083, 37.160054292819765, 163.63793110254423, 65.09067188950593, 74.91890411002962, 115.58428452718766, 45.7076474882609, 40.892124929554, 32.9863470466753, 35.7364749685521, 64.83690740886694, 53.85677052285177, 43.87084603329354, 60.67117749756472, 129.44825810694724, 109.43300111180854, 49.71218131735948, 68.7494509271083, 97.71267162761835, 57.21836996148928, 55.24173574865985, 64.33234259239626, 77.29708541258049, 34.09998263323346, 36.15772522700737, 66.89505178172818, 102.40200064874995, 50.5937537501147, 26.505274475545082, 120.1886389803891, 77.29708541258049, 65.8579401324037, 70.93179276322707, 130.4635333784854, 56.773093073800744, 51.894808174781225, 108.15806803146498, 49.9067492938035, 64.58413226217957, 106.06610162605153, 86.90744918987926, 33.4404292647954, 41.943695056893915, 55.24173574865985, 99.63988138921447, 90.36944839098756, 55.6750020519895, 26.401940203271806, 139.42126035437516, 76.69555472159655, 66.37447035867858, 60.434642992688794, 64.33234259239626, 37.160054292819765, 48.27689708476307, 31.90908051733889, 103.20514949397284, 34.56939489431364, 58.57526450222688, 46.7915731119465, 62.597087408320306, 161.10096012939127, 64.33234259239626, 39.78912962559818, 34.70469568042362, 72.89809449073893, 61.14702744030778, 37.89297069129353, 38.867415430562446, 70.93179276322707, 83.25223705241488, 73.46984133199462, 166.21485355527525, 40.25815129472595, 43.699809813576366, 36.228414794400415, 40.81233544283598, 37.45150421083523, 26.87013978811538, 69.55984769951485, 73.75739416067401, 56.33128135834348, 68.21443845541918, 44.64883097460398, 39.40245503911115, 56.9952966774873, 39.32557222442699, 88.27604237373367, 58.80452160545304, 59.73055807527518, 81.9615307508321, 79.43983955226133, 73.18340956555208, 76.99573263552006, 90.01713130052181, 106.89798837141365, 64.33234259239626, 130.9741532108186, 32.72964525738904, 29.975807060355134, 29.975807060355134, 54.49161709130268, 40.57389988868238, 61.626609520407456, 50.89107172411317, 150.1626064581185, 114.23768682435218, 92.1518260518184, 100.02986085865793, 57.89284200037607, 104.4216991379013, 204.44776193681219, 123.52039078374908, 106.89798837141365, 66.63425269055388, 104.83039411284926, 59.49769069799599, 41.861853726676614, 25.840760898054555, 44.04255166984753, 45.351947753528236, 66.37447035867858, 66.37447035867858, 81.00664999449546, 99.2514223056366, 104.01459751366006, 45.351947753528236, 50.29817277977263, 45.351947753528236, 45.351947753528236, 48.94146538270724, 30.329152388418176, 90.01713130052181, 85.56007407666469, 104.01459751366006, 97.33172602285453, 112.90677745995042, 67.1568716116849, 81.32370105824417, 84.2335880783952, 61.38635013907805, 113.7923162908356, 87.93188669987823, 47.52843316962603, 96.19777706766972, 122.08133464248264, 48.08868329990042, 32.411575418595035, 45.7076474882609, 87.24759532868651, 159.2240770935855, 73.18340956555208, 113.34868209247811, 38.64034257045414, 29.053548433146837, 27.346642204097876, 40.41571724385996, 45.79700747446617, 65.34542957624015, 51.09025374715126, 97.71267162761835, 40.1011996370117, 34.367432111764096, 103.60908302823987, 66.89505178172818, 215.09800253180504, 40.97207040739876, 35.18243227767251, 100.02986085865793, 77.59961765106212, 61.626609520407456, 61.867809250367884, 54.067559962798995, 37.01518097066168, 53.646802873050504, 44.91121316574036, 70.10541234668786, 45.61846186294896, 49.42175070681059, 63.3349620664254, 69.28865955361954, 72.61389175398708, 76.69555472159655, 48.65553748142455, 48.65553748142455, 112.90677745995042, 145.54258875480053, 97.33172602285453, 93.6030061542134, 79.13013296849392, 51.692489491288626, 31.72265993608251, 64.83690740886694, 146.11222634120347, 95.0770390179677, 40.97207040739876, 44.73612093895834, 28.380523708580828, 43.87084603329354, 56.9952966774873, 51.591626129191596, 43.78524440968234, 43.78524440968234, 50.1020787877761, 25.094689498803355, 34.63697922255266, 67.68358948381629, 73.46984133199462, 120.65904401308495, 40.73270164287187, 123.03883041717654, 56.773093073800744, 70.37979717263202, 51.39048944047805, 59.73055807527518, 47.99485175252902, 69.55984769951485, 112.02812994692015, 89.66618776422116, 74.91890411002962, 78.51433738825796, 63.3349620664254, 84.563269020706, 215.09800253180504, 96.95226558425567, 28.436008585761353, 55.457945789238345, 55.457945789238345, 33.4404292647954, 60.199030648454944, 83.57807709540792, 65.60118435636834, 36.65546388385301, 100.02986085865793, 80.37625124290746, 45.17513726542669, 33.31005738633129, 61.38635013907805, 43.87084603329354, 84.2335880783952, 51.09025374715126, 37.74523999888851, 72.61389175398708, 164.27839185643572, 81.641993024771, 81.641993024771, 81.641993024771, 130.4635333784854, 83.57807709540792, 79.13013296849392, 79.13013296849392, 48.94146538270724, 60.199030648454944, 74.04607243930377, 119.2533235714264, 119.2533235714264, 174.8734378511126, 60.67117749756472, 69.83209724580306, 55.6750020519895, 96.95226558425567, 89.66618776422116, 99.2514223056366, 78.82163381626928, 116.94675549762589, 116.94675549762589, 36.798929313140206, 69.55984769951485, 65.09067188950593, 76.69555472159655, 76.69555472159655, 76.69555472159655, 86.56862915519338, 51.591626129191596, 102.40200064874995, 72.04880596519355, 143.8469662358426, 71.76791429061606, 276.1913023296018, 99.2514223056366, 78.2082389954826, 176.93479059943397, 176.93479059943397, 63.08804244970428, 101.6051019574268, 59.265731184583274, 61.14702744030778, 89.66618776422116, 100.02986085865793, 134.60489048229144, 104.4216991379013, 42.77096031537661, 42.77096031537661, 45.79700747446617, 87.93188669987823, 128.94358717585138, 59.03467599561125, 110.29129477712178, 81.641993024771, 53.333385249853684, 65.09067188950593, 35.32013242426123, 64.08153455750669, 36.58394105020859, 48.182698291098816, 79.75075829332496, 30.26997357328364, 164.9213593028435, 45.263456176258764, 60.908637772317434, 44.73612093895834, 63.3349620664254, 81.9615307508321, 38.71588573814723, 97.71267162761835, 84.2335880783952, 58.119428177447986, 68.7494509271083, 102.80279074731456, 59.03467599561125, 59.49769069799599, 42.190180314697045, 95.0770390179677, 70.37979717263202, 48.56059987746486, 57.66713919909174, 62.84208548133934, 69.83209724580306, 112.02812994692015, 74.04607243930377, 51.591626129191596, 63.08804244970428, 62.10995300937982, 83.90519244027122, 91.43469485205186, 48.56059987746486, 61.867809250367884, 53.02179868836547, 65.60118435636834, 108.15806803146498, 115.13366410675533, 47.15856396830611, 74.62682298320091, 69.55984769951485, 41.2128458403547, 147.25819873544805, 101.20898120494374, 44.82358155812517, 51.09025374715126, 75.80202444429054, 35.94648303373688, 45.61846186294896, 59.49769069799599, 75.50650035756513, 35.32013242426123, 66.37447035867858, 52.301824199806006, 85.2265069597304, 26.453556883236853, 53.333385249853684, 57.21836996148928, 72.61389175398708, 98.09510821132314, 26.817710263716034, 75.50650035756513, 28.049880592281973, 24.370158584949497, 36.87087245606033, 65.60118435636834, 95.82273747708687, 101.20898120494374, 55.457945789238345, 77.90333396724647, 96.95226558425567, 73.18340956555208, 121.13129015902297, 51.894808174781225, 54.173263858709824, 93.96935796203766, 98.09510821132314, 50.20003003478527, 88.27604237373367, 32.79363281328884, 21.506589466713315, 36.15772522700737, 73.46984133199462, 74.62682298320091, 78.51433738825796, 31.10908815096766, 44.82358155812517, 32.411575418595035, 60.908637772317434, 48.84596987082467, 36.15772522700737, 86.56862915519338, 53.75168417471723, 46.60915005647103, 37.74523999888851, 52.81508628524973, 47.807737558209425, 40.022953418667065, 67.9484955630781, 56.9952966774873, 100.42136666458902, 74.91890411002962, 90.36944839098756, 110.29129477712178, 55.89290784893326, 123.03883041717654, 40.1011996370117, 45.17513726542669, 41.617286632367644, 74.04607243930377, 101.20898120494374, 42.3553080116137, 72.89809449073893, 70.10541234668786, 22.582776184718522, 42.10785803778799, 45.97625189374871, 68.7494509271083, 52.19977189231385, 52.9183415530491, 44.387981683814154, 59.49769069799599, 31.41437607743883, 76.39654709045266, 41.2128458403547, 52.097918711212095, 58.34690118774256, 61.626609520407456, 50.6926662381911, 62.597087408320306, 53.85677052285177, 47.52843316962603, 47.62135285300038, 66.37447035867858, 52.71203249107963, 113.7923162908356, 62.10995300937982, 56.773093073800744, 59.03467599561125, 50.89107172411317, 49.32531805312982, 38.0412795852805, 29.396022468352484, 73.75739416067401, 130.9741532108186, 130.9741532108186, 59.265731184583274, 51.290215346585704, 52.609179777419676, 96.19777706766972, 90.01713130052181, 101.20898120494374, 46.246434773093995, 76.09870517958821, 124.97641006343288, 89.31661242710538, 97.33172602285453, 97.33172602285453, 238.09178486008656, 132.00139622598326, 66.11570082213345, 63.83170433047907, 116.94675549762589, 69.28865955361954, 46.88305220915621, 34.501942437892716, 47.435694792893365, 112.46659565031442, 95.44916002730179, 79.75075829332496, 35.66674528344124, 28.827451925481085, 24.22778221261098, 31.9714638416083, 60.908637772317434, 47.807737558209425, 46.246434773093995, 16.39333746248769, 154.32526724792663, 32.9863470466753, 33.11545195869231, 16.94678474038272, 25.840760898054555, 50.20003003478527, 26.043432497666576, 33.90076227087091, 33.96703939987107, 20.4816886421048, 19.053782246709744, 31.784678801853513, 18.2524054167373, 22.319678873017494, 22.40703562619394, 26.713157919740368, 21.33922385967176, 23.574279122424027, 30.151962130172098, 19.091033031037714, 22.319678873017494, 31.048387510438282, 15.703855414343392, 18.721779026482544, 36.72712654703974, 25.24216031657971, 17.247311047115943, 19.774139536218833, 16.04489330813532, 16.88071548801052, 14.89717913158109, 13.940044882238906, 21.887977204970085, 22.538712245181248, 21.887977204970085, 19.053782246709744, 23.25416777248505, 27.61500750152876, 20.322298876557472, 22.80439145296227, 23.992334296139543, 23.52828054362271, 13.379852088930456, 11.69292987543966, 12.968197316969134, 13.537569551425428, 47.99485175252902, 20.4816886421048, 32.47494113727571, 49.133016862271724, 20.602050815688813, 36.51255777326842, 20.046345644076645, 31.53732846903916, 26.817710263716034, 19.697047472120772, 41.861853726676614, 31.53732846903916, 52.9183415530491, 58.119428177447986, 34.97688798216538, 20.36202965350754, 67.1568716116849, 40.97207040739876, 24.133327041205547, 29.110349096271467, 31.53732846903916, 23.75917448515314, 45.88654216244616, 19.24076585887165, 23.4365524707797, 27.028044172168716, 54.067559962798995, 43.10641711474324, 47.52843316962603, 28.491601937740562, 59.964336869699544, 72.33079701870139, 41.45503620813183, 32.79363281328884, 24.133327041205547, 41.45503620813183, 61.38635013907805, 25.045724266551254, 36.228414794400415, 34.367432111764096, 19.46756991825522, 24.80232751057381, 52.81508628524973, 74.33588057277046, 51.894808174781225, 24.80232751057381, 60.199030648454944, 44.5617113320772, 56.11166650505244, 64.08153455750669, 38.79157659527073, 32.79363281328884, 22.893645317537526, 23.94552000018515, 48.94146538270724, 31.660762082694095, 36.798929313140206, 28.60311491479713, 59.265731184583274, 31.598985005566618, 29.74253347911242, 24.322607071104724, 18.503663694167408, 54.173263858709824, 37.5980852538711, 48.94146538270724, 48.182698291098816, 146.11222634120347, 28.715064341305684, 43.444504934395866, 43.699809813576366, 40.25815129472595, 26.14536361045317, 72.04880596519355, 44.5617113320772, 37.8190332109868, 49.133016862271724, 20.64232851707703, 78.2082389954826, 48.94146538270724, 27.561124577174567, 46.88305220915621, 51.99626426796157, 37.524723159601, 55.457945789238345, 60.434642992688794, 48.56059987746486, 27.668995768810042, 34.367432111764096, 80.69083499570341, 59.964336869699544, 27.186876493132687, 33.76859572409541, 56.11166650505244, 40.57389988868238, 24.086237637543274, 36.228414794400415, 51.99626426796157, 15.16135869683449, 49.71218131735948, 49.71218131735948, 55.457945789238345, 23.028181534732802, 21.675267738479917, 76.39654709045266, 29.396022468352484, 28.436008585761353, 28.996858600549206, 39.172256347947744, 32.9863470466753, 107.31637526839133, 40.65322322588273, 23.345182011801924, 20.203571081597193, 30.388446900124176, 31.53732846903916, 30.26997357328364, 76.69555472159655, 55.02636863114642, 27.83159468972004, 39.248839424943355, 63.08804244970428, 70.65525591042105, 74.62682298320091, 41.780172086809024, 48.75066069158903, 58.80452160545304, 12.742232560259533, 20.046345644076645, 11.738694709013188, 16.04489330813532, 17.484733434196265, 13.069908149118522, 11.005966116567095, 33.702705802144536, 22.80439145296227, 17.213657767955564, 36.58394105020859, 28.883810562338418, 46.51820540231633, 68.48142221881122, 29.56877039069624, 47.52843316962603, 29.22428378123494, 20.24306974171993, 20.401838105461145, 32.60204446315635, 29.858942463791767, 27.668995768810042, 29.56877039069624, 23.299630451611073, 28.049880592281973, 41.53608219643496, 48.37128004023317, 29.626578330054702, 41.698649825700315, 28.940279382223952, 46.15619785585283, 37.305494633128845, 50.79177210350426, 36.15772522700737, 26.09434828302453, 23.75917448515314, 27.186876493132687, 47.62135285300038, 52.81508628524973, 39.095822701298815, 40.73270164287187, 26.975306541751994, 59.49769069799599, 53.646802873050504, 32.53843073806492, 63.08804244970428, 98.09510821132314, 123.03883041717654, 74.91890411002962, 56.773093073800744, 53.22932014095188, 23.75917448515314, 29.975807060355134, 54.49161709130268, 60.434642992688794, 60.199030648454944, 39.095822701298815, 69.01852867010916, 45.351947753528236, 58.119428177447986, 19.24076585887165, 40.17959882934157, 39.86691871057298, 36.65546388385301, 25.740017318702005, 72.04880596519355, 32.85774546700983, 44.21492934326177, 45.086990684122, 69.55984769951485, 106.89798837141365, 67.68358948381629, 50.49503426195374, 61.38635013907805, 47.34313736903344, 42.190180314697045, 31.72265993608251, 48.46584751755162, 51.692489491288626, 62.597087408320306, 27.133828946090556, 57.21836996148928, 20.124804822565828, 30.388446900124176, 67.41971617548266, 28.325147094540903, 48.27689708476307, 55.89290784893326, 40.022953418667065, 51.09025374715126, 84.89424029774196, 57.66713919909174, 39.78912962559818, 26.76538304079522, 81.641993024771, 58.119428177447986, 27.4536739354601, 29.053548433146837, 90.7231444115574, 46.33684810653952, 33.96703939987107, 57.89284200037607, 24.753932740556596, 29.975807060355134, 17.416566918622852, 63.582848099201826, 27.507346790200362, 37.45150421083523, 50.6926662381911, 35.7364749685521, 44.21492934326177, 26.505274475545082, 62.84208548133934, 55.24173574865985, 65.09067188950593, 82.28231911219243, 87.24759532868651, 101.20898120494374, 50.990565478815945, 50.990565478815945, 52.50652775191775, 52.50652775191775, 38.64034257045414, 28.883810562338418, 23.20879380099744, 93.6030061542134, 59.964336869699544, 26.817710263716034, 69.01852867010916, 39.634006510082365, 41.943695056893915, 44.12865633779556, 40.65322322588273, 36.51255777326842, 55.24173574865985, 66.63425269055388, 59.265731184583274, 40.97207040739876, 27.346642204097876, 36.44131378072682, 52.301824199806006, 51.894808174781225, 65.8579401324037, 53.22932014095188, 36.228414794400415, 41.05217218133816, 85.56007407666469, 48.182698291098816, 29.9173176993003, 69.28865955361954, 40.1011996370117, 41.943695056893915, 21.214554942517648, 22.494734284035275, 66.89505178172818, 25.63966650134361, 52.609179777419676, 43.95661501095434, 28.60311491479713, 58.34690118774256, 37.89297069129353, 24.417803063601138, 58.57526450222688, 62.3530444922685, 65.34542957624015, 87.24759532868651, 65.09067188950593, 43.275130861102994, 54.067559962798995, 44.82358155812517, 45.88654216244616, 52.609179777419676, 26.713157919740368, 71.76791429061606, 39.556671887793456, 22.36331459495464, 65.8579401324037, 26.87013978811538, 43.02230705927991, 50.004318665090636, 28.15966472087403, 72.04880596519355, 18.648789727196185, 22.75989509352673, 50.29817277977263, 48.27689708476307, 68.7494509271083, 38.48969815122384, 25.192907336481866, 38.79157659527073, 42.27266353442275, 31.90908051733889, 27.028044172168716, 42.77096031537661, 63.08804244970428, 29.396022468352484, 30.329152388418176, 31.10908815096766, 48.94146538270724, 38.79157659527073, 27.346642204097876, 37.232703447205715, 75.80202444429054, 82.28231911219243, 56.11166650505244, 49.133016862271724, 56.9952966774873, 39.172256347947744, 62.597087408320306, 48.75066069158903, 60.67117749756472, 49.32531805312982, 57.89284200037607, 32.348333340301785, 26.453556883236853, 32.033969127372174, 73.18340956555208, 41.780172086809024, 54.811841150452466, 32.9863470466753, 87.24759532868651, 41.780172086809024, 21.008389858307645, 27.83159468972004, 23.75917448515314, 33.31005738633129, 22.494734284035275, 19.851533330124635, 67.41971617548266, 72.61389175398708, 42.190180314697045, 14.89717913158109, 80.69083499570341, 33.05083646319603, 46.246434773093995, 43.10641711474324, 21.760102337787384, 21.717643615002626, 22.016603538363892, 30.806767041912167, 20.723120306578686, 20.763634702888442, 47.71445419747699, 42.3553080116137, 42.3553080116137, 29.511075247281195, 24.948080241282657, 38.33964103892552, 72.61389175398708, 38.71588573814723, 46.066137010589394, 103.60908302823987, 81.00664999449546, 38.71588573814723, 118.3252868447224, 54.811841150452466, 59.265731184583274, 113.7923162908356, 50.1020787877761, 75.21212841006654, 70.37979717263202, 20.322298876557472, 31.9714638416083, 52.609179777419676, 73.18340956555208, 62.3530444922685, 66.89505178172818, 38.41459632502361, 38.56494680401705, 28.491601937740562, 17.113091407702502, 25.39049775763493, 37.9670527218579, 20.8042283062039, 32.09659661306934, 23.852165264858517, 36.942956250241515, 19.543763832601815, 33.4404292647954, 46.246434773093995, 35.38918446958933, 32.60204446315635, 42.190180314697045, 45.529450258314064, 76.69555472159655, 23.75917448515314, 31.53732846903916, 21.380942759123343, 18.503663694167408, 30.74665629604239, 27.186876493132687, 27.72308958496738, 59.03467599561125, 66.37447035867858, 37.160054292819765, 56.9952966774873, 53.12545808585662, 72.61389175398708, 44.04255166984753, 78.82163381626928, 91.79256012938836, 84.2335880783952, 69.83209724580306, 51.99626426796157, 72.89809449073893, 27.293282919182133, 32.348333340301785, 65.34542957624015, 91.79256012938836, 75.80202444429054, 103.20514949397284, 51.39048944047805, 60.908637772317434, 69.01852867010916, 51.49095957359226, 55.457945789238345, 87.93188669987823, 17.76007792103387, 44.12865633779556, 92.87458178129451, 56.11166650505244, 30.447857334592324, 40.1011996370117, 47.06654728734682, 39.78912962559818, 45.529450258314064, 104.01459751366006, 73.46984133199462, 30.447857334592324, 31.72265993608251, 39.86691871057298, 45.086990684122, 54.598150033144236, 33.96703939987107, 56.55175575986736, 82.60436300370014, 61.14702744030778, 48.182698291098816, 70.10541234668786, 60.434642992688794, 83.25223705241488, 87.24759532868651, 28.1047190510635, 59.73055807527518, 41.37414835829223, 53.02179868836547, 81.641993024771, 31.10908815096766, 58.57526450222688, 55.457945789238345, 80.69083499570341, 24.465540688808968, 37.9670527218579, 49.61518196998445, 63.83170433047907, 77.29708541258049, 116.03666862607915, 55.89290784893326, 35.11378365112735, 37.74523999888851, 53.43765380953904, 33.90076227087091, 24.561296107667808, 27.668995768810042, 38.115651564714454, 25.44013703364339, 19.812798643245316, 18.758380733362646, 34.97688798216538, 21.008389858307645, 19.429584428039178, 37.45150421083523, 25.34095533871963, 68.7494509271083, 23.852165264858517, 46.97471015107385, 82.28231911219243, 56.9952966774873, 70.65525591042105, 110.29129477712178, 58.34690118774256, 63.08804244970428, 58.57526450222688, 51.09025374715126, 44.30137101535166, 31.846818916590372, 76.09870517958821, 69.83209724580306, 85.89494673838159, 100.42136666458902, 64.58413226217957, 47.250760544967605, 52.301824199806006, 82.60436300370014, 34.4346215959791, 64.33234259239626, 45.44061233480379, 37.67159077349981, 82.28231911219243, 25.045724266551254, 37.89297069129353, 40.57389988868238, 23.299630451611073, 45.79700747446617, 67.68358948381629, 35.25121511458854, 56.55175575986736, 50.004318665090636, 30.388446900124176, 22.84897480427519, 34.300373728940066, 29.453492679719986, 26.66103470132838, 53.542126217761066, 48.84596987082467, 41.132430556936306, 33.76859572409541, 40.81233544283598, 22.232662691545976, 36.798929313140206, 44.04255166984753, 30.927341318986464, 36.44131378072682, 46.15619785585283, 38.33964103892552, 33.90076227087091, 25.2915095879081, 47.71445419747699, 49.32531805312982, 37.89297069129353, 29.453492679719986, 49.32531805312982, 34.03344610288473, 32.9863470466753, 52.40407602298735, 75.21212841006654, 81.00664999449546, 38.26483200699776, 43.19069160819034, 45.17513726542669, 47.15856396830611, 48.182698291098816, 31.475792237910618, 40.73270164287187, 59.49769069799599, 34.56939489431364, 16.8149038152427, 50.5937537501147, 30.927341318986464, 35.04526897307925, 40.41571724385996, 44.30137101535166, 28.2698785323965, 31.291903031786116, 39.556671887793456, 24.609314266596865, 20.68268496272443, 28.771203256660744, 47.807737558209425, 60.199030648454944, 49.71218131735948, 50.396507397123145, 50.89107172411317, 53.12545808585662, 32.72964525738904, 43.275130861102994, 50.6926662381911, 30.806767041912167, 39.248839424943355, 93.96935796203766, 25.790339917193062, 57.66713919909174, 44.99901609609206, 50.79177210350426, 22.36331459495464, 21.973644325590612, 21.590763877931302, 35.04526897307925, 25.24216031657971, 37.9670527218579, 13.484791569471833, 91.07822475921115, 42.77096031537661, 33.1801937796528, 30.093129052017023, 22.98324851186175, 47.52843316962603, 73.75739416067401, 78.82163381626928, 81.00664999449546, 50.89107172411317, 27.61500750152876, 23.118311288923124, 25.539706914745466, 23.620367629992042, 26.817710263716034, 43.3597351955915, 23.4365524707797, 39.634006510082365, 16.91371785382974, 19.812798643245316, 31.291903031786116, 32.033969127372174, 24.9968545761854, 29.33866439394716, 34.09998263323346, 35.87634357692165, 41.943695056893915, 20.322298876557472, 45.351947753528236, 43.87084603329354, 10.522497930618114, 42.190180314697045, 24.465540688808968, 28.214717811314713, 24.22778221261098, 18.467558995986426, 30.74665629604239, 27.561124577174567, 34.23344619169951, 11.761644251483835, 27.4536739354601, 58.34690118774256, 25.1437504597293, 45.61846186294896, 24.948080241282657, 23.16350836406023, 39.32557222442699, 35.32013242426123, 64.33234259239626, 50.1020787877761, 105.65258908406659, 25.689792910615846, 75.21212841006654, 31.41437607743883, 35.32013242426123, 20.84490127137742, 54.173263858709824, 33.50580642730562, 70.37979717263202, 21.845268970328604, 23.25416777248505, 16.2975636122563, 29.22428378123494, 23.852165264858517, 22.80439145296227, 36.51255777326842, 24.753932740556596, 16.2975636122563, 23.898797049169467, 25.2915095879081, 17.180070153709277, 22.626926270737744, 28.1047190510635, 26.713157919740368, 11.761644251483835, 20.24306974171993, 31.291903031786116, 27.080884906424416, 27.72308958496738, 26.2990087931472, 24.465540688808968, 28.325147094540903, 29.56877039069624, 21.590763877931302, 34.4346215959791, 23.16350836406023, 32.53843073806492, 21.590763877931302, 32.28521466114651, 20.723120306578686, 60.434642992688794, 24.322607071104724, 27.886006392229614, 31.41437607743883, 29.33866439394716, 24.561296107667808, 32.72964525738904, 34.03344610288473, 34.03344610288473, 27.668995768810042, 27.40010580820802, 46.15619785585283, 38.26483200699776, 20.44172438427552, 20.8042283062039, 40.33685733281881, 35.04526897307925, 53.43765380953904, 23.118311288923124, 46.246434773093995, 54.067559962798995, 27.4536739354601, 19.278382202442366, 44.47476167904283, 33.90076227087091, 31.53732846903916, 57.21836996148928, 15.796140293991145, 45.263456176258764, 19.505629671380603, 27.346642204097876, 50.49503426195374, 95.0770390179677, 33.57131140436464, 55.89290784893326, 77.90333396724647, 77.90333396724647, 94.70636877096143, 27.080884906424416, 55.02636863114642, 55.6750020519895, 56.773093073800744, 81.32370105824417, 46.15619785585283, 64.08153455750669, 61.867809250367884, 27.886006392229614, 35.87634357692165, 36.299242562272255, 39.32557222442699, 72.04880596519355, 53.02179868836547, 34.908640417532865, 30.62678644311483, 61.867809250367884, 43.78524440968234, 58.34690118774256, 90.7231444115574, 46.427438201089, 37.08754689283638, 61.38635013907805, 100.42136666458902, 26.2990087931472, 20.56185170491249, 77.90333396724647, 41.132430556936306, 83.90519244027122, 22.494734284035275, 20.64232851707703, 28.715064341305684, 60.67117749756472, 68.21443845541918, 60.908637772317434, 37.45150421083523, 25.045724266551254, 102.40200064874995, 77.29708541258049, 22.145985754016134, 57.66713919909174, 63.3349620664254, 28.65903496526287, 31.846818916590372, 30.093129052017023, 48.08868329990042, 28.214717811314713, 28.65903496526287, 40.1011996370117, 75.21212841006654, 25.489873356103875, 45.79700747446617, 40.73270164287187, 50.5937537501147, 48.46584751755162, 51.79355004464666, 28.214717811314713, 33.4404292647954, 35.806340977539655, 38.26483200699776, 28.547303976590324, 21.214554942517648, 36.299242562272255, 48.94146538270724, 48.94146538270724, 21.090614371389922, 24.561296107667808, 31.784678801853513, 42.02569638966092, 51.99626426796157, 48.08868329990042, 23.80562446936611, 34.70469568042362, 40.25815129472595, 32.9863470466753, 19.581972547389178, 75.50650035756513, 49.133016862271724, 42.02569638966092, 32.79363281328884, 65.60118435636834, 80.37625124290746, 44.91121316574036, 29.053548433146837, 30.093129052017023, 41.943695056893915, 35.66674528344124, 28.827451925481085, 50.004318665090636, 55.457945789238345, 34.4346215959791, 29.74253347911242, 93.96935796203766, 36.37020880080945, 25.992616060150944, 47.15856396830611, 22.10277408821932, 41.05217218133816, 99.63988138921447, 44.04255166984753, 28.60311491479713, 112.02812994692015, 64.58413226217957, 37.305494633128845, 38.94340253332466, 34.367432111764096, 40.33685733281881, 33.57131140436464, 18.039758452611142, 60.434642992688794, 36.37020880080945, 28.827451925481085, 51.79355004464666, 38.115651564714454, 45.88654216244616, 63.83170433047907, 69.83209724580306, 30.507383918455513, 46.427438201089, 47.62135285300038, 62.84208548133934, 26.76538304079522, 26.76538304079522, 18.467558995986426, 43.78524440968234, 39.172256347947744, 58.34690118774256, 61.867809250367884, 34.4346215959791, 55.89290784893326, 69.83209724580306, 24.657426302639607, 68.48142221881122, 42.52108200006278, 37.8190332109868, 22.582776184718522, 58.57526450222688, 67.68358948381629, 76.39654709045266, 49.133016862271724, 41.45503620813183, 29.33866439394716, 78.51433738825796, 25.94189877662824, 58.57526450222688, 19.505629671380603, 37.232703447205715, 59.03467599561125, 26.35042423874788, 86.23113005463887, 81.641993024771, 46.246434773093995, 41.2128458403547, 59.265731184583274, 71.76791429061606, 37.08754689283638, 37.08754689283638, 45.61846186294896, 16.88071548801052, 49.51837188956857, 26.817710263716034, 20.124804822565828, 38.26483200699776, 54.598150033144236, 23.345182011801924, 24.513371642677637, 28.940279382223952, 25.192907336481866, 31.291903031786116, 40.1011996370117, 20.007230835729846, 38.79157659527073, 27.4536739354601, 24.948080241282657, 61.626609520407456, 34.63697922255266, 30.507383918455513, 22.319678873017494, 26.196478674560588, 62.3530444922685, 34.63697922255266, 63.582848099201826, 51.591626129191596, 45.97625189374871, 48.08868329990042, 73.46984133199462, 44.82358155812517, 77.90333396724647, 67.9484955630781, 72.61389175398708, 46.427438201089, 28.2698785323965, 39.479488162280596, 31.35307975333966, 50.396507397123145, 68.7494509271083, 57.21836996148928, 26.453556883236853, 31.475792237910618, 43.95661501095434, 39.7114923241556, 47.435694792893365, 54.49161709130268, 100.02986085865793, 69.01852867010916, 60.199030648454944, 63.3349620664254, 19.128356641900336, 141.0647140528757, 79.75075829332496, 59.73055807527518, 64.58413226217957, 98.86447768050158, 113.7923162908356, 111.59137365930344, 44.99901609609206, 76.69555472159655, 62.3530444922685, 52.71203249107963, 41.698649825700315, 23.482371718117374, 111.15632012308374, 69.83209724580306, 40.41571724385996, 64.08153455750669, 36.942956250241515, 55.6750020519895, 50.6926662381911, 62.10995300937982, 16.04489330813532, 18.181246031735814, 41.37414835829223, 31.72265993608251, 35.11378365112735, 16.61900425204837, 23.620367629992042, 20.8042283062039, 56.9952966774873, 35.18243227767251, 66.89505178172818, 28.827451925481085, 20.763634702888442, 73.75739416067401, 45.529450258314064, 43.444504934395866, 42.854578979512624, 36.228414794400415, 69.83209724580306, 52.19977189231385, 87.24759532868651, 78.51433738825796, 70.65525591042105, 25.740017318702005, 23.118311288923124, 21.717643615002626, 38.19016894386643, 22.10277408821932, 25.790339917193062, 40.57389988868238, 23.75917448515314, 42.438114061533675, 39.095822701298815, 59.964336869699544, 57.21836996148928, 83.25223705241488, 23.16350836406023, 29.9173176993003, 56.9952966774873, 50.1020787877761, 30.093129052017023, 25.489873356103875, 24.9968545761854, 31.90908051733889, 51.09025374715126, 19.128356641900336, 27.507346790200362, 65.34542957624015, 25.44013703364339, 42.854578979512624, 41.132430556936306, 42.52108200006278, 66.11570082213345, 28.15966472087403, 37.9670527218579, 25.34095533871963, 17.013112580497378, 33.57131140436464, 27.77728915635237, 46.60915005647103, 30.388446900124176, 108.15806803146498, 49.71218131735948, 32.47494113727571, 40.892124929554, 60.434642992688794, 20.84490127137742, 20.24306974171993, 106.89798837141365, 24.133327041205547, 58.119428177447986, 41.05217218133816, 43.5294404008871, 44.30137101535166, 50.990565478815945, 45.17513726542669, 56.9952966774873, 55.457945789238345, 22.27612829392538, 28.771203256660744, 29.110349096271467, 25.192907336481866, 26.87013978811538, 63.582848099201826, 30.447857334592324, 33.702705802144536, 63.83170433047907, 69.83209724580306, 18.721779026482544, 52.301824199806006, 42.02569638966092, 93.2380826167318, 110.29129477712178, 49.51837188956857, 70.10541234668786, 29.33866439394716, 53.22932014095188, 30.866995306309306, 66.89505178172818, 44.5617113320772, 51.290215346585704, 77.59961765106212, 72.04880596519355, 50.89107172411317, 45.351947753528236, 28.940279382223952, 105.24068867470233, 73.18340956555208, 76.09870517958821, 65.34542957624015, 20.723120306578686, 23.073202403172917, 101.20898120494374, 98.09510821132314, 119.72006788311512, 119.72006788311512, 119.72006788311512, 95.44916002730179, 38.71588573814723, 41.861853726676614, 31.10908815096766, 51.39048944047805, 27.080884906424416, 115.13366410675533, 61.14702744030778, 57.66713919909174, 41.943695056893915, 46.33684810653952, 42.27266353442275, 59.265731184583274, 35.806340977539655, 66.63425269055388, 50.49503426195374, 124.00383592430612, 44.5617113320772, 57.442316329639176, 39.78912962559818, 43.10641711474324, 35.32013242426123, 21.256030110048442, 22.016603538363892, 21.173160702117308, 33.50580642730562, 40.49473132867618, 19.96819234893607, 45.88654216244616, 33.245062173048, 31.846818916590372, 49.42175070681059, 35.11378365112735, 34.70469568042362, 18.28808948244362, 23.25416777248505, 15.520899957093135, 27.668995768810042, 36.65546388385301, 41.53608219643496, 38.867415430562446, 39.01953819342525, 31.475792237910618, 69.28865955361954, 50.49503426195374, 35.87634357692165, 23.16350836406023, 31.9714638416083, 29.9173176993003, 23.666546242140512, 31.9714638416083, 57.21836996148928, 43.02230705927991, 28.214717811314713, 31.048387510438282, 42.190180314697045, 23.16350836406023, 24.89940107578365, 34.501942437892716, 40.57389988868238, 76.09870517958821, 39.248839424943355, 33.05083646319603, 57.89284200037607, 79.13013296849392, 53.75168417471723, 23.75917448515314, 14.694892728788941, 22.319678873017494, 43.275130861102994, 27.668995768810042, 78.2082389954826, 85.2265069597304, 57.442316329639176, 43.275130861102994, 53.75168417471723, 45.086990684122, 65.60118435636834, 27.940524471445343, 64.33234259239626, 18.2524054167373, 38.0412795852805, 75.80202444429054, 50.79177210350426, 40.022953418667065, 92.1518260518184, 32.60204446315635, 68.7494509271083, 34.300373728940066, 33.702705802144536, 66.63425269055388, 78.2082389954826, 14.159568646729724, 28.436008585761353, 43.10641711474324, 44.30137101535166, 17.934365811960372, 19.697047472120772, 43.699809813576366, 79.43983955226133, 30.74665629604239, 56.773093073800744, 83.25223705241488, 42.854578979512624, 36.15772522700737, 20.124804822565828, 30.21091022897123, 30.507383918455513, 36.65546388385301, 43.699809813576366, 56.11166650505244, 44.5617113320772, 40.57389988868238, 49.037147591523336, 37.01518097066168, 44.04255166984753, 67.1568716116849, 31.41437607743883, 49.51837188956857, 50.6926662381911, 51.49095957359226, 21.590763877931302, 33.245062173048, 66.63425269055388, 40.1011996370117, 92.1518260518184, 32.53843073806492, 45.351947753528236, 29.626578330054702, 37.08754689283638, 29.167260806600495, 60.199030648454944, 50.20003003478527, 41.780172086809024, 52.9183415530491, 19.429584428039178, 39.634006510082365, 55.24173574865985, 57.21836996148928, 56.11166650505244, 36.08717359043372, 28.60311491479713, 75.50650035756513, 60.434642992688794, 45.7076474882609, 62.84208548133934, 104.4216991379013, 49.9067492938035, 14.552086348683558, 35.66674528344124, 45.88654216244616, 33.90076227087091, 31.291903031786116, 44.64883097460398, 28.2698785323965, 46.427438201089, 33.4404292647954, 55.89290784893326, 56.773093073800744, 37.305494633128845, 63.582848099201826, 59.03467599561125, 34.300373728940066, 57.89284200037607, 27.028044172168716, 36.65546388385301, 69.01852867010916, 45.61846186294896, 26.922671813996086, 40.022953418667065, 46.33684810653952, 32.72964525738904, 47.34313736903344, 60.434642992688794, 25.2915095879081, 52.609179777419676, 30.74665629604239, 27.940524471445343, 22.67116267165818, 71.48811770889668, 25.44013703364339, 28.65903496526287, 37.74523999888851, 30.806767041912167, 56.773093073800744, 66.37447035867858, 45.351947753528236, 44.82358155812517, 61.38635013907805, 83.25223705241488, 43.02230705927991, 40.022953418667065, 49.9067492938035, 47.807737558209425, 31.660762082694095, 66.11570082213345, 50.79177210350426, 55.6750020519895, 55.6750020519895, 40.1011996370117, 60.199030648454944, 60.67117749756472, 47.52843316962603, 47.34313736903344, 35.45837151398557, 97.71267162761835, 29.975807060355134, 23.028181534732802, 76.99573263552006, 31.784678801853513, 57.442316329639176, 69.83209724580306, 28.547303976590324, 66.89505178172818, 109.43300111180854, 61.867809250367884, 70.65525591042105, 61.14702744030778, 37.232703447205715, 62.597087408320306, 54.811841150452466, 43.95661501095434, 32.66578255521719, 70.93179276322707, 54.173263858709824, 36.299242562272255, 40.892124929554, 47.435694792893365, 102.40200064874995, 42.52108200006278, 27.940524471445343, 33.31005738633129, 22.189281900071105, 29.858942463791767, 24.753932740556596, 18.648789727196185, 18.758380733362646, 24.561296107667808, 22.538712245181248, 36.942956250241515, 22.84897480427519, 38.0412795852805, 19.505629671380603, 49.42175070681059, 23.299630451611073, 46.97471015107385, 19.620255961497417, 35.66674528344124, 35.18243227767251, 34.56939489431364, 61.14702744030778, 43.78524440968234, 59.265731184583274, 56.33128135834348, 23.75917448515314, 24.322607071104724, 22.84897480427519, 21.845268970328604, 22.10277408821932, 35.52769382137767, 24.322607071104724, 31.291903031786116, 42.27266353442275, 39.94485987582193, 44.5617113320772, 69.55984769951485, 24.513371642677637, 41.861853726676614, 35.32013242426123, 38.56494680401705, 46.066137010589394, 44.04255166984753, 42.77096031537661, 108.15806803146498, 82.60436300370014, 75.80202444429054, 91.79256012938836, 29.626578330054702, 48.84596987082467, 49.32531805312982, 55.457945789238345, 61.38635013907805, 65.09067188950593, 43.87084603329354, 57.442316329639176, 68.21443845541918, 80.69083499570341, 96.95226558425567, 97.71267162761835, 99.2514223056366, 41.37414835829223, 29.74253347911242, 23.482371718117374, 60.908637772317434, 66.11570082213345, 27.4536739354601, 30.74665629604239, 57.21836996148928, 45.79700747446617, 50.396507397123145, 35.32013242426123, 45.44061233480379, 75.21212841006654, 41.37414835829223, 60.434642992688794, 53.12545808585662, 54.811841150452466, 34.300373728940066, 31.475792237910618, 73.75739416067401, 59.964336869699544, 54.27917440928786, 22.84897480427519, 21.464625403032095, 30.74665629604239, 46.70027251047952, 76.69555472159655, 61.14702744030778, 35.18243227767251, 40.25815129472595, 60.67117749756472, 52.9183415530491, 67.1568716116849, 52.097918711212095, 44.64883097460398, 41.37414835829223, 41.37414835829223, 42.68750480955783, 30.866995306309306, 66.89505178172818, 22.40703562619394, 76.69555472159655, 26.401940203271806, 55.6750020519895, 140.51475485198003, 34.09998263323346, 65.60118435636834, 51.09025374715126, 32.033969127372174, 78.51433738825796, 41.617286632367644, 55.6750020519895, 106.89798837141365, 25.44013703364339, 34.367432111764096, 102.80279074731456, 50.1020787877761, 32.47494113727571, 34.97688798216538, 21.049461966173517, 40.1011996370117, 31.846818916590372, 16.94678474038272, 31.660762082694095, 46.88305220915621, 38.64034257045414, 38.79157659527073, 43.275130861102994, 48.27689708476307, 55.24173574865985, 32.09659661306934, 35.04526897307925, 53.75168417471723, 50.79177210350426, 19.429584428039178, 68.7494509271083, 73.18340956555208, 73.18340956555208, 58.57526450222688, 51.99626426796157, 51.894808174781225, 55.457945789238345, 92.1518260518184, 49.133016862271724, 47.250760544967605, 24.370158584949497, 57.89284200037607, 21.297586362924953, 45.17513726542669, 40.1011996370117, 30.447857334592324, 52.81508628524973, 22.450842133517945, 27.080884906424416, 95.44916002730179, 47.250760544967605, 77.59961765106212, 55.24173574865985, 48.75066069158903, 59.03467599561125, 68.48142221881122, 65.09067188950593, 56.11166650505244, 30.866995306309306, 40.65322322588273, 37.9670527218579, 75.80202444429054, 63.83170433047907, 70.37979717263202, 40.81233544283598, 55.457945789238345, 68.21443845541918, 50.20003003478527, 60.434642992688794, 128.94358717585138, 30.987805310145493, 62.3530444922685, 34.77254452624438, 65.60118435636834, 16.36135047271935, 30.329152388418176, 122.55914747655109, 39.479488162280596, 41.698649825700315, 40.892124929554, 61.14702744030778, 42.77096031537661, 67.9484955630781, 55.457945789238345, 22.84897480427519, 28.547303976590324, 52.9183415530491, 59.73055807527518, 116.49082330627213, 46.60915005647103, 34.97688798216538, 30.26997357328364, 34.09998263323346, 43.444504934395866, 76.39654709045266, 28.547303976590324, 21.973644325590612, 41.37414835829223, 76.09870517958821, 21.008389858307645, 80.06289393593435, 42.3553080116137, 47.807737558209425, 37.89297069129353, 79.75075829332496, 164.27839185643572, 50.6926662381911, 53.333385249853684, 60.434642992688794, 65.09067188950593, 43.61454191906839, 32.348333340301785, 41.780172086809024, 22.016603538363892, 18.145770440989054, 36.44131378072682, 42.60421214369869, 18.868615759264884, 18.868615759264884, 19.658614220966218, 25.489873356103875, 29.975807060355134, 44.91121316574036, 31.90908051733889, 53.12545808585662, 58.57526450222688, 56.11166650505244, 41.37414835829223, 41.617286632367644, 32.033969127372174, 51.290215346585704, 83.57807709540792, 41.617286632367644, 92.87458178129451, 36.65546388385301, 35.52769382137767, 57.89284200037607, 26.09434828302453, 83.25223705241488, 61.14702744030778, 62.3530444922685, 42.3553080116137, 42.854578979512624, 46.7915731119465, 55.89290784893326, 32.53843073806492, 24.370158584949497, 30.329152388418176, 24.948080241282657, 31.9714638416083, 27.080884906424416, 57.21836996148928, 67.68358948381629, 109.00636227269737, 41.861853726676614, 31.048387510438282, 58.34690118774256, 44.5617113320772, 51.49095957359226, 37.160054292819765, 24.18050850623376, 45.17513726542669, 49.71218131735948, 75.21212841006654, 26.09434828302453, 59.49769069799599, 43.02230705927991, 62.84208548133934, 40.17959882934157, 46.427438201089, 55.24173574865985, 52.609179777419676, 40.33685733281881, 28.60311491479713, 19.543763832601815, 39.248839424943355, 23.25416777248505, 27.507346790200362, 23.898797049169467, 34.501942437892716, 38.79157659527073, 40.57389988868238, 14.75240691276831, 37.01518097066168, 19.658614220966218, 55.02636863114642, 47.62135285300038, 42.27266353442275, 24.18050850623376, 28.771203256660744, 38.48969815122384, 29.74253347911242, 28.715064341305684, 34.70469568042362, 52.097918711212095, 36.15772522700737, 28.996858600549206, 45.79700747446617, 33.4404292647954, 33.63694444585419, 43.3597351955915, 44.64883097460398, 43.02230705927991, 59.03467599561125, 29.22428378123494, 27.4536739354601, 40.81233544283598, 31.72265993608251, 66.37447035867858, 42.02569638966092, 56.11166650505244, 76.39654709045266, 32.22221914035047, 24.753932740556596, 78.82163381626928, 71.48811770889668, 61.626609520407456, 52.40407602298735, 39.32557222442699, 27.080884906424416, 27.028044172168716, 50.6926662381911, 40.25815129472595, 47.250760544967605, 30.987805310145493, 25.192907336481866, 24.513371642677637, 48.27689708476307, 42.93836112094585, 37.9670527218579, 39.40245503911115, 61.38635013907805, 50.990565478815945, 42.68750480955783, 56.33128135834348, 32.53843073806492, 130.4635333784854, 100.02986085865793, 65.34542957624015, 62.10995300937982, 69.28865955361954, 38.19016894386643, 24.513371642677637, 32.033969127372174, 38.94340253332466, 23.16350836406023, 30.56702687878971, 21.590763877931302, 76.99573263552006, 57.21836996148928, 43.5294404008871, 123.52039078374908, 80.37625124290746, 43.699809813576366, 42.02569638966092, 85.89494673838159, 33.96703939987107, 43.61454191906839, 30.68666283939559, 26.2990087931472, 32.22221914035047, 30.447857334592324, 49.42175070681059, 36.798929313140206, 50.89107172411317, 30.21091022897123, 66.63425269055388, 39.556671887793456, 39.556671887793456, 39.556671887793456, 59.73055807527518, 40.022953418667065, 60.199030648454944, 39.172256347947744, 96.19777706766972, 103.60908302823987, 41.53608219643496, 49.9067492938035, 41.29341833835342, 22.582776184718522, 27.995149135337257, 42.3553080116137, 41.2128458403547, 57.66713919909174, 40.022953418667065, 39.095822701298815, 47.06654728734682, 33.1801937796528, 74.91890411002962, 56.773093073800744, 90.01713130052181, 79.75075829332496, 36.87087245606033, 50.6926662381911, 59.964336869699544, 47.62135285300038, 31.598985005566618, 45.61846186294896, 29.626578330054702, 51.79355004464666, 25.44013703364339, 40.33685733281881, 43.19069160819034, 23.482371718117374, 51.39048944047805, 25.790339917193062, 68.7494509271083, 124.00383592430612, 53.85677052285177, 135.13171912803952, 107.31637526839133, 61.14702744030778, 33.50580642730562, 47.34313736903344, 37.5980852538711, 26.14536361045317, 36.798929313140206, 40.1011996370117, 42.60421214369869, 54.27917440928786, 50.89107172411317, 35.806340977539655, 54.49161709130268, 45.351947753528236, 69.28865955361954, 88.27604237373367, 43.78524440968234, 44.47476167904283, 17.348666159108166, 22.494734284035275, 46.33684810653952, 38.19016894386643, 50.004318665090636, 83.90519244027122, 16.88071548801052, 34.300373728940066, 21.090614371389922, 104.01459751366006, 59.73055807527518, 66.37447035867858, 32.72964525738904, 36.798929313140206, 80.06289393593435, 76.39654709045266, 37.232703447205715, 62.3530444922685, 48.65553748142455, 27.995149135337257, 21.464625403032095, 36.72712654703974, 63.3349620664254, 45.351947753528236, 42.77096031537661, 47.435694792893365, 126.44959243764188, 67.68358948381629, 76.39654709045266, 60.199030648454944, 45.7076474882609, 32.53843073806492, 29.053548433146837, 37.232703447205715, 122.08133464248264, 24.275148340672132, 37.524723159601, 50.49503426195374, 53.96206231832688, 34.70469568042362, 65.09067188950593, 112.90677745995042, 59.49769069799599, 97.33172602285453, 69.83209724580306, 75.80202444429054, 60.199030648454944, 34.77254452624438, 37.74523999888851, 35.94648303373688, 26.505274475545082, 68.7494509271083, 33.702705802144536, 44.30137101535166, 28.15966472087403, 54.49161709130268, 45.97625189374871, 25.094689498803355, 42.93836112094585, 55.6750020519895, 64.83690740886694, 70.65525591042105, 57.21836996148928, 41.617286632367644, 61.38635013907805, 47.435694792893365, 70.10541234668786, 39.78912962559818, 52.19977189231385, 35.25121511458854, 95.0770390179677, 89.31661242710538, 37.8190332109868, 34.4346215959791, 42.438114061533675, 46.97471015107385, 41.2128458403547, 45.351947753528236, 76.09870517958821, 34.908640417532865, 84.2335880783952, 39.32557222442699, 41.45503620813183, 50.990565478815945, 22.319678873017494, 37.89297069129353, 50.1020787877761, 35.806340977539655, 36.942956250241515, 46.97471015107385, 57.442316329639176, 73.75739416067401, 25.539706914745466, 38.64034257045414, 40.49473132867618, 37.45150421083523, 20.007230835729846, 72.89809449073893, 59.49769069799599, 43.87084603329354, 47.71445419747699, 50.89107172411317, 69.01852867010916, 59.964336869699544, 55.02636863114642, 53.12545808585662, 53.12545808585662, 67.41971617548266, 39.172256347947744, 65.8579401324037, 60.908637772317434, 72.89809449073893, 26.66103470132838, 38.56494680401705, 45.61846186294896, 28.827451925481085, 78.2082389954826, 23.666546242140512, 156.14440466345445, 35.32013242426123, 35.32013242426123, 31.660762082694095, 40.17959882934157, 32.159346537604755, 28.65903496526287, 31.35307975333966, 44.21492934326177, 36.65546388385301, 55.457945789238345, 38.41459632502361, 38.41459632502361, 31.846818916590372, 22.232662691545976, 55.24173574865985, 39.172256347947744, 45.529450258314064, 54.173263858709824, 70.37979717263202, 32.60204446315635, 29.22428378123494, 29.22428378123494, 37.8190332109868, 42.77096031537661, 31.72265993608251, 43.95661501095434, 29.56877039069624, 37.74523999888851, 39.94485987582193, 30.034410770075546, 42.68750480955783, 37.74523999888851, 25.1437504597293, 41.53608219643496, 44.47476167904283, 48.182698291098816, 47.62135285300038, 64.08153455750669, 55.6750020519895, 82.28231911219243, 70.37979717263202, 56.33128135834348, 58.119428177447986, 56.33128135834348, 53.85677052285177, 59.49769069799599, 68.21443845541918, 43.444504934395866, 165.5668432526847, 30.62678644311483, 23.073202403172917, 53.96206231832688, 44.30137101535166, 103.20514949397284, 49.133016862271724, 46.70027251047952, 101.20898120494374, 40.25815129472595, 50.5937537501147, 52.609179777419676, 55.02636863114642, 54.067559962798995, 34.70469568042362, 82.60436300370014, 70.93179276322707, 69.28865955361954, 104.83039411284926, 105.24068867470233, 38.0412795852805, 52.50652775191775, 66.37447035867858, 45.17513726542669, 24.948080241282657, 57.442316329639176, 75.21212841006654, 27.668995768810042, 19.128356641900336, 74.62682298320091, 89.31661242710538, 59.73055807527518, 35.66674528344124, 27.77728915635237, 27.77728915635237, 29.453492679719986, 42.27266353442275, 17.6218678981495, 18.28808948244362, 47.250760544967605, 35.94648303373688, 62.597087408320306, 54.067559962798995, 16.01358620918395, 17.690837939037834, 40.892124929554, 36.942956250241515, 35.7364749685521, 41.05217218133816, 66.89505178172818, 85.56007407666469, 76.39654709045266, 77.90333396724647, 62.3530444922685, 44.64883097460398, 144.4099673433467, 67.9484955630781, 78.82163381626928, 31.846818916590372, 30.56702687878971, 63.3349620664254, 51.1901369093998, 44.5617113320772, 55.02636863114642, 44.82358155812517, 21.090614371389922, 27.080884906424416, 19.774139536218833, 43.95661501095434, 20.203571081597193, 21.632974546568068, 25.44013703364339, 49.71218131735948, 20.322298876557472, 37.67159077349981, 52.609179777419676, 58.34690118774256, 69.01852867010916, 45.263456176258764, 54.598150033144236, 62.3530444922685, 32.09659661306934, 28.827451925481085, 86.90744918987926, 76.39654709045266, 26.14536361045317, 28.325147094540903, 95.0770390179677, 37.160054292819765, 59.03467599561125, 49.22907356066477, 60.67117749756472, 41.943695056893915, 32.53843073806492, 25.992616060150944, 62.10995300937982, 37.9670527218579, 32.72964525738904, 25.689792910615846, 106.89798837141365, 38.33964103892552, 63.83170433047907, 79.75075829332496, 30.151962130172098, 237.16355295871682, 54.38529201854992, 28.996858600549206, 117.86398026897247, 48.65553748142455, 57.442316329639176, 73.75739416067401, 44.5617113320772, 34.501942437892716, 53.43765380953904, 46.7915731119465, 64.58413226217957, 46.51820540231633, 52.609179777419676, 46.066137010589394, 62.84208548133934, 44.91121316574036, 28.996858600549206, 35.18243227767251, 38.56494680401705, 35.18243227767251, 55.24173574865985, 70.37979717263202, 44.04255166984753, 46.15619785585283, 35.94648303373688, 46.246434773093995, 40.33685733281881, 38.48969815122384, 64.58413226217957, 33.245062173048, 33.76859572409541, 28.996858600549206, 58.80452160545304, 36.37020880080945, 33.11545195869231, 75.21212841006654, 38.867415430562446, 42.27266353442275, 39.86691871057298, 49.133016862271724, 35.45837151398557, 20.36202965350754, 54.811841150452466, 56.9952966774873, 37.8190332109868, 32.72964525738904, 19.24076585887165, 58.119428177447986, 43.95661501095434, 41.780172086809024, 42.93836112094585, 22.80439145296227, 54.067559962798995, 51.99626426796157, 36.016759615546256, 32.348333340301785, 36.15772522700737, 44.82358155812517, 84.563269020706, 41.45503620813183, 30.388446900124176, 38.79157659527073, 27.886006392229614, 29.167260806600495, 44.82358155812517, 37.8190332109868, 61.38635013907805, 37.45150421083523, 79.43983955226133, 66.63425269055388, 66.63425269055388, 50.29817277977263, 37.01518097066168, 42.438114061533675, 17.79479946994832, 16.554212894695087, 35.52769382137767, 33.57131140436464, 33.57131140436464, 81.9615307508321, 29.858942463791767, 31.9714638416083, 46.246434773093995, 32.411575418595035, 47.250760544967605, 72.33079701870139, 43.87084603329354, 62.10995300937982, 50.79177210350426, 51.09025374715126, 70.37979717263202, 27.940524471445343, 39.556671887793456, 82.60436300370014, 86.56862915519338, 101.20898120494374, 27.561124577174567, 59.265731184583274, 34.97688798216538, 38.71588573814723, 40.41571724385996, 41.132430556936306, 51.09025374715126, 37.01518097066168, 48.182698291098816, 25.44013703364339, 55.457945789238345, 45.17513726542669, 24.609314266596865, 31.846818916590372, 54.811841150452466, 28.380523708580828, 35.04526897307925, 39.7114923241556, 53.85677052285177, 53.333385249853684, 44.99901609609206, 56.11166650505244, 58.34690118774256, 67.41971617548266, 66.11570082213345, 65.60118435636834, 31.72265993608251, 25.992616060150944, 45.351947753528236, 114.23768682435218, 81.641993024771, 155.5356553203632, 147.25819873544805, 49.037147591523336, 25.58963789966816, 38.19016894386643, 17.79479946994832, 81.9615307508321, 42.10785803778799, 23.16350836406023, 100.81440478091984, 116.94675549762589, 99.2514223056366, 95.44916002730179, 46.246434773093995, 152.5273234252381, 88.96839995507139, 24.22778221261098, 30.56702687878971, 50.49503426195374, 54.598150033144236, 69.83209724580306, 51.894808174781225, 67.41971617548266, 75.50650035756513, 44.47476167904283, 79.75075829332496, 38.19016894386643, 41.2128458403547, 28.827451925481085, 34.70469568042362, 28.883810562338418, 76.39654709045266, 56.33128135834348, 66.11570082213345, 81.00664999449546, 158.60332124553608, 135.66060972280735, 37.67159077349981, 27.940524471445343, 33.96703939987107, 36.65546388385301, 58.57526450222688, 77.90333396724647, 53.96206231832688, 45.7076474882609, 42.854578979512624, 61.14702744030778, 50.29817277977263, 126.44959243764188, 42.77096031537661, 28.325147094540903, 72.33079701870139, 44.04255166984753, 41.132430556936306, 48.182698291098816, 38.26483200699776, 49.9067492938035, 63.08804244970428, 46.97471015107385, 32.09659661306934, 81.32370105824417, 66.63425269055388, 24.039240115615065, 29.281418237700255, 38.867415430562446, 87.24759532868651, 48.84596987082467, 52.81508628524973, 65.60118435636834, 46.97471015107385, 55.89290784893326, 59.73055807527518, 43.10641711474324, 31.53732846903916, 35.52769382137767, 52.9183415530491, 86.23113005463887, 86.23113005463887, 60.199030648454944, 59.73055807527518, 60.908637772317434, 44.5617113320772, 103.60908302823987, 57.66713919909174, 57.442316329639176, 34.908640417532865, 69.28865955361954, 44.64883097460398, 106.48123261036584, 70.10541234668786, 25.94189877662824, 27.72308958496738, 58.80452160545304, 43.95661501095434, 38.26483200699776, 63.582848099201826, 88.62154503481175, 94.70636877096143, 58.34690118774256, 36.58394105020859, 57.442316329639176, 50.6926662381911, 54.067559962798995, 26.557093177483523, 27.886006392229614, 21.887977204970085, 41.943695056893915, 55.457945789238345, 80.06289393593435, 46.15619785585283, 36.15772522700737, 99.2514223056366, 77.59961765106212, 59.03467599561125, 42.60421214369869, 82.92766733936132, 35.94648303373688, 50.004318665090636, 51.290215346585704, 59.265731184583274, 28.214717811314713, 23.20879380099744, 23.20879380099744, 48.65553748142455, 43.10641711474324, 45.7076474882609, 43.61454191906839, 69.01852867010916, 93.6030061542134, 180.4245166495895, 85.89494673838159, 97.33172602285453, 37.45150421083523, 82.92766733936132, 63.3349620664254, 40.81233544283598, 42.3553080116137, 75.21212841006654, 60.199030648454944, 71.20941195067294, 66.89505178172818, 35.45837151398557, 32.79363281328884, 31.846818916590372, 26.817710263716034, 66.89505178172818, 35.806340977539655, 61.867809250367884, 39.78912962559818, 120.65904401308495, 130.9741532108186, 64.08153455750669, 168.17414165184545, 60.434642992688794, 66.63425269055388, 66.63425269055388, 29.167260806600495, 45.351947753528236, 53.646802873050504, 73.18340956555208, 25.840760898054555, 36.016759615546256, 76.69555472159655, 86.90744918987926, 44.99901609609206, 57.442316329639176, 61.626609520407456, 66.89505178172818, 41.2128458403547, 67.9484955630781, 59.964336869699544, 54.811841150452466, 53.85677052285177, 130.4635333784854, 21.845268970328604, 33.1801937796528, 67.9484955630781, 66.89505178172818, 27.507346790200362, 91.43469485205186, 84.563269020706, 60.908637772317434, 106.48123261036584, 47.435694792893365, 34.56939489431364, 44.04255166984753, 53.75168417471723, 40.33685733281881, 36.44131378072682, 33.90076227087091, 71.20941195067294, 44.47476167904283, 68.48142221881122, 52.81508628524973, 39.01953819342525, 45.351947753528236, 87.24759532868651, 54.811841150452466, 79.75075829332496, 99.63988138921447, 30.21091022897123, 45.44061233480379, 45.7076474882609, 45.7076474882609, 81.00664999449546, 64.83690740886694, 69.83209724580306, 40.57389988868238, 35.11378365112735, 76.39654709045266, 53.43765380953904, 57.66713919909174, 77.29708541258049, 50.79177210350426, 44.21492934326177, 57.89284200037607, 51.39048944047805, 28.380523708580828, 56.33128135834348, 88.96839995507139, 54.27917440928786, 44.04255166984753, 50.49503426195374, 31.90908051733889, 51.79355004464666, 30.987805310145493, 68.21443845541918, 24.70563239932894, 24.80232751057381, 18.61240185635776, 75.50650035756513, 61.626609520407456, 36.016759615546256, 37.89297069129353, 90.36944839098756, 64.83690740886694, 57.89284200037607, 49.9067492938035, 59.265731184583274, 25.992616060150944, 46.70027251047952, 76.39654709045266, 54.811841150452466, 54.598150033144236, 112.90677745995042, 56.55175575986736, 63.08804244970428, 47.62135285300038, 67.1568716116849, 62.3530444922685, 56.11166650505244, 80.06289393593435, 88.96839995507139, 91.79256012938836, 32.79363281328884, 70.37979717263202, 72.61389175398708, 57.442316329639176, 14.354485905365031, 21.256030110048442, 19.851533330124635, 19.543763832601815, 20.007230835729846, 18.979498589532987, 15.16135869683449, 14.75240691276831, 18.6852487375444, 17.51891667538985, 24.086237637543274, 24.086237637543274, 15.370066359341997, 16.84777751703098, 23.118311288923124, 21.760102337787384, 24.80232751057381, 11.489186055851967, 21.131847230940878, 27.080884906424416, 13.121062323276538, 24.561296107667808, 23.4365524707797, 24.039240115615065, 21.590763877931302, 32.159346537604755, 32.159346537604755, 19.812798643245316, 29.511075247281195, 34.367432111764096, 26.76538304079522, 19.24076585887165, 19.24076585887165, 18.359667040887704, 25.34095533871963, 28.15966472087403, 28.65903496526287, 24.039240115615065, 22.27612829392538, 16.716667072215767, 22.059646737841184, 17.046373787079762, 20.322298876557472, 27.61500750152876, 14.049378009312553, 23.16350836406023, 17.553166745968216, 28.883810562338418, 22.494734284035275, 22.938403163153815, 21.42274322042474, 19.735555861572347, 32.28521466114651, 21.464625403032095, 34.23344619169951, 28.771203256660744, 45.529450258314064, 43.95661501095434, 26.196478674560588, 18.039758452611142, 21.090614371389922, 27.40010580820802, 24.89940107578365, 18.216790978454146, 16.139182322446413, 28.1047190510635, 22.893645317537526, 19.391673055829287, 15.220698576992893, 16.521912001744084, 37.08754689283638, 38.26483200699776, 31.35307975333966, 28.883810562338418, 26.2990087931472, 24.22778221261098, 21.131847230940878, 25.44013703364339, 18.6852487375444, 22.67116267165818, 29.453492679719986, 23.25416777248505, 15.520899957093135, 14.242778409807016, 25.489873356103875, 28.325147094540903, 37.74523999888851, 34.03344610288473, 48.46584751755162, 37.8190332109868, 40.81233544283598, 27.240027749911206, 22.80439145296227, 25.94189877662824, 19.543763832601815, 30.68666283939559, 26.24769367033534, 18.868615759264884, 21.173160702117308, 19.31607208722833, 21.675267738479917, 23.80562446936611, 25.2915095879081, 31.169907463288272, 21.930768935644906, 15.370066359341997, 23.118311288923124, 18.00455893510183, 17.934365811960372, 19.091033031037714, 22.10277408821932, 20.046345644076645, 28.380523708580828, 24.89940107578365, 31.598985005566618, 29.396022468352484, 23.80562446936611, 19.697047472120772, 26.922671813996086, 22.189281900071105, 16.61900425204837, 32.85774546700983, 15.400115355370453, 19.20322291302118, 13.777638440111984, 18.039758452611142, 21.845268970328604, 27.72308958496738, 32.9863470466753, 23.666546242140512, 25.489873356103875, 22.80439145296227, 24.370158584949497, 19.31607208722833, 18.2524054167373, 25.689792910615846, 49.809370301716946, 18.359667040887704, 27.72308958496738, 27.40010580820802, 25.790339917193062, 42.93836112094585, 34.501942437892716, 27.995149135337257, 28.715064341305684, 15.280270706875589, 40.892124929554, 20.203571081597193, 18.648789727196185, 10.941666595318791, 13.172416709211404, 30.093129052017023, 22.75989509352673, 26.2990087931472, 16.01358620918395, 17.725424121461643, 22.189281900071105, 22.189281900071105, 15.612109685326937, 16.489674134905655, 17.934365811960372, 17.934365811960372, 28.940279382223952, 32.79363281328884, 28.214717811314713, 24.609314266596865, 24.948080241282657, 22.582776184718522, 22.626926270737744, 27.507346790200362, 27.507346790200362, 19.278382202442366, 24.22778221261098, 31.230845679407448, 32.348333340301785, 21.548635571548648, 20.203571081597193, 24.275148340672132, 29.74253347911242, 29.74253347911242, 26.043432497666576, 29.053548433146837, 12.76714405293921, 22.494734284035275, 18.110364070885183, 28.380523708580828, 27.561124577174567, 16.979916273809664, 18.79505399780919, 20.8042283062039, 20.203571081597193, 17.79479946994832, 21.33922385967176, 30.21091022897123, 26.043432497666576, 22.319678873017494, 28.2698785323965, 33.57131140436464, 24.086237637543274, 23.299630451611073, 27.293282919182133, 24.417803063601138, 25.2915095879081, 24.039240115615065, 32.411575418595035, 29.975807060355134, 28.883810562338418, 21.845268970328604, 24.513371642677637, 18.039758452611142, 26.817710263716034, 24.850816893992036, 23.118311288923124, 29.167260806600495, 32.53843073806492, 14.2706235200274, 15.280270706875589, 26.609013186724894, 29.33866439394716, 23.16350836406023, 24.133327041205547, 23.574279122424027, 30.806767041912167, 20.124804822565828, 21.008389858307645, 19.016604146815915, 31.169907463288272, 17.51891667538985, 28.325147094540903, 30.151962130172098, 32.411575418595035, 21.887977204970085, 13.885697797673076, 24.275148340672132, 18.90550453688997, 16.684049195604416, 23.52828054362271, 18.39556080667227, 24.417803063601138, 23.073202403172917, 21.717643615002626, 22.059646737841184, 13.96729813824377, 29.110349096271467, 21.548635571548648, 23.118311288923124, 19.278382202442366, 17.89937193856263, 22.016603538363892, 18.576084986220376, 15.340075995482293, 19.620255961497417, 22.36331459495464, 21.214554942517648, 18.145770440989054, 34.4346215959791, 23.20879380099744, 21.506589466713315, 15.581647041956762, 14.637602771363136, 19.24076585887165, 16.61900425204837, 14.552086348683558, 21.297586362924953, 14.609041604294383, 22.27612829392538, 34.84052601883783, 36.51255777326842, 21.214554942517648, 23.52828054362271, 19.128356641900336, 18.648789727196185, 19.278382202442366, 15.765318566620541, 14.723621737689907, 26.35042423874788, 16.8149038152427, 12.792104248423877, 12.087689040557068, 21.049461966173517, 13.146714440868186, 11.715789946123383, 16.139182322446413, 28.049880592281973, 25.63966650134361, 15.072782365110845, 15.072782365110845, 11.3553329839266, 18.28808948244362, 15.16135869683449, 13.301683750542033, 15.10225016094451, 13.198169226352599, 14.467069533669397, 25.94189877662824, 30.329152388418176, 23.80562446936611, 28.214717811314713, 32.9863470466753, 55.457945789238345, 27.83159468972004, 25.790339917193062, 24.70563239932894, 21.717643615002626, 16.36135047271935, 15.703855414343392, 27.995149135337257, 26.76538304079522, 27.61500750152876, 26.24769367033534, 19.620255961497417, 23.16350836406023, 24.417803063601138, 32.22221914035047, 33.90076227087091, 27.561124577174567, 24.561296107667808, 16.23402543319486, 17.89937193856263, 18.758380733362646, 10.440611202065584, 11.244980766441616, 11.113972772314547, 23.118311288923124, 14.076845015253456, 17.416566918622852, 17.690837939037834, 16.23402543319486, 19.20322291302118, 14.868111474214228, 12.742232560259533, 18.216790978454146, 35.04526897307925, 20.124804822565828, 16.91371785382974, 10.85651787033819, 14.354485905365031, 19.620255961497417, 20.124804822565828, 13.406010149206963, 14.694892728788941, 15.520899957093135, 13.912844803302042, 23.299630451611073, 28.715064341305684, 31.475792237910618, 31.475792237910618, 17.113091407702502, 18.758380733362646, 27.886006392229614, 27.940524471445343, 26.609013186724894, 24.850816893992036, 17.690837939037834, 40.33685733281881, 28.60311491479713, 32.348333340301785, 24.086237637543274, 27.240027749911206, 16.39333746248769, 16.94678474038272, 11.534153458662256, 35.806340977539655, 30.806767041912167, 17.89937193856263, 16.076261613516785, 17.14654807625046, 42.10785803778799, 24.609314266596865, 21.930768935644906, 22.10277408821932, 27.4536739354601, 32.159346537604755, 30.74665629604239, 21.717643615002626, 15.581647041956762, 27.72308958496738, 20.16414949200632, 18.323843311697065, 41.2128458403547, 24.609314266596865, 20.64232851707703, 20.56185170491249, 30.56702687878971, 29.800681131145613, 18.2524054167373, 19.96819234893607, 32.411575418595035, 35.04526897307925, 20.56185170491249, 42.3553080116137, 21.802644068801417, 25.094689498803355, 25.2915095879081, 27.240027749911206, 23.20879380099744, 29.511075247281195, 28.715064341305684, 26.24769367033534, 22.10277408821932, 20.124804822565828, 38.56494680401705, 22.016603538363892, 23.073202403172917, 21.008389858307645, 23.4365524707797, 16.202349296067883, 27.080884906424416, 24.18050850623376, 15.951155153095918, 18.075026786359576, 25.24216031657971, 21.380942759123343, 19.31607208722833, 41.29341833835342, 24.465540688808968, 21.049461966173517, 29.053548433146837, 17.89937193856263, 17.89937193856263, 22.10277408821932, 13.67042000714299, 23.118311288923124, 14.438841114275835, 22.40703562619394, 27.240027749911206, 12.867278004795136, 12.206311145618837, 11.511647800524438, 18.075026786359576, 20.521731031400577, 17.690837939037834, 21.049461966173517, 18.467558995986426, 14.926303617194517, 14.523692042874359, 17.416566918622852, 27.4536739354601, 28.2698785323965, 13.249825400173327, 17.690837939037834, 20.68268496272443, 22.98324851186175, 34.4346215959791, 29.800681131145613, 27.61500750152876, 11.092286978670202, 32.159346537604755, 29.626578330054702, 21.548635571548648, 15.888967492629165, 24.370158584949497, 43.699809813576366, 44.21492934326177, 23.992334296139543, 31.291903031786116, 12.302046430466076, 25.045724266551254, 48.75066069158903, 28.2698785323965, 30.62678644311483, 27.133828946090556, 25.094689498803355, 21.090614371389922, 25.94189877662824, 34.70469568042362, 18.539838978246124, 16.74934871787156, 14.104365720113616, 13.432219349369987, 11.624616945432633, 20.124804822565828, 16.979916273809664, 19.581972547389178, 17.348666159108166, 17.247311047115943, 15.340075995482293, 16.94678474038272, 20.203571081597193, 24.657426302639607, 16.84777751703098, 19.505629671380603, 29.453492679719986, 12.76714405293921, 14.868111474214228, 16.04489330813532, 29.33866439394716, 33.83461446305701, 33.63694444585419, 32.66578255521719, 19.053782246709744, 16.74934871787156, 10.814192318320675, 16.94678474038272, 13.777638440111984, 15.703855414343392, 26.817710263716034, 16.61900425204837, 15.131775567308885, 18.6852487375444, 18.145770440989054, 30.034410770075546, 24.561296107667808, 22.715485556228362, 16.2975636122563, 26.557093177483523, 29.453492679719986, 21.008389858307645, 33.11545195869231, 38.867415430562446, 36.65546388385301, 15.581647041956762, 19.812798643245316, 31.41437607743883, 17.28103011956753, 11.923552252478798, 14.467069533669397, 13.831562592310346, 25.094689498803355, 14.382549407491597, 15.888967492629165, 14.049378009312553, 14.049378009312553, 19.697047472120772, 15.400115355370453, 28.883810562338418, 19.128356641900336, 14.868111474214228, 25.891280453627324, 22.36331459495464, 20.521731031400577, 46.97471015107385, 34.16664924473404, 22.938403163153815, 31.53732846903916, 29.33866439394716, 27.83159468972004, 32.66578255521719, 17.934365811960372, 17.247311047115943, 26.453556883236853, 15.43022309819552, 29.626578330054702, 23.852165264858517, 28.827451925481085, 20.723120306578686, 15.642631884188171, 37.89297069129353, 31.784678801853513, 17.416566918622852, 30.447857334592324, 28.771203256660744, 28.771203256660744, 19.128356641900336, 19.278382202442366, 26.87013978811538, 22.715485556228362, 29.281418237700255, 27.561124577174567, 27.83159468972004, 32.60204446315635, 25.891280453627324, 20.64232851707703, 23.118311288923124, 23.299630451611073, 14.382549407491597, 16.58657693697683, 29.33866439394716, 34.70469568042362, 32.348333340301785, 26.196478674560588, 25.24216031657971, 22.538712245181248, 34.03344610288473, 16.91371785382974, 23.666546242140512, 33.90076227087091, 22.582776184718522, 26.817710263716034, 26.713157919740368, 30.56702687878971, 20.44172438427552, 22.27612829392538, 23.345182011801924, 36.15772522700737, 31.230845679407448, 21.802644068801417, 20.723120306578686, 23.94552000018515, 28.214717811314713, 19.053782246709744, 22.582776184718522, 16.94678474038272, 16.170734966018113, 23.118311288923124, 23.390822626823073, 26.35042423874788, 21.973644325590612, 14.298523068373607, 22.538712245181248, 30.74665629604239, 15.280270706875589, 15.951155153095918, 13.406010149206963, 20.282645623050005, 25.840760898054555, 37.74523999888851, 29.56877039069624, 26.817710263716034, 28.771203256660744, 23.482371718117374, 17.046373787079762, 14.495353140571094, 18.359667040887704, 19.697047472120772, 15.250455553989443, 19.96819234893607, 16.84777751703098, 18.28808948244362, 31.291903031786116, 31.230845679407448, 18.2524054167373, 27.028044172168716, 31.72265993608251, 20.046345644076645, 16.23402543319486, 24.89940107578365, 17.079700020438295, 26.35042423874788, 29.56877039069624, 15.612109685326937, 12.520205133467638, 16.979916273809664, 18.503663694167408, 16.979916273809664, 17.51891667538985, 15.734556979186685, 28.771203256660744, 28.771203256660744, 17.113091407702502, 15.612109685326937, 26.922671813996086, 42.68750480955783, 39.78912962559818, 9.693779343581, 33.90076227087091, 15.190999662372045, 31.35307975333966, 25.63966650134361, 31.35307975333966, 34.501942437892716, 15.827022278874091, 21.049461966173517, 17.079700020438295, 25.58963789966816, 24.133327041205547, 31.41437607743883, 36.72712654703974, 27.995149135337257, 22.938403163153815, 20.322298876557472, 27.40010580820802, 23.898797049169467, 27.240027749911206, 13.484791569471833, 13.458479789399725, 17.416566918622852, 18.648789727196185, 16.265763498233948, 19.96819234893607, 18.539838978246124, 18.758380733362646, 28.436008585761353, 33.83461446305701, 16.265763498233948, 19.278382202442366, 16.01358620918395, 19.658614220966218, 17.348666159108166, 35.32013242426123, 18.2524054167373, 30.388446900124176, 35.52769382137767, 19.774139536218833, 20.16414949200632, 16.74934871787156, 19.581972547389178, 14.523692042874359, 21.464625403032095, 21.464625403032095, 21.802644068801417, 24.70563239932894, 25.58963789966816, 21.380942759123343, 18.359667040887704, 22.10277408821932, 19.851533330124635, 16.265763498233948, 16.58657693697683, 19.391673055829287, 19.505629671380603, 28.380523708580828, 14.076845015253456, 37.01518097066168, 28.60311491479713, 27.186876493132687, 31.230845679407448, 29.975807060355134, 34.63697922255266, 23.574279122424027, 25.489873356103875, 24.561296107667808, 24.22778221261098, 26.76538304079522, 20.046345644076645, 20.322298876557472, 22.938403163153815, 34.4346215959791, 21.802644068801417, 23.992334296139543, 43.275130861102994, 27.240027749911206, 25.045724266551254, 21.42274322042474, 21.008389858307645, 32.60204446315635, 30.927341318986464, 24.275148340672132, 27.080884906424416, 23.20879380099744, 28.827451925481085, 17.89937193856263, 23.073202403172917, 20.92648590822237, 27.561124577174567, 29.858942463791767, 26.35042423874788, 22.27612829392538, 32.159346537604755, 21.632974546568068, 22.715485556228362, 23.94552000018515, 22.494734284035275, 28.436008585761353, 28.380523708580828, 17.51891667538985, 21.802644068801417, 24.657426302639607, 20.92648590822237, 16.8149038152427, 20.16414949200632, 20.401838105461145, 21.256030110048442, 22.75989509352673, 33.4404292647954, 11.556703116117722, 23.073202403172917, 25.63966650134361, 22.10277408821932, 20.24306974171993, 31.846818916590372, 19.89034374461794, 26.817710263716034, 22.232662691545976, 28.65903496526287, 26.817710263716034, 25.689792910615846, 23.852165264858517, 27.940524471445343, 24.18050850623376, 27.080884906424416, 34.09998263323346, 30.68666283939559, 15.581647041956762, 24.18050850623376, 33.11545195869231, 27.940524471445343, 31.10908815096766, 25.39049775763493, 20.16414949200632, 16.265763498233948, 19.278382202442366, 19.391673055829287, 28.049880592281973, 16.521912001744084, 15.370066359341997, 17.864446345871784, 24.948080241282657, 22.84897480427519, 19.581972547389178, 22.626926270737744, 21.008389858307645, 23.390822626823073, 24.753932740556596, 24.465540688808968, 23.118311288923124, 24.657426302639607, 14.81014620089916, 18.61240185635776, 16.01358620918395, 18.145770440989054, 24.850816893992036, 24.850816893992036, 19.735555861572347, 24.322607071104724, 21.632974546568068, 27.40010580820802, 19.46756991825522, 22.27612829392538, 22.626926270737744, 23.94552000018515, 26.043432497666576, 22.67116267165818, 27.028044172168716, 22.80439145296227, 24.465540688808968, 25.24216031657971, 27.995149135337257, 15.490615283867873, 24.948080241282657, 25.489873356103875, 28.827451925481085, 21.173160702117308, 22.80439145296227, 20.401838105461145, 14.781248363830914, 17.45061689198871, 32.9863470466753, 35.59715165620933, 28.491601937740562, 23.620367629992042, 29.396022468352484, 28.715064341305684, 28.491601937740562, 26.09434828302453, 24.9968545761854, 24.9968545761854, 23.16350836406023, 20.8042283062039, 27.133828946090556, 38.19016894386643, 16.265763498233948, 31.72265993608251, 20.521731031400577, 25.992616060150944, 24.275148340672132, 20.44172438427552, 23.345182011801924, 25.2915095879081, 35.59715165620933, 14.81014620089916, 23.620367629992042, 22.75989509352673, 28.325147094540903, 40.41571724385996, 18.79505399780919, 18.90550453688997, 13.777638440111984, 18.00455893510183, 25.539706914745466, 39.634006510082365, 41.05217218133816, 38.115651564714454, 28.940279382223952, 26.505274475545082, 24.039240115615065, 19.812798643245316, 26.043432497666576, 20.44172438427552, 19.774139536218833, 28.940279382223952, 26.453556883236853, 29.281418237700255, 22.893645317537526, 23.345182011801924, 17.14654807625046, 27.61500750152876, 22.450842133517945, 15.765318566620541, 27.507346790200362, 27.507346790200362, 17.113091407702502, 19.505629671380603, 28.049880592281973, 28.049880592281973, 31.72265993608251, 31.41437607743883, 28.049880592281973, 40.81233544283598, 25.094689498803355, 23.390822626823073, 29.975807060355134, 28.049880592281973, 19.620255961497417, 38.41459632502361, 38.48969815122384, 18.79505399780919, 18.83179895971962, 20.24306974171993, 20.24306974171993, 20.602050815688813, 19.429584428039178, 19.429584428039178, 34.09998263323346, 31.846818916590372, 20.84490127137742, 19.391673055829287, 23.94552000018515, 24.086237637543274, 28.491601937740562, 26.817710263716034, 45.351947753528236, 20.8856537535641, 20.8042283062039, 33.90076227087091, 23.94552000018515, 32.033969127372174, 32.033969127372174, 24.80232751057381, 40.57389988868238, 29.74253347911242, 17.28103011956753, 23.028181534732802, 32.411575418595035, 23.20879380099744, 27.028044172168716, 20.282645623050005, 33.1801937796528, 20.723120306578686, 22.67116267165818, 21.380942759123343, 17.416566918622852, 17.180070153709277, 25.790339917193062, 45.97625189374871, 23.898797049169467, 20.521731031400577, 15.734556979186685, 21.008389858307645, 29.800681131145613, 17.180070153709277, 19.581972547389178, 23.118311288923124, 28.996858600549206, 25.58963789966816, 22.80439145296227, 25.1437504597293, 24.039240115615065, 18.6852487375444, 23.94552000018515, 27.186876493132687, 17.656319241826594, 20.8042283062039, 22.10277408821932, 23.25416777248505, 22.016603538363892, 19.89034374461794, 28.940279382223952, 26.609013186724894, 18.539838978246124, 19.165753221675935, 20.16414949200632, 35.45837151398557, 43.699809813576366, 21.131847230940878, 17.969428099556087, 25.2915095879081, 27.133828946090556, 28.60311491479713, 20.602050815688813, 20.8042283062039, 15.520899957093135, 16.651494963610144, 28.940279382223952, 27.346642204097876, 28.547303976590324, 43.444504934395866, 42.60421214369869, 21.090614371389922, 29.053548433146837, 23.028181534732802, 23.20879380099744, 18.79505399780919, 20.8042283062039, 18.145770440989054, 19.20322291302118, 27.080884906424416, 17.416566918622852, 30.329152388418176, 20.723120306578686, 23.25416777248505, 24.948080241282657, 28.547303976590324, 20.401838105461145, 26.2990087931472, 25.39049775763493, 30.093129052017023, 33.1801937796528, 34.63697922255266, 20.085536923187668, 17.382583384208317, 35.59715165620933, 15.796140293991145, 24.039240115615065, 21.33922385967176, 20.085536923187668, 21.548635571548648, 22.938403163153815, 23.073202403172917, 35.25121511458854, 23.75917448515314, 25.790339917193062, 23.52828054362271, 19.429584428039178, 18.90550453688997, 13.643746025356307, 15.796140293991145, 28.940279382223952, 23.80562446936611, 12.350195278830133, 21.33922385967176, 29.22428378123494, 26.922671813996086, 22.626926270737744, 17.89937193856263, 25.44013703364339, 24.322607071104724, 26.043432497666576, 20.84490127137742, 17.934365811960372, 15.888967492629165, 14.438841114275835, 31.35307975333966, 12.842171128856814, 14.159568646729724, 14.382549407491597, 13.885697797673076, 14.382549407491597, 19.697047472120772, 19.581972547389178, 13.432219349369987, 25.1437504597293, 13.85860376179437, 14.523692042874359, 11.69292987543966, 15.98234019723552, 17.553166745968216, 12.742232560259533, 13.804574186067095, 16.88071548801052, 19.353835657005096, 13.458479789399725, 25.840760898054555, 25.891280453627324, 20.84490127137742, 14.382549407491597, 19.31607208722833, 18.2524054167373, 14.580536166314408, 22.80439145296227, 21.090614371389922, 17.382583384208317, 18.216790978454146, 15.340075995482293, 23.75917448515314, 20.763634702888442, 22.84897480427519, 23.16350836406023, 20.007230835729846, 13.198169226352599, 14.049378009312553, 14.926303617194517, 20.322298876557472, 16.84777751703098, 13.777638440111984, 23.390822626823073, 16.651494963610144, 20.203571081597193, 28.883810562338418, 21.973644325590612, 19.697047472120772, 18.00455893510183, 19.89034374461794, 18.79505399780919, 16.84777751703098, 14.95548504215564, 23.390822626823073, 20.723120306578686, 14.694892728788941, 16.91371785382974, 20.763634702888442, 17.725424121461643, 19.774139536218833, 22.893645317537526, 18.216790978454146, 18.758380733362646, 16.782094257242417, 18.28808948244362, 24.465540688808968, 20.68268496272443, 22.232662691545976, 19.929230034775262, 21.973644325590612, 22.40703562619394, 22.40703562619394, 29.453492679719986, 15.581647041956762, 15.796140293991145, 19.053782246709744, 13.146714440868186, 12.422772124124576, 12.76714405293921, 14.552086348683558, 13.67042000714299, 22.938403163153815, 18.721779026482544, 17.656319241826594, 18.110364070885183, 12.942893525362278, 15.734556979186685, 15.400115355370453, 15.310144149387147, 17.013112580497378, 15.04337206739713, 15.04337206739713, 15.581647041956762, 15.581647041956762, 12.692555303381242, 14.984723517782793, 12.618402319374093, 14.438841114275835, 12.398532576458457, 12.398532576458457, 12.398532576458457, 12.398532576458457, 13.172416709211404, 14.438841114275835, 14.4106677747075, 14.438841114275835, 12.326097344475713, 14.580536166314408, 14.2706235200274, 13.301683750542033, 13.301683750542033, 13.301683750542033, 14.326477161274006, 17.180070153709277, 19.543763832601815, 20.56185170491249, 12.326097344475713, 17.76007792103387, 18.28808948244362, 17.553166745968216, 18.648789727196185, 16.076261613516785, 15.400115355370453, 20.56185170491249, 15.85796463907484, 18.79505399780919, 18.79505399780919, 16.01358620918395, 13.804574186067095, 13.121062323276538, 12.495775472672934, 15.04337206739713, 15.10225016094451, 15.10225016094451, 15.85796463907484, 23.16350836406023, 20.124804822565828, 22.145985754016134, 17.45061689198871, 16.107691244988956, 16.782094257242417, 17.484733434196265, 21.887977204970085, 17.416566918622852, 16.94678474038272, 16.94678474038272, 16.94678474038272, 16.94678474038272, 16.94678474038272, 16.94678474038272, 19.735555861572347, 18.539838978246124, 18.539838978246124, 14.984723517782793, 17.213657767955564, 16.716667072215767, 16.716667072215767, 13.777638440111984, 12.867278004795136, 12.867278004795136, 11.970219716515004, 11.3553329839266, 11.157471631854794, 17.934365811960372, 14.159568646729724, 13.379852088930456, 12.593781054375055, 13.831562592310346, 12.842171128856814, 12.842171128856814, 12.111320878668954, 12.302046430466076, 12.302046430466076, 11.970219716515004, 20.64232851707703, 20.521731031400577, 18.979498589532987, 24.465540688808968, 24.948080241282657, 19.20322291302118, 18.075026786359576, 23.75917448515314, 18.075026786359576, 16.23402543319486, 22.938403163153815, 20.36202965350754, 23.75917448515314, 18.181246031735814, 20.085536923187668, 22.715485556228362, 21.380942759123343, 26.453556883236853, 24.18050850623376, 20.401838105461145, 20.124804822565828, 24.322607071104724, 17.28103011956753, 16.716667072215767, 19.165753221675935, 20.521731031400577, 22.67116267165818, 20.602050815688813, 34.84052601883783, 33.76859572409541, 25.790339917193062, 25.992616060150944, 26.401940203271806, 22.938403163153815, 24.70563239932894, 28.1047190510635, 21.131847230940878, 17.969428099556087, 25.840760898054555, 24.322607071104724, 22.319678873017494, 20.8042283062039, 13.32768898909124, 16.716667072215767, 13.885697797673076, 16.554212894695087, 18.503663694167408, 15.888967492629165, 20.44172438427552, 28.715064341305684, 13.994604675279588, 19.20322291302118, 17.382583384208317, 16.2975636122563, 14.666219776472907, 12.040563607027826, 10.688203161633492, 16.23402543319486, 15.612109685326937, 14.723621737689907, 21.008389858307645, 12.182493960703473, 14.523692042874359, 24.89940107578365, 21.049461966173517, 21.090614371389922, 20.36202965350754, 33.702705802144536, 22.450842133517945, 13.249825400173327, 24.513371642677637, 20.84490127137742, 20.085536923187668, 20.44172438427552, 16.979916273809664, 12.374340325455691, 21.590763877931302, 23.75917448515314, 27.240027749911206, 24.89940107578365, 23.073202403172917, 33.11545195869231, 26.196478674560588, 16.23402543319486, 20.4816886421048, 21.717643615002626, 22.319678873017494, 24.657426302639607, 23.25416777248505, 16.2975636122563, 16.88071548801052, 33.1801937796528, 30.866995306309306, 11.399776500625777, 19.620255961497417, 20.602050815688813, 19.31607208722833, 18.39556080667227, 18.145770440989054, 20.92648590822237, 22.016603538363892, 36.72712654703974, 22.938403163153815, 19.620255961497417, 14.984723517782793, 15.280270706875589, 20.24306974171993, 19.543763832601815, 13.85860376179437, 19.391673055829287, 14.242778409807016, 17.416566918622852, 18.110364070885183, 21.297586362924953, 19.812798643245316, 37.89297069129353, 23.345182011801924, 26.713157919740368, 15.310144149387147, 17.013112580497378, 13.537569551425428, 15.43022309819552, 27.4536739354601, 22.938403163153815, 22.36331459495464, 26.09434828302453, 26.14536361045317, 26.043432497666576, 25.1437504597293, 22.189281900071105, 31.41437607743883, 18.359667040887704, 28.380523708580828, 16.651494963610144, 14.609041604294383, 28.049880592281973, 15.460389702669161, 16.58657693697683, 11.288992376773315, 14.694892728788941, 20.68268496272443, 18.79505399780919, 17.247311047115943, 18.576084986220376, 18.110364070885183, 26.14536361045317, 18.979498589532987, 16.554212894695087, 17.6218678981495, 17.89937193856263, 23.25416777248505, 17.348666159108166, 19.31607208722833, 15.400115355370453, 20.4816886421048, 22.059646737841184, 14.95548504215564, 26.505274475545082, 19.128356641900336, 38.0412795852805, 30.329152388418176, 20.322298876557472, 28.380523708580828, 20.64232851707703, 23.073202403172917, 22.84897480427519, 18.216790978454146, 13.564035954639001, 21.33922385967176, 13.85860376179437, 17.28103011956753, 17.79479946994832, 22.232662691545976, 14.104365720113616, 14.104365720113616, 14.104365720113616, 23.71281513502692, 21.256030110048442, 15.072782365110845, 18.323843311697065, 16.139182322446413, 17.247311047115943, 18.181246031735814, 23.20879380099744, 12.182493960703473, 12.742232560259533, 12.742232560259533, 12.742232560259533, 12.742232560259533, 12.087689040557068, 13.59055410055989, 14.694892728788941, 14.495353140571094, 13.35374506875535, 16.23402543319486, 14.637602771363136, 14.382549407491597, 19.505629671380603, 25.34095533871963, 15.920030957803608, 18.181246031735814, 13.432219349369987, 19.89034374461794, 13.72392451825968, 20.68268496272443, 18.216790978454146, 13.537569551425428, 10.62576008857486, 14.104365720113616, 12.968197316969134, 14.214987631491761, 15.796140293991145, 13.804574186067095, 19.697047472120772, 17.484733434196265, 15.98234019723552, 18.359667040887704, 23.80562446936611, 27.995149135337257, 15.734556979186685, 22.84897480427519, 17.51891667538985, 23.52828054362271, 25.39049775763493, 16.94678474038272, 16.651494963610144, 26.09434828302453, 18.00455893510183, 10.278744528592322, 25.790339917193062, 14.83910053420958, 16.521912001744084, 20.723120306578686, 16.329425896569767, 10.688203161633492, 13.301683750542033, 30.987805310145493, 25.539706914745466, 16.39333746248769, 19.929230034775262, 18.431524745974645, 21.548635571548648, 25.689792910615846, 29.396022468352484, 21.464625403032095, 22.494734284035275, 34.908640417532865, 27.4536739354601, 19.46756991825522, 19.543763832601815, 15.796140293991145, 28.65903496526287, 28.827451925481085, 24.609314266596865, 18.61240185635776, 15.827022278874091, 29.800681131145613, 20.046345644076645, 16.651494963610144, 19.543763832601815, 22.538712245181248, 21.380942759123343, 33.4404292647954, 26.401940203271806, 22.98324851186175, 26.505274475545082, 29.511075247281195, 32.348333340301785, 30.329152388418176, 33.76859572409541, 17.864446345871784, 33.702705802144536, 22.10277408821932, 25.790339917193062, 46.88305220915621, 24.417803063601138, 31.230845679407448, 22.626926270737744, 15.490615283867873, 26.557093177483523, 18.145770440989054, 18.075026786359576, 18.721779026482544, 11.601934773230344, 23.345182011801924, 14.4106677747075, 17.28103011956753, 27.507346790200362, 16.521912001744084, 17.6218678981495, 17.113091407702502, 26.24769367033534, 33.245062173048, 20.282645623050005, 22.10277408821932, 27.940524471445343, 33.375179667439795, 35.45837151398557, 35.45837151398557, 23.4365524707797, 22.75989509352673, 21.131847230940878, 28.65903496526287, 24.322607071104724, 29.56877039069624, 21.973644325590612, 11.005966116567095, 23.4365524707797, 31.169907463288272, 29.9173176993003, 23.80562446936611, 18.145770440989054, 31.475792237910618, 26.24769367033534, 10.730035606033903, 25.63966650134361, 31.72265993608251, 16.202349296067883, 10.85651787033819, 37.160054292819765, 19.429584428039178, 17.553166745968216, 17.28103011956753, 23.574279122424027, 27.080884906424416, 18.431524745974645, 21.380942759123343, 37.524723159601, 31.660762082694095, 24.89940107578365, 23.482371718117374, 23.482371718117374, 28.715064341305684, 21.717643615002626, 14.984723517782793, 18.539838978246124, 25.58963789966816, 21.297586362924953, 29.110349096271467, 33.11545195869231, 17.484733434196265, 22.059646737841184, 18.39556080667227, 28.771203256660744, 19.429584428039178, 16.2975636122563, 22.715485556228362, 24.850816893992036, 28.380523708580828, 27.995149135337257, 22.016603538363892, 15.220698576992893, 36.44131378072682, 14.214987631491761, 33.702705802144536, 19.091033031037714, 16.329425896569767, 23.852165264858517, 15.460389702669161, 22.36331459495464, 22.67116267165818, 35.87634357692165, 30.806767041912167, 30.388446900124176, 30.388446900124176, 30.388446900124176, 28.715064341305684, 24.9968545761854, 25.740017318702005, 24.80232751057381, 28.715064341305684, 21.887977204970085, 19.24076585887165, 32.28521466114651, 28.771203256660744, 17.656319241826594, 9.267952672428878, 9.693779343581, 19.429584428039178, 19.658614220966218, 13.59055410055989, 22.189281900071105, 15.490615283867873, 13.406010149206963, 14.580536166314408, 18.942465433314446, 28.214717811314713, 25.1437504597293, 18.942465433314446, 20.401838105461145, 27.028044172168716, 22.059646737841184, 50.20003003478527, 17.89937193856263, 24.948080241282657, 13.885697797673076, 20.8856537535641, 17.725424121461643, 22.189281900071105, 18.359667040887704, 23.52828054362271, 21.297586362924953, 17.113091407702502, 21.090614371389922, 22.232662691545976, 37.305494633128845, 16.651494963610144, 16.04489330813532, 17.247311047115943, 24.465540688808968, 26.66103470132838, 34.97688798216538, 23.299630451611073, 18.648789727196185, 36.65546388385301, 18.942465433314446, 15.642631884188171, 24.370158584949497, 15.340075995482293, 25.1437504597293, 15.220698576992893, 12.495775472672934, 16.265763498233948, 18.721779026482544, 22.715485556228362, 16.076261613516785, 23.620367629992042, 25.94189877662824, 20.84490127137742, 20.602050815688813, 27.293282919182133, 21.297586362924953, 26.76538304079522, 13.72392451825968, 18.075026786359576, 16.651494963610144, 21.090614371389922, 11.647343462043494, 19.128356641900336, 17.14654807625046, 20.124804822565828, 15.370066359341997, 19.620255961497417, 21.214554942517648, 27.668995768810042, 16.651494963610144, 21.33922385967176, 16.2975636122563, 13.697146137460072, 20.64232851707703, 18.110364070885183, 21.049461966173517, 32.09659661306934, 22.893645317537526, 26.922671813996086, 29.453492679719986, 24.133327041205547, 25.24216031657971, 30.806767041912167, 13.406010149206963, 31.41437607743883, 25.44013703364339, 28.771203256660744, 32.411575418595035, 20.84490127137742, 38.19016894386643, 31.10908815096766, 19.128356641900336, 22.626926270737744, 22.626926270737744, 11.923552252478798, 18.145770440989054, 19.278382202442366, 17.348666159108166, 16.01358620918395, 26.196478674560588, 22.27612829392538, 20.16414949200632, 23.898797049169467, 16.684049195604416, 23.028181534732802, 21.845268970328604, 22.27612829392538, 12.717369675355013, 13.804574186067095, 17.113091407702502, 23.992334296139543, 15.520899957093135, 18.39556080667227, 15.951155153095918, 34.77254452624438, 22.715485556228362, 24.275148340672132, 13.750755251693265, 22.016603538363892, 26.09434828302453, 22.715485556228362, 32.47494113727571, 26.14536361045317, 27.83159468972004, 26.817710263716034, 27.346642204097876, 25.94189877662824, 32.411575418595035, 31.169907463288272, 33.4404292647954, 28.547303976590324, 22.40703562619394, 21.590763877931302, 25.045724266551254, 28.65903496526287, 26.09434828302453, 18.28808948244362, 20.44172438427552, 26.66103470132838, 19.165753221675935, 19.391673055829287, 18.431524745974645, 14.495353140571094, 20.24306974171993, 25.689792910615846, 30.806767041912167, 25.1437504597293, 30.034410770075546, 34.367432111764096, 32.66578255521719, 31.9714638416083, 21.590763877931302, 26.557093177483523, 35.59715165620933, 23.75917448515314, 23.666546242140512, 19.165753221675935, 26.14536361045317, 17.348666159108166, 36.15772522700737, 28.15966472087403, 31.598985005566618, 27.668995768810042, 27.668995768810042, 21.131847230940878, 27.507346790200362, 15.10225016094451, 18.181246031735814, 27.186876493132687, 21.802644068801417, 16.170734966018113, 18.503663694167408, 22.84897480427519, 20.16414949200632, 24.948080241282657, 21.008389858307645, 29.975807060355134, 31.291903031786116, 25.790339917193062, 34.70469568042362, 21.464625403032095, 17.45061689198871, 13.67042000714299, 10.751013186076355, 22.016603538363892, 16.684049195604416, 17.013112580497378, 26.713157919740368, 19.851533330124635, 23.028181534732802, 14.467069533669397, 21.506589466713315, 21.42274322042474, 27.886006392229614, 23.992334296139543, 30.21091022897123, 18.942465433314446, 19.46756991825522, 21.760102337787384, 18.2524054167373, 16.61900425204837, 16.61900425204837, 15.460389702669161, 18.359667040887704, 16.8149038152427, 26.817710263716034, 19.581972547389178, 30.507383918455513, 23.4365524707797, 17.213657767955564, 14.868111474214228, 16.36135047271935, 32.348333340301785, 18.79505399780919, 28.996858600549206, 17.725424121461643, 25.891280453627324, 13.484791569471833, 21.42274322042474, 24.70563239932894, 20.046345644076645, 15.888967492629165, 37.5980852538711, 20.16414949200632, 15.310144149387147, 22.016603538363892, 17.934365811960372, 15.490615283867873, 21.464625403032095, 23.574279122424027, 24.80232751057381, 23.299630451611073, 16.107691244988956, 15.01401915561195, 19.812798643245316, 25.539706914745466, 19.620255961497417, 14.81014620089916, 18.61240185635776, 23.898797049169467, 22.319678873017494, 19.24076585887165, 24.80232751057381, 24.657426302639607, 23.073202403172917, 19.929230034775262, 20.124804822565828, 19.24076585887165, 14.81014620089916, 12.569207830853442, 12.230174893930755, 13.777638440111984, 14.467069533669397, 16.684049195604416, 16.684049195604416, 16.684049195604416, 16.684049195604416, 17.51891667538985, 18.868615759264884, 14.187251079068192, 12.643071719773257, 15.01401915561195, 13.018953406185608, 21.464625403032095, 19.391673055829287, 13.643746025356307, 15.85796463907484, 14.75240691276831, 36.72712654703974, 18.6852487375444, 22.98324851186175, 30.866995306309306, 16.01358620918395, 14.382549407491597, 13.617124090346826, 27.133828946090556, 20.282645623050005, 14.637602771363136, 21.464625403032095, 20.007230835729846, 13.912844803302042, 14.926303617194517, 17.656319241826594, 21.33922385967176, 12.520205133467638, 12.520205133467638, 12.520205133467638, 26.66103470132838, 13.994604675279588, 14.214987631491761, 20.401838105461145, 23.299630451611073, 21.008389858307645, 14.580536166314408, 20.4816886421048, 14.580536166314408, 22.319678873017494, 12.495775472672934, 27.507346790200362, 19.851533330124635, 22.582776184718522, 25.045724266551254, 28.60311491479713, 44.91121316574036, 19.658614220966218, 15.888967492629165, 23.992334296139543, 22.450842133517945, 18.83179895971962, 21.380942759123343, 23.345182011801924, 22.189281900071105, 17.013112580497378, 15.551243837871848, 15.796140293991145, 14.104365720113616, 21.675267738479917, 18.942465433314446, 14.354485905365031, 13.146714440868186, 15.01401915561195, 13.750755251693265, 12.968197316969134, 13.275729253905588, 16.01358620918395, 17.348666159108166, 16.107691244988956, 16.107691244988956, 16.107691244988956, 12.087689040557068, 16.84777751703098, 17.314815113938415, 17.416566918622852, 16.04489330813532, 15.10225016094451, 13.643746025356307, 13.223972090529857, 25.44013703364339, 17.348666159108166, 19.774139536218833, 20.68268496272443, 26.196478674560588, 13.59055410055989, 15.920030957803608, 21.717643615002626, 18.868615759264884, 22.450842133517945, 13.750755251693265, 12.968197316969134, 13.96729813824377, 19.24076585887165, 14.438841114275835, 16.425386987895497, 15.01401915561195, 19.016604146815915, 13.59055410055989, 13.223972090529857, 13.223972090529857, 11.738694709013188, 15.310144149387147, 14.382549407491597, 14.382549407491597, 16.651494963610144, 13.643746025356307, 16.91371785382974, 15.85796463907484, 20.046345644076645, 18.539838978246124, 13.223972090529857, 14.926303617194517, 16.84777751703098, 21.049461966173517, 15.951155153095918, 20.282645623050005, 15.04337206739713, 14.4106677747075, 17.14654807625046, 15.490615283867873, 17.725424121461643, 16.8149038152427, 23.852165264858517, 12.182493960703473, 13.511154789957821, 15.951155153095918, 14.694892728788941, 9.846434155787346, 19.391673055829287, 20.44172438427552, 16.329425896569767, 18.181246031735814, 20.124804822565828, 16.23402543319486, 17.45061689198871, 18.503663694167408, 16.457499171202052, 24.561296107667808, 15.85796463907484, 20.046345644076645, 17.6218678981495, 14.382549407491597, 14.694892728788941, 15.01401915561195, 16.425386987895497, 12.398532576458457, 20.36202965350754, 15.85796463907484, 22.98324851186175, 17.934365811960372, 18.431524745974645, 17.213657767955564, 18.942465433314446, 13.831562592310346, 25.39049775763493, 23.666546242140512, 20.203571081597193, 24.561296107667808, 21.33922385967176, 23.666546242140512, 18.039758452611142, 16.265763498233948, 15.796140293991145, 15.220698576992893, 16.58657693697683, 13.804574186067095, 13.069908149118522, 14.89717913158109, 17.046373787079762, 17.046373787079762, 17.113091407702502, 15.400115355370453, 15.734556979186685, 15.734556979186685, 30.62678644311483, 17.247311047115943, 19.581972547389178, 28.940279382223952, 24.133327041205547, 23.390822626823073, 16.076261613516785, 17.76007792103387, 19.20322291302118, 15.85796463907484, 25.1437504597293, 27.507346790200362, 27.028044172168716, 27.61500750152876, 14.298523068373607, 19.581972547389178, 15.85796463907484, 17.725424121461643, 22.98324851186175, 33.245062173048, 18.503663694167408, 15.131775567308885, 14.89717913158109, 14.723621737689907, 19.46756991825522, 22.626926270737744, 15.280270706875589, 17.725424121461643, 24.322607071104724, 24.322607071104724, 16.489674134905655, 25.44013703364339, 19.851533330124635, 27.886006392229614, 27.886006392229614, 27.886006392229614, 28.827451925481085, 20.8042283062039, 21.590763877931302, 22.059646737841184, 15.581647041956762, 22.98324851186175, 18.576084986220376, 24.657426302639607, 20.8042283062039, 21.845268970328604, 17.656319241826594, 20.56185170491249, 18.323843311697065, 13.750755251693265, 17.046373787079762, 18.90550453688997, 16.489674134905655, 22.582776184718522, 9.808046546977115, 15.220698576992893, 15.340075995482293, 13.564035954639001, 24.609314266596865, 19.851533330124635, 20.401838105461145, 15.796140293991145, 20.8042283062039, 14.984723517782793, 22.450842133517945, 22.450842133517945, 27.77728915635237, 26.76538304079522, 18.79505399780919, 32.66578255521719, 12.326097344475713, 18.359667040887704, 22.016603538363892, 20.56185170491249, 23.118311288923124, 21.632974546568068, 17.829588900657228, 11.807678025521156, 12.158723248329283, 21.380942759123343, 17.046373787079762, 25.740017318702005, 13.804574186067095, 28.380523708580828, 26.609013186724894, 30.447857334592324, 20.36202965350754, 13.432219349369987, 12.667789349678872, 16.076261613516785, 20.8042283062039, 33.245062173048, 12.618402319374093, 12.544682555069812, 33.375179667439795, 18.79505399780919, 26.505274475545082, 29.74253347911242, 15.340075995482293, 26.87013978811538, 16.139182322446413, 21.297586362924953, 22.715485556228362, 21.008389858307645, 16.04489330813532, 13.643746025356307, 15.827022278874091, 14.326477161274006, 16.139182322446413, 31.53732846903916, 19.165753221675935, 12.087689040557068, 9.322416488758655, 16.554212894695087, 32.159346537604755, 22.36331459495464, 32.28521466114651, 10.963057921054194, 14.021964597512564, 12.064103313334172, 17.6218678981495, 16.39333746248769, 15.250455553989443, 16.94678474038272, 21.297586362924953, 18.83179895971962, 32.411575418595035, 36.15772522700737, 18.359667040887704, 15.581647041956762, 36.51255777326842, 19.697047472120772, 24.133327041205547, 22.36331459495464, 27.886006392229614, 25.740017318702005, 28.2698785323965, 24.70563239932894, 27.293282919182133, 29.56877039069624, 23.75917448515314, 13.458479789399725, 20.8856537535641, 16.74934871787156, 22.232662691545976, 19.697047472120772, 28.325147094540903, 26.922671813996086, 25.2915095879081, 23.118311288923124, 28.325147094540903, 28.940279382223952, 20.967397891114594, 51.591626129191596, 31.230845679407448, 25.094689498803355, 12.692555303381242, 23.20879380099744, 18.868615759264884, 15.370066359341997, 15.400115355370453, 28.491601937740562, 18.39556080667227, 21.973644325590612, 31.10908815096766, 18.648789727196185, 17.725424121461643, 22.67116267165818, 15.340075995482293, 16.170734966018113, 19.774139536218833, 15.888967492629165, 13.777638440111984, 15.01401915561195, 11.57929685891098, 20.521731031400577, 19.96819234893607, 18.503663694167408, 16.425386987895497, 16.74934871787156, 22.016603538363892, 35.66674528344124, 22.67116267165818, 26.505274475545082, 26.975306541751994, 16.684049195604416, 13.72392451825968, 19.929230034775262, 24.18050850623376, 24.465540688808968, 17.013112580497378, 15.612109685326937, 17.180070153709277, 23.898797049169467, 20.085536923187668, 23.75917448515314, 26.609013186724894, 16.58657693697683, 15.250455553989443, 18.216790978454146, 19.929230034775262, 20.24306974171993, 23.299630451611073, 26.505274475545082, 25.2915095879081, 27.83159468972004, 22.450842133517945, 13.67042000714299, 16.265763498233948, 11.877066727642635, 19.429584428039178, 14.021964597512564, 14.131940228876227, 18.503663694167408, 16.91371785382974, 24.465540688808968, 29.396022468352484, 37.160054292819765, 31.048387510438282, 20.84490127137742, 31.291903031786116, 18.868615759264884, 25.192907336481866, 23.852165264858517, 23.390822626823073, 32.72964525738904, 17.180070153709277, 16.329425896569767, 18.28808948244362, 24.80232751057381, 16.651494963610144, 22.80439145296227, 15.310144149387147, 23.028181534732802, 24.850816893992036, 16.58657693697683, 22.016603538363892, 22.189281900071105, 21.675267738479917, 9.59957426409205, 13.32768898909124, 15.520899957093135, 12.040563607027826, 11.3775330413472, 11.3775330413472, 15.250455553989443, 16.489674134905655, 15.827022278874091, 15.072782365110845, 15.888967492629165, 18.576084986220376, 16.84777751703098, 22.189281900071105, 21.33922385967176, 15.10225016094451, 15.520899957093135, 23.345182011801924, 19.053782246709744, 23.4365524707797, 15.131775567308885, 20.36202965350754, 16.651494963610144, 13.223972090529857, 19.851533330124635, 17.934365811960372, 15.520899957093135, 14.4106677747075, 18.90550453688997, 11.3553329839266, 14.076845015253456, 17.213657767955564, 18.721779026482544, 25.2915095879081, 27.080884906424416, 20.68268496272443, 20.56185170491249, 41.53608219643496, 13.994604675279588, 12.182493960703473, 13.994604675279588, 15.920030957803608, 12.064103313334172, 11.422063446614425, 14.95548504215564, 13.379852088930456, 23.574279122424027, 8.912902981198737, 25.34095533871963, 21.256030110048442, 10.77203177799245, 18.61240185635776, 17.864446345871784, 18.181246031735814, 37.37842812826559, 16.58657693697683, 24.18050850623376, 17.934365811960372, 16.04489330813532, 20.68268496272443, 20.68268496272443, 18.075026786359576, 19.697047472120772, 23.20879380099744, 18.467558995986426, 15.581647041956762, 29.396022468352484, 27.293282919182133, 38.56494680401705, 23.992334296139543, 43.02230705927991, 19.24076585887165, 19.31607208722833, 18.942465433314446, 23.898797049169467, 23.028181534732802, 22.75989509352673, 23.16350836406023, 19.581972547389178, 26.87013978811538, 21.887977204970085, 20.84490127137742, 18.539838978246124, 15.520899957093135, 21.930768935644906, 18.648789727196185, 24.465540688808968, 17.656319241826594, 29.396022468352484, 17.829588900657228, 15.131775567308885, 23.574279122424027, 25.192907336481866, 16.61900425204837, 18.979498589532987, 15.370066359341997, 14.926303617194517, 25.63966650134361, 19.31607208722833, 18.110364070885183, 32.66578255521719, 17.934365811960372, 31.660762082694095, 16.61900425204837, 22.582776184718522, 24.465540688808968, 16.651494963610144, 36.72712654703974, 21.049461966173517, 18.979498589532987, 26.09434828302453, 26.401940203271806, 32.9863470466753, 15.072782365110845, 18.075026786359576, 22.016603538363892, 22.016603538363892, 20.763634702888442, 23.898797049169467, 23.898797049169467, 22.67116267165818, 18.6852487375444, 17.348666159108166, 13.32768898909124, 14.926303617194517, 15.490615283867873, 14.83910053420958, 16.107691244988956, 20.8042283062039, 15.220698576992893, 26.505274475545082, 23.620367629992042, 22.189281900071105, 16.74934871787156, 14.694892728788941, 16.651494963610144, 15.581647041956762, 20.24306974171993, 15.340075995482293, 18.28808948244362, 17.587483776585056, 13.96729813824377, 13.96729813824377, 16.554212894695087, 15.250455553989443, 17.725424121461643, 20.44172438427552, 27.133828946090556, 25.689792910615846, 17.113091407702502, 14.666219776472907, 13.0444058974144, 12.76714405293921, 12.76714405293921, 12.76714405293921, 12.76714405293921, 12.76714405293921, 12.76714405293921, 18.61240185635776, 13.249825400173327, 14.495353140571094, 15.250455553989443, 19.165753221675935, 16.01358620918395, 17.829588900657228, 11.027483150026422, 16.457499171202052, 19.735555861572347, 19.429584428039178, 17.45061689198871, 16.39333746248769, 22.893645317537526, 24.753932740556596, 20.007230835729846, 20.007230835729846, 25.689792910615846, 18.467558995986426, 17.416566918622852, 20.4816886421048, 28.827451925481085, 16.716667072215767, 18.323843311697065, 32.92198346312245, 17.79479946994832, 31.475792237910618, 22.494734284035275, 16.8149038152427, 23.52828054362271, 23.52828054362271, 15.16135869683449, 16.170734966018113, 18.758380733362646, 15.310144149387147, 19.31607208722833, 19.278382202442366, 21.464625403032095, 25.34095533871963, 15.703855414343392, 22.40703562619394, 17.89937193856263, 23.299630451611073, 24.133327041205547, 31.10908815096766, 28.547303976590324, 23.073202403172917, 16.979916273809664, 30.388446900124176, 30.388446900124176, 20.4816886421048, 20.4816886421048, 25.740017318702005, 29.33866439394716, 18.110364070885183, 15.190999662372045, 20.723120306578686, 17.969428099556087, 24.370158584949497, 39.248839424943355, 27.028044172168716, 28.2698785323965, 31.598985005566618, 35.806340977539655, 14.984723517782793, 17.79479946994832, 18.467558995986426, 21.297586362924953, 14.382549407491597, 24.133327041205547, 24.657426302639607, 21.049461966173517, 29.9173176993003, 20.322298876557472, 34.367432111764096, 30.507383918455513, 27.133828946090556, 19.851533330124635, 33.1801937796528, 35.59715165620933, 28.996858600549206, 24.657426302639607, 22.145985754016134, 23.16350836406023, 26.2990087931472, 26.14536361045317, 30.62678644311483, 35.25121511458854, 20.203571081597193, 26.35042423874788, 29.053548433146837, 30.56702687878971, 25.790339917193062, 22.893645317537526, 15.951155153095918, 18.075026786359576, 28.491601937740562, 32.09659661306934, 36.51255777326842, 30.034410770075546, 15.280270706875589, 39.248839424943355, 16.23402543319486, 18.758380733362646, 28.436008585761353, 18.503663694167408, 24.850816893992036, 19.053782246709744, 31.53732846903916, 30.866995306309306, 28.883810562338418, 23.028181534732802, 22.893645317537526, 27.346642204097876, 33.76859572409541, 33.76859572409541, 22.27612829392538, 15.612109685326937, 30.74665629604239, 19.053782246709744, 30.151962130172098, 38.867415430562446, 19.658614220966218, 18.145770440989054, 37.67159077349981, 27.995149135337257, 47.99485175252902, 28.771203256660744, 29.858942463791767, 30.21091022897123, 21.845268970328604, 30.987805310145493, 15.888967492629165, 22.538712245181248, 19.581972547389178, 22.538712245181248, 16.554212894695087, 16.88071548801052, 20.521731031400577, 30.388446900124176, 22.145985754016134, 23.574279122424027, 24.513371642677637, 28.65903496526287, 27.346642204097876, 30.388446900124176, 30.62678644311483, 31.230845679407448, 31.230845679407448, 33.11545195869231, 26.76538304079522, 21.717643615002626, 23.852165264858517, 24.22778221261098, 31.10908815096766, 37.160054292819765, 24.086237637543274, 13.249825400173327, 19.697047472120772, 32.348333340301785, 15.340075995482293, 25.891280453627324, 31.10908815096766, 31.9714638416083, 16.716667072215767, 26.043432497666576, 27.72308958496738, 24.039240115615065, 18.323843311697065, 14.984723517782793, 15.920030957803608, 20.521731031400577, 19.929230034775262, 26.2990087931472, 19.165753221675935, 17.45061689198871, 17.690837939037834, 23.345182011801924, 20.24306974171993, 23.71281513502692, 19.24076585887165, 22.450842133517945, 30.21091022897123, 23.94552000018515, 19.24076585887165, 19.165753221675935, 17.829588900657228, 21.845268970328604, 22.450842133517945, 23.16350836406023, 20.967397891114594, 19.053782246709744, 19.774139536218833, 43.5294404008871, 28.547303976590324, 21.380942759123343, 40.41571724385996, 26.76538304079522, 26.2990087931472, 19.812798643245316, 26.505274475545082, 26.14536361045317, 27.40010580820802, 26.14536361045317, 32.159346537604755, 18.61240185635776, 18.039758452611142, 25.094689498803355, 29.858942463791767, 22.40703562619394, 15.250455553989443, 18.503663694167408, 17.348666159108166, 15.85796463907484, 19.620255961497417, 18.721779026482544, 18.83179895971962, 23.073202403172917, 16.457499171202052, 37.45150421083523, 28.214717811314713, 28.547303976590324, 20.36202965350754, 19.391673055829287, 18.6852487375444, 23.16350836406023, 18.431524745974645, 17.553166745968216, 24.89940107578365, 18.648789727196185, 21.717643615002626, 29.281418237700255, 21.380942759123343, 26.505274475545082, 18.503663694167408, 15.796140293991145, 26.196478674560588, 18.868615759264884, 17.484733434196265, 13.85860376179437, 13.301683750542033, 16.23402543319486, 22.059646737841184, 17.046373787079762, 13.643746025356307, 17.113091407702502, 22.016603538363892, 14.95548504215564, 21.380942759123343, 20.521731031400577, 15.190999662372045, 13.750755251693265, 22.938403163153815, 23.75917448515314, 27.83159468972004, 18.758380733362646, 24.89940107578365, 20.44172438427552, 22.938403163153815, 15.01401915561195, 19.053782246709744, 31.10908815096766, 17.553166745968216, 20.16414949200632, 25.2915095879081, 23.16350836406023, 32.033969127372174, 18.83179895971962, 20.68268496272443, 28.325147094540903, 25.63966650134361, 42.10785803778799, 23.299630451611073, 21.049461966173517, 31.41437607743883, 29.053548433146837, 27.83159468972004, 19.735555861572347, 36.65546388385301, 25.63966650134361, 16.61900425204837, 21.590763877931302, 17.314815113938415, 29.975807060355134, 23.94552000018515, 16.489674134905655, 11.69292987543966, 20.007230835729846, 21.973644325590612, 30.62678644311483, 27.133828946090556, 33.05083646319603, 34.03344610288473, 33.90076227087091, 17.656319241826594, 25.94189877662824, 18.467558995986426, 18.721779026482544, 14.75240691276831, 17.213657767955564, 13.146714440868186, 19.091033031037714, 24.513371642677637, 31.048387510438282, 24.370158584949497, 27.293282919182133, 15.310144149387147, 23.20879380099744, 26.557093177483523, 25.39049775763493, 19.391673055829287, 17.213657767955564, 20.36202965350754, 20.401838105461145, 21.131847230940878, 27.028044172168716, 32.033969127372174, 29.396022468352484, 19.165753221675935, 23.52828054362271, 28.1047190510635, 34.97688798216538, 17.046373787079762, 37.9670527218579, 35.59715165620933, 28.325147094540903, 25.44013703364339, 33.375179667439795, 26.557093177483523, 25.39049775763493, 31.846818916590372, 21.930768935644906, 26.557093177483523, 22.84897480427519, 27.61500750152876, 24.086237637543274, 22.36331459495464, 22.67116267165818, 17.382583384208317, 20.602050815688813, 31.291903031786116, 46.97471015107385, 16.202349296067883, 30.329152388418176, 13.617124090346826, 25.740017318702005, 26.817710263716034, 39.556671887793456, 27.507346790200362, 11.853891970655884, 18.79505399780919, 23.94552000018515, 26.196478674560588, 15.490615283867873, 18.868615759264884, 21.845268970328604, 18.721779026482544, 22.538712245181248, 31.10908815096766, 20.124804822565828, 13.831562592310346, 35.66674528344124, 11.556703116117722, 17.829588900657228, 32.79363281328884, 32.66578255521719, 26.76538304079522, 23.52828054362271, 31.53732846903916, 22.80439145296227, 28.325147094540903, 39.172256347947744, 27.133828946090556, 27.995149135337257, 33.05083646319603, 41.780172086809024, 23.574279122424027, 22.40703562619394, 22.40703562619394, 18.979498589532987, 22.67116267165818, 19.353835657005096, 19.353835657005096, 26.401940203271806, 19.20322291302118, 22.059646737841184, 27.83159468972004, 16.94678474038272, 16.36135047271935, 26.609013186724894, 25.58963789966816, 17.046373787079762, 15.920030957803608, 33.63694444585419, 28.380523708580828, 13.750755251693265, 28.049880592281973, 179.02044192165667, 43.10641711474324, 52.301824199806006, 62.597087408320306, 49.9067492938035, 34.501942437892716, 42.02569638966092, 32.79363281328884, 35.66674528344124, 44.91121316574036, 44.91121316574036, 43.5294404008871, 76.09870517958821, 41.132430556936306, 84.89424029774196, 129.44825810694724, 109.00636227269737, 48.94146538270724, 82.28231911219243, 39.095822701298815, 61.14702744030778, 56.55175575986736, 54.38529201854992, 66.89505178172818, 67.41971617548266, 47.71445419747699, 46.15619785585283, 49.809370301716946, 65.60118435636834, 60.908637772317434, 53.96206231832688, 53.22932014095188, 79.43983955226133, 71.48811770889668, 42.854578979512624, 46.066137010589394, 40.1011996370117, 52.19977189231385, 47.62135285300038, 47.62135285300038, 47.62135285300038, 79.13013296849392, 37.89297069129353, 32.66578255521719, 55.24173574865985, 84.2335880783952, 47.71445419747699, 63.08804244970428, 73.75739416067401, 73.75739416067401, 73.18340956555208, 69.83209724580306, 38.867415430562446, 38.867415430562446, 54.38529201854992, 47.250760544967605, 47.250760544967605, 53.43765380953904, 101.6051019574268, 105.65258908406659, 76.69555472159655, 50.990565478815945, 58.34690118774256, 58.57526450222688, 70.37979717263202, 35.806340977539655, 37.89297069129353, 49.32531805312982, 80.37625124290746, 54.49161709130268, 102.40200064874995, 72.89809449073893, 34.97688798216538, 38.79157659527073, 53.333385249853684, 45.086990684122, 63.83170433047907, 51.591626129191596, 54.067559962798995, 104.4216991379013, 58.119428177447986, 96.95226558425567, 61.626609520407456, 56.33128135834348, 79.43983955226133, 75.80202444429054, 53.85677052285177, 44.82358155812517, 40.022953418667065, 47.34313736903344, 44.30137101535166, 69.55984769951485, 92.51249810131188, 81.9615307508321, 53.333385249853684, 44.99901609609206, 106.48123261036584, 56.773093073800744, 91.07822475921115, 50.29817277977263, 63.3349620664254, 71.20941195067294, 45.44061233480379, 43.61454191906839, 40.65322322588273, 62.597087408320306, 31.846818916590372, 54.067559962798995, 51.49095957359226, 41.780172086809024, 75.50650035756513, 70.93179276322707, 52.9183415530491, 52.9183415530491, 74.04607243930377, 53.96206231832688, 65.8579401324037, 100.81440478091984, 91.07822475921115, 73.75739416067401, 115.58428452718766, 135.13171912803952, 73.46984133199462, 89.66618776422116, 56.55175575986736, 62.84208548133934, 47.435694792893365, 132.00139622598326, 119.2533235714264, 66.37447035867858, 60.67117749756472, 98.86447768050158, 46.70027251047952, 43.61454191906839, 63.3349620664254, 56.55175575986736, 63.582848099201826, 72.61389175398708, 32.60204446315635, 79.75075829332496, 95.44916002730179, 71.48811770889668, 36.65546388385301, 102.00277308269969, 132.00139622598326, 39.78912962559818, 81.641993024771, 66.89505178172818, 58.57526450222688, 70.37979717263202, 62.84208548133934, 36.72712654703974, 46.427438201089, 67.68358948381629, 99.2514223056366, 67.1568716116849, 56.33128135834348, 39.40245503911115, 55.02636863114642, 55.02636863114642, 38.41459632502361, 49.51837188956857, 79.13013296849392, 67.41971617548266, 82.28231911219243, 89.66618776422116, 92.87458178129451, 59.964336869699544, 47.06654728734682, 41.45503620813183, 41.45503620813183, 41.45503620813183, 59.265731184583274, 44.64883097460398, 93.6030061542134, 56.11166650505244, 60.908637772317434, 71.48811770889668, 42.77096031537661, 28.827451925481085, 28.827451925481085, 91.07822475921115, 47.15856396830611, 55.6750020519895, 42.190180314697045, 42.190180314697045, 42.52108200006278, 59.03467599561125, 55.89290784893326, 50.79177210350426, 62.597087408320306, 52.301824199806006, 43.87084603329354, 76.39654709045266, 73.18340956555208, 66.37447035867858, 69.83209724580306, 66.37447035867858, 59.964336869699544, 70.37979717263202, 102.40200064874995, 111.59137365930344, 60.434642992688794, 37.37842812826559, 55.02636863114642, 53.646802873050504, 47.06654728734682, 40.41571724385996, 50.20003003478527, 81.9615307508321, 54.067559962798995, 57.442316329639176, 64.58413226217957, 58.80452160545304, 55.24173574865985, 95.0770390179677, 63.582848099201826, 57.442316329639176, 38.19016894386643, 45.97625189374871, 79.13013296849392, 71.20941195067294, 25.992616060150944, 75.21212841006654, 71.76791429061606, 63.582848099201826, 67.68358948381629, 86.23113005463887, 41.943695056893915, 41.943695056893915, 45.97625189374871, 61.626609520407456, 48.94146538270724, 83.25223705241488, 60.908637772317434, 52.609179777419676, 74.91890411002962, 60.67117749756472, 47.15856396830611, 32.9863470466753, 89.66618776422116, 77.59961765106212, 66.63425269055388, 69.55984769951485, 64.33234259239626, 55.457945789238345, 50.29817277977263, 52.19977189231385, 53.85677052285177, 62.3530444922685, 63.3349620664254, 63.3349620664254, 63.3349620664254, 35.806340977539655, 43.78524440968234, 56.33128135834348, 37.160054292819765, 42.77096031537661, 43.275130861102994, 58.34690118774256, 68.21443845541918, 68.48142221881122, 62.10995300937982, 48.75066069158903, 73.18340956555208, 121.13129015902297, 76.09870517958821, 48.182698291098816, 53.85677052285177, 66.89505178172818, 74.04607243930377, 62.597087408320306, 48.182698291098816, 48.37128004023317, 57.21836996148928, 86.56862915519338, 47.71445419747699, 69.28865955361954, 64.08153455750669, 40.57389988868238, 66.63425269055388, 81.9615307508321, 34.03344610288473, 64.83690740886694, 56.9952966774873, 55.6750020519895, 88.27604237373367, 98.86447768050158, 33.31005738633129, 115.13366410675533, 56.33128135834348, 119.2533235714264, 98.86447768050158, 73.46984133199462, 49.037147591523336, 31.53732846903916, 41.2128458403547, 36.15772522700737, 57.66713919909174, 60.199030648454944, 40.33685733281881, 80.06289393593435, 63.08804244970428, 147.25819873544805, 26.35042423874788, 31.41437607743883, 40.81233544283598, 58.34690118774256, 80.06289393593435, 85.2265069597304, 59.73055807527518, 59.73055807527518, 117.86398026897247, 39.634006510082365, 54.38529201854992, 50.89107172411317, 73.46984133199462, 89.66618776422116, 45.97625189374871, 53.75168417471723, 36.798929313140206, 33.245062173048, 47.435694792893365, 42.52108200006278, 37.9670527218579, 77.29708541258049, 34.16664924473404, 71.48811770889668, 83.90519244027122, 50.89107172411317, 71.20941195067294, 76.99573263552006, 68.7494509271083, 43.699809813576366, 101.20898120494374, 35.25121511458854, 40.81233544283598, 117.86398026897247, 82.28231911219243, 126.94450214932196, 46.51820540231633, 31.53732846903916, 50.990565478815945, 50.990565478815945, 62.3530444922685, 69.83209724580306, 83.25223705241488, 50.1020787877761, 63.582848099201826, 55.02636863114642, 48.75066069158903, 61.38635013907805, 51.692489491288626, 45.7076474882609, 61.14702744030778, 40.57389988868238, 59.73055807527518, 51.09025374715126, 61.867809250367884, 62.84208548133934, 52.50652775191775, 67.41971617548266, 62.10995300937982, 59.73055807527518, 50.990565478815945, 47.52843316962603, 42.77096031537661, 57.89284200037607, 31.90908051733889, 77.29708541258049, 60.199030648454944, 43.95661501095434, 80.37625124290746, 91.07822475921115, 54.811841150452466, 81.9615307508321, 46.88305220915621, 79.13013296849392, 74.33588057277046, 75.21212841006654, 68.48142221881122, 95.82273747708687, 36.87087245606033, 55.02636863114642, 93.2380826167318, 66.37447035867858, 76.39654709045266, 55.6750020519895, 64.33234259239626, 69.55984769951485, 37.8190332109868, 72.89809449073893, 65.34542957624015, 43.87084603329354, 69.55984769951485, 82.60436300370014, 95.0770390179677, 77.59961765106212, 85.2265069597304, 74.91890411002962, 56.11166650505244, 79.13013296849392, 68.7494509271083, 113.7923162908356, 113.7923162908356, 59.265731184583274, 69.28865955361954, 125.9566121960733, 125.9566121960733, 88.62154503481175, 57.66713919909174, 46.70027251047952, 91.07822475921115, 81.9615307508321, 107.7363996853844, 87.93188669987823, 51.290215346585704, 66.37447035867858, 77.29708541258049, 55.02636863114642, 79.43983955226133, 90.01713130052181, 71.48811770889668, 63.3349620664254, 63.3349620664254, 72.61389175398708, 58.57526450222688, 72.33079701870139, 106.89798837141365, 51.290215346585704, 79.13013296849392, 83.90519244027122, 88.96839995507139, 35.38918446958933, 41.53608219643496, 36.228414794400415, 76.09870517958821, 90.01713130052181, 92.51249810131188, 59.265731184583274, 98.4790416094955, 44.82358155812517, 38.94340253332466, 56.773093073800744, 57.21836996148928, 103.60908302823987, 33.63694444585419, 87.24759532868651, 67.41971617548266, 74.91890411002962, 99.63988138921447, 61.38635013907805, 61.38635013907805, 64.33234259239626, 67.68358948381629, 38.41459632502361, 31.10908815096766, 70.93179276322707, 98.86447768050158, 76.69555472159655, 62.3530444922685, 65.34542957624015, 68.48142221881122, 65.34542957624015, 35.66674528344124, 59.73055807527518, 46.33684810653952, 42.3553080116137, 84.89424029774196, 88.62154503481175, 66.63425269055388, 47.90120329104557, 33.96703939987107, 81.641993024771, 59.03467599561125, 66.63425269055388, 58.57526450222688, 48.08868329990042, 47.15856396830611, 63.3349620664254, 68.21443845541918, 49.32531805312982, 46.88305220915621, 40.022953418667065, 61.38635013907805, 59.03467599561125, 62.10995300937982, 81.00664999449546, 59.49769069799599, 75.50650035756513, 61.867809250367884, 39.479488162280596, 67.41971617548266, 42.02569638966092, 51.894808174781225, 44.30137101535166, 49.32531805312982, 24.80232751057381, 54.38529201854992, 55.24173574865985, 51.49095957359226, 51.09025374715126, 67.68358948381629, 63.83170433047907, 56.9952966774873, 76.99573263552006, 48.08868329990042, 38.19016894386643, 50.5937537501147, 59.73055807527518, 25.34095533871963, 41.698649825700315, 40.73270164287187, 56.55175575986736, 58.57526450222688, 52.301824199806006, 64.08153455750669, 87.58907276183992, 51.79355004464666, 65.34542957624015, 59.03467599561125, 61.626609520407456, 54.811841150452466, 31.41437607743883, 47.34313736903344, 59.49769069799599, 59.964336869699544, 58.119428177447986, 40.41571724385996, 62.597087408320306, 74.62682298320091, 55.02636863114642, 41.45503620813183, 76.99573263552006, 77.90333396724647, 79.75075829332496, 73.75739416067401, 79.43983955226133, 64.58413226217957, 66.37447035867858, 33.57131140436464, 46.066137010589394, 67.9484955630781, 74.91890411002962, 44.73612093895834, 53.02179868836547, 55.89290784893326, 47.06654728734682, 58.119428177447986, 36.299242562272255, 89.66618776422116, 29.281418237700255, 89.66618776422116, 68.21443845541918, 49.809370301716946, 32.60204446315635, 51.79355004464666, 33.57131140436464, 76.99573263552006, 80.37625124290746, 236.23893989057657, 219.3404304920078, 56.11166650505244, 69.28865955361954, 58.34690118774256, 44.73612093895834, 26.713157919740368, 35.25121511458854, 68.21443845541918, 55.89290784893326, 73.18340956555208, 45.7076474882609, 26.76538304079522, 70.10541234668786, 58.57526450222688, 51.591626129191596, 60.199030648454944, 52.097918711212095, 56.55175575986736, 93.96935796203766, 57.66713919909174, 45.44061233480379, 59.265731184583274, 91.43469485205186, 128.94358717585138, 53.75168417471723, 105.24068867470233, 106.89798837141365, 103.60908302823987, 111.15632012308374, 115.58428452718766, 96.95226558425567, 32.85774546700983, 23.574279122424027, 134.60489048229144, 134.60489048229144, 44.82358155812517, 92.1518260518184, 89.66618776422116, 90.36944839098756, 79.43983955226133, 26.76538304079522, 38.41459632502361, 92.1518260518184, 28.883810562338418, 46.7915731119465, 46.7915731119465, 51.49095957359226, 51.49095957359226, 51.49095957359226, 45.351947753528236, 79.75075829332496, 103.20514949397284, 54.173263858709824, 87.58907276183992, 87.24759532868651, 29.858942463791767, 27.995149135337257, 38.64034257045414, 32.47494113727571, 24.657426302639607, 50.396507397123145, 65.09067188950593, 56.11166650505244, 47.99485175252902, 80.69083499570341, 97.33172602285453, 37.8190332109868, 49.42175070681059, 43.275130861102994, 56.773093073800744, 36.37020880080945, 50.990565478815945, 50.6926662381911, 36.37020880080945, 81.32370105824417, 73.46984133199462, 87.93188669987823, 60.199030648454944, 55.6750020519895, 47.99485175252902, 56.773093073800744, 41.45503620813183, 36.15772522700737, 76.69555472159655, 44.47476167904283, 44.82358155812517, 33.245062173048, 41.53608219643496, 54.38529201854992, 109.43300111180854, 58.34690118774256, 34.63697922255266, 64.83690740886694, 74.33588057277046, 55.02636863114642, 48.56059987746486, 53.333385249853684, 40.57389988868238, 30.26997357328364, 42.3553080116137, 43.61454191906839, 40.49473132867618, 46.427438201089, 34.367432111764096, 40.892124929554, 60.199030648454944, 67.1568716116849, 38.115651564714454, 42.93836112094585, 66.11570082213345, 74.04607243930377, 59.964336869699544, 93.6030061542134, 84.563269020706, 74.62682298320091, 74.62682298320091, 45.7076474882609, 76.39654709045266, 104.83039411284926, 108.15806803146498, 81.641993024771, 64.83690740886694, 53.43765380953904, 57.442316329639176, 66.11570082213345, 58.119428177447986, 93.6030061542134, 93.6030061542134, 74.62682298320091, 58.34690118774256, 52.50652775191775, 47.90120329104557, 50.29817277977263, 48.65553748142455, 62.10995300937982, 55.02636863114642, 43.444504934395866, 48.08868329990042, 51.894808174781225, 51.79355004464666, 76.69555472159655, 64.58413226217957, 69.28865955361954, 60.908637772317434, 55.6750020519895, 37.67159077349981, 31.048387510438282, 74.62682298320091, 71.48811770889668, 53.96206231832688, 96.95226558425567, 84.2335880783952, 28.380523708580828, 50.990565478815945, 37.160054292819765, 58.119428177447986, 51.49095957359226, 39.32557222442699, 53.646802873050504, 56.33128135834348, 100.81440478091984, 50.1020787877761, 58.119428177447986, 61.38635013907805, 31.660762082694095, 57.442316329639176, 45.97625189374871, 49.9067492938035, 52.19977189231385, 49.9067492938035, 42.77096031537661, 43.95661501095434, 28.996858600549206, 52.097918711212095, 62.10995300937982, 50.1020787877761, 61.867809250367884, 68.7494509271083, 45.79700747446617, 54.067559962798995, 51.1901369093998, 69.83209724580306, 84.563269020706, 74.33588057277046, 102.40200064874995, 60.199030648454944, 47.34313736903344, 50.79177210350426, 77.59961765106212, 82.60436300370014, 66.11570082213345, 115.58428452718766, 69.01852867010916, 57.21836996148928, 88.27604237373367, 61.867809250367884, 76.99573263552006, 104.4216991379013, 134.60489048229144, 72.33079701870139, 54.811841150452466, 61.626609520407456, 66.11570082213345, 60.908637772317434, 103.20514949397284, 86.90744918987926, 105.65258908406659, 99.63988138921447, 138.8777083722429, 144.97517197488295, 67.1568716116849, 28.325147094540903, 30.034410770075546, 36.798929313140206, 46.70027251047952, 31.9714638416083, 32.66578255521719, 51.09025374715126, 77.90333396724647, 83.90519244027122, 78.82163381626928, 74.04607243930377, 29.74253347911242, 36.37020880080945, 55.457945789238345, 79.75075829332496, 66.37447035867858, 91.07822475921115, 64.08153455750669, 42.93836112094585, 37.160054292819765, 32.92198346312245, 90.36944839098756, 43.95661501095434, 59.49769069799599, 65.8579401324037, 88.62154503481175, 58.57526450222688, 48.75066069158903, 59.964336869699544, 76.99573263552006, 67.9484955630781, 46.427438201089, 102.00277308269969, 68.7494509271083, 46.60915005647103, 41.132430556936306, 34.908640417532865, 41.943695056893915, 54.811841150452466, 53.85677052285177, 79.13013296849392, 38.26483200699776, 32.72964525738904, 76.39654709045266, 49.51837188956857, 51.894808174781225, 38.71588573814723, 55.89290784893326, 78.82163381626928, 67.41971617548266, 28.15966472087403, 56.33128135834348, 38.71588573814723, 46.7915731119465, 53.75168417471723, 93.6030061542134, 50.89107172411317, 85.2265069597304, 40.892124929554, 52.301824199806006, 44.5617113320772, 31.598985005566618, 36.58394105020859, 81.32370105824417, 32.85774546700983, 32.85774546700983, 44.64883097460398, 62.3530444922685, 28.325147094540903, 33.4404292647954, 24.70563239932894, 44.30137101535166, 42.93836112094585, 76.09870517958821, 67.9484955630781, 84.563269020706, 52.301824199806006, 60.908637772317434, 59.49769069799599, 172.1622764913071, 67.41971617548266, 69.83209724580306, 73.75739416067401, 98.09510821132314, 27.028044172168716, 62.84208548133934, 59.964336869699544, 94.70636877096143, 60.67117749756472, 47.807737558209425, 58.119428177447986, 40.33685733281881, 48.56059987746486, 41.943695056893915, 44.82358155812517, 51.79355004464666, 48.94146538270724, 69.83209724580306, 54.49161709130268, 101.6051019574268, 37.305494633128845, 38.64034257045414, 60.434642992688794, 36.58394105020859, 42.93836112094585, 56.55175575986736, 38.94340253332466, 56.33128135834348, 55.02636863114642, 58.57526450222688, 56.11166650505244, 40.022953418667065, 34.70469568042362, 32.92198346312245, 40.73270164287187, 33.63694444585419, 30.034410770075546, 46.60915005647103, 50.6926662381911, 116.94675549762589, 50.004318665090636, 42.190180314697045, 81.9615307508321, 146.11222634120347, 90.01713130052181, 43.02230705927991, 67.68358948381629, 38.115651564714454, 31.90908051733889, 38.94340253332466, 36.228414794400415, 46.33684810653952, 25.689792910615846, 32.79363281328884, 38.19016894386643, 39.095822701298815, 45.529450258314064, 47.71445419747699, 58.80452160545304, 41.780172086809024, 56.9952966774873, 41.45503620813183, 42.93836112094585, 75.50650035756513, 75.50650035756513, 37.8190332109868, 40.17959882934157, 39.94485987582193, 66.37447035867858, 80.06289393593435, 52.9183415530491, 36.58394105020859, 35.18243227767251, 30.151962130172098, 39.86691871057298, 32.92198346312245, 59.03467599561125, 35.32013242426123, 35.806340977539655, 27.4536739354601, 95.0770390179677, 46.60915005647103, 34.367432111764096, 27.77728915635237, 29.800681131145613, 42.77096031537661, 65.34542957624015, 43.78524440968234, 31.660762082694095, 37.45150421083523, 54.598150033144236, 47.06654728734682, 47.52843316962603, 59.265731184583274, 38.71588573814723, 44.64883097460398, 51.591626129191596, 36.65546388385301, 45.351947753528236, 50.004318665090636, 55.89290784893326, 66.63425269055388, 40.81233544283598, 37.45150421083523, 37.45150421083523, 36.51255777326842, 30.26997357328364, 46.70027251047952, 39.172256347947744, 34.300373728940066, 42.10785803778799, 33.96703939987107, 30.507383918455513, 46.246434773093995, 24.70563239932894, 23.345182011801924, 47.62135285300038, 38.94340253332466, 38.94340253332466, 81.641993024771, 54.27917440928786, 62.3530444922685, 46.246434773093995, 74.91890411002962, 58.119428177447986, 74.62682298320091, 74.91890411002962, 67.41971617548266, 53.333385249853684, 52.40407602298735, 35.38918446958933, 53.43765380953904, 59.49769069799599, 45.97625189374871, 45.97625189374871, 65.8579401324037, 49.22907356066477, 63.582848099201826, 93.6030061542134, 99.63988138921447, 46.70027251047952, 56.55175575986736, 25.34095533871963, 53.02179868836547, 31.35307975333966, 37.67159077349981, 40.65322322588273, 42.10785803778799, 54.38529201854992, 33.90076227087091, 27.83159468972004, 36.72712654703974, 53.646802873050504, 43.87084603329354, 71.20941195067294, 85.89494673838159, 44.21492934326177, 83.57807709540792, 47.71445419747699, 74.33588057277046, 48.182698291098816, 45.7076474882609, 43.19069160819034, 42.52108200006278, 38.64034257045414, 33.4404292647954, 47.807737558209425, 37.9670527218579, 31.598985005566618, 33.31005738633129, 21.090614371389922, 30.093129052017023, 30.093129052017023, 46.246434773093995, 31.35307975333966, 26.76538304079522, 59.49769069799599, 59.49769069799599, 74.33588057277046, 43.87084603329354, 33.57131140436464, 46.7915731119465, 66.37447035867858, 89.31661242710538, 38.48969815122384, 32.28521466114651, 43.444504934395866, 45.61846186294896, 42.10785803778799, 38.71588573814723, 68.7494509271083, 61.14702744030778, 104.4216991379013, 59.964336869699544, 68.48142221881122, 62.84208548133934, 81.641993024771, 40.57389988868238, 45.79700747446617, 42.27266353442275, 75.80202444429054, 58.119428177447986, 43.275130861102994, 41.132430556936306, 59.964336869699544, 46.88305220915621, 42.68750480955783, 48.182698291098816, 74.33588057277046, 63.83170433047907, 55.89290784893326, 64.83690740886694, 62.597087408320306, 121.13129015902297, 88.62154503481175, 33.702705802144536, 46.7915731119465, 44.99901609609206, 66.11570082213345, 36.87087245606033, 39.32557222442699, 175.55787313507827, 65.09067188950593, 70.93179276322707, 59.03467599561125, 57.21836996148928, 34.77254452624438, 40.1011996370117, 41.05217218133816, 58.57526450222688, 42.52108200006278, 42.52108200006278, 44.91121316574036, 33.375179667439795, 32.09659661306934, 35.25121511458854, 70.65525591042105, 75.80202444429054, 60.434642992688794, 59.964336869699544, 57.89284200037607, 66.11570082213345, 41.698649825700315, 100.02986085865793, 40.33685733281881, 40.97207040739876, 103.60908302823987, 87.58907276183992, 62.3530444922685, 62.84208548133934, 92.51249810131188, 36.65546388385301, 56.9952966774873, 56.55175575986736, 45.97625189374871, 41.780172086809024, 65.60118435636834, 44.99901609609206, 43.95661501095434, 51.591626129191596, 35.59715165620933, 33.57131140436464, 26.09434828302453, 60.434642992688794, 46.33684810653952, 44.73612093895834, 49.809370301716946, 44.73612093895834, 45.44061233480379, 55.89290784893326, 52.9183415530491, 29.9173176993003, 36.87087245606033, 81.00664999449546, 39.32557222442699, 87.24759532868651, 74.33588057277046, 68.48142221881122, 51.79355004464666, 90.7231444115574, 35.25121511458854, 76.99573263552006, 105.65258908406659, 74.33588057277046, 41.2128458403547, 43.275130861102994, 73.18340956555208, 24.850816893992036, 65.34542957624015, 49.71218131735948, 60.199030648454944, 66.89505178172818, 44.47476167904283, 53.75168417471723, 81.00664999449546, 71.76791429061606, 46.60915005647103, 59.265731184583274, 51.49095957359226, 39.78912962559818, 75.50650035756513, 87.93188669987823, 54.598150033144236, 51.591626129191596, 73.75739416067401, 73.75739416067401, 28.940279382223952, 20.8856537535641, 139.42126035437516, 96.57428452170761, 82.60436300370014, 49.809370301716946, 85.56007407666469, 57.89284200037607, 56.11166650505244, 46.33684810653952, 44.99901609609206, 35.32013242426123, 85.56007407666469, 38.19016894386643, 59.49769069799599, 67.41971617548266, 43.3597351955915, 130.9741532108186, 35.7364749685521, 55.24173574865985, 31.10908815096766, 63.83170433047907, 59.49769069799599, 57.66713919909174, 107.31637526839133, 90.36944839098756, 66.89505178172818, 20.602050815688813, 95.0770390179677, 39.32557222442699, 54.38529201854992, 37.305494633128845, 47.71445419747699, 52.097918711212095, 36.798929313140206, 36.942956250241515, 44.47476167904283, 65.60118435636834, 46.33684810653952, 113.34868209247811, 66.37447035867858, 61.626609520407456, 62.597087408320306, 96.95226558425567, 40.73270164287187, 93.96935796203766, 85.2265069597304, 80.37625124290746, 33.50580642730562, 33.11545195869231, 33.83461446305701, 100.81440478091984, 43.61454191906839, 63.582848099201826, 62.3530444922685, 53.85677052285177, 46.97471015107385, 69.01852867010916, 83.57807709540792, 44.64883097460398, 31.90908051733889, 56.11166650505244, 49.71218131735948, 37.67159077349981, 77.90333396724647, 47.06654728734682, 74.33588057277046, 92.87458178129451, 38.56494680401705, 36.15772522700737, 88.62154503481175, 56.55175575986736, 79.75075829332496, 28.827451925481085, 55.457945789238345, 33.11545195869231, 52.40407602298735, 49.809370301716946, 40.65322322588273, 62.84208548133934, 106.48123261036584, 51.79355004464666, 52.81508628524973, 43.10641711474324, 102.00277308269969, 66.63425269055388, 73.18340956555208, 43.02230705927991, 33.702705802144536, 171.49107938408403, 58.119428177447986, 81.9615307508321, 41.2128458403547, 50.89107172411317, 47.06654728734682, 76.09870517958821, 30.26997357328364, 48.56059987746486, 52.81508628524973, 54.067559962798995, 70.10541234668786, 56.9952966774873, 29.453492679719986, 33.96703939987107, 39.78912962559818, 45.61846186294896, 37.89297069129353, 57.21836996148928, 39.248839424943355, 41.943695056893915, 50.990565478815945, 46.33684810653952, 42.77096031537661, 47.06654728734682, 63.08804244970428, 28.325147094540903, 28.325147094540903, 28.325147094540903, 45.17513726542669, 33.83461446305701, 44.30137101535166, 80.37625124290746, 65.09067188950593, 49.51837188956857, 42.190180314697045, 39.7114923241556, 37.89297069129353, 62.84208548133934, 51.39048944047805, 42.854578979512624, 57.442316329639176, 53.12545808585662, 60.434642992688794, 49.22907356066477, 56.773093073800744, 57.66713919909174, 81.32370105824417, 59.03467599561125, 53.75168417471723, 48.27689708476307, 60.908637772317434, 45.263456176258764, 148.99403178365867, 68.21443845541918, 38.79157659527073, 69.83209724580306, 134.60489048229144, 87.58907276183992, 53.333385249853684, 65.60118435636834, 85.56007407666469, 116.03666862607915, 86.56862915519338, 76.69555472159655, 71.48811770889668, 48.75066069158903, 56.9952966774873, 96.57428452170761, 55.02636863114642, 51.1901369093998, 51.49095957359226, 92.1518260518184, 50.5937537501147, 50.20003003478527, 87.24759532868651, 83.90519244027122, 76.69555472159655, 107.31637526839133, 64.33234259239626, 75.21212841006654, 82.92766733936132, 55.457945789238345, 55.457945789238345, 59.03467599561125, 55.02636863114642, 65.34542957624015, 72.04880596519355, 130.9741532108186, 54.598150033144236, 83.25223705241488, 66.63425269055388, 56.773093073800744, 56.55175575986736, 75.80202444429054, 41.2128458403547, 59.03467599561125, 27.561124577174567, 25.34095533871963, 46.70027251047952, 67.1568716116849, 62.3530444922685, 60.908637772317434, 47.807737558209425, 46.427438201089, 45.88654216244616, 102.80279074731456, 41.617286632367644, 43.61454191906839, 45.88654216244616, 32.72964525738904, 39.40245503911115, 81.00664999449546, 98.09510821132314, 31.230845679407448, 49.51837188956857, 43.78524440968234, 71.48811770889668, 69.28865955361954, 62.10995300937982, 71.20941195067294, 72.61389175398708, 79.13013296849392, 25.891280453627324, 78.51433738825796, 67.41971617548266, 90.7231444115574, 65.60118435636834, 85.56007407666469, 79.13013296849392, 71.48811770889668, 95.0770390179677, 71.20941195067294, 78.82163381626928, 38.71588573814723, 73.18340956555208, 80.69083499570341, 93.96935796203766, 73.46984133199462, 73.46984133199462, 43.699809813576366, 37.08754689283638, 46.33684810653952, 84.563269020706, 32.033969127372174, 57.66713919909174, 72.61389175398708, 42.68750480955783, 48.27689708476307, 43.3597351955915, 40.57389988868238, 67.1568716116849, 58.80452160545304, 50.29817277977263, 69.55984769951485, 65.09067188950593, 97.71267162761835, 85.56007407666469, 37.74523999888851, 39.78912962559818, 34.09998263323346, 47.435694792893365, 38.79157659527073, 38.56494680401705, 32.47494113727571, 96.95226558425567, 91.79256012938836, 69.28865955361954, 55.24173574865985, 128.94358717585138, 76.39654709045266, 52.9183415530491, 83.25223705241488, 88.62154503481175, 88.62154503481175, 52.9183415530491, 102.00277308269969, 68.48142221881122, 68.21443845541918, 62.84208548133934, 93.6030061542134, 105.24068867470233, 51.39048944047805, 45.88654216244616, 38.56494680401705, 54.49161709130268, 65.09067188950593, 43.95661501095434, 39.095822701298815, 35.806340977539655, 59.265731184583274, 41.05217218133816, 102.40200064874995, 95.44916002730179, 49.22907356066477, 38.94340253332466, 45.17513726542669, 55.24173574865985, 61.626609520407456, 50.6926662381911, 47.71445419747699, 50.990565478815945, 59.265731184583274, 54.38529201854992, 32.85774546700983, 61.867809250367884, 53.646802873050504, 34.4346215959791, 37.8190332109868, 81.32370105824417, 74.91890411002962, 66.37447035867858, 76.69555472159655, 33.375179667439795, 56.55175575986736, 24.465540688808968, 55.24173574865985, 34.77254452624438, 30.56702687878971, 48.37128004023317, 36.72712654703974, 24.80232751057381, 36.15772522700737, 52.609179777419676, 75.50650035756513, 67.68358948381629, 36.299242562272255, 40.33685733281881, 37.74523999888851, 31.53732846903916, 46.51820540231633, 62.84208548133934, 72.33079701870139, 43.78524440968234, 40.81233544283598, 64.33234259239626, 37.89297069129353, 32.033969127372174, 34.501942437892716, 75.80202444429054, 65.34542957624015, 65.34542957624015, 88.62154503481175, 45.086990684122, 50.20003003478527, 56.55175575986736, 82.60436300370014, 72.89809449073893, 32.53843073806492, 67.1568716116849, 84.563269020706, 36.51255777326842, 47.06654728734682, 74.33588057277046, 47.90120329104557, 39.172256347947744, 41.132430556936306, 44.30137101535166, 45.88654216244616, 61.626609520407456, 36.65546388385301, 45.61846186294896, 47.34313736903344, 50.396507397123145, 56.33128135834348, 70.65525591042105, 54.067559962798995, 74.33588057277046, 70.37979717263202, 62.84208548133934, 81.9615307508321, 73.75739416067401, 66.11570082213345, 115.58428452718766, 62.3530444922685, 49.22907356066477, 46.60915005647103, 75.50650035756513, 69.55984769951485, 95.0770390179677, 61.14702744030778, 28.883810562338418, 36.72712654703974, 96.95226558425567, 96.95226558425567, 42.190180314697045, 38.0412795852805, 43.444504934395866, 34.97688798216538, 37.37842812826559, 79.13013296849392, 55.24173574865985, 52.097918711212095, 35.7364749685521, 34.300373728940066, 38.33964103892552, 29.511075247281195, 59.964336869699544, 37.67159077349981, 35.87634357692165, 43.275130861102994, 52.097918711212095, 34.300373728940066, 63.582848099201826, 45.88654216244616, 29.453492679719986, 34.367432111764096, 33.76859572409541, 50.6926662381911, 50.89107172411317, 54.38529201854992, 67.9484955630781, 74.33588057277046, 25.58963789966816, 59.49769069799599, 34.84052601883783, 41.780172086809024, 65.09067188950593, 59.49769069799599, 46.51820540231633, 56.55175575986736, 70.37979717263202, 53.43765380953904, 43.19069160819034, 88.62154503481175, 106.48123261036584, 44.5617113320772, 53.333385249853684, 62.10995300937982, 88.62154503481175, 60.434642992688794, 64.58413226217957, 59.03467599561125, 49.51837188956857, 51.894808174781225, 51.894808174781225, 48.65553748142455, 39.78912962559818, 161.73149145963927, 55.89290784893326, 41.45503620813183, 51.290215346585704, 57.442316329639176, 54.811841150452466, 37.08754689283638, 39.172256347947744, 68.48142221881122, 46.60915005647103, 45.88654216244616, 31.784678801853513, 43.5294404008871, 50.004318665090636, 51.79355004464666, 41.780172086809024, 80.37625124290746, 41.943695056893915, 98.4790416094955, 98.4790416094955, 66.11570082213345, 70.37979717263202, 59.73055807527518, 51.79355004464666, 31.72265993608251, 42.68750480955783, 41.2128458403547, 42.68750480955783, 28.491601937740562, 40.892124929554, 56.11166650505244, 34.84052601883783, 43.10641711474324, 56.55175575986736, 78.51433738825796, 73.18340956555208, 74.91890411002962, 29.858942463791767, 74.62682298320091, 83.57807709540792, 56.9952966774873, 48.08868329990042, 100.42136666458902, 91.07822475921115, 47.34313736903344, 98.86447768050158, 78.2082389954826, 34.84052601883783, 126.44959243764188, 72.04880596519355, 52.9183415530491, 63.08804244970428, 50.004318665090636, 73.46984133199462, 76.09870517958821, 75.21212841006654, 162.3644906197382, 41.45503620813183, 71.20941195067294, 68.48142221881122, 36.942956250241515, 35.45837151398557, 54.173263858709824, 42.68750480955783, 43.699809813576366, 91.43469485205186, 37.45150421083523, 82.92766733936132, 62.84208548133934, 100.81440478091984, 33.90076227087091, 53.646802873050504, 52.301824199806006, 42.60421214369869, 33.245062173048, 87.93188669987823, 71.48811770889668, 53.75168417471723, 27.886006392229614, 44.387981683814154, 38.79157659527073, 52.301824199806006, 55.6750020519895, 71.48811770889668, 31.9714638416083, 46.70027251047952, 60.67117749756472, 39.86691871057298, 58.80452160545304, 109.86130976812342, 42.93836112094585, 43.444504934395866, 38.48969815122384, 62.597087408320306, 40.81233544283598, 74.33588057277046, 49.32531805312982, 46.066137010589394, 68.48142221881122, 54.811841150452466, 55.6750020519895, 43.19069160819034, 67.9484955630781, 76.39654709045266, 218.48530319325408, 78.2082389954826, 90.7231444115574, 93.6030061542134, 98.4790416094955, 112.02812994692015, 36.37020880080945, 32.92198346312245, 44.47476167904283, 45.351947753528236, 67.41971617548266, 32.92198346312245, 47.807737558209425, 32.033969127372174, 39.32557222442699, 38.115651564714454, 74.91890411002962, 60.67117749756472, 46.246434773093995, 33.50580642730562, 39.32557222442699, 92.51249810131188, 70.10541234668786, 87.93188669987823, 103.60908302823987, 164.27839185643572, 77.29708541258049, 54.38529201854992, 31.846818916590372, 33.96703939987107, 79.75075829332496, 82.92766733936132, 75.50650035756513, 112.46659565031442, 61.38635013907805, 81.9615307508321, 106.06610162605153, 49.133016862271724, 45.17513726542669, 84.563269020706, 29.800681131145613, 41.943695056893915, 46.15619785585283, 36.65546388385301, 95.0770390179677, 121.60538462411665, 108.5813867407896, 28.65903496526287, 35.66674528344124, 45.529450258314064, 46.246434773093995, 31.230845679407448, 53.02179868836547, 54.811841150452466, 41.29341833835342, 23.299630451611073, 36.942956250241515, 24.322607071104724, 42.77096031537661, 29.22428378123494, 29.22428378123494, 29.22428378123494, 26.505274475545082, 29.22428378123494, 30.56702687878971, 33.05083646319603, 34.77254452624438, 28.60311491479713, 27.346642204097876, 34.84052601883783, 29.56877039069624, 27.40010580820802, 59.73055807527518, 55.24173574865985, 41.780172086809024, 26.401940203271806, 31.9714638416083, 36.228414794400415, 25.840760898054555, 49.9067492938035, 27.668995768810042, 31.35307975333966, 30.093129052017023, 43.699809813576366, 57.66713919909174, 40.022953418667065, 46.51820540231633, 37.5980852538711, 30.507383918455513, 33.50580642730562, 45.086990684122, 27.40010580820802, 12.206311145618837, 39.248839424943355, 43.61454191906839, 28.883810562338418, 66.37447035867858, 35.59715165620933, 49.71218131735948, 45.88654216244616, 35.45837151398557, 37.74523999888851, 20.763634702888442, 23.852165264858517, 33.1801937796528, 63.582848099201826, 64.83690740886694, 46.88305220915621, 43.699809813576366, 29.684499285876434, 38.867415430562446, 42.10785803778799, 15.01401915561195, 37.45150421083523, 29.800681131145613, 36.299242562272255, 56.33128135834348, 30.26997357328364, 31.291903031786116, 36.08717359043372, 28.771203256660744, 24.18050850623376, 28.15966472087403, 43.78524440968234, 26.609013186724894, 41.2128458403547, 37.9670527218579, 29.975807060355134, 30.093129052017023, 41.780172086809024, 30.329152388418176, 34.300373728940066, 50.1020787877761, 37.74523999888851, 64.33234259239626, 75.21212841006654, 64.83690740886694, 94.33714363029665, 52.71203249107963, 50.1020787877761, 67.9484955630781, 46.70027251047952, 35.94648303373688, 31.169907463288272, 32.033969127372174, 39.172256347947744, 54.067559962798995, 54.067559962798995, 91.07822475921115, 79.75075829332496, 54.811841150452466, 51.09025374715126, 39.94485987582193, 43.3597351955915, 90.7231444115574, 44.64883097460398, 46.7915731119465, 25.39049775763493, 40.33685733281881, 41.698649825700315, 34.70469568042362, 51.39048944047805, 22.893645317537526, 49.61518196998445, 33.05083646319603, 25.192907336481866, 30.447857334592324, 24.322607071104724, 38.56494680401705, 90.01713130052181, 32.72964525738904, 22.98324851186175, 45.7076474882609, 57.66713919909174, 36.72712654703974, 31.90908051733889, 33.245062173048, 20.92648590822237, 39.634006510082365, 34.03344610288473, 38.94340253332466, 75.21212841006654, 84.89424029774196, 87.58907276183992, 69.01852867010916, 77.29708541258049, 61.14702744030778, 96.57428452170761, 52.50652775191775, 32.28521466114651, 50.990565478815945, 31.35307975333966, 41.2128458403547, 33.63694444585419, 30.26997357328364, 38.71588573814723, 31.230845679407448, 40.1011996370117, 20.203571081597193, 30.987805310145493, 30.507383918455513, 45.086990684122, 34.300373728940066, 28.049880592281973, 28.325147094540903, 57.66713919909174, 35.32013242426123, 27.561124577174567, 31.41437607743883, 36.08717359043372, 30.806767041912167, 30.447857334592324, 44.5617113320772, 41.132430556936306, 39.78912962559818, 36.65546388385301, 23.073202403172917, 21.173160702117308, 27.346642204097876, 16.74934871787156, 26.557093177483523, 34.16664924473404, 22.36331459495464, 21.008389858307645, 27.40010580820802, 31.660762082694095, 28.827451925481085, 17.51891667538985, 50.6926662381911, 38.0412795852805, 24.70563239932894, 32.79363281328884, 38.0412795852805, 33.90076227087091, 16.782094257242417, 73.46984133199462, 34.4346215959791, 33.90076227087091, 32.28521466114651, 22.494734284035275, 26.505274475545082, 48.182698291098816, 44.387981683814154, 42.52108200006278, 35.18243227767251, 40.892124929554, 49.61518196998445, 52.097918711212095, 112.90677745995042, 20.8856537535641, 13.511154789957821, 22.27612829392538, 77.59961765106212, 39.40245503911115, 45.44061233480379, 33.1801937796528, 65.8579401324037, 41.780172086809024, 66.37447035867858, 39.172256347947744, 115.58428452718766, 86.56862915519338, 179.02044192165667, 47.52843316962603, 56.773093073800744, 27.995149135337257, 62.597087408320306, 37.67159077349981, 54.49161709130268, 59.03467599561125, 51.692489491288626, 35.806340977539655, 166.21485355527525, 150.7503262825534, 85.2265069597304, 49.133016862271724, 33.90076227087091, 32.348333340301785, 63.582848099201826, 21.049461966173517, 29.9173176993003, 27.668995768810042, 30.987805310145493, 29.453492679719986, 30.151962130172098, 30.447857334592324, 30.447857334592324, 35.87634357692165, 26.505274475545082, 42.60421214369869, 44.387981683814154, 40.41571724385996, 26.922671813996086, 26.2990087931472, 18.758380733362646, 51.49095957359226, 40.1011996370117, 49.133016862271724, 19.278382202442366, 77.90333396724647, 35.52769382137767, 39.32557222442699, 64.08153455750669, 97.71267162761835, 29.396022468352484, 29.396022468352484, 35.38918446958933, 32.92198346312245, 35.04526897307925, 50.89107172411317, 60.67117749756472, 37.37842812826559, 68.48142221881122, 34.4346215959791, 41.45503620813183, 47.435694792893365, 36.016759615546256, 44.47476167904283, 54.38529201854992, 42.190180314697045, 53.333385249853684, 58.119428177447986, 37.45150421083523, 29.684499285876434, 32.09659661306934, 33.4404292647954, 53.02179868836547, 31.291903031786116, 32.92198346312245, 37.305494633128845, 24.850816893992036, 36.228414794400415, 31.291903031786116, 27.668995768810042, 22.189281900071105, 45.79700747446617, 41.943695056893915, 44.99901609609206, 30.034410770075546, 32.9863470466753, 33.83461446305701, 40.57389988868238, 38.94340253332466, 43.699809813576366, 23.028181534732802, 24.657426302639607, 112.46659565031442, 63.582848099201826, 72.33079701870139, 27.186876493132687, 86.23113005463887, 119.2533235714264, 30.866995306309306, 33.245062173048, 85.89494673838159, 83.90519244027122, 53.75168417471723, 59.964336869699544, 64.58413226217957, 45.7076474882609, 67.1568716116849, 42.3553080116137, 86.23113005463887, 66.89505178172818, 57.89284200037607, 98.4790416094955, 94.70636877096143, 51.79355004464666, 98.4790416094955, 48.182698291098816, 41.617286632367644, 57.66713919909174, 37.5980852538711, 61.38635013907805, 44.04255166984753, 42.190180314697045, 64.58413226217957, 50.89107172411317, 21.717643615002626, 46.33684810653952, 48.46584751755162, 40.81233544283598, 44.387981683814154, 30.74665629604239, 38.26483200699776, 79.13013296849392, 64.83690740886694, 54.173263858709824, 70.37979717263202, 37.232703447205715, 85.89494673838159, 43.444504934395866, 56.11166650505244, 66.89505178172818, 30.56702687878971, 27.668995768810042, 77.90333396724647, 75.50650035756513, 81.9615307508321, 84.2335880783952, 55.02636863114642, 44.82358155812517, 111.59137365930344, 76.39654709045266, 131.48677155266952, 151.3403463773431, 91.43469485205186, 67.41971617548266, 69.55984769951485, 77.29708541258049, 54.811841150452466, 85.56007407666469, 68.48142221881122, 97.33172602285453, 47.06654728734682, 53.646802873050504, 42.3553080116137, 69.28865955361954, 53.02179868836547, 67.1568716116849, 38.94340253332466, 87.24759532868651, 31.53732846903916, 76.69555472159655, 71.76791429061606, 65.8579401324037, 51.894808174781225, 81.00664999449546, 61.867809250367884, 38.115651564714454, 48.94146538270724, 220.19890466437903, 47.90120329104557, 57.89284200037607, 175.55787313507827, 45.88654216244616, 91.07822475921115, 76.99573263552006, 81.00664999449546, 64.33234259239626, 45.086990684122, 61.14702744030778, 56.11166650505244, 25.34095533871963, 58.34690118774256, 25.740017318702005, 59.49769069799599, 59.73055807527518, 56.11166650505244, 110.29129477712178, 54.811841150452466, 56.11166650505244, 43.444504934395866, 38.64034257045414, 47.34313736903344, 72.61389175398708, 45.44061233480379, 45.7076474882609, 45.7076474882609, 45.7076474882609, 55.6750020519895, 36.44131378072682, 40.17959882934157, 44.47476167904283, 36.44131378072682, 77.90333396724647, 18.90550453688997, 36.44131378072682, 85.89494673838159, 96.19777706766972, 48.94146538270724, 56.55175575986736, 40.49473132867618, 42.77096031537661, 46.51820540231633, 27.995149135337257, 77.59961765106212, 56.11166650505244, 75.21212841006654, 122.55914747655109, 28.547303976590324, 36.65546388385301, 91.79256012938836, 32.72964525738904, 49.42175070681059, 38.867415430562446, 44.12865633779556, 28.883810562338418, 74.62682298320091, 27.61500750152876, 64.58413226217957, 38.19016894386643, 35.32013242426123, 48.94146538270724, 20.36202965350754, 38.41459632502361, 66.63425269055388, 92.1518260518184, 48.75066069158903, 64.58413226217957, 34.367432111764096, 54.598150033144236, 39.32557222442699, 85.56007407666469, 79.75075829332496, 111.59137365930344, 72.89809449073893, 19.278382202442366, 75.80202444429054, 36.51255777326842, 90.7231444115574, 48.27689708476307, 56.33128135834348, 57.66713919909174, 63.83170433047907, 63.83170433047907, 63.83170433047907, 32.79363281328884, 39.01953819342525, 14.467069533669397, 37.8190332109868, 69.55984769951485, 27.72308958496738, 27.886006392229614, 56.773093073800744, 108.15806803146498, 39.01953819342525, 66.63425269055388, 43.275130861102994, 88.27604237373367, 68.21443845541918, 117.86398026897247, 72.61389175398708, 43.61454191906839, 54.173263858709824, 53.02179868836547, 61.38635013907805, 44.91121316574036, 44.5617113320772, 32.72964525738904, 33.375179667439795, 27.028044172168716, 57.66713919909174, 34.70469568042362, 77.29708541258049, 39.40245503911115, 32.92198346312245, 78.51433738825796, 38.19016894386643, 52.097918711212095, 100.02986085865793, 32.60204446315635, 72.61389175398708, 44.82358155812517, 37.37842812826559, 102.00277308269969, 46.7915731119465, 66.37447035867858, 124.48917321564447, 98.86447768050158, 99.2514223056366, 51.79355004464666, 35.52769382137767, 67.9484955630781, 71.76791429061606, 72.89809449073893, 38.48969815122384, 88.27604237373367, 34.908640417532865, 81.32370105824417, 35.45837151398557, 67.68358948381629, 81.9615307508321, 100.02986085865793, 67.41971617548266, 54.173263858709824, 60.199030648454944, 57.89284200037607, 46.33684810653952, 36.15772522700737, 27.4536739354601, 35.52769382137767, 48.08868329990042, 42.854578979512624, 69.28865955361954, 80.06289393593435, 71.48811770889668, 68.48142221881122, 45.529450258314064, 96.19777706766972, 51.1901369093998, 53.646802873050504, 46.70027251047952, 53.22932014095188, 51.09025374715126, 56.11166650505244, 56.11166650505244, 48.27689708476307, 76.69555472159655, 49.22907356066477, 72.89809449073893, 32.53843073806492, 22.626926270737744, 47.52843316962603, 21.256030110048442, 19.851533330124635, 33.76859572409541, 34.367432111764096, 54.38529201854992, 66.11570082213345, 42.27266353442275, 89.66618776422116, 42.854578979512624, 57.21836996148928, 71.20941195067294, 43.87084603329354, 30.21091022897123, 82.60436300370014, 79.13013296849392, 57.89284200037607, 17.382583384208317, 67.9484955630781, 85.89494673838159, 93.96935796203766, 84.563269020706, 21.33922385967176, 91.43469485205186, 51.49095957359226, 68.48142221881122, 53.43765380953904, 26.76538304079522, 40.25815129472595, 40.57389988868238, 64.58413226217957, 35.11378365112735, 78.51433738825796, 39.248839424943355, 48.65553748142455, 31.475792237910618, 51.894808174781225, 21.049461966173517, 45.79700747446617, 31.9714638416083, 77.59961765106212, 45.086990684122, 62.597087408320306, 56.33128135834348, 44.387981683814154, 88.96839995507139, 90.01713130052181, 45.351947753528236, 43.699809813576366, 55.457945789238345, 53.12545808585662, 64.83690740886694, 81.32370105824417, 37.9670527218579, 81.641993024771, 85.56007407666469, 84.89424029774196, 38.48969815122384, 35.18243227767251, 37.232703447205715, 24.753932740556596, 41.617286632367644, 102.80279074731456, 114.23768682435218, 63.582848099201826, 55.24173574865985, 41.2128458403547, 32.9863470466753, 39.172256347947744, 50.49503426195374, 41.780172086809024, 63.08804244970428, 56.773093073800744, 33.83461446305701, 82.60436300370014, 82.60436300370014, 20.84490127137742, 52.71203249107963, 35.32013242426123, 18.6852487375444, 25.689792910615846, 58.34690118774256, 38.71588573814723, 18.6852487375444, 39.479488162280596, 26.817710263716034, 24.133327041205547, 148.99403178365867, 29.975807060355134, 39.634006510082365, 39.40245503911115, 52.50652775191775, 26.505274475545082, 80.69083499570341, 37.89297069129353, 33.1801937796528, 48.94146538270724, 63.3349620664254, 65.34542957624015, 43.10641711474324, 77.90333396724647, 51.79355004464666, 41.780172086809024, 50.79177210350426, 56.11166650505244, 54.49161709130268, 32.92198346312245, 36.72712654703974, 76.69555472159655, 29.281418237700255, 32.28521466114651, 24.370158584949497, 26.453556883236853, 29.453492679719986, 26.87013978811538, 26.87013978811538, 26.87013978811538, 26.87013978811538, 26.87013978811538, 29.396022468352484, 39.01953819342525, 32.79363281328884, 94.70636877096143, 55.457945789238345, 55.457945789238345, 45.61846186294896, 60.199030648454944, 35.7364749685521, 37.67159077349981, 19.929230034775262, 54.598150033144236, 52.19977189231385, 44.64883097460398, 65.8579401324037, 46.70027251047952, 34.77254452624438, 27.346642204097876, 43.5294404008871, 27.886006392229614, 60.908637772317434, 28.436008585761353, 29.56877039069624, 20.64232851707703, 42.3553080116137, 41.861853726676614, 78.82163381626928, 59.73055807527518, 41.861853726676614, 72.04880596519355, 34.501942437892716, 49.61518196998445, 46.066137010589394, 28.380523708580828, 17.76007792103387, 22.494734284035275, 31.846818916590372, 41.45503620813183, 31.660762082694095, 25.840760898054555, 31.72265993608251, 60.199030648454944, 22.938403163153815, 41.617286632367644, 20.723120306578686, 40.25815129472595, 26.505274475545082, 25.2915095879081, 65.8579401324037, 25.045724266551254, 36.51255777326842, 33.1801937796528, 35.32013242426123, 31.9714638416083, 32.9863470466753, 28.15966472087403, 49.9067492938035, 35.25121511458854, 40.73270164287187, 38.48969815122384, 47.250760544967605, 25.045724266551254, 29.511075247281195, 61.38635013907805, 18.90550453688997, 46.60915005647103, 57.21836996148928, 48.75066069158903, 25.489873356103875, 37.01518097066168, 41.132430556936306, 62.84208548133934, 67.9484955630781, 36.37020880080945, 28.940279382223952, 147.25819873544805, 36.942956250241515, 25.94189877662824, 50.1020787877761, 24.850816893992036, 45.97625189374871, 45.97625189374871, 36.87087245606033, 28.1047190510635, 42.854578979512624, 24.89940107578365, 77.90333396724647, 24.039240115615065, 23.028181534732802, 33.83461446305701, 45.086990684122, 84.89424029774196, 22.67116267165818, 31.41437607743883, 45.17513726542669, 19.391673055829287, 72.89809449073893, 28.2698785323965, 31.784678801853513, 25.045724266551254, 24.275148340672132, 40.41571724385996, 30.21091022897123, 53.646802873050504, 28.60311491479713, 24.086237637543274, 31.53732846903916, 34.300373728940066, 38.41459632502361, 20.64232851707703, 31.10908815096766, 30.62678644311483, 33.57131140436464, 30.507383918455513, 24.753932740556596, 18.431524745974645, 24.850816893992036, 23.94552000018515, 20.085536923187668, 39.86691871057298, 32.47494113727571, 71.48811770889668, 31.784678801853513, 37.305494633128845, 19.543763832601815, 15.920030957803608, 67.1568716116849, 49.51837188956857, 77.29708541258049, 18.00455893510183, 48.56059987746486, 27.886006392229614, 55.89290784893326, 36.37020880080945, 66.11570082213345, 98.86447768050158, 26.14536361045317, 57.21836996148928, 35.66674528344124, 58.80452160545304, 71.76791429061606, 90.7231444115574, 19.96819234893607, 38.71588573814723, 39.172256347947744, 33.245062173048, 41.943695056893915, 75.21212841006654, 53.646802873050504, 53.333385249853684, 41.45503620813183, 58.57526450222688, 27.40010580820802, 50.79177210350426, 87.93188669987823, 34.63697922255266, 100.02986085865793, 56.55175575986736, 50.5937537501147, 51.894808174781225, 34.56939489431364, 27.61500750152876, 44.5617113320772, 61.38635013907805, 54.173263858709824, 47.71445419747699, 54.173263858709824, 29.396022468352484, 51.692489491288626, 30.447857334592324, 49.9067492938035, 67.41971617548266, 79.13013296849392, 31.784678801853513, 42.52108200006278, 68.21443845541918, 34.97688798216538, 32.348333340301785, 56.773093073800744, 78.51433738825796, 19.091033031037714, 68.21443845541918, 69.55984769951485, 39.40245503911115, 32.22221914035047, 24.850816893992036, 41.2128458403547, 32.92198346312245, 31.72265993608251, 76.09870517958821, 39.172256347947744, 32.09659661306934, 38.0412795852805, 52.71203249107963, 26.817710263716034, 51.692489491288626, 81.9615307508321, 63.582848099201826, 72.04880596519355, 64.83690740886694, 57.89284200037607, 47.807737558209425, 108.15806803146498, 41.861853726676614, 98.09510821132314, 110.29129477712178, 53.333385249853684, 23.620367629992042, 47.90120329104557, 31.598985005566618, 63.08804244970428, 71.20941195067294, 40.81233544283598, 33.83461446305701, 32.92198346312245, 72.33079701870139, 62.597087408320306, 29.110349096271467, 91.43469485205186, 50.6926662381911, 106.89798837141365, 82.92766733936132, 40.1011996370117, 121.60538462411665, 118.78839892336089, 37.8190332109868, 28.827451925481085, 40.33685733281881, 45.88654216244616, 74.04607243930377, 31.846818916590372, 46.60915005647103, 40.022953418667065, 61.867809250367884, 23.028181534732802, 29.053548433146837, 86.90744918987926, 64.08153455750669, 41.37414835829223, 47.52843316962603, 42.27266353442275, 52.301824199806006, 34.908640417532865, 34.300373728940066, 46.33684810653952, 68.48142221881122, 21.464625403032095, 30.74665629604239, 23.482371718117374, 78.51433738825796, 78.51433738825796, 27.61500750152876, 43.61454191906839, 52.301824199806006, 106.06610162605153, 33.90076227087091, 82.92766733936132, 48.08868329990042, 64.58413226217957, 67.1568716116849, 68.48142221881122, 47.06654728734682, 47.06654728734682, 72.89809449073893, 83.57807709540792, 77.90333396724647, 76.99573263552006, 54.598150033144236, 69.55984769951485, 76.99573263552006, 61.867809250367884, 84.563269020706, 90.7231444115574, 27.028044172168716, 88.27604237373367, 87.24759532868651, 84.563269020706, 18.979498589532987, 48.84596987082467, 37.305494633128845, 94.70636877096143, 59.964336869699544, 89.31661242710538, 39.94485987582193, 28.049880592281973, 13.511154789957821, 22.80439145296227, 99.2514223056366, 51.09025374715126, 81.641993024771, 68.21443845541918, 38.79157659527073, 78.82163381626928, 43.5294404008871, 69.55984769951485, 50.6926662381911, 43.275130861102994, 24.465540688808968, 63.08804244970428, 54.27917440928786, 27.186876493132687, 34.09998263323346, 37.524723159601, 51.09025374715126, 46.60915005647103, 24.322607071104724, 45.263456176258764, 50.1020787877761, 51.894808174781225, 20.4816886421048, 104.83039411284926, 31.90908051733889, 29.053548433146837, 39.248839424943355, 46.15619785585283, 31.9714638416083, 43.3597351955915, 34.84052601883783, 28.996858600549206, 36.08717359043372, 46.246434773093995, 37.232703447205715, 32.348333340301785, 46.246434773093995, 109.86130976812342, 54.598150033144236, 42.77096031537661, 48.56059987746486, 27.995149135337257, 59.03467599561125, 65.8579401324037, 32.92198346312245, 34.56939489431364, 28.547303976590324, 56.773093073800744, 25.34095533871963, 36.15772522700737, 42.68750480955783, 57.66713919909174, 26.557093177483523, 81.641993024771, 86.23113005463887, 79.13013296849392, 38.79157659527073, 37.67159077349981, 40.81233544283598, 28.1047190510635, 38.0412795852805, 85.56007407666469, 83.25223705241488, 29.396022468352484, 59.964336869699544, 61.626609520407456, 41.29341833835342, 42.68750480955783, 64.33234259239626, 104.01459751366006, 43.87084603329354, 84.563269020706, 46.88305220915621, 18.503663694167408, 65.34542957624015, 61.867809250367884, 21.380942759123343, 20.24306974171993, 25.2915095879081, 100.81440478091984, 71.20941195067294, 63.3349620664254, 38.33964103892552, 42.52108200006278, 86.90744918987926, 26.14536361045317, 92.87458178129451, 105.24068867470233, 85.89494673838159, 64.58413226217957, 52.9183415530491, 55.89290784893326, 69.83209724580306, 46.33684810653952, 66.11570082213345, 53.646802873050504, 71.48811770889668, 150.1626064581185, 44.5617113320772, 108.5813867407896, 47.90120329104557, 52.40407602298735, 42.68750480955783, 115.58428452718766, 77.59961765106212, 35.32013242426123, 47.90120329104557, 58.57526450222688, 24.039240115615065, 20.401838105461145, 75.80202444429054, 65.60118435636834, 21.090614371389922, 24.370158584949497, 95.44916002730179, 42.27266353442275, 20.36202965350754, 23.482371718117374, 49.61518196998445, 48.08868329990042, 44.30137101535166, 55.02636863114642, 18.979498589532987, 49.22907356066477, 79.75075829332496, 39.556671887793456, 58.119428177447986, 74.62682298320091, 37.232703447205715, 103.20514949397284, 43.19069160819034, 40.1011996370117, 83.90519244027122, 36.44131378072682, 30.388446900124176, 26.196478674560588, 38.867415430562446, 41.53608219643496, 26.196478674560588, 62.10995300937982, 90.01713130052181, 29.511075247281195, 32.66578255521719, 39.095822701298815, 39.479488162280596, 45.88654216244616, 35.87634357692165, 40.97207040739876, 52.81508628524973, 47.52843316962603, 45.44061233480379, 36.228414794400415, 26.043432497666576, 55.02636863114642, 55.457945789238345, 46.33684810653952, 27.507346790200362, 47.62135285300038, 38.0412795852805, 33.57131140436464, 37.160054292819765, 51.49095957359226, 40.41571724385996, 51.591626129191596, 53.646802873050504, 45.351947753528236, 31.9714638416083, 40.73270164287187, 30.093129052017023, 52.097918711212095, 31.72265993608251, 42.3553080116137, 43.19069160819034, 57.89284200037607, 72.61389175398708, 74.04607243930377, 76.39654709045266, 59.265731184583274, 67.41971617548266, 82.28231911219243, 25.94189877662824, 45.17513726542669, 109.43300111180854, 52.9183415530491, 39.01953819342525, 34.09998263323346, 72.61389175398708, 66.63425269055388, 29.74253347911242, 31.660762082694095, 116.94675549762589, 44.387981683814154, 48.27689708476307, 34.367432111764096, 72.89809449073893, 46.33684810653952, 59.73055807527518, 32.47494113727571, 86.23113005463887, 164.9213593028435, 36.44131378072682, 156.7555365841094, 47.99485175252902, 37.305494633128845, 84.89424029774196, 131.48677155266952, 51.894808174781225, 41.780172086809024, 54.598150033144236, 78.2082389954826, 59.964336869699544, 40.33685733281881, 70.37979717263202, 77.90333396724647, 27.40010580820802, 28.380523708580828, 40.81233544283598, 93.2380826167318, 148.99403178365867, 32.66578255521719, 81.9615307508321, 104.01459751366006, 141.61682573322432, 25.39049775763493, 46.066137010589394, 45.263456176258764, 55.6750020519895, 36.08717359043372, 44.47476167904283, 37.89297069129353, 36.87087245606033, 32.72964525738904, 31.9714638416083, 37.89297069129353, 52.301824199806006, 16.04489330813532, 46.066137010589394, 32.22221914035047, 46.51820540231633, 43.10641711474324, 67.9484955630781, 63.3349620664254, 34.03344610288473, 37.74523999888851, 44.04255166984753, 101.6051019574268, 48.84596987082467, 74.62682298320091, 51.591626129191596, 61.38635013907805, 36.15772522700737, 50.990565478815945, 34.23344619169951, 27.940524471445343, 54.27917440928786, 62.10995300937982, 41.2128458403547, 43.78524440968234, 40.49473132867618, 40.41571724385996, 36.58394105020859, 46.7915731119465, 77.29708541258049, 72.33079701870139, 50.89107172411317, 53.646802873050504, 39.40245503911115, 42.3553080116137, 50.6926662381911, 29.453492679719986, 32.411575418595035, 61.626609520407456, 55.457945789238345, 38.48969815122384, 30.034410770075546, 36.228414794400415, 52.609179777419676, 72.89809449073893, 28.380523708580828, 26.24769367033534, 35.32013242426123, 52.50652775191775, 45.88654216244616, 40.65322322588273, 39.94485987582193, 44.82358155812517, 31.90908051733889, 52.40407602298735, 49.61518196998445, 45.88654216244616, 77.59961765106212, 53.85677052285177, 45.44061233480379, 101.20898120494374, 67.68358948381629, 54.811841150452466, 52.81508628524973, 40.49473132867618, 95.44916002730179, 59.73055807527518, 38.26483200699776, 85.89494673838159, 65.60118435636834, 51.39048944047805, 35.32013242426123, 73.18340956555208, 44.99901609609206, 112.02812994692015, 28.214717811314713, 95.0770390179677, 71.20941195067294, 90.01713130052181, 131.48677155266952, 59.265731184583274, 34.84052601883783, 98.86447768050158, 131.48677155266952, 21.887977204970085, 51.894808174781225, 74.62682298320091, 37.5980852538711, 100.42136666458902, 39.86691871057298, 32.47494113727571, 48.75066069158903, 34.367432111764096, 47.62135285300038, 33.76859572409541, 62.3530444922685, 79.13013296849392, 33.11545195869231, 85.89494673838159, 77.59961765106212, 48.75066069158903, 38.0412795852805, 56.33128135834348, 59.49769069799599, 53.22932014095188, 79.75075829332496, 104.83039411284926, 106.89798837141365, 106.89798837141365, 41.53608219643496, 113.34868209247811, 81.641993024771, 134.08011574678568, 72.04880596519355, 56.33128135834348, 81.9615307508321, 43.02230705927991, 59.73055807527518, 72.04880596519355, 55.457945789238345, 29.281418237700255, 103.20514949397284, 38.115651564714454, 31.784678801853513, 63.3349620664254, 61.38635013907805, 61.38635013907805, 46.066137010589394, 51.99626426796157, 66.89505178172818, 76.99573263552006, 33.702705802144536, 19.735555861572347, 78.51433738825796, 88.96839995507139, 87.58907276183992, 106.48123261036584, 22.145985754016134, 66.89505178172818, 34.63697922255266, 48.37128004023317, 78.82163381626928, 55.457945789238345, 116.03666862607915, 47.71445419747699, 85.2265069597304, 89.31661242710538, 38.64034257045414, 77.59961765106212, 111.15632012308374, 161.73149145963927, 81.9615307508321, 29.167260806600495, 62.597087408320306, 55.6750020519895, 69.01852867010916, 60.199030648454944, 87.24759532868651, 195.84896001735729, 42.10785803778799, 118.78839892336089, 53.96206231832688, 37.01518097066168, 45.79700747446617, 151.3403463773431, 49.71218131735948, 39.634006510082365, 44.64883097460398, 67.68358948381629, 42.60421214369869, 44.12865633779556, 83.57807709540792, 37.67159077349981, 74.04607243930377, 53.75168417471723, 73.18340956555208, 32.66578255521719, 52.097918711212095, 45.351947753528236, 121.13129015902297, 122.08133464248264, 105.24068867470233, 42.10785803778799, 54.811841150452466, 94.70636877096143, 46.88305220915621, 71.48811770889668, 95.44916002730179, 79.13013296849392, 80.06289393593435, 42.27266353442275, 73.75739416067401, 54.598150033144236, 36.016759615546256, 43.10641711474324, 36.58394105020859, 59.49769069799599, 110.7229626998625, 40.022953418667065, 64.33234259239626, 109.43300111180854, 73.46984133199462, 158.60332124553608, 62.84208548133934, 90.7231444115574, 35.18243227767251, 31.475792237910618, 56.9952966774873, 91.79256012938836, 85.56007407666469, 50.5937537501147, 74.04607243930377, 60.199030648454944, 37.37842812826559, 43.5294404008871, 37.67159077349981, 55.457945789238345, 25.740017318702005, 68.7494509271083, 62.597087408320306, 35.87634357692165, 73.18340956555208, 64.58413226217957, 66.11570082213345, 45.97625189374871, 71.76791429061606, 99.2514223056366, 41.05217218133816, 37.74523999888851, 60.908637772317434, 32.22221914035047, 85.56007407666469, 59.49769069799599, 156.7555365841094, 51.591626129191596, 65.60118435636834, 42.68750480955783, 95.0770390179677, 83.57807709540792, 98.4790416094955, 57.89284200037607, 76.39654709045266, 66.37447035867858, 61.626609520407456, 115.13366410675533, 75.80202444429054, 73.75739416067401, 85.2265069597304, 58.119428177447986, 59.49769069799599, 102.40200064874995, 65.34542957624015, 32.28521466114651, 28.883810562338418, 93.6030061542134, 39.94485987582193, 29.281418237700255, 67.68358948381629, 48.27689708476307, 63.3349620664254, 46.70027251047952, 85.56007407666469, 57.21836996148928, 80.37625124290746, 51.49095957359226, 73.18340956555208, 24.039240115615065, 90.36944839098756, 83.57807709540792, 43.3597351955915, 25.740017318702005, 26.817710263716034, 21.760102337787384, 31.9714638416083, 26.401940203271806, 43.02230705927991, 44.30137101535166, 126.44959243764188, 41.780172086809024, 54.811841150452466, 71.48811770889668, 54.598150033144236, 106.89798837141365, 93.96935796203766, 38.33964103892552, 46.51820540231633, 49.71218131735948, 97.33172602285453, 80.37625124290746, 44.12865633779556, 31.72265993608251, 45.529450258314064, 65.8579401324037, 57.21836996148928, 27.83159468972004, 31.048387510438282, 36.37020880080945, 62.597087408320306, 39.78912962559818, 29.167260806600495, 56.773093073800744, 86.23113005463887, 40.49473132867618, 34.97688798216538, 61.14702744030778, 59.265731184583274, 29.22428378123494, 34.300373728940066, 41.37414835829223, 45.79700747446617, 70.37979717263202, 33.4404292647954, 27.240027749911206, 44.12865633779556, 43.95661501095434, 42.68750480955783, 102.00277308269969, 79.75075829332496, 46.60915005647103, 44.12865633779556, 55.89290784893326, 72.89809449073893, 54.811841150452466, 38.48969815122384, 53.02179868836547, 29.167260806600495, 41.05217218133816, 42.3553080116137, 45.44061233480379, 62.597087408320306, 55.24173574865985, 26.35042423874788, 78.2082389954826, 41.780172086809024, 58.57526450222688, 47.34313736903344, 69.83209724580306, 68.48142221881122, 72.04880596519355, 42.3553080116137, 45.351947753528236, 34.300373728940066, 41.45503620813183, 31.660762082694095, 63.582848099201826, 84.563269020706, 34.23344619169951, 85.2265069597304, 68.21443845541918, 44.82358155812517, 39.634006510082365, 62.3530444922685, 58.80452160545304, 69.55984769951485, 55.02636863114642, 28.2698785323965, 69.83209724580306, 23.345182011801924, 27.83159468972004, 43.78524440968234, 27.561124577174567, 27.186876493132687, 83.57807709540792, 34.84052601883783, 25.790339917193062, 56.11166650505244, 24.80232751057381, 46.427438201089, 59.03467599561125, 59.03467599561125, 59.03467599561125, 29.281418237700255, 40.81233544283598, 46.246434773093995, 34.908640417532865, 48.65553748142455, 91.43469485205186, 85.89494673838159, 88.62154503481175, 68.21443845541918, 30.034410770075546, 39.095822701298815, 73.75739416067401, 69.28865955361954, 38.115651564714454, 39.86691871057298, 82.28231911219243, 67.41971617548266, 43.95661501095434, 84.89424029774196, 24.948080241282657, 46.88305220915621, 38.56494680401705, 33.4404292647954, 33.76859572409541, 47.71445419747699, 33.375179667439795, 72.61389175398708, 35.11378365112735, 45.7076474882609, 89.66618776422116, 129.9549042642197, 53.22932014095188, 31.53732846903916, 65.09067188950593, 82.92766733936132, 55.89290784893326, 60.67117749756472, 32.22221914035047, 40.33685733281881, 30.62678644311483, 87.58907276183992, 80.37625124290746, 72.89809449073893, 53.333385249853684, 63.3349620664254, 112.46659565031442, 60.908637772317434, 45.7076474882609, 84.2335880783952, 48.84596987082467, 68.7494509271083, 46.88305220915621, 39.78912962559818, 47.99485175252902, 87.93188669987823, 125.9566121960733, 29.800681131145613, 66.63425269055388, 42.438114061533675, 26.2990087931472, 76.39654709045266, 59.03467599561125, 55.02636863114642, 129.9549042642197, 116.49082330627213, 42.77096031537661, 128.94358717585138, 68.48142221881122, 76.39654709045266, 29.281418237700255, 52.71203249107963, 48.75066069158903, 143.28616006164455, 50.29817277977263, 56.55175575986736, 78.51433738825796, 201.27809318052155, 75.50650035756513, 40.49473132867618, 71.20941195067294, 82.92766733936132, 29.975807060355134, 109.43300111180854, 32.09659661306934, 27.77728915635237, 40.49473132867618, 86.56862915519338, 33.1801937796528, 39.095822701298815, 34.56939489431364, 39.01953819342525, 31.291903031786116, 22.626926270737744, 42.27266353442275, 53.75168417471723, 37.9670527218579, 54.067559962798995, 56.773093073800744, 53.75168417471723, 37.37842812826559, 39.479488162280596, 36.72712654703974, 113.34868209247811, 81.00664999449546, 70.93179276322707, 21.090614371389922, 23.94552000018515, 50.79177210350426, 34.908640417532865, 40.17959882934157, 25.891280453627324, 23.898797049169467, 18.039758452611142, 34.16664924473404, 76.39654709045266, 22.98324851186175, 56.55175575986736, 43.02230705927991, 55.89290784893326, 50.396507397123145, 34.56939489431364, 39.94485987582193, 19.96819234893607, 132.51803508331912, 112.90677745995042, 45.086990684122, 84.2335880783952, 31.598985005566618, 70.37979717263202, 52.71203249107963, 26.609013186724894, 74.04607243930377, 83.90519244027122, 47.250760544967605, 69.83209724580306, 63.3349620664254, 75.21212841006654, 46.70027251047952, 37.89297069129353, 100.42136666458902, 62.84208548133934, 23.20879380099744, 22.938403163153815, 43.87084603329354, 29.74253347911242, 33.50580642730562, 41.37414835829223, 66.11570082213345, 56.11166650505244, 33.245062173048, 22.98324851186175, 48.182698291098816, 94.33714363029665, 67.1568716116849, 40.17959882934157, 34.4346215959791, 106.89798837141365, 37.74523999888851, 79.75075829332496, 117.86398026897247, 47.52843316962603, 47.15856396830611, 76.39654709045266, 35.59715165620933, 92.1518260518184, 84.2335880783952, 94.33714363029665, 27.186876493132687, 33.57131140436464, 33.57131140436464, 34.16664924473404, 42.3553080116137, 63.83170433047907, 66.11570082213345, 86.23113005463887, 133.5573869140851, 108.5813867407896, 70.65525591042105, 78.2082389954826, 51.894808174781225, 72.61389175398708, 47.06654728734682, 75.80202444429054, 95.82273747708687, 39.556671887793456, 58.57526450222688, 39.479488162280596, 45.79700747446617, 54.067559962798995, 68.48142221881122, 57.89284200037607, 53.96206231832688, 50.89107172411317, 24.513371642677637, 20.64232851707703, 62.597087408320306, 71.20941195067294, 39.7114923241556, 51.09025374715126, 92.87458178129451, 149.57717793613415, 106.48123261036584, 66.63425269055388, 96.57428452170761, 66.89505178172818, 61.626609520407456, 67.41971617548266, 88.96839995507139, 80.37625124290746, 85.56007407666469, 83.90519244027122, 79.13013296849392, 185.4260684919039, 137.7969534714142, 76.99573263552006, 70.93179276322707, 53.22932014095188, 67.1568716116849, 58.80452160545304, 47.34313736903344, 60.908637772317434, 57.21836996148928, 32.66578255521719, 48.75066069158903, 48.182698291098816, 58.57526450222688, 73.46984133199462, 65.60118435636834, 93.6030061542134, 47.435694792893365, 48.75066069158903, 100.81440478091984, 59.265731184583274, 59.265731184583274, 111.59137365930344, 122.08133464248264, 91.79256012938836, 62.10995300937982, 33.31005738633129, 59.73055807527518, 62.597087408320306, 22.10277408821932, 69.83209724580306, 63.582848099201826, 21.548635571548648, 70.65525591042105, 54.27917440928786, 63.582848099201826, 77.90333396724647, 43.444504934395866, 30.866995306309306, 90.36944839098756, 98.86447768050158, 55.02636863114642, 50.004318665090636, 77.90333396724647, 56.55175575986736, 108.5813867407896, 57.66713919909174, 65.09067188950593, 63.08804244970428, 116.49082330627213, 65.8579401324037, 70.65525591042105, 43.5294404008871, 79.13013296849392, 28.996858600549206, 36.798929313140206, 59.03467599561125, 23.345182011801924, 80.69083499570341, 112.02812994692015, 42.438114061533675, 87.93188669987823, 71.20941195067294, 74.91890411002962, 76.39654709045266, 84.89424029774196, 30.447857334592324, 48.08868329990042, 39.40245503911115, 68.7494509271083, 91.43469485205186, 83.25223705241488, 119.2533235714264, 42.10785803778799, 42.10785803778799, 85.2265069597304, 86.23113005463887, 61.14702744030778, 76.99573263552006, 90.36944839098756, 66.89505178172818, 62.84208548133934, 56.11166650505244, 73.75739416067401, 69.83209724580306, 64.58413226217957, 69.28865955361954, 105.24068867470233, 100.42136666458902, 85.89494673838159, 68.21443845541918, 61.626609520407456, 87.24759532868651, 70.10541234668786, 56.11166650505244, 61.38635013907805, 50.49503426195374, 51.1901369093998, 80.37625124290746, 101.20898120494374, 64.83690740886694, 49.42175070681059, 55.89290784893326, 38.19016894386643, 69.83209724580306, 61.14702744030778, 47.435694792893365, 60.67117749756472, 34.84052601883783, 60.908637772317434, 78.51433738825796, 75.50650035756513, 40.73270164287187, 52.40407602298735, 74.33588057277046, 67.68358948381629, 25.094689498803355, 103.60908302823987, 22.626926270737744, 61.14702744030778, 42.77096031537661, 66.89505178172818, 130.4635333784854, 54.173263858709824, 46.51820540231633, 50.89107172411317, 80.69083499570341, 30.21091022897123, 122.08133464248264, 81.641993024771, 94.70636877096143, 67.41971617548266, 55.02636863114642, 55.89290784893326, 127.44134888284603, 62.84208548133934, 73.18340956555208, 56.33128135834348, 66.63425269055388, 55.24173574865985, 43.3597351955915, 47.52843316962603, 33.1801937796528, 68.21443845541918, 60.908637772317434, 49.42175070681059, 75.21212841006654, 60.67117749756472, 74.62682298320091, 67.68358948381629, 112.90677745995042, 46.97471015107385, 129.9549042642197, 46.15619785585283, 49.42175070681059, 61.626609520407456, 172.83610058973207, 94.33714363029665, 86.56862915519338, 134.08011574678568, 50.6926662381911, 76.09870517958821, 63.83170433047907, 43.444504934395866, 43.5294404008871, 37.160054292819765, 45.44061233480379, 47.52843316962603, 59.265731184583274, 62.10995300937982, 43.19069160819034, 80.69083499570341, 64.33234259239626, 99.2514223056366, 43.3597351955915, 62.10995300937982, 81.641993024771, 101.6051019574268, 45.44061233480379, 65.34542957624015, 52.097918711212095, 51.290215346585704, 56.11166650505244, 55.24173574865985, 43.699809813576366, 41.45503620813183, 112.46659565031442, 43.95661501095434, 68.7494509271083, 47.99485175252902, 63.08804244970428, 60.908637772317434, 87.24759532868651, 64.33234259239626, 53.646802873050504, 47.250760544967605, 72.33079701870139, 48.08868329990042, 130.4635333784854, 49.809370301716946, 39.32557222442699, 72.04880596519355, 59.265731184583274, 38.115651564714454, 49.22907356066477, 70.10541234668786, 48.84596987082467, 48.84596987082467, 51.591626129191596, 51.79355004464666, 43.699809813576366, 59.03467599561125, 43.275130861102994, 35.11378365112735, 64.58413226217957, 59.03467599561125, 120.1886389803891, 146.6840934260825, 119.72006788311512, 56.33128135834348, 62.84208548133934, 56.773093073800744, 85.89494673838159, 39.248839424943355, 81.641993024771, 65.60118435636834, 174.8734378511126, 78.51433738825796, 65.8579401324037, 33.83461446305701, 34.16664924473404, 44.12865633779556, 63.3349620664254, 73.75739416067401, 32.53843073806492, 45.61846186294896, 124.00383592430612, 42.438114061533675, 57.442316329639176, 31.169907463288272, 56.773093073800744, 66.89505178172818, 52.097918711212095, 43.5294404008871, 44.04255166984753, 47.71445419747699, 30.093129052017023, 94.70636877096143, 68.48142221881122, 58.119428177447986, 85.2265069597304, 112.02812994692015, 78.2082389954826, 32.159346537604755, 52.50652775191775, 51.49095957359226, 96.57428452170761, 41.698649825700315, 70.65525591042105, 31.784678801853513, 41.698649825700315, 59.265731184583274, 51.591626129191596, 60.199030648454944, 44.91121316574036, 41.29341833835342, 59.03467599561125, 44.64883097460398, 27.507346790200362, 86.56862915519338, 49.133016862271724, 96.19777706766972, 73.46984133199462, 69.28865955361954, 52.50652775191775, 65.34542957624015, 83.57807709540792, 86.56862915519338, 27.346642204097876, 42.438114061533675, 48.37128004023317, 65.34542957624015, 50.20003003478527, 63.83170433047907, 36.228414794400415, 32.92198346312245, 58.80452160545304, 36.44131378072682, 54.49161709130268, 42.02569638966092, 126.94450214932196, 78.2082389954826, 94.70636877096143, 102.80279074731456, 78.51433738825796, 105.24068867470233, 109.00636227269737, 59.03467599561125, 36.87087245606033, 76.69555472159655, 39.01953819342525, 74.62682298320091, 68.48142221881122, 119.72006788311512, 84.563269020706, 103.60908302823987, 76.09870517958821, 58.80452160545304, 66.63425269055388, 110.7229626998625, 59.964336869699544, 58.80452160545304, 91.43469485205186, 63.83170433047907, 57.66713919909174, 82.28231911219243, 84.563269020706, 84.563269020706, 40.65322322588273, 58.80452160545304, 34.16664924473404, 63.08804244970428, 76.39654709045266, 70.10541234668786, 107.31637526839133, 107.7363996853844, 144.4099673433467, 71.76791429061606, 96.19777706766972, 82.28231911219243, 79.13013296849392, 70.93179276322707, 30.507383918455513, 56.773093073800744, 34.300373728940066, 33.1801937796528, 29.33866439394716, 78.82163381626928, 65.60118435636834, 56.9952966774873, 74.91890411002962, 63.08804244970428, 104.01459751366006, 42.854578979512624, 71.48811770889668, 73.18340956555208, 42.02569638966092, 76.09870517958821, 39.556671887793456, 73.18340956555208, 66.37447035867858, 74.91890411002962, 123.52039078374908, 68.7494509271083, 50.89107172411317, 56.55175575986736, 74.91890411002962, 53.96206231832688, 58.34690118774256, 43.61454191906839, 51.591626129191596, 79.75075829332496, 49.71218131735948, 42.68750480955783, 127.94014021950323, 113.34868209247811, 42.93836112094585, 61.38635013907805, 69.83209724580306, 54.811841150452466, 137.7969534714142, 63.08804244970428, 43.5294404008871, 55.02636863114642, 82.92766733936132, 38.48969815122384, 62.10995300937982, 112.02812994692015, 77.29708541258049, 80.69083499570341, 70.93179276322707, 58.34690118774256, 47.435694792893365, 60.67117749756472, 59.49769069799599, 65.60118435636834, 92.1518260518184, 77.90333396724647, 47.435694792893365, 71.20941195067294, 72.89809449073893, 91.43469485205186, 88.62154503481175, 132.00139622598326, 109.43300111180854, 170.82249902639472, 149.57717793613415, 129.44825810694724, 65.09067188950593, 81.00664999449546, 51.99626426796157, 150.1626064581185, 58.80452160545304, 47.250760544967605, 64.08153455750669, 62.84208548133934, 51.79355004464666, 92.87458178129451, 95.44916002730179, 79.75075829332496, 77.29708541258049, 68.21443845541918, 31.35307975333966, 29.74253347911242, 119.72006788311512, 73.46984133199462, 61.14702744030778, 105.24068867470233, 74.33588057277046, 70.10541234668786, 44.5617113320772, 49.037147591523336, 59.49769069799599, 43.444504934395866, 35.806340977539655, 87.58907276183992, 55.24173574865985, 65.34542957624015, 45.17513726542669, 83.90519244027122, 58.57526450222688, 49.22907356066477, 56.33128135834348, 78.51433738825796, 56.773093073800744, 63.3349620664254, 94.70636877096143, 75.80202444429054, 205.2479478525057, 60.434642992688794, 71.20941195067294, 43.95661501095434, 70.93179276322707, 104.83039411284926, 73.18340956555208, 83.25223705241488, 29.626578330054702, 50.396507397123145, 57.21836996148928, 119.2533235714264, 65.8579401324037, 98.4790416094955, 109.86130976812342, 63.83170433047907, 82.92766733936132, 67.41971617548266, 82.92766733936132, 135.66060972280735, 68.48142221881122, 70.10541234668786, 164.27839185643572, 54.598150033144236, 40.81233544283598, 236.23893989057657, 70.37979717263202, 87.58907276183992, 58.57526450222688, 118.3252868447224, 64.33234259239626, 57.21836996148928, 58.57526450222688, 126.94450214932196, 88.96839995507139, 108.5813867407896, 41.45503620813183, 95.0770390179677, 106.48123261036584, 84.563269020706, 46.246434773093995, 136.72460907194932, 157.98498549518746, 81.641993024771, 68.21443845541918, 67.41971617548266, 55.02636863114642, 60.908637772317434, 112.90677745995042, 77.59961765106212, 112.46659565031442, 90.36944839098756, 85.56007407666469, 80.69083499570341, 99.2514223056366, 112.90677745995042, 135.13171912803952, 143.28616006164455, 74.91890411002962, 91.07822475921115, 50.20003003478527, 52.609179777419676, 55.24173574865985, 56.33128135834348, 59.73055807527518, 70.65525591042105, 74.04607243930377, 173.5125619611118, 63.582848099201826, 100.81440478091984, 135.66060972280735, 91.07822475921115, 111.15632012308374, 74.91890411002962, 104.4216991379013, 162.99996726850102, 104.4216991379013, 64.83690740886694, 82.60436300370014, 164.27839185643572, 97.71267162761835, 87.24759532868651, 73.18340956555208, 149.57717793613415, 72.61389175398708, 156.14440466345445, 54.38529201854992, 150.7503262825534, 145.54258875480053, 88.96839995507139, 222.79453967267415, 196.6154911738957, 90.36944839098756, 70.65525591042105, 72.89809449073893, 31.598985005566618, 66.37447035867858, 64.33234259239626, 81.00664999449546, 70.37979717263202, 148.99403178365867, 95.0770390179677, 75.21212841006654, 116.94675549762589, 66.11570082213345, 98.86447768050158, 123.52039078374908, 94.33714363029665, 90.36944839098756, 95.44916002730179, 186.15180560442604, 49.9067492938035, 74.62682298320091, 35.66674528344124, 58.80452160545304, 52.19977189231385, 104.01459751366006, 82.28231911219243, 148.4131591025766, 96.19777706766972, 155.5356553203632, 127.94014021950323, 127.94014021950323, 127.94014021950323, 154.92927926604594, 154.92927926604594, 154.92927926604594, 154.92927926604594, 139.96693973881514, 238.09178486008656, 200.49338426145144, 79.43983955226133, 73.75739416067401, 53.43765380953904, 51.591626129191596, 73.46984133199462, 154.32526724792663, 55.457945789238345, 72.89809449073893, 132.00139622598326, 70.37979717263202, 56.773093073800744, 49.9067492938035, 96.57428452170761, 148.99403178365867, 62.597087408320306, 158.60332124553608, 205.2479478525057, 104.4216991379013, 77.59961765106212, 103.60908302823987, 69.01852867010916, 79.13013296849392, 65.34542957624015, 88.96839995507139, 170.82249902639472, 97.71267162761835, 56.11166650505244, 64.08153455750669, 78.2082389954826, 61.867809250367884, 38.0412795852805, 70.65525591042105, 67.9484955630781, 105.65258908406659, 51.79355004464666, 96.57428452170761, 142.7275402635186, 69.55984769951485, 82.28231911219243, 69.28865955361954, 54.598150033144236, 96.57428452170761, 80.37625124290746, 76.99573263552006, 105.24068867470233, 84.2335880783952, 181.83960367209863, 111.59137365930344, 99.2514223056366, 101.6051019574268, 86.23113005463887, 110.29129477712178, 86.23113005463887, 72.33079701870139, 70.93179276322707, 53.85677052285177, 70.65525591042105, 133.5573869140851, 179.7211081206401, 181.13067824168084, 61.626609520407456, 154.92927926604594, 156.7555365841094, 104.83039411284926, 57.442316329639176, 65.09067188950593, 70.93179276322707, 64.08153455750669, 95.44916002730179, 218.48530319325408, 170.1565252164996, 80.06289393593435, 84.563269020706, 234.40051387501347, 82.60436300370014, 70.65525591042105, 69.28865955361954, 81.9615307508321, 150.1626064581185, 97.33172602285453, 122.55914747655109, 97.71267162761835, 98.86447768050158, 210.11526266986428, 135.66060972280735, 148.4131591025766, 73.46984133199462, 49.42175070681059, 116.03666862607915, 141.61682573322432, 91.07822475921115, 104.01459751366006, 44.30137101535166, 103.60908302823987, 82.60436300370014, 127.44134888284603, 152.5273234252381, 117.86398026897247, 232.57639460420748, 55.89290784893326, 31.10908815096766, 55.24173574865985, 68.21443845541918, 61.626609520407456, 72.61389175398708, 97.71267162761835, 164.27839185643572, 162.3644906197382, 74.04607243930377, 55.89290784893326, 64.83690740886694, 77.59961765106212, 72.61389175398708, 80.69083499570341, 88.27604237373367, 141.61682573322432, 135.13171912803952, 40.49473132867618, 81.641993024771, 29.053548433146837, 66.89505178172818, 51.09025374715126, 46.246434773093995, 50.5937537501147, 98.4790416094955, 107.7363996853844, 80.37625124290746, 67.41971617548266, 51.591626129191596, 62.597087408320306, 105.65258908406659, 86.90744918987926, 53.43765380953904, 66.63425269055388, 84.2335880783952, 115.13366410675533, 111.59137365930344, 48.37128004023317, 35.59715165620933, 83.25223705241488, 45.263456176258764, 73.46984133199462, 151.3403463773431, 67.9484955630781, 53.02179868836547, 58.119428177447986, 57.89284200037607, 55.6750020519895, 52.40407602298735, 99.63988138921447, 74.91890411002962, 61.38635013907805, 32.28521466114651, 35.18243227767251, 64.33234259239626, 58.34690118774256, 84.89424029774196, 83.90519244027122, 56.773093073800744, 63.08804244970428, 77.29708541258049, 126.94450214932196, 172.1622764913071, 124.48917321564447, 183.26578935968087, 161.10096012939127, 121.13129015902297, 72.89809449073893, 105.65258908406659, 61.14702744030778, 67.41971617548266, 65.09067188950593, 109.43300111180854, 93.2380826167318, 56.773093073800744, 69.83209724580306, 51.591626129191596, 78.51433738825796, 74.33588057277046, 128.44088377025506, 93.2380826167318, 80.69083499570341, 33.4404292647954, 84.89424029774196, 191.3121237449989, 173.5125619611118, 155.5356553203632, 92.1518260518184, 56.9952966774873, 58.80452160545304, 86.56862915519338, 50.20003003478527, 55.457945789238345, 200.49338426145144, 87.93188669987823, 179.7211081206401, 91.43469485205186, 33.63694444585419, 173.5125619611118, 140.51475485198003, 56.773093073800744, 75.21212841006654, 159.2240770935855, 219.3404304920078, 32.28521466114651, 102.00277308269969, 138.33627549846275, 59.49769069799599, 87.93188669987823, 67.41971617548266, 56.9952966774873, 75.21212841006654, 65.09067188950593, 59.03467599561125, 60.67117749756472, 55.457945789238345, 54.598150033144236, 33.90076227087091, 47.06654728734682, 112.46659565031442, 57.21836996148928, 55.89290784893326, 60.908637772317434, 42.854578979512624, 56.773093073800744, 46.60915005647103, 42.190180314697045, 59.964336869699544, 72.04880596519355, 67.1568716116849, 23.073202403172917, 67.68358948381629, 39.32557222442699, 48.56059987746486, 48.27689708476307, 70.93179276322707, 134.60489048229144, 65.09067188950593, 76.09870517958821, 71.76791429061606, 75.80202444429054, 64.08153455750669, 70.37979717263202, 61.626609520407456, 73.46984133199462, 65.8579401324037, 57.442316329639176, 63.08804244970428, 69.83209724580306, 56.773093073800744, 69.83209724580306, 69.83209724580306, 72.89809449073893, 91.79256012938836, 71.20941195067294, 72.04880596519355, 57.21836996148928, 47.99485175252902, 53.646802873050504, 53.646802873050504, 77.29708541258049, 36.65546388385301, 55.02636863114642, 35.18243227767251, 40.73270164287187, 92.51249810131188, 45.88654216244616, 25.94189877662824, 66.11570082213345, 66.37447035867858, 53.333385249853684, 97.33172602285453, 134.60489048229144, 65.34542957624015, 39.32557222442699, 43.5294404008871, 56.773093073800744, 36.798929313140206, 81.32370105824417, 75.50650035756513, 66.11570082213345, 47.250760544967605, 58.119428177447986, 110.7229626998625, 112.46659565031442, 62.84208548133934, 90.36944839098756, 178.32250736130783, 144.4099673433467, 126.44959243764188, 95.82273747708687, 68.21443845541918, 56.9952966774873, 76.39654709045266, 76.39654709045266, 50.1020787877761, 88.96839995507139, 81.00664999449546, 69.28865955361954, 51.39048944047805, 139.42126035437516, 98.86447768050158, 161.73149145963927, 92.87458178129451, 73.46984133199462, 66.63425269055388, 82.92766733936132, 71.48811770889668, 98.4790416094955, 85.89494673838159, 60.67117749756472, 96.57428452170761, 114.23768682435218, 77.59961765106212, 88.96839995507139, 52.50652775191775, 77.90333396724647, 61.14702744030778, 82.28231911219243, 106.48123261036584, 36.798929313140206, 50.6926662381911, 53.85677052285177, 69.01852867010916, 61.626609520407456, 89.31661242710538, 66.63425269055388, 185.4260684919039, 172.1622764913071, 56.33128135834348, 71.48811770889668, 70.93179276322707, 38.56494680401705, 46.15619785585283, 49.22907356066477, 44.73612093895834, 39.479488162280596, 57.442316329639176, 65.34542957624015, 42.68750480955783, 40.33685733281881, 53.02179868836547, 49.32531805312982, 45.79700747446617, 51.99626426796157, 72.89809449073893, 61.867809250367884, 43.444504934395866, 119.72006788311512, 54.27917440928786, 70.65525591042105, 34.56939489431364, 75.50650035756513, 75.50650035756513, 75.50650035756513, 56.773093073800744, 56.33128135834348, 69.83209724580306, 145.54258875480053, 71.76791429061606, 64.08153455750669, 91.07822475921115, 110.7229626998625, 96.57428452170761, 112.02812994692015, 79.43983955226133, 32.22221914035047, 88.96839995507139, 77.29708541258049, 106.48123261036584, 80.37625124290746, 70.10541234668786, 51.09025374715126, 96.19777706766972, 47.807737558209425, 58.34690118774256, 36.37020880080945, 58.80452160545304, 56.9952966774873, 78.2082389954826, 54.811841150452466, 30.74665629604239, 34.70469568042362, 37.89297069129353, 51.79355004464666, 60.199030648454944, 53.75168417471723, 81.9615307508321, 91.07822475921115, 31.598985005566618, 67.9484955630781, 82.28231911219243, 48.182698291098816, 88.27604237373367, 77.59961765106212, 69.55984769951485, 62.597087408320306, 86.23113005463887, 81.9615307508321, 194.3248513188351, 77.59961765106212, 82.28231911219243, 31.9714638416083, 85.2265069597304, 95.0770390179677, 47.250760544967605, 107.7363996853844, 42.854578979512624, 39.172256347947744, 68.21443845541918, 53.02179868836547, 75.80202444429054, 166.21485355527525, 125.9566121960733, 132.51803508331912, 44.91121316574036, 31.169907463288272, 50.29817277977263, 33.83461446305701, 77.29708541258049, 35.45837151398557, 50.29817277977263, 45.61846186294896, 42.68750480955783, 58.80452160545304, 41.617286632367644, 49.42175070681059, 67.41971617548266, 34.908640417532865, 36.72712654703974, 48.27689708476307, 41.698649825700315, 32.9863470466753, 42.854578979512624, 39.01953819342525, 37.524723159601, 47.99485175252902, 114.23768682435218, 33.05083646319603, 78.82163381626928, 44.387981683814154, 50.990565478815945, 51.1901369093998, 57.66713919909174, 37.524723159601, 64.33234259239626, 62.597087408320306, 114.68480048885151, 67.1568716116849, 39.248839424943355, 25.094689498803355, 81.32370105824417, 59.73055807527518, 109.86130976812342, 87.24759532868651, 54.811841150452466, 52.301824199806006, 62.10995300937982, 62.10995300937982, 72.61389175398708, 206.05126560732333, 45.351947753528236, 80.37625124290746, 80.69083499570341, 85.2265069597304, 127.94014021950323, 95.0770390179677, 68.21443845541918, 83.25223705241488, 75.50650035756513, 75.50650035756513, 96.19777706766972, 66.11570082213345, 70.10541234668786, 60.67117749756472, 60.67117749756472, 60.67117749756472, 55.457945789238345, 69.28865955361954, 94.70636877096143, 67.9484955630781, 67.9484955630781, 67.9484955630781, 67.9484955630781, 67.9484955630781, 67.9484955630781, 65.60118435636834, 71.48811770889668, 71.48811770889668, 47.99485175252902, 88.62154503481175, 88.62154503481175, 88.62154503481175, 77.29708541258049, 45.351947753528236, 43.699809813576366, 96.95226558425567, 97.33172602285453, 64.58413226217957, 116.94675549762589, 87.58907276183992, 87.58907276183992, 79.43983955226133, 94.33714363029665, 61.38635013907805, 56.55175575986736, 57.66713919909174, 80.69083499570341, 63.83170433047907, 63.83170433047907, 63.83170433047907, 28.1047190510635, 50.1020787877761, 72.04880596519355, 89.31661242710538, 70.65525591042105, 70.65525591042105, 70.65525591042105, 89.66618776422116, 62.84208548133934, 68.48142221881122, 68.48142221881122, 66.89505178172818, 40.892124929554, 59.73055807527518, 64.58413226217957, 64.58413226217957, 64.58413226217957, 64.58413226217957, 64.58413226217957, 64.58413226217957, 64.58413226217957, 64.58413226217957, 64.58413226217957, 64.58413226217957, 64.58413226217957, 55.457945789238345, 55.457945789238345, 55.457945789238345, 55.457945789238345, 51.79355004464666, 101.6051019574268, 86.90744918987926, 93.96935796203766, 114.23768682435218, 67.41971617548266, 67.41971617548266, 67.41971617548266, 67.41971617548266, 67.41971617548266, 67.41971617548266, 67.41971617548266, 67.41971617548266, 67.41971617548266, 67.41971617548266, 67.41971617548266, 67.41971617548266, 45.7076474882609, 47.52843316962603, 44.04255166984753, 84.89424029774196, 84.89424029774196, 84.89424029774196, 75.50650035756513, 71.76791429061606, 71.76791429061606, 71.76791429061606, 71.76791429061606, 71.76791429061606, 89.66618776422116, 60.908637772317434, 45.88654216244616, 61.626609520407456, 61.626609520407456, 61.626609520407456, 85.56007407666469, 77.59961765106212, 77.59961765106212, 81.641993024771, 81.641993024771, 72.89809449073893, 89.31661242710538, 84.563269020706, 69.83209724580306, 70.65525591042105, 70.65525591042105, 73.75739416067401, 77.59961765106212, 60.67117749756472, 124.48917321564447, 76.69555472159655, 76.69555472159655, 75.50650035756513, 61.626609520407456, 74.91890411002962, 74.91890411002962, 74.91890411002962, 74.91890411002962, 61.626609520407456, 64.58413226217957, 95.44916002730179, 70.65525591042105, 59.73055807527518, 77.29708541258049, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 66.89505178172818, 37.37842812826559, 48.37128004023317, 48.08868329990042, 45.7076474882609, 42.190180314697045, 45.17513726542669, 56.55175575986736, 56.11166650505244, 56.11166650505244, 56.11166650505244, 56.11166650505244, 63.3349620664254, 81.32370105824417, 45.7076474882609, 88.96839995507139, 52.097918711212095, 83.57807709540792, 66.11570082213345, 137.25973406168575, 88.96839995507139, 86.56862915519338, 79.75075829332496, 79.75075829332496, 67.1568716116849, 54.811841150452466, 57.89284200037607, 61.38635013907805, 73.18340956555208, 77.29708541258049, 82.28231911219243, 93.96935796203766, 83.90519244027122, 89.31661242710538, 91.43469485205186, 91.43469485205186, 91.43469485205186, 91.43469485205186, 91.43469485205186, 42.93836112094585, 53.333385249853684, 80.37625124290746, 80.37625124290746, 80.37625124290746, 64.08153455750669, 65.60118435636834, 51.99626426796157, 38.867415430562446, 31.291903031786116, 55.457945789238345, 97.33172602285453, 62.597087408320306, 27.346642204097876, 31.72265993608251, 40.49473132867618, 43.78524440968234, 80.06289393593435, 100.81440478091984, 48.94146538270724, 53.333385249853684, 35.59715165620933, 71.48811770889668, 65.34542957624015, 45.97625189374871, 58.119428177447986, 71.76791429061606, 65.8579401324037, 49.22907356066477, 46.066137010589394, 46.246434773093995, 82.60436300370014, 78.51433738825796, 96.19777706766972, 131.48677155266952, 63.83170433047907, 82.60436300370014, 82.60436300370014, 63.83170433047907, 40.022953418667065, 178.32250736130783, 172.1622764913071, 76.99573263552006, 61.14702744030778, 105.65258908406659, 41.05217218133816, 41.780172086809024, 57.66713919909174, 144.97517197488295, 81.32370105824417, 105.24068867470233, 43.3597351955915, 54.27917440928786, 39.479488162280596, 202.06587336346374, 171.49107938408403, 61.626609520407456, 72.04880596519355, 56.33128135834348, 110.29129477712178, 184.70316076024525, 70.93179276322707, 69.01852867010916, 64.33234259239626, 139.42126035437516, 44.99901609609206, 66.37447035867858, 167.51849280886478, 148.99403178365867, 66.89505178172818, 168.17414165184545, 32.66578255521719, 74.33588057277046, 28.214717811314713, 33.50580642730562, 135.66060972280735, 90.7231444115574, 38.79157659527073, 63.3349620664254, 60.199030648454944, 52.81508628524973, 81.32370105824417, 57.66713919909174, 76.39654709045266, 48.56059987746486, 56.773093073800744, 55.89290784893326, 36.72712654703974, 73.46984133199462, 77.59961765106212, 48.94146538270724, 68.7494509271083, 85.2265069597304, 115.13366410675533, 88.27604237373367, 65.8579401324037, 46.15619785585283, 123.52039078374908, 122.55914747655109, 59.964336869699544, 80.69083499570341, 47.250760544967605, 82.28231911219243, 61.38635013907805, 95.44916002730179, 55.24173574865985, 72.04880596519355, 83.25223705241488, 52.9183415530491, 67.9484955630781, 98.4790416094955, 34.84052601883783, 109.86130976812342, 66.63425269055388, 46.97471015107385, 193.56725052076862, 55.457945789238345, 32.79363281328884, 46.246434773093995, 54.067559962798995, 48.46584751755162, 54.811841150452466, 141.61682573322432, 68.7494509271083, 60.908637772317434, 22.319678873017494, 45.44061233480379, 73.18340956555208, 141.0647140528757, 124.48917321564447, 92.87458178129451, 62.3530444922685, 63.83170433047907, 68.48142221881122, 76.69555472159655, 79.13013296849392, 87.93188669987823, 53.646802873050504, 64.08153455750669, 60.199030648454944, 93.96935796203766, 72.04880596519355, 116.03666862607915, 98.4790416094955, 98.4790416094955, 47.435694792893365, 87.58907276183992, 87.24759532868651, 102.00277308269969, 66.89505178172818, 69.28865955361954, 83.57807709540792, 38.94340253332466, 141.0647140528757, 53.22932014095188, 65.60118435636834, 63.83170433047907, 81.32370105824417, 87.24759532868651, 47.34313736903344, 62.3530444922685, 43.78524440968234, 47.90120329104557, 55.24173574865985, 66.37447035867858, 48.08868329990042, 71.76791429061606, 39.556671887793456, 53.02179868836547, 74.04607243930377, 25.790339917193062, 33.83461446305701, 34.63697922255266, 78.82163381626928, 69.83209724580306, 43.87084603329354, 64.83690740886694, 59.964336869699544, 62.10995300937982, 27.133828946090556, 46.97471015107385, 73.46984133199462, 44.99901609609206, 38.115651564714454, 39.172256347947744, 53.542126217761066, 66.63425269055388, 115.13366410675533, 58.119428177447986, 62.10995300937982, 47.90120329104557, 61.14702744030778, 42.854578979512624, 42.93836112094585, 53.333385249853684, 50.004318665090636, 59.265731184583274, 59.03467599561125, 59.964336869699544, 59.964336869699544, 60.908637772317434, 36.51255777326842, 48.65553748142455, 38.64034257045414, 62.3530444922685, 49.51837188956857, 52.81508628524973, 51.1901369093998, 96.57428452170761, 114.23768682435218, 33.96703939987107, 70.10541234668786, 70.93179276322707, 124.97641006343288, 45.351947753528236, 108.5813867407896, 70.65525591042105, 136.72460907194932, 48.75066069158903, 39.634006510082365, 93.96935796203766, 66.89505178172818, 48.75066069158903, 78.82163381626928, 141.61682573322432, 222.79453967267415, 96.19777706766972, 100.42136666458902, 72.61389175398708, 70.65525591042105, 56.33128135834348, 53.22932014095188, 76.39654709045266, 62.3530444922685, 42.93836112094585, 61.626609520407456, 38.56494680401705, 48.08868329990042, 124.48917321564447, 34.63697922255266, 69.83209724580306, 74.04607243930377, 90.7231444115574, 69.55984769951485, 91.43469485205186, 44.21492934326177, 125.9566121960733, 74.04607243930377, 168.17414165184545, 164.27839185643572, 68.7494509271083, 56.9952966774873, 39.32557222442699, 40.17959882934157, 63.08804244970428, 98.09510821132314, 81.9615307508321, 44.64883097460398, 59.49769069799599, 41.29341833835342, 111.59137365930344, 93.6030061542134, 62.10995300937982, 81.641993024771, 107.7363996853844, 66.63425269055388, 57.66713919909174, 57.66713919909174, 47.807737558209425, 47.90120329104557, 57.66713919909174, 66.63425269055388, 66.11570082213345, 64.33234259239626, 67.1568716116849, 96.95226558425567, 121.13129015902297, 134.60489048229144, 53.646802873050504, 74.33588057277046, 87.93188669987823, 52.9183415530491, 75.80202444429054, 75.21212841006654, 84.563269020706, 73.18340956555208, 45.79700747446617, 45.79700747446617, 100.02986085865793, 45.79700747446617, 34.23344619169951, 347.77735828917787, 98.86447768050158, 52.301824199806006, 109.00636227269737, 67.1568716116849, 31.230845679407448, 65.34542957624015, 185.4260684919039, 42.3553080116137, 84.563269020706, 236.23893989057657, 90.36944839098756, 56.9952966774873, 74.62682298320091, 52.097918711212095, 63.83170433047907, 49.22907356066477, 35.18243227767251, 55.6750020519895, 48.46584751755162, 72.04880596519355, 49.71218131735948, 60.199030648454944, 48.27689708476307, 57.442316329639176, 66.89505178172818, 49.9067492938035, 72.89809449073893, 51.09025374715126, 84.2335880783952, 110.29129477712178, 41.2128458403547, 75.50650035756513, 131.48677155266952, 76.39654709045266, 52.81508628524973, 74.33588057277046, 97.33172602285453, 73.75739416067401, 79.43983955226133, 103.20514949397284, 119.72006788311512, 81.32370105824417, 72.89809449073893, 38.48969815122384, 54.811841150452466, 45.529450258314064, 43.61454191906839, 52.50652775191775, 69.83209724580306, 129.9549042642197, 56.773093073800744, 64.33234259239626, 61.626609520407456, 63.08804244970428, 83.25223705241488, 109.00636227269737, 41.617286632367644, 67.41971617548266, 69.55984769951485, 37.74523999888851, 92.1518260518184, 50.49503426195374, 62.3530444922685, 78.2082389954826, 51.591626129191596, 103.60908302823987, 48.27689708476307, 98.4790416094955, 92.1518260518184, 39.479488162280596, 65.34542957624015, 60.908637772317434, 82.28231911219243, 107.7363996853844, 99.63988138921447, 58.80452160545304, 100.42136666458902, 95.82273747708687, 105.65258908406659, 88.96839995507139, 168.83235663184237, 199.71173463253027, 59.73055807527518, 81.00664999449546, 172.83610058973207, 162.99996726850102, 41.53608219643496, 72.61389175398708, 199.71173463253027, 140.51475485198003, 141.61682573322432, 254.43935071813848, 60.67117749756472, 74.91890411002962, 66.63425269055388, 55.89290784893326, 33.57131140436464, 63.83170433047907, 52.81508628524973, 112.46659565031442, 80.06289393593435, 51.290215346585704, 68.21443845541918, 32.22221914035047, 65.34542957624015, 39.7114923241556, 29.684499285876434, 47.71445419747699, 40.57389988868238, 34.4346215959791, 39.86691871057298, 78.51433738825796, 111.15632012308374, 26.817710263716034, 76.39654709045266, 31.35307975333966, 60.908637772317434, 59.03467599561125, 74.33588057277046, 43.19069160819034, 75.50650035756513, 115.13366410675533, 42.93836112094585, 43.61454191906839, 62.10995300937982, 30.447857334592324, 34.09998263323346, 74.62682298320091, 37.74523999888851, 43.19069160819034, 44.99901609609206, 39.248839424943355, 62.84208548133934, 45.351947753528236, 75.80202444429054, 75.21212841006654, 44.64883097460398, 31.291903031786116, 35.66674528344124, 42.93836112094585, 50.990565478815945, 50.990565478815945, 42.68750480955783, 46.066137010589394, 59.265731184583274, 47.435694792893365, 29.626578330054702, 31.41437607743883, 128.44088377025506, 70.93179276322707, 49.71218131735948, 51.99626426796157, 84.563269020706, 88.27604237373367, 48.75066069158903, 70.93179276322707, 66.89505178172818, 55.24173574865985, 56.773093073800744, 64.58413226217957, 68.21443845541918, 66.37447035867858, 65.34542957624015, 74.33588057277046, 67.41971617548266, 55.89290784893326, 68.7494509271083, 104.01459751366006, 53.333385249853684, 72.33079701870139, 64.33234259239626, 96.19777706766972, 73.46984133199462, 53.85677052285177, 63.83170433047907, 172.83610058973207, 73.46984133199462, 52.097918711212095, 51.09025374715126, 152.5273234252381, 122.55914747655109, 60.434642992688794, 51.894808174781225, 27.72308958496738, 20.521731031400577, 35.11378365112735, 124.48917321564447, 80.69083499570341, 45.529450258314064, 62.3530444922685, 72.61389175398708, 50.396507397123145, 63.3349620664254, 24.086237637543274, 80.37625124290746, 102.00277308269969, 163.63793110254423, 98.4790416094955, 144.4099673433467, 129.44825810694724, 98.09510821132314, 81.641993024771, 157.98498549518746, 51.99626426796157, 94.70636877096143, 86.23113005463887, 132.51803508331912, 84.2335880783952, 92.87458178129451, 70.37979717263202, 44.82358155812517, 78.51433738825796, 90.36944839098756, 90.36944839098756, 47.06654728734682, 91.43469485205186, 27.240027749911206, 44.82358155812517, 46.15619785585283, 27.186876493132687, 91.07822475921115, 54.38529201854992, 76.99573263552006, 155.5356553203632, 75.80202444429054, 60.67117749756472, 34.16664924473404, 61.14702744030778, 51.591626129191596, 98.4790416094955, 51.79355004464666, 50.1020787877761, 138.33627549846275, 62.10995300937982, 89.66618776422116, 58.34690118774256, 95.0770390179677, 125.4655539023251, 85.56007407666469, 103.20514949397284, 43.3597351955915, 69.01852867010916, 93.2380826167318, 48.182698291098816, 111.59137365930344, 42.3553080116137, 50.49503426195374, 65.60118435636834, 36.15772522700737, 59.265731184583274, 69.83209724580306, 62.597087408320306, 50.1020787877761, 89.31661242710538, 82.28231911219243, 83.90519244027122, 65.09067188950593, 151.93267574549122, 88.62154503481175, 45.7076474882609, 134.60489048229144, 40.892124929554, 38.64034257045414, 74.33588057277046, 37.9670527218579, 70.37979717263202, 35.52769382137767, 82.60436300370014, 77.90333396724647, 46.33684810653952, 124.48917321564447, 108.15806803146498, 59.265731184583274, 80.37625124290746, 115.13366410675533, 73.18340956555208, 103.60908302823987, 74.04607243930377, 33.4404292647954, 107.7363996853844, 88.96839995507139, 290.5788592888401, 106.06610162605153, 138.8777083722429, 115.13366410675533, 125.4655539023251, 75.50650035756513, 227.18876822599648, 51.49095957359226, 69.01852867010916, 57.21836996148928, 75.21212841006654, 136.72460907194932, 45.086990684122, 83.90519244027122, 76.09870517958821, 115.58428452718766, 51.1901369093998, 163.63793110254423, 103.60908302823987, 71.48811770889668, 81.00664999449546, 103.20514949397284, 77.59961765106212, 102.40200064874995, 64.58413226217957, 45.263456176258764, 87.58907276183992, 43.78524440968234, 49.037147591523336, 80.37625124290746, 76.39654709045266, 96.95226558425567, 89.66618776422116, 124.00383592430612, 98.4790416094955, 65.60118435636834, 180.4245166495895, 86.23113005463887, 64.33234259239626, 186.15180560442604, 87.24759532868651, 55.02636863114642, 72.89809449073893, 51.39048944047805, 98.86447768050158, 65.09067188950593, 142.7275402635186, 65.8579401324037, 49.9067492938035, 77.29708541258049, 168.17414165184545, 147.834551029463, 89.66618776422116, 227.18876822599648, 131.48677155266952, 98.86447768050158, 69.83209724580306, 48.65553748142455, 77.90333396724647, 81.9615307508321, 111.15632012308374, 102.80279074731456, 112.90677745995042, 97.33172602285453, 296.31001375229687, 48.65553748142455, 77.59961765106212, 161.10096012939127, 200.49338426145144, 189.08326999196993, 92.51249810131188, 77.59961765106212, 84.2335880783952, 166.86540009848025, 157.98498549518746, 156.7555365841094, 194.3248513188351, 109.43300111180854, 93.2380826167318, 123.03883041717654, 104.83039411284926, 102.40200064874995, 85.89494673838159, 153.7236100495016, 95.44916002730179, 81.9615307508321, 79.75075829332496, 79.75075829332496, 177.62729378994385, 77.90333396724647, 187.61181231093738, 177.62729378994385, 297.46973835596765, 143.8469662358426, 198.15756558346607, 198.15756558346607, 92.1518260518184, 130.4635333784854, 146.6840934260825, 209.29610089202527, 168.17414165184545, 70.37979717263202, 168.17414165184545, 71.48811770889668, 100.81440478091984, 51.09025374715126, 75.50650035756513, 95.0770390179677, 63.582848099201826, 100.02986085865793, 55.24173574865985, 118.78839892336089, 118.78839892336089, 198.15756558346607, 69.83209724580306, 109.43300111180854, 57.442316329639176, 36.15772522700737, 84.563269020706, 82.60436300370014, 66.37447035867858, 116.03666862607915, 59.03467599561125, 113.34868209247811, 75.21212841006654, 57.21836996148928, 69.83209724580306, 88.27604237373367, 42.77096031537661, 101.6051019574268, 115.13366410675533, 123.03883041717654, 103.20514949397284, 87.58907276183992, 155.5356553203632, 96.19777706766972, 219.3404304920078, 116.03666862607915, 83.90519244027122, 71.20941195067294, 58.119428177447986, 90.01713130052181, 225.42077120508256, 262.51612271179147, 76.69555472159655, 138.33627549846275, 166.21485355527525, 91.43469485205186, 281.63868765896893, 351.87684773779097, 114.23768682435218, 53.542126217761066, 95.44916002730179, 92.51249810131188, 234.40051387501347, 76.09870517958821, 172.1622764913071, 117.86398026897247, 61.38635013907805, 140.51475485198003, 91.43469485205186, 161.10096012939127, 57.89284200037607, 137.7969534714142, 119.2533235714264, 119.72006788311512, 337.0773738522583, 45.79700747446617, 81.00664999449546, 160.47288700783744, 70.10541234668786, 37.08754689283638, 125.4655539023251, 135.13171912803952, 173.5125619611118, 127.94014021950323, 164.9213593028435, 107.31637526839133, 80.06289393593435, 148.99403178365867, 102.40200064874995, 134.60489048229144, 77.90333396724647, 133.5573869140851, 179.02044192165667, 184.70316076024525, 159.8472625113303, 136.72460907194932, 81.641993024771, 90.01713130052181, 86.56862915519338, 114.23768682435218, 151.93267574549122, 151.93267574549122, 90.7231444115574, 106.06610162605153, 68.7494509271083, 83.25223705241488, 80.37625124290746, 120.65904401308495, 104.83039411284926, 75.50650035756513, 84.563269020706, 27.72308958496738, 95.82273747708687, 164.9213593028435, 106.06610162605153, 66.37447035867858, 67.41971617548266, 92.51249810131188, 126.94450214932196, 209.29610089202527, 59.265731184583274, 43.61454191906839, 98.86447768050158, 74.04607243930377, 76.39654709045266, 102.80279074731456, 294.00411093116844, 53.75168417471723, 97.33172602285453, 83.90519244027122, 143.28616006164455, 112.02812994692015, 128.44088377025506, 111.15632012308374, 125.9566121960733, 170.1565252164996, 71.48811770889668, 164.27839185643572, 86.90744918987926, 198.15756558346607, 148.4131591025766, 102.80279074731456, 53.02179868836547, 116.49082330627213, 156.7555365841094, 81.641993024771, 55.02636863114642, 166.21485355527525, 192.0608982264211, 81.32370105824417, 144.97517197488295, 51.99626426796157, 130.9741532108186, 94.70636877096143, 85.89494673838159, 60.434642992688794, 132.00139622598326, 112.90677745995042, 112.46659565031442, 69.01852867010916, 88.27604237373367, 86.56862915519338, 174.8734378511126, 45.97625189374871, 167.51849280886478, 138.8777083722429, 75.80202444429054, 81.9615307508321, 99.63988138921447, 98.4790416094955, 114.23768682435218, 94.33714363029665, 142.1710983175922, 69.55984769951485, 81.9615307508321, 79.13013296849392, 138.33627549846275, 147.834551029463, 134.60489048229144, 110.7229626998625, 120.65904401308495, 94.33714363029665, 76.09870517958821, 76.09870517958821, 144.97517197488295, 89.31661242710538, 134.60489048229144, 59.49769069799599, 92.1518260518184, 67.1568716116849, 208.48013272330223, 176.93479059943397, 87.24759532868651, 79.13013296849392, 74.62682298320091, 146.6840934260825, 137.7969534714142, 154.92927926604594, 92.1518260518184, 93.2380826167318, 116.49082330627213, 194.3248513188351, 87.93188669987823, 179.02044192165667, 104.01459751366006, 164.27839185643572, 164.9213593028435, 198.9331323667161, 217.63350971989422, 110.7229626998625, 91.43469485205186, 128.44088377025506, 163.63793110254423, 68.7494509271083, 74.91890411002962, 96.95226558425567, 95.44916002730179, 114.68480048885151, 143.8469662358426, 156.7555365841094, 265.61057969554065, 225.42077120508256, 176.24498722300464, 370.20707083592094, 147.25819873544805, 112.46659565031442, 76.99573263552006, 144.4099673433467, 63.582848099201826, 195.08541728258788, 55.02636863114642, 67.9484955630781, 133.5573869140851, 87.24759532868651, 109.86130976812342, 99.63988138921447, 302.15420510895507, 65.34542957624015, 79.13013296849392, 92.51249810131188, 96.95226558425567, 48.84596987082467, 91.43469485205186, 58.57526450222688, 51.49095957359226, 90.36944839098756, 49.22907356066477, 164.9213593028435, 109.86130976812342, 134.60489048229144, 63.08804244970428, 84.89424029774196, 131.48677155266952, 121.60538462411665, 64.58413226217957, 68.48142221881122, 63.83170433047907, 196.6154911738957, 65.09067188950593, 63.582848099201826, 90.7231444115574, 119.2533235714264, 74.04607243930377, 80.37625124290746, 72.33079701870139, 63.582848099201826, 85.2265069597304, 153.7236100495016, 166.21485355527525, 92.1518260518184, 76.99573263552006, 124.00383592430612, 151.93267574549122, 59.265731184583274, 124.48917321564447, 94.70636877096143, 98.86447768050158, 76.09870517958821, 77.29708541258049, 129.44825810694724, 63.3349620664254, 65.60118435636834, 69.83209724580306, 76.99573263552006, 86.56862915519338, 134.08011574678568, 329.26898489707213, 166.86540009848025, 135.13171912803952, 154.92927926604594, 126.94450214932196, 140.51475485198003, 100.81440478091984, 54.811841150452466, 52.301824199806006, 62.597087408320306, 60.199030648454944, 127.94014021950323, 80.06289393593435, 190.56626845863, 177.62729378994385, 155.5356553203632, 46.7915731119465, 207.6673457129932, 164.9213593028435, 150.7503262825534, 141.0647140528757, 90.7231444115574, 118.78839892336089, 178.32250736130783, 188.3461041828892, 235.3179315471721, 276.1913023296018, 132.51803508331912, 96.19777706766972, 46.427438201089, 98.4790416094955, 84.89424029774196, 129.9549042642197, 108.15806803146498, 62.597087408320306, 88.62154503481175, 94.70636877096143, 156.7555365841094, 82.60436300370014, 93.96935796203766, 154.32526724792663, 77.29708541258049, 65.09067188950593, 90.36944839098756, 116.94675549762589, 128.94358717585138, 108.5813867407896, 57.21836996148928, 188.3461041828892, 161.73149145963927, 95.0770390179677, 116.03666862607915, 82.28231911219243, 99.2514223056366, 240.8983354780937, 86.56862915519338, 126.44959243764188, 131.48677155266952, 127.94014021950323, 58.57526450222688, 174.8734378511126, 111.15632012308374, 150.7503262825534, 215.9398723106141, 206.85772745893686, 294.00411093116844, 198.9331323667161, 139.96693973881514, 101.20898120494374, 193.56725052076862, 96.95226558425567, 174.8734378511126, 195.84896001735729, 89.31661242710538, 108.15806803146498, 133.5573869140851, 110.7229626998625, 92.51249810131188, 95.44916002730179, 90.7231444115574, 96.95226558425567, 65.60118435636834, 45.88654216244616, 195.08541728258788, 211.76321709954237, 77.90333396724647, 72.04880596519355, 166.21485355527525, 109.43300111180854, 66.89505178172818, 176.93479059943397, 222.79453967267415, 177.62729378994385, 121.60538462411665, 72.33079701870139, 180.4245166495895, 70.93179276322707, 151.93267574549122, 102.40200064874995, 20.322298876557472, 83.25223705241488, 14.694892728788941, 22.67116267165818, 22.67116267165818, 35.45837151398557, 34.97688798216538, 30.806767041912167, 27.4536739354601, 23.620367629992042, 19.96819234893607, 14.049378009312553, 42.02569638966092, 55.89290784893326, 46.51820540231633, 36.44131378072682, 19.812798643245316, 26.817710263716034, 21.464625403032095, 16.88071548801052, 13.85860376179437, 24.086237637543274, 45.88654216244616, 44.387981683814154, 23.52828054362271, 22.27612829392538, 23.16350836406023, 14.89717913158109, 13.198169226352599, 34.03344610288473, 26.196478674560588, 17.382583384208317, 21.675267738479917, 21.380942759123343, 14.242778409807016, 29.626578330054702, 27.83159468972004, 30.74665629604239, 36.72712654703974, 39.556671887793456, 20.521731031400577, 14.694892728788941, 22.626926270737744, 16.58657693697683, 25.63966650134361, 28.883810562338418, 14.666219776472907, 17.416566918622852, 26.35042423874788, 14.723621737689907, 24.465540688808968, 13.249825400173327, 27.72308958496738, 17.690837939037834, 36.65546388385301, 23.898797049169467, 20.56185170491249, 49.037147591523336, 35.11378365112735, 38.94340253332466, 27.668995768810042, 23.028181534732802, 32.85774546700983, 14.326477161274006, 27.995149135337257, 37.524723159601, 25.2915095879081, 25.840760898054555, 27.83159468972004, 10.835334427562556, 11.244980766441616, 13.146714440868186, 40.41571724385996, 12.374340325455691, 18.79505399780919, 13.85860376179437, 28.380523708580828, 29.22428378123494, 17.484733434196265, 22.319678873017494, 26.609013186724894, 17.76007792103387, 17.416566918622852, 17.6218678981495, 22.232662691545976, 20.007230835729846, 20.68268496272443, 13.912844803302042, 25.045724266551254, 23.390822626823073, 11.511647800524438, 13.564035954639001, 22.27612829392538, 20.521731031400577, 13.249825400173327, 12.471393479493909, 24.370158584949497, 28.15966472087403, 20.8042283062039, 18.323843311697065, 24.275148340672132, 18.942465433314446, 19.053782246709744, 13.940044882238906, 17.690837939037834, 16.58657693697683, 21.464625403032095, 12.302046430466076, 24.80232751057381, 23.118311288923124, 34.03344610288473, 13.537569551425428, 18.942465433314446, 10.877742727456024, 14.021964597512564, 11.489186055851967, 22.715485556228362, 15.520899957093135, 13.223972090529857, 18.721779026482544, 13.85860376179437, 13.275729253905588, 12.471393479493909, 13.069908149118522, 15.250455553989443, 27.507346790200362, 19.091033031037714, 34.56939489431364, 29.22428378123494, 16.554212894695087, 12.134998917818166, 13.697146137460072, 20.68268496272443, 14.104365720113616, 19.128356641900336, 20.24306974171993, 10.899009079882486, 16.139182322446413, 11.761644251483835, 19.24076585887165, 12.520205133467638, 13.537569551425428, 27.995149135337257, 11.807678025521156, 13.458479789399725, 10.079935612761957, 9.962500805803561, 15.250455553989443, 12.040563607027826, 17.180070153709277, 21.632974546568068, 14.75240691276831, 12.742232560259533, 16.457499171202052, 21.506589466713315, 15.490615283867873, 19.31607208722833, 10.751013186076355, 13.458479789399725, 13.940044882238906, 15.131775567308885, 13.432219349369987, 14.187251079068192, 23.52828054362271, 22.938403163153815, 41.05217218133816, 37.9670527218579, 20.602050815688813, 12.792104248423877, 15.765318566620541, 14.382549407491597, 25.740017318702005, 30.034410770075546, 26.35042423874788, 34.56939489431364, 19.128356641900336, 18.181246031735814, 29.511075247281195, 23.028181534732802, 23.852165264858517, 25.24216031657971, 17.656319241826594, 51.99626426796157, 31.048387510438282, 16.716667072215767, 29.511075247281195, 30.62678644311483, 27.186876493132687, 24.9968545761854, 17.28103011956753, 22.67116267165818, 16.716667072215767, 20.84490127137742, 14.666219776472907, 25.045724266551254, 16.74934871787156, 15.04337206739713, 21.845268970328604, 31.72265993608251, 16.170734966018113, 25.045724266551254, 32.47494113727571, 15.16135869683449, 23.80562446936611, 24.89940107578365, 13.59055410055989, 14.326477161274006, 12.302046430466076, 13.564035954639001, 21.131847230940878, 36.08717359043372, 25.58963789966816, 25.34095533871963, 21.297586362924953, 15.400115355370453, 26.453556883236853, 29.453492679719986, 25.689792910615846, 30.329152388418176, 16.01358620918395, 22.319678873017494, 18.83179895971962, 32.72964525738904, 33.11545195869231, 25.840760898054555, 22.715485556228362, 22.715485556228362, 34.300373728940066, 10.751013186076355, 19.658614220966218, 37.74523999888851, 20.007230835729846, 16.202349296067883, 17.416566918622852, 18.83179895971962, 25.740017318702005, 39.86691871057298, 17.013112580497378, 35.25121511458854, 24.133327041205547, 19.851533330124635, 19.278382202442366, 34.77254452624438, 17.79479946994832, 19.46756991825522, 22.67116267165818, 17.587483776585056, 29.22428378123494, 21.632974546568068, 22.582776184718522, 24.086237637543274, 41.53608219643496, 24.18050850623376, 27.293282919182133, 16.2975636122563, 19.053782246709744, 28.771203256660744, 23.118311288923124, 18.942465433314446, 27.186876493132687, 24.657426302639607, 39.7114923241556, 46.70027251047952, 42.27266353442275, 68.21443845541918, 45.17513726542669, 34.70469568042362, 21.464625403032095, 23.299630451611073, 23.80562446936611, 17.829588900657228, 47.250760544967605, 22.84897480427519, 39.556671887793456, 47.435694792893365, 25.094689498803355, 36.15772522700737, 36.51255777326842, 25.2915095879081, 44.47476167904283, 30.74665629604239, 14.95548504215564, 12.643071719773257, 15.190999662372045, 36.228414794400415, 18.039758452611142, 15.673213754973451, 27.40010580820802, 11.57929685891098, 13.223972090529857, 44.5617113320772, 15.888967492629165, 13.67042000714299, 28.436008585761353, 32.60204446315635, 25.489873356103875, 16.076261613516785, 18.90550453688997, 29.56877039069624, 18.00455893510183, 30.093129052017023, 32.09659661306934, 14.214987631491761, 37.45150421083523, 19.812798643245316, 21.297586362924953, 35.59715165620933, 29.396022468352484, 38.0412795852805, 37.9670527218579, 17.656319241826594, 17.113091407702502, 21.887977204970085, 21.173160702117308, 18.61240185635776, 26.09434828302453, 19.505629671380603, 16.84777751703098, 18.216790978454146, 29.281418237700255, 18.83179895971962, 24.948080241282657, 60.434642992688794, 23.52828054362271, 50.89107172411317, 33.1801937796528, 51.99626426796157, 24.275148340672132, 16.04489330813532, 35.11378365112735, 23.898797049169467, 20.007230835729846, 43.02230705927991, 18.2524054167373, 29.74253347911242, 21.802644068801417, 40.81233544283598, 27.995149135337257, 37.74523999888851, 33.76859572409541, 37.5980852538711, 43.275130861102994, 24.322607071104724, 35.59715165620933, 29.56877039069624, 23.118311288923124, 47.52843316962603, 62.597087408320306, 60.908637772317434, 28.60311491479713, 46.60915005647103, 45.44061233480379, 29.33866439394716, 25.740017318702005, 55.89290784893326, 40.73270164287187, 45.351947753528236, 102.40200064874995, 27.77728915635237, 50.29817277977263, 70.93179276322707, 76.69555472159655, 29.22428378123494, 22.016603538363892, 22.893645317537526, 26.609013186724894, 16.94678474038272, 26.401940203271806, 22.27612829392538, 15.703855414343392, 36.228414794400415, 26.66103470132838, 21.760102337787384, 21.845268970328604, 23.574279122424027, 26.14536361045317, 15.703855414343392, 14.95548504215564, 31.291903031786116, 21.256030110048442, 18.216790978454146, 24.133327041205547, 25.992616060150944, 26.2990087931472, 19.658614220966218, 58.119428177447986, 36.51255777326842, 47.250760544967605, 49.22907356066477, 31.660762082694095, 32.9863470466753, 50.89107172411317, 54.811841150452466, 20.401838105461145, 23.898797049169467, 55.6750020519895, 22.189281900071105, 38.48969815122384, 20.322298876557472, 37.232703447205715, 23.574279122424027, 29.053548433146837, 27.40010580820802, 56.9952966774873, 38.33964103892552, 23.52828054362271, 34.84052601883783, 28.60311491479713, 25.689792910615846, 53.333385249853684, 59.964336869699544, 27.40010580820802, 70.93179276322707, 54.598150033144236, 78.51433738825796, 44.387981683814154, 31.90908051733889, 36.016759615546256, 60.908637772317434, 41.05217218133816, 27.346642204097876, 15.370066359341997, 26.2990087931472, 44.99901609609206, 25.2915095879081, 21.760102337787384, 21.33922385967176, 22.538712245181248, 17.864446345871784, 28.715064341305684, 16.425386987895497, 16.716667072215767, 40.81233544283598, 31.598985005566618, 16.651494963610144, 14.354485905365031, 41.698649825700315, 46.70027251047952, 26.043432497666576, 46.066137010589394, 26.609013186724894, 20.203571081597193, 21.131847230940878, 29.74253347911242, 39.01953819342525, 36.228414794400415, 18.979498589532987, 16.36135047271935, 20.8042283062039, 24.609314266596865, 33.63694444585419, 23.80562446936611, 41.37414835829223, 73.75739416067401, 25.891280453627324, 22.75989509352673, 43.699809813576366, 22.059646737841184, 20.56185170491249, 16.94678474038272, 28.940279382223952, 35.38918446958933, 56.9952966774873, 41.698649825700315, 24.609314266596865, 21.930768935644906, 16.91371785382974, 22.450842133517945, 31.846818916590372, 21.632974546568068, 15.796140293991145, 29.053548433146837, 23.94552000018515, 35.59715165620933, 18.110364070885183, 27.72308958496738, 42.60421214369869, 32.411575418595035, 26.817710263716034, 31.846818916590372, 42.02569638966092, 20.84490127137742, 34.501942437892716, 49.32531805312982, 44.64883097460398, 27.507346790200362, 30.74665629604239, 28.436008585761353, 21.548635571548648, 33.96703939987107, 28.049880592281973, 27.77728915635237, 11.311062736078481, 24.22778221261098, 35.45837151398557, 35.25121511458854, 40.57389988868238, 13.172416709211404, 15.85796463907484, 56.773093073800744, 33.50580642730562, 52.19977189231385, 18.145770440989054, 26.76538304079522, 41.780172086809024, 23.80562446936611, 27.83159468972004, 27.61500750152876, 31.846818916590372, 43.78524440968234, 30.388446900124176, 25.094689498803355, 44.47476167904283, 47.435694792893365, 21.802644068801417, 22.059646737841184, 18.145770440989054, 19.929230034775262, 26.87013978811538, 18.039758452611142, 23.666546242140512, 20.36202965350754, 45.351947753528236, 25.44013703364339, 17.934365811960372, 17.969428099556087, 27.240027749911206, 17.046373787079762, 17.079700020438295, 39.248839424943355, 41.29341833835342, 18.075026786359576, 20.046345644076645, 34.23344619169951, 26.557093177483523, 43.95661501095434, 78.82163381626928, 37.232703447205715, 33.31005738633129, 47.52843316962603, 31.048387510438282, 29.800681131145613, 37.160054292819765, 40.41571724385996, 23.4365524707797, 26.401940203271806, 25.539706914745466, 36.08717359043372, 23.94552000018515, 40.57389988868238, 35.18243227767251, 26.453556883236853, 19.429584428039178, 17.180070153709277, 18.503663694167408, 13.994604675279588, 16.107691244988956, 13.564035954639001, 19.735555861572347, 17.28103011956753, 22.232662691545976, 11.57929685891098, 18.2524054167373, 17.76007792103387, 14.076845015253456, 23.52828054362271, 21.632974546568068, 23.94552000018515, 12.158723248329283, 15.490615283867873, 19.543763832601815, 16.36135047271935, 16.457499171202052, 42.10785803778799, 18.110364070885183, 14.021964597512564, 13.777638440111984, 11.946863197656471, 27.72308958496738, 16.94678474038272, 13.85860376179437, 18.79505399780919, 17.213657767955564, 17.656319241826594, 10.040637668042795, 26.196478674560588, 18.075026786359576, 13.994604675279588, 19.96819234893607, 39.634006510082365, 25.689792910615846, 21.173160702117308, 21.131847230940878, 21.42274322042474, 19.128356641900336, 10.835334427562556, 25.689792910615846, 23.16350836406023, 20.36202965350754, 14.723621737689907, 16.425386987895497, 16.076261613516785, 18.075026786359576, 19.774139536218833, 23.390822626823073, 24.89940107578365, 29.800681131145613, 14.868111474214228, 26.87013978811538, 35.7364749685521, 33.90076227087091, 25.39049775763493, 22.98324851186175, 13.223972090529857, 16.94678474038272, 34.23344619169951, 16.202349296067883, 37.5980852538711, 36.65546388385301, 23.52828054362271, 13.804574186067095, 29.281418237700255, 33.4404292647954, 23.852165264858517, 15.072782365110845, 20.44172438427552, 16.782094257242417, 24.465540688808968, 29.56877039069624, 41.05217218133816, 34.97688798216538, 20.007230835729846, 65.09067188950593, 23.390822626823073, 20.723120306578686, 28.1047190510635, 41.05217218133816, 46.427438201089, 19.016604146815915, 36.58394105020859, 19.31607208722833, 20.92648590822237, 46.33684810653952, 51.49095957359226, 61.626609520407456, 43.10641711474324, 22.84897480427519, 52.301824199806006, 74.04607243930377, 51.79355004464666, 100.42136666458902, 39.94485987582193, 41.53608219643496, 24.657426302639607, 22.450842133517945, 34.4346215959791, 25.1437504597293, 18.942465433314446, 44.30137101535166, 36.37020880080945, 22.715485556228362, 16.23402543319486, 22.319678873017494, 18.323843311697065, 18.00455893510183, 20.56185170491249, 30.21091022897123, 40.73270164287187, 40.892124929554, 27.668995768810042, 25.192907336481866, 24.18050850623376, 23.666546242140512, 38.48969815122384, 47.52843316962603, 53.12545808585662, 23.4365524707797, 23.4365524707797, 21.632974546568068, 46.15619785585283, 38.0412795852805, 26.35042423874788, 26.401940203271806, 23.75917448515314, 25.1437504597293, 39.095822701298815, 23.75917448515314, 23.71281513502692, 53.02179868836547, 54.38529201854992, 22.10277408821932, 34.77254452624438, 32.033969127372174, 27.507346790200362, 27.40010580820802, 22.626926270737744, 22.016603538363892, 23.118311288923124, 19.053782246709744, 32.60204446315635, 30.68666283939559, 23.71281513502692, 22.189281900071105, 55.457945789238345, 49.51837188956857, 49.51837188956857, 33.96703939987107, 29.56877039069624, 37.67159077349981, 36.299242562272255, 45.44061233480379, 67.1568716116849, 44.21492934326177, 16.94678474038272, 19.353835657005096, 53.12545808585662, 30.093129052017023, 54.598150033144236, 89.66618776422116, 66.63425269055388, 29.281418237700255, 31.72265993608251, 41.05217218133816, 18.868615759264884, 24.513371642677637, 20.085536923187668, 24.417803063601138, 19.89034374461794, 32.159346537604755, 32.60204446315635, 47.250760544967605, 36.51255777326842, 41.132430556936306, 38.48969815122384, 38.48969815122384, 69.28865955361954, 70.93179276322707, 49.809370301716946, 56.11166650505244, 37.74523999888851, 23.620367629992042, 50.5937537501147, 17.587483776585056, 50.20003003478527, 31.598985005566618, 26.043432497666576, 71.48811770889668, 41.617286632367644, 31.41437607743883, 37.8190332109868, 25.39049775763493, 30.329152388418176, 30.507383918455513, 20.521731031400577, 29.453492679719986, 57.89284200037607, 49.71218131735948, 28.715064341305684, 30.74665629604239, 38.64034257045414, 29.281418237700255, 25.891280453627324, 43.87084603329354, 18.2524054167373, 27.995149135337257, 55.89290784893326, 21.973644325590612, 72.33079701870139, 36.58394105020859, 34.23344619169951, 24.417803063601138, 66.63425269055388, 22.98324851186175, 19.505629671380603, 17.51891667538985, 72.61389175398708, 41.132430556936306, 31.598985005566618, 40.97207040739876, 22.715485556228362, 41.2128458403547, 28.214717811314713, 16.457499171202052, 19.658614220966218, 19.505629671380603, 17.079700020438295, 24.417803063601138, 24.948080241282657, 24.753932740556596, 25.2915095879081, 25.992616060150944, 19.31607208722833, 23.80562446936611, 25.58963789966816, 21.717643615002626, 35.11378365112735, 34.03344610288473, 25.1437504597293, 24.753932740556596, 36.08717359043372, 29.453492679719986, 23.073202403172917, 27.61500750152876, 55.24173574865985, 43.699809813576366, 44.99901609609206, 26.922671813996086, 43.699809813576366, 44.47476167904283, 50.990565478815945, 38.64034257045414, 59.265731184583274, 52.71203249107963, 29.800681131145613, 26.609013186724894, 28.883810562338418, 20.24306974171993, 24.086237637543274, 21.930768935644906, 26.09434828302453, 30.034410770075546, 29.800681131145613, 34.97688798216538, 38.94340253332466, 54.067559962798995, 45.97625189374871, 16.84777751703098, 11.113972772314547, 15.01401915561195, 12.792104248423877, 48.56059987746486, 20.046345644076645, 16.23402543319486, 26.043432497666576, 12.111320878668954, 34.300373728940066, 33.57131140436464, 18.39556080667227, 19.429584428039178, 25.790339917193062, 25.58963789966816, 26.557093177483523, 27.080884906424416, 20.322298876557472, 27.668995768810042, 14.984723517782793, 20.723120306578686, 28.436008585761353, 30.987805310145493, 52.301824199806006, 25.24216031657971, 11.422063446614425, 22.67116267165818, 22.40703562619394, 32.72964525738904, 29.858942463791767, 20.124804822565828, 44.91121316574036, 30.806767041912167, 26.975306541751994, 21.760102337787384, 22.893645317537526, 37.160054292819765, 28.827451925481085, 45.263456176258764, 52.40407602298735, 76.69555472159655, 26.76538304079522, 35.87634357692165, 29.22428378123494, 33.96703939987107, 39.86691871057298, 19.391673055829287, 15.673213754973451, 14.81014620089916, 14.868111474214228, 33.05083646319603, 24.513371642677637, 27.133828946090556, 24.039240115615065, 48.182698291098816, 29.800681131145613, 20.521731031400577, 22.145985754016134, 21.506589466713315, 13.0444058974144, 18.323843311697065, 41.05217218133816, 16.651494963610144, 28.214717811314713, 22.626926270737744, 22.232662691545976, 25.992616060150944, 15.131775567308885, 59.49769069799599, 16.01358620918395, 16.489674134905655, 16.489674134905655, 22.98324851186175, 34.501942437892716, 28.2698785323965, 32.53843073806492, 26.713157919740368, 20.007230835729846, 52.097918711212095, 19.620255961497417, 19.620255961497417, 29.396022468352484, 32.85774546700983, 40.57389988868238, 44.04255166984753, 17.690837939037834, 50.79177210350426, 29.053548433146837, 26.817710263716034, 25.39049775763493, 25.740017318702005, 54.27917440928786, 46.51820540231633, 29.626578330054702, 28.715064341305684, 34.09998263323346, 44.21492934326177, 19.391673055829287, 22.84897480427519, 73.46984133199462, 32.033969127372174, 38.48969815122384, 48.75066069158903, 34.77254452624438, 53.12545808585662, 31.72265993608251, 49.133016862271724, 14.187251079068192, 63.582848099201826, 41.53608219643496, 64.08153455750669, 15.490615283867873, 44.73612093895834, 18.868615759264884, 67.68358948381629, 27.40010580820802, 15.827022278874091, 19.505629671380603, 19.391673055829287, 32.85774546700983, 24.370158584949497, 33.90076227087091, 59.73055807527518, 53.43765380953904, 31.90908051733889, 23.898797049169467, 25.192907336481866, 45.086990684122, 13.643746025356307, 25.840760898054555, 17.314815113938415, 47.71445419747699, 40.49473132867618, 41.943695056893915, 21.173160702117308, 28.65903496526287, 40.65322322588273, 31.41437607743883, 44.04255166984753, 27.668995768810042, 20.24306974171993, 56.55175575986736, 32.53843073806492, 32.79363281328884, 43.02230705927991, 35.52769382137767, 21.973644325590612, 18.467558995986426, 62.597087408320306, 80.69083499570341, 31.169907463288272, 65.60118435636834, 52.50652775191775, 34.63697922255266, 57.66713919909174, 59.73055807527518, 76.99573263552006, 61.14702744030778, 50.20003003478527, 47.15856396830611, 13.67042000714299, 21.464625403032095, 16.202349296067883, 21.845268970328604, 21.090614371389922, 29.167260806600495, 44.04255166984753, 22.67116267165818, 28.827451925481085, 37.45150421083523, 47.15856396830611, 44.91121316574036, 22.10277408821932, 42.93836112094585, 48.182698291098816, 11.070643498756118, 14.159568646729724, 17.113091407702502, 17.76007792103387, 35.25121511458854, 18.721779026482544, 25.790339917193062, 48.27689708476307, 23.028181534732802, 24.275148340672132, 15.734556979186685, 16.94678474038272, 14.354485905365031, 47.52843316962603, 29.511075247281195, 44.21492934326177, 36.44131378072682, 26.557093177483523, 28.1047190510635, 40.33685733281881, 26.713157919740368, 38.26483200699776, 36.51255777326842, 13.617124090346826, 12.618402319374093, 10.793091461961774, 32.159346537604755, 15.16135869683449, 24.70563239932894, 36.15772522700737, 14.926303617194517, 24.609314266596865, 19.774139536218833, 27.507346790200362, 16.202349296067883, 13.249825400173327, 22.582776184718522, 17.829588900657228, 20.282645623050005, 55.89290784893326, 12.667789349678872, 12.717369675355013, 54.811841150452466, 26.196478674560588, 30.093129052017023, 18.942465433314446, 53.333385249853684, 38.33964103892552, 21.590763877931302, 32.348333340301785, 27.668995768810042, 27.668995768810042, 25.24216031657971, 15.370066359341997, 20.56185170491249, 27.561124577174567, 20.56185170491249, 20.046345644076645, 17.725424121461643, 29.281418237700255, 19.016604146815915, 46.60915005647103, 11.049042250008942, 17.484733434196265, 16.01358620918395, 18.28808948244362, 32.033969127372174, 12.643071719773257, 23.482371718117374, 25.58963789966816, 13.67042000714299, 13.912844803302042, 25.34095533871963, 12.254085296672228, 29.511075247281195, 25.891280453627324, 28.380523708580828, 20.085536923187668, 18.145770440989054, 16.84777751703098, 15.98234019723552, 16.107691244988956, 14.4106677747075, 24.465540688808968, 13.994604675279588, 13.406010149206963, 10.667348137697372, 9.267952672428878, 10.793091461961774, 27.77728915635237, 17.76007792103387, 17.76007792103387, 15.765318566620541, 22.36331459495464, 18.2524054167373, 20.92648590822237, 23.71281513502692, 20.203571081597193, 20.085536923187668, 16.457499171202052, 16.58657693697683, 22.016603538363892, 19.929230034775262, 15.951155153095918, 16.716667072215767, 24.561296107667808, 14.438841114275835, 18.61240185635776, 23.20879380099744, 21.760102337787384, 14.580536166314408, 21.173160702117308, 19.165753221675935, 21.802644068801417, 18.79505399780919, 23.71281513502692, 14.868111474214228, 24.417803063601138, 24.80232751057381, 20.64232851707703, 19.016604146815915, 24.89940107578365, 22.715485556228362, 30.866995306309306, 16.39333746248769, 16.39333746248769, 13.885697797673076, 14.104365720113616, 22.75989509352673, 14.354485905365031, 17.45061689198871, 12.422772124124576, 20.16414949200632, 15.072782365110845, 23.620367629992042, 15.370066359341997, 21.090614371389922, 47.99485175252902, 21.887977204970085, 26.922671813996086, 26.975306541751994, 19.89034374461794, 32.85774546700983, 47.06654728734682, 47.250760544967605, 53.22932014095188, 19.391673055829287, 37.232703447205715, 23.898797049169467, 22.189281900071105, 39.479488162280596, 32.033969127372174, 28.325147094540903, 38.867415430562446, 39.94485987582193, 38.64034257045414, 66.37447035867858, 61.867809250367884, 38.115651564714454, 54.38529201854992, 49.61518196998445, 43.61454191906839, 49.133016862271724, 36.798929313140206, 49.51837188956857, 15.370066359341997, 49.22907356066477, 29.858942463791767, 26.713157919740368, 26.453556883236853, 30.62678644311483, 32.22221914035047, 36.51255777326842, 27.028044172168716, 27.61500750152876, 27.61500750152876, 25.39049775763493, 20.64232851707703, 20.68268496272443, 25.489873356103875, 33.83461446305701, 21.632974546568068, 51.1901369093998, 31.291903031786116, 33.702705802144536, 39.40245503911115, 47.250760544967605, 55.89290784893326, 22.98324851186175, 25.63966650134361, 16.39333746248769, 20.203571081597193, 22.319678873017494, 16.04489330813532, 91.79256012938836, 34.63697922255266, 21.548635571548648, 20.56185170491249, 37.74523999888851, 20.282645623050005, 54.173263858709824, 42.60421214369869, 42.60421214369869, 20.36202965350754, 14.926303617194517, 39.94485987582193, 20.967397891114594, 29.626578330054702, 24.850816893992036, 40.97207040739876, 15.340075995482293, 20.92648590822237, 38.48969815122384, 17.969428099556087, 14.81014620089916, 23.666546242140512, 21.717643615002626, 20.763634702888442, 13.750755251693265, 31.35307975333966, 40.022953418667065, 26.043432497666576, 35.45837151398557, 19.735555861572347, 22.893645317537526, 27.668995768810042, 17.113091407702502, 17.113091407702502, 30.806767041912167, 17.690837939037834, 18.61240185635776, 12.374340325455691, 13.301683750542033, 37.160054292819765, 12.717369675355013, 23.25416777248505, 48.46584751755162, 54.811841150452466, 70.37979717263202, 32.28521466114651, 65.34542957624015, 56.33128135834348, 22.67116267165818, 33.50580642730562, 47.435694792893365, 44.82358155812517, 52.50652775191775, 35.45837151398557, 36.65546388385301, 63.3349620664254, 48.182698291098816, 45.44061233480379, 50.990565478815945, 38.71588573814723, 16.782094257242417, 46.33684810653952, 51.1901369093998, 37.232703447205715, 61.867809250367884, 37.8190332109868, 35.11378365112735, 43.02230705927991, 38.19016894386643, 37.8190332109868, 31.169907463288272, 109.86130976812342, 17.314815113938415, 19.353835657005096, 25.94189877662824, 20.763634702888442, 43.444504934395866, 32.348333340301785, 16.01358620918395, 41.29341833835342, 27.346642204097876, 21.33922385967176, 16.076261613516785, 23.94552000018515, 22.10277408821932, 25.539706914745466, 99.2514223056366, 60.908637772317434, 88.27604237373367, 24.039240115615065, 42.3553080116137, 29.684499285876434, 12.520205133467638, 25.790339917193062, 29.975807060355134, 30.034410770075546, 42.02569638966092, 29.975807060355134, 25.94189877662824, 63.3349620664254, 18.28808948244362, 23.345182011801924, 43.19069160819034, 35.52769382137767, 35.66674528344124, 49.133016862271724, 44.91121316574036, 19.46756991825522, 27.240027749911206, 22.75989509352673, 29.22428378123494, 25.63966650134361, 25.094689498803355, 37.8190332109868, 25.1437504597293, 53.96206231832688, 44.12865633779556, 30.987805310145493, 27.995149135337257, 26.713157919740368, 12.158723248329283, 27.028044172168716, 21.049461966173517, 29.453492679719986, 17.013112580497378, 11.333176243677455, 34.16664924473404, 30.987805310145493, 20.64232851707703, 29.453492679719986, 42.190180314697045, 56.33128135834348, 56.33128135834348, 15.827022278874091, 18.979498589532987, 24.948080241282657, 26.401940203271806, 23.073202403172917, 20.602050815688813, 23.028181534732802, 15.951155153095918, 17.013112580497378, 17.28103011956753, 25.39049775763493, 39.40245503911115, 15.04337206739713, 25.1437504597293, 16.39333746248769, 35.32013242426123, 15.43022309819552, 19.96819234893607, 24.039240115615065, 23.118311288923124, 27.028044172168716, 52.71203249107963, 28.60311491479713, 38.41459632502361, 38.41459632502361, 80.06289393593435, 47.34313736903344, 44.5617113320772, 13.940044882238906, 16.076261613516785, 15.220698576992893, 34.908640417532865, 69.01852867010916, 40.022953418667065, 49.32531805312982, 37.01518097066168, 32.85774546700983, 27.133828946090556, 22.84897480427519, 60.434642992688794, 32.47494113727571, 28.325147094540903, 23.71281513502692, 108.5813867407896, 25.1437504597293, 39.634006510082365, 32.348333340301785, 18.359667040887704, 23.4365524707797, 48.08868329990042, 23.118311288923124, 44.64883097460398, 57.442316329639176, 39.01953819342525, 24.609314266596865, 37.160054292819765, 42.438114061533675, 12.692555303381242, 34.56939489431364, 22.494734284035275, 34.300373728940066, 58.80452160545304, 38.26483200699776, 102.40200064874995, 42.68750480955783, 57.89284200037607, 62.84208548133934, 31.230845679407448, 21.131847230940878, 36.44131378072682, 39.248839424943355, 22.450842133517945, 14.723621737689907, 36.228414794400415, 42.60421214369869, 23.898797049169467, 23.345182011801924, 28.827451925481085, 46.97471015107385, 26.24769367033534, 61.38635013907805, 39.095822701298815, 39.095822701298815, 52.609179777419676, 43.275130861102994, 46.60915005647103, 11.900286792057758, 20.723120306578686, 37.37842812826559, 35.7364749685521, 29.511075247281195, 47.435694792893365, 49.22907356066477, 54.27917440928786, 34.97688798216538, 23.118311288923124, 45.263456176258764, 17.046373787079762, 38.41459632502361, 51.692489491288626, 66.11570082213345, 27.186876493132687, 11.647343462043494, 31.048387510438282, 21.297586362924953, 30.866995306309306, 17.725424121461643, 21.380942759123343, 15.460389702669161, 27.293282919182133, 28.996858600549206, 17.690837939037834, 41.861853726676614, 18.979498589532987, 30.74665629604239, 14.868111474214228, 15.10225016094451, 7.850261748528355, 12.842171128856814, 22.232662691545976, 22.232662691545976, 34.63697922255266, 33.375179667439795, 33.4404292647954, 49.037147591523336, 21.887977204970085, 25.740017318702005, 18.721779026482544, 21.632974546568068, 19.278382202442366, 29.053548433146837, 24.039240115615065, 29.396022468352484, 18.868615759264884, 12.447059060920614, 38.19016894386643, 15.04337206739713, 24.9968545761854, 35.87634357692165, 48.56059987746486, 37.9670527218579, 32.28521466114651, 41.37414835829223, 31.41437607743883, 26.24769367033534, 29.511075247281195, 27.346642204097876, 46.51820540231633, 38.79157659527073, 29.9173176993003, 26.24769367033534, 28.2698785323965, 15.04337206739713, 13.301683750542033, 20.602050815688813, 27.668995768810042, 20.44172438427552, 28.380523708580828, 55.89290784893326, 15.612109685326937, 15.98234019723552, 42.854578979512624, 39.248839424943355, 39.248839424943355, 21.506589466713315, 14.552086348683558, 32.60204446315635, 21.380942759123343, 19.658614220966218, 56.33128135834348, 40.97207040739876, 19.31607208722833, 25.34095533871963, 20.8856537535641, 45.61846186294896, 43.444504934395866, 13.172416709211404, 20.723120306578686, 18.979498589532987, 50.89107172411317, 28.715064341305684, 17.180070153709277, 24.561296107667808, 64.33234259239626, 43.275130861102994, 40.49473132867618, 50.396507397123145, 34.63697922255266, 40.1011996370117, 23.992334296139543, 29.858942463791767, 19.165753221675935, 57.66713919909174, 20.602050815688813, 15.460389702669161, 64.83690740886694, 47.250760544967605, 47.250760544967605, 20.4816886421048, 15.43022309819552, 24.417803063601138, 50.89107172411317, 43.3597351955915, 33.96703939987107, 35.52769382137767, 33.90076227087091, 24.609314266596865, 43.3597351955915, 32.72964525738904, 33.57131140436464, 36.15772522700737, 15.370066359341997, 61.38635013907805, 29.858942463791767, 24.039240115615065, 25.840760898054555, 30.093129052017023, 24.370158584949497, 36.15772522700737, 21.887977204970085, 24.609314266596865, 32.159346537604755, 59.49769069799599, 17.28103011956753, 24.370158584949497, 32.79363281328884, 16.329425896569767, 68.48142221881122, 36.72712654703974, 15.400115355370453, 33.245062173048, 52.40407602298735, 38.41459632502361, 18.181246031735814, 41.132430556936306, 28.547303976590324, 34.63697922255266, 41.780172086809024, 39.94485987582193, 20.8856537535641, 33.57131140436464, 24.609314266596865, 41.37414835829223, 30.927341318986464, 26.713157919740368, 26.87013978811538, 30.806767041912167, 64.58413226217957, 63.83170433047907, 70.65525591042105, 27.668995768810042, 35.66674528344124, 40.41571724385996, 40.41571724385996, 38.79157659527073, 53.646802873050504, 53.22932014095188, 28.771203256660744, 80.06289393593435, 32.60204446315635, 33.4404292647954, 34.97688798216538, 14.694892728788941, 57.21836996148928, 32.348333340301785, 32.09659661306934, 12.76714405293921, 17.45061689198871, 37.305494633128845, 35.806340977539655, 19.278382202442366, 31.660762082694095, 35.66674528344124, 19.016604146815915, 38.867415430562446, 28.049880592281973, 47.99485175252902, 34.77254452624438, 48.27689708476307, 58.119428177447986, 28.883810562338418, 63.582848099201826, 52.71203249107963, 64.33234259239626, 97.33172602285453, 51.591626129191596, 36.87087245606033, 69.83209724580306, 58.80452160545304, 93.6030061542134, 39.556671887793456, 38.867415430562446, 22.145985754016134, 46.427438201089, 42.77096031537661, 74.04607243930377, 26.557093177483523, 77.90333396724647, 80.37625124290746, 35.45837151398557, 35.45837151398557, 27.72308958496738, 46.51820540231633, 53.333385249853684, 64.08153455750669, 24.22778221261098, 71.48811770889668, 52.097918711212095, 39.32557222442699, 87.24759532868651, 48.56059987746486, 38.71588573814723, 31.35307975333966, 66.37447035867858, 65.34542957624015, 108.15806803146498, 42.10785803778799, 105.24068867470233, 36.72712654703974, 69.55984769951485, 44.21492934326177, 78.51433738825796, 26.35042423874788, 53.333385249853684, 34.03344610288473, 18.2524054167373, 46.15619785585283, 38.71588573814723, 48.65553748142455, 77.90333396724647, 65.34542957624015, 76.99573263552006, 41.861853726676614, 39.78912962559818, 29.56877039069624, 74.04607243930377, 85.89494673838159, 36.65546388385301, 35.45837151398557, 67.9484955630781, 59.73055807527518, 83.90519244027122, 48.182698291098816, 39.095822701298815, 101.6051019574268, 40.892124929554, 48.75066069158903, 48.182698291098816, 42.77096031537661, 43.95661501095434, 57.21836996148928, 63.08804244970428, 36.37020880080945, 16.8149038152427, 22.715485556228362, 19.46756991825522, 33.50580642730562, 19.96819234893607, 20.322298876557472, 39.479488162280596, 71.48811770889668, 16.425386987895497, 23.345182011801924, 10.399907116482579, 15.98234019723552, 20.64232851707703, 17.213657767955564, 13.940044882238906, 12.817113241929157, 15.280270706875589, 15.131775567308885, 14.076845015253456, 13.172416709211404, 13.172416709211404, 13.172416709211404, 13.172416709211404, 13.172416709211404, 13.172416709211404, 13.172416709211404, 13.172416709211404, 13.172416709211404, 12.134998917818166, 17.14654807625046, 17.180070153709277, 23.666546242140512, 26.975306541751994, 17.725424121461643, 23.4365524707797, 11.135700962413912, 13.750755251693265, 15.612109685326937, 9.432306184336436, 10.920317008742302, 15.131775567308885, 11.399776500625777, 15.43022309819552, 17.51891667538985, 15.551243837871848, 14.049378009312553, 14.049378009312553, 11.399776500625777, 11.027483150026422, 8.93032800592033, 16.36135047271935, 17.656319241826594, 18.576084986220376, 16.01358620918395, 16.170734966018113, 21.887977204970085, 25.840760898054555, 14.214987631491761, 16.202349296067883, 15.581647041956762, 30.866995306309306, 30.26997357328364, 32.411575418595035, 22.36331459495464, 36.228414794400415, 15.460389702669161, 20.282645623050005, 18.110364070885183, 25.44013703364339, 16.489674134905655, 25.34095533871963, 10.877742727456024, 13.406010149206963, 15.827022278874091, 14.2706235200274, 15.796140293991145, 18.323843311697065, 33.702705802144536, 32.348333340301785, 26.66103470132838, 28.491601937740562, 14.049378009312553, 11.57929685891098, 17.247311047115943, 17.14654807625046, 17.14654807625046, 20.085536923187668, 25.63966650134361, 36.08717359043372, 15.951155153095918, 39.86691871057298, 23.25416777248505, 51.39048944047805, 33.31005738633129, 32.72964525738904, 36.942956250241515, 11.993621898152476, 13.432219349369987, 18.2524054167373, 16.04489330813532, 15.673213754973451, 14.382549407491597, 24.322607071104724, 23.852165264858517, 19.581972547389178, 25.992616060150944, 20.4816886421048, 21.548635571548648, 32.60204446315635, 30.987805310145493, 10.751013186076355, 34.03344610288473, 83.25223705241488, 75.21212841006654, 20.723120306578686, 16.521912001744084, 16.107691244988956, 17.180070153709277, 12.842171128856814, 22.80439145296227, 27.886006392229614, 33.63694444585419, 31.846818916590372, 45.529450258314064, 63.3349620664254, 41.780172086809024, 45.263456176258764, 24.513371642677637, 43.87084603329354, 34.03344610288473, 42.190180314697045, 38.48969815122384, 35.52769382137767, 25.790339917193062, 30.927341318986464, 28.715064341305684, 26.609013186724894, 44.12865633779556, 110.7229626998625, 19.016604146815915, 44.30137101535166, 35.66674528344124, 43.87084603329354, 37.37842812826559, 35.66674528344124, 31.475792237910618, 46.427438201089, 44.73612093895834, 59.265731184583274, 57.66713919909174, 43.19069160819034, 32.159346537604755, 28.940279382223952, 38.71588573814723, 22.626926270737744, 36.299242562272255, 51.290215346585704, 16.716667072215767, 37.37842812826559, 29.56877039069624, 43.444504934395866, 58.34690118774256, 55.457945789238345, 26.453556883236853, 22.626926270737744, 17.587483776585056, 18.467558995986426, 19.46756991825522, 26.24769367033534, 42.438114061533675, 23.574279122424027, 28.049880592281973, 28.049880592281973, 39.556671887793456, 70.65525591042105, 26.24769367033534, 25.539706914745466, 36.228414794400415, 29.858942463791767, 24.513371642677637, 16.329425896569767, 21.717643615002626, 27.186876493132687, 69.55984769951485, 36.87087245606033, 25.045724266551254, 45.61846186294896, 25.539706914745466, 37.45150421083523, 40.17959882934157, 67.41971617548266, 33.702705802144536, 18.979498589532987, 53.85677052285177, 32.53843073806492, 31.72265993608251, 83.90519244027122, 76.09870517958821, 25.34095533871963, 28.214717811314713, 56.9952966774873, 24.70563239932894, 12.278042445054236, 43.95661501095434, 18.979498589532987, 19.620255961497417, 21.548635571548648, 16.782094257242417, 27.240027749911206, 15.734556979186685, 13.617124090346826, 18.503663694167408, 27.77728915635237, 34.97688798216538, 25.192907336481866, 37.524723159601, 27.346642204097876, 36.299242562272255, 17.180070153709277, 17.79479946994832, 24.657426302639607, 34.908640417532865, 34.84052601883783, 25.34095533871963, 33.31005738633129, 33.375179667439795, 26.975306541751994, 58.57526450222688, 14.926303617194517, 32.72964525738904, 72.04880596519355, 33.05083646319603, 32.66578255521719, 36.51255777326842, 47.34313736903344, 19.89034374461794, 16.36135047271935, 31.41437607743883, 20.24306974171993, 25.094689498803355, 33.245062173048, 21.675267738479917, 22.84897480427519, 32.47494113727571, 20.602050815688813, 18.2524054167373, 42.190180314697045, 15.04337206739713, 28.214717811314713, 16.58657693697683, 55.89290784893326, 23.71281513502692, 18.61240185635776, 39.40245503911115, 35.7364749685521, 45.17513726542669, 39.479488162280596, 81.00664999449546, 29.511075247281195, 41.617286632367644, 32.159346537604755, 37.37842812826559, 17.348666159108166, 21.632974546568068, 26.817710263716034, 59.265731184583274, 55.89290784893326, 16.23402543319486, 50.990565478815945, 25.840760898054555, 53.75168417471723, 69.55984769951485, 44.21492934326177, 32.79363281328884, 34.300373728940066, 45.44061233480379, 47.15856396830611, 56.773093073800744, 41.780172086809024, 50.89107172411317, 43.5294404008871, 15.280270706875589, 28.771203256660744, 26.196478674560588, 33.375179667439795, 17.76007792103387, 33.31005738633129, 31.169907463288272, 29.9173176993003, 28.049880592281973, 44.73612093895834, 18.79505399780919, 65.09067188950593, 74.04607243930377, 70.65525591042105, 61.38635013907805, 38.48969815122384, 16.170734966018113, 24.850816893992036, 23.4365524707797, 20.68268496272443, 56.11166650505244, 21.297586362924953, 56.55175575986736, 85.89494673838159, 18.539838978246124, 13.484791569471833, 13.484791569471833, 14.984723517782793, 14.984723517782793, 15.642631884188171, 15.612109685326937, 31.846818916590372, 13.198169226352599, 13.198169226352599, 14.723621737689907, 16.88071548801052, 16.076261613516785, 15.43022309819552, 17.484733434196265, 16.265763498233948, 15.581647041956762, 12.182493960703473, 15.072782365110845, 14.637602771363136, 14.637602771363136, 14.637602771363136, 14.637602771363136, 14.637602771363136, 14.637602771363136, 14.637602771363136, 19.505629671380603, 13.121062323276538, 13.564035954639001, 22.80439145296227, 17.382583384208317, 16.8149038152427, 17.382583384208317, 16.457499171202052, 13.223972090529857, 13.223972090529857, 14.723621737689907, 20.521731031400577, 14.242778409807016, 19.581972547389178, 17.484733434196265, 18.576084986220376, 13.537569551425428, 17.76007792103387, 23.75917448515314, 16.554212894695087, 18.216790978454146, 20.16414949200632, 20.4816886421048, 15.642631884188171, 32.22221914035047, 21.42274322042474, 23.20879380099744, 24.9968545761854, 12.618402319374093, 12.742232560259533, 17.829588900657228, 13.885697797673076, 18.79505399780919, 19.543763832601815, 16.076261613516785, 17.247311047115943, 21.675267738479917, 20.602050815688813, 20.203571081597193, 16.91371785382974, 35.52769382137767, 19.053782246709744, 13.777638440111984, 15.888967492629165, 19.620255961497417, 20.401838105461145, 17.079700020438295, 20.124804822565828, 22.538712245181248, 20.84490127137742, 36.65546388385301, 28.491601937740562, 29.800681131145613, 23.620367629992042, 40.17959882934157, 36.65546388385301, 27.886006392229614, 57.21836996148928, 59.49769069799599, 45.97625189374871, 59.265731184583274, 47.34313736903344, 41.617286632367644, 67.68358948381629, 22.626926270737744, 13.994604675279588, 47.34313736903344, 39.248839424943355, 40.022953418667065, 19.165753221675935, 13.994604675279588, 49.22907356066477, 46.427438201089, 18.942465433314446, 56.11166650505244, 52.50652775191775, 31.9714638416083, 48.84596987082467, 25.34095533871963, 49.71218131735948, 45.44061233480379, 29.9173176993003, 40.49473132867618, 74.62682298320091, 34.63697922255266, 25.992616060150944, 48.65553748142455, 52.19977189231385, 51.79355004464666, 49.61518196998445, 48.75066069158903, 47.62135285300038, 65.09067188950593, 47.71445419747699, 47.71445419747699, 73.18340956555208, 24.513371642677637, 41.05217218133816, 52.40407602298735, 62.597087408320306, 48.46584751755162, 44.91121316574036, 37.45150421083523, 43.444504934395866, 36.87087245606033, 23.4365524707797, 49.809370301716946, 62.10995300937982, 62.10995300937982, 59.73055807527518, 43.78524440968234, 24.80232751057381, 66.37447035867858, 20.68268496272443, 19.505629671380603, 16.74934871787156, 21.675267738479917, 17.864446345871784, 22.450842133517945, 34.367432111764096, 28.771203256660744, 35.32013242426123, 33.702705802144536, 42.60421214369869, 26.401940203271806, 59.964336869699544, 21.760102337787384, 53.542126217761066, 24.948080241282657, 39.7114923241556, 81.32370105824417, 21.464625403032095, 17.6218678981495, 23.482371718117374, 11.26696508157019, 65.60118435636834, 29.453492679719986, 24.133327041205547, 44.73612093895834, 19.429584428039178, 27.940524471445343, 26.505274475545082, 19.429584428039178, 26.975306541751994, 24.039240115615065, 20.36202965350754, 19.929230034775262, 22.98324851186175, 42.190180314697045, 37.45150421083523, 26.505274475545082, 22.538712245181248, 29.74253347911242, 24.561296107667808, 15.581647041956762, 29.396022468352484, 19.505629671380603, 25.44013703364339, 39.32557222442699, 43.444504934395866, 37.08754689283638, 32.85774546700983, 33.4404292647954, 18.431524745974645, 24.513371642677637, 44.5617113320772, 34.367432111764096, 31.846818916590372, 52.50652775191775, 24.948080241282657, 42.77096031537661, 20.36202965350754, 40.25815129472595, 36.08717359043372, 40.892124929554, 18.39556080667227, 73.46984133199462, 73.46984133199462, 43.10641711474324, 84.89424029774196, 72.89809449073893, 38.56494680401705, 33.90076227087091, 12.968197316969134, 48.84596987082467, 26.66103470132838, 24.417803063601138, 33.63694444585419, 21.760102337787384, 28.827451925481085, 50.1020787877761, 32.60204446315635, 43.275130861102994, 25.840760898054555, 35.04526897307925, 32.60204446315635, 32.60204446315635, 37.5980852538711, 45.086990684122, 40.57389988868238, 31.048387510438282, 12.968197316969134, 17.829588900657228, 45.7076474882609, 39.01953819342525, 16.107691244988956, 24.322607071104724, 16.84777751703098, 25.790339917193062, 40.892124929554, 21.214554942517648, 20.967397891114594, 56.11166650505244, 29.396022468352484, 13.172416709211404, 22.36331459495464, 19.016604146815915, 17.046373787079762, 14.467069533669397, 37.305494633128845, 23.620367629992042, 15.703855414343392, 20.24306974171993, 37.232703447205715, 36.798929313140206, 27.186876493132687, 20.967397891114594, 20.085536923187668, 27.40010580820802, 21.173160702117308, 24.275148340672132, 18.6852487375444, 51.39048944047805, 52.81508628524973, 53.85677052285177, 14.438841114275835, 15.520899957093135, 10.522497930618114, 23.16350836406023, 60.434642992688794, 27.668995768810042, 51.09025374715126, 60.908637772317434, 33.83461446305701, 23.028181534732802, 17.113091407702502, 14.868111474214228, 21.845268970328604, 12.667789349678872, 16.202349296067883, 11.715789946123383, 13.198169226352599, 27.77728915635237, 16.74934871787156, 27.61500750152876, 17.013112580497378, 22.40703562619394, 17.046373787079762, 17.046373787079762, 16.8149038152427, 11.807678025521156, 18.79505399780919, 28.60311491479713, 17.864446345871784, 18.83179895971962, 13.198169226352599, 17.382583384208317, 30.74665629604239, 13.301683750542033, 26.401940203271806, 26.401940203271806, 20.401838105461145, 20.401838105461145, 20.401838105461145, 20.401838105461145, 20.401838105461145, 13.511154789957821, 19.851533330124635, 18.110364070885183, 23.16350836406023, 27.346642204097876, 19.505629671380603, 22.40703562619394, 30.329152388418176, 14.926303617194517, 21.214554942517648, 31.475792237910618, 22.938403163153815, 29.110349096271467, 29.453492679719986, 28.2698785323965, 25.63966650134361, 86.23113005463887, 45.351947753528236, 41.2128458403547, 46.88305220915621, 32.47494113727571, 49.133016862271724, 45.44061233480379, 19.505629671380603, 44.387981683814154, 24.18050850623376, 27.561124577174567, 35.52769382137767, 29.511075247281195, 27.61500750152876, 49.61518196998445, 57.89284200037607, 71.48811770889668, 36.798929313140206, 104.01459751366006, 45.79700747446617, 31.598985005566618, 71.20941195067294, 52.097918711212095, 82.28231911219243, 26.922671813996086, 23.390822626823073, 29.453492679719986, 61.626609520407456, 25.39049775763493, 31.9714638416083, 44.82358155812517, 42.52108200006278, 32.66578255521719, 15.827022278874091, 19.697047472120772, 13.511154789957821, 20.763634702888442, 22.98324851186175, 23.52828054362271, 20.64232851707703, 36.87087245606033, 36.15772522700737, 58.119428177447986, 23.992334296139543, 28.827451925481085, 58.80452160545304, 26.557093177483523, 15.827022278874091, 16.139182322446413, 20.56185170491249, 25.790339917193062, 19.620255961497417, 18.868615759264884, 56.9952966774873, 28.325147094540903, 24.9968545761854, 23.898797049169467, 36.58394105020859, 57.89284200037607, 63.83170433047907, 38.41459632502361, 33.31005738633129, 64.58413226217957, 51.79355004464666, 53.542126217761066, 52.609179777419676, 71.76791429061606, 63.08804244970428, 66.89505178172818, 36.228414794400415, 33.57131140436464, 64.83690740886694, 54.173263858709824, 61.38635013907805, 25.39049775763493, 32.85774546700983, 24.561296107667808, 67.9484955630781, 48.37128004023317, 34.97688798216538, 55.02636863114642, 51.894808174781225, 48.56059987746486, 34.63697922255266, 45.7076474882609, 17.046373787079762, 31.72265993608251, 15.131775567308885, 32.92198346312245, 62.3530444922685, 18.942465433314446, 32.28521466114651, 28.65903496526287, 34.84052601883783, 15.888967492629165, 16.84777751703098, 36.72712654703974, 52.50652775191775, 50.396507397123145, 30.927341318986464, 13.146714440868186, 11.333176243677455, 18.075026786359576, 26.557093177483523, 21.973644325590612, 20.602050815688813, 24.89940107578365, 20.282645623050005, 27.133828946090556, 30.21091022897123, 23.898797049169467, 33.1801937796528, 17.51891667538985, 35.11378365112735, 44.99901609609206, 22.059646737841184, 33.1801937796528, 101.6051019574268, 38.48969815122384, 37.305494633128845, 28.436008585761353, 21.008389858307645, 41.29341833835342, 11.005966116567095, 44.73612093895834, 44.73612093895834, 44.21492934326177, 30.806767041912167, 22.67116267165818, 59.964336869699544, 36.15772522700737, 35.806340977539655, 56.773093073800744, 19.20322291302118, 66.11570082213345, 13.750755251693265, 36.299242562272255, 61.626609520407456, 35.25121511458854, 33.375179667439795, 40.57389988868238, 47.807737558209425, 27.77728915635237, 16.457499171202052, 18.539838978246124, 22.626926270737744, 12.01706983184115, 29.9173176993003, 23.52828054362271, 23.852165264858517, 46.97471015107385, 22.98324851186175, 17.180070153709277, 24.133327041205547, 61.626609520407456, 34.4346215959791, 13.940044882238906, 28.547303976590324, 36.72712654703974, 40.1011996370117, 23.482371718117374, 31.53732846903916, 34.56939489431364, 23.71281513502692, 18.2524054167373, 15.340075995482293, 31.784678801853513, 24.039240115615065, 22.80439145296227, 30.388446900124176, 54.173263858709824, 50.990565478815945, 36.37020880080945, 21.675267738479917, 19.735555861572347, 24.657426302639607, 24.9968545761854, 64.33234259239626, 92.87458178129451, 49.809370301716946, 18.039758452611142, 25.539706914745466, 31.41437607743883, 14.187251079068192, 12.892433965519283, 56.55175575986736, 33.4404292647954, 26.09434828302453, 27.028044172168716, 32.85774546700983, 23.80562446936611, 31.169907463288272, 28.214717811314713, 29.858942463791767, 26.043432497666576, 39.479488162280596, 55.02636863114642, 49.809370301716946, 55.24173574865985, 55.89290784893326, 45.263456176258764, 47.250760544967605, 50.1020787877761, 50.1020787877761, 30.68666283939559, 30.329152388418176, 38.64034257045414, 35.04526897307925, 45.351947753528236, 39.94485987582193, 36.37020880080945, 82.28231911219243, 37.305494633128845, 19.016604146815915, 101.20898120494374, 26.975306541751994, 75.80202444429054, 43.61454191906839, 30.388446900124176, 17.45061689198871, 25.790339917193062, 84.2335880783952, 32.72964525738904, 35.7364749685521, 70.37979717263202, 41.2128458403547, 32.47494113727571, 21.131847230940878, 26.87013978811538, 58.34690118774256, 47.250760544967605, 63.3349620664254, 25.34095533871963, 27.240027749911206, 22.98324851186175, 20.84490127137742, 25.790339917193062, 64.33234259239626, 52.9183415530491, 12.817113241929157, 27.240027749911206, 29.626578330054702, 21.930768935644906, 23.20879380099744, 18.83179895971962, 21.760102337787384, 34.09998263323346, 38.0412795852805, 44.5617113320772, 53.333385249853684, 46.066137010589394, 66.11570082213345, 56.773093073800744, 54.598150033144236, 17.690837939037834, 34.63697922255266, 27.080884906424416, 13.67042000714299, 34.09998263323346, 38.56494680401705, 39.78912962559818, 42.52108200006278, 37.232703447205715, 53.43765380953904, 45.17513726542669, 45.97625189374871, 38.79157659527073, 28.325147094540903, 37.232703447205715, 43.87084603329354, 56.9952966774873, 47.435694792893365, 45.529450258314064, 47.15856396830611, 56.55175575986736, 52.81508628524973, 42.60421214369869, 17.348666159108166, 30.68666283939559, 57.89284200037607, 45.17513726542669, 44.73612093895834, 58.34690118774256, 20.4816886421048, 27.028044172168716, 47.435694792893365, 34.63697922255266, 33.702705802144536, 32.53843073806492, 40.17959882934157, 19.89034374461794, 26.2990087931472, 20.84490127137742, 25.34095533871963, 14.076845015253456, 30.806767041912167, 42.438114061533675, 29.626578330054702, 32.159346537604755, 22.36331459495464, 27.4536739354601, 16.170734966018113, 15.04337206739713, 25.39049775763493, 21.33922385967176, 22.80439145296227, 14.83910053420958, 21.380942759123343, 18.075026786359576, 18.145770440989054, 20.322298876557472, 25.63966650134361, 14.523692042874359, 35.66674528344124, 48.182698291098816, 52.81508628524973, 42.68750480955783, 37.45150421083523, 19.128356641900336, 24.086237637543274, 18.61240185635776, 21.297586362924953, 31.230845679407448, 18.942465433314446, 40.41571724385996, 54.811841150452466, 59.03467599561125, 39.01953819342525, 31.9714638416083, 55.457945789238345, 55.457945789238345, 40.1011996370117, 34.09998263323346, 46.97471015107385, 33.96703939987107, 30.927341318986464, 20.723120306578686, 41.617286632367644, 51.894808174781225, 22.938403163153815, 33.1801937796528, 35.45837151398557, 39.40245503911115, 31.35307975333966, 46.427438201089, 61.14702744030778, 33.90076227087091, 65.60118435636834, 38.26483200699776, 34.03344610288473, 51.39048944047805, 47.807737558209425, 47.90120329104557, 76.39654709045266, 36.798929313140206, 53.646802873050504, 45.79700747446617, 34.84052601883783, 44.21492934326177, 30.21091022897123, 36.016759615546256, 23.75917448515314, 49.9067492938035, 42.77096031537661, 20.282645623050005, 42.60421214369869, 47.71445419747699, 50.49503426195374, 43.444504934395866, 42.438114061533675, 39.01953819342525, 35.66674528344124, 42.68750480955783, 24.70563239932894, 34.23344619169951, 60.908637772317434, 37.232703447205715, 91.79256012938836, 71.48811770889668, 48.65553748142455, 27.293282919182133, 46.066137010589394, 21.590763877931302, 33.1801937796528, 78.51433738825796, 49.61518196998445, 54.811841150452466, 52.19977189231385, 80.69083499570341, 42.93836112094585, 49.71218131735948, 60.67117749756472, 53.542126217761066, 32.66578255521719, 23.390822626823073, 20.401838105461145, 32.85774546700983, 41.943695056893915, 45.529450258314064, 40.57389988868238, 36.798929313140206, 30.927341318986464, 29.800681131145613, 82.28231911219243, 40.49473132867618, 28.65903496526287, 28.715064341305684, 59.03467599561125, 23.390822626823073, 17.969428099556087, 29.053548433146837, 30.866995306309306, 34.300373728940066, 32.79363281328884, 27.186876493132687, 19.812798643245316, 33.11545195869231, 23.118311288923124, 15.220698576992893, 29.33866439394716, 52.609179777419676, 18.942465433314446, 106.06610162605153, 35.04526897307925, 59.265731184583274, 42.438114061533675, 34.63697922255266, 41.37414835829223, 47.435694792893365, 22.10277408821932, 106.48123261036584, 50.990565478815945, 42.93836112094585, 28.60311491479713, 20.967397891114594, 17.51891667538985, 30.74665629604239, 36.15772522700737, 28.325147094540903, 32.28521466114651, 28.996858600549206, 30.151962130172098, 43.275130861102994, 38.41459632502361, 34.23344619169951, 50.396507397123145, 39.40245503911115, 39.40245503911115, 37.37842812826559, 37.01518097066168, 32.411575418595035, 16.01358620918395, 12.471393479493909, 13.67042000714299, 33.11545195869231, 16.457499171202052, 27.507346790200362, 30.927341318986464, 42.60421214369869, 43.699809813576366, 43.699809813576366, 34.4346215959791, 41.45503620813183, 19.505629671380603, 24.039240115615065, 25.689792910615846, 42.77096031537661, 61.867809250367884, 52.301824199806006, 59.73055807527518, 70.65525591042105, 55.24173574865985, 73.46984133199462, 24.086237637543274, 17.89937193856263, 17.14654807625046, 54.173263858709824, 51.79355004464666, 26.609013186724894, 31.169907463288272, 65.60118435636834, 86.23113005463887, 33.245062173048, 58.57526450222688, 24.753932740556596, 47.06654728734682, 80.37625124290746, 73.75739416067401, 27.507346790200362, 28.715064341305684, 24.609314266596865, 59.73055807527518, 56.773093073800744, 37.74523999888851, 51.1901369093998, 23.620367629992042, 54.49161709130268, 33.4404292647954, 44.30137101535166, 45.88654216244616, 40.1011996370117, 31.846818916590372, 52.9183415530491, 70.65525591042105, 34.56939489431364, 63.3349620664254, 59.73055807527518, 38.26483200699776, 34.4346215959791, 35.38918446958933, 37.89297069129353, 35.11378365112735, 27.028044172168716, 32.28521466114651, 23.80562446936611, 30.507383918455513, 71.48811770889668, 77.29708541258049, 77.29708541258049, 69.01852867010916, 63.582848099201826, 49.9067492938035, 47.62135285300038, 40.022953418667065, 50.396507397123145, 54.067559962798995, 53.12545808585662, 46.51820540231633, 67.9484955630781, 35.7364749685521, 22.582776184718522, 27.507346790200362, 45.086990684122, 85.89494673838159, 53.646802873050504, 24.370158584949497, 58.34690118774256, 31.598985005566618, 32.47494113727571, 22.80439145296227, 32.411575418595035, 43.02230705927991, 34.501942437892716, 24.609314266596865, 20.521731031400577, 20.602050815688813, 39.32557222442699, 38.48969815122384, 36.016759615546256, 36.016759615546256, 47.15856396830611, 43.02230705927991, 35.25121511458854, 40.57389988868238, 38.26483200699776, 23.898797049169467, 44.5617113320772, 24.417803063601138, 34.908640417532865, 34.56939489431364, 80.69083499570341, 49.61518196998445, 38.19016894386643, 65.8579401324037, 46.7915731119465, 43.5294404008871, 57.89284200037607, 40.97207040739876, 18.721779026482544, 23.666546242140512, 44.04255166984753, 38.48969815122384, 65.34542957624015, 59.265731184583274, 39.94485987582193, 15.220698576992893, 70.37979717263202, 24.657426302639607, 24.370158584949497, 13.275729253905588, 21.33922385967176, 20.56185170491249, 33.05083646319603, 19.20322291302118, 22.016603538363892, 25.891280453627324, 41.698649825700315, 41.53608219643496, 41.45503620813183, 40.33685733281881, 40.33685733281881, 37.08754689283638, 13.96729813824377, 41.45503620813183, 56.9952966774873, 22.582776184718522, 38.115651564714454, 38.115651564714454, 27.72308958496738, 26.196478674560588, 27.346642204097876, 18.467558995986426, 32.9863470466753, 27.83159468972004, 56.55175575986736, 71.48811770889668, 45.44061233480379, 34.908640417532865, 66.89505178172818, 64.58413226217957, 34.56939489431364, 44.30137101535166, 29.684499285876434, 32.72964525738904, 27.4536739354601, 52.609179777419676, 43.3597351955915, 49.22907356066477, 45.88654216244616, 44.5617113320772, 50.396507397123145, 47.52843316962603, 47.52843316962603, 38.33964103892552, 43.61454191906839, 59.73055807527518, 39.7114923241556, 24.370158584949497, 46.066137010589394, 25.094689498803355, 29.9173176993003, 22.626926270737744, 33.57131140436464, 41.37414835829223, 34.70469568042362, 42.190180314697045, 35.52769382137767, 42.02569638966092, 68.48142221881122, 38.71588573814723, 25.34095533871963, 20.44172438427552, 49.42175070681059, 71.76791429061606, 71.76791429061606, 41.132430556936306, 52.81508628524973, 35.11378365112735, 71.20941195067294, 40.25815129472595, 39.556671887793456, 34.84052601883783, 39.172256347947744, 43.275130861102994, 28.049880592281973, 47.71445419747699, 27.668995768810042, 38.115651564714454, 35.25121511458854, 24.133327041205547, 39.7114923241556, 30.034410770075546, 27.668995768810042, 43.5294404008871, 26.453556883236853, 42.10785803778799, 34.63697922255266, 22.80439145296227, 23.620367629992042, 27.561124577174567, 30.093129052017023, 27.886006392229614, 27.83159468972004, 24.657426302639607, 28.65903496526287, 41.45503620813183, 20.085536923187668, 19.812798643245316, 19.543763832601815, 26.35042423874788, 23.852165264858517, 35.32013242426123, 42.190180314697045, 47.52843316962603, 29.33866439394716, 41.05217218133816, 36.798929313140206, 25.045724266551254, 50.79177210350426, 42.52108200006278, 38.48969815122384, 63.582848099201826, 28.325147094540903, 37.160054292819765, 32.28521466114651, 59.265731184583274, 41.53608219643496, 44.12865633779556, 45.88654216244616, 24.561296107667808, 25.39049775763493, 34.63697922255266, 69.55984769951485, 57.21836996148928, 38.26483200699776, 64.08153455750669, 56.11166650505244, 15.827022278874091, 46.60915005647103, 18.431524745974645, 28.049880592281973, 46.70027251047952, 32.09659661306934, 30.151962130172098, 52.9183415530491, 41.05217218133816, 48.84596987082467, 48.56059987746486, 37.89297069129353, 54.598150033144236, 56.9952966774873, 47.250760544967605, 23.620367629992042, 27.940524471445343, 24.850816893992036, 26.505274475545082, 17.416566918622852, 18.6852487375444, 23.80562446936611, 20.046345644076645, 26.557093177483523, 40.892124929554, 34.63697922255266, 36.942956250241515, 40.022953418667065, 40.892124929554, 44.47476167904283, 47.52843316962603, 23.4365524707797, 24.561296107667808, 21.049461966173517, 26.66103470132838, 57.66713919909174, 57.89284200037607, 34.367432111764096, 37.524723159601, 67.9484955630781, 20.4816886421048, 44.21492934326177, 73.75739416067401, 72.33079701870139, 51.692489491288626, 41.617286632367644, 30.21091022897123, 48.94146538270724, 38.64034257045414, 43.78524440968234, 49.61518196998445, 57.66713919909174, 81.641993024771, 42.77096031537661, 49.42175070681059, 63.582848099201826, 60.908637772317434, 48.84596987082467, 42.190180314697045, 37.08754689283638, 40.49473132867618, 33.702705802144536, 50.1020787877761, 41.943695056893915, 79.13013296849392, 54.811841150452466, 52.50652775191775, 76.99573263552006, 77.90333396724647, 28.827451925481085, 23.620367629992042, 70.65525591042105, 40.41571724385996, 36.44131378072682, 27.83159468972004, 25.790339917193062, 25.44013703364339, 48.56059987746486, 41.617286632367644, 19.20322291302118, 18.539838978246124, 62.597087408320306, 32.033969127372174, 23.028181534732802, 25.045724266551254, 38.64034257045414, 44.64883097460398, 52.81508628524973, 49.809370301716946, 47.99485175252902, 21.930768935644906, 52.301824199806006, 34.4346215959791, 38.64034257045414, 421.1414445251494, 134.60489048229144, 347.77735828917787, 371.6560203434465, 129.9549042642197, 227.18876822599648, 169.4931477924318, 167.51849280886478, 265.61057969554065, 80.69083499570341, 457.14471326890896, 314.1906602856942, 314.1906602856942, 136.19157033683516, 100.81440478091984, 96.95226558425567, 113.34868209247811, 161.10096012939127, 217.63350971989422, 70.10541234668786, 110.7229626998625, 84.563269020706, 96.19777706766972, 65.8579401324037, 62.597087408320306, 137.7969534714142, 110.7229626998625, 76.99573263552006, 64.58413226217957, 110.29129477712178, 100.81440478091984, 83.57807709540792, 93.2380826167318, 157.98498549518746, 102.00277308269969, 276.1913023296018, 62.84208548133934, 78.82163381626928, 176.93479059943397, 376.03698390417554, 176.93479059943397, 337.0773738522583, 197.38502244855516, 106.89798837141365, 126.94450214932196, 195.84896001735729, 206.85772745893686, 397.1742097789884, 114.23768682435218, 416.23499808144635, 333.1503034969411, 194.3248513188351, 60.908637772317434, 119.72006788311512, 155.5356553203632, 109.43300111180854, 88.62154503481175, 381.95870474451476, 105.65258908406659, 82.60436300370014, 99.2514223056366, 268.7415131590023, 135.66060972280735, 162.3644906197382, 211.76321709954237, 46.427438201089, 260.4732060371668, 97.33172602285453, 81.32370105824417, 139.42126035437516, 302.15420510895507, 86.56862915519338, 77.90333396724647, 113.34868209247811, 146.6840934260825, 187.61181231093738, 129.44825810694724, 232.57639460420748, 165.5668432526847, 69.83209724580306, 169.4931477924318, 163.63793110254423, 74.33588057277046, 377.5087510463025, 122.55914747655109, 193.56725052076862, 131.48677155266952, 77.59961765106212, 164.9213593028435, 98.86447768050158, 109.86130976812342, 174.8734378511126, 102.80279074731456, 490.4446850170856, 488.53262239485935, 502.07524555352444, 106.48123261036584, 29.858942463791767, 211.76321709954237, 107.7363996853844, 149.57717793613415, 204.44776193681219, 87.93188669987823, 220.19890466437903, 70.65525591042105, 102.80279074731456, 184.70316076024525, 239.95916187280093, 586.9854309417088, 263.5435817656175, 520.0402695399388, 113.7923162908356, 327.98528678000895, 79.13013296849392, 358.8169966923127, 283.84760726337464, 107.7363996853844, 115.13366410675533, 114.23768682435218, 164.27839185643572, 179.7211081206401, 380.4695888770434, 76.39654709045266, 357.4181017943673, 287.1935130368687, 338.3966573919768, 277.272284523134, 69.28865955361954, 77.59961765106212, 170.1565252164996, 124.48917321564447, 125.9566121960733, 122.55914747655109, 358.8169966923127, 112.46659565031442, 164.9213593028435, 210.93763055625195, 141.0647140528757, 220.19890466437903, 205.2479478525057, 146.6840934260825, 205.2479478525057, 305.71590326490076, 429.4477152409339, 253.44738569742526, 226.30304315814422, 363.0466179050314, 181.13067824168084, 99.2514223056366, 326.70659332761613, 419.4995696321248, 95.0770390179677, 229.86679753407884, 262.51612271179147, 206.85772745893686, 206.85772745893686, 133.0366960079705, 85.89494673838159, 115.58428452718766, 115.58428452718766, 158.60332124553608, 158.60332124553608, 159.2240770935855, 93.6030061542134, 286.07385163895896, 117.40447215712238, 124.97641006343288, 329.26898489707213, 329.26898489707213, 215.09800253180504, 249.51804818745066, 109.43300111180854, 246.61107487460225, 246.61107487460225, 179.02044192165667, 551.4217815078388, 341.05073527023916, 342.38557010684895, 279.4469580033028, 265.61057969554065, 148.99403178365867, 166.86540009848025, 314.1906602856942, 100.81440478091984, 66.37447035867858, 97.33172602285453, 247.5762833312724, 266.6501501115634, 130.4635333784854, 181.83960367209863, 198.9331323667161, 279.4469580033028, 197.38502244855516, 612.7571861239999, 642.1640174789829, 289.4459996766781, 228.97063181940615, 309.31958559164264, 247.5762833312724, 320.3875157950033, 403.4287934927351, 252.45928798182638, 361.6312332766962, 381.95870474451476, 515.9932840552193, 356.0246606707791, 356.0246606707791, 248.54526950703038, 351.87684773779097, 311.7456091692961, 225.42077120508256, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 248.54526950703038, 195.08541728258788, 187.61181231093738, 187.61181231093738, 187.61181231093738, 187.61181231093738, 187.61181231093738, 187.61181231093738, 187.61181231093738, 187.61181231093738, 278.35749756134834, 242.7877245401199, 136.19157033683516, 202.06587336346374, 199.71173463253027, 119.72006788311512, 176.24498722300464, 144.4099673433467, 121.60538462411665, 211.76321709954237, 217.63350971989422, 168.83235663184237, 427.7734572673073, 138.8777083722429, 74.62682298320091, 66.63425269055388, 306.91244147925255, 441.35244443864343, 232.57639460420748, 334.45421691786504, 170.82249902639472, 276.1913023296018, 198.9331323667161, 138.8777083722429, 108.5813867407896, 157.98498549518746, 702.5295750239817, 580.1468435322063, 194.3248513188351, 226.30304315814422, 500.11783967291126, 112.90677745995042, 434.50990901347194, 261.4926693412009, 343.72562933929424, 259.4577172438939, 228.97063181940615, 207.6673457129932, 191.3121237449989, 549.27199171561, 270.8492789701163, 335.7632337116253, 217.63350971989422, 143.8469662358426, 202.8567368308649, 120.65904401308495, 353.25405478056325, 212.59203489720244, 458.93392708825405, 194.3248513188351, 287.1935130368687, 324.16414244755515, 309.31958559164264, 654.829567836133, 199.71173463253027, 133.5573869140851, 193.56725052076862, 118.78839892336089, 146.6840934260825, 228.97063181940615, 271.90935307537654, 475.3552883908416, 28.325147094540903, 90.01713130052181, 75.21212841006654, 221.06073880966076, 326.70659332761613, 243.73796882667915, 215.9398723106141, 133.5573869140851, 515.9932840552193, 286.07385163895896, 132.51803508331912, 105.65258908406659, 161.10096012939127, 152.5273234252381, 222.79453967267415, 260.4732060371668, 183.26578935968087, 236.23893989057657, 147.834551029463, 202.06587336346374, 174.19167092744075, 159.8472625113303, 189.08326999196993, 129.9549042642197, 167.51849280886478, 65.34542957624015, 94.70636877096143, 82.60436300370014, 111.15632012308374, 64.08153455750669, 115.13366410675533, 110.7229626998625, 107.31637526839133, 90.36944839098756, 103.20514949397284, 181.83960367209863, 226.30304315814422, 90.01713130052181, 74.62682298320091, 147.834551029463, 50.5937537501147, 100.02986085865793, 66.37447035867858, 70.37979717263202, 151.3403463773431, 217.63350971989422, 78.2082389954826, 74.33588057277046, 71.76791429061606, 90.36944839098756, 144.4099673433467, 70.37979717263202, 27.995149135337257, 82.92766733936132, 106.06610162605153, 88.96839995507139, 98.4790416094955, 111.59137365930344, 116.03666862607915, 137.25973406168575, 65.34542957624015, 143.8469662358426, 202.06587336346374, 206.85772745893686, 164.9213593028435, 159.2240770935855, 106.06610162605153, 228.07795992374852, 327.98528678000895, 212.59203489720244, 183.98307137873957, 80.37625124290746, 129.44825810694724, 78.82163381626928, 300.97621750018493, 155.5356553203632, 172.1622764913071, 257.43860126944566, 247.5762833312724, 247.5762833312724, 209.29610089202527, 275.1145344862516, 312.9657469853004, 244.69193226422038, 219.3404304920078, 319.1384432445317, 319.1384432445317, 792.9667171920373, 792.9667171920373, 170.82249902639472, 122.08133464248264, 139.96693973881514, 179.02044192165667, 144.4099673433467, 65.8579401324037, 78.51433738825796, 156.7555365841094, 195.08541728258788, 299.8028224298701, 98.4790416094955, 210.93763055625195, 128.44088377025506, 94.33714363029665, 89.66618776422116, 99.63988138921447, 72.33079701870139, 84.563269020706, 153.7236100495016, 109.00636227269737, 136.19157033683516, 127.94014021950323, 73.18340956555208, 298.6340019934001, 136.19157033683516, 688.941423011762, 74.91890411002962, 294.00411093116844, 43.95661501095434, 105.24068867470233, 321.64147707721304, 177.62729378994385, 196.6154911738957, 271.90935307537654, 223.66653284614108, 739.1262534528506, 568.9257878221232, 231.6696651716735, 169.4931477924318, 118.78839892336089, 191.3121237449989, 224.54193890439277, 335.7632337116253, 99.63988138921447, 232.57639460420748, 133.0366960079705, 276.1913023296018, 235.3179315471721, 107.7363996853844, 159.2240770935855, 1022.1745001752669, 154.32526724792663, 239.02364975839862, 176.93479059943397, 228.07795992374852, 245.649629409089, 427.7734572673073, 361.6312332766962, 292.8578975283812, 102.40200064874995, 178.32250736130783, 170.82249902639472, 141.61682573322432, 179.02044192165667, 232.57639460420748, 124.48917321564447, 152.5273234252381, 450.0573414090999, 287.1935130368687, 70.10541234668786, 209.29610089202527, 294.00411093116844, 589.282826961637, 405.0077691463688, 494.29129035606036, 367.3260965541814, 398.728705684109, 264.57506218047985, 464.34369499541174, 302.15420510895507, 253.44738569742526, 174.8734378511126, 61.14702744030778, 217.63350971989422, 96.57428452170761, 432.81591541665557, 347.77735828917787, 206.85772745893686, 291.71615278815983, 53.02179868836547, 108.15806803146498, 138.8777083722429, 69.55984769951485, 81.9615307508321, 53.542126217761066, 81.641993024771, 376.03698390417554, 330.5577072665093, 330.5577072665093, 338.3966573919768, 424.4444978122241, 286.07385163895896, 203.65069565035918, 270.8492789701163, 184.70316076024525, 184.70316076024525, 395.6257742790621, 331.8514735526883, 509.98178020909006, 55.6750020519895, 637.1666574479115, 298.6340019934001, 376.03698390417554, 441.35244443864343, 314.1906602856942, 387.9736791189793, 327.98528678000895, 464.34369499541174, 384.95444399467493, 294.00411093116844, 80.37625124290746, 333.1503034969411, 287.1935130368687, 532.3726691377844, 320.3875157950033, 471.65604418826433, 242.7877245401199, 316.65488817567297, 374.5709546383618, 242.7877245401199, 310.5302282198318, 206.85772745893686, 223.66653284614108, 201.27809318052155, 166.21485355527525, 259.4577172438939, 259.4577172438939, 157.98498549518746, 192.81260332830294, 189.8233209864514, 361.6312332766962, 166.86540009848025, 315.42036776119477, 363.0466179050314, 286.07385163895896, 223.66653284614108, 266.6501501115634, 292.8578975283812, 191.3121237449989, 231.6696651716735, 373.11064087900144, 566.7077563764824, 409.7818725681369, 132.00139622598326, 351.87684773779097, 473.50205375098, 411.38571347022946, 93.6030061542134, 95.44916002730179, 122.55914747655109, 102.80279074731456, 79.43983955226133, 84.563269020706, 237.16355295871682, 169.4931477924318, 123.03883041717654, 72.04880596519355, 109.86130976812342, 126.44959243764188, 110.7229626998625, 150.7503262825534, 342.38557010684895, 69.83209724580306, 467.98558773246197, 287.1935130368687, 83.25223705241488, 166.86540009848025, 295.1548104863723, 408.18428444915037, 215.9398723106141, 417.86409580268446, 223.66653284614108, 186.15180560442604, 193.56725052076862, 225.42077120508256, 170.82249902639472, 205.2479478525057, 121.60538462411665, 498.16806499462353, 124.97641006343288, 178.32250736130783, 739.1262534528506, 377.5087510463025, 557.921766975976, 771.577792875611, 49.037147591523336, 174.19167092744075, 233.4866728754002, 208.48013272330223, 532.3726691377844, 179.7211081206401, 335.7632337116253, 209.29610089202527, 271.90935307537654, 324.16414244755515, 242.7877245401199, 409.7818725681369, 277.272284523134, 367.3260965541814, 373.11064087900144, 462.53339048974414, 353.25405478056325, 133.0366960079705, 368.76377024718164, 302.15420510895507, 275.1145344862516, 194.3248513188351, 158.60332124553608, 139.96693973881514, 165.5668432526847, 92.87458178129451, 316.65488817567297, 370.20707083592094, 564.4983722159518, 739.1262534528506, 200.49338426145144, 338.3966573919768, 288.31755666558627, 164.27839185643572, 176.93479059943397, 378.98627852215554, 349.1385203839046, 360.2213667100846, 427.7734572673073, 311.7456091692961, 184.70316076024525, 77.90333396724647, 104.01459751366006, 294.00411093116844, 191.3121237449989, 255.43519818017026, 175.55787313507827, 421.1414445251494, 183.98307137873957, 116.49082330627213, 125.9566121960733, 267.6937892911538, 176.93479059943397, 513.9816168847873, 424.4444978122241, 179.7211081206401, 90.01713130052181, 149.57717793613415, 136.72460907194932, 95.44916002730179, 86.56862915519338, 479.0835461252901, 114.68480048885151, 181.83960367209863, 564.4983722159518, 98.4790416094955, 494.29129035606036, 95.44916002730179, 538.6481096972173, 358.8169966923127, 341.05073527023916, 279.4469580033028, 571.1525003973912, 409.7818725681369, 321.64147707721304, 384.95444399467493, 557.921766975976, 93.6030061542134, 190.56626845863, 586.9854309417088, 349.1385203839046, 236.23893989057657, 257.43860126944566, 179.7211081206401, 56.11166650505244, 279.4469580033028, 162.3644906197382, 321.64147707721304, 183.26578935968087, 320.3875157950033, 189.08326999196993, 265.61057969554065, 127.44134888284603, 360.2213667100846, 847.4124245067931, 116.03666862607915, 77.90333396724647, 76.39654709045266, 108.15806803146498, 81.32370105824417, 243.73796882667915, 305.71590326490076, 137.25973406168575, 90.01713130052181, 108.5813867407896, 305.71590326490076, 97.71267162761835, 81.641993024771, 51.49095957359226, 59.73055807527518, 68.21443845541918, 221.06073880966076, 486.6280141983472, 259.4577172438939, 141.0647140528757, 50.1020787877761, 65.60118435636834, 29.396022468352484, 52.50652775191775, 118.3252868447224, 128.94358717585138, 62.597087408320306, 40.892124929554, 66.11570082213345, 101.20898120494374, 174.8734378511126, 117.86398026897247, 154.92927926604594, 95.0770390179677, 114.68480048885151, 137.7969534714142, 51.99626426796157, 95.0770390179677, 87.58907276183992, 88.62154503481175, 49.61518196998445, 93.96935796203766, 69.01852867010916, 67.41971617548266, 57.66713919909174, 126.94450214932196, 125.4655539023251, 138.8777083722429, 138.33627549846275, 186.15180560442604, 72.61389175398708, 90.36944839098756, 57.66713919909174, 67.41971617548266, 130.4635333784854, 64.58413226217957, 137.7969534714142, 65.60118435636834, 146.11222634120347, 327.98528678000895, 103.20514949397284, 174.19167092744075, 126.44959243764188, 202.8567368308649, 89.66618776422116, 70.93179276322707, 112.90677745995042, 86.56862915519338, 101.20898120494374, 166.21485355527525, 77.59961765106212, 132.51803508331912, 146.11222634120347, 98.4790416094955, 186.88038317169537, 169.4931477924318, 167.51849280886478, 267.6937892911538, 181.83960367209863, 32.09659661306934, 86.56862915519338, 71.48811770889668, 56.9952966774873, 76.69555472159655, 90.7231444115574, 121.13129015902297, 85.89494673838159, 105.65258908406659, 111.59137365930344, 57.442316329639176, 108.5813867407896, 86.90744918987926, 95.82273747708687, 90.36944839098756, 104.01459751366006, 147.834551029463, 147.834551029463, 83.57807709540792, 45.263456176258764, 109.00636227269737, 60.199030648454944, 132.00139622598326, 114.68480048885151, 81.641993024771, 174.19167092744075, 50.79177210350426, 91.07822475921115, 93.96935796203766, 145.54258875480053, 193.56725052076862, 74.04607243930377, 30.74665629604239, 41.53608219643496, 53.646802873050504, 137.25973406168575, 137.25973406168575, 70.10541234668786, 109.00636227269737, 113.7923162908356, 129.9549042642197, 135.13171912803952, 141.0647140528757, 137.25973406168575, 74.91890411002962, 54.811841150452466, 142.1710983175922, 142.1710983175922, 132.00139622598326, 132.00139622598326, 96.19777706766972, 129.44825810694724, 82.28231911219243, 80.06289393593435, 92.51249810131188, 46.70027251047952, 65.60118435636834, 76.09870517958821, 169.4931477924318, 103.20514949397284, 69.55984769951485, 120.65904401308495, 190.56626845863, 56.11166650505244, 52.71203249107963, 143.8469662358426, 176.93479059943397, 135.66060972280735, 96.95226558425567, 67.1568716116849, 58.57526450222688, 121.60538462411665, 81.9615307508321, 46.7915731119465, 56.33128135834348, 89.31661242710538, 34.84052601883783, 81.9615307508321, 69.55984769951485, 63.83170433047907, 50.89107172411317, 136.19157033683516, 104.83039411284926, 58.34690118774256, 48.27689708476307, 50.004318665090636, 66.63425269055388, 86.56862915519338, 82.60436300370014, 121.60538462411665, 71.20941195067294, 104.01459751366006, 84.89424029774196, 90.7231444115574, 72.61389175398708, 124.00383592430612, 85.89494673838159, 80.69083499570341, 74.62682298320091, 57.66713919909174, 25.63966650134361, 174.8734378511126, 80.69083499570341, 102.40200064874995, 156.7555365841094, 142.7275402635186, 48.27689708476307, 147.25819873544805, 117.86398026897247, 119.2533235714264, 147.25819873544805, 140.51475485198003, 29.053548433146837, 84.89424029774196, 49.61518196998445, 192.81260332830294, 147.25819873544805, 118.78839892336089, 141.0647140528757, 36.08717359043372, 58.80452160545304, 125.9566121960733, 64.83690740886694, 120.65904401308495, 126.94450214932196, 536.5481197318883, 148.4131591025766, 43.275130861102994, 51.894808174781225, 342.38557010684895, 46.88305220915621, 159.2240770935855, 351.87684773779097, 305.71590326490076, 207.6673457129932, 253.44738569742526, 176.24498722300464, 60.199030648454944, 47.435694792893365, 434.50990901347194, 114.68480048885151, 268.7415131590023, 142.7275402635186, 112.90677745995042, 305.71590326490076, 161.10096012939127, 249.51804818745066, 221.06073880966076, 210.11526266986428, 62.3530444922685, 162.3644906197382, 160.47288700783744, 325.4328850285552, 598.5626811957673, 325.4328850285552, 64.58413226217957, 564.4983722159518, 274.0419645628891, 39.248839424943355, 289.4459996766781, 110.7229626998625, 206.05126560732333, 342.38557010684895, 373.11064087900144, 330.5577072665093, 268.7415131590023, 280.5406824728656, 97.33172602285453, 259.4577172438939, 108.5813867407896, 132.00139622598326, 303.3368032308679, 215.09800253180504, 247.5762833312724, 206.05126560732333, 217.63350971989422, 195.84896001735729, 93.6030061542134, 215.09800253180504, 63.582848099201826, 72.61389175398708, 95.82273747708687, 78.82163381626928, 74.62682298320091, 84.89424029774196, 152.5273234252381, 298.6340019934001, 71.20941195067294, 180.4245166495895, 48.75066069158903, 281.63868765896893, 247.5762833312724, 346.4215028625498, 144.97517197488295, 77.90333396724647, 92.87458178129451, 135.66060972280735, 72.61389175398708, 172.83610058973207, 60.908637772317434, 392.54699007778345, 341.05073527023916, 212.59203489720244, 145.54258875480053, 146.6840934260825, 174.8734378511126, 200.49338426145144, 133.5573869140851, 183.26578935968087, 515.9932840552193, 680.9150123157852, 303.3368032308679, 496.225891767423, 212.59203489720244, 164.9213593028435, 214.2594148922178, 409.7818725681369, 444.8140145626792, 783.7283751097743, 236.23893989057657, 179.02044192165667, 316.65488817567297, 311.7456091692961, 283.84760726337464, 270.8492789701163, 57.21836996148928, 113.7923162908356, 59.03467599561125, 92.87458178129451, 502.07524555352444, 193.56725052076862, 83.25223705241488, 71.76791429061606, 137.25973406168575, 66.37447035867858, 126.44959243764188, 51.39048944047805, 192.0608982264211, 199.71173463253027, 89.66618776422116, 72.33079701870139, 162.3644906197382, 84.563269020706, 169.4931477924318, 267.6937892911538, 377.5087510463025, 287.1935130368687, 234.40051387501347, 176.24498722300464, 304.5240299109616, 35.18243227767251, 202.8567368308649, 334.45421691786504, 377.5087510463025, 262.51612271179147, 460.7301436924908, 353.25405478056325, 338.3966573919768, 106.89798837141365, 272.97357619337527, 118.3252868447224, 316.65488817567297, 368.76377024718164, 280.5406824728656, 171.49107938408403, 277.272284523134, 165.5668432526847, 509.98178020909006, 596.2291064528091, 228.97063181940615, 464.34369499541174, 35.87634357692165, 153.12429849019884, 63.83170433047907, 78.2082389954826, 176.24498722300464, 125.9566121960733, 170.1565252164996, 58.80452160545304, 64.08153455750669, 148.99403178365867, 88.96839995507139, 135.66060972280735, 46.246434773093995, 99.2514223056366, 54.38529201854992, 71.76791429061606, 118.78839892336089, 129.44825810694724, 130.4635333784854, 150.1626064581185, 80.69083499570341, 116.94675549762589, 111.59137365930344, 98.4790416094955, 151.3403463773431, 118.3252868447224, 94.70636877096143, 174.19167092744075, 158.60332124553608, 300.97621750018493, 199.71173463253027, 198.9331323667161, 194.3248513188351, 42.3553080116137, 81.9615307508321, 97.33172602285453, 83.57807709540792, 122.08133464248264, 110.7229626998625, 135.13171912803952, 104.4216991379013, 234.40051387501347, 125.9566121960733, 61.867809250367884, 65.34542957624015, 44.21492934326177, 107.7363996853844, 97.33172602285453, 72.61389175398708, 92.51249810131188, 91.79256012938836, 75.21212841006654, 43.3597351955915, 116.49082330627213, 110.7229626998625, 209.29610089202527, 139.42126035437516, 122.08133464248264, 179.7211081206401, 231.6696651716735, 174.8734378511126, 79.13013296849392, 78.51433738825796, 88.27604237373367, 81.00664999449546, 118.3252868447224, 143.28616006164455, 71.48811770889668, 161.73149145963927, 111.59137365930344, 162.99996726850102, 151.93267574549122, 155.5356553203632, 155.5356553203632, 62.10995300937982, 73.46984133199462, 122.08133464248264, 119.72006788311512, 138.8777083722429, 158.60332124553608, 160.47288700783744, 111.59137365930344, 89.31661242710538, 127.94014021950323, 168.83235663184237, 175.55787313507827, 200.49338426145144, 121.13129015902297, 100.02986085865793, 54.811841150452466, 78.2082389954826, 125.4655539023251, 125.4655539023251, 104.4216991379013, 70.10541234668786, 204.44776193681219, 81.32370105824417, 97.71267162761835, 63.582848099201826, 108.5813867407896, 154.92927926604594, 157.3690604074729, 62.597087408320306, 171.49107938408403, 61.626609520407456, 78.82163381626928, 26.14536361045317, 102.80279074731456, 34.97688798216538, 193.56725052076862, 102.40200064874995, 38.64034257045414, 184.70316076024525, 266.6501501115634, 103.60908302823987, 288.31755666558627, 334.45421691786504, 166.86540009848025, 248.54526950703038, 174.8734378511126, 86.23113005463887, 135.13171912803952, 127.44134888284603, 115.58428452718766, 157.3690604074729, 130.4635333784854, 150.1626064581185, 232.57639460420748, 180.4245166495895, 192.0608982264211, 627.2883040361144, 175.55787313507827, 214.2594148922178, 93.96935796203766, 229.86679753407884, 289.4459996766781, 317.89424036643936, 61.38635013907805, 349.1385203839046, 73.75739416067401, 376.03698390417554, 158.60332124553608, 213.42409659600423, 127.44134888284603, 325.4328850285552, 112.90677745995042, 231.6696651716735, 154.92927926604594, 228.97063181940615, 96.19777706766972, 260.4732060371668, 223.66653284614108, 272.97357619337527, 295.1548104863723, 124.97641006343288, 89.31661242710538, 123.03883041717654, 157.98498549518746, 156.14440466345445, 434.50990901347194, 249.51804818745066, 103.20514949397284, 84.563269020706, 351.87684773779097, 189.08326999196993, 326.70659332761613, 118.78839892336089, 229.86679753407884, 129.44825810694724, 252.45928798182638, 276.1913023296018, 175.55787313507827, 331.8514735526883, 143.8469662358426, 114.23768682435218, 126.94450214932196, 98.4790416094955, 156.7555365841094, 68.21443845541918, 498.16806499462353, 217.63350971989422, 210.11526266986428, 160.47288700783744, 188.3461041828892, 114.23768682435218, 141.0647140528757, 110.7229626998625, 150.1626064581185, 192.81260332830294, 106.89798837141365, 125.4655539023251, 189.08326999196993, 195.84896001735729, 299.8028224298701, 166.21485355527525, 236.23893989057657, 374.5709546383618, 387.9736791189793, 507.9935496710831, 528.2297121579952, 41.698649825700315, 384.95444399467493, 422.78974553481265, 117.86398026897247, 92.87458178129451, 265.61057969554065, 280.5406824728656, 165.5668432526847, 176.93479059943397, 200.49338426145144, 327.98528678000895, 242.7877245401199, 162.99996726850102, 284.95855538715824, 232.57639460420748, 147.834551029463, 255.43519818017026, 259.4577172438939, 191.3121237449989, 324.16414244755515, 287.1935130368687, 272.97357619337527, 100.42136666458902, 488.53262239485935, 110.29129477712178, 252.45928798182638, 288.31755666558627, 210.93763055625195, 141.0647140528757, 392.54699007778345, 371.6560203434465, 357.4181017943673, 278.35749756134834, 394.08337555704935, 274.0419645628891, 380.4695888770434, 276.1913023296018, 229.86679753407884, 107.31637526839133, 306.91244147925255, 306.91244147925255, 272.97357619337527, 166.21485355527525, 230.76647074218755, 235.3179315471721, 179.02044192165667, 210.93763055625195, 342.38557010684895, 325.4328850285552, 185.4260684919039, 391.0165943978527, 288.31755666558627, 490.4446850170856, 287.1935130368687, 223.66653284614108, 363.0466179050314, 238.09178486008656, 182.5513037582002, 240.8983354780937, 168.83235663184237, 171.49107938408403, 422.78974553481265, 422.78974553481265, 349.1385203839046, 261.4926693412009, 257.43860126944566, 166.21485355527525, 220.19890466437903, 448.3027341176846, 245.649629409089, 302.15420510895507, 286.07385163895896, 335.7632337116253, 335.7632337116253, 327.98528678000895, 341.05073527023916, 252.45928798182638, 159.2240770935855, 247.5762833312724, 227.18876822599648, 157.3690604074729, 57.66713919909174, 327.98528678000895, 192.81260332830294, 199.71173463253027, 134.60489048229144, 186.88038317169537, 132.51803508331912, 311.7456091692961, 241.84118490494708, 282.74099031586354, 210.93763055625195, 153.12429849019884, 225.42077120508256, 144.4099673433467, 228.07795992374852, 198.9331323667161, 277.272284523134, 210.11526266986428, 276.1913023296018, 276.1913023296018, 152.5273234252381, 71.20941195067294, 163.63793110254423, 297.46973835596765, 247.5762833312724, 172.83610058973207, 221.06073880966076, 171.49107938408403, 252.45928798182638, 139.42126035437516, 365.89402781973274, 221.92594607841514, 139.96693973881514, 228.07795992374852, 283.84760726337464, 242.7877245401199, 250.49463421597682, 250.49463421597682, 295.1548104863723, 108.5813867407896, 276.1913023296018, 227.18876822599648, 157.98498549518746, 164.9213593028435, 197.38502244855516, 221.92594607841514, 222.79453967267415, 90.36944839098756, 118.78839892336089, 257.43860126944566, 291.71615278815983, 124.00383592430612, 150.7503262825534, 143.8469662358426, 161.73149145963927, 233.4866728754002, 383.45364884670335, 128.94358717585138, 256.43494327896633, 403.4287934927351, 210.11526266986428, 417.86409580268446, 108.5813867407896, 156.14440466345445, 173.5125619611118, 164.9213593028435, 142.1710983175922, 112.46659565031442, 266.6501501115634, 149.57717793613415, 329.26898489707213, 16.139182322446413, 193.56725052076862, 257.43860126944566, 350.5050099164416, 166.86540009848025, 427.7734572673073, 133.0366960079705, 221.06073880966076, 300.97621750018493, 167.51849280886478, 226.30304315814422, 261.4926693412009, 169.4931477924318, 206.85772745893686, 392.54699007778345, 255.43519818017026, 477.21577638600166, 498.16806499462353, 239.95916187280093, 176.24498722300464, 239.02364975839862, 1022.1745001752669, 786.7958012412769, 637.1666574479115, 102.80279074731456, 242.7877245401199, 204.44776193681219, 354.636652059297, 297.46973835596765, 246.61107487460225, 292.8578975283812, 136.72460907194932, 164.9213593028435, 146.11222634120347, 183.98307137873957, 419.4995696321248, 275.1145344862516, 186.88038317169537, 231.6696651716735, 169.4931477924318, 195.08541728258788, 235.3179315471721, 166.21485355527525, 122.55914747655109, 276.1913023296018, 91.07822475921115, 166.86540009848025, 208.48013272330223, 419.4995696321248, 136.72460907194932, 88.27604237373367, 200.49338426145144, 233.4866728754002, 148.99403178365867, 353.25405478056325, 384.95444399467493, 302.15420510895507, 106.06610162605153, 281.63868765896893, 383.45364884670335, 426.1057266069316, 118.78839892336089, 231.6696651716735, 179.02044192165667, 59.49769069799599, 258.44618746623325, 311.7456091692961, 305.71590326490076, 305.71590326490076, 221.92594607841514, 291.71615278815983, 864.1261369598841, 192.0608982264211, 319.1384432445317, 240.8983354780937, 280.5406824728656, 308.1136628117644, 555.7466361419574, 395.6257742790621, 242.7877245401199, 224.54193890439277, 197.38502244855516, 343.72562933929424, 266.6501501115634, 263.5435817656175, 142.7275402635186, 217.63350971989422, 229.86679753407884, 387.9736791189793, 241.84118490494708, 367.3260965541814, 409.7818725681369, 315.42036776119477, 450.0573414090999, 175.55787313507827, 239.95916187280093, 70.65525591042105, 96.57428452170761, 102.80279074731456, 119.2533235714264, 132.51803508331912, 252.45928798182638, 176.93479059943397, 106.48123261036584, 172.83610058973207, 68.21443845541918, 145.54258875480053, 157.3690604074729, 139.42126035437516, 48.37128004023317, 292.8578975283812, 471.65604418826433, 139.42126035437516, 333.1503034969411, 358.8169966923127, 290.5788592888401, 364.467542192173, 195.84896001735729, 152.5273234252381, 156.14440466345445, 124.00383592430612, 222.79453967267415, 278.35749756134834, 43.3597351955915, 394.08337555704935, 394.08337555704935, 111.59137365930344, 159.2240770935855, 86.90744918987926, 228.97063181940615, 95.44916002730179, 68.21443845541918, 74.62682298320091, 154.32526724792663, 401.85597368179066, 177.62729378994385, 256.43494327896633, 286.07385163895896, 298.6340019934001, 147.25819873544805, 54.598150033144236, 80.06289393593435, 165.5668432526847, 322.9003462251159, 114.68480048885151, 141.0647140528757, 629.7434410413515, 584.6969916200433, 573.3879280792671, 515.9932840552193, 189.8233209864514, 353.25405478056325, 174.19167092744075, 37.524723159601, 102.40200064874995, 310.5302282198318, 106.89798837141365, 310.5302282198318, 234.40051387501347, 471.65604418826433, 139.42126035437516, 283.84760726337464, 221.06073880966076, 115.13366410675533, 191.3121237449989, 112.46659565031442, 119.2533235714264, 156.7555365841094, 66.11570082213345, 157.98498549518746, 56.773093073800744, 55.89290784893326, 380.4695888770434, 164.27839185643572, 370.20707083592094, 224.54193890439277, 580.1468435322063, 219.3404304920078, 350.5050099164416, 314.1906602856942, 299.8028224298701, 49.133016862271724, 124.00383592430612, 142.1710983175922, 117.86398026897247, 115.13366410675533, 88.96839995507139, 88.96839995507139, 81.00664999449546, 368.76377024718164, 203.65069565035918, 39.634006510082365, 56.33128135834348, 118.78839892336089, 409.7818725681369, 221.06073880966076, 294.00411093116844, 156.14440466345445, 156.7555365841094, 197.38502244855516, 212.59203489720244, 249.51804818745066, 188.3461041828892, 51.39048944047805, 96.57428452170761, 314.1906602856942, 134.08011574678568, 350.5050099164416, 129.44825810694724, 142.7275402635186, 133.5573869140851, 120.1886389803891, 268.7415131590023, 52.81508628524973, 43.61454191906839, 179.02044192165667, 75.80202444429054, 148.4131591025766, 303.3368032308679, 63.83170433047907, 43.3597351955915, 45.79700747446617, 109.00636227269737, 115.58428452718766, 168.83235663184237, 53.43765380953904, 31.9714638416083, 331.8514735526883, 166.21485355527525, 147.25819873544805, 98.09510821132314, 127.94014021950323, 131.48677155266952, 267.6937892911538, 121.13129015902297, 153.12429849019884, 246.61107487460225, 103.60908302823987, 181.83960367209863, 183.26578935968087, 45.7076474882609, 414.6122516103204, 98.09510821132314, 76.09870517958821, 314.1906602856942, 237.16355295871682, 451.81881603928605, 159.2240770935855, 259.4577172438939, 168.17414165184545, 206.85772745893686, 172.1622764913071, 357.4181017943673, 297.46973835596765, 286.07385163895896, 80.69083499570341, 184.70316076024525, 142.7275402635186, 205.2479478525057, 69.01852867010916, 133.0366960079705, 607.9886718701034, 96.57428452170761, 130.4635333784854, 189.8233209864514, 150.1626064581185, 139.96693973881514, 122.08133464248264, 98.09510821132314, 100.42136666458902, 93.2380826167318, 100.81440478091984, 170.1565252164996, 203.65069565035918, 132.00139622598326, 198.9331323667161, 126.44959243764188, 231.6696651716735, 150.7503262825534, 189.8233209864514, 231.6696651716735, 524.1189959252079, 203.65069565035918, 180.4245166495895, 275.1145344862516, 154.92927926604594, 170.1565252164996, 106.48123261036584, 138.8777083722429, 41.617286632367644, 353.25405478056325, 105.24068867470233, 150.1626064581185, 71.48811770889668, 55.457945789238345, 50.990565478815945, 183.26578935968087, 46.7915731119465, 53.85677052285177, 77.90333396724647, 112.90677745995042, 62.597087408320306, 109.86130976812342, 134.08011574678568, 198.9331323667161, 93.2380826167318, 104.4216991379013, 38.867415430562446, 113.34868209247811, 60.908637772317434, 130.4635333784854, 172.1622764913071, 102.40200064874995, 79.13013296849392, 173.5125619611118, 77.29708541258049, 16.782094257242417, 64.08153455750669, 58.57526450222688, 253.44738569742526, 157.98498549518746, 97.33172602285453, 74.04607243930377, 96.57428452170761, 86.23113005463887, 87.24759532868651, 59.964336869699544, 39.78912962559818, 126.44959243764188, 129.44825810694724, 130.4635333784854, 86.23113005463887, 156.14440466345445, 56.33128135834348, 23.4365524707797, 54.38529201854992, 98.4790416094955, 190.56626845863, 50.1020787877761, 55.89290784893326, 276.1913023296018, 106.48123261036584, 118.3252868447224, 96.95226558425567, 98.09510821132314, 107.31637526839133, 231.6696651716735, 54.067559962798995, 116.49082330627213, 194.3248513188351, 139.42126035437516, 157.98498549518746, 76.09870517958821, 58.57526450222688, 71.76791429061606, 111.15632012308374, 210.11526266986428, 67.68358948381629, 178.32250736130783, 28.940279382223952, 32.9863470466753, 49.51837188956857, 98.4790416094955, 90.7231444115574, 184.70316076024525, 78.82163381626928, 88.96839995507139, 89.31661242710538, 82.28231911219243, 162.99996726850102, 123.52039078374908, 122.08133464248264, 243.73796882667915, 58.119428177447986, 141.0647140528757, 19.735555861572347, 113.7923162908356, 62.84208548133934, 95.82273747708687, 100.02986085865793, 49.51837188956857, 47.435694792893365, 92.51249810131188, 103.20514949397284, 138.33627549846275, 104.01459751366006, 115.58428452718766, 92.1518260518184, 98.09510821132314, 143.28616006164455, 41.698649825700315, 52.50652775191775, 14.694892728788941, 97.33172602285453, 123.03883041717654, 127.94014021950323, 159.8472625113303, 44.12865633779556, 135.13171912803952, 134.60489048229144, 124.00383592430612, 124.00383592430612, 32.66578255521719, 126.94450214932196, 150.1626064581185, 188.3461041828892, 85.56007407666469, 54.811841150452466, 52.097918711212095, 53.22932014095188, 48.75066069158903, 97.71267162761835, 42.68750480955783, 98.86447768050158, 97.33172602285453, 53.02179868836547, 63.3349620664254, 80.69083499570341, 68.48142221881122, 249.51804818745066, 47.71445419747699, 30.26997357328364, 95.44916002730179, 46.88305220915621, 242.7877245401199, 142.1710983175922, 198.15756558346607, 289.4459996766781, 66.37447035867858, 349.1385203839046, 119.72006788311512, 137.25973406168575, 214.2594148922178, 181.83960367209863, 121.13129015902297, 183.98307137873957, 398.728705684109, 93.2380826167318, 281.63868765896893, 124.00383592430612, 115.13366410675533, 41.780172086809024, 112.02812994692015, 374.5709546383618, 31.846818916590372, 109.43300111180854, 56.11166650505244, 72.04880596519355, 57.89284200037607, 96.57428452170761, 179.02044192165667, 81.32370105824417, 90.36944839098756, 28.049880592281973, 264.57506218047985, 44.73612093895834, 85.56007407666469, 76.99573263552006, 43.275130861102994, 111.59137365930344, 117.40447215712238, 86.23113005463887, 161.73149145963927, 50.49503426195374, 193.56725052076862, 222.79453967267415, 130.9741532108186, 134.08011574678568, 101.20898120494374, 228.97063181940615, 121.60538462411665, 114.23768682435218, 186.15180560442604, 145.54258875480053, 289.4459996766781, 163.63793110254423, 146.11222634120347, 257.43860126944566, 223.66653284614108, 278.35749756134834, 54.49161709130268, 85.2265069597304, 155.5356553203632, 173.5125619611118, 181.83960367209863, 148.4131591025766, 103.20514949397284, 89.31661242710538, 145.54258875480053, 101.6051019574268, 192.0608982264211, 132.00139622598326, 125.4655539023251, 321.64147707721304, 65.8579401324037, 327.98528678000895, 287.1935130368687, 208.48013272330223, 78.2082389954826, 189.08326999196993, 116.03666862607915, 197.38502244855516, 143.8469662358426, 168.17414165184545, 90.01713130052181, 216.78503707457477, 133.0366960079705, 108.15806803146498, 473.50205375098, 96.95226558425567, 124.97641006343288, 87.24759532868651, 110.29129477712178, 123.03883041717654, 263.5435817656175, 166.21485355527525, 218.48530319325408, 91.79256012938836, 179.02044192165667, 183.26578935968087, 282.74099031586354, 242.7877245401199, 279.4469580033028, 105.65258908406659, 161.10096012939127, 110.7229626998625, 59.964336869699544, 170.82249902639472, 80.06289393593435, 65.09067188950593, 62.10995300937982, 134.08011574678568, 55.89290784893326, 205.2479478525057, 204.44776193681219, 159.2240770935855, 170.82249902639472, 84.2335880783952, 161.73149145963927, 276.1913023296018, 159.2240770935855, 64.83690740886694, 103.20514949397284, 115.13366410675533, 70.65525591042105, 155.5356553203632, 81.00664999449546, 50.396507397123145, 106.89798837141365, 124.00383592430612, 65.09067188950593, 210.11526266986428, 672.9821121368258, 458.93392708825405, 287.1935130368687, 219.3404304920078, 268.7415131590023, 115.13366410675533, 26.09434828302453, 350.5050099164416, 129.44825810694724, 96.19777706766972, 120.65904401308495, 166.21485355527525, 242.7877245401199, 411.38571347022946, 230.76647074218755, 79.13013296849392, 61.867809250367884, 232.57639460420748, 300.97621750018493, 91.07822475921115, 62.84208548133934, 40.57389988868238, 71.20941195067294, 50.990565478815945, 38.33964103892552, 151.93267574549122, 245.649629409089, 64.83690740886694, 142.7275402635186, 254.43935071813848, 53.43765380953904, 166.21485355527525, 95.82273747708687, 67.41971617548266, 174.19167092744075, 81.9615307508321, 220.19890466437903, 64.08153455750669, 83.57807709540792, 33.50580642730562, 263.5435817656175, 28.2698785323965, 216.78503707457477, 209.29610089202527, 234.40051387501347, 316.65488817567297, 87.93188669987823, 316.65488817567297, 86.90744918987926, 206.85772745893686, 85.89494673838159, 262.51612271179147, 82.28231911219243, 469.8172315347884, 319.1384432445317, 207.6673457129932, 403.4287934927351, 354.636652059297, 125.9566121960733, 90.01713130052181, 135.66060972280735, 192.81260332830294, 169.4931477924318, 83.90519244027122, 99.63988138921447, 39.556671887793456, 164.27839185643572, 183.26578935968087, 192.0608982264211, 175.55787313507827, 278.35749756134834, 290.5788592888401, 341.05073527023916, 356.0246606707791, 350.5050099164416, 417.86409580268446, 319.1384432445317, 201.27809318052155, 170.82249902639472, 148.99403178365867, 429.4477152409339, 358.8169966923127, 175.55787313507827, 895.045898320867, 282.74099031586354, 226.30304315814422, 317.89424036643936, 317.89424036643936, 317.89424036643936, 431.12852607499315, 253.44738569742526, 329.26898489707213, 139.96693973881514, 389.49216516524257, 484.7308313654974, 209.29610089202527, 283.84760726337464, 280.5406824728656, 219.3404304920078, 249.51804818745066, 353.25405478056325, 330.5577072665093, 287.1935130368687, 225.42077120508256, 227.18876822599648, 101.6051019574268, 134.08011574678568, 127.94014021950323, 76.99573263552006, 204.44776193681219, 371.6560203434465, 412.9958316281291, 178.32250736130783, 108.5813867407896, 139.96693973881514, 54.811841150452466, 100.02986085865793, 103.20514949397284, 179.7211081206401, 169.4931477924318, 119.2533235714264, 283.84760726337464, 142.7275402635186, 88.96839995507139, 111.59137365930344, 397.1742097789884, 51.79355004464666, 176.93479059943397, 184.70316076024525, 221.92594607841514, 60.908637772317434, 241.84118490494708, 208.48013272330223, 140.51475485198003, 274.0419645628891, 113.7923162908356, 123.03883041717654, 180.4245166495895, 110.29129477712178, 351.87684773779097, 251.47504249414803, 381.95870474451476, 288.31755666558627, 250.49463421597682, 518.012824668342, 289.4459996766781, 141.61682573322432, 106.06610162605153, 142.1710983175922, 317.89424036643936, 453.58718488624703, 412.9958316281291, 282.74099031586354, 80.06289393593435, 334.45421691786504, 315.42036776119477, 256.43494327896633, 124.48917321564447, 280.5406824728656, 269.79333770212673, 232.57639460420748, 252.45928798182638, 337.0773738522583, 528.2297121579952, 267.6937892911538, 267.6937892911538, 37.524723159601, 109.00636227269737, 177.62729378994385, 200.49338426145144, 109.00636227269737, 43.95661501095434, 265.61057969554065, 109.00636227269737, 174.8734378511126, 177.62729378994385, 136.19157033683516, 136.19157033683516, 189.8233209864514, 350.5050099164416, 250.49463421597682, 186.88038317169537, 102.80279074731456, 241.84118490494708, 308.1136628117644, 291.71615278815983, 370.20707083592094, 120.1886389803891, 612.7571861239999, 205.2479478525057, 76.39654709045266, 154.32526724792663, 140.51475485198003, 87.58907276183992, 66.11570082213345, 55.02636863114642, 268.7415131590023, 99.2514223056366, 214.2594148922178, 158.60332124553608, 116.03666862607915, 84.89424029774196, 280.5406824728656, 211.76321709954237, 126.94450214932196, 389.49216516524257, 316.65488817567297, 316.65488817567297, 51.1901369093998, 37.160054292819765, 268.7415131590023, 417.86409580268446, 265.61057969554065, 120.65904401308495, 181.13067824168084, 195.08541728258788, 538.6481096972173, 61.38635013907805, 174.19167092744075, 74.62682298320091, 92.51249810131188, 67.1568716116849, 110.7229626998625, 245.649629409089, 325.4328850285552, 100.81440478091984, 186.15180560442604, 67.1568716116849, 125.9566121960733, 451.81881603928605, 705.279198037542, 364.467542192173, 80.06289393593435, 93.96935796203766, 215.9398723106141, 129.44825810694724, 79.75075829332496, 280.5406824728656, 221.92594607841514, 670.3584035283843, 60.908637772317434, 62.10995300937982, 159.8472625113303, 254.43935071813848, 96.95226558425567, 380.4695888770434, 786.7958012412769, 316.65488817567297, 119.72006788311512, 279.4469580033028, 186.88038317169537, 159.8472625113303, 241.84118490494708, 161.10096012939127, 239.02364975839862, 47.71445419747699, 43.5294404008871, 59.265731184583274, 168.17414165184545, 88.27604237373367, 60.199030648454944, 73.18340956555208, 90.7231444115574, 137.25973406168575, 156.7555365841094, 170.82249902639472, 113.7923162908356, 86.23113005463887, 184.70316076024525, 108.5813867407896, 95.0770390179677, 117.86398026897247, 79.75075829332496, 297.46973835596765, 434.50990901347194, 249.51804818745066, 241.84118490494708, 71.20941195067294, 66.63425269055388, 45.086990684122, 398.728705684109, 162.99996726850102, 215.9398723106141, 159.8472625113303, 93.6030061542134, 228.97063181940615, 55.457945789238345, 263.5435817656175, 190.56626845863, 202.06587336346374, 179.7211081206401, 147.834551029463, 70.93179276322707, 103.20514949397284, 439.63177432431667, 72.33079701870139, 195.84896001735729, 296.31001375229687, 331.8514735526883, 458.93392708825405, 654.829567836133, 624.8427387029609, 394.08337555704935, 206.85772745893686, 83.90519244027122, 484.7308313654974, 343.72562933929424, 137.7969534714142, 303.3368032308679, 263.5435817656175, 203.65069565035918, 265.61057969554065, 246.61107487460225, 251.47504249414803, 329.26898489707213, 274.0419645628891, 198.9331323667161, 334.45421691786504, 147.25819873544805, 206.85772745893686, 125.4655539023251, 119.2533235714264, 383.45364884670335, 225.42077120508256, 143.28616006164455, 83.25223705241488, 267.6937892911538, 77.90333396724647, 86.90744918987926, 205.2479478525057, 309.31958559164264, 239.95916187280093, 471.65604418826433, 373.11064087900144, 274.0419645628891, 125.9566121960733, 100.42136666458902, 441.35244443864343, 221.92594607841514, 15.920030957803608, 311.7456091692961, 121.60538462411665, 72.89809449073893, 392.54699007778345, 437.9178124670295, 357.4181017943673, 124.48917321564447, 189.8233209864514, 106.89798837141365, 38.19016894386643, 39.556671887793456, 114.68480048885151, 157.3690604074729, 129.9549042642197, 114.23768682435218, 41.45503620813183, 254.43935071813848, 213.42409659600423, 155.5356553203632, 126.44959243764188, 29.110349096271467, 206.05126560732333, 170.82249902639472, 77.59961765106212, 144.97517197488295, 170.82249902639472, 254.43935071813848, 168.17414165184545, 354.636652059297, 89.66618776422116, 191.3121237449989, 150.1626064581185, 109.00636227269737, 163.63793110254423, 158.60332124553608, 21.506589466713315, 94.33714363029665, 540.7563187908823, 72.33079701870139, 78.82163381626928, 88.27604237373367, 92.87458178129451, 219.3404304920078, 98.4790416094955, 39.78912962559818, 23.073202403172917, 575.6321049777138, 571.1525003973912, 560.1054110313753, 134.08011574678568, 187.61181231093738, 341.05073527023916, 227.18876822599648, 119.2533235714264, 245.649629409089, 276.1913023296018, 269.79333770212673, 104.4216991379013, 83.57807709540792, 314.1906602856942, 70.37979717263202, 183.98307137873957, 36.299242562272255, 20.92648590822237, 30.56702687878971, 34.16664924473404, 22.67116267165818, 29.800681131145613, 22.059646737841184, 25.891280453627324, 31.35307975333966, 21.506589466713315, 23.118311288923124, 21.506589466713315, 26.975306541751994, 30.093129052017023, 21.506589466713315, 32.22221914035047, 35.11378365112735, 26.35042423874788, 29.800681131145613, 21.256030110048442, 34.16664924473404, 27.77728915635237, 16.139182322446413, 19.851533330124635, 13.85860376179437, 12.618402319374093, 17.864446345871784, 19.429584428039178, 17.690837939037834, 17.348666159108166, 16.91371785382974, 19.053782246709744, 24.370158584949497, 20.723120306578686, 20.967397891114594, 16.684049195604416, 32.53843073806492, 36.65546388385301, 20.723120306578686, 31.846818916590372, 22.27612829392538, 14.926303617194517, 16.651494963610144, 17.314815113938415, 25.24216031657971, 25.539706914745466, 18.467558995986426, 25.992616060150944, 24.039240115615065, 14.868111474214228, 23.345182011801924, 13.379852088930456, 23.20879380099744, 21.930768935644906, 16.39333746248769, 16.425386987895497, 19.774139536218833, 13.095460258581369, 18.39556080667227, 14.552086348683558, 14.076845015253456, 15.581647041956762, 19.96819234893607, 12.643071719773257, 15.310144149387147, 22.27612829392538, 14.2706235200274, 15.703855414343392, 14.984723517782793, 20.8856537535641, 15.131775567308885, 14.354485905365031, 14.214987631491761, 12.374340325455691, 12.643071719773257, 14.076845015253456, 23.75917448515314, 17.416566918622852, 22.67116267165818, 17.656319241826594, 19.735555861572347, 24.086237637543274, 33.83461446305701, 19.735555861572347, 16.139182322446413, 20.763634702888442, 24.18050850623376, 15.370066359341997, 24.561296107667808, 17.013112580497378, 15.400115355370453, 34.84052601883783, 20.602050815688813, 21.33922385967176, 30.507383918455513, 22.538712245181248, 33.375179667439795, 34.03344610288473, 30.447857334592324, 24.322607071104724, 20.64232851707703, 19.31607208722833, 18.431524745974645, 18.110364070885183, 17.89937193856263, 24.9968545761854, 33.1801937796528, 31.72265993608251, 16.23402543319486, 27.507346790200362, 18.758380733362646, 35.38918446958933, 28.827451925481085, 19.353835657005096, 14.781248363830914, 19.774139536218833, 22.27612829392538, 28.049880592281973, 21.42274322042474, 16.782094257242417, 23.20879380099744, 22.27612829392538, 16.04489330813532, 17.14654807625046, 21.632974546568068, 16.170734966018113, 21.760102337787384, 18.758380733362646, 17.51891667538985, 26.66103470132838, 21.845268970328604, 26.76538304079522, 20.124804822565828, 22.893645317537526, 13.172416709211404, 20.046345644076645, 11.993621898152476, 15.612109685326937, 14.552086348683558, 21.049461966173517, 13.885697797673076, 18.216790978454146, 16.979916273809664, 18.868615759264884, 16.521912001744084, 15.703855414343392, 20.521731031400577, 18.145770440989054, 12.917639106991658, 18.28808948244362, 20.085536923187668, 18.721779026482544, 19.89034374461794, 10.218693612096475, 19.851533330124635, 22.494734284035275, 25.94189877662824, 22.98324851186175, 17.725424121461643, 12.667789349678872, 28.15966472087403, 25.44013703364339, 17.314815113938415, 24.275148340672132, 42.60421214369869, 30.74665629604239, 20.521731031400577, 24.275148340672132, 22.98324851186175, 28.491601937740562, 37.89297069129353, 18.145770440989054, 17.79479946994832, 17.213657767955564, 24.133327041205547, 12.717369675355013, 18.110364070885183, 15.920030957803608, 12.867278004795136, 20.4816886421048, 21.214554942517648, 20.44172438427552, 21.256030110048442, 16.88071548801052, 29.511075247281195, 20.124804822565828, 16.265763498233948, 10.198754704063631, 23.345182011801924, 13.912844803302042, 17.864446345871784, 12.792104248423877, 24.753932740556596, 19.429584428039178, 16.91371785382974, 22.450842133517945, 36.51255777326842, 13.804574186067095, 34.77254452624438, 39.78912962559818, 19.278382202442366, 23.52828054362271, 29.800681131145613, 19.929230034775262, 28.60311491479713, 33.4404292647954, 45.529450258314064, 22.450842133517945, 42.52108200006278, 23.118311288923124, 18.758380733362646, 17.829588900657228, 45.263456176258764, 24.513371642677637, 24.22778221261098, 27.40010580820802, 23.75917448515314, 25.2915095879081, 21.008389858307645, 20.44172438427552, 30.927341318986464, 15.551243837871848, 24.133327041205547, 18.359667040887704, 20.967397891114594, 16.170734966018113, 16.782094257242417, 18.648789727196185, 34.77254452624438, 30.56702687878971, 18.467558995986426, 18.181246031735814, 13.804574186067095, 28.491601937740562, 36.016759615546256, 20.282645623050005, 15.04337206739713, 14.637602771363136, 21.380942759123343, 19.091033031037714, 14.187251079068192, 14.187251079068192, 13.32768898909124, 16.01358620918395, 16.651494963610144, 20.085536923187668, 18.181246031735814, 14.242778409807016, 12.867278004795136, 19.697047472120772, 15.220698576992893, 23.75917448515314, 25.689792910615846, 17.864446345871784, 21.33922385967176, 19.89034374461794, 22.40703562619394, 34.367432111764096, 17.45061689198871, 26.817710263716034, 34.16664924473404, 23.94552000018515, 23.118311288923124, 13.018953406185608, 37.160054292819765, 20.085536923187668, 25.2915095879081, 24.417803063601138, 19.89034374461794, 21.760102337787384, 18.110364070885183, 30.74665629604239, 23.20879380099744, 30.927341318986464, 32.47494113727571, 23.80562446936611, 30.447857334592324, 25.44013703364339, 31.784678801853513, 28.547303976590324, 29.22428378123494, 28.214717811314713, 22.538712245181248, 20.282645623050005, 17.76007792103387, 26.14536361045317, 20.92648590822237, 13.831562592310346, 18.039758452611142, 13.484791569471833, 17.013112580497378, 21.33922385967176, 18.721779026482544, 17.180070153709277, 20.085536923187668, 20.92648590822237, 23.80562446936611, 24.18050850623376, 22.40703562619394, 16.8149038152427, 21.845268970328604, 21.214554942517648, 31.230845679407448, 20.64232851707703, 22.494734284035275, 18.79505399780919, 19.735555861572347, 16.107691244988956, 23.992334296139543, 14.89717913158109, 21.380942759123343, 26.609013186724894, 20.8042283062039, 25.689792910615846, 23.852165264858517, 28.15966472087403, 18.539838978246124, 28.883810562338418, 37.67159077349981, 33.1801937796528, 20.24306974171993, 13.643746025356307, 22.715485556228362, 29.684499285876434, 32.159346537604755, 30.093129052017023, 32.60204446315635, 34.84052601883783, 43.3597351955915, 26.609013186724894, 36.798929313140206, 29.511075247281195, 37.8190332109868, 23.71281513502692, 21.548635571548648, 33.76859572409541, 26.609013186724894, 19.812798643245316, 20.36202965350754, 22.40703562619394, 40.892124929554, 39.556671887793456, 37.160054292819765, 22.494734284035275, 26.24769367033534, 33.375179667439795, 26.609013186724894, 27.028044172168716, 22.98324851186175, 34.300373728940066, 17.484733434196265, 21.049461966173517, 33.83461446305701, 37.5980852538711, 13.59055410055989, 26.557093177483523, 46.246434773093995, 28.049880592281973, 33.4404292647954, 27.028044172168716, 31.41437607743883, 20.36202965350754, 31.291903031786116, 26.713157919740368, 36.08717359043372, 30.806767041912167, 30.987805310145493, 20.84490127137742, 22.582776184718522, 27.028044172168716, 26.453556883236853, 23.620367629992042, 26.14536361045317, 26.09434828302453, 16.01358620918395, 27.346642204097876, 19.735555861572347, 18.6852487375444, 21.297586362924953, 25.58963789966816, 20.723120306578686, 22.98324851186175, 26.557093177483523, 26.713157919740368, 42.190180314697045, 22.450842133517945, 35.32013242426123, 39.172256347947744, 29.56877039069624, 26.817710263716034, 23.16350836406023, 42.68750480955783, 43.61454191906839, 42.60421214369869, 42.190180314697045, 30.62678644311483, 32.22221914035047, 55.6750020519895, 24.850816893992036, 34.908640417532865, 33.05083646319603, 24.133327041205547, 28.940279382223952, 22.059646737841184, 38.79157659527073, 18.6852487375444, 31.41437607743883, 27.61500750152876, 27.4536739354601, 15.642631884188171, 25.192907336481866, 19.851533330124635, 14.95548504215564, 13.406010149206963, 20.602050815688813, 26.609013186724894, 15.10225016094451, 13.912844803302042, 14.666219776472907, 29.453492679719986, 30.21091022897123, 17.314815113938415, 27.561124577174567, 13.432219349369987, 25.790339917193062, 19.24076585887165, 24.18050850623376, 26.505274475545082, 31.598985005566618, 34.16664924473404, 37.232703447205715, 43.5294404008871, 21.717643615002626, 25.689792910615846, 26.87013978811538, 19.053782246709744, 27.346642204097876, 21.590763877931302, 14.694892728788941, 30.987805310145493, 17.656319241826594, 24.89940107578365, 19.658614220966218, 17.934365811960372, 15.01401915561195, 15.673213754973451, 31.660762082694095, 31.048387510438282, 20.046345644076645, 22.10277408821932, 26.196478674560588, 14.926303617194517, 45.88654216244616, 26.975306541751994, 27.561124577174567, 16.58657693697683, 21.42274322042474, 32.09659661306934, 50.1020787877761, 20.085536923187668, 18.868615759264884, 20.723120306578686, 17.314815113938415, 16.107691244988956, 17.113091407702502, 20.8042283062039, 17.51891667538985, 37.89297069129353, 22.84897480427519, 21.049461966173517, 17.382583384208317, 20.124804822565828, 35.87634357692165, 30.447857334592324, 21.760102337787384, 16.01358620918395, 18.539838978246124, 60.434642992688794, 28.049880592281973, 35.87634357692165, 20.16414949200632, 21.506589466713315, 16.329425896569767, 17.79479946994832, 20.44172438427552, 16.94678474038272, 30.329152388418176, 30.68666283939559, 36.798929313140206, 50.1020787877761, 32.033969127372174, 17.113091407702502, 21.548635571548648, 29.975807060355134, 30.447857334592324, 23.299630451611073, 22.145985754016134, 26.24769367033534, 19.929230034775262, 14.2706235200274, 22.938403163153815, 21.008389858307645, 15.920030957803608, 28.214717811314713, 18.6852487375444, 38.56494680401705, 30.806767041912167, 30.806767041912167, 18.503663694167408, 16.84777751703098, 23.574279122424027, 24.850816893992036, 25.689792910615846, 14.868111474214228, 21.802644068801417, 18.758380733362646, 15.951155153095918, 16.2975636122563, 19.391673055829287, 18.648789727196185, 24.417803063601138, 30.507383918455513, 18.6852487375444, 17.013112580497378, 22.938403163153815, 20.64232851707703, 19.128356641900336, 36.299242562272255, 15.370066359341997, 25.39049775763493, 20.046345644076645, 29.858942463791767, 20.967397891114594, 9.693779343581, 20.4816886421048, 14.81014620089916, 14.984723517782793, 27.995149135337257, 18.576084986220376, 15.734556979186685, 12.040563607027826, 14.984723517782793, 15.673213754973451, 20.56185170491249, 20.203571081597193, 24.513371642677637, 33.11545195869231, 27.40010580820802, 17.587483776585056, 20.44172438427552, 25.489873356103875, 15.220698576992893, 21.297586362924953, 22.27612829392538, 17.113091407702502, 30.866995306309306, 17.046373787079762, 30.034410770075546, 15.888967492629165, 19.89034374461794, 24.850816893992036, 18.431524745974645, 13.831562592310346, 19.278382202442366, 33.702705802144536, 15.072782365110845, 23.482371718117374, 22.494734284035275, 18.61240185635776, 17.553166745968216, 31.41437607743883, 23.4365524707797, 18.648789727196185, 21.214554942517648, 26.196478674560588, 17.587483776585056, 19.24076585887165, 22.145985754016134, 17.113091407702502, 15.370066359341997, 20.84490127137742, 23.898797049169467, 37.74523999888851, 19.96819234893607, 43.78524440968234, 13.511154789957821, 13.697146137460072, 18.145770440989054, 19.31607208722833, 19.735555861572347, 30.329152388418176, 17.587483776585056, 15.765318566620541, 22.626926270737744, 13.96729813824377, 16.425386987895497, 21.380942759123343, 9.904297461775021, 26.609013186724894, 29.511075247281195, 29.281418237700255, 20.44172438427552, 30.151962130172098, 16.554212894695087, 18.539838978246124, 28.883810562338418, 20.56185170491249, 11.422063446614425, 18.39556080667227, 15.01401915561195, 22.938403163153815, 13.994604675279588, 15.43022309819552, 17.969428099556087, 16.521912001744084, 15.340075995482293, 16.139182322446413, 16.139182322446413, 15.131775567308885, 15.796140293991145, 17.013112580497378, 19.774139536218833, 17.180070153709277, 24.609314266596865, 14.89717913158109, 13.96729813824377, 21.297586362924953, 18.145770440989054, 25.840760898054555, 13.750755251693265, 24.370158584949497, 25.689792910615846, 22.450842133517945, 22.450842133517945, 22.450842133517945, 21.049461966173517, 23.75917448515314, 27.293282919182133, 28.547303976590324, 16.202349296067883, 42.190180314697045, 30.151962130172098, 24.513371642677637, 17.6218678981495, 26.24769367033534, 28.996858600549206, 22.319678873017494, 27.72308958496738, 13.697146137460072, 21.675267738479917, 22.893645317537526, 31.90908051733889, 21.590763877931302, 23.620367629992042, 16.88071548801052, 16.782094257242417, 26.922671813996086, 30.56702687878971, 22.67116267165818, 13.35374506875535, 28.436008585761353, 20.046345644076645, 27.028044172168716, 19.391673055829287, 31.41437607743883, 27.940524471445343, 16.91371785382974, 23.118311288923124, 23.898797049169467, 32.09659661306934, 14.467069533669397, 25.094689498803355, 15.951155153095918, 25.24216031657971, 32.033969127372174, 27.028044172168716, 23.80562446936611, 23.390822626823073, 26.35042423874788, 35.87634357692165, 29.74253347911242, 55.457945789238345, 18.79505399780919, 43.87084603329354, 27.186876493132687, 19.812798643245316, 18.6852487375444, 21.845268970328604, 22.36331459495464, 11.556703116117722, 13.564035954639001, 26.66103470132838, 23.028181534732802, 21.590763877931302, 16.651494963610144, 17.113091407702502, 16.58657693697683, 32.47494113727571, 24.80232751057381, 37.01518097066168, 27.133828946090556, 24.70563239932894, 30.987805310145493, 31.230845679407448, 21.131847230940878, 27.028044172168716, 16.04489330813532, 24.22778221261098, 25.094689498803355, 22.450842133517945, 22.80439145296227, 24.086237637543274, 25.63966650134361, 18.648789727196185, 26.14536361045317, 17.28103011956753, 18.868615759264884, 18.28808948244362, 12.569207830853442, 16.202349296067883, 16.329425896569767, 19.278382202442366, 19.128356641900336, 19.543763832601815, 14.104365720113616, 14.523692042874359, 11.511647800524438, 27.561124577174567, 13.777638440111984, 13.484791569471833, 32.92198346312245, 27.346642204097876, 22.27612829392538, 21.049461966173517, 24.417803063601138, 13.172416709211404, 15.280270706875589, 27.61500750152876, 20.8856537535641, 40.65322322588273, 36.72712654703974, 26.505274475545082, 29.110349096271467, 37.08754689283638, 21.049461966173517, 29.56877039069624, 17.314815113938415, 19.053782246709744, 31.90908051733889, 18.758380733362646, 12.792104248423877, 22.319678873017494, 22.494734284035275, 30.151962130172098, 27.028044172168716, 38.41459632502361, 22.494734284035275, 18.181246031735814, 44.21492934326177, 32.72964525738904, 18.110364070885183, 24.513371642677637, 31.784678801853513, 29.22428378123494, 34.77254452624438, 13.697146137460072, 19.735555861572347, 34.97688798216538, 14.4106677747075, 26.24769367033534, 23.073202403172917, 16.8149038152427, 25.44013703364339, 35.87634357692165, 40.33685733281881, 37.67159077349981, 39.248839424943355, 19.20322291302118, 21.214554942517648, 29.396022468352484, 34.16664924473404, 26.557093177483523, 20.44172438427552, 22.319678873017494, 39.634006510082365, 31.660762082694095, 23.620367629992042, 21.506589466713315, 20.68268496272443, 21.506589466713315, 22.232662691545976, 14.242778409807016, 13.018953406185608, 22.494734284035275, 14.354485905365031, 14.868111474214228, 13.069908149118522, 27.028044172168716, 22.626926270737744, 11.3553329839266, 22.450842133517945, 25.489873356103875, 18.942465433314446, 27.346642204097876, 21.256030110048442, 19.543763832601815, 19.658614220966218, 20.24306974171993, 22.319678873017494, 22.27612829392538, 20.602050815688813, 16.61900425204837, 24.275148340672132, 28.1047190510635, 35.11378365112735, 14.81014620089916, 23.345182011801924, 23.80562446936611, 12.817113241929157, 18.539838978246124, 35.25121511458854, 22.626926270737744, 23.75917448515314, 17.45061689198871, 11.333176243677455, 34.300373728940066, 18.868615759264884, 42.3553080116137, 48.37128004023317, 22.145985754016134, 19.851533330124635, 33.76859572409541, 44.5617113320772, 30.806767041912167, 15.01401915561195, 45.086990684122, 23.898797049169467, 34.501942437892716, 19.278382202442366, 36.15772522700737, 28.049880592281973, 20.4816886421048, 27.028044172168716, 22.67116267165818, 23.299630451611073, 23.482371718117374, 26.817710263716034, 22.626926270737744, 29.975807060355134, 24.417803063601138, 21.973644325590612, 22.145985754016134, 30.034410770075546, 32.79363281328884, 28.827451925481085, 20.322298876557472, 20.68268496272443, 15.642631884188171, 17.76007792103387, 27.83159468972004, 21.802644068801417, 36.299242562272255, 21.632974546568068, 28.771203256660744, 17.969428099556087, 35.11378365112735, 25.045724266551254, 28.996858600549206, 28.771203256660744, 18.868615759264884, 45.086990684122, 22.98324851186175, 27.507346790200362, 29.110349096271467, 20.723120306578686, 23.75917448515314, 24.70563239932894, 25.192907336481866, 26.922671813996086, 32.411575418595035, 19.505629671380603, 21.297586362924953, 28.65903496526287, 29.9173176993003, 37.08754689283638, 30.507383918455513, 29.281418237700255, 24.948080241282657, 28.883810562338418, 23.666546242140512, 22.319678873017494, 42.77096031537661, 22.626926270737744, 26.2990087931472, 40.1011996370117, 30.034410770075546, 19.543763832601815, 32.66578255521719, 20.967397891114594, 29.453492679719986, 29.800681131145613, 28.65903496526287, 17.51891667538985, 24.039240115615065, 20.68268496272443, 30.093129052017023, 29.9173176993003, 15.280270706875589, 32.47494113727571, 26.453556883236853, 35.59715165620933, 26.453556883236853, 39.095822701298815, 25.94189877662824, 24.753932740556596, 20.56185170491249, 24.753932740556596, 22.189281900071105, 25.24216031657971, 37.524723159601, 29.858942463791767, 22.84897480427519, 22.715485556228362, 22.80439145296227, 27.507346790200362, 28.491601937740562, 29.800681131145613, 39.78912962559818, 25.790339917193062, 44.91121316574036, 32.159346537604755, 31.90908051733889, 30.21091022897123, 35.94648303373688, 32.9863470466753, 22.715485556228362, 36.08717359043372, 33.11545195869231, 38.19016894386643, 25.58963789966816, 22.938403163153815, 25.790339917193062, 22.98324851186175, 28.65903496526287, 24.465540688808968, 21.42274322042474, 27.4536739354601, 27.886006392229614, 17.690837939037834, 24.850816893992036, 26.043432497666576, 25.689792910615846, 22.494734284035275, 33.4404292647954, 28.65903496526287, 22.938403163153815, 40.73270164287187, 40.97207040739876, 40.41571724385996, 44.04255166984753, 27.293282919182133, 25.39049775763493, 26.043432497666576, 23.94552000018515, 27.028044172168716, 34.56939489431364, 31.9714638416083, 32.348333340301785, 23.852165264858517, 34.03344610288473, 36.016759615546256, 27.4536739354601, 20.401838105461145, 25.24216031657971, 47.435694792893365, 33.50580642730562, 27.080884906424416, 14.2706235200274, 20.282645623050005, 22.84897480427519, 18.648789727196185, 17.382583384208317, 28.491601937740562, 25.2915095879081, 19.581972547389178, 20.046345644076645, 22.27612829392538, 22.538712245181248, 20.967397891114594, 34.300373728940066, 15.98234019723552, 21.33922385967176, 16.329425896569767, 20.8042283062039, 33.4404292647954, 29.053548433146837, 21.590763877931302, 28.325147094540903, 25.045724266551254, 22.84897480427519, 21.464625403032095, 25.840760898054555, 25.740017318702005, 29.396022468352484, 17.213657767955564, 20.967397891114594, 20.521731031400577, 18.039758452611142, 17.28103011956753, 29.110349096271467, 16.94678474038272, 21.380942759123343, 16.139182322446413, 16.139182322446413, 15.072782365110845, 16.554212894695087, 16.23402543319486, 18.2524054167373, 18.2524054167373, 18.2524054167373, 18.2524054167373, 18.2524054167373, 16.94678474038272, 19.581972547389178, 15.43022309819552, 16.521912001744084, 16.521912001744084, 16.521912001744084, 16.521912001744084, 15.673213754973451, 16.076261613516785, 21.049461966173517, 16.74934871787156, 18.539838978246124, 15.888967492629165, 18.359667040887704, 24.561296107667808, 16.91371785382974, 16.2975636122563, 25.489873356103875, 29.281418237700255, 25.2915095879081, 32.9863470466753, 21.464625403032095, 18.83179895971962, 21.42274322042474, 17.314815113938415, 19.581972547389178, 15.920030957803608, 16.107691244988956, 19.697047472120772, 13.018953406185608, 21.675267738479917, 16.684049195604416, 14.580536166314408, 16.91371785382974, 30.034410770075546, 29.800681131145613, 21.887977204970085, 25.689792910615846, 27.293282919182133, 18.61240185635776, 26.557093177483523, 24.086237637543274, 26.609013186724894, 19.165753221675935, 29.453492679719986, 29.33866439394716, 20.68268496272443, 21.590763877931302, 19.581972547389178, 33.90076227087091, 22.232662691545976, 19.543763832601815, 19.620255961497417, 23.898797049169467, 30.56702687878971, 37.37842812826559, 23.852165264858517, 23.620367629992042, 22.189281900071105, 34.908640417532865, 48.75066069158903, 19.391673055829287, 18.539838978246124, 23.666546242140512, 37.45150421083523, 28.1047190510635, 31.475792237910618, 36.51255777326842, 24.322607071104724, 40.892124929554, 38.115651564714454, 27.346642204097876, 29.053548433146837, 23.345182011801924, 28.65903496526287, 16.58657693697683, 18.39556080667227, 23.75917448515314, 19.24076585887165, 23.299630451611073, 27.561124577174567, 31.35307975333966, 23.073202403172917, 26.09434828302453, 34.97688798216538, 34.70469568042362, 18.110364070885183, 29.453492679719986, 19.505629671380603, 24.948080241282657, 18.039758452611142, 29.56877039069624, 35.87634357692165, 23.073202403172917, 21.548635571548648, 20.8042283062039, 53.43765380953904, 22.98324851186175, 37.01518097066168, 28.436008585761353, 27.293282919182133, 17.79479946994832, 20.8856537535641, 47.71445419747699, 20.723120306578686, 28.15966472087403, 29.053548433146837, 32.033969127372174, 32.53843073806492, 24.609314266596865, 44.387981683814154, 31.048387510438282, 28.771203256660744, 29.684499285876434, 25.2915095879081, 30.987805310145493, 27.028044172168716, 28.883810562338418, 34.70469568042362, 22.938403163153815, 27.40010580820802, 22.75989509352673, 17.314815113938415, 19.851533330124635, 29.167260806600495, 20.68268496272443, 29.396022468352484, 33.11545195869231, 21.464625403032095, 40.892124929554, 47.250760544967605, 22.189281900071105, 30.74665629604239, 24.9968545761854, 42.854578979512624, 24.22778221261098, 21.506589466713315, 24.513371642677637, 17.934365811960372, 19.697047472120772, 31.475792237910618, 24.322607071104724, 23.4365524707797, 26.14536361045317, 34.70469568042362, 48.56059987746486, 20.282645623050005, 19.016604146815915, 22.67116267165818, 41.698649825700315, 19.46756991825522, 21.760102337787384, 40.25815129472595, 37.9670527218579, 28.436008585761353, 21.297586362924953, 29.281418237700255, 33.375179667439795, 24.465540688808968, 23.52828054362271, 29.858942463791767, 39.86691871057298, 55.89290784893326, 29.396022468352484, 33.702705802144536, 19.053782246709744, 25.24216031657971, 20.007230835729846, 18.28808948244362, 18.868615759264884, 35.52769382137767, 30.034410770075546, 24.561296107667808, 22.938403163153815, 16.36135047271935, 29.975807060355134, 28.715064341305684, 25.24216031657971, 24.133327041205547, 24.513371642677637, 28.60311491479713, 31.41437607743883, 25.24216031657971, 25.840760898054555, 17.76007792103387, 20.44172438427552, 16.84777751703098, 16.84777751703098, 16.84777751703098, 24.133327041205547, 17.587483776585056, 21.42274322042474, 30.151962130172098, 25.24216031657971, 24.80232751057381, 28.883810562338418, 32.28521466114651, 18.00455893510183, 27.72308958496738, 36.87087245606033, 26.922671813996086, 45.44061233480379, 30.26997357328364, 24.609314266596865, 19.053782246709744, 22.189281900071105, 19.774139536218833, 45.086990684122, 38.115651564714454, 19.851533330124635, 35.32013242426123, 38.867415430562446, 26.817710263716034, 28.940279382223952, 26.505274475545082, 33.76859572409541, 36.228414794400415, 34.70469568042362, 31.846818916590372, 38.48969815122384, 31.53732846903916, 36.65546388385301, 59.265731184583274, 31.230845679407448, 28.883810562338418, 32.92198346312245, 39.172256347947744, 45.529450258314064, 18.721779026482544, 27.346642204097876, 31.10908815096766, 34.84052601883783, 53.542126217761066, 49.9067492938035, 41.37414835829223, 31.72265993608251, 45.086990684122, 39.78912962559818, 18.79505399780919, 32.60204446315635, 59.03467599561125, 23.52828054362271, 24.850816893992036, 48.08868329990042, 39.248839424943355, 43.61454191906839, 27.83159468972004, 20.282645623050005, 45.79700747446617, 31.35307975333966, 34.77254452624438, 24.275148340672132, 44.64883097460398, 35.18243227767251, 31.598985005566618, 27.293282919182133, 28.436008585761353, 26.557093177483523, 20.44172438427552, 44.5617113320772, 33.90076227087091, 41.45503620813183, 34.09998263323346, 35.45837151398557, 29.053548433146837, 45.086990684122, 26.401940203271806, 36.08717359043372, 24.753932740556596, 26.609013186724894, 54.173263858709824, 29.511075247281195, 34.03344610288473, 18.503663694167408, 29.396022468352484, 36.299242562272255, 17.725424121461643, 28.380523708580828, 27.668995768810042, 30.987805310145493, 24.948080241282657, 27.346642204097876, 19.353835657005096, 26.196478674560588, 19.091033031037714, 31.230845679407448, 25.24216031657971, 22.059646737841184, 16.36135047271935, 20.282645623050005, 24.039240115615065, 26.14536361045317, 19.353835657005096, 20.84490127137742, 19.929230034775262, 24.609314266596865, 35.18243227767251, 40.33685733281881, 29.396022468352484, 20.602050815688813, 22.582776184718522, 17.76007792103387, 29.167260806600495, 25.045724266551254, 29.22428378123494, 16.04489330813532, 45.351947753528236, 40.41571724385996, 40.41571724385996, 48.46584751755162, 29.22428378123494, 19.812798643245316, 18.467558995986426, 22.67116267165818, 35.52769382137767, 40.892124929554, 41.780172086809024, 39.78912962559818, 18.145770440989054, 20.322298876557472, 22.582776184718522, 38.867415430562446, 33.4404292647954, 20.085536923187668, 27.61500750152876, 40.1011996370117, 29.053548433146837, 16.716667072215767, 16.716667072215767, 40.73270164287187, 19.505629671380603, 36.44131378072682, 41.943695056893915, 20.124804822565828, 25.44013703364339, 26.76538304079522, 26.196478674560588, 35.7364749685521, 36.299242562272255, 25.094689498803355, 22.98324851186175, 22.626926270737744, 34.97688798216538, 31.10908815096766, 19.429584428039178, 19.812798643245316, 22.145985754016134, 20.967397891114594, 18.075026786359576, 18.758380733362646, 23.20879380099744, 23.574279122424027, 27.77728915635237, 29.858942463791767, 35.806340977539655, 23.299630451611073, 20.763634702888442, 19.620255961497417, 21.760102337787384, 23.94552000018515, 30.21091022897123, 46.60915005647103, 33.31005738633129, 17.079700020438295, 28.2698785323965, 30.388446900124176, 26.922671813996086, 19.812798643245316, 15.16135869683449, 20.967397891114594, 32.60204446315635, 31.660762082694095, 40.49473132867618, 24.039240115615065, 20.007230835729846, 19.581972547389178, 28.547303976590324, 19.091033031037714, 15.703855414343392, 18.181246031735814, 20.203571081597193, 26.505274475545082, 23.25416777248505, 21.845268970328604, 21.845268970328604, 17.656319241826594, 20.085536923187668, 26.66103470132838, 22.189281900071105, 19.812798643245316, 18.79505399780919, 19.620255961497417, 23.52828054362271, 21.090614371389922, 24.465540688808968, 23.25416777248505, 22.36331459495464, 24.039240115615065, 38.0412795852805, 35.11378365112735, 29.684499285876434, 37.89297069129353, 25.094689498803355, 40.33685733281881, 38.56494680401705, 24.370158584949497, 23.574279122424027, 18.359667040887704, 29.110349096271467, 30.034410770075546, 38.71588573814723, 41.45503620813183, 21.131847230940878, 25.1437504597293, 17.76007792103387, 32.66578255521719, 27.72308958496738, 24.370158584949497, 18.039758452611142, 35.45837151398557, 39.40245503911115, 20.44172438427552, 27.240027749911206, 31.048387510438282, 24.086237637543274, 39.7114923241556, 30.866995306309306, 24.18050850623376, 33.245062173048, 34.56939489431364, 32.159346537604755, 23.94552000018515, 25.094689498803355, 22.232662691545976, 30.56702687878971, 32.22221914035047, 25.489873356103875, 22.67116267165818, 17.829588900657228, 19.851533330124635, 30.26997357328364, 32.9863470466753, 24.22778221261098, 29.858942463791767, 25.2915095879081, 55.457945789238345, 24.086237637543274, 36.51255777326842, 30.21091022897123, 25.94189877662824, 23.71281513502692, 26.401940203271806, 26.24769367033534, 31.048387510438282, 31.90908051733889, 24.322607071104724, 17.79479946994832, 26.557093177483523, 31.598985005566618, 46.70027251047952, 39.86691871057298, 46.246434773093995, 29.858942463791767, 29.22428378123494, 39.095822701298815, 23.345182011801924, 38.19016894386643, 28.996858600549206, 28.827451925481085, 33.11545195869231, 24.657426302639607, 18.039758452611142, 31.660762082694095, 31.10908815096766, 41.780172086809024, 22.059646737841184, 30.987805310145493, 25.24216031657971, 30.866995306309306, 28.547303976590324, 32.47494113727571, 17.79479946994832, 35.04526897307925, 36.942956250241515, 26.196478674560588, 30.093129052017023, 31.784678801853513, 38.115651564714454, 35.66674528344124, 40.41571724385996, 54.27917440928786, 37.8190332109868, 37.45150421083523, 26.713157919740368, 36.65546388385301, 35.59715165620933, 45.79700747446617, 43.78524440968234, 25.689792910615846, 43.02230705927991, 42.77096031537661, 34.84052601883783, 24.561296107667808, 38.56494680401705, 18.758380733362646, 40.57389988868238, 26.24769367033534, 51.692489491288626, 23.574279122424027, 34.03344610288473, 24.948080241282657, 36.51255777326842, 26.87013978811538, 24.850816893992036, 32.22221914035047, 18.942465433314446, 27.77728915635237, 34.23344619169951, 22.75989509352673, 62.10995300937982, 41.698649825700315, 69.55984769951485, 91.43469485205186, 67.41971617548266, 31.9714638416083, 30.093129052017023, 20.763634702888442, 49.22907356066477, 39.095822701298815, 19.774139536218833, 16.457499171202052, 30.927341318986464, 14.89717913158109, 20.68268496272443, 23.390822626823073, 14.868111474214228, 9.884972009450424, 25.489873356103875, 13.069908149118522, 17.656319241826594, 27.240027749911206, 12.742232560259533, 20.92648590822237, 29.800681131145613, 27.186876493132687, 23.073202403172917, 16.2975636122563, 29.22428378123494, 25.1437504597293, 22.80439145296227, 15.43022309819552, 19.658614220966218, 10.218693612096475, 12.278042445054236, 38.19016894386643, 17.14654807625046, 10.501966233859342, 18.979498589532987, 17.28103011956753, 38.71588573814723, 31.598985005566618, 31.35307975333966, 17.969428099556087, 14.049378009312553, 20.602050815688813, 12.326097344475713, 16.8149038152427, 24.275148340672132, 40.97207040739876, 21.464625403032095, 12.302046430466076, 9.413901690309483, 51.99626426796157, 24.370158584949497, 46.246434773093995, 69.28865955361954, 76.39654709045266, 46.066137010589394, 75.21212841006654, 37.08754689283638, 69.83209724580306, 53.333385249853684, 40.49473132867618, 27.77728915635237, 84.563269020706, 27.72308958496738, 27.40010580820802, 53.75168417471723, 98.86447768050158, 29.053548433146837, 12.968197316969134, 12.593781054375055, 113.7923162908356, 17.829588900657228, 49.133016862271724, 23.75917448515314, 38.48969815122384, 25.840760898054555, 37.89297069129353, 33.96703939987107, 29.33866439394716, 56.9952966774873, 18.942465433314446, 18.39556080667227, 26.453556883236853, 28.2698785323965, 83.57807709540792, 47.52843316962603, 45.351947753528236, 76.99573263552006, 31.784678801853513, 44.99901609609206, 99.2514223056366, 17.864446345871784, 57.89284200037607, 20.521731031400577, 29.975807060355134, 34.300373728940066, 28.60311491479713, 16.8149038152427, 24.133327041205547, 54.067559962798995, 40.41571724385996, 35.87634357692165, 25.840760898054555, 27.886006392229614, 31.53732846903916, 33.31005738633129, 28.771203256660744, 21.42274322042474, 28.214717811314713, 19.016604146815915, 22.10277408821932, 27.61500750152876, 35.59715165620933, 62.597087408320306, 48.84596987082467, 33.57131140436464, 28.547303976590324, 55.457945789238345, 27.561124577174567, 40.73270164287187, 54.598150033144236, 35.66674528344124, 57.442316329639176, 24.275148340672132, 22.27612829392538, 44.47476167904283, 15.43022309819552, 29.167260806600495, 65.60118435636834, 35.18243227767251, 70.93179276322707, 67.41971617548266, 46.246434773093995, 100.81440478091984, 79.43983955226133, 105.65258908406659, 54.38529201854992, 87.93188669987823, 52.19977189231385, 12.158723248329283, 55.24173574865985, 204.44776193681219, 62.597087408320306, 53.542126217761066, 26.87013978811538, 20.24306974171993, 34.908640417532865, 22.84897480427519, 14.4106677747075, 18.00455893510183, 22.059646737841184, 21.131847230940878, 19.016604146815915, 20.282645623050005, 38.19016894386643, 18.181246031735814, 13.85860376179437, 35.87634357692165, 16.39333746248769, 17.864446345871784, 20.401838105461145, 33.96703939987107, 14.868111474214228, 10.62576008857486, 16.36135047271935, 21.380942759123343, 87.58907276183992, 35.87634357692165, 36.58394105020859, 47.99485175252902, 68.48142221881122, 41.780172086809024, 47.807737558209425, 50.396507397123145, 50.396507397123145, 45.17513726542669, 20.16414949200632, 79.43983955226133, 14.495353140571094, 30.447857334592324, 48.37128004023317, 71.76791429061606, 28.771203256660744, 24.22778221261098, 11.923552252478798, 26.2990087931472, 44.99901609609206, 34.84052601883783, 28.65903496526287, 34.23344619169951, 26.35042423874788, 40.25815129472595, 19.581972547389178, 31.230845679407448, 33.96703939987107, 15.581647041956762, 23.992334296139543, 41.617286632367644, 16.74934871787156, 59.03467599561125, 40.81233544283598, 30.329152388418176, 28.547303976590324, 28.380523708580828, 32.033969127372174, 30.26997357328364, 31.53732846903916, 20.085536923187668, 41.05217218133816, 17.76007792103387, 27.40010580820802, 70.37979717263202, 27.561124577174567, 60.67117749756472, 78.51433738825796, 64.08153455750669, 28.380523708580828, 63.582848099201826, 82.28231911219243, 26.09434828302453, 80.06289393593435, 75.50650035756513, 46.97471015107385, 68.48142221881122, 90.7231444115574, 59.964336869699544, 106.06610162605153, 29.110349096271467, 39.86691871057298, 42.52108200006278, 92.1518260518184, 144.4099673433467, 40.65322322588273, 72.33079701870139, 43.275130861102994, 52.40407602298735, 50.396507397123145, 46.88305220915621, 99.63988138921447, 56.11166650505244, 24.70563239932894, 112.46659565031442, 77.29708541258049, 89.31661242710538, 57.442316329639176, 65.34542957624015, 56.11166650505244, 100.42136666458902, 21.930768935644906, 24.9968545761854, 90.7231444115574, 166.21485355527525, 74.91890411002962, 106.48123261036584, 83.90519244027122, 100.02986085865793, 76.99573263552006, 71.20941195067294, 73.46984133199462, 98.86447768050158, 45.61846186294896, 27.72308958496738, 19.851533330124635, 64.83690740886694, 56.55175575986736, 37.8190332109868, 73.75739416067401, 92.87458178129451, 109.43300111180854, 58.57526450222688, 52.71203249107963, 21.090614371389922, 22.98324851186175, 57.89284200037607, 17.079700020438295, 55.24173574865985, 45.79700747446617, 48.182698291098816, 61.867809250367884, 30.866995306309306, 55.89290784893326, 26.817710263716034, 19.165753221675935, 97.33172602285453, 17.829588900657228, 17.690837939037834, 40.65322322588273, 33.1801937796528, 137.25973406168575, 79.43983955226133, 41.780172086809024, 50.396507397123145, 30.329152388418176, 42.10785803778799, 95.82273747708687, 62.597087408320306, 67.9484955630781, 66.37447035867858, 160.47288700783744, 91.79256012938836, 98.09510821132314, 88.27604237373367, 81.32370105824417, 46.7915731119465, 44.82358155812517, 55.6750020519895, 24.561296107667808, 36.65546388385301, 35.45837151398557, 53.96206231832688, 62.84208548133934, 48.08868329990042, 37.305494633128845, 62.84208548133934, 26.09434828302453, 45.17513726542669, 17.51891667538985, 43.275130861102994, 14.021964597512564, 27.186876493132687, 9.524869784493507, 56.9952966774873, 50.6926662381911, 62.3530444922685, 55.24173574865985, 41.698649825700315, 23.345182011801924, 60.67117749756472, 64.58413226217957, 28.940279382223952, 32.92198346312245, 124.97641006343288, 28.436008585761353, 46.246434773093995, 42.68750480955783, 34.300373728940066, 25.2915095879081, 33.50580642730562, 54.067559962798995, 40.41571724385996, 20.602050815688813, 93.2380826167318, 70.65525591042105, 14.89717913158109, 61.38635013907805, 33.63694444585419, 47.15856396830611, 69.55984769951485, 75.50650035756513, 58.57526450222688, 57.442316329639176, 59.73055807527518, 95.44916002730179, 44.47476167904283, 37.9670527218579, 54.811841150452466, 86.23113005463887, 108.5813867407896, 52.50652775191775, 87.58907276183992, 60.199030648454944, 63.582848099201826, 44.73612093895834, 45.79700747446617, 76.99573263552006, 29.281418237700255, 41.29341833835342, 51.692489491288626, 52.301824199806006, 36.15772522700737, 63.3349620664254, 86.90744918987926, 43.02230705927991, 107.31637526839133, 17.484733434196265, 69.83209724580306, 58.57526450222688, 66.37447035867858, 73.46984133199462, 38.56494680401705, 93.2380826167318, 43.444504934395866, 72.61389175398708, 61.14702744030778, 76.09870517958821, 61.14702744030778, 70.93179276322707, 36.51255777326842, 134.60489048229144, 93.96935796203766, 53.96206231832688, 144.97517197488295, 44.12865633779556, 101.6051019574268, 70.10541234668786, 50.6926662381911, 41.780172086809024, 43.02230705927991, 58.119428177447986, 51.1901369093998, 54.811841150452466, 45.529450258314064, 35.7364749685521, 29.053548433146837, 23.75917448515314, 53.02179868836547, 24.70563239932894, 31.10908815096766, 46.70027251047952, 90.01713130052181, 72.04880596519355, 26.453556883236853, 42.52108200006278, 17.79479946994832, 108.5813867407896, 28.325147094540903, 64.58413226217957, 79.43983955226133, 63.08804244970428, 74.33588057277046, 52.301824199806006, 33.702705802144536, 62.10995300937982, 78.82163381626928, 72.33079701870139, 61.867809250367884, 89.66618776422116, 28.715064341305684, 108.5813867407896, 22.80439145296227, 36.15772522700737, 80.37625124290746, 88.62154503481175, 69.83209724580306, 47.99485175252902, 78.2082389954826, 52.301824199806006, 81.641993024771, 33.4404292647954, 15.703855414343392, 66.37447035867858, 39.40245503911115, 58.34690118774256, 116.49082330627213, 54.811841150452466, 22.10277408821932, 42.52108200006278, 122.08133464248264, 40.022953418667065, 66.63425269055388, 37.01518097066168, 64.08153455750669, 37.37842812826559, 76.09870517958821, 79.75075829332496, 76.69555472159655, 47.34313736903344, 50.5937537501147, 45.97625189374871, 21.214554942517648, 64.33234259239626, 153.12429849019884, 71.20941195067294, 64.33234259239626, 105.65258908406659, 8.878154865526355, 78.51433738825796, 17.28103011956753, 23.390822626823073, 116.03666862607915, 121.13129015902297, 14.694892728788941, 32.9863470466753, 23.345182011801924, 59.49769069799599, 81.32370105824417, 62.597087408320306, 58.57526450222688, 29.626578330054702, 72.04880596519355, 34.23344619169951, 96.57428452170761, 36.299242562272255, 90.7231444115574, 64.33234259239626, 28.380523708580828, 104.83039411284926, 95.82273747708687, 67.41971617548266, 27.293282919182133, 96.57428452170761, 60.199030648454944, 51.39048944047805, 37.01518097066168, 12.968197316969134, 27.507346790200362, 24.18050850623376, 11.311062736078481, 25.2915095879081, 174.19167092744075, 18.539838978246124, 20.322298876557472, 31.598985005566618, 24.657426302639607, 21.33922385967176, 55.24173574865985, 45.086990684122, 49.61518196998445, 57.89284200037607, 64.33234259239626, 86.23113005463887, 33.50580642730562, 83.25223705241488, 42.27266353442275, 56.9952966774873, 84.89424029774196, 71.48811770889668, 116.03666862607915, 22.145985754016134, 15.04337206739713, 18.075026786359576, 30.987805310145493, 91.79256012938836, 24.609314266596865, 123.03883041717654, 52.40407602298735, 82.92766733936132, 53.85677052285177, 49.809370301716946, 33.90076227087091, 11.993621898152476, 50.49503426195374, 108.15806803146498, 88.96839995507139, 79.75075829332496, 23.16350836406023, 87.58907276183992, 25.58963789966816, 42.27266353442275, 80.69083499570341, 76.99573263552006, 42.190180314697045, 39.634006510082365, 21.131847230940878, 24.465540688808968, 120.65904401308495, 57.442316329639176, 87.58907276183992, 58.34690118774256, 69.83209724580306, 104.83039411284926, 50.5937537501147, 60.67117749756472, 32.159346537604755, 60.434642992688794, 48.65553748142455, 39.86691871057298, 43.19069160819034, 32.159346537604755, 126.44959243764188, 81.9615307508321, 20.68268496272443, 93.96935796203766, 46.066137010589394, 35.11378365112735, 51.39048944047805, 56.773093073800744, 61.14702744030778, 67.1568716116849, 61.626609520407456, 63.3349620664254, 15.520899957093135, 29.167260806600495, 53.02179868836547, 31.72265993608251, 68.48142221881122, 17.76007792103387, 68.48142221881122, 81.641993024771, 69.83209724580306, 46.246434773093995, 38.33964103892552, 113.7923162908356, 39.78912962559818, 34.77254452624438, 22.36331459495464, 15.400115355370453, 19.353835657005096, 101.20898120494374, 36.15772522700737, 53.333385249853684, 43.699809813576366, 20.046345644076645, 65.8579401324037, 19.20322291302118, 29.33866439394716, 19.016604146815915, 23.574279122424027, 22.319678873017494, 69.55984769951485, 41.29341833835342, 37.305494633128845, 16.521912001744084, 22.36331459495464, 10.379614621236398, 20.8856537535641, 22.75989509352673, 37.5980852538711, 91.79256012938836, 20.16414949200632, 42.10785803778799, 47.52843316962603, 59.73055807527518, 97.71267162761835, 40.73270164287187, 84.563269020706, 36.08717359043372, 59.73055807527518, 21.090614371389922, 40.57389988868238, 78.2082389954826, 60.908637772317434, 10.77203177799245, 21.802644068801417, 17.79479946994832, 22.40703562619394, 38.71588573814723, 87.24759532868651, 41.132430556936306, 86.90744918987926, 21.973644325590612, 15.98234019723552, 57.442316329639176, 75.21212841006654, 66.37447035867858, 37.305494633128845, 45.17513726542669, 35.66674528344124, 65.8579401324037, 47.250760544967605, 41.53608219643496, 31.660762082694095, 30.987805310145493, 53.02179868836547, 113.7923162908356, 78.2082389954826, 40.81233544283598, 18.039758452611142, 41.45503620813183, 16.139182322446413, 28.049880592281973, 17.969428099556087, 94.70636877096143, 30.56702687878971, 28.60311491479713, 39.32557222442699, 101.20898120494374, 30.388446900124176, 16.457499171202052, 14.781248363830914, 45.61846186294896, 75.50650035756513, 13.301683750542033, 78.82163381626928, 66.89505178172818, 83.90519244027122, 26.043432497666576, 59.73055807527518, 25.39049775763493, 83.25223705241488, 39.78912962559818, 30.56702687878971, 144.97517197488295, 30.806767041912167, 21.760102337787384, 107.7363996853844, 33.76859572409541, 74.33588057277046, 26.35042423874788, 53.85677052285177, 38.94340253332466, 98.86447768050158, 53.646802873050504, 54.598150033144236, 58.80452160545304, 52.097918711212095, 21.506589466713315, 45.351947753528236, 69.83209724580306, 137.7969534714142, 96.19777706766972, 77.90333396724647, 30.21091022897123, 59.265731184583274, 50.49503426195374, 112.90677745995042, 33.96703939987107, 53.542126217761066, 64.58413226217957, 60.908637772317434, 55.24173574865985, 56.773093073800744, 22.319678873017494, 72.04880596519355, 29.56877039069624, 21.548635571548648, 124.48917321564447, 40.25815129472595, 76.39654709045266, 28.380523708580828, 30.329152388418176, 47.15856396830611, 60.199030648454944, 52.71203249107963, 68.21443845541918, 40.41571724385996, 27.77728915635237, 41.29341833835342, 57.21836996148928, 56.773093073800744, 59.73055807527518, 50.79177210350426, 30.151962130172098, 29.800681131145613, 53.542126217761066, 91.43469485205186, 36.08717359043372, 53.12545808585662, 64.33234259239626, 50.1020787877761, 141.61682573322432, 61.626609520407456, 56.773093073800744, 26.2990087931472, 32.09659661306934, 42.854578979512624, 48.65553748142455, 16.61900425204837, 15.460389702669161, 48.46584751755162, 52.50652775191775, 97.71267162761835, 80.69083499570341, 64.83690740886694, 66.37447035867858, 37.67159077349981, 95.0770390179677, 34.908640417532865, 21.590763877931302, 15.888967492629165, 88.62154503481175, 100.42136666458902, 33.31005738633129, 141.0647140528757, 18.039758452611142, 51.39048944047805, 109.43300111180854, 28.325147094540903, 47.250760544967605, 28.380523708580828, 25.891280453627324, 28.65903496526287, 16.39333746248769, 15.951155153095918, 23.94552000018515, 47.62135285300038, 22.67116267165818, 51.39048944047805, 16.01358620918395, 83.57807709540792, 43.61454191906839, 82.60436300370014, 102.00277308269969, 50.990565478815945, 49.71218131735948, 68.48142221881122, 47.34313736903344, 84.2335880783952, 51.894808174781225, 13.940044882238906, 13.537569551425428, 28.60311491479713, 58.34690118774256, 41.2128458403547, 14.104365720113616, 73.46984133199462, 37.305494633128845, 48.84596987082467, 82.60436300370014, 49.42175070681059, 80.06289393593435, 40.17959882934157, 80.69083499570341, 46.427438201089, 42.27266353442275, 72.61389175398708, 74.91890411002962, 88.27604237373367, 86.90744918987926, 72.89809449073893, 77.59961765106212, 19.429584428039178, 41.943695056893915, 34.77254452624438, 111.15632012308374, 24.417803063601138, 34.367432111764096, 58.119428177447986, 54.811841150452466, 44.91121316574036, 93.6030061542134, 60.434642992688794, 65.34542957624015, 66.63425269055388, 65.34542957624015, 73.18340956555208, 73.46984133199462, 93.96935796203766, 84.563269020706, 55.02636863114642, 127.94014021950323, 20.763634702888442, 27.668995768810042, 30.388446900124176, 34.300373728940066, 81.32370105824417, 29.281418237700255, 24.039240115615065, 19.505629671380603, 35.38918446958933, 61.867809250367884, 46.15619785585283, 43.275130861102994, 42.854578979512624, 41.617286632367644, 26.609013186724894, 18.90550453688997, 40.022953418667065, 52.301824199806006, 57.21836996148928, 22.10277408821932, 79.75075829332496, 71.48811770889668, 46.51820540231633, 15.490615283867873, 20.046345644076645, 21.173160702117308, 30.927341318986464, 71.48811770889668, 28.1047190510635, 26.922671813996086, 58.80452160545304, 25.58963789966816, 19.391673055829287, 31.846818916590372, 61.14702744030778, 20.36202965350754, 57.442316329639176, 15.888967492629165, 90.01713130052181, 17.45061689198871, 11.923552252478798, 13.379852088930456, 84.563269020706, 24.948080241282657, 31.048387510438282, 26.557093177483523, 32.85774546700983, 13.301683750542033, 30.388446900124176, 19.89034374461794, 10.001492931490674, 71.20941195067294, 14.382549407491597, 36.65546388385301, 13.198169226352599, 60.199030648454944, 27.186876493132687, 52.50652775191775, 54.173263858709824, 31.10908815096766, 55.02636863114642, 22.80439145296227, 16.139182322446413, 21.506589466713315, 12.544682555069812, 39.32557222442699, 85.2265069597304, 48.27689708476307, 17.348666159108166, 101.6051019574268, 14.81014620089916, 78.51433738825796, 71.76791429061606, 21.090614371389922, 22.319678873017494, 17.553166745968216, 17.934365811960372, 17.14654807625046, 12.134998917818166, 11.027483150026422, 28.715064341305684, 13.804574186067095, 16.84777751703098, 19.165753221675935, 16.076261613516785, 20.401838105461145, 33.57131140436464, 32.348333340301785, 14.159568646729724, 22.494734284035275, 13.0444058974144, 12.942893525362278, 23.852165264858517, 17.013112580497378, 13.121062323276538, 12.817113241929157, 30.447857334592324, 17.180070153709277, 24.086237637543274, 22.84897480427519, 15.072782365110845, 13.249825400173327, 12.206311145618837, 24.89940107578365, 73.75739416067401, 42.854578979512624, 39.40245503911115, 24.753932740556596, 31.41437607743883, 29.684499285876434, 19.89034374461794, 23.80562446936611, 30.987805310145493, 35.806340977539655, 36.37020880080945, 55.24173574865985, 87.24759532868651, 38.19016894386643, 29.110349096271467, 28.049880592281973, 47.62135285300038, 22.319678873017494, 42.10785803778799, 75.80202444429054, 51.591626129191596, 40.41571724385996, 60.67117749756472, 29.110349096271467, 41.45503620813183, 39.7114923241556, 91.07822475921115, 41.780172086809024, 56.55175575986736, 70.93179276322707, 32.033969127372174, 45.351947753528236, 49.61518196998445, 44.30137101535166, 30.927341318986464, 59.964336869699544, 43.95661501095434, 79.75075829332496, 79.75075829332496, 88.27604237373367, 25.39049775763493, 19.429584428039178, 26.713157919740368, 31.784678801853513, 17.180070153709277, 69.55984769951485, 44.47476167904283, 26.196478674560588, 22.582776184718522, 70.65525591042105, 61.626609520407456, 59.03467599561125, 84.89424029774196, 31.846818916590372, 70.65525591042105, 63.83170433047907, 50.004318665090636, 65.8579401324037, 91.07822475921115, 27.080884906424416, 69.01852867010916, 72.89809449073893, 72.89809449073893, 118.78839892336089, 23.666546242140512, 30.866995306309306, 39.172256347947744, 56.33128135834348, 27.995149135337257, 26.87013978811538, 48.08868329990042, 26.401940203271806, 61.14702744030778, 24.275148340672132, 20.16414949200632, 18.61240185635776, 35.11378365112735, 56.9952966774873, 38.33964103892552, 60.67117749756472, 60.199030648454944, 15.520899957093135, 27.886006392229614, 51.99626426796157, 50.49503426195374, 28.15966472087403, 88.27604237373367, 36.15772522700737, 69.55984769951485, 20.4816886421048, 30.447857334592324, 34.70469568042362, 29.396022468352484, 43.78524440968234, 93.6030061542134, 52.609179777419676, 35.94648303373688, 53.12545808585662, 25.891280453627324, 103.60908302823987, 15.490615283867873, 23.75917448515314, 39.40245503911115, 45.351947753528236, 17.934365811960372, 27.995149135337257, 21.973644325590612, 28.325147094540903, 53.542126217761066, 21.675267738479917, 39.86691871057298, 82.60436300370014, 17.45061689198871, 23.390822626823073, 14.076845015253456, 29.33866439394716, 22.75989509352673, 26.87013978811538, 14.187251079068192, 25.24216031657971, 65.34542957624015, 61.626609520407456, 29.684499285876434, 10.688203161633492, 8.554730245048765, 90.36944839098756, 21.131847230940878, 14.89717913158109, 38.79157659527073, 30.927341318986464, 81.9615307508321, 24.417803063601138, 13.069908149118522, 40.97207040739876, 53.02179868836547, 55.02636863114642, 34.367432111764096, 26.09434828302453, 30.56702687878971, 35.94648303373688, 21.845268970328604, 56.55175575986736, 69.83209724580306, 16.076261613516785, 32.47494113727571, 45.086990684122, 50.396507397123145, 73.18340956555208, 16.170734966018113, 16.684049195604416, 56.11166650505244, 9.904297461775021, 26.14536361045317, 40.022953418667065, 48.84596987082467, 36.016759615546256, 31.169907463288272, 45.263456176258764, 22.75989509352673, 22.319678873017494, 20.282645623050005, 56.11166650505244, 19.581972547389178, 33.90076227087091, 14.781248363830914, 32.033969127372174, 48.37128004023317, 41.05217218133816, 16.94678474038272, 24.22778221261098, 43.699809813576366, 78.51433738825796, 49.809370301716946, 69.28865955361954, 94.70636877096143, 27.028044172168716, 23.4365524707797, 16.489674134905655, 33.375179667439795, 13.458479789399725, 14.926303617194517, 11.807678025521156, 20.723120306578686, 140.51475485198003, 10.318974397214378, 12.742232560259533, 14.159568646729724, 14.354485905365031, 20.046345644076645, 52.301824199806006, 72.89809449073893, 42.52108200006278, 18.181246031735814, 15.673213754973451, 38.867415430562446, 22.189281900071105, 50.49503426195374, 71.20941195067294, 10.984491067549964, 56.773093073800744, 80.06289393593435, 22.450842133517945, 106.89798837141365, 67.41971617548266, 55.89290784893326, 46.51820540231633, 43.02230705927991, 64.08153455750669, 21.887977204970085, 27.668995768810042, 20.84490127137742, 39.634006510082365, 79.13013296849392, 51.99626426796157, 83.90519244027122, 92.1518260518184, 119.72006788311512, 90.36944839098756, 81.32370105824417, 56.55175575986736, 22.145985754016134, 39.479488162280596, 84.563269020706, 70.65525591042105, 69.55984769951485, 77.59961765106212, 35.25121511458854, 79.13013296849392, 80.69083499570341, 80.69083499570341, 69.55984769951485, 69.55984769951485, 137.25973406168575, 75.21212841006654, 76.39654709045266, 98.09510821132314, 70.65525591042105, 67.9484955630781, 67.9484955630781, 23.4365524707797, 24.513371642677637, 66.63425269055388, 50.396507397123145, 33.76859572409541, 20.046345644076645, 16.2975636122563, 44.73612093895834, 128.44088377025506, 13.484791569471833, 30.034410770075546, 82.28231911219243, 19.735555861572347, 29.110349096271467, 39.634006510082365, 64.33234259239626, 40.57389988868238, 85.2265069597304, 12.230174893930755, 61.38635013907805, 28.2698785323965, 33.90076227087091, 26.713157919740368, 35.11378365112735, 32.159346537604755, 44.387981683814154, 35.38918446958933, 38.0412795852805, 24.948080241282657, 26.557093177483523, 20.84490127137742, 26.817710263716034, 26.2990087931472, 60.908637772317434, 28.2698785323965, 29.110349096271467, 24.513371642677637, 27.080884906424416, 43.275130861102994, 68.48142221881122, 21.675267738479917, 52.50652775191775, 78.51433738825796, 11.601934773230344, 31.230845679407448, 34.16664924473404, 70.65525591042105, 26.14536361045317, 12.942893525362278, 33.11545195869231, 27.186876493132687, 12.968197316969134, 74.91890411002962, 39.32557222442699, 22.582776184718522, 22.893645317537526, 21.131847230940878, 17.046373787079762, 29.626578330054702, 36.15772522700737, 151.93267574549122, 23.80562446936611, 66.11570082213345, 72.89809449073893, 86.90744918987926, 19.20322291302118, 18.28808948244362, 40.17959882934157, 103.20514949397284, 73.75739416067401, 69.28865955361954, 72.04880596519355, 19.165753221675935, 66.89505178172818, 26.14536361045317, 71.20941195067294, 23.75917448515314, 107.31637526839133, 53.22932014095188, 44.387981683814154, 78.2082389954826, 64.08153455750669, 75.21212841006654, 77.90333396724647, 78.51433738825796, 60.199030648454944, 28.771203256660744, 44.12865633779556, 14.242778409807016, 18.61240185635776, 17.314815113938415, 17.213657767955564, 34.16664924473404, 16.01358620918395, 70.65525591042105, 87.58907276183992, 42.3553080116137, 59.03467599561125, 51.1901369093998, 19.658614220966218, 52.9183415530491, 48.08868329990042, 81.9615307508321, 30.56702687878971, 87.58907276183992, 30.806767041912167, 85.2265069597304, 61.14702744030778, 51.79355004464666, 23.25416777248505, 24.417803063601138, 29.975807060355134, 52.81508628524973, 63.83170433047907, 28.2698785323965, 58.80452160545304, 63.582848099201826, 25.539706914745466, 49.22907356066477, 26.76538304079522, 80.69083499570341, 69.01852867010916, 82.28231911219243, 51.39048944047805, 33.83461446305701, 81.9615307508321, 40.97207040739876, 15.951155153095918, 88.96839995507139, 81.00664999449546, 116.03666862607915, 14.83910053420958, 46.066137010589394, 59.73055807527518, 27.940524471445343, 58.34690118774256, 32.66578255521719, 55.24173574865985, 69.01852867010916, 64.83690740886694, 39.7114923241556, 103.60908302823987, 52.19977189231385, 76.09870517958821, 21.008389858307645, 58.119428177447986, 70.65525591042105, 58.57526450222688, 18.868615759264884, 26.2990087931472, 58.57526450222688, 31.660762082694095, 16.88071548801052, 27.346642204097876, 10.730035606033903, 15.340075995482293, 17.314815113938415, 45.263456176258764, 33.83461446305701, 15.796140293991145, 33.96703939987107, 42.68750480955783, 28.214717811314713, 27.080884906424416, 17.013112580497378, 31.41437607743883, 35.66674528344124, 49.61518196998445, 66.11570082213345, 52.50652775191775, 21.464625403032095, 13.0444058974144, 13.275729253905588, 26.557093177483523, 21.173160702117308, 54.173263858709824, 34.70469568042362, 41.29341833835342, 38.115651564714454, 18.075026786359576, 27.346642204097876, 31.048387510438282, 38.48969815122384, 19.278382202442366, 16.58657693697683, 14.214987631491761, 33.375179667439795, 37.232703447205715, 39.248839424943355, 29.453492679719986, 33.11545195869231, 23.666546242140512, 10.318974397214378, 17.079700020438295, 26.975306541751994, 92.1518260518184, 24.086237637543274, 30.62678644311483, 110.29129477712178, 34.23344619169951, 43.02230705927991, 57.89284200037607, 15.04337206739713, 10.563681823081545, 20.24306974171993, 119.2533235714264, 11.624616945432633, 20.4816886421048, 81.00664999449546, 46.97471015107385, 82.60436300370014, 10.399907116482579, 12.593781054375055, 22.893645317537526, 20.763634702888442, 18.467558995986426, 19.016604146815915, 16.58657693697683, 10.54306976753373, 17.348666159108166, 36.08717359043372, 15.310144149387147, 15.827022278874091, 30.507383918455513, 23.482371718117374, 18.979498589532987, 21.675267738479917, 20.203571081597193, 20.085536923187668, 16.979916273809664, 14.781248363830914, 8.655570698112841, 38.64034257045414, 31.846818916590372, 21.380942759123343, 22.450842133517945, 79.13013296849392, 30.507383918455513, 43.87084603329354, 32.22221914035047, 27.83159468972004, 63.08804244970428, 103.20514949397284, 107.7363996853844, 31.41437607743883, 19.053782246709744, 51.894808174781225, 38.0412795852805, 69.28865955361954, 29.396022468352484, 36.58394105020859, 33.702705802144536, 40.49473132867618, 48.27689708476307, 18.503663694167408, 34.23344619169951, 37.8190332109868, 58.119428177447986, 65.60118435636834, 80.37625124290746, 77.90333396724647, 22.40703562619394, 44.387981683814154, 25.1437504597293, 96.95226558425567, 30.56702687878971, 83.57807709540792, 146.11222634120347, 81.641993024771, 78.51433738825796, 59.49769069799599, 64.83690740886694, 75.50650035756513, 35.94648303373688, 17.113091407702502, 65.8579401324037, 36.08717359043372, 30.507383918455513, 26.76538304079522, 59.49769069799599, 13.172416709211404, 17.013112580497378, 22.10277408821932, 10.278744528592322, 42.77096031537661, 55.6750020519895, 25.34095533871963, 77.90333396724647, 37.08754689283638, 15.85796463907484, 91.07822475921115, 58.34690118774256, 77.90333396724647, 76.09870517958821, 68.21443845541918, 64.08153455750669, 20.84490127137742, 104.4216991379013, 74.33588057277046, 88.62154503481175, 28.827451925481085, 34.908640417532865, 33.4404292647954, 50.20003003478527, 17.348666159108166, 56.9952966774873, 43.02230705927991, 67.41971617548266, 77.29708541258049, 37.9670527218579, 70.93179276322707, 31.90908051733889, 66.63425269055388, 35.66674528344124, 92.87458178129451, 17.79479946994832, 17.180070153709277, 33.31005738633129, 55.457945789238345, 23.20879380099744, 10.33914833878563, 17.046373787079762, 20.602050815688813, 39.634006510082365, 29.684499285876434, 28.771203256660744, 36.65546388385301, 54.173263858709824, 27.40010580820802, 53.22932014095188, 114.68480048885151, 46.60915005647103, 82.28231911219243, 71.76791429061606, 144.4099673433467, 30.034410770075546, 64.83690740886694, 36.65546388385301, 70.65525591042105, 86.56862915519338, 35.59715165620933, 168.83235663184237, 35.59715165620933, 70.37979717263202, 70.37979717263202, 32.53843073806492, 65.8579401324037, 79.43983955226133, 31.048387510438282, 22.10277408821932, 34.4346215959791, 29.453492679719986, 83.90519244027122, 39.248839424943355, 33.702705802144536, 120.1886389803891, 60.67117749756472, 88.62154503481175, 106.89798837141365, 77.29708541258049, 100.02986085865793, 26.043432497666576, 18.467558995986426, 89.66618776422116, 36.37020880080945, 61.626609520407456, 183.26578935968087, 81.641993024771, 42.68750480955783, 51.290215346585704, 47.71445419747699, 28.60311491479713, 62.3530444922685, 38.115651564714454, 137.25973406168575, 9.637145935456955, 64.83690740886694, 81.9615307508321, 81.9615307508321, 46.427438201089, 121.60538462411665, 66.89505178172818, 12.593781054375055, 79.75075829332496, 61.14702744030778, 26.713157919740368, 51.39048944047805, 35.32013242426123, 76.99573263552006, 44.82358155812517, 84.2335880783952, 19.053782246709744, 84.89424029774196, 58.34690118774256, 52.097918711212095, 32.60204446315635, 33.63694444585419, 133.0366960079705, 85.56007407666469, 16.88071548801052, 22.626926270737744, 26.975306541751994, 119.72006788311512, 73.18340956555208, 51.1901369093998, 14.214987631491761, 29.975807060355134, 98.86447768050158, 51.79355004464666, 24.9968545761854, 56.33128135834348, 21.590763877931302, 47.34313736903344, 86.23113005463887, 34.09998263323346, 37.5980852538711, 69.28865955361954, 63.3349620664254, 58.34690118774256, 26.043432497666576, 37.37842812826559, 74.62682298320091, 58.57526450222688, 64.83690740886694, 58.57526450222688, 107.31637526839133, 61.38635013907805, 40.022953418667065, 39.479488162280596, 24.417803063601138, 59.03467599561125, 18.467558995986426, 81.641993024771, 22.67116267165818, 98.09510821132314, 73.75739416067401, 80.06289393593435, 42.93836112094585, 71.76791429061606, 25.24216031657971, 38.0412795852805, 91.43469485205186, 29.511075247281195, 15.642631884188171, 79.13013296849392, 85.56007407666469, 14.2706235200274, 45.529450258314064, 10.688203161633492, 74.04607243930377, 13.198169226352599, 20.84490127137742, 46.88305220915621, 80.37625124290746, 36.72712654703974, 31.72265993608251, 24.80232751057381, 51.49095957359226, 73.75739416067401, 42.3553080116137, 30.987805310145493, 25.992616060150944, 83.90519244027122, 25.44013703364339, 91.07822475921115, 71.48811770889668, 77.29708541258049, 41.617286632367644, 49.71218131735948, 56.55175575986736, 64.33234259239626, 68.48142221881122, 41.45503620813183, 29.33866439394716, 92.87458178129451, 53.96206231832688, 126.44959243764188, 20.56185170491249, 51.894808174781225, 119.2533235714264, 24.22778221261098, 51.894808174781225, 44.91121316574036, 68.21443845541918, 56.55175575986736, 30.388446900124176, 27.995149135337257, 27.028044172168716, 29.453492679719986, 16.88071548801052, 25.39049775763493, 49.809370301716946, 25.2915095879081, 60.199030648454944, 88.96839995507139, 30.21091022897123, 42.10785803778799, 48.65553748142455, 46.70027251047952, 47.71445419747699, 66.11570082213345, 51.290215346585704, 55.02636863114642, 100.02986085865793, 60.67117749756472, 57.66713919909174, 35.59715165620933, 48.65553748142455, 26.24769367033534, 51.79355004464666, 17.656319241826594, 43.444504934395866, 27.186876493132687, 83.90519244027122, 69.01852867010916, 19.929230034775262, 128.94358717585138, 95.0770390179677, 62.84208548133934, 40.49473132867618, 48.75066069158903, 79.75075829332496, 23.666546242140512, 16.94678474038272, 26.09434828302453, 125.4655539023251, 26.35042423874788, 63.3349620664254, 84.89424029774196, 80.06289393593435, 80.06289393593435, 28.436008585761353, 35.38918446958933, 66.37447035867858, 19.89034374461794, 49.133016862271724, 32.09659661306934, 61.38635013907805, 40.73270164287187, 37.9670527218579, 156.7555365841094, 31.41437607743883, 22.36331459495464, 63.08804244970428, 70.65525591042105, 22.938403163153815, 25.24216031657971, 96.95226558425567, 11.946863197656471, 20.203571081597193, 25.63966650134361, 39.479488162280596, 35.94648303373688, 59.73055807527518, 80.37625124290746, 59.265731184583274, 51.894808174781225, 63.08804244970428, 33.245062173048, 25.63966650134361, 25.489873356103875, 31.41437607743883, 40.73270164287187, 49.71218131735948, 16.36135047271935, 43.10641711474324, 23.898797049169467, 25.891280453627324, 39.32557222442699, 67.9484955630781, 70.10541234668786, 52.50652775191775, 48.08868329990042, 73.18340956555208, 61.14702744030778, 18.979498589532987, 78.2082389954826, 55.89290784893326, 50.004318665090636, 25.539706914745466, 123.52039078374908, 105.24068867470233, 100.02986085865793, 33.90076227087091, 79.13013296849392, 90.01713130052181, 31.291903031786116, 44.04255166984753, 19.697047472120772, 48.27689708476307, 131.48677155266952, 51.591626129191596, 100.42136666458902, 129.9549042642197, 132.51803508331912, 146.6840934260825, 27.080884906424416, 75.50650035756513, 51.591626129191596, 60.199030648454944, 35.59715165620933, 49.809370301716946, 36.65546388385301, 38.26483200699776, 39.94485987582193, 43.10641711474324, 27.72308958496738, 77.90333396724647, 21.802644068801417, 42.438114061533675, 29.167260806600495, 32.47494113727571, 22.40703562619394, 33.76859572409541, 72.33079701870139, 29.56877039069624, 46.246434773093995, 80.37625124290746, 73.75739416067401, 86.56862915519338, 72.89809449073893, 22.40703562619394, 14.354485905365031, 49.809370301716946, 61.38635013907805, 25.44013703364339, 62.10995300937982, 50.79177210350426, 28.1047190510635, 58.80452160545304, 45.88654216244616, 24.70563239932894, 26.87013978811538, 36.87087245606033, 31.598985005566618, 31.598985005566618, 69.01852867010916, 33.05083646319603, 37.37842812826559, 39.634006510082365, 34.56939489431364, 41.45503620813183, 40.49473132867618, 30.927341318986464, 32.92198346312245, 53.96206231832688, 35.7364749685521, 51.591626129191596, 46.60915005647103, 28.491601937740562, 47.99485175252902, 32.09659661306934, 63.582848099201826, 40.17959882934157, 41.132430556936306, 22.626926270737744, 59.03467599561125, 18.758380733362646, 41.37414835829223, 35.32013242426123, 51.1901369093998, 63.83170433047907, 82.92766733936132, 73.46984133199462, 70.93179276322707, 114.23768682435218, 73.46984133199462, 67.68358948381629, 60.199030648454944, 69.01852867010916, 42.854578979512624, 59.03467599561125, 87.58907276183992, 90.7231444115574, 113.7923162908356, 162.3644906197382, 106.06610162605153, 75.50650035756513, 42.52108200006278, 54.811841150452466, 30.62678644311483, 67.68358948381629, 45.529450258314064, 31.53732846903916, 36.51255777326842, 89.31661242710538, 32.85774546700983, 36.08717359043372, 82.28231911219243, 62.597087408320306, 40.022953418667065, 31.784678801853513, 45.17513726542669, 74.91890411002962, 95.44916002730179, 39.095822701298815, 77.29708541258049, 92.87458178129451, 30.21091022897123, 42.60421214369869, 87.24759532868651, 151.93267574549122, 52.609179777419676, 23.666546242140512, 23.20879380099744, 15.340075995482293, 26.14536361045317, 89.66618776422116, 43.19069160819034, 34.63697922255266, 44.99901609609206, 48.84596987082467, 40.892124929554, 29.858942463791767, 39.172256347947744, 53.333385249853684, 67.41971617548266, 30.21091022897123, 43.5294404008871, 39.01953819342525, 50.004318665090636, 79.75075829332496, 72.04880596519355, 40.81233544283598, 59.49769069799599, 75.80202444429054, 29.74253347911242, 46.7915731119465, 32.348333340301785, 48.75066069158903, 43.699809813576366, 36.44131378072682, 38.115651564714454, 27.668995768810042, 91.43469485205186, 45.263456176258764, 17.725424121461643, 41.780172086809024, 39.7114923241556, 143.8469662358426, 75.50650035756513, 31.90908051733889, 24.850816893992036, 71.76791429061606, 23.20879380099744, 81.32370105824417, 43.78524440968234, 54.811841150452466, 33.11545195869231, 56.773093073800744, 46.60915005647103, 72.89809449073893, 33.702705802144536, 116.03666862607915, 34.16664924473404, 35.11378365112735, 62.84208548133934, 73.18340956555208, 35.87634357692165, 34.84052601883783, 53.43765380953904, 66.11570082213345, 82.28231911219243, 46.15619785585283, 42.3553080116137, 58.57526450222688, 66.37447035867858, 36.798929313140206, 47.52843316962603, 52.9183415530491, 84.2335880783952, 72.33079701870139, 69.01852867010916, 95.0770390179677, 112.46659565031442, 53.75168417471723, 48.08868329990042, 81.00664999449546, 110.29129477712178, 68.21443845541918, 50.004318665090636, 52.609179777419676, 29.74253347911242, 20.046345644076645, 27.507346790200362, 33.702705802144536, 43.78524440968234, 39.40245503911115, 47.435694792893365, 120.65904401308495, 63.83170433047907, 71.48811770889668, 28.214717811314713, 38.19016894386643, 34.84052601883783, 33.05083646319603, 47.99485175252902, 47.06654728734682, 45.086990684122, 50.6926662381911, 91.79256012938836, 71.76791429061606, 71.76791429061606, 59.49769069799599, 22.016603538363892, 48.46584751755162, 43.61454191906839, 37.5980852538711, 56.9952966774873, 34.03344610288473, 78.82163381626928, 58.119428177447986, 53.542126217761066, 46.60915005647103, 22.27612829392538, 26.817710263716034, 39.7114923241556, 54.173263858709824, 26.453556883236853, 55.457945789238345, 49.22907356066477, 25.58963789966816, 92.1518260518184, 51.09025374715126, 121.13129015902297, 49.51837188956857, 46.51820540231633, 75.21212841006654, 41.05217218133816, 54.27917440928786, 30.56702687878971, 53.542126217761066, 39.556671887793456, 70.65525591042105, 45.263456176258764, 72.33079701870139, 72.33079701870139, 79.43983955226133, 64.08153455750669, 32.79363281328884, 28.883810562338418, 38.33964103892552, 44.04255166984753, 44.21492934326177, 63.3349620664254, 71.76791429061606, 36.016759615546256, 47.71445419747699, 55.457945789238345, 41.132430556936306, 79.13013296849392, 81.32370105824417, 87.93188669987823, 45.44061233480379, 42.93836112094585, 88.62154503481175, 77.59961765106212, 102.00277308269969, 102.00277308269969, 136.72460907194932, 88.96839995507139, 115.58428452718766, 76.69555472159655, 49.42175070681059, 45.351947753528236, 60.908637772317434, 81.00664999449546, 71.76791429061606, 64.83690740886694, 70.37979717263202, 68.21443845541918, 52.19977189231385, 38.0412795852805, 43.95661501095434, 107.7363996853844, 83.90519244027122, 86.23113005463887, 56.9952966774873, 56.9952966774873, 92.1518260518184, 106.06610162605153, 52.609179777419676, 67.1568716116849, 56.55175575986736, 45.88654216244616, 62.10995300937982, 88.96839995507139, 57.89284200037607, 86.90744918987926, 86.90744918987926, 66.89505178172818, 66.89505178172818, 72.04880596519355, 62.10995300937982, 126.94450214932196, 156.14440466345445, 119.72006788311512, 82.28231911219243, 80.69083499570341, 58.34690118774256, 104.83039411284926, 104.83039411284926, 57.21836996148928, 67.1568716116849, 76.69555472159655, 111.59137365930344, 92.87458178129451, 58.34690118774256, 50.6926662381911, 34.501942437892716, 56.33128135834348, 96.19777706766972, 90.36944839098756, 141.0647140528757, 59.265731184583274, 31.846818916590372, 56.55175575986736, 43.95661501095434, 88.27604237373367, 88.27604237373367, 27.507346790200362, 25.891280453627324, 56.55175575986736, 77.90333396724647, 56.11166650505244, 66.37447035867858, 64.58413226217957, 80.06289393593435, 58.119428177447986, 29.053548433146837, 75.50650035756513, 72.61389175398708, 118.78839892336089, 75.80202444429054, 72.33079701870139, 57.442316329639176, 86.90744918987926, 40.892124929554, 116.03666862607915, 104.83039411284926, 48.37128004023317, 47.90120329104557, 49.133016862271724, 38.48969815122384, 49.133016862271724, 108.15806803146498, 71.76791429061606, 43.61454191906839, 44.99901609609206, 40.65322322588273, 73.18340956555208, 120.65904401308495, 125.9566121960733, 23.482371718117374, 54.38529201854992, 42.190180314697045, 50.004318665090636, 46.88305220915621, 40.41571724385996, 132.51803508331912, 34.70469568042362, 38.33964103892552, 36.228414794400415, 39.40245503911115, 30.866995306309306, 48.94146538270724, 31.475792237910618, 55.6750020519895, 58.34690118774256, 65.60118435636834, 78.2082389954826, 61.867809250367884, 38.56494680401705, 114.23768682435218, 37.305494633128845, 47.250760544967605, 37.89297069129353, 55.6750020519895, 33.57131140436464, 55.02636863114642, 29.281418237700255, 40.25815129472595, 43.95661501095434, 57.21836996148928, 41.05217218133816, 35.59715165620933, 20.085536923187668, 30.987805310145493, 110.7229626998625, 70.65525591042105, 40.41571724385996, 32.66578255521719, 93.96935796203766, 30.21091022897123, 102.40200064874995, 90.01713130052181, 136.72460907194932, 136.72460907194932, 59.03467599561125, 42.93836112094585, 66.11570082213345, 70.37979717263202, 34.63697922255266, 37.524723159601, 31.10908815096766, 50.79177210350426, 68.21443845541918, 45.79700747446617, 66.11570082213345, 26.76538304079522, 72.89809449073893, 54.811841150452466, 121.60538462411665, 41.05217218133816, 70.37979717263202, 112.02812994692015, 54.598150033144236, 45.17513726542669, 56.33128135834348, 68.21443845541918, 108.15806803146498, 108.15806803146498, 94.33714363029665, 35.04526897307925, 28.827451925481085, 34.501942437892716, 31.10908815096766, 49.809370301716946, 22.715485556228362, 42.77096031537661, 59.03467599561125, 46.33684810653952, 32.9863470466753, 61.38635013907805, 65.8579401324037, 75.21212841006654, 69.83209724580306, 61.14702744030778, 42.10785803778799, 43.87084603329354, 33.76859572409541, 60.199030648454944, 82.92766733936132, 146.6840934260825, 111.59137365930344, 83.25223705241488, 80.37625124290746, 68.7494509271083, 84.2335880783952, 34.908640417532865, 25.58963789966816, 58.57526450222688, 44.64883097460398, 70.93179276322707, 34.501942437892716, 32.60204446315635, 72.33079701870139, 66.37447035867858, 57.89284200037607, 66.63425269055388, 44.5617113320772, 53.542126217761066, 59.265731184583274, 39.095822701298815, 39.248839424943355, 84.2335880783952, 58.34690118774256, 179.7211081206401, 56.55175575986736, 76.69555472159655, 86.90744918987926, 75.21212841006654, 58.80452160545304, 147.25819873544805, 40.17959882934157, 70.37979717263202, 94.33714363029665, 125.9566121960733, 120.65904401308495, 84.89424029774196, 34.97688798216538, 40.022953418667065, 17.247311047115943, 22.80439145296227, 22.27612829392538, 39.94485987582193, 50.29817277977263, 73.75739416067401, 125.9566121960733, 55.02636863114642, 117.86398026897247, 69.01852867010916, 23.992334296139543, 29.281418237700255, 71.20941195067294, 47.99485175252902, 69.83209724580306, 28.827451925481085, 37.8190332109868, 148.99403178365867, 123.03883041717654, 15.310144149387147, 36.87087245606033, 22.36331459495464, 55.24173574865985, 39.78912962559818, 26.817710263716034, 25.94189877662824, 101.20898120494374, 44.99901609609206, 25.24216031657971, 35.11378365112735, 29.396022468352484, 31.660762082694095, 126.44959243764188, 58.57526450222688, 92.87458178129451, 25.192907336481866, 31.230845679407448, 75.80202444429054, 41.617286632367644, 38.26483200699776, 57.21836996148928, 139.42126035437516, 116.49082330627213, 100.42136666458902, 76.39654709045266, 50.5937537501147, 40.022953418667065, 46.066137010589394, 66.11570082213345, 49.133016862271724, 83.57807709540792, 71.48811770889668, 25.539706914745466, 22.80439145296227, 38.71588573814723, 128.94358717585138, 208.48013272330223, 81.641993024771, 132.51803508331912, 124.97641006343288, 45.17513726542669, 50.5937537501147, 57.89284200037607, 65.8579401324037, 67.1568716116849, 104.01459751366006, 52.81508628524973, 53.333385249853684, 61.14702744030778, 24.753932740556596, 47.62135285300038, 42.02569638966092, 33.245062173048, 46.51820540231633, 79.13013296849392, 72.33079701870139, 56.773093073800744, 55.02636863114642, 58.119428177447986, 70.37979717263202, 95.0770390179677, 161.10096012939127, 187.61181231093738, 129.44825810694724, 30.26997357328364, 45.61846186294896, 66.89505178172818, 42.60421214369869, 79.13013296849392, 64.83690740886694, 91.07822475921115, 76.39654709045266, 55.02636863114642, 56.9952966774873, 59.49769069799599, 52.609179777419676, 107.31637526839133, 120.65904401308495, 82.92766733936132, 62.10995300937982, 63.3349620664254, 44.64883097460398, 57.89284200037607, 62.597087408320306, 62.597087408320306, 39.095822701298815, 54.811841150452466, 54.811841150452466, 54.811841150452466, 49.133016862271724, 74.04607243930377, 47.52843316962603, 36.65546388385301, 63.582848099201826, 74.04607243930377, 30.927341318986464, 51.49095957359226, 46.15619785585283, 55.457945789238345, 50.89107172411317, 45.7076474882609, 38.115651564714454, 50.004318665090636, 58.34690118774256, 80.37625124290746, 70.37979717263202, 65.34542957624015, 78.51433738825796, 63.3349620664254, 55.89290784893326, 70.93179276322707, 161.10096012939127, 48.08868329990042, 55.457945789238345, 56.773093073800744, 49.9067492938035, 77.29708541258049, 77.59961765106212, 64.08153455750669, 59.73055807527518, 90.7231444115574, 60.434642992688794, 118.3252868447224, 98.09510821132314, 76.69555472159655, 50.5937537501147, 87.24759532868651, 25.094689498803355, 58.57526450222688, 109.43300111180854, 118.78839892336089, 122.08133464248264, 108.5813867407896, 107.7363996853844, 52.097918711212095, 64.33234259239626, 141.0647140528757, 76.99573263552006, 119.2533235714264, 59.964336869699544, 150.1626064581185, 130.4635333784854, 78.82163381626928, 183.98307137873957, 193.56725052076862, 54.811841150452466, 30.987805310145493, 58.80452160545304, 125.4655539023251, 89.66618776422116, 102.00277308269969, 44.64883097460398, 121.13129015902297, 126.44959243764188, 62.84208548133934, 59.265731184583274, 49.9067492938035, 81.32370105824417, 123.03883041717654, 53.75168417471723, 25.840760898054555, 63.582848099201826, 172.1622764913071, 88.62154503481175, 135.13171912803952, 88.96839995507139, 108.15806803146498, 44.64883097460398, 57.89284200037607, 59.03467599561125, 43.3597351955915, 84.563269020706, 93.2380826167318, 54.598150033144236, 211.76321709954237, 203.65069565035918, 92.1518260518184, 58.119428177447986, 92.51249810131188, 64.08153455750669, 75.21212841006654, 122.08133464248264, 34.908640417532865, 69.28865955361954, 59.265731184583274, 135.66060972280735, 176.93479059943397, 126.94450214932196, 59.49769069799599, 180.4245166495895, 155.5356553203632, 105.24068867470233, 58.80452160545304, 165.5668432526847, 124.97641006343288, 95.0770390179677, 152.5273234252381, 161.10096012939127, 60.199030648454944, 56.773093073800744, 102.00277308269969, 47.52843316962603, 158.60332124553608, 137.7969534714142, 180.4245166495895, 71.76791429061606, 101.6051019574268, 88.96839995507139, 83.25223705241488, 69.01852867010916, 73.75739416067401, 61.867809250367884, 57.442316329639176, 61.626609520407456, 101.6051019574268, 208.48013272330223, 162.99996726850102, 53.96206231832688, 139.42126035437516, 121.60538462411665, 98.86447768050158, 157.3690604074729, 157.3690604074729, 106.06610162605153, 139.96693973881514, 124.00383592430612, 126.44959243764188, 113.34868209247811, 136.19157033683516, 116.49082330627213, 153.12429849019884, 125.9566121960733, 160.47288700783744, 112.02812994692015, 284.95855538715824, 141.61682573322432, 198.9331323667161, 210.93763055625195, 44.21492934326177, 166.86540009848025, 56.33128135834348, 99.2514223056366, 64.08153455750669, 78.82163381626928, 162.99996726850102, 156.14440466345445, 58.119428177447986, 124.00383592430612, 54.598150033144236, 92.51249810131188, 67.1568716116849, 43.10641711474324, 42.3553080116137, 178.32250736130783, 156.14440466345445, 54.38529201854992, 48.56059987746486, 62.10995300937982, 161.73149145963927, 43.275130861102994, 77.90333396724647, 58.34690118774256, 44.82358155812517, 46.97471015107385, 74.62682298320091, 53.43765380953904, 36.08717359043372, 63.3349620664254, 43.10641711474324, 137.7969534714142, 59.73055807527518, 69.01852867010916, 76.99573263552006, 75.50650035756513, 55.24173574865985, 64.83690740886694, 57.442316329639176, 69.01852867010916, 74.91890411002962, 154.92927926604594, 206.05126560732333, 238.09178486008656, 153.7236100495016, 58.119428177447986, 157.3690604074729, 92.87458178129451, 41.53608219643496, 96.95226558425567, 68.48142221881122, 119.2533235714264, 106.48123261036584, 69.83209724580306, 136.72460907194932, 78.2082389954826, 90.36944839098756, 213.42409659600423, 191.3121237449989, 87.24759532868651, 119.72006788311512, 121.13129015902297, 160.47288700783744, 260.4732060371668, 82.28231911219243, 91.43469485205186, 245.649629409089, 135.13171912803952, 259.4577172438939, 77.90333396724647, 105.24068867470233, 136.72460907194932, 116.03666862607915, 59.49769069799599, 55.24173574865985, 51.79355004464666, 212.59203489720244, 202.8567368308649, 174.8734378511126, 111.15632012308374, 175.55787313507827, 151.93267574549122, 112.46659565031442, 112.46659565031442, 88.96839995507139, 95.82273747708687, 119.2533235714264, 96.57428452170761, 126.94450214932196, 107.7363996853844, 125.9566121960733, 69.01852867010916, 148.99403178365867, 112.90677745995042, 137.7969534714142, 104.01459751366006, 168.17414165184545, 88.27604237373367, 104.01459751366006, 167.51849280886478, 113.34868209247811, 129.9549042642197, 157.98498549518746, 51.79355004464666, 104.01459751366006, 63.582848099201826, 49.32531805312982, 90.36944839098756, 47.435694792893365, 130.4635333784854, 109.43300111180854, 181.83960367209863, 89.31661242710538, 57.89284200037607, 69.28865955361954, 89.66618776422116, 57.89284200037607, 132.00139622598326, 132.00139622598326, 134.60489048229144, 176.24498722300464, 105.24068867470233, 111.15632012308374, 117.40447215712238, 157.3690604074729, 123.52039078374908, 59.265731184583274, 57.21836996148928, 85.89494673838159, 69.83209724580306, 85.89494673838159, 207.6673457129932, 170.1565252164996, 99.2514223056366, 143.28616006164455, 37.5980852538711, 131.48677155266952, 50.79177210350426, 53.96206231832688, 70.37979717263202, 98.86447768050158, 63.08804244970428, 92.51249810131188, 119.72006788311512, 151.3403463773431, 78.2082389954826, 55.02636863114642, 43.275130861102994, 71.20941195067294, 46.33684810653952, 69.55984769951485, 53.542126217761066, 172.83610058973207, 197.38502244855516, 70.10541234668786, 138.8777083722429, 43.78524440968234, 39.634006510082365, 111.15632012308374, 69.28865955361954, 144.4099673433467, 29.858942463791767, 105.24068867470233, 69.55984769951485, 66.37447035867858, 66.37447035867858, 66.37447035867858, 55.457945789238345, 80.37625124290746, 49.51837188956857, 47.807737558209425, 49.32531805312982, 53.542126217761066, 171.49107938408403, 83.25223705241488, 74.33588057277046, 65.8579401324037, 126.44959243764188, 52.9183415530491, 73.75739416067401, 70.37979717263202, 94.33714363029665, 76.39654709045266, 202.8567368308649, 202.8567368308649, 151.93267574549122, 61.626609520407456, 49.32531805312982, 65.09067188950593, 72.04880596519355, 68.7494509271083, 115.58428452718766, 44.91121316574036, 62.3530444922685, 56.9952966774873, 51.39048944047805, 56.55175575986736, 55.6750020519895, 223.66653284614108, 96.95226558425567, 83.25223705241488, 146.6840934260825, 63.3349620664254, 148.99403178365867, 64.83690740886694, 41.132430556936306, 57.442316329639176, 38.0412795852805, 58.34690118774256, 78.82163381626928, 60.199030648454944, 71.48811770889668, 77.90333396724647, 104.83039411284926, 36.798929313140206, 126.44959243764188, 147.25819873544805, 63.582848099201826, 69.28865955361954, 56.773093073800744, 64.33234259239626, 54.811841150452466, 104.83039411284926, 195.08541728258788, 171.49107938408403, 218.48530319325408, 136.72460907194932, 111.15632012308374, 99.63988138921447, 148.4131591025766, 130.4635333784854, 133.0366960079705, 139.42126035437516, 74.91890411002962, 70.65525591042105, 75.21212841006654, 109.86130976812342, 82.60436300370014, 39.556671887793456, 91.79256012938836, 180.4245166495895, 48.94146538270724, 199.71173463253027, 99.2514223056366, 113.7923162908356, 138.8777083722429, 150.7503262825534, 40.1011996370117, 72.33079701870139, 114.23768682435218, 89.31661242710538, 56.773093073800744, 70.65525591042105, 179.7211081206401, 98.09510821132314, 176.93479059943397, 201.27809318052155, 202.06587336346374, 148.99403178365867, 36.87087245606033, 104.83039411284926, 30.62678644311483, 79.43983955226133, 93.6030061542134, 45.17513726542669, 119.2533235714264, 79.13013296849392, 64.58413226217957, 98.4790416094955, 76.69555472159655, 171.49107938408403, 159.8472625113303, 151.3403463773431, 147.834551029463, 108.15806803146498, 97.71267162761835, 67.41971617548266, 117.86398026897247, 143.28616006164455, 123.52039078374908, 180.4245166495895, 124.97641006343288, 320.3875157950033, 38.64034257045414, 87.93188669987823, 97.71267162761835, 73.46984133199462, 123.03883041717654, 133.0366960079705, 63.83170433047907, 85.2265069597304, 129.9549042642197, 27.561124577174567, 54.811841150452466, 36.08717359043372, 36.299242562272255, 50.49503426195374, 32.92198346312245, 38.33964103892552, 43.02230705927991, 37.01518097066168, 27.995149135337257, 41.698649825700315, 29.110349096271467, 28.827451925481085, 34.300373728940066, 33.4404292647954, 27.293282919182133, 32.92198346312245, 44.30137101535166, 26.14536361045317, 43.444504934395866, 36.942956250241515, 28.996858600549206, 28.827451925481085, 28.996858600549206, 40.41571724385996, 30.329152388418176, 26.817710263716034, 31.660762082694095, 25.740017318702005, 25.1437504597293, 26.2990087931472, 27.40010580820802, 27.61500750152876, 24.609314266596865, 22.84897480427519, 24.513371642677637, 25.34095533871963, 25.34095533871963, 26.557093177483523, 29.626578330054702, 34.63697922255266, 31.9714638416083, 25.94189877662824, 26.87013978811538, 32.92198346312245, 23.80562446936611, 25.58963789966816, 25.44013703364339, 34.84052601883783, 23.118311288923124, 31.53732846903916, 27.133828946090556, 34.23344619169951, 32.72964525738904, 26.76538304079522, 29.975807060355134, 21.506589466713315, 26.35042423874788, 28.15966472087403, 25.045724266551254, 33.50580642730562, 27.668995768810042, 25.44013703364339, 26.24769367033534, 37.9670527218579, 32.09659661306934, 38.867415430562446, 29.684499285876434, 33.245062173048, 35.11378365112735, 27.561124577174567, 34.4346215959791, 37.37842812826559, 36.37020880080945, 53.646802873050504, 34.84052601883783, 45.351947753528236, 37.45150421083523, 35.45837151398557, 45.7076474882609, 42.60421214369869, 41.37414835829223, 32.79363281328884, 50.79177210350426, 42.27266353442275, 44.64883097460398, 41.05217218133816, 38.33964103892552, 26.975306541751994, 39.634006510082365, 36.08717359043372, 42.438114061533675, 42.93836112094585, 37.37842812826559, 35.87634357692165, 36.37020880080945, 28.547303976590324, 39.248839424943355, 39.40245503911115, 42.60421214369869, 49.42175070681059, 52.19977189231385, 39.94485987582193, 41.29341833835342, 31.169907463288272, 26.66103470132838, 42.27266353442275, 43.10641711474324, 37.67159077349981, 26.505274475545082, 40.81233544283598, 37.01518097066168, 34.300373728940066, 49.037147591523336, 34.908640417532865, 38.94340253332466, 32.159346537604755, 35.45837151398557, 37.8190332109868, 45.263456176258764, 39.248839424943355, 44.82358155812517, 39.248839424943355, 32.28521466114651, 41.2128458403547, 28.883810562338418, 23.75917448515314, 40.73270164287187, 46.7915731119465, 42.93836112094585, 35.38918446958933, 41.05217218133816, 40.57389988868238, 35.11378365112735, 43.3597351955915, 36.87087245606033, 28.60311491479713, 42.190180314697045, 39.248839424943355, 37.89297069129353, 40.65322322588273, 34.908640417532865, 35.806340977539655, 40.73270164287187, 37.524723159601, 25.992616060150944, 38.94340253332466, 40.57389988868238, 27.940524471445343, 38.19016894386643, 29.110349096271467, 25.539706914745466, 32.72964525738904, 40.65322322588273, 28.15966472087403, 27.028044172168716, 30.56702687878971, 39.172256347947744, 34.03344610288473, 44.82358155812517, 35.25121511458854, 37.89297069129353, 36.37020880080945, 35.52769382137767, 38.26483200699776, 41.780172086809024, 51.894808174781225, 57.66713919909174, 33.245062173048, 50.79177210350426, 41.05217218133816, 38.33964103892552, 37.37842812826559, 35.52769382137767, 36.798929313140206, 39.40245503911115, 42.68750480955783, 37.232703447205715, 39.556671887793456, 43.02230705927991, 32.22221914035047, 33.31005738633129, 42.854578979512624, 30.927341318986464, 41.2128458403547, 46.15619785585283, 76.69555472159655, 73.75739416067401, 42.93836112094585, 43.5294404008871, 40.892124929554, 25.192907336481866, 34.77254452624438, 34.77254452624438, 41.943695056893915, 25.840760898054555, 23.25416777248505, 34.4346215959791, 38.33964103892552, 37.8190332109868, 30.866995306309306, 22.938403163153815, 38.64034257045414, 30.093129052017023, 47.807737558209425, 56.9952966774873, 49.133016862271724, 47.435694792893365, 39.01953819342525, 37.45150421083523, 38.56494680401705, 28.940279382223952, 39.556671887793456, 39.556671887793456, 39.94485987582193, 39.94485987582193, 38.71588573814723, 45.61846186294896, 36.37020880080945, 47.71445419747699, 41.05217218133816, 46.97471015107385, 38.64034257045414, 44.04255166984753, 28.491601937740562, 24.18050850623376, 47.34313736903344, 46.427438201089, 38.115651564714454, 36.15772522700737, 41.698649825700315, 48.37128004023317, 38.867415430562446, 37.01518097066168, 49.42175070681059, 48.37128004023317, 69.83209724580306, 30.21091022897123, 23.898797049169467, 35.7364749685521, 45.086990684122, 49.809370301716946, 47.15856396830611, 48.84596987082467, 44.21492934326177, 26.922671813996086, 37.305494633128845, 44.387981683814154, 40.25815129472595, 38.71588573814723, 42.60421214369869, 53.85677052285177, 36.08717359043372, 40.892124929554, 35.94648303373688, 43.3597351955915, 44.91121316574036, 43.19069160819034, 31.230845679407448, 39.556671887793456, 49.133016862271724, 29.167260806600495, 33.375179667439795, 45.529450258314064, 30.56702687878971, 34.63697922255266, 51.79355004464666, 34.908640417532865, 40.65322322588273, 44.99901609609206, 35.18243227767251, 36.228414794400415, 41.943695056893915, 51.591626129191596, 45.529450258314064, 42.60421214369869, 42.52108200006278, 37.45150421083523, 47.34313736903344, 45.88654216244616, 31.475792237910618, 40.022953418667065, 41.132430556936306, 36.37020880080945, 38.71588573814723, 28.325147094540903, 51.1901369093998, 61.14702744030778, 32.60204446315635, 47.90120329104557, 28.996858600549206, 47.34313736903344, 31.72265993608251, 38.64034257045414, 34.77254452624438, 35.45837151398557, 47.435694792893365, 43.444504934395866, 38.0412795852805, 43.61454191906839, 35.18243227767251, 46.427438201089, 37.67159077349981, 44.64883097460398, 27.995149135337257, 33.96703939987107, 37.160054292819765, 37.232703447205715, 30.866995306309306, 67.1568716116849, 46.15619785585283, 65.8579401324037, 29.053548433146837, 43.61454191906839, 33.83461446305701, 32.9863470466753, 43.02230705927991, 36.016759615546256, 53.43765380953904, 42.93836112094585, 40.33685733281881, 38.79157659527073, 38.94340253332466, 35.94648303373688, 40.57389988868238, 36.44131378072682, 35.18243227767251, 46.7915731119465, 46.15619785585283, 45.44061233480379, 39.556671887793456, 52.301824199806006, 36.72712654703974, 44.73612093895834, 39.94485987582193, 52.81508628524973, 64.33234259239626, 56.773093073800744, 28.214717811314713, 49.809370301716946, 41.45503620813183, 33.4404292647954, 48.84596987082467, 34.56939489431364, 31.230845679407448, 29.626578330054702, 46.51820540231633, 45.97625189374871, 39.94485987582193, 29.167260806600495, 25.891280453627324, 27.40010580820802, 61.867809250367884, 56.9952966774873, 50.004318665090636, 41.45503620813183, 46.97471015107385, 37.160054292819765, 24.561296107667808, 25.689792910615846, 37.37842812826559, 25.891280453627324, 33.1801937796528, 17.382583384208317, 23.80562446936611, 48.84596987082467, 37.45150421083523, 37.160054292819765, 28.65903496526287, 26.35042423874788, 24.80232751057381, 24.948080241282657, 30.507383918455513, 28.940279382223952, 30.093129052017023, 42.190180314697045, 34.908640417532865, 34.908640417532865, 50.79177210350426, 37.5980852538711, 25.790339917193062, 56.773093073800744, 64.83690740886694, 22.016603538363892, 41.943695056893915, 26.09434828302453, 29.281418237700255, 23.992334296139543, 24.370158584949497, 51.692489491288626, 33.245062173048, 17.484733434196265, 31.10908815096766, 36.51255777326842, 34.501942437892716, 25.489873356103875, 25.790339917193062, 31.846818916590372, 33.245062173048, 23.4365524707797, 42.68750480955783, 35.45837151398557, 35.52769382137767, 33.63694444585419, 26.87013978811538, 25.790339917193062, 37.8190332109868, 40.022953418667065, 32.72964525738904, 55.24173574865985, 40.41571724385996, 36.72712654703974, 27.561124577174567, 39.172256347947744, 24.275148340672132, 38.79157659527073, 38.79157659527073, 27.61500750152876, 40.17959882934157, 34.908640417532865, 38.0412795852805, 43.95661501095434, 50.1020787877761, 41.05217218133816, 61.14702744030778, 61.14702744030778, 35.806340977539655, 41.05217218133816, 33.57131140436464, 48.27689708476307, 30.68666283939559, 29.281418237700255, 24.9968545761854, 32.159346537604755, 61.867809250367884, 57.442316329639176, 35.18243227767251, 34.09998263323346, 28.771203256660744, 39.172256347947744, 51.692489491288626, 25.192907336481866, 42.60421214369869, 37.89297069129353, 27.346642204097876, 40.022953418667065, 35.94648303373688, 35.7364749685521, 39.479488162280596, 32.9863470466753, 25.44013703364339, 22.319678873017494, 21.632974546568068, 29.800681131145613, 27.028044172168716, 43.61454191906839, 35.11378365112735, 32.72964525738904, 39.86691871057298, 43.5294404008871, 39.172256347947744, 47.807737558209425, 43.699809813576366, 47.06654728734682, 37.45150421083523, 41.05217218133816, 34.63697922255266, 34.63697922255266, 59.265731184583274, 43.10641711474324, 38.94340253332466, 36.942956250241515, 40.65322322588273, 52.609179777419676, 37.5980852538711, 38.48969815122384, 30.034410770075546, 43.5294404008871, 46.88305220915621, 44.21492934326177, 50.396507397123145, 33.90076227087091, 50.6926662381911, 44.04255166984753, 44.82358155812517, 55.6750020519895, 24.18050850623376, 51.79355004464666, 33.4404292647954, 33.4404292647954, 40.57389988868238, 42.27266353442275, 49.9067492938035, 52.19977189231385, 50.20003003478527, 33.11545195869231, 39.172256347947744, 47.435694792893365, 40.25815129472595, 40.1011996370117, 40.57389988868238, 41.617286632367644, 34.367432111764096, 29.281418237700255, 38.26483200699776, 29.33866439394716, 36.72712654703974, 21.214554942517648, 33.375179667439795, 41.05217218133816, 28.491601937740562, 31.72265993608251, 26.713157919740368, 24.657426302639607, 28.60311491479713, 27.240027749911206, 26.09434828302453, 25.39049775763493, 28.547303976590324, 27.668995768810042, 27.293282919182133, 30.56702687878971, 30.26997357328364, 33.702705802144536, 34.63697922255266, 26.975306541751994, 32.66578255521719, 26.975306541751994, 33.05083646319603, 45.44061233480379, 24.513371642677637, 21.632974546568068, 30.68666283939559, 50.5937537501147, 31.230845679407448, 47.807737558209425, 45.529450258314064, 44.47476167904283, 41.698649825700315, 45.88654216244616, 43.95661501095434, 43.699809813576366, 41.617286632367644, 42.190180314697045, 33.76859572409541, 27.72308958496738, 50.1020787877761, 35.32013242426123, 35.25121511458854, 40.1011996370117, 21.632974546568068, 39.095822701298815, 39.095822701298815, 24.80232751057381, 26.609013186724894, 34.23344619169951, 25.992616060150944, 32.159346537604755, 39.94485987582193, 29.56877039069624, 29.56877039069624, 69.01852867010916, 33.83461446305701, 33.96703939987107, 41.943695056893915, 39.479488162280596, 44.99901609609206, 39.479488162280596, 39.78912962559818, 41.617286632367644, 45.61846186294896, 39.86691871057298, 37.305494633128845, 36.58394105020859, 48.56059987746486, 47.06654728734682, 51.39048944047805, 45.086990684122, 41.943695056893915, 50.89107172411317, 39.32557222442699, 40.1011996370117, 55.6750020519895, 53.02179868836547, 39.556671887793456, 41.53608219643496, 47.62135285300038, 36.08717359043372, 50.89107172411317, 48.46584751755162, 27.940524471445343, 51.1901369093998, 33.375179667439795, 34.908640417532865, 34.908640417532865, 44.64883097460398, 50.6926662381911, 40.1011996370117, 46.97471015107385, 32.22221914035047, 33.31005738633129, 54.27917440928786, 43.87084603329354, 30.21091022897123, 40.97207040739876, 44.47476167904283, 55.02636863114642, 34.09998263323346, 32.60204446315635, 41.617286632367644, 46.066137010589394, 49.133016862271724, 39.634006510082365, 50.49503426195374, 30.866995306309306, 30.806767041912167, 48.46584751755162, 48.182698291098816, 38.115651564714454, 42.438114061533675, 46.15619785585283, 40.49473132867618, 46.70027251047952, 38.867415430562446, 27.028044172168716, 36.798929313140206, 41.05217218133816, 37.5980852538711, 42.02569638966092, 38.48969815122384, 46.60915005647103, 36.87087245606033, 31.10908815096766, 31.41437607743883, 50.89107172411317, 30.62678644311483, 47.807737558209425, 50.990565478815945, 32.60204446315635, 41.861853726676614, 56.55175575986736, 51.591626129191596, 55.89290784893326, 55.457945789238345, 37.74523999888851, 43.61454191906839, 42.438114061533675, 50.29817277977263, 64.83690740886694, 49.32531805312982, 53.85677052285177, 42.93836112094585, 52.097918711212095, 37.08754689283638, 39.7114923241556, 42.27266353442275, 47.34313736903344, 42.190180314697045, 64.33234259239626, 66.11570082213345, 42.438114061533675, 42.77096031537661, 44.91121316574036, 37.89297069129353, 34.97688798216538, 44.91121316574036, 46.15619785585283, 30.329152388418176, 48.08868329990042, 33.702705802144536, 37.524723159601, 46.246434773093995, 46.60915005647103, 53.96206231832688, 50.49503426195374, 53.85677052285177, 32.92198346312245, 45.88654216244616, 27.40010580820802, 77.59961765106212, 47.06654728734682, 28.940279382223952, 29.56877039069624, 55.457945789238345, 35.04526897307925, 31.230845679407448, 37.232703447205715, 37.5980852538711, 46.427438201089, 42.10785803778799, 57.89284200037607, 52.50652775191775, 48.75066069158903, 43.61454191906839, 42.93836112094585, 46.88305220915621, 30.447857334592324, 51.591626129191596, 50.49503426195374, 28.940279382223952, 36.228414794400415, 36.942956250241515, 48.75066069158903, 35.11378365112735, 53.333385249853684, 49.61518196998445, 43.87084603329354, 37.74523999888851, 33.83461446305701, 54.067559962798995, 57.442316329639176, 32.033969127372174, 41.780172086809024, 48.27689708476307, 57.66713919909174, 59.265731184583274, 40.73270164287187, 58.34690118774256, 57.89284200037607, 48.65553748142455, 43.02230705927991, 29.74253347911242, 28.214717811314713, 43.3597351955915, 42.854578979512624, 51.39048944047805, 80.06289393593435, 39.86691871057298, 39.095822701298815, 34.09998263323346, 42.10785803778799, 47.52843316962603, 36.942956250241515, 47.15856396830611, 43.95661501095434, 48.182698291098816, 54.811841150452466, 57.66713919909174, 43.78524440968234, 59.49769069799599, 66.89505178172818, 46.427438201089, 50.89107172411317, 48.27689708476307, 50.004318665090636, 32.348333340301785, 40.25815129472595, 39.78912962559818, 42.77096031537661, 47.71445419747699, 42.27266353442275, 37.45150421083523, 45.17513726542669, 32.348333340301785, 43.3597351955915, 54.811841150452466, 47.06654728734682, 40.81233544283598, 41.05217218133816, 54.27917440928786, 38.115651564714454, 41.37414835829223, 58.34690118774256, 65.09067188950593, 61.14702744030778, 37.232703447205715, 39.78912962559818, 33.05083646319603, 22.75989509352673, 26.14536361045317, 40.33685733281881, 54.811841150452466, 45.97625189374871, 56.11166650505244, 39.78912962559818, 48.37128004023317, 31.846818916590372, 38.56494680401705, 59.03467599561125, 57.66713919909174, 30.866995306309306, 37.67159077349981, 26.24769367033534, 59.964336869699544, 40.65322322588273, 28.996858600549206, 36.15772522700737, 34.16664924473404, 45.17513726542669, 43.699809813576366, 49.22907356066477, 27.240027749911206, 52.40407602298735, 43.87084603329354, 49.133016862271724, 33.50580642730562, 41.617286632367644, 36.798929313140206, 61.14702744030778, 37.9670527218579, 54.173263858709824, 52.301824199806006, 52.50652775191775, 26.713157919740368, 42.10785803778799, 52.609179777419676, 39.32557222442699, 58.34690118774256, 44.21492934326177, 44.91121316574036, 47.71445419747699, 42.93836112094585, 25.790339917193062, 27.940524471445343, 43.275130861102994, 43.5294404008871, 51.49095957359226, 42.190180314697045, 35.66674528344124, 50.004318665090636, 43.444504934395866, 38.41459632502361, 71.20941195067294, 41.132430556936306, 43.95661501095434, 62.84208548133934, 37.232703447205715, 45.61846186294896, 54.27917440928786, 42.10785803778799, 31.10908815096766, 49.22907356066477, 54.27917440928786, 46.60915005647103, 47.250760544967605, 48.08868329990042, 46.7915731119465, 43.5294404008871, 42.77096031537661, 50.396507397123145, 44.387981683814154, 35.7364749685521, 41.132430556936306, 32.92198346312245, 26.713157919740368, 30.987805310145493, 31.660762082694095, 31.598985005566618, 23.620367629992042, 30.21091022897123, 35.18243227767251, 34.56939489431364, 31.72265993608251, 45.61846186294896, 44.04255166984753, 38.56494680401705, 29.800681131145613, 30.56702687878971, 34.03344610288473, 42.93836112094585, 28.65903496526287, 29.396022468352484, 34.63697922255266, 36.942956250241515, 53.646802873050504, 25.790339917193062, 22.67116267165818, 37.305494633128845, 29.167260806600495, 28.15966472087403, 37.8190332109868, 34.4346215959791, 40.65322322588273, 51.39048944047805, 27.72308958496738, 31.41437607743883, 30.034410770075546, 43.61454191906839, 61.38635013907805, 45.61846186294896, 49.037147591523336, 53.542126217761066, 39.556671887793456, 25.94189877662824, 61.867809250367884, 43.87084603329354, 53.85677052285177, 55.6750020519895, 44.47476167904283, 45.529450258314064, 27.240027749911206, 40.17959882934157, 46.7915731119465, 42.10785803778799, 47.15856396830611, 32.22221914035047, 43.61454191906839, 41.05217218133816, 44.21492934326177, 34.09998263323346, 43.3597351955915, 52.097918711212095, 61.14702744030778, 57.89284200037607, 39.095822701298815, 40.022953418667065, 36.15772522700737, 45.086990684122, 29.33866439394716, 27.668995768810042, 29.975807060355134, 35.806340977539655, 32.79363281328884, 43.02230705927991, 27.186876493132687, 21.548635571548648, 35.87634357692165, 37.8190332109868, 30.866995306309306, 43.87084603329354, 29.22428378123494, 31.598985005566618, 30.447857334592324, 27.028044172168716, 41.37414835829223, 61.38635013907805, 32.79363281328884, 32.60204446315635, 40.25815129472595, 37.8190332109868, 44.64883097460398, 57.89284200037607, 80.69083499570341, 62.10995300937982, 57.66713919909174, 44.64883097460398, 52.19977189231385, 53.43765380953904, 55.89290784893326, 33.702705802144536, 44.73612093895834, 49.71218131735948, 48.65553748142455, 34.4346215959791, 50.990565478815945, 44.30137101535166, 41.698649825700315, 52.81508628524973, 46.97471015107385, 48.84596987082467, 51.39048944047805, 51.692489491288626, 33.57131140436464, 36.87087245606033, 29.626578330054702, 48.46584751755162, 56.773093073800744, 54.598150033144236, 46.70027251047952, 51.290215346585704, 41.861853726676614, 40.41571724385996, 48.75066069158903, 54.811841150452466, 49.61518196998445, 54.598150033144236, 50.20003003478527, 53.85677052285177, 45.61846186294896, 32.348333340301785, 36.15772522700737, 61.38635013907805, 61.626609520407456, 51.39048944047805, 35.32013242426123, 41.05217218133816, 52.9183415530491, 42.52108200006278, 42.93836112094585, 52.609179777419676, 44.5617113320772, 69.01852867010916, 51.1901369093998, 45.97625189374871, 38.33964103892552, 55.02636863114642, 52.40407602298735, 63.3349620664254, 41.698649825700315, 48.84596987082467, 68.21443845541918, 28.65903496526287, 65.60118435636834, 40.65322322588273, 39.248839424943355, 31.660762082694095, 33.05083646319603, 20.723120306578686, 28.1047190510635, 29.453492679719986, 37.45150421083523, 38.94340253332466, 42.438114061533675, 38.0412795852805, 24.18050850623376, 31.10908815096766, 31.048387510438282, 36.016759615546256, 33.83461446305701, 42.52108200006278, 35.45837151398557, 45.88654216244616, 45.086990684122, 51.09025374715126, 34.908640417532865, 64.33234259239626, 48.27689708476307, 50.004318665090636, 46.60915005647103, 45.529450258314064, 38.94340253332466, 33.31005738633129, 30.388446900124176, 44.04255166984753, 48.65553748142455, 41.698649825700315, 47.250760544967605, 60.199030648454944, 43.61454191906839, 44.82358155812517, 48.84596987082467, 48.46584751755162, 41.780172086809024, 44.30137101535166, 48.37128004023317, 51.692489491288626, 40.022953418667065, 53.22932014095188, 48.94146538270724, 43.444504934395866, 55.89290784893326, 54.173263858709824, 33.4404292647954, 39.78912962559818, 30.62678644311483, 38.867415430562446, 35.66674528344124, 50.5937537501147, 50.396507397123145, 45.61846186294896, 42.93836112094585, 28.547303976590324, 42.27266353442275, 46.51820540231633, 51.1901369093998, 27.133828946090556, 43.19069160819034, 29.800681131145613, 40.33685733281881, 44.5617113320772, 30.093129052017023, 43.87084603329354, 49.22907356066477, 40.25815129472595, 31.90908051733889, 57.21836996148928, 47.06654728734682, 64.08153455750669, 60.908637772317434, 34.77254452624438, 56.773093073800744, 44.21492934326177, 59.03467599561125, 33.05083646319603, 40.57389988868238, 58.80452160545304, 51.894808174781225, 37.232703447205715, 42.854578979512624, 60.67117749756472, 69.01852867010916, 54.811841150452466, 53.75168417471723, 36.51255777326842, 42.60421214369869, 48.65553748142455, 45.97625189374871, 64.33234259239626, 49.51837188956857, 29.684499285876434, 39.172256347947744, 28.049880592281973, 31.9714638416083, 35.94648303373688, 48.56059987746486, 42.438114061533675, 50.6926662381911, 38.33964103892552, 26.76538304079522, 24.086237637543274, 27.886006392229614, 47.807737558209425, 34.09998263323346, 41.29341833835342, 47.71445419747699, 37.524723159601, 44.387981683814154, 70.10541234668786, 47.250760544967605, 41.780172086809024, 60.434642992688794, 37.232703447205715, 32.47494113727571, 48.27689708476307, 38.26483200699776, 55.89290784893326, 54.173263858709824, 47.435694792893365, 52.71203249107963, 44.21492934326177, 51.79355004464666, 51.591626129191596, 40.57389988868238, 37.74523999888851, 43.3597351955915, 54.067559962798995, 40.022953418667065, 53.96206231832688, 54.27917440928786, 63.3349620664254, 54.811841150452466, 46.97471015107385, 43.19069160819034, 42.854578979512624, 55.6750020519895, 21.33922385967176, 32.033969127372174, 30.329152388418176, 39.7114923241556, 36.15772522700737, 33.57131140436464, 35.806340977539655, 31.35307975333966, 32.79363281328884, 40.022953418667065, 60.908637772317434, 50.20003003478527, 50.49503426195374, 46.97471015107385, 51.09025374715126, 41.45503620813183, 69.55984769951485, 36.72712654703974, 40.892124929554, 32.033969127372174, 28.60311491479713, 42.190180314697045, 39.86691871057298, 42.93836112094585, 53.542126217761066, 49.32531805312982, 47.06654728734682, 50.396507397123145, 58.57526450222688, 57.21836996148928, 46.066137010589394, 41.861853726676614, 37.9670527218579, 42.93836112094585, 49.22907356066477, 51.692489491288626, 39.32557222442699, 39.32557222442699, 38.19016894386643, 37.5980852538711, 41.2128458403547, 48.37128004023317, 47.250760544967605, 66.63425269055388, 50.1020787877761, 37.9670527218579, 36.299242562272255, 34.77254452624438, 68.7494509271083, 49.22907356066477, 36.44131378072682, 39.01953819342525, 47.62135285300038, 36.44131378072682, 56.773093073800744, 35.7364749685521, 45.263456176258764, 59.964336869699544, 23.94552000018515, 42.52108200006278, 30.987805310145493, 49.51837188956857, 48.84596987082467, 63.3349620664254, 47.71445419747699, 32.92198346312245, 37.232703447205715, 46.246434773093995, 37.89297069129353, 32.92198346312245, 52.81508628524973, 53.96206231832688, 76.99573263552006, 50.89107172411317, 51.49095957359226, 51.39048944047805, 39.556671887793456, 47.99485175252902, 61.14702744030778, 51.09025374715126, 44.73612093895834, 44.5617113320772, 47.71445419747699, 29.858942463791767, 47.250760544967605, 46.246434773093995, 56.11166650505244, 50.004318665090636, 63.08804244970428, 50.990565478815945, 48.37128004023317, 37.9670527218579, 44.04255166984753, 50.89107172411317, 49.133016862271724, 39.556671887793456, 32.159346537604755, 43.19069160819034, 24.18050850623376, 29.167260806600495, 44.47476167904283, 25.689792910615846, 53.333385249853684, 41.2128458403547, 30.866995306309306, 44.73612093895834, 28.771203256660744, 27.83159468972004, 31.230845679407448, 45.263456176258764, 42.10785803778799, 47.807737558209425, 42.77096031537661, 39.248839424943355, 59.49769069799599, 50.6926662381911, 47.34313736903344, 56.9952966774873, 36.942956250241515, 42.27266353442275, 41.37414835829223, 55.24173574865985, 50.20003003478527, 76.99573263552006, 66.37447035867858, 51.692489491288626, 49.51837188956857, 44.12865633779556, 54.811841150452466, 45.263456176258764, 32.159346537604755, 51.99626426796157, 31.230845679407448, 42.27266353442275, 54.49161709130268, 48.46584751755162, 55.457945789238345, 51.09025374715126, 51.99626426796157, 57.89284200037607, 66.11570082213345, 69.55984769951485, 44.30137101535166, 46.7915731119465, 31.10908815096766, 52.71203249107963, 48.37128004023317, 23.20879380099744, 31.41437607743883, 43.444504934395866, 52.097918711212095, 27.995149135337257, 26.35042423874788, 44.387981683814154, 44.387981683814154, 41.943695056893915, 39.01953819342525, 53.22932014095188, 46.51820540231633, 35.87634357692165, 41.05217218133816, 39.78912962559818, 39.78912962559818, 39.78912962559818, 40.81233544283598, 48.75066069158903, 47.06654728734682, 29.975807060355134, 43.275130861102994, 51.692489491288626, 40.17959882934157, 39.095822701298815, 48.56059987746486, 55.89290784893326, 37.01518097066168, 49.42175070681059, 24.039240115615065, 73.46984133199462, 55.24173574865985, 52.19977189231385, 57.21836996148928, 52.301824199806006, 48.37128004023317, 47.52843316962603, 36.08717359043372, 50.79177210350426, 51.290215346585704, 43.444504934395866, 22.582776184718522, 40.73270164287187, 47.71445419747699, 57.89284200037607, 38.26483200699776, 40.57389988868238, 53.96206231832688, 52.097918711212095, 48.08868329990042, 40.1011996370117, 55.6750020519895, 25.840760898054555, 25.045724266551254, 43.5294404008871, 50.396507397123145, 50.79177210350426, 33.05083646319603, 42.02569638966092, 24.80232751057381, 29.281418237700255, 52.81508628524973, 45.529450258314064, 37.74523999888851, 31.660762082694095, 46.066137010589394, 37.74523999888851, 42.10785803778799, 33.375179667439795, 52.81508628524973, 43.444504934395866, 23.390822626823073, 63.582848099201826, 40.41571724385996, 34.4346215959791, 48.182698291098816, 18.431524745974645, 37.9670527218579, 35.18243227767251, 34.23344619169951, 38.71588573814723, 30.74665629604239, 28.214717811314713, 29.975807060355134, 34.23344619169951, 25.192907336481866, 60.908637772317434, 28.436008585761353, 36.51255777326842, 29.858942463791767, 24.657426302639607, 50.6926662381911, 34.70469568042362, 23.992334296139543, 35.806340977539655, 39.40245503911115, 48.56059987746486, 36.228414794400415, 38.64034257045414, 56.9952966774873, 50.1020787877761, 33.96703939987107, 34.84052601883783, 51.692489491288626, 24.465540688808968, 21.760102337787384, 32.09659661306934, 27.668995768810042, 29.396022468352484, 33.1801937796528, 31.475792237910618, 36.15772522700737, 30.26997357328364, 32.411575418595035, 36.228414794400415, 27.028044172168716, 59.03467599561125, 56.773093073800744, 49.71218131735948, 53.02179868836547, 36.51255777326842, 46.97471015107385, 52.81508628524973, 31.35307975333966, 45.351947753528236, 26.817710263716034, 43.02230705927991, 39.40245503911115, 32.66578255521719, 40.41571724385996, 26.87013978811538, 30.68666283939559, 28.1047190510635, 37.08754689283638, 19.543763832601815, 50.89107172411317, 53.12545808585662, 46.97471015107385, 47.06654728734682, 38.48969815122384, 36.87087245606033, 30.447857334592324, 38.19016894386643, 26.453556883236853, 30.447857334592324, 43.95661501095434, 29.33866439394716, 34.09998263323346, 46.70027251047952, 35.18243227767251, 51.1901369093998, 40.65322322588273, 34.77254452624438, 38.19016894386643, 30.806767041912167, 46.88305220915621, 32.66578255521719, 44.91121316574036, 36.51255777326842, 30.56702687878971, 42.52108200006278, 44.387981683814154, 39.095822701298815, 48.56059987746486, 33.90076227087091, 32.92198346312245, 28.1047190510635, 57.21836996148928, 48.56059987746486, 33.11545195869231, 34.908640417532865, 27.995149135337257, 29.626578330054702, 28.436008585761353, 27.133828946090556, 26.35042423874788, 36.51255777326842, 42.27266353442275, 28.380523708580828, 29.975807060355134, 43.5294404008871, 42.190180314697045, 47.71445419747699, 32.60204446315635, 31.598985005566618, 27.4536739354601, 44.73612093895834, 45.17513726542669, 32.09659661306934, 32.033969127372174, 21.297586362924953, 31.72265993608251, 42.02569638966092, 24.322607071104724, 53.646802873050504, 51.39048944047805, 32.79363281328884, 50.004318665090636, 39.634006510082365, 56.9952966774873, 59.49769069799599, 45.97625189374871, 48.65553748142455, 51.894808174781225, 46.15619785585283, 47.250760544967605, 39.40245503911115, 52.9183415530491, 39.479488162280596, 36.15772522700737, 25.24216031657971, 28.883810562338418, 25.2915095879081, 36.299242562272255, 40.25815129472595, 40.17959882934157, 32.22221914035047, 24.322607071104724, 27.40010580820802, 40.81233544283598, 46.7915731119465, 43.699809813576366, 46.33684810653952, 38.71588573814723, 55.24173574865985, 59.964336869699544, 46.88305220915621, 58.80452160545304, 47.435694792893365, 38.0412795852805, 41.698649825700315, 74.91890411002962, 62.10995300937982, 36.016759615546256, 63.08804244970428, 56.33128135834348, 35.25121511458854, 39.7114923241556, 30.507383918455513, 29.74253347911242, 45.7076474882609, 31.598985005566618, 38.19016894386643, 30.21091022897123, 32.66578255521719, 30.68666283939559, 28.214717811314713, 54.598150033144236, 40.49473132867618, 31.784678801853513, 45.88654216244616, 36.37020880080945, 24.513371642677637, 30.987805310145493, 28.771203256660744, 37.45150421083523, 52.19977189231385, 39.32557222442699, 53.22932014095188, 43.275130861102994, 33.702705802144536, 36.44131378072682, 30.447857334592324, 55.24173574865985, 44.387981683814154, 52.71203249107963, 49.9067492938035, 42.77096031537661, 57.89284200037607, 33.90076227087091, 66.63425269055388, 38.41459632502361, 60.67117749756472, 56.33128135834348, 63.08804244970428, 37.67159077349981, 37.37842812826559, 29.9173176993003, 56.9952966774873, 34.97688798216538, 29.110349096271467, 57.21836996148928, 31.598985005566618, 44.73612093895834, 43.87084603329354, 53.12545808585662, 21.973644325590612, 40.25815129472595, 48.94146538270724, 39.7114923241556, 32.60204446315635, 56.773093073800744, 33.11545195869231, 42.60421214369869, 26.401940203271806, 50.29817277977263, 67.41971617548266, 48.08868329990042, 50.6926662381911, 28.715064341305684, 62.3530444922685, 41.698649825700315, 57.21836996148928, 32.9863470466753, 25.63966650134361, 51.09025374715126, 45.086990684122, 28.214717811314713, 53.12545808585662, 48.84596987082467, 55.89290784893326, 58.80452160545304, 36.37020880080945, 42.438114061533675, 37.9670527218579, 56.33128135834348, 58.34690118774256, 63.582848099201826, 65.8579401324037, 47.435694792893365, 46.51820540231633, 47.34313736903344, 43.699809813576366, 51.99626426796157, 53.96206231832688, 48.94146538270724, 31.660762082694095, 31.9714638416083, 44.73612093895834, 58.34690118774256, 29.396022468352484, 28.996858600549206, 38.48969815122384, 45.97625189374871, 30.866995306309306, 47.71445419747699, 64.58413226217957, 48.46584751755162, 43.95661501095434, 52.50652775191775, 39.86691871057298, 31.660762082694095, 52.50652775191775, 49.61518196998445, 28.715064341305684, 26.922671813996086, 30.62678644311483, 27.507346790200362, 37.08754689283638, 34.908640417532865, 39.634006510082365, 47.99485175252902, 26.043432497666576, 20.44172438427552, 31.784678801853513, 41.132430556936306, 33.11545195869231, 44.21492934326177, 50.5937537501147, 49.809370301716946, 42.52108200006278, 39.7114923241556, 48.46584751755162, 45.7076474882609, 45.529450258314064, 42.438114061533675, 39.01953819342525, 35.94648303373688, 42.60421214369869, 91.79256012938836, 41.780172086809024, 30.866995306309306, 33.83461446305701, 47.15856396830611, 55.89290784893326, 29.684499285876434, 28.436008585761353, 24.275148340672132, 33.50580642730562, 29.396022468352484, 33.96703939987107, 53.02179868836547, 44.73612093895834, 35.38918446958933, 27.028044172168716, 58.80452160545304, 45.88654216244616, 42.10785803778799, 41.2128458403547, 25.790339917193062, 49.42175070681059, 33.245062173048, 51.894808174781225, 34.908640417532865, 43.5294404008871, 45.086990684122, 24.22778221261098, 36.51255777326842, 28.380523708580828, 21.42274322042474, 53.22932014095188, 31.72265993608251, 34.70469568042362, 54.598150033144236, 44.21492934326177, 39.40245503911115, 28.65903496526287, 42.02569638966092, 34.56939489431364, 35.18243227767251, 30.26997357328364, 46.51820540231633, 51.290215346585704, 40.17959882934157, 26.505274475545082, 34.97688798216538, 33.375179667439795, 29.053548433146837, 34.300373728940066, 44.82358155812517, 34.300373728940066, 25.539706914745466, 55.24173574865985, 45.61846186294896, 23.16350836406023, 62.10995300937982, 34.97688798216538, 30.093129052017023, 46.60915005647103, 37.524723159601, 24.086237637543274, 50.396507397123145, 32.22221914035047, 26.922671813996086, 27.507346790200362, 25.2915095879081, 35.52769382137767, 46.427438201089, 40.73270164287187, 32.28521466114651, 51.99626426796157, 44.47476167904283, 50.79177210350426, 47.250760544967605, 55.6750020519895, 45.086990684122, 57.89284200037607, 52.71203249107963, 55.89290784893326, 29.74253347911242, 49.71218131735948, 42.02569638966092, 52.301824199806006, 53.333385249853684, 38.94340253332466, 36.15772522700737, 52.50652775191775, 51.1901369093998, 50.1020787877761, 52.609179777419676, 44.5617113320772, 49.22907356066477, 57.21836996148928, 47.62135285300038, 37.305494633128845, 24.89940107578365, 39.634006510082365, 31.784678801853513, 43.275130861102994, 44.387981683814154, 30.329152388418176, 33.96703939987107, 44.47476167904283, 46.33684810653952, 55.02636863114642, 49.9067492938035, 51.1901369093998, 43.02230705927991, 42.10785803778799, 59.73055807527518, 43.87084603329354, 57.66713919909174, 54.598150033144236, 42.52108200006278, 50.20003003478527, 50.49503426195374, 79.43983955226133, 78.2082389954826, 38.115651564714454, 37.01518097066168, 67.41971617548266, 38.33964103892552, 47.71445419747699, 26.922671813996086, 66.37447035867858, 49.037147591523336, 103.20514949397284, 75.21212841006654, 118.3252868447224, 46.33684810653952, 79.75075829332496, 47.71445419747699, 64.08153455750669, 99.2514223056366, 102.40200064874995, 43.699809813576366, 108.5813867407896, 65.8579401324037, 53.85677052285177, 48.08868329990042, 48.75066069158903, 78.2082389954826, 53.333385249853684, 62.84208548133934, 56.33128135834348, 33.96703939987107, 59.03467599561125, 61.867809250367884, 62.10995300937982, 92.51249810131188, 35.04526897307925, 51.692489491288626, 72.89809449073893, 51.591626129191596, 54.067559962798995, 59.964336869699544, 40.17959882934157, 42.02569638966092, 88.62154503481175, 76.09870517958821, 64.58413226217957, 78.51433738825796, 98.09510821132314, 74.33588057277046, 59.265731184583274, 51.39048944047805, 39.94485987582193, 47.99485175252902, 102.80279074731456, 61.867809250367884, 50.1020787877761, 34.501942437892716, 69.55984769951485, 68.48142221881122, 59.49769069799599, 65.09067188950593, 73.18340956555208, 51.99626426796157, 74.91890411002962, 90.7231444115574, 82.92766733936132, 35.04526897307925, 35.32013242426123, 36.016759615546256, 75.50650035756513, 100.42136666458902, 87.24759532868651, 118.78839892336089, 48.65553748142455, 53.333385249853684, 58.34690118774256, 58.119428177447986, 44.47476167904283, 107.7363996853844, 63.83170433047907, 56.773093073800744, 96.57428452170761, 88.27604237373367, 72.89809449073893, 54.27917440928786, 69.28865955361954, 61.38635013907805, 54.811841150452466, 47.807737558209425, 60.67117749756472, 49.809370301716946, 48.08868329990042, 50.396507397123145, 98.4790416094955, 107.7363996853844, 84.563269020706, 109.86130976812342, 28.325147094540903, 100.42136666458902, 110.7229626998625, 57.66713919909174, 38.867415430562446, 97.33172602285453, 74.91890411002962, 26.817710263716034, 71.20941195067294, 80.37625124290746, 52.301824199806006, 66.11570082213345, 68.7494509271083, 80.06289393593435, 93.6030061542134, 83.90519244027122, 59.265731184583274, 118.3252868447224, 72.33079701870139, 92.51249810131188, 84.563269020706, 94.33714363029665, 110.7229626998625, 74.91890411002962, 49.809370301716946, 118.78839892336089, 42.68750480955783, 87.93188669987823, 63.3349620664254, 92.1518260518184, 69.01852867010916, 83.90519244027122, 73.75739416067401, 73.75739416067401, 80.69083499570341, 80.37625124290746, 91.43469485205186, 75.80202444429054, 63.582848099201826, 84.563269020706, 76.39654709045266, 51.79355004464666, 104.83039411284926, 104.83039411284926, 85.2265069597304, 70.65525591042105, 74.33588057277046, 73.18340956555208, 63.3349620664254, 76.39654709045266, 66.37447035867858, 79.75075829332496, 55.24173574865985, 58.80452160545304, 57.89284200037607, 92.1518260518184, 45.263456176258764, 59.265731184583274, 82.92766733936132, 97.71267162761835, 106.06610162605153, 53.22932014095188, 47.06654728734682, 96.57428452170761, 59.03467599561125, 74.62682298320091, 66.63425269055388, 94.33714363029665, 107.7363996853844, 88.62154503481175, 89.66618776422116, 65.60118435636834, 47.807737558209425, 71.20941195067294, 96.95226558425567, 85.2265069597304, 44.21492934326177, 86.56862915519338, 64.58413226217957, 70.65525591042105, 51.49095957359226, 71.48811770889668, 70.10541234668786, 90.36944839098756, 90.36944839098756, 69.28865955361954, 64.33234259239626, 99.63988138921447, 69.83209724580306, 72.61389175398708, 71.20941195067294, 61.38635013907805, 61.38635013907805, 44.04255166984753, 96.57428452170761, 67.68358948381629, 74.04607243930377, 69.28865955361954, 83.90519244027122, 50.20003003478527, 64.08153455750669, 76.09870517958821, 81.9615307508321, 32.72964525738904, 76.09870517958821, 90.36944839098756, 55.89290784893326, 76.99573263552006, 62.10995300937982, 72.89809449073893, 67.68358948381629, 57.89284200037607, 114.68480048885151, 105.65258908406659, 79.13013296849392, 67.9484955630781, 85.89494673838159, 100.02986085865793, 48.46584751755162, 57.21836996148928, 42.190180314697045, 68.7494509271083, 73.46984133199462, 55.6750020519895, 81.9615307508321, 86.23113005463887, 73.18340956555208, 58.119428177447986, 97.33172602285453, 40.33685733281881, 69.83209724580306, 89.66618776422116, 70.65525591042105, 74.33588057277046, 69.83209724580306, 94.33714363029665, 40.41571724385996, 61.626609520407456, 54.49161709130268, 62.10995300937982, 91.79256012938836, 47.250760544967605, 47.34313736903344, 47.52843316962603, 70.37979717263202, 36.58394105020859, 53.333385249853684, 75.80202444429054, 78.82163381626928, 88.27604237373367, 67.41971617548266, 78.2082389954826, 68.21443845541918, 36.016759615546256, 93.6030061542134, 72.89809449073893, 62.3530444922685, 74.62682298320091, 76.99573263552006, 92.87458178129451, 54.49161709130268, 67.68358948381629, 67.41971617548266, 63.582848099201826, 48.65553748142455, 82.92766733936132, 70.65525591042105, 85.2265069597304, 93.2380826167318, 67.68358948381629, 70.37979717263202, 64.08153455750669, 72.04880596519355, 76.39654709045266, 75.50650035756513, 56.33128135834348, 59.03467599561125, 55.6750020519895, 93.2380826167318, 86.90744918987926, 75.80202444429054, 126.94450214932196, 70.93179276322707, 50.89107172411317, 32.159346537604755, 57.21836996148928, 61.626609520407456, 89.31661242710538, 84.2335880783952, 116.49082330627213, 57.442316329639176, 81.9615307508321, 81.00664999449546, 30.26997357328364, 69.83209724580306, 70.65525591042105, 54.49161709130268, 108.15806803146498, 80.06289393593435, 74.62682298320091, 128.94358717585138, 131.48677155266952, 95.82273747708687, 66.63425269055388, 68.7494509271083, 70.65525591042105, 58.119428177447986, 59.265731184583274, 99.2514223056366, 67.9484955630781, 91.43469485205186, 91.43469485205186, 79.75075829332496, 92.51249810131188, 79.13013296849392, 49.32531805312982, 59.73055807527518, 56.55175575986736, 49.71218131735948, 55.24173574865985, 58.119428177447986, 58.119428177447986, 59.964336869699544, 54.067559962798995, 58.34690118774256, 53.43765380953904, 99.63988138921447, 42.02569638966092, 54.598150033144236, 63.08804244970428, 109.00636227269737, 75.21212841006654, 62.10995300937982, 69.55984769951485, 68.21443845541918, 46.15619785585283, 99.2514223056366, 88.27604237373367, 94.33714363029665, 59.03467599561125, 91.43469485205186, 69.83209724580306, 44.99901609609206, 53.75168417471723, 48.94146538270724, 40.1011996370117, 74.04607243930377, 56.33128135834348, 57.89284200037607, 105.24068867470233, 53.43765380953904, 53.43765380953904, 75.21212841006654, 66.11570082213345, 40.49473132867618, 79.75075829332496, 67.1568716116849, 86.90744918987926, 71.76791429061606, 56.773093073800744, 65.8579401324037, 41.617286632367644, 44.47476167904283, 48.56059987746486, 54.811841150452466, 59.964336869699544, 64.83690740886694, 42.02569638966092, 61.14702744030778, 63.08804244970428, 69.28865955361954, 125.9566121960733, 60.199030648454944, 70.37979717263202, 61.38635013907805, 61.626609520407456, 53.02179868836547, 81.00664999449546, 92.87458178129451, 76.09870517958821, 81.32370105824417, 70.37979717263202, 74.62682298320091, 81.9615307508321, 88.96839995507139, 100.02986085865793, 90.01713130052181, 83.25223705241488, 70.65525591042105, 57.21836996148928, 56.11166650505244, 71.76791429061606, 56.773093073800744, 54.49161709130268, 51.49095957359226, 67.41971617548266, 73.18340956555208, 87.58907276183992, 75.50650035756513, 74.33588057277046, 65.09067188950593, 33.702705802144536, 103.20514949397284, 82.28231911219243, 87.93188669987823, 71.48811770889668, 63.582848099201826, 34.84052601883783, 33.702705802144536, 25.192907336481866, 32.033969127372174, 35.806340977539655, 49.22907356066477, 22.36331459495464, 52.81508628524973, 17.587483776585056, 18.431524745974645, 37.9670527218579, 26.66103470132838, 56.9952966774873, 40.41571724385996, 29.110349096271467, 27.940524471445343, 16.74934871787156, 57.89284200037607, 51.894808174781225, 53.542126217761066, 21.973644325590612, 39.86691871057298, 33.1801937796528, 22.145985754016134, 34.16664924473404, 82.60436300370014, 32.22221914035047, 49.9067492938035, 37.232703447205715, 20.085536923187668, 38.0412795852805, 56.9952966774873, 40.892124929554, 78.51433738825796, 56.9952966774873, 42.438114061533675, 38.41459632502361, 33.57131140436464, 42.02569638966092, 67.9484955630781, 31.72265993608251, 45.7076474882609, 39.01953819342525, 36.08717359043372, 22.626926270737744, 29.22428378123494, 25.539706914745466, 22.626926270737744, 51.894808174781225, 46.066137010589394, 16.94678474038272, 27.186876493132687, 19.165753221675935, 23.4365524707797, 51.1901369093998, 34.84052601883783, 49.71218131735948, 20.203571081597193, 31.598985005566618, 25.790339917193062, 23.75917448515314, 32.47494113727571, 33.11545195869231, 45.97625189374871, 22.494734284035275, 26.817710263716034, 40.65322322588273, 23.75917448515314, 24.9968545761854, 19.165753221675935, 25.94189877662824, 17.382583384208317, 49.32531805312982, 34.56939489431364, 67.9484955630781, 33.83461446305701, 37.5980852538711, 44.387981683814154, 45.529450258314064, 33.50580642730562, 52.50652775191775, 18.39556080667227, 23.898797049169467, 27.83159468972004, 36.942956250241515, 58.34690118774256, 33.31005738633129, 53.02179868836547, 94.33714363029665, 48.94146538270724, 53.75168417471723, 30.151962130172098, 26.401940203271806, 98.4790416094955, 44.47476167904283, 57.442316329639176, 44.82358155812517, 22.10277408821932, 31.048387510438282, 28.2698785323965, 76.39654709045266, 41.861853726676614, 58.57526450222688, 52.097918711212095, 56.11166650505244, 57.442316329639176, 23.574279122424027, 53.02179868836547, 32.85774546700983, 45.88654216244616, 58.119428177447986, 33.50580642730562, 24.513371642677637, 31.72265993608251, 31.53732846903916, 22.938403163153815, 26.35042423874788, 25.045724266551254, 35.806340977539655, 57.89284200037607, 28.2698785323965, 20.322298876557472, 46.15619785585283, 69.01852867010916, 53.85677052285177, 24.133327041205547, 63.582848099201826, 34.23344619169951, 26.196478674560588, 30.034410770075546, 36.942956250241515, 44.04255166984753, 45.17513726542669, 53.85677052285177, 30.806767041912167, 85.2265069597304, 52.71203249107963, 47.34313736903344, 50.20003003478527, 59.49769069799599, 35.806340977539655, 49.42175070681059, 64.83690740886694, 68.7494509271083, 52.50652775191775, 58.34690118774256, 35.52769382137767, 59.265731184583274, 53.75168417471723, 53.12545808585662, 55.02636863114642, 66.89505178172818, 30.806767041912167, 78.82163381626928, 54.598150033144236, 46.97471015107385, 63.582848099201826, 20.68268496272443, 62.3530444922685, 45.263456176258764, 68.21443845541918, 26.2990087931472, 21.590763877931302, 48.75066069158903, 20.322298876557472, 16.265763498233948, 40.41571724385996, 59.49769069799599, 71.76791429061606, 53.12545808585662, 22.538712245181248, 72.33079701870139, 34.77254452624438, 15.98234019723552, 42.190180314697045, 64.83690740886694, 33.90076227087091, 81.32370105824417, 61.626609520407456, 65.09067188950593, 30.74665629604239, 30.74665629604239, 30.74665629604239, 86.23113005463887, 55.24173574865985, 53.646802873050504, 63.3349620664254, 29.511075247281195, 27.940524471445343, 22.538712245181248, 37.89297069129353, 24.513371642677637, 19.391673055829287, 34.300373728940066, 47.15856396830611, 48.56059987746486, 41.53608219643496, 25.489873356103875, 27.507346790200362, 22.538712245181248, 20.124804822565828, 20.401838105461145, 30.927341318986464, 31.169907463288272, 36.87087245606033, 27.080884906424416, 109.00636227269737, 66.11570082213345, 32.411575418595035, 53.646802873050504, 76.39654709045266, 106.06610162605153, 70.93179276322707, 82.60436300370014, 65.60118435636834, 51.09025374715126, 46.60915005647103, 60.199030648454944, 60.67117749756472, 47.435694792893365, 84.2335880783952, 73.18340956555208, 35.38918446958933, 39.095822701298815, 59.73055807527518, 48.84596987082467, 27.507346790200362, 35.7364749685521, 32.79363281328884, 20.92648590822237, 42.3553080116137, 65.8579401324037, 81.00664999449546, 89.66618776422116, 45.97625189374871, 64.58413226217957, 61.14702744030778, 69.28865955361954, 36.942956250241515, 39.01953819342525, 32.92198346312245, 23.118311288923124, 20.92648590822237, 33.31005738633129, 37.9670527218579, 29.453492679719986, 24.850816893992036, 47.52843316962603, 34.908640417532865, 48.56059987746486, 57.89284200037607, 30.034410770075546, 49.133016862271724, 46.246434773093995, 45.351947753528236, 40.57389988868238, 42.77096031537661, 78.82163381626928, 25.891280453627324, 84.563269020706, 36.15772522700737, 26.453556883236853, 70.93179276322707, 30.806767041912167, 65.34542957624015, 106.89798837141365, 118.78839892336089, 85.89494673838159, 26.453556883236853, 23.16350836406023, 21.717643615002626, 66.63425269055388, 50.89107172411317, 63.582848099201826, 19.774139536218833, 44.21492934326177, 57.89284200037607, 47.250760544967605, 44.04255166984753, 40.17959882934157, 60.434642992688794, 23.620367629992042, 35.11378365112735, 44.73612093895834, 45.529450258314064, 45.086990684122, 32.411575418595035, 23.482371718117374, 69.01852867010916, 102.80279074731456, 20.723120306578686, 38.115651564714454, 14.298523068373607, 70.37979717263202, 30.151962130172098, 76.39654709045266, 47.807737558209425, 18.467558995986426, 51.49095957359226, 20.4816886421048, 67.1568716116849, 31.169907463288272, 96.57428452170761, 60.67117749756472, 69.28865955361954, 35.59715165620933, 54.811841150452466, 42.3553080116137, 43.19069160819034, 47.62135285300038, 51.894808174781225, 30.68666283939559, 98.86447768050158, 76.39654709045266, 53.02179868836547, 27.293282919182133, 25.045724266551254, 33.83461446305701, 56.55175575986736, 85.89494673838159, 51.1901369093998, 62.84208548133934, 60.67117749756472, 26.66103470132838, 74.04607243930377, 31.784678801853513, 26.713157919740368, 23.852165264858517, 76.39654709045266, 56.11166650505244, 30.866995306309306, 36.15772522700737, 32.47494113727571, 39.40245503911115, 34.908640417532865, 36.08717359043372, 30.68666283939559, 39.94485987582193, 28.65903496526287, 28.65903496526287, 70.65525591042105, 37.37842812826559, 27.72308958496738, 26.922671813996086, 44.04255166984753, 46.97471015107385, 45.086990684122, 49.9067492938035, 74.33588057277046, 74.33588057277046, 22.80439145296227, 16.94678474038272, 38.867415430562446, 26.87013978811538, 34.908640417532865, 20.16414949200632, 90.7231444115574, 65.09067188950593, 54.811841150452466, 58.119428177447986, 64.08153455750669, 64.83690740886694, 65.34542957624015, 63.08804244970428, 34.501942437892716, 45.44061233480379, 41.943695056893915, 34.09998263323346, 54.598150033144236, 59.49769069799599, 24.561296107667808, 33.702705802144536, 35.806340977539655, 41.132430556936306, 47.250760544967605, 47.15856396830611, 43.87084603329354, 23.75917448515314, 23.482371718117374, 40.81233544283598, 118.78839892336089, 33.11545195869231, 67.41971617548266, 43.444504934395866, 50.20003003478527, 60.199030648454944, 67.9484955630781, 16.8149038152427, 45.97625189374871, 53.22932014095188, 26.609013186724894, 52.19977189231385, 31.784678801853513, 22.80439145296227, 35.66674528344124, 25.992616060150944, 50.004318665090636, 28.715064341305684, 37.67159077349981, 36.44131378072682, 81.00664999449546, 82.28231911219243, 71.76791429061606, 67.41971617548266, 34.70469568042362, 86.90744918987926, 56.773093073800744, 33.50580642730562, 72.61389175398708, 73.18340956555208, 103.60908302823987, 31.72265993608251, 59.49769069799599, 29.22428378123494, 36.016759615546256, 53.12545808585662, 52.81508628524973, 34.97688798216538, 52.71203249107963, 55.02636863114642, 27.940524471445343, 17.113091407702502, 65.60118435636834, 59.73055807527518, 47.62135285300038, 46.70027251047952, 83.57807709540792, 18.467558995986426, 24.70563239932894, 35.806340977539655, 54.27917440928786, 45.79700747446617, 20.521731031400577, 50.29817277977263, 46.066137010589394, 52.097918711212095, 20.92648590822237, 46.60915005647103, 37.37842812826559, 41.132430556936306, 64.58413226217957, 38.48969815122384, 51.09025374715126, 25.39049775763493, 49.037147591523336, 28.491601937740562, 29.33866439394716, 31.598985005566618, 43.95661501095434, 40.1011996370117, 32.22221914035047, 44.12865633779556, 46.51820540231633, 44.91121316574036, 47.99485175252902, 24.133327041205547, 28.771203256660744, 42.438114061533675, 23.028181534732802, 48.182698291098816, 55.6750020519895, 24.513371642677637, 22.80439145296227, 50.990565478815945, 62.84208548133934, 51.591626129191596, 37.160054292819765, 59.03467599561125, 37.8190332109868, 32.411575418595035, 89.31661242710538, 47.435694792893365, 42.10785803778799, 51.79355004464666, 50.79177210350426, 44.5617113320772, 55.6750020519895, 62.3530444922685, 33.05083646319603, 43.78524440968234, 32.22221914035047, 105.24068867470233, 42.77096031537661, 41.943695056893915, 51.99626426796157, 40.81233544283598, 24.22778221261098, 25.63966650134361, 50.49503426195374, 36.228414794400415, 20.92648590822237, 18.648789727196185, 48.75066069158903, 44.64883097460398, 42.52108200006278, 51.09025374715126, 38.115651564714454, 56.55175575986736, 21.548635571548648, 16.489674134905655, 35.7364749685521, 93.6030061542134, 76.69555472159655, 62.3530444922685, 25.790339917193062, 35.04526897307925, 22.538712245181248, 32.79363281328884, 57.21836996148928, 51.99626426796157, 55.457945789238345, 49.9067492938035, 47.435694792893365, 42.10785803778799, 37.232703447205715, 34.501942437892716, 31.230845679407448, 38.48969815122384, 29.684499285876434, 46.51820540231633, 49.037147591523336, 35.38918446958933, 51.39048944047805, 121.60538462411665, 104.4216991379013, 66.63425269055388, 61.14702744030778, 39.78912962559818, 43.275130861102994, 55.89290784893326, 42.3553080116137, 72.33079701870139, 96.19777706766972, 53.542126217761066, 42.77096031537661, 50.89107172411317, 69.01852867010916, 97.33172602285453, 37.160054292819765, 53.333385249853684, 67.1568716116849, 113.34868209247811, 71.48811770889668, 109.43300111180854, 39.01953819342525, 41.617286632367644, 75.50650035756513, 43.10641711474324, 38.115651564714454, 81.32370105824417, 31.230845679407448, 56.55175575986736, 58.80452160545304, 39.7114923241556, 95.82273747708687, 48.182698291098816, 54.38529201854992, 46.33684810653952, 38.79157659527073, 36.72712654703974, 53.96206231832688, 57.442316329639176, 51.39048944047805, 81.641993024771, 84.563269020706, 111.15632012308374, 52.9183415530491, 52.9183415530491, 38.79157659527073, 28.65903496526287, 37.5980852538711, 26.196478674560588, 58.119428177447986, 79.13013296849392, 85.89494673838159, 69.83209724580306, 61.867809250367884, 61.867809250367884, 61.867809250367884, 52.609179777419676, 60.434642992688794, 72.61389175398708, 68.48142221881122, 74.04607243930377, 44.5617113320772, 62.10995300937982, 91.07822475921115, 93.2380826167318, 84.563269020706, 73.46984133199462, 59.49769069799599, 93.6030061542134, 76.39654709045266, 76.39654709045266, 75.21212841006654, 106.48123261036584, 48.65553748142455, 45.351947753528236, 70.65525591042105, 23.028181534732802, 78.51433738825796, 61.14702744030778, 101.20898120494374, 70.37979717263202, 46.97471015107385, 59.964336869699544, 40.41571724385996, 41.53608219643496, 66.37447035867858, 59.265731184583274, 80.37625124290746, 45.17513726542669, 39.94485987582193, 48.94146538270724, 30.151962130172098, 40.81233544283598, 41.45503620813183, 65.60118435636834, 48.65553748142455, 61.38635013907805, 79.75075829332496, 33.63694444585419, 75.21212841006654, 94.33714363029665, 77.90333396724647, 56.55175575986736, 72.04880596519355, 53.542126217761066, 67.9484955630781, 70.65525591042105, 63.08804244970428, 54.598150033144236, 60.434642992688794, 60.67117749756472, 60.67117749756472, 38.867415430562446, 84.89424029774196, 92.51249810131188, 95.82273747708687, 116.03666862607915, 42.27266353442275, 44.21492934326177, 64.33234259239626, 56.33128135834348, 59.265731184583274, 44.73612093895834, 86.56862915519338, 67.68358948381629, 57.442316329639176, 64.83690740886694, 39.248839424943355, 47.90120329104557, 46.70027251047952, 33.4404292647954, 41.2128458403547, 53.85677052285177, 63.3349620664254, 55.6750020519895, 109.00636227269737, 73.75739416067401, 73.75739416067401, 66.89505178172818, 102.00277308269969, 58.57526450222688, 50.396507397123145, 80.69083499570341, 63.582848099201826, 77.29708541258049, 44.47476167904283, 49.71218131735948, 75.50650035756513, 55.89290784893326, 95.0770390179677, 74.91890411002962, 56.33128135834348, 62.597087408320306, 57.66713919909174, 59.73055807527518, 44.91121316574036, 45.61846186294896, 60.199030648454944, 70.10541234668786, 32.9863470466753, 38.64034257045414, 63.83170433047907, 60.199030648454944, 49.42175070681059, 34.03344610288473, 49.22907356066477, 75.80202444429054, 93.6030061542134, 54.598150033144236, 52.81508628524973, 70.37979717263202, 107.7363996853844, 80.69083499570341, 93.2380826167318, 78.51433738825796, 81.32370105824417, 61.867809250367884, 62.597087408320306, 87.93188669987823, 67.1568716116849, 56.33128135834348, 49.61518196998445, 73.18340956555208, 81.641993024771, 55.89290784893326, 70.10541234668786, 91.79256012938836, 102.40200064874995, 52.50652775191775, 52.301824199806006, 99.2514223056366, 118.3252868447224, 81.9615307508321, 86.56862915519338, 95.82273747708687, 95.82273747708687, 55.89290784893326, 64.83690740886694, 42.52108200006278, 54.811841150452466, 77.90333396724647, 49.71218131735948, 83.57807709540792, 46.60915005647103, 71.48811770889668, 72.89809449073893, 70.10541234668786, 32.09659661306934, 37.74523999888851, 66.37447035867858, 47.807737558209425, 56.773093073800744, 74.91890411002962, 52.71203249107963, 62.84208548133934, 80.37625124290746, 69.55984769951485, 53.646802873050504, 45.7076474882609, 47.71445419747699, 88.62154503481175, 101.20898120494374, 75.21212841006654, 69.01852867010916, 51.894808174781225, 49.809370301716946, 70.65525591042105, 54.598150033144236, 43.95661501095434, 102.00277308269969, 90.7231444115574, 67.1568716116849, 68.48142221881122, 87.93188669987823, 79.13013296849392, 48.94146538270724, 77.29708541258049, 56.11166650505244, 74.91890411002962, 86.23113005463887, 77.90333396724647, 39.94485987582193, 63.83170433047907, 68.21443845541918, 91.43469485205186, 62.84208548133934, 91.79256012938836, 80.69083499570341, 39.479488162280596, 62.10995300937982, 36.299242562272255, 65.60118435636834, 74.33588057277046, 76.99573263552006, 58.80452160545304, 47.71445419747699, 61.38635013907805, 73.75739416067401, 63.582848099201826, 88.27604237373367, 147.25819873544805, 172.83610058973207, 194.3248513188351, 482.8410449475606, 315.42036776119477, 443.0798490653855, 180.4245166495895, 371.6560203434465, 371.6560203434465, 371.6560203434465, 94.70636877096143, 174.8734378511126, 110.29129477712178, 124.48917321564447, 319.1384432445317, 115.13366410675533, 120.65904401308495, 182.5513037582002, 166.21485355527525, 259.4577172438939, 304.5240299109616, 124.00383592430612, 188.3461041828892, 305.71590326490076, 330.5577072665093, 177.62729378994385, 291.71615278815983, 137.7969534714142, 168.83235663184237, 132.51803508331912, 105.24068867470233, 150.1626064581185, 185.4260684919039, 198.15756558346607, 164.27839185643572, 203.65069565035918, 124.00383592430612, 172.83610058973207, 135.13171912803952, 210.93763055625195, 197.38502244855516, 258.44618746623325, 320.3875157950033, 201.27809318052155, 214.2594148922178, 326.70659332761613, 305.71590326490076, 176.93479059943397, 92.1518260518184, 117.40447215712238, 226.30304315814422, 166.86540009848025, 376.03698390417554, 157.3690604074729, 105.24068867470233, 88.27604237373367, 47.71445419747699, 266.6501501115634, 220.19890466437903, 148.4131591025766, 96.95226558425567, 124.00383592430612, 170.1565252164996, 76.69555472159655, 93.6030061542134, 134.08011574678568, 157.98498549518746, 241.84118490494708, 269.79333770212673, 235.3179315471721, 571.1525003973912, 591.5892147353535, 490.4446850170856, 421.1414445251494, 112.46659565031442, 98.4790416094955, 106.48123261036584, 117.40447215712238, 169.4931477924318, 257.43860126944566, 153.12429849019884, 237.16355295871682, 196.6154911738957, 201.27809318052155, 151.93267574549122, 195.08541728258788, 195.08541728258788, 167.51849280886478, 195.08541728258788, 176.24498722300464, 138.33627549846275, 228.97063181940615, 253.44738569742526, 258.44618746623325, 151.93267574549122, 480.95862610864765, 200.49338426145144, 220.19890466437903, 243.73796882667915, 400.2892857141792, 384.95444399467493, 444.8140145626792, 282.74099031586354, 140.51475485198003, 158.60332124553608, 443.0798490653855, 309.31958559164264, 469.8172315347884, 288.31755666558627, 119.72006788311512, 506.013070509352, 224.54193890439277, 524.1189959252079, 351.87684773779097, 346.4215028625498, 337.0773738522583, 143.28616006164455, 280.5406824728656, 156.7555365841094, 148.99403178365867, 124.48917321564447, 351.87684773779097, 158.60332124553608, 446.5549673918236, 575.6321049777138, 183.98307137873957, 229.86679753407884, 257.43860126944566, 281.63868765896893, 130.4635333784854, 170.1565252164996, 244.69193226422038, 179.7211081206401, 205.2479478525057, 270.8492789701163, 380.4695888770434, 200.49338426145144, 167.51849280886478, 288.31755666558627, 255.43519818017026, 248.54526950703038, 304.5240299109616, 122.55914747655109, 321.64147707721304, 183.98307137873957, 360.2213667100846, 536.5481197318883, 477.21577638600166, 221.06073880966076, 211.76321709954237, 150.7503262825534, 203.65069565035918, 88.96839995507139, 235.3179315471721, 265.61057969554065, 98.4790416094955, 179.7211081206401, 176.93479059943397, 154.92927926604594, 222.79453967267415, 197.38502244855516, 216.78503707457477, 192.81260332830294, 154.92927926604594, 80.69083499570341, 215.09800253180504, 95.44916002730179, 180.4245166495895, 221.92594607841514, 269.79333770212673, 257.43860126944566, 387.9736791189793, 226.30304315814422, 164.9213593028435, 380.4695888770434, 227.18876822599648, 260.4732060371668, 172.1622764913071, 237.16355295871682, 127.94014021950323, 85.2265069597304, 289.4459996766781, 295.1548104863723, 247.5762833312724, 339.72110446147576, 421.1414445251494, 265.61057969554065, 233.4866728754002, 281.63868765896893, 392.54699007778345, 167.51849280886478, 163.63793110254423, 211.76321709954237, 229.86679753407884, 267.6937892911538, 205.2479478525057, 92.1518260518184, 160.47288700783744, 194.3248513188351, 209.29610089202527, 368.76377024718164, 77.29708541258049, 274.0419645628891, 218.48530319325408, 138.8777083722429, 277.272284523134, 280.5406824728656, 312.9657469853004, 202.8567368308649, 347.77735828917787, 122.08133464248264, 560.1054110313753, 135.66060972280735, 159.2240770935855, 421.1414445251494, 339.72110446147576, 417.86409580268446, 161.10096012939127, 384.95444399467493, 343.72562933929424, 174.19167092744075, 206.05126560732333, 206.05126560732333, 95.0770390179677, 667.7449237904252, 292.8578975283812, 429.4477152409339, 429.4477152409339, 206.85772745893686, 403.4287934927351, 460.7301436924908, 419.4995696321248, 286.07385163895896, 245.649629409089, 555.7466361419574, 532.3726691377844, 308.1136628117644, 267.6937892911538, 267.6937892911538, 153.7236100495016, 210.11526266986428, 492.3642312408232, 473.50205375098, 473.50205375098, 412.9958316281291, 566.7077563764824, 392.54699007778345, 437.9178124670295, 358.8169966923127, 306.91244147925255, 119.2533235714264, 680.9150123157852, 304.5240299109616, 117.40447215712238, 164.27839185643572, 213.42409659600423, 154.32526724792663, 221.06073880966076, 513.9816168847873, 198.15756558346607, 198.15756558346607, 144.4099673433467, 260.4732060371668, 217.63350971989422, 142.7275402635186, 196.6154911738957, 223.66653284614108, 189.8233209864514, 153.7236100495016, 206.05126560732333, 147.25819873544805, 206.85772745893686, 91.43469485205186, 181.83960367209863, 144.97517197488295, 170.82249902639472, 151.3403463773431, 124.97641006343288, 284.95855538715824, 292.8578975283812, 453.58718488624703, 183.98307137873957, 431.12852607499315, 799.1860322364524, 799.1860322364524, 473.50205375098, 265.61057969554065, 296.31001375229687, 360.2213667100846, 469.8172315347884, 133.5573869140851, 68.48142221881122, 469.8172315347884, 189.8233209864514, 141.61682573322432, 295.1548104863723, 444.8140145626792, 174.19167092744075, 324.16414244755515, 268.7415131590023, 232.57639460420748, 177.62729378994385, 130.9741532108186, 451.81881603928605, 138.33627549846275, 170.1565252164996, 170.1565252164996, 162.99996726850102, 647.2005722905537, 771.577792875611, 290.5788592888401, 466.1610848325832, 610.368272250772, 160.47288700783744, 412.9958316281291, 424.4444978122241, 429.4477152409339, 306.91244147925255, 305.71590326490076, 237.16355295871682, 300.97621750018493, 308.1136628117644, 263.5435817656175, 221.92594607841514, 408.18428444915037, 292.8578975283812, 304.5240299109616, 341.05073527023916, 424.4444978122241, 580.1468435322063, 305.71590326490076, 326.70659332761613, 347.77735828917787, 441.35244443864343, 551.4217815078388, 249.51804818745066, 270.8492789701163, 482.8410449475606, 708.0395827712994, 824.5549201253252, 397.1742097789884, 238.09178486008656, 358.8169966923127, 107.7363996853844, 498.16806499462353, 176.93479059943397, 511.9777924614019, 228.07795992374852, 408.18428444915037, 304.5240299109616, 210.11526266986428, 203.65069565035918, 298.6340019934001, 320.3875157950033, 119.72006788311512, 145.54258875480053, 551.4217815078388, 358.8169966923127, 358.8169966923127, 243.73796882667915, 267.6937892911538, 351.87684773779097, 419.4995696321248, 381.95870474451476, 302.15420510895507, 151.3403463773431, 267.6937892911538, 145.54258875480053, 466.1610848325832, 575.6321049777138, 422.78974553481265, 405.0077691463688, 453.58718488624703, 227.18876822599648, 198.15756558346607, 321.64147707721304, 213.42409659600423, 191.3121237449989, 345.0709334152821, 417.86409580268446, 144.97517197488295, 360.2213667100846, 284.95855538715824, 339.72110446147576, 573.3879280792671, 232.57639460420748, 360.2213667100846, 194.3248513188351, 210.93763055625195, 241.84118490494708, 494.29129035606036, 494.29129035606036, 347.77735828917787, 756.6541274363462, 335.7632337116253, 335.7632337116253, 181.13067824168084, 480.95862610864765, 322.9003462251159, 464.34369499541174, 265.61057969554065, 443.0798490653855, 214.2594148922178, 657.3924983001216, 234.40051387501347, 284.95855538715824, 387.9736791189793, 160.47288700783744, 275.1145344862516, 342.38557010684895, 377.5087510463025, 427.7734572673073, 176.24498722300464, 135.66060972280735, 115.58428452718766, 325.4328850285552, 383.45364884670335, 451.81881603928605, 347.77735828917787, 160.47288700783744, 134.60489048229144, 480.95862610864765, 675.6160896504167, 403.4287934927351, 434.50990901347194, 437.9178124670295, 403.4287934927351, 78.82163381626928, 424.4444978122241, 235.3179315471721, 392.54699007778345, 60.67117749756472, 60.67117749756472, 128.44088377025506, 215.09800253180504, 283.84760726337464, 432.81591541665557, 455.3624749331844, 426.1057266069316, 304.5240299109616, 124.97641006343288, 181.83960367209863, 250.49463421597682, 117.86398026897247, 170.1565252164996, 279.4469580033028, 183.26578935968087, 356.0246606707791, 116.49082330627213, 116.94675549762589, 193.56725052076862, 165.5668432526847, 131.48677155266952, 162.99996726850102, 189.8233209864514, 280.5406824728656, 339.72110446147576, 106.89798837141365, 192.81260332830294, 134.08011574678568, 162.3644906197382, 92.87458178129451, 92.87458178129451, 210.11526266986428, 281.63868765896893, 329.26898489707213, 259.4577172438939, 133.5573869140851, 119.2533235714264, 93.96935796203766, 263.5435817656175, 143.8469662358426, 186.88038317169537, 226.30304315814422, 198.15756558346607, 391.0165943978527, 215.09800253180504, 195.84896001735729, 166.21485355527525, 168.83235663184237, 203.65069565035918, 164.9213593028435, 226.30304315814422, 168.83235663184237, 150.7503262825534, 148.4131591025766, 267.6937892911538, 243.73796882667915, 304.5240299109616, 634.6825800645618, 181.83960367209863, 339.72110446147576, 195.84896001735729, 319.1384432445317, 316.65488817567297, 210.11526266986428, 204.44776193681219, 122.55914747655109, 206.05126560732333, 183.98307137873957, 226.30304315814422, 271.90935307537654, 298.6340019934001, 150.7503262825534, 244.69193226422038, 269.79333770212673, 202.06587336346374, 179.02044192165667, 130.9741532108186, 268.7415131590023, 320.3875157950033, 299.8028224298701, 299.8028224298701, 329.26898489707213, 151.3403463773431, 212.59203489720244, 302.15420510895507, 255.43519818017026, 265.61057969554065, 265.61057969554065, 190.56626845863, 506.013070509352, 357.4181017943673, 304.5240299109616, 203.65069565035918, 174.19167092744075, 193.56725052076862, 171.49107938408403, 149.57717793613415, 185.4260684919039, 151.3403463773431, 342.38557010684895, 451.81881603928605, 241.84118490494708, 290.5788592888401, 345.0709334152821, 368.76377024718164, 190.56626845863, 306.91244147925255, 106.89798837141365, 208.48013272330223, 100.42136666458902, 66.63425269055388, 89.31661242710538, 326.70659332761613, 392.54699007778345, 244.69193226422038, 229.86679753407884, 108.5813867407896, 110.7229626998625, 361.6312332766962, 189.8233209864514, 216.78503707457477, 281.63868765896893, 200.49338426145144, 170.1565252164996, 119.72006788311512, 193.56725052076862, 342.38557010684895, 176.24498722300464, 276.1913023296018, 271.90935307537654, 292.8578975283812, 99.2514223056366, 124.97641006343288, 458.93392708825405, 223.66653284614108, 168.17414165184545, 274.0419645628891, 186.15180560442604, 282.74099031586354, 151.3403463773431, 130.4635333784854, 253.44738569742526, 100.42136666458902, 247.5762833312724, 161.10096012939127, 241.84118490494708, 356.0246606707791, 223.66653284614108, 212.59203489720244, 237.16355295871682, 85.2265069597304, 173.5125619611118, 180.4245166495895, 153.7236100495016, 275.1145344862516, 116.03666862607915, 135.66060972280735, 210.11526266986428, 137.7969534714142, 188.3461041828892, 408.18428444915037, 116.94675549762589, 127.94014021950323, 235.3179315471721, 317.89424036643936, 249.51804818745066, 283.84760726337464, 43.699809813576366, 162.3644906197382, 297.46973835596765, 250.49463421597682, 256.43494327896633, 419.4995696321248, 139.96693973881514, 198.15756558346607, 386.4611130887752, 365.89402781973274, 383.45364884670335, 383.45364884670335, 383.45364884670335, 386.4611130887752, 117.86398026897247, 159.8472625113303, 458.93392708825405, 99.2514223056366, 538.6481096972173, 387.9736791189793, 477.21577638600166, 224.54193890439277, 357.4181017943673, 374.5709546383618, 286.07385163895896, 577.8850653361966, 210.11526266986428, 115.58428452718766, 314.1906602856942, 164.27839185643572, 319.1384432445317, 195.84896001735729, 135.13171912803952, 315.42036776119477, 181.83960367209863, 374.5709546383618, 329.26898489707213, 329.26898489707213, 397.1742097789884, 322.9003462251159, 265.61057969554065, 232.57639460420748, 176.24498722300464, 384.95444399467493, 178.32250736130783, 577.8850653361966, 540.7563187908823, 132.51803508331912, 368.76377024718164, 164.9213593028435, 395.6257742790621, 350.5050099164416, 286.07385163895896, 617.56310029872, 441.35244443864343, 364.467542192173, 363.0466179050314, 363.0466179050314, 228.07795992374852, 412.9958316281291, 171.49107938408403, 295.1548104863723, 389.49216516524257, 117.40447215712238, 215.09800253180504, 92.87458178129451, 83.25223705241488, 175.55787313507827, 215.09800253180504, 409.7818725681369, 380.4695888770434, 335.7632337116253, 59.03467599561125, 196.6154911738957, 250.49463421597682, 443.0798490653855, 498.16806499462353, 144.4099673433467, 260.4732060371668, 245.649629409089, 316.65488817567297, 387.9736791189793, 114.68480048885151, 84.563269020706, 141.0647140528757, 146.6840934260825, 151.3403463773431, 274.0419645628891, 719.1895837886203, 228.97063181940615, 165.5668432526847, 192.81260332830294, 405.0077691463688, 439.63177432431667, 130.4635333784854, 351.87684773779097, 321.64147707721304, 130.4635333784854, 114.23768682435218, 881.1694978531985, 345.0709334152821, 441.35244443864343, 180.4245166495895, 555.7466361419574, 193.56725052076862, 206.85772745893686, 403.4287934927351, 334.45421691786504, 600.905389292032, 600.905389292032, 263.5435817656175, 617.56310029872, 327.98528678000895, 292.8578975283812, 302.15420510895507, 208.48013272330223, 149.57717793613415, 272.97357619337527, 284.95855538715824, 356.0246606707791, 277.272284523134, 357.4181017943673, 403.4287934927351, 542.872779181642, 181.13067824168084, 551.4217815078388, 534.4563168515505, 353.25405478056325, 176.24498722300464, 257.43860126944566, 229.86679753407884, 678.2603762605156, 542.872779181642, 575.6321049777138, 295.1548104863723, 119.72006788311512, 250.49463421597682, 455.3624749331844, 202.06587336346374, 297.46973835596765, 733.3743273150056, 239.95916187280093, 346.4215028625498, 346.4215028625498, 269.79333770212673, 71.20941195067294, 275.1145344862516, 387.9736791189793, 310.5302282198318, 259.4577172438939, 233.4866728754002, 143.8469662358426, 190.56626845863, 354.636652059297, 178.32250736130783, 262.51612271179147, 591.5892147353535, 304.5240299109616, 270.8492789701163, 218.48530319325408, 253.44738569742526, 342.38557010684895, 466.1610848325832, 408.18428444915037, 292.8578975283812, 186.88038317169537, 309.31958559164264, 245.649629409089, 504.04031250414465, 504.04031250414465, 275.1145344862516, 343.72562933929424, 573.3879280792671, 450.0573414090999, 310.5302282198318, 315.42036776119477, 297.46973835596765, 370.20707083592094, 221.92594607841514, 494.29129035606036, 629.7434410413515, 361.6312332766962, 326.70659332761613, 281.63868765896893, 298.6340019934001, 248.54526950703038, 649.7336437102142, 586.9854309417088, 279.4469580033028, 270.8492789701163, 199.71173463253027, 422.78974553481265, 349.1385203839046, 186.88038317169537, 424.4444978122241, 351.87684773779097, 278.35749756134834, 217.63350971989422, 211.76321709954237, 197.38502244855516, 53.43765380953904, 131.48677155266952, 68.21443845541918, 59.49769069799599, 96.95226558425567, 180.4245166495895, 136.72460907194932, 124.00383592430612, 152.5273234252381, 228.07795992374852, 174.19167092744075, 289.4459996766781, 136.19157033683516, 249.51804818745066, 172.1622764913071, 417.86409580268446, 177.62729378994385, 302.15420510895507, 193.56725052076862, 189.8233209864514, 189.8233209864514, 180.4245166495895, 143.28616006164455, 148.4131591025766, 178.32250736130783, 192.0608982264211, 329.26898489707213, 92.1518260518184, 150.1626064581185, 173.5125619611118, 238.09178486008656, 268.7415131590023, 289.4459996766781, 170.1565252164996, 221.92594607841514, 200.49338426145144, 91.07822475921115, 181.83960367209863, 150.7503262825534, 122.55914747655109, 161.10096012939127, 121.13129015902297, 206.85772745893686, 221.92594607841514, 237.16355295871682, 103.60908302823987, 85.89494673838159, 181.13067824168084, 195.84896001735729, 156.7555365841094, 215.09800253180504, 97.33172602285453, 213.42409659600423, 139.96693973881514, 241.84118490494708, 240.8983354780937, 143.28616006164455, 213.42409659600423, 157.98498549518746, 70.65525591042105, 69.55984769951485, 111.15632012308374, 166.86540009848025, 195.84896001735729, 98.86447768050158, 95.0770390179677, 102.00277308269969, 110.7229626998625, 166.21485355527525, 253.44738569742526, 170.82249902639472, 149.57717793613415, 103.60908302823987, 115.58428452718766, 183.26578935968087, 163.63793110254423, 209.29610089202527, 197.38502244855516, 191.3121237449989, 86.56862915519338, 92.51249810131188, 129.9549042642197, 129.9549042642197, 204.44776193681219, 122.08133464248264, 143.28616006164455, 103.60908302823987, 196.6154911738957, 166.86540009848025, 166.86540009848025, 193.56725052076862, 193.56725052076862, 193.56725052076862, 200.49338426145144, 165.5668432526847, 176.24498722300464, 170.82249902639472, 179.02044192165667, 200.49338426145144, 65.60118435636834, 159.2240770935855, 195.08541728258788, 223.66653284614108, 196.6154911738957, 181.83960367209863, 190.56626845863, 181.13067824168084, 179.7211081206401, 267.6937892911538, 249.51804818745066, 148.99403178365867, 177.62729378994385, 182.5513037582002, 206.05126560732333, 204.44776193681219, 257.43860126944566, 231.6696651716735, 172.1622764913071, 290.5788592888401, 268.7415131590023, 253.44738569742526, 460.7301436924908, 192.0608982264211, 429.4477152409339, 376.03698390417554, 96.95226558425567, 253.44738569742526, 95.82273747708687, 135.13171912803952, 188.3461041828892, 265.61057969554065, 178.32250736130783, 295.1548104863723, 129.9549042642197, 206.05126560732333, 153.12429849019884, 88.27604237373367, 82.60436300370014, 106.89798837141365, 93.2380826167318, 135.13171912803952, 145.54258875480053, 98.86447768050158, 143.8469662358426, 103.20514949397284, 183.98307137873957, 170.1565252164996, 186.88038317169537, 143.8469662358426, 192.0608982264211, 242.7877245401199, 164.9213593028435, 80.69083499570341, 111.59137365930344, 159.2240770935855, 116.03666862607915, 205.2479478525057, 205.2479478525057, 124.48917321564447, 145.54258875480053, 184.70316076024525, 157.98498549518746, 137.7969534714142, 167.51849280886478, 124.48917321564447, 106.06610162605153, 134.60489048229144, 103.60908302823987, 183.26578935968087, 199.71173463253027, 120.65904401308495, 346.4215028625498, 165.5668432526847, 119.72006788311512, 145.54258875480053, 217.63350971989422, 329.26898489707213, 222.79453967267415, 253.44738569742526, 230.76647074218755, 84.2335880783952, 179.02044192165667, 176.93479059943397, 342.38557010684895, 192.81260332830294, 73.75739416067401, 381.95870474451476, 103.60908302823987, 203.65069565035918, 358.8169966923127, 138.8777083722429, 168.17414165184545, 321.64147707721304, 78.51433738825796, 130.4635333784854, 240.8983354780937, 252.45928798182638, 245.649629409089, 178.32250736130783, 170.1565252164996, 334.45421691786504, 219.3404304920078, 129.9549042642197, 168.83235663184237, 253.44738569742526, 303.3368032308679, 170.82249902639472, 68.21443845541918, 233.4866728754002, 108.5813867407896, 326.70659332761613, 158.60332124553608, 206.85772745893686, 154.32526724792663, 232.57639460420748, 206.05126560732333, 230.76647074218755, 144.97517197488295, 247.5762833312724, 178.32250736130783, 222.79453967267415, 122.55914747655109, 283.84760726337464, 245.649629409089, 180.4245166495895, 140.51475485198003, 99.2514223056366, 220.19890466437903, 163.63793110254423, 95.0770390179677, 73.46984133199462, 171.49107938408403, 194.3248513188351, 174.8734378511126, 106.48123261036584, 192.81260332830294, 85.56007407666469, 142.1710983175922, 154.32526724792663, 90.7231444115574, 176.24498722300464, 81.32370105824417, 104.01459751366006, 134.08011574678568, 142.1710983175922, 121.60538462411665, 129.44825810694724, 90.01713130052181, 151.3403463773431, 239.95916187280093, 239.95916187280093, 102.80279074731456, 96.95226558425567, 213.42409659600423, 281.63868765896893, 253.44738569742526, 59.49769069799599, 254.43935071813848, 102.40200064874995, 168.83235663184237, 188.3461041828892, 51.591626129191596, 77.90333396724647, 68.21443845541918, 163.63793110254423, 84.2335880783952, 81.00664999449546, 116.49082330627213, 91.43469485205186, 198.15756558346607, 62.84208548133934, 148.4131591025766, 139.96693973881514, 163.63793110254423, 144.97517197488295, 162.99996726850102, 156.7555365841094, 127.94014021950323, 90.01713130052181, 99.63988138921447, 122.08133464248264, 168.83235663184237, 166.86540009848025, 198.15756558346607, 252.45928798182638, 270.8492789701163, 136.72460907194932, 111.15632012308374, 105.65258908406659, 147.25819873544805, 204.44776193681219, 222.79453967267415, 360.2213667100846, 179.02044192165667, 156.7555365841094, 192.81260332830294, 126.44959243764188, 148.99403178365867, 220.19890466437903, 161.10096012939127, 109.86130976812342, 126.44959243764188, 79.75075829332496, 159.2240770935855, 268.7415131590023, 173.5125619611118, 215.09800253180504, 159.8472625113303, 265.61057969554065, 258.44618746623325, 95.0770390179677, 334.45421691786504, 219.3404304920078, 177.62729378994385, 427.7734572673073, 35.45837151398557, 294.00411093116844, 152.5273234252381, 185.4260684919039, 226.30304315814422, 106.06610162605153, 122.08133464248264, 248.54526950703038, 241.84118490494708, 241.84118490494708, 216.78503707457477, 237.16355295871682, 326.70659332761613, 327.98528678000895, 160.47288700783744, 146.6840934260825, 249.51804818745066, 122.08133464248264, 132.00139622598326, 333.1503034969411, 329.26898489707213, 149.57717793613415, 142.1710983175922, 373.11064087900144, 93.6030061542134, 414.6122516103204, 289.4459996766781, 411.38571347022946, 123.03883041717654, 421.1414445251494, 263.5435817656175, 231.6696651716735, 113.7923162908356, 109.86130976812342, 173.5125619611118, 86.56862915519338, 180.4245166495895, 218.48530319325408, 184.70316076024525, 243.73796882667915, 263.5435817656175, 235.3179315471721, 479.0835461252901, 174.19167092744075, 296.31001375229687, 186.88038317169537, 199.71173463253027, 112.02812994692015, 137.7969534714142, 159.2240770935855, 233.4866728754002, 243.73796882667915, 275.1145344862516, 371.6560203434465, 191.3121237449989, 263.5435817656175, 173.5125619611118, 166.21485355527525, 204.44776193681219, 187.61181231093738, 356.0246606707791, 87.24759532868651, 302.15420510895507, 179.7211081206401, 186.88038317169537, 269.79333770212673, 182.5513037582002, 110.29129477712178, 180.4245166495895, 134.08011574678568, 140.51475485198003, 432.81591541665557, 195.08541728258788, 188.3461041828892, 575.6321049777138, 494.29129035606036, 419.4995696321248, 389.49216516524257, 416.23499808144635, 416.23499808144635, 186.88038317169537, 431.12852607499315, 314.1906602856942, 295.1548104863723, 316.65488817567297, 330.5577072665093, 82.92766733936132, 99.2514223056366, 153.7236100495016, 153.7236100495016, 100.81440478091984, 183.26578935968087, 156.7555365841094, 141.61682573322432, 120.65904401308495, 177.62729378994385, 163.63793110254423, 119.72006788311512, 110.29129477712178, 201.27809318052155, 166.86540009848025, 146.6840934260825, 200.49338426145144, 159.2240770935855, 201.27809318052155, 86.56862915519338, 228.07795992374852, 280.5406824728656, 121.60538462411665, 154.92927926604594, 246.61107487460225, 146.11222634120347, 245.649629409089, 97.71267162761835, 161.73149145963927, 188.3461041828892, 295.1548104863723, 419.4995696321248, 274.0419645628891, 182.5513037582002, 412.9958316281291, 309.31958559164264, 324.16414244755515, 439.63177432431667, 95.44916002730179, 334.45421691786504, 295.1548104863723, 343.72562933929424, 154.32526724792663, 283.84760726337464, 82.92766733936132, 333.1503034969411, 263.5435817656175, 296.31001375229687, 339.72110446147576, 291.71615278815983, 108.5813867407896, 288.31755666558627, 331.8514735526883, 163.63793110254423, 142.7275402635186, 254.43935071813848, 191.3121237449989, 170.1565252164996, 357.4181017943673, 232.57639460420748, 119.72006788311512, 154.32526724792663, 259.4577172438939, 238.09178486008656, 141.61682573322432, 113.34868209247811, 185.4260684919039, 222.79453967267415, 184.70316076024525, 134.60489048229144, 119.2533235714264, 339.72110446147576, 284.95855538715824, 198.9331323667161, 250.49463421597682, 269.79333770212673, 374.5709546383618, 101.20898120494374, 101.20898120494374, 161.10096012939127, 238.09178486008656, 283.84760726337464, 480.95862610864765, 351.87684773779097, 185.4260684919039, 184.70316076024525, 380.4695888770434, 380.4695888770434, 211.76321709954237, 208.48013272330223, 235.3179315471721, 414.6122516103204, 151.3403463773431, 112.02812994692015, 119.72006788311512, 237.16355295871682, 339.72110446147576, 203.65069565035918, 226.30304315814422, 218.48530319325408, 322.9003462251159, 202.8567368308649, 236.23893989057657, 262.51612271179147, 148.4131591025766, 343.72562933929424, 215.9398723106141, 133.5573869140851, 227.18876822599648, 311.7456091692961, 337.0773738522583, 183.26578935968087, 257.43860126944566, 118.3252868447224, 432.81591541665557, 322.9003462251159, 194.3248513188351, 135.66060972280735, 133.0366960079705, 114.68480048885151, 312.9657469853004, 403.4287934927351, 116.49082330627213, 97.33172602285453, 288.31755666558627, 209.29610089202527, 181.13067824168084, 196.6154911738957, 65.8579401324037, 151.3403463773431, 277.272284523134, 245.649629409089, 164.27839185643572, 160.47288700783744, 173.5125619611118, 172.83610058973207, 215.09800253180504, 276.1913023296018, 147.834551029463, 343.72562933929424, 136.72460907194932, 498.16806499462353, 450.0573414090999, 405.0077691463688, 243.73796882667915, 148.99403178365867, 202.8567368308649, 271.90935307537654, 443.0798490653855, 358.8169966923127, 347.77735828917787, 153.12429849019884, 373.11064087900144, 321.64147707721304, 199.71173463253027, 190.56626845863, 264.57506218047985, 255.43519818017026, 255.43519818017026, 255.43519818017026, 137.7969534714142, 339.72110446147576, 291.71615278815983, 403.4287934927351, 308.1136628117644, 197.38502244855516, 202.8567368308649, 239.02364975839862, 197.38502244855516, 312.9657469853004, 257.43860126944566, 151.3403463773431, 114.68480048885151, 310.5302282198318, 326.70659332761613, 249.51804818745066, 373.11064087900144, 183.26578935968087, 297.46973835596765, 330.5577072665093, 251.47504249414803, 187.61181231093738, 106.48123261036584, 95.44916002730179, 208.48013272330223, 118.3252868447224, 288.31755666558627, 221.06073880966076, 235.3179315471721, 235.3179315471721, 326.70659332761613, 109.00636227269737, 134.08011574678568, 234.40051387501347, 441.35244443864343, 243.73796882667915, 167.51849280886478, 141.0647140528757, 228.97063181940615, 211.76321709954237, 205.2479478525057, 172.83610058973207, 414.6122516103204, 185.4260684919039, 115.58428452718766, 229.86679753407884, 234.40051387501347, 270.8492789701163, 101.6051019574268, 306.91244147925255, 398.728705684109, 178.32250736130783, 236.23893989057657, 111.15632012308374, 288.31755666558627, 151.3403463773431, 178.32250736130783, 168.83235663184237, 116.49082330627213, 86.90744918987926, 263.5435817656175, 181.13067824168084, 373.11064087900144, 259.4577172438939, 287.1935130368687, 254.43935071813848, 329.26898489707213, 475.3552883908416, 268.7415131590023, 240.8983354780937, 532.3726691377844, 471.65604418826433, 235.3179315471721, 166.86540009848025, 431.12852607499315, 580.1468435322063, 205.2479478525057, 197.38502244855516, 172.1622764913071, 264.57506218047985, 137.25973406168575, 177.62729378994385, 143.8469662358426, 288.31755666558627, 326.70659332761613, 228.97063181940615, 200.49338426145144, 159.8472625113303, 251.47504249414803, 175.55787313507827, 303.3368032308679, 198.15756558346607, 163.63793110254423, 180.4245166495895, 331.8514735526883, 177.62729378994385, 175.55787313507827, 172.1622764913071, 153.7236100495016, 364.467542192173, 364.467542192173, 154.32526724792663, 164.9213593028435, 157.3690604074729, 138.8777083722429, 169.4931477924318, 167.51849280886478, 129.44825810694724, 282.74099031586354, 299.8028224298701, 311.7456091692961, 339.72110446147576, 387.9736791189793, 108.5813867407896, 198.9331323667161, 258.44618746623325, 128.44088377025506, 156.14440466345445, 183.26578935968087, 290.5788592888401, 76.09870517958821, 141.61682573322432, 70.93179276322707, 98.4790416094955, 85.89494673838159, 85.89494673838159, 108.15806803146498, 202.06587336346374, 193.56725052076862, 184.70316076024525, 124.00383592430612, 189.8233209864514, 160.47288700783744, 292.8578975283812, 437.9178124670295, 150.1626064581185, 123.52039078374908, 210.93763055625195, 133.0366960079705, 44.21492934326177, 287.1935130368687, 213.42409659600423, 124.97641006343288, 82.60436300370014, 154.32526724792663, 83.25223705241488, 179.7211081206401, 405.0077691463688, 338.3966573919768, 327.98528678000895, 312.9657469853004, 261.4926693412009, 246.61107487460225, 436.21053271376616, 269.79333770212673, 144.4099673433467, 277.272284523134, 239.02364975839862, 142.7275402635186, 220.19890466437903, 384.95444399467493, 216.78503707457477, 237.16355295871682, 122.08133464248264, 271.90935307537654, 266.6501501115634, 168.83235663184237, 406.59292473597867, 337.0773738522583, 102.80279074731456, 120.1886389803891, 120.1886389803891, 185.4260684919039, 221.06073880966076, 436.21053271376616, 316.65488817567297, 98.86447768050158, 143.28616006164455, 356.0246606707791, 108.5813867407896, 241.84118490494708, 52.097918711212095, 127.94014021950323, 118.3252868447224, 350.5050099164416, 542.872779181642, 280.5406824728656, 427.7734572673073, 255.43519818017026, 121.13129015902297, 164.27839185643572, 81.9615307508321, 80.06289393593435, 137.7969534714142, 159.8472625113303, 221.06073880966076, 295.1548104863723, 371.6560203434465, 276.1913023296018, 235.3179315471721, 150.1626064581185, 161.10096012939127, 458.93392708825405, 511.9777924614019, 76.09870517958821, 153.7236100495016, 139.42126035437516, 350.5050099164416, 306.91244147925255, 122.55914747655109, 101.20898120494374, 121.13129015902297, 140.51475485198003, 95.44916002730179, 129.44825810694724, 126.44959243764188, 161.73149145963927, 98.4790416094955, 133.5573869140851, 67.68358948381629, 116.03666862607915, 152.5273234252381, 127.94014021950323, 139.42126035437516, 271.90935307537654, 79.43983955226133, 86.23113005463887, 103.60908302823987, 105.24068867470233, 98.86447768050158, 43.10641711474324, 126.94450214932196, 169.4931477924318, 316.65488817567297, 151.3403463773431, 315.42036776119477, 106.89798837141365, 166.86540009848025, 175.55787313507827, 159.8472625113303, 137.25973406168575, 100.42136666458902, 147.834551029463, 161.73149145963927, 143.8469662358426, 89.31661242710538, 235.3179315471721, 135.13171912803952, 112.90677745995042, 335.7632337116253, 132.00139622598326, 132.00139622598326, 334.45421691786504, 145.54258875480053, 161.73149145963927, 108.5813867407896, 161.73149145963927, 97.33172602285453, 269.79333770212673, 156.14440466345445, 138.8777083722429, 172.1622764913071, 212.59203489720244, 243.73796882667915, 333.1503034969411, 92.87458178129451, 342.38557010684895, 215.9398723106141, 305.71590326490076, 305.71590326490076, 264.57506218047985, 61.38635013907805, 81.32370105824417, 169.4931477924318, 185.4260684919039, 156.7555365841094, 259.4577172438939, 268.7415131590023, 128.44088377025506, 199.71173463253027, 114.68480048885151, 247.5762833312724, 228.07795992374852, 368.76377024718164, 315.42036776119477, 365.89402781973274, 128.94358717585138, 84.89424029774196, 91.07822475921115, 130.4635333784854, 151.3403463773431, 136.19157033683516, 83.90519244027122, 153.7236100495016, 89.66618776422116, 267.6937892911538, 134.60489048229144, 288.31755666558627, 303.3368032308679, 364.467542192173, 136.72460907194932, 303.3368032308679, 186.88038317169537, 219.3404304920078, 221.06073880966076, 221.06073880966076, 263.5435817656175, 263.5435817656175, 147.834551029463, 159.8472625113303, 256.43494327896633, 227.18876822599648, 169.4931477924318, 54.598150033144236, 263.5435817656175, 312.9657469853004, 101.6051019574268, 257.43860126944566, 135.13171912803952, 141.0647140528757, 55.6750020519895, 151.93267574549122, 185.4260684919039, 259.4577172438939, 173.5125619611118, 73.18340956555208, 109.00636227269737, 249.51804818745066, 225.42077120508256, 133.5573869140851, 106.06610162605153, 205.2479478525057, 198.15756558346607, 116.49082330627213, 147.834551029463, 201.27809318052155, 277.272284523134, 291.71615278815983, 225.42077120508256, 237.16355295871682, 203.65069565035918, 146.11222634120347, 111.59137365930344, 96.95226558425567, 239.02364975839862, 231.6696651716735, 148.99403178365867, 224.54193890439277, 289.4459996766781, 158.60332124553608, 248.54526950703038, 212.59203489720244, 314.1906602856942, 271.90935307537654, 206.85772745893686, 232.57639460420748, 189.8233209864514, 111.15632012308374, 249.51804818745066, 303.3368032308679, 327.98528678000895, 343.72562933929424, 164.9213593028435, 170.82249902639472, 172.83610058973207, 170.82249902639472, 345.0709334152821, 156.14440466345445, 153.12429849019884, 221.92594607841514, 139.42126035437516, 146.11222634120347, 164.9213593028435, 80.69083499570341, 346.4215028625498, 254.43935071813848, 297.46973835596765, 96.19777706766972, 236.23893989057657, 146.11222634120347, 207.6673457129932, 488.53262239485935, 93.6030061542134, 99.2514223056366, 99.2514223056366, 141.0647140528757, 116.49082330627213, 166.21485355527525, 198.15756558346607, 182.5513037582002, 259.4577172438939, 239.95916187280093, 147.834551029463, 380.4695888770434, 102.40200064874995, 295.1548104863723, 266.6501501115634, 49.9067492938035, 142.1710983175922, 138.33627549846275, 202.8567368308649, 167.51849280886478, 113.7923162908356, 108.15806803146498, 142.7275402635186, 95.82273747708687, 146.11222634120347, 156.7555365841094, 216.78503707457477, 104.4216991379013, 137.25973406168575, 248.54526950703038, 221.06073880966076, 115.13366410675533, 85.56007407666469, 124.97641006343288, 148.99403178365867, 181.13067824168084, 197.38502244855516, 181.83960367209863, 99.63988138921447, 172.83610058973207, 190.56626845863, 129.44825810694724, 277.272284523134, 83.25223705241488, 83.25223705241488, 104.4216991379013, 268.7415131590023, 138.33627549846275, 130.9741532108186, 124.48917321564447, 160.47288700783744, 113.34868209247811, 218.48530319325408, 166.86540009848025, 143.8469662358426, 55.89290784893326, 100.81440478091984, 168.17414165184545, 141.0647140528757, 193.56725052076862, 161.10096012939127, 127.94014021950323, 180.4245166495895, 181.83960367209863, 117.86398026897247, 148.4131591025766, 166.86540009848025, 312.9657469853004, 269.79333770212673, 109.00636227269737, 201.27809318052155, 161.10096012939127, 159.8472625113303, 239.95916187280093, 221.06073880966076, 129.9549042642197, 126.94450214932196, 94.33714363029665, 230.76647074218755, 227.18876822599648, 135.66060972280735, 179.7211081206401, 245.649629409089, 71.20941195067294, 67.41971617548266, 84.563269020706, 87.58907276183992, 343.72562933929424, 462.53339048974414, 58.57526450222688, 192.81260332830294, 210.93763055625195, 138.8777083722429, 177.62729378994385, 94.33714363029665, 218.48530319325408, 127.44134888284603, 172.83610058973207, 260.4732060371668, 148.4131591025766, 164.27839185643572, 175.55787313507827, 88.62154503481175, 197.38502244855516, 122.08133464248264, 176.24498722300464, 137.7969534714142, 115.13366410675533, 266.6501501115634, 83.57807709540792, 116.94675549762589, 187.61181231093738, 180.4245166495895, 189.08326999196993, 142.7275402635186, 193.56725052076862, 59.03467599561125, 259.4577172438939, 351.87684773779097, 182.5513037582002, 426.1057266069316, 195.08541728258788, 232.57639460420748, 201.27809318052155, 136.72460907194932, 254.43935071813848, 297.46973835596765, 160.47288700783744, 94.33714363029665, 208.48013272330223, 150.7503262825534, 60.199030648454944, 221.06073880966076, 176.24498722300464, 142.7275402635186, 253.44738569742526, 364.467542192173, 310.5302282198318, 195.08541728258788, 276.1913023296018, 206.05126560732333, 189.8233209864514, 230.76647074218755, 276.1913023296018, 309.31958559164264, 177.62729378994385, 198.15756558346607, 161.73149145963927, 267.6937892911538, 113.7923162908356, 187.61181231093738, 208.48013272330223, 132.51803508331912, 178.32250736130783, 524.1189959252079, 166.21485355527525, 144.97517197488295, 405.0077691463688, 444.8140145626792, 486.6280141983472, 251.47504249414803, 186.15180560442604, 183.26578935968087, 170.1565252164996, 144.97517197488295, 214.2594148922178, 334.45421691786504, 277.272284523134, 277.272284523134, 290.5788592888401, 263.5435817656175, 248.54526950703038, 129.9549042642197, 419.4995696321248, 326.70659332761613, 314.1906602856942, 119.72006788311512, 383.45364884670335, 249.51804818745066, 267.6937892911538, 159.2240770935855, 319.1384432445317, 467.98558773246197, 115.58428452718766, 144.4099673433467, 166.21485355527525, 182.5513037582002, 240.8983354780937, 280.5406824728656, 297.46973835596765, 193.56725052076862, 274.0419645628891, 179.7211081206401, 141.61682573322432, 116.49082330627213, 480.95862610864765, 389.49216516524257, 192.81260332830294, 166.21485355527525, 133.5573869140851, 161.10096012939127, 176.93479059943397, 103.60908302823987, 201.27809318052155, 221.92594607841514, 343.72562933929424, 156.7555365841094, 132.51803508331912, 172.1622764913071, 92.87458178129451, 184.70316076024525, 321.64147707721304, 321.64147707721304, 217.63350971989422, 417.86409580268446, 286.07385163895896, 91.07822475921115, 364.467542192173, 159.2240770935855, 151.93267574549122, 168.83235663184237, 222.79453967267415, 225.42077120508256, 147.834551029463, 211.76321709954237, 349.1385203839046, 120.1886389803891, 176.24498722300464, 174.8734378511126, 314.1906602856942, 144.4099673433467, 236.23893989057657, 247.5762833312724, 370.20707083592094, 81.32370105824417, 68.48142221881122, 215.09800253180504, 220.19890466437903, 269.79333770212673, 288.31755666558627, 241.84118490494708, 227.18876822599648, 141.61682573322432, 358.8169966923127, 239.95916187280093, 441.35244443864343, 206.05126560732333, 48.182698291098816, 93.96935796203766, 212.59203489720244, 148.99403178365867, 196.6154911738957, 106.06610162605153, 145.54258875480053, 144.4099673433467, 128.44088377025506, 246.61107487460225, 431.12852607499315, 183.26578935968087, 146.6840934260825, 100.42136666458902, 73.75739416067401, 157.98498549518746, 102.80279074731456, 93.96935796203766, 170.82249902639472, 191.3121237449989, 94.33714363029665, 275.1145344862516, 75.21212841006654, 195.08541728258788, 186.88038317169537, 202.06587336346374, 226.30304315814422, 85.2265069597304, 166.86540009848025, 162.99996726850102, 162.99996726850102, 141.61682573322432, 150.7503262825534, 179.02044192165667, 110.29129477712178, 189.8233209864514, 163.63793110254423, 205.2479478525057, 165.5668432526847, 49.809370301716946, 42.10785803778799, 170.82249902639472, 141.61682573322432, 169.4931477924318, 124.00383592430612, 181.83960367209863, 110.29129477712178, 93.6030061542134, 176.24498722300464, 306.91244147925255, 357.4181017943673, 208.48013272330223, 210.11526266986428, 151.93267574549122, 598.5626811957673, 507.9935496710831, 338.3966573919768, 189.8233209864514, 220.19890466437903, 196.6154911738957, 371.6560203434465, 239.95916187280093, 292.8578975283812, 216.78503707457477, 412.9958316281291, 292.8578975283812, 229.86679753407884, 144.4099673433467, 252.45928798182638, 284.95855538715824, 180.4245166495895, 338.3966573919768, 192.81260332830294, 122.08133464248264, 314.1906602856942, 164.27839185643572, 271.90935307537654, 294.00411093116844, 229.86679753407884, 245.649629409089, 252.45928798182638, 130.9741532108186, 334.45421691786504, 156.14440466345445, 342.38557010684895, 106.06610162605153, 209.29610089202527, 216.78503707457477, 224.54193890439277, 184.70316076024525, 100.02986085865793, 173.5125619611118, 189.8233209864514, 219.3404304920078, 147.25819873544805, 194.3248513188351, 327.98528678000895, 170.82249902639472, 305.71590326490076, 215.09800253180504, 302.15420510895507, 128.44088377025506, 205.2479478525057, 251.47504249414803, 33.57131140436464, 151.3403463773431, 138.8777083722429, 109.00636227269737, 100.42136666458902, 142.7275402635186, 279.4469580033028, 120.65904401308495, 234.40051387501347, 439.63177432431667, 615.1554499417665, 146.6840934260825, 48.65553748142455, 176.24498722300464, 102.40200064874995, 250.49463421597682, 292.8578975283812, 132.00139622598326, 211.76321709954237, 86.23113005463887, 207.6673457129932, 236.23893989057657, 143.28616006164455, 116.03666862607915, 530.2971447966088, 179.02044192165667, 192.0608982264211, 130.4635333784854, 138.33627549846275, 110.7229626998625, 142.1710983175922, 128.44088377025506, 173.5125619611118, 342.38557010684895, 124.97641006343288, 81.9615307508321, 166.86540009848025, 315.42036776119477, 168.83235663184237, 203.65069565035918, 169.4931477924318, 191.3121237449989, 99.2514223056366, 118.78839892336089, 122.55914747655109, 271.90935307537654, 139.96693973881514, 209.29610089202527, 115.13366410675533, 59.49769069799599, 190.56626845863, 171.49107938408403, 179.02044192165667, 169.4931477924318, 157.98498549518746, 178.32250736130783, 101.6051019574268, 205.2479478525057, 305.71590326490076, 146.6840934260825, 221.92594607841514, 429.4477152409339, 224.54193890439277, 159.2240770935855, 181.83960367209863, 172.1622764913071, 349.1385203839046, 417.86409580268446, 208.48013272330223, 214.2594148922178, 181.83960367209863, 338.3966573919768, 233.4866728754002, 395.6257742790621, 409.7818725681369, 221.92594607841514, 277.272284523134, 408.18428444915037, 186.15180560442604, 475.3552883908416, 432.81591541665557, 193.56725052076862, 127.94014021950323, 86.56862915519338, 198.9331323667161, 320.3875157950033, 180.4245166495895, 190.56626845863, 146.6840934260825, 143.8469662358426, 139.42126035437516, 119.72006788311512, 143.8469662358426, 142.1710983175922, 142.1710983175922, 260.4732060371668, 265.61057969554065, 109.86130976812342, 246.61107487460225, 518.012824668342, 289.4459996766781, 263.5435817656175, 416.23499808144635, 213.42409659600423, 194.3248513188351, 195.08541728258788, 276.1913023296018, 210.93763055625195, 180.4245166495895, 236.23893989057657, 197.38502244855516, 322.9003462251159, 319.1384432445317, 282.74099031586354, 398.728705684109, 225.42077120508256, 240.8983354780937, 248.54526950703038, 417.86409580268446, 86.56862915519338, 300.97621750018493, 257.43860126944566, 212.59203489720244, 251.47504249414803, 110.7229626998625, 133.5573869140851, 288.31755666558627, 288.31755666558627, 138.33627549846275, 284.95855538715824, 260.4732060371668, 241.84118490494708, 292.8578975283812, 480.95862610864765, 284.95855538715824, 189.8233209864514, 237.16355295871682, 287.1935130368687, 258.44618746623325, 88.27604237373367, 259.4577172438939, 345.0709334152821, 197.38502244855516, 444.8140145626792, 358.8169966923127, 287.1935130368687, 286.07385163895896, 154.92927926604594, 347.77735828917787, 205.2479478525057, 262.51612271179147, 462.53339048974414, 528.2297121579952, 250.49463421597682, 132.51803508331912, 270.8492789701163, 213.42409659600423, 189.8233209864514, 205.2479478525057, 241.84118490494708, 74.62682298320091, 172.1622764913071, 135.13171912803952, 267.6937892911538, 265.61057969554065, 181.83960367209863, 231.6696651716735, 263.5435817656175, 129.44825810694724, 238.09178486008656, 146.6840934260825, 98.09510821132314, 296.31001375229687, 195.08541728258788, 612.7571861239999, 274.0419645628891, 167.51849280886478, 166.86540009848025, 282.74099031586354, 255.43519818017026, 236.23893989057657, 227.18876822599648, 164.27839185643572, 215.9398723106141, 246.61107487460225, 243.73796882667915, 477.21577638600166, 159.2240770935855, 530.2971447966088, 161.10096012939127, 342.38557010684895, 368.76377024718164, 383.45364884670335, 427.7734572673073, 327.98528678000895, 387.9736791189793, 95.82273747708687, 642.1640174789829, 492.3642312408232, 172.83610058973207, 142.7275402635186, 221.06073880966076, 189.8233209864514, 343.72562933929424, 211.76321709954237, 310.5302282198318, 195.84896001735729, 180.4245166495895, 378.98627852215554, 240.8983354780937, 240.8983354780937, 136.19157033683516, 464.34369499541174, 780.6729077394361, 401.85597368179066, 227.18876822599648, 236.23893989057657, 434.50990901347194, 215.09800253180504, 268.7415131590023, 229.86679753407884, 443.0798490653855, 173.5125619611118, 394.08337555704935, 360.2213667100846, 380.4695888770434, 380.4695888770434, 380.4695888770434, 380.4695888770434, 380.4695888770434, 380.4695888770434, 394.08337555704935, 460.7301436924908, 378.98627852215554, 368.76377024718164, 288.31755666558627, 117.86398026897247, 263.5435817656175, 414.6122516103204, 373.11064087900144, 354.636652059297, 304.5240299109616, 269.79333770212673, 325.4328850285552, 200.49338426145144, 536.5481197318883, 419.4995696321248, 321.64147707721304, 203.65069565035918, 203.65069565035918, 432.81591541665557, 538.6481096972173, 844.108676557209, 844.108676557209, 245.649629409089, 488.53262239485935, 466.1610848325832, 530.2971447966088, 603.2572664885378, 376.03698390417554, 351.87684773779097, 392.54699007778345, 392.54699007778345, 384.95444399467493, 534.4563168515505, 335.7632337116253, 343.72562933929424, 327.98528678000895, 327.98528678000895, 327.98528678000895, 327.98528678000895, 327.98528678000895, 327.98528678000895, 327.98528678000895, 327.98528678000895, 327.98528678000895, 289.4459996766781, 409.7818725681369, 380.4695888770434, 347.77735828917787, 347.77735828917787, 139.96693973881514, 408.18428444915037, 457.14471326890896, 377.5087510463025, 598.5626811957673, 486.6280141983472, 479.0835461252901, 479.0835461252901, 351.87684773779097, 228.07795992374852, 374.5709546383618, 354.636652059297, 289.4459996766781, 380.4695888770434, 264.57506218047985, 181.83960367209863, 256.43494327896633, 319.1384432445317, 228.97063181940615, 405.0077691463688, 652.2766292910974, 416.23499808144635, 346.4215028625498, 520.0402695399388, 238.09178486008656, 350.5050099164416, 724.8302576041393, 238.09178486008656, 458.93392708825405, 295.1548104863723, 295.1548104863723, 401.85597368179066, 305.71590326490076, 274.0419645628891, 270.8492789701163, 506.013070509352, 327.98528678000895, 439.63177432431667, 201.27809318052155, 387.9736791189793, 511.9777924614019, 326.70659332761613, 405.0077691463688, 248.54526950703038, 201.27809318052155, 195.08541728258788, 77.90333396724647, 72.04880596519355, 99.63988138921447, 335.7632337116253, 458.93392708825405, 357.4181017943673, 129.44825810694724, 490.4446850170856, 490.4446850170856, 147.25819873544805, 189.08326999196993, 107.31637526839133, 147.25819873544805, 277.272284523134, 135.66060972280735, 116.03666862607915, 380.4695888770434, 298.6340019934001, 296.31001375229687, 234.40051387501347, 303.3368032308679, 361.6312332766962, 381.95870474451476, 627.2883040361144, 627.2883040361144, 239.02364975839862, 286.07385163895896, 167.51849280886478, 298.6340019934001, 192.81260332830294, 197.38502244855516, 219.3404304920078, 245.649629409089, 268.7415131590023, 154.92927926604594, 108.5813867407896, 241.84118490494708, 315.42036776119477, 341.05073527023916, 207.6673457129932, 434.50990901347194, 484.7308313654974, 128.44088377025506, 286.07385163895896, 167.51849280886478, 212.59203489720244, 256.43494327896633, 292.8578975283812, 85.56007407666469, 300.97621750018493, 480.95862610864765, 232.57639460420748, 112.02812994692015, 215.09800253180504, 141.61682573322432, 125.4655539023251, 105.24068867470233, 309.31958559164264, 371.6560203434465, 172.1622764913071, 179.02044192165667, 309.31958559164264, 151.93267574549122, 62.3530444922685, 168.83235663184237, 358.8169966923127, 308.1136628117644, 243.73796882667915, 68.48142221881122, 97.33172602285453, 276.1913023296018, 534.4563168515505, 506.013070509352, 334.45421691786504, 195.84896001735729, 431.12852607499315, 439.63177432431667, 498.16806499462353, 467.98558773246197, 500.11783967291126, 566.7077563764824, 479.0835461252901, 515.9932840552193, 394.08337555704935, 530.2971447966088, 530.2971447966088, 460.7301436924908, 547.1305831594982, 622.4067077254782, 513.9816168847873, 283.84760726337464, 378.98627852215554, 184.70316076024525, 494.29129035606036, 453.58718488624703, 542.872779181642, 164.9213593028435, 172.1622764913071, 389.49216516524257, 389.49216516524257, 424.4444978122241, 409.7818725681369, 221.92594607841514, 242.7877245401199, 792.9667171920373, 197.38502244855516, 330.5577072665093, 422.78974553481265, 226.30304315814422, 427.7734572673073, 146.11222634120347, 312.9657469853004, 329.26898489707213, 368.76377024718164, 251.47504249414803, 398.728705684109, 511.9777924614019, 269.79333770212673, 201.27809318052155, 206.85772745893686, 180.4245166495895, 557.921766975976, 171.49107938408403, 209.29610089202527, 211.76321709954237, 108.15806803146498, 303.3368032308679, 453.58718488624703, 553.5799853394149, 549.27199171561, 553.5799853394149, 571.1525003973912, 173.5125619611118, 354.636652059297, 373.11064087900144, 360.2213667100846, 315.42036776119477, 173.5125619611118, 156.14440466345445, 141.61682573322432, 370.20707083592094, 320.3875157950033, 416.23499808144635, 446.5549673918236, 334.45421691786504, 444.8140145626792, 314.1906602856942, 303.3368032308679, 345.0709334152821, 391.0165943978527, 482.8410449475606, 479.0835461252901, 398.728705684109, 231.6696651716735, 306.91244147925255, 444.8140145626792, 321.64147707721304, 196.6154911738957, 215.09800253180504, 148.99403178365867, 179.02044192165667, 302.15420510895507, 235.3179315471721, 106.89798837141365, 252.45928798182638, 151.3403463773431, 219.3404304920078, 431.12852607499315, 315.42036776119477, 990.7254970041143, 397.1742097789884, 394.08337555704935, 253.44738569742526, 210.11526266986428, 170.1565252164996, 109.86130976812342, 214.2594148922178, 205.2479478525057, 139.96693973881514, 180.4245166495895, 108.5813867407896, 113.34868209247811, 144.97517197488295, 149.57717793613415, 179.02044192165667, 287.1935130368687, 157.3690604074729, 243.73796882667915, 257.43860126944566, 272.97357619337527, 272.97357619337527, 292.8578975283812, 121.60538462411665, 279.4469580033028, 231.6696651716735, 231.6696651716735, 241.84118490494708, 243.73796882667915, 106.06610162605153, 116.94675549762589, 133.0366960079705, 142.1710983175922, 126.44959243764188, 283.84760726337464, 102.40200064874995, 146.6840934260825, 169.4931477924318, 121.13129015902297, 141.0647140528757, 109.43300111180854, 152.5273234252381, 153.7236100495016, 119.72006788311512, 159.8472625113303, 234.40051387501347, 353.25405478056325, 297.46973835596765, 150.1626064581185, 365.89402781973274, 321.64147707721304, 212.59203489720244, 617.56310029872, 294.00411093116844, 157.3690604074729, 383.45364884670335, 248.54526950703038, 161.10096012939127, 249.51804818745066, 176.24498722300464, 159.8472625113303, 141.0647140528757, 64.58413226217957, 118.3252868447224, 120.65904401308495, 345.0709334152821, 253.44738569742526, 108.5813867407896, 202.8567368308649, 286.07385163895896, 210.93763055625195, 135.66060972280735, 125.4655539023251, 163.63793110254423, 168.83235663184237, 354.636652059297, 509.98178020909006, 264.57506218047985, 327.98528678000895, 206.05126560732333, 247.5762833312724, 128.44088377025506, 106.06610162605153, 262.51612271179147, 414.6122516103204, 298.6340019934001, 202.06587336346374, 218.48530319325408, 127.94014021950323, 181.13067824168084, 154.92927926604594, 206.05126560732333, 136.19157033683516, 95.44916002730179, 56.55175575986736, 213.42409659600423, 210.93763055625195, 62.3530444922685, 166.21485355527525, 424.4444978122241, 119.2533235714264, 406.59292473597867, 281.63868765896893, 132.51803508331912, 137.25973406168575, 124.48917321564447, 125.9566121960733, 254.43935071813848, 411.38571347022946, 308.1136628117644, 136.72460907194932, 163.63793110254423, 74.62682298320091, 296.31001375229687, 210.11526266986428, 106.48123261036584, 291.71615278815983, 247.5762833312724, 167.51849280886478, 167.51849280886478, 194.3248513188351, 334.45421691786504, 184.70316076024525, 154.32526724792663, 486.6280141983472, 469.8172315347884, 203.65069565035918, 248.54526950703038, 250.49463421597682, 305.71590326490076, 457.14471326890896, 198.15756558346607, 258.44618746623325, 386.4611130887752, 257.43860126944566, 315.42036776119477, 280.5406824728656, 358.8169966923127, 325.4328850285552, 299.8028224298701, 466.1610848325832, 466.1610848325832, 99.63988138921447, 605.6183486721279, 605.6183486721279, 450.0573414090999, 132.51803508331912, 183.98307137873957, 376.03698390417554, 315.42036776119477, 439.63177432431667, 269.79333770212673, 358.8169966923127, 122.08133464248264, 94.33714363029665, 180.4245166495895, 180.4245166495895, 154.92927926604594, 289.4459996766781, 130.4635333784854, 143.28616006164455, 139.42126035437516, 381.95870474451476, 657.3924983001216, 361.6312332766962, 197.38502244855516, 162.3644906197382, 118.3252868447224, 357.4181017943673, 81.00664999449546, 209.29610089202527, 334.45421691786504, 223.66653284614108, 282.74099031586354, 282.74099031586354, 374.5709546383618, 471.65604418826433, 378.98627852215554, 286.07385163895896, 370.20707083592094, 243.73796882667915, 437.9178124670295, 148.99403178365867, 170.1565252164996, 342.38557010684895, 159.8472625113303, 205.2479478525057, 324.16414244755515, 134.60489048229144, 319.1384432445317, 292.8578975283812, 377.5087510463025, 384.95444399467493, 309.31958559164264, 198.15756558346607, 198.15756558346607, 243.73796882667915, 457.14471326890896, 378.98627852215554, 305.71590326490076, 389.49216516524257, 389.49216516524257, 346.4215028625498, 246.61107487460225, 316.65488817567297, 329.26898489707213, 247.5762833312724, 191.3121237449989, 150.1626064581185, 302.15420510895507, 302.15420510895507, 364.467542192173, 255.43519818017026, 254.43935071813848, 389.49216516524257, 389.49216516524257, 580.1468435322063, 276.1913023296018, 218.48530319325408, 245.649629409089, 281.63868765896893, 281.63868765896893, 236.23893989057657, 284.95855538715824, 446.5549673918236, 365.89402781973274, 162.99996726850102, 108.5813867407896, 185.4260684919039, 228.97063181940615, 299.8028224298701, 277.272284523134, 133.0366960079705, 195.84896001735729, 168.83235663184237, 127.44134888284603, 370.20707083592094, 220.19890466437903, 161.10096012939127, 302.15420510895507, 156.7555365841094, 509.98178020909006, 509.98178020909006, 179.7211081206401, 221.06073880966076, 292.8578975283812, 199.71173463253027, 231.6696651716735, 159.8472625113303, 128.94358717585138, 160.47288700783744, 131.48677155266952, 186.15180560442604, 207.6673457129932, 272.97357619337527, 496.225891767423, 153.7236100495016, 234.40051387501347, 262.51612271179147, 192.0608982264211, 287.1935130368687, 238.09178486008656, 164.27839185643572, 125.9566121960733, 124.97641006343288, 333.1503034969411, 380.4695888770434, 195.84896001735729, 289.4459996766781, 275.1145344862516, 183.26578935968087, 220.19890466437903, 317.89424036643936, 403.4287934927351, 553.5799853394149, 192.81260332830294, 270.8492789701163, 326.70659332761613, 180.4245166495895, 189.08326999196993, 240.8983354780937, 137.7969534714142, 270.8492789701163, 158.60332124553608, 253.44738569742526, 176.24498722300464, 202.8567368308649, 152.5273234252381, 172.1622764913071, 172.1622764913071, 215.9398723106141, 342.38557010684895, 73.75739416067401, 210.11526266986428, 338.3966573919768, 226.30304315814422, 114.68480048885151, 231.6696651716735, 60.67117749756472, 61.38635013907805, 266.6501501115634, 115.13366410675533, 106.89798837141365, 167.51849280886478, 224.54193890439277, 239.95916187280093, 132.51803508331912, 189.8233209864514, 168.83235663184237, 121.13129015902297, 181.13067824168084, 259.4577172438939, 194.3248513188351, 218.48530319325408, 156.7555365841094, 122.55914747655109, 122.08133464248264, 257.43860126944566, 162.3644906197382, 422.78974553481265, 401.85597368179066, 82.92766733936132, 254.43935071813848, 284.95855538715824, 256.43494327896633, 350.5050099164416, 317.89424036643936, 291.71615278815983, 241.84118490494708, 347.77735828917787, 102.40200064874995, 146.6840934260825, 133.0366960079705, 199.71173463253027, 85.89494673838159, 99.63988138921447, 157.98498549518746, 308.1136628117644, 86.90744918987926, 86.90744918987926, 271.90935307537654, 179.7211081206401, 236.23893989057657, 342.38557010684895, 250.49463421597682, 398.728705684109, 411.38571347022946, 268.7415131590023, 223.66653284614108, 321.64147707721304, 149.57717793613415, 275.1145344862516, 268.7415131590023, 90.36944839098756, 123.03883041717654, 330.5577072665093, 270.8492789701163, 69.83209724580306, 302.15420510895507, 159.2240770935855, 397.1742097789884, 234.40051387501347, 151.93267574549122, 148.4131591025766, 158.60332124553608, 168.83235663184237, 242.7877245401199, 185.4260684919039, 327.98528678000895, 157.3690604074729, 82.92766733936132, 200.49338426145144, 146.11222634120347, 144.4099673433467, 85.2265069597304, 38.867415430562446, 77.59961765106212, 106.48123261036584, 157.98498549518746, 397.1742097789884, 174.8734378511126, 131.48677155266952, 150.7503262825534, 291.71615278815983, 119.72006788311512, 216.78503707457477, 304.5240299109616, 409.7818725681369, 460.7301436924908, 329.26898489707213, 374.5709546383618, 384.95444399467493, 335.7632337116253, 441.35244443864343, 309.31958559164264, 492.3642312408232, 172.83610058973207, 303.3368032308679, 104.4216991379013, 205.2479478525057, 295.1548104863723, 416.23499808144635, 211.76321709954237, 75.21212841006654, 157.3690604074729, 109.43300111180854, 311.7456091692961, 446.5549673918236, 446.5549673918236, 506.013070509352, 251.47504249414803, 353.25405478056325, 334.45421691786504, 469.8172315347884, 335.7632337116253, 153.7236100495016, 411.38571347022946, 317.89424036643936, 304.5240299109616, 71.48811770889668, 204.44776193681219, 204.44776193681219, 138.33627549846275, 232.57639460420748, 175.55787313507827, 150.1626064581185, 266.6501501115634, 295.1548104863723, 237.16355295871682, 177.62729378994385, 380.4695888770434, 154.92927926604594, 357.4181017943673, 320.3875157950033, 555.7466361419574, 331.8514735526883, 315.42036776119477, 206.85772745893686, 345.0709334152821, 419.4995696321248, 357.4181017943673, 168.17414165184545, 304.5240299109616, 339.72110446147576, 339.72110446147576, 424.4444978122241, 381.95870474451476, 257.43860126944566, 417.86409580268446, 426.1057266069316, 403.4287934927351, 195.84896001735729, 132.51803508331912, 59.49769069799599, 254.43935071813848, 195.08541728258788, 135.13171912803952, 197.38502244855516, 186.15180560442604, 181.13067824168084, 186.88038317169537, 406.59292473597867, 490.4446850170856, 376.03698390417554, 186.15180560442604, 26.557093177483523, 21.297586362924953, 16.88071548801052, 13.301683750542033, 21.090614371389922, 19.053782246709744, 21.464625403032095, 25.094689498803355, 25.992616060150944, 19.697047472120772, 23.75917448515314, 18.648789727196185, 18.110364070885183, 22.319678873017494, 19.774139536218833, 26.922671813996086, 18.758380733362646, 21.131847230940878, 22.189281900071105, 20.84490127137742, 22.84897480427519, 24.086237637543274, 22.893645317537526, 18.145770440989054, 24.70563239932894, 20.8042283062039, 25.1437504597293, 21.675267738479917, 18.79505399780919, 22.40703562619394, 21.802644068801417, 20.401838105461145, 17.348666159108166, 17.348666159108166, 22.626926270737744, 20.124804822565828, 19.735555861572347, 23.25416777248505, 16.521912001744084, 18.758380733362646, 18.359667040887704, 19.581972547389178, 21.973644325590612, 25.1437504597293, 25.1437504597293, 20.44172438427552, 20.602050815688813, 20.967397891114594, 20.92648590822237, 23.620367629992042, 21.802644068801417, 24.80232751057381, 22.450842133517945, 20.8042283062039, 24.417803063601138, 22.27612829392538, 23.80562446936611, 23.482371718117374, 27.028044172168716, 19.89034374461794, 20.8856537535641, 21.675267738479917, 19.697047472120772, 19.165753221675935, 22.494734284035275, 20.401838105461145, 19.543763832601815, 21.131847230940878, 24.086237637543274, 23.073202403172917, 17.6218678981495, 18.758380733362646, 23.898797049169467, 24.657426302639607, 22.494734284035275, 22.494734284035275, 23.028181534732802, 24.609314266596865, 23.25416777248505, 22.27612829392538, 20.763634702888442, 16.94678474038272, 19.165753221675935, 16.01358620918395, 28.214717811314713, 20.967397891114594, 18.216790978454146, 24.086237637543274, 15.98234019723552, 16.076261613516785, 16.23402543319486, 22.016603538363892, 21.973644325590612, 20.4816886421048, 19.735555861572347, 16.74934871787156, 17.864446345871784, 19.505629671380603, 28.1047190510635, 23.574279122424027, 19.46756991825522, 20.44172438427552, 26.922671813996086, 25.790339917193062, 27.561124577174567, 26.401940203271806, 28.1047190510635, 20.322298876557472, 19.581972547389178, 23.80562446936611, 19.543763832601815, 26.09434828302453, 15.43022309819552, 19.46756991825522, 19.20322291302118, 20.124804822565828, 20.124804822565828, 22.98324851186175, 22.494734284035275, 18.648789727196185, 24.370158584949497, 24.370158584949497, 21.297586362924953, 20.203571081597193, 28.547303976590324, 21.887977204970085, 22.893645317537526, 16.202349296067883, 21.887977204970085, 19.581972547389178, 23.992334296139543, 21.506589466713315, 20.124804822565828, 21.548635571548648, 21.802644068801417, 24.948080241282657, 22.319678873017494, 21.380942759123343, 19.128356641900336, 21.464625403032095, 19.20322291302118, 21.131847230940878, 17.45061689198871, 25.740017318702005, 21.380942759123343, 22.319678873017494, 23.390822626823073, 21.297586362924953, 21.008389858307645, 21.845268970328604, 20.521731031400577, 22.319678873017494, 15.734556979186685, 20.723120306578686, 21.802644068801417, 25.63966650134361, 21.506589466713315, 22.84897480427519, 23.52828054362271, 20.68268496272443, 26.401940203271806, 23.71281513502692, 25.44013703364339, 27.61500750152876, 18.039758452611142, 18.181246031735814, 21.131847230940878, 23.4365524707797, 18.6852487375444, 15.250455553989443, 20.602050815688813, 18.868615759264884, 20.36202965350754, 24.417803063601138, 19.581972547389178, 18.431524745974645, 23.898797049169467, 22.36331459495464, 17.89937193856263, 22.938403163153815, 18.39556080667227, 23.992334296139543, 23.80562446936611, 20.8042283062039, 19.091033031037714, 29.511075247281195, 25.489873356103875, 16.04489330813532, 19.20322291302118, 17.348666159108166, 20.085536923187668, 19.091033031037714, 20.24306974171993, 20.56185170491249, 27.83159468972004, 21.380942759123343, 23.620367629992042, 24.086237637543274, 15.642631884188171, 20.967397891114594, 16.8149038152427, 25.790339917193062, 28.436008585761353, 25.539706914745466, 18.79505399780919, 18.83179895971962, 21.590763877931302, 19.091033031037714, 18.83179895971962, 21.632974546568068, 18.039758452611142, 25.992616060150944, 17.416566918622852, 19.353835657005096, 17.829588900657228, 25.992616060150944, 21.008389858307645, 31.784678801853513, 15.310144149387147, 17.934365811960372, 22.059646737841184, 22.016603538363892, 21.973644325590612, 21.214554942517648, 21.632974546568068, 24.657426302639607, 18.576084986220376, 18.79505399780919, 24.948080241282657, 22.10277408821932, 18.503663694167408, 28.380523708580828, 23.992334296139543, 23.073202403172917, 18.868615759264884, 21.973644325590612, 20.8856537535641, 20.8856537535641, 20.046345644076645, 22.538712245181248, 19.016604146815915, 24.850816893992036, 18.90550453688997, 19.429584428039178, 24.70563239932894, 24.80232751057381, 19.658614220966218, 23.20879380099744, 26.975306541751994, 40.892124929554, 15.85796463907484, 20.085536923187668, 18.979498589532987, 30.151962130172098, 22.582776184718522, 26.09434828302453, 19.543763832601815, 27.4536739354601, 25.1437504597293, 23.80562446936611, 13.912844803302042, 22.059646737841184, 23.299630451611073, 37.45150421083523, 19.20322291302118, 22.84897480427519, 21.506589466713315, 20.92648590822237, 18.216790978454146, 18.216790978454146, 18.539838978246124, 21.008389858307645, 22.715485556228362, 16.554212894695087, 19.20322291302118, 17.934365811960372, 24.18050850623376, 19.46756991825522, 22.938403163153815, 20.085536923187668, 19.31607208722833, 20.763634702888442, 18.539838978246124, 24.22778221261098, 27.61500750152876, 22.84897480427519, 20.723120306578686, 18.323843311697065, 20.8042283062039, 21.008389858307645, 25.790339917193062, 20.68268496272443, 22.626926270737744, 21.33922385967176, 21.33922385967176, 25.689792910615846, 24.370158584949497, 29.053548433146837, 22.189281900071105, 28.827451925481085, 24.039240115615065, 20.401838105461145, 26.24769367033534, 23.898797049169467, 18.503663694167408, 23.574279122424027, 21.930768935644906, 20.92648590822237, 22.059646737841184, 19.391673055829287, 21.802644068801417, 19.96819234893607, 23.20879380099744, 19.505629671380603, 21.173160702117308, 20.322298876557472, 18.979498589532987, 24.18050850623376, 22.319678873017494, 22.319678873017494, 22.319678873017494, 24.18050850623376, 22.626926270737744, 23.4365524707797, 20.007230835729846, 22.40703562619394, 22.40703562619394, 22.40703562619394, 21.506589466713315, 21.380942759123343, 21.464625403032095, 22.059646737841184, 20.68268496272443, 23.25416777248505, 21.297586362924953, 18.79505399780919, 19.505629671380603, 23.4365524707797, 21.632974546568068, 19.89034374461794, 18.039758452611142, 18.28808948244362, 20.763634702888442, 21.131847230940878, 20.124804822565828, 23.992334296139543, 22.84897480427519, 18.39556080667227, 18.61240185635776, 18.721779026482544, 16.457499171202052, 16.202349296067883, 15.280270706875589, 19.20322291302118, 17.14654807625046, 10.139171107507485, 17.51891667538985, 16.170734966018113, 16.170734966018113, 25.63966650134361, 20.203571081597193, 20.124804822565828, 18.61240185635776, 18.648789727196185, 18.648789727196185, 20.4816886421048, 19.016604146815915, 19.091033031037714, 22.189281900071105, 19.091033031037714, 18.61240185635776, 27.240027749911206, 24.086237637543274, 22.715485556228362, 18.979498589532987, 24.80232751057381, 24.370158584949497, 23.71281513502692, 30.093129052017023, 18.216790978454146, 22.626926270737744, 22.626926270737744, 19.735555861572347, 22.319678873017494, 20.64232851707703, 23.666546242140512, 21.297586362924953, 21.090614371389922, 21.008389858307645, 21.008389858307645, 21.008389858307645, 21.008389858307645, 18.979498589532987, 21.214554942517648, 21.590763877931302, 22.059646737841184, 19.697047472120772, 21.506589466713315, 19.774139536218833, 25.045724266551254, 21.380942759123343, 21.131847230940878, 19.620255961497417, 21.42274322042474, 21.930768935644906, 19.46756991825522, 24.086237637543274, 24.039240115615065, 22.450842133517945, 22.059646737841184, 23.852165264858517, 22.145985754016134, 21.131847230940878, 18.00455893510183, 23.898797049169467, 21.845268970328604, 26.453556883236853, 21.33922385967176, 22.538712245181248, 25.489873356103875, 28.325147094540903, 21.930768935644906, 19.735555861572347, 24.322607071104724, 20.124804822565828, 27.72308958496738, 22.189281900071105, 22.84897480427519, 21.930768935644906, 23.25416777248505, 23.992334296139543, 23.852165264858517, 17.51891667538985, 23.345182011801924, 18.83179895971962, 23.852165264858517, 30.329152388418176, 22.232662691545976, 21.590763877931302, 24.70563239932894, 28.380523708580828, 18.503663694167408, 24.275148340672132, 19.353835657005096, 18.2524054167373, 19.697047472120772, 21.760102337787384, 25.045724266551254, 16.554212894695087, 17.6218678981495, 19.053782246709744, 21.675267738479917, 21.297586362924953, 19.391673055829287, 30.56702687878971, 29.858942463791767, 21.717643615002626, 22.450842133517945, 30.866995306309306, 17.829588900657228, 21.131847230940878, 24.80232751057381, 25.539706914745466, 22.84897480427519, 21.42274322042474, 19.620255961497417, 21.173160702117308, 23.898797049169467, 23.898797049169467, 22.189281900071105, 21.380942759123343, 18.075026786359576, 31.846818916590372, 24.322607071104724, 26.401940203271806, 17.484733434196265, 26.505274475545082, 17.046373787079762, 20.046345644076645, 18.83179895971962, 20.203571081597193, 21.464625403032095, 22.80439145296227, 22.715485556228362, 17.79479946994832, 21.632974546568068, 22.715485556228362, 26.2990087931472, 26.87013978811538, 21.930768935644906, 20.967397891114594, 20.322298876557472, 19.20322291302118, 20.24306974171993, 22.626926270737744, 20.124804822565828, 20.36202965350754, 22.27612829392538, 21.802644068801417, 18.431524745974645, 19.735555861572347, 25.39049775763493, 20.046345644076645, 20.046345644076645, 19.851533330124635, 24.513371642677637, 26.557093177483523, 19.851533330124635, 21.173160702117308, 22.84897480427519, 25.840760898054555, 25.489873356103875, 18.39556080667227, 22.98324851186175, 27.61500750152876, 23.390822626823073, 25.39049775763493, 23.118311288923124, 20.602050815688813, 19.581972547389178, 22.538712245181248, 30.329152388418176, 18.110364070885183, 16.329425896569767, 19.24076585887165, 20.967397891114594, 21.214554942517648, 23.620367629992042, 19.353835657005096, 20.44172438427552, 19.620255961497417, 20.56185170491249, 18.28808948244362, 21.090614371389922, 18.83179895971962, 19.812798643245316, 29.110349096271467, 22.84897480427519, 21.590763877931302, 22.319678873017494, 19.46756991825522, 20.763634702888442, 16.91371785382974, 16.04489330813532, 13.35374506875535, 17.969428099556087, 20.36202965350754, 23.028181534732802, 24.465540688808968, 21.214554942517648, 22.98324851186175, 22.67116267165818, 22.27612829392538, 20.763634702888442, 25.44013703364339, 16.554212894695087, 21.214554942517648, 21.590763877931302, 23.16350836406023, 23.94552000018515, 24.89940107578365, 26.609013186724894, 23.80562446936611, 15.765318566620541, 18.6852487375444, 22.059646737841184, 24.417803063601138, 20.322298876557472, 22.232662691545976, 21.845268970328604, 21.930768935644906, 24.039240115615065, 10.54306976753373, 24.753932740556596, 21.590763877931302, 23.390822626823073, 19.735555861572347, 22.582776184718522, 23.20879380099744, 28.65903496526287, 19.165753221675935, 19.543763832601815, 14.159568646729724, 22.319678873017494, 19.620255961497417, 20.602050815688813, 21.506589466713315, 23.390822626823073, 28.214717811314713, 25.44013703364339, 24.609314266596865, 20.763634702888442, 24.657426302639607, 21.008389858307645, 21.008389858307645, 24.70563239932894, 27.83159468972004, 20.56185170491249, 28.491601937740562, 23.073202403172917, 19.165753221675935, 20.322298876557472, 17.934365811960372, 26.14536361045317, 20.282645623050005, 19.774139536218833, 18.110364070885183, 25.34095533871963, 22.016603538363892, 18.467558995986426, 24.22778221261098, 21.845268970328604, 19.165753221675935, 24.513371642677637, 19.46756991825522, 29.511075247281195, 22.450842133517945, 27.40010580820802, 18.503663694167408, 18.503663694167408, 18.503663694167408, 22.494734284035275, 17.690837939037834, 21.590763877931302, 21.632974546568068, 19.053782246709744, 19.053782246709744, 22.938403163153815, 17.14654807625046, 21.675267738479917, 22.059646737841184, 21.464625403032095, 19.016604146815915, 19.016604146815915, 19.016604146815915, 19.016604146815915, 22.145985754016134, 21.049461966173517, 24.322607071104724, 16.61900425204837, 25.192907336481866, 22.582776184718522, 24.22778221261098, 24.22778221261098, 23.482371718117374, 28.15966472087403, 24.513371642677637, 23.073202403172917, 25.63966650134361, 23.992334296139543, 23.028181534732802, 20.84490127137742, 23.482371718117374, 24.9968545761854, 24.133327041205547, 30.56702687878971, 21.131847230940878, 21.090614371389922, 18.503663694167408, 20.36202965350754, 20.64232851707703, 18.90550453688997, 22.67116267165818, 20.44172438427552, 25.740017318702005, 26.2990087931472, 21.380942759123343, 20.763634702888442, 24.22778221261098, 21.090614371389922, 22.36331459495464, 21.930768935644906, 19.429584428039178, 19.429584428039178, 19.053782246709744, 23.574279122424027, 13.617124090346826, 18.075026786359576, 23.620367629992042, 19.24076585887165, 24.850816893992036, 21.42274322042474, 21.297586362924953, 24.513371642677637, 27.668995768810042, 30.21091022897123, 24.275148340672132, 17.829588900657228, 25.94189877662824, 24.417803063601138, 26.14536361045317, 25.489873356103875, 18.90550453688997, 23.94552000018515, 22.538712245181248, 19.735555861572347, 27.995149135337257, 18.942465433314446, 20.763634702888442, 20.68268496272443, 19.774139536218833, 15.888967492629165, 23.25416777248505, 18.145770440989054, 15.673213754973451, 25.992616060150944, 23.574279122424027, 18.979498589532987, 20.967397891114594, 12.569207830853442, 29.396022468352484, 17.864446345871784, 16.74934871787156, 21.131847230940878, 19.20322291302118, 19.505629671380603, 24.22778221261098, 26.24769367033534, 14.495353140571094, 17.079700020438295, 25.840760898054555, 27.40010580820802, 21.131847230940878, 20.8042283062039, 20.401838105461145, 20.282645623050005, 17.213657767955564, 19.96819234893607, 21.590763877931302, 23.16350836406023, 21.42274322042474, 28.491601937740562, 22.75989509352673, 16.651494963610144, 17.484733434196265, 21.33922385967176, 17.314815113938415, 16.23402543319486, 13.750755251693265, 21.802644068801417, 20.401838105461145, 19.353835657005096, 19.128356641900336, 20.64232851707703, 21.548635571548648, 21.090614371389922, 23.482371718117374, 25.840760898054555, 22.715485556228362, 21.33922385967176, 22.319678873017494, 22.494734284035275, 26.713157919740368, 26.09434828302453, 21.42274322042474, 30.806767041912167, 23.073202403172917, 22.938403163153815, 18.539838978246124, 22.232662691545976, 25.840760898054555, 17.79479946994832, 20.007230835729846, 23.992334296139543, 14.495353140571094, 18.431524745974645, 22.319678873017494, 15.310144149387147, 15.340075995482293, 16.04489330813532, 19.053782246709744, 29.626578330054702, 20.007230835729846, 28.883810562338418, 21.675267738479917, 24.948080241282657, 20.401838105461145, 20.44172438427552, 16.716667072215767, 20.64232851707703, 23.028181534732802, 18.83179895971962, 17.314815113938415, 23.345182011801924, 22.36331459495464, 18.145770440989054, 16.8149038152427, 24.89940107578365, 25.539706914745466, 24.417803063601138, 24.18050850623376, 19.391673055829287, 23.482371718117374, 21.590763877931302, 18.721779026482544, 17.829588900657228, 18.2524054167373, 22.538712245181248, 25.63966650134361, 22.494734284035275, 24.370158584949497, 20.124804822565828, 22.84897480427519, 20.4816886421048, 19.96819234893607, 15.43022309819552, 27.4536739354601, 20.84490127137742, 19.96819234893607, 32.92198346312245, 17.180070153709277, 19.505629671380603, 19.128356641900336, 25.094689498803355, 17.14654807625046, 21.802644068801417, 22.016603538363892, 22.189281900071105, 23.898797049169467, 25.24216031657971, 9.846434155787346, 12.643071719773257, 16.782094257242417, 20.84490127137742, 20.602050815688813, 17.247311047115943, 22.98324851186175, 25.192907336481866, 20.521731031400577, 16.74934871787156, 25.58963789966816, 21.008389858307645, 17.28103011956753, 19.697047472120772, 18.868615759264884, 14.4106677747075, 13.198169226352599, 19.353835657005096, 21.845268970328604, 19.581972547389178, 22.67116267165818, 20.68268496272443, 20.68268496272443, 16.74934871787156, 21.675267738479917, 23.898797049169467, 18.6852487375444, 22.40703562619394, 21.930768935644906, 16.107691244988956, 19.165753221675935, 23.71281513502692, 12.968197316969134, 21.548635571548648, 20.44172438427552, 16.489674134905655, 20.967397891114594, 18.648789727196185, 23.992334296139543, 16.91371785382974, 16.139182322446413, 14.382549407491597, 20.967397891114594, 15.280270706875589, 14.580536166314408, 14.242778409807016, 20.36202965350754, 18.359667040887704, 20.046345644076645, 26.87013978811538, 22.494734284035275, 26.453556883236853, 23.620367629992042, 19.353835657005096, 20.36202965350754, 14.926303617194517, 25.94189877662824, 23.482371718117374, 26.2990087931472, 21.973644325590612, 20.36202965350754, 13.804574186067095, 20.723120306578686, 19.812798643245316, 23.4365524707797, 20.322298876557472, 18.145770440989054, 21.887977204970085, 26.24769367033534, 28.827451925481085, 27.940524471445343, 24.948080241282657, 25.489873356103875, 24.039240115615065, 17.484733434196265, 23.25416777248505, 14.694892728788941, 22.232662691545976, 20.203571081597193, 20.24306974171993, 22.80439145296227, 22.27612829392538, 19.812798643245316, 16.716667072215767, 18.79505399780919, 24.657426302639607, 20.68268496272443, 16.94678474038272, 17.829588900657228, 19.774139536218833, 21.717643615002626, 22.67116267165818, 22.232662691545976, 24.22778221261098, 20.602050815688813, 27.668995768810042, 22.494734284035275, 21.973644325590612, 17.934365811960372, 18.6852487375444, 18.79505399780919, 16.489674134905655, 23.345182011801924, 23.345182011801924, 21.380942759123343, 27.133828946090556, 14.049378009312553, 24.417803063601138, 22.319678873017494, 21.675267738479917, 19.24076585887165, 19.505629671380603, 23.52828054362271, 19.128356641900336, 21.930768935644906, 18.6852487375444, 20.68268496272443, 22.189281900071105, 16.61900425204837, 28.380523708580828, 8.912902981198737, 21.506589466713315, 19.735555861572347, 27.346642204097876, 28.827451925481085, 21.675267738479917, 21.802644068801417, 25.39049775763493, 18.503663694167408, 21.802644068801417, 24.322607071104724, 21.717643615002626, 20.24306974171993, 22.36331459495464, 27.4536739354601, 21.464625403032095, 17.314815113938415, 27.61500750152876, 23.992334296139543, 23.118311288923124, 21.632974546568068, 17.6218678981495, 21.464625403032095, 20.007230835729846, 19.353835657005096, 25.790339917193062, 21.131847230940878, 22.938403163153815, 19.581972547389178, 15.85796463907484, 17.348666159108166, 14.159568646729724, 26.453556883236853, 17.6218678981495, 16.554212894695087, 20.282645623050005, 21.090614371389922, 21.632974546568068, 17.690837939037834, 18.942465433314446, 18.758380733362646, 18.00455893510183, 15.734556979186685, 21.548635571548648, 21.802644068801417, 23.299630451611073, 22.27612829392538, 25.740017318702005, 23.852165264858517, 17.587483776585056, 19.20322291302118, 26.66103470132838, 19.165753221675935, 20.401838105461145, 18.90550453688997, 23.992334296139543, 19.20322291302118, 21.548635571548648, 19.96819234893607, 18.431524745974645, 20.4816886421048, 22.145985754016134, 21.973644325590612, 26.09434828302453, 19.46756991825522, 21.590763877931302, 18.28808948244362, 27.40010580820802, 19.851533330124635, 24.370158584949497, 18.359667040887704, 20.64232851707703, 22.450842133517945, 21.214554942517648, 20.8042283062039, 25.1437504597293, 25.489873356103875, 14.926303617194517, 24.753932740556596, 15.888967492629165, 21.973644325590612, 31.10908815096766, 14.609041604294383, 21.717643615002626, 26.401940203271806, 23.20879380099744, 16.076261613516785, 21.930768935644906, 26.557093177483523, 24.609314266596865, 23.16350836406023, 23.620367629992042, 24.9968545761854, 20.521731031400577, 23.52828054362271, 25.1437504597293, 18.39556080667227, 22.893645317537526, 23.25416777248505, 24.609314266596865, 20.203571081597193, 21.845268970328604, 12.692555303381242, 22.36331459495464, 20.203571081597193, 25.790339917193062, 19.774139536218833, 19.505629671380603, 17.046373787079762, 26.14536361045317, 22.10277408821932, 32.159346537604755, 22.582776184718522, 19.016604146815915, 18.942465433314446, 20.322298876557472, 24.086237637543274, 20.44172438427552, 18.79505399780919, 21.845268970328604, 30.034410770075546, 17.28103011956753, 16.91371785382974, 17.587483776585056, 20.763634702888442, 21.930768935644906, 19.128356641900336, 18.61240185635776, 19.620255961497417, 27.293282919182133, 27.028044172168716, 22.715485556228362, 21.930768935644906, 14.723621737689907, 19.391673055829287, 21.297586362924953, 30.329152388418176, 18.90550453688997, 22.450842133517945, 25.34095533871963, 21.887977204970085, 22.67116267165818, 22.450842133517945, 21.090614371389922, 20.723120306578686, 25.34095533871963, 19.128356641900336, 16.39333746248769, 23.992334296139543, 20.8042283062039, 20.085536923187668, 24.561296107667808, 23.299630451611073, 24.039240115615065, 23.898797049169467, 21.131847230940878, 16.61900425204837, 20.56185170491249, 20.56185170491249, 15.520899957093135, 21.297586362924953, 18.075026786359576, 20.36202965350754, 21.214554942517648, 22.75989509352673, 22.450842133517945, 22.494734284035275, 22.189281900071105, 24.465540688808968, 25.992616060150944, 20.401838105461145, 26.09434828302453, 22.98324851186175, 21.33922385967176, 21.548635571548648, 21.33922385967176, 21.760102337787384, 19.96819234893607, 20.007230835729846, 19.016604146815915, 21.590763877931302, 17.969428099556087, 18.539838978246124, 20.322298876557472, 26.196478674560588, 20.44172438427552, 22.715485556228362, 20.401838105461145, 19.429584428039178, 18.503663694167408, 22.319678873017494, 22.319678873017494, 21.506589466713315, 26.14536361045317, 22.232662691545976, 22.319678873017494, 21.049461966173517, 22.75989509352673, 13.85860376179437, 23.898797049169467, 19.091033031037714, 19.091033031037714, 21.464625403032095, 16.39333746248769, 12.892433965519283, 27.186876493132687, 18.110364070885183, 19.278382202442366, 22.893645317537526, 23.345182011801924, 19.812798643245316, 17.934365811960372, 22.67116267165818, 21.380942759123343, 21.930768935644906, 22.67116267165818, 19.505629671380603, 23.75917448515314, 23.852165264858517, 19.96819234893607, 19.278382202442366, 21.973644325590612, 24.513371642677637, 17.14654807625046, 25.840760898054555, 24.9968545761854, 26.505274475545082, 20.92648590822237, 25.740017318702005, 18.90550453688997, 23.71281513502692, 20.046345644076645, 26.505274475545082, 22.582776184718522, 20.68268496272443, 22.80439145296227, 24.609314266596865, 25.58963789966816, 20.24306974171993, 16.39333746248769, 21.760102337787384, 18.79505399780919, 16.139182322446413, 14.83910053420958, 17.76007792103387, 22.189281900071105, 22.10277408821932, 25.44013703364339, 22.938403163153815, 20.723120306578686, 24.753932740556596, 25.840760898054555, 19.505629671380603, 17.416566918622852, 16.74934871787156, 17.314815113938415, 18.39556080667227, 22.84897480427519, 30.507383918455513, 17.046373787079762, 22.319678873017494, 22.40703562619394, 18.868615759264884, 14.495353140571094, 14.552086348683558, 19.24076585887165, 16.489674134905655, 19.851533330124635, 19.735555861572347, 22.145985754016134, 22.67116267165818, 23.852165264858517, 18.721779026482544, 20.203571081597193, 19.46756991825522, 19.46756991825522, 19.46756991825522, 19.46756991825522, 17.348666159108166, 23.390822626823073, 20.203571081597193, 23.620367629992042, 25.094689498803355, 16.94678474038272, 21.049461966173517, 20.521731031400577, 17.013112580497378, 19.543763832601815, 17.969428099556087, 19.20322291302118, 19.505629671380603, 21.632974546568068, 16.94678474038272, 22.10277408821932, 14.666219776472907, 18.942465433314446, 19.128356641900336, 21.930768935644906, 26.09434828302453, 18.2524054167373, 24.275148340672132, 18.90550453688997, 23.20879380099744, 18.979498589532987, 17.213657767955564, 20.203571081597193, 19.96819234893607, 19.091033031037714, 18.61240185635776, 16.74934871787156, 17.28103011956753, 17.14654807625046, 18.942465433314446, 16.782094257242417, 20.92648590822237, 25.489873356103875, 20.521731031400577, 25.539706914745466, 21.049461966173517, 22.27612829392538, 21.256030110048442, 19.89034374461794, 22.059646737841184, 16.716667072215767, 16.716667072215767, 15.673213754973451, 22.232662691545976, 16.684049195604416, 22.10277408821932, 20.085536923187668, 19.353835657005096, 21.930768935644906, 26.196478674560588, 21.42274322042474, 21.049461966173517, 20.84490127137742, 20.84490127137742, 16.979916273809664, 21.802644068801417, 21.675267738479917, 20.967397891114594, 23.620367629992042, 27.346642204097876, 21.33922385967176, 25.045724266551254, 18.323843311697065, 27.133828946090556, 22.538712245181248, 21.802644068801417, 22.016603538363892, 18.868615759264884, 22.582776184718522, 25.1437504597293, 22.938403163153815, 10.877742727456024, 18.216790978454146, 21.802644068801417, 19.543763832601815, 19.812798643245316, 20.8856537535641, 21.33922385967176, 20.124804822565828, 21.675267738479917, 20.8042283062039, 21.675267738479917, 20.521731031400577, 20.521731031400577, 20.521731031400577, 20.521731031400577, 20.521731031400577, 20.521731031400577, 20.521731031400577, 20.521731031400577, 20.521731031400577, 19.128356641900336, 16.651494963610144, 22.538712245181248, 14.83910053420958, 19.929230034775262, 21.173160702117308, 21.173160702117308, 21.173160702117308, 21.173160702117308, 21.173160702117308, 21.173160702117308, 21.173160702117308, 21.173160702117308, 18.216790978454146, 20.44172438427552, 23.25416777248505, 21.131847230940878, 19.391673055829287, 20.723120306578686, 29.626578330054702, 19.929230034775262, 23.71281513502692, 24.753932740556596, 20.4816886421048, 30.388446900124176, 23.390822626823073, 22.450842133517945, 22.059646737841184, 21.802644068801417, 22.67116267165818, 21.131847230940878, 21.214554942517648, 24.039240115615065, 25.2915095879081, 26.196478674560588, 25.992616060150944, 20.203571081597193, 20.967397891114594, 20.967397891114594, 23.992334296139543, 20.203571081597193, 18.6852487375444, 20.8042283062039, 21.33922385967176, 21.464625403032095, 22.189281900071105, 24.80232751057381, 20.36202965350754, 18.467558995986426, 25.39049775763493, 25.63966650134361, 22.626926270737744, 17.079700020438295, 20.602050815688813, 15.131775567308885, 26.817710263716034, 25.489873356103875, 19.812798643245316, 22.40703562619394, 20.44172438427552, 20.967397891114594, 21.464625403032095, 29.33866439394716, 20.68268496272443, 23.4365524707797, 20.8856537535641, 20.322298876557472, 20.723120306578686, 15.703855414343392, 18.323843311697065, 20.8042283062039, 28.214717811314713, 20.203571081597193, 23.482371718117374, 20.68268496272443, 22.450842133517945, 22.938403163153815, 16.265763498233948, 23.52828054362271, 20.4816886421048, 23.073202403172917, 22.40703562619394, 21.33922385967176, 24.039240115615065, 21.802644068801417, 17.247311047115943, 17.079700020438295, 22.10277408821932, 21.930768935644906, 31.846818916590372, 26.817710263716034, 25.94189877662824, 19.505629671380603, 25.2915095879081, 21.675267738479917, 22.145985754016134, 20.68268496272443, 25.891280453627324, 24.133327041205547, 20.967397891114594, 24.039240115615065, 25.44013703364339, 22.938403163153815, 20.56185170491249, 17.829588900657228, 19.391673055829287, 24.18050850623376, 20.84490127137742, 17.969428099556087, 21.256030110048442, 17.51891667538985, 16.139182322446413, 22.538712245181248, 13.72392451825968, 22.059646737841184, 15.250455553989443, 21.33922385967176, 19.46756991825522, 19.128356641900336, 12.76714405293921, 22.059646737841184, 22.938403163153815, 22.938403163153815, 24.18050850623376, 24.89940107578365, 22.145985754016134, 25.045724266551254, 23.992334296139543, 21.760102337787384, 11.57929685891098, 26.2990087931472, 32.22221914035047, 19.391673055829287, 19.165753221675935, 26.817710263716034, 22.715485556228362, 17.51891667538985, 18.039758452611142, 16.265763498233948, 15.01401915561195, 23.390822626823073, 19.053782246709744, 22.75989509352673, 21.380942759123343, 14.666219776472907, 13.121062323276538, 26.2990087931472, 21.845268970328604, 19.20322291302118, 27.240027749911206, 19.016604146815915, 19.46756991825522, 25.63966650134361, 12.134998917818166, 20.68268496272443, 21.632974546568068, 21.632974546568068, 12.398532576458457, 11.556703116117722, 17.969428099556087, 22.626926270737744, 25.1437504597293, 20.16414949200632, 19.543763832601815, 19.24076585887165, 20.085536923187668, 18.83179895971962, 25.34095533871963, 26.713157919740368, 22.10277408821932, 19.851533330124635, 20.967397891114594, 21.632974546568068, 21.131847230940878, 27.293282919182133, 20.763634702888442, 20.763634702888442, 28.60311491479713, 23.073202403172917, 16.076261613516785, 29.110349096271467, 23.4365524707797, 19.016604146815915, 21.297586362924953, 21.802644068801417, 20.521731031400577, 18.90550453688997, 33.4404292647954, 25.44013703364339, 19.620255961497417, 23.852165264858517, 20.521731031400577, 21.173160702117308, 20.046345644076645, 20.602050815688813, 25.489873356103875, 23.20879380099744, 26.043432497666576, 15.673213754973451, 21.845268970328604, 21.214554942517648, 21.973644325590612, 18.61240185635776, 25.24216031657971, 16.782094257242417, 23.16350836406023, 23.16350836406023, 17.6218678981495, 23.028181534732802, 18.79505399780919, 21.42274322042474, 20.64232851707703, 25.1437504597293, 17.51891667538985, 19.46756991825522, 24.370158584949497, 25.790339917193062, 21.214554942517648, 20.56185170491249, 22.582776184718522, 23.4365524707797, 23.345182011801924, 21.506589466713315, 24.80232751057381, 31.169907463288272, 23.71281513502692, 21.214554942517648, 22.10277408821932, 17.046373787079762, 22.27612829392538, 15.703855414343392, 27.995149135337257, 13.617124090346826, 23.482371718117374, 22.67116267165818, 24.370158584949497, 24.22778221261098, 21.930768935644906, 24.513371642677637, 22.84897480427519, 20.4816886421048, 24.948080241282657, 25.44013703364339, 25.24216031657971, 18.359667040887704, 20.4816886421048, 22.189281900071105, 18.758380733362646, 20.763634702888442, 24.18050850623376, 19.016604146815915, 23.666546242140512, 27.133828946090556, 24.9968545761854, 19.929230034775262, 22.80439145296227, 20.967397891114594, 24.039240115615065, 22.538712245181248, 25.24216031657971, 18.323843311697065, 18.90550453688997, 21.548635571548648, 19.658614220966218, 19.053782246709744, 25.790339917193062, 32.159346537604755, 24.370158584949497, 18.721779026482544, 21.930768935644906, 18.979498589532987, 18.503663694167408, 22.80439145296227, 19.620255961497417, 19.581972547389178, 29.626578330054702, 16.8149038152427, 25.1437504597293, 21.008389858307645, 18.648789727196185, 20.763634702888442, 24.89940107578365, 26.14536361045317, 24.370158584949497, 19.658614220966218, 20.84490127137742, 24.275148340672132, 23.16350836406023, 19.543763832601815, 19.851533330124635, 27.346642204097876, 28.491601937740562, 20.967397891114594, 25.39049775763493, 19.31607208722833, 17.51891667538985, 18.539838978246124, 20.046345644076645, 23.20879380099744, 19.581972547389178, 22.319678873017494, 22.715485556228362, 22.80439145296227, 17.6218678981495, 25.34095533871963, 20.046345644076645, 17.725424121461643, 20.8042283062039, 20.007230835729846, 17.934365811960372, 19.96819234893607, 29.281418237700255, 22.40703562619394, 28.771203256660744, 17.829588900657228, 18.868615759264884, 17.89937193856263, 25.192907336481866, 18.6852487375444, 21.380942759123343, 18.323843311697065, 33.50580642730562, 15.703855414343392, 24.086237637543274, 22.494734284035275, 25.1437504597293, 19.505629671380603, 28.547303976590324, 17.046373787079762, 25.192907336481866, 28.996858600549206, 19.46756991825522, 19.697047472120772, 20.84490127137742, 17.348666159108166, 21.887977204970085, 22.319678873017494, 14.131940228876227, 17.553166745968216, 14.868111474214228, 22.715485556228362, 20.8042283062039, 14.75240691276831, 22.10277408821932, 27.4536739354601, 19.20322291302118, 16.265763498233948, 18.039758452611142, 18.2524054167373, 20.16414949200632, 20.521731031400577, 18.467558995986426, 20.124804822565828, 19.46756991825522, 24.9968545761854, 24.370158584949497, 17.690837939037834, 19.96819234893607, 20.763634702888442, 20.16414949200632, 20.124804822565828, 14.723621737689907, 11.738694709013188, 21.380942759123343, 18.359667040887704, 24.133327041205547, 14.4106677747075, 16.36135047271935, 16.684049195604416, 24.513371642677637, 19.24076585887165, 15.888967492629165, 10.730035606033903, 26.505274475545082, 24.133327041205547, 23.574279122424027, 23.118311288923124, 19.165753221675935, 23.71281513502692, 20.967397891114594, 18.90550453688997, 23.390822626823073, 15.220698576992893, 15.220698576992893, 17.046373787079762, 22.059646737841184, 23.028181534732802, 25.992616060150944, 24.561296107667808, 22.319678873017494, 18.075026786359576, 20.723120306578686, 19.391673055829287, 18.539838978246124, 25.24216031657971, 16.91371785382974, 18.539838978246124, 22.016603538363892, 20.203571081597193, 20.967397891114594, 16.202349296067883, 18.2524054167373, 26.401940203271806, 22.494734284035275, 21.590763877931302, 19.20322291302118, 19.543763832601815, 21.675267738479917, 19.774139536218833, 20.24306974171993, 17.113091407702502, 26.975306541751994, 24.80232751057381, 23.118311288923124, 25.58963789966816, 24.275148340672132, 23.666546242140512, 22.715485556228362, 21.930768935644906, 20.763634702888442, 21.380942759123343, 25.39049775763493, 22.27612829392538, 22.145985754016134, 27.080884906424416, 23.118311288923124, 22.893645317537526, 23.94552000018515, 21.590763877931302, 21.887977204970085, 20.36202965350754, 21.760102337787384, 20.967397891114594, 20.44172438427552, 19.851533330124635, 22.189281900071105, 19.543763832601815, 17.829588900657228, 18.145770440989054, 24.039240115615065, 37.160054292819765, 21.008389858307645, 15.490615283867873, 20.8856537535641, 21.214554942517648, 23.482371718117374, 23.75917448515314, 21.930768935644906, 21.802644068801417, 24.948080241282657, 20.322298876557472, 22.538712245181248, 23.80562446936611, 17.28103011956753, 25.840760898054555, 25.790339917193062, 17.348666159108166, 21.760102337787384, 24.465540688808968, 19.505629671380603, 17.079700020438295, 20.16414949200632, 17.213657767955564, 26.24769367033534, 21.131847230940878, 18.110364070885183, 21.675267738479917, 15.131775567308885, 16.170734966018113, 19.96819234893607, 16.8149038152427, 16.8149038152427, 22.319678873017494, 16.74934871787156, 22.494734284035275, 16.684049195604416, 21.380942759123343, 14.81014620089916, 21.802644068801417, 18.979498589532987, 18.942465433314446, 22.232662691545976, 19.091033031037714, 18.576084986220376, 23.992334296139543, 23.898797049169467, 22.98324851186175, 20.16414949200632, 19.658614220966218, 22.494734284035275, 22.36331459495464, 17.314815113938415, 18.2524054167373, 17.314815113938415, 23.852165264858517, 12.569207830853442, 23.574279122424027, 22.189281900071105, 17.079700020438295, 16.425386987895497, 21.973644325590612, 28.715064341305684, 22.938403163153815, 16.076261613516785, 20.967397891114594, 16.23402543319486, 21.33922385967176, 36.37020880080945, 19.053782246709744, 15.642631884188171, 20.085536923187668, 20.4816886421048, 18.942465433314446, 24.753932740556596, 21.42274322042474, 19.581972547389178, 19.929230034775262, 18.648789727196185, 24.657426302639607, 17.76007792103387, 28.60311491479713, 12.158723248329283, 21.090614371389922, 17.314815113938415, 24.322607071104724, 19.96819234893607, 20.4816886421048, 18.145770440989054, 24.086237637543274, 22.84897480427519, 28.65903496526287, 19.812798643245316, 19.812798643245316, 22.10277408821932, 23.482371718117374, 20.8042283062039, 20.44172438427552, 26.505274475545082, 18.868615759264884, 25.094689498803355, 23.992334296139543, 20.4816886421048, 19.96819234893607, 19.735555861572347, 18.6852487375444, 21.802644068801417, 15.520899957093135, 20.401838105461145, 21.256030110048442, 19.96819234893607, 23.4365524707797, 16.329425896569767, 18.648789727196185, 23.992334296139543, 23.71281513502692, 16.265763498233948, 26.922671813996086, 26.975306541751994, 24.70563239932894, 19.429584428039178, 23.390822626823073, 23.390822626823073, 19.96819234893607, 16.651494963610144, 15.04337206739713, 26.35042423874788, 19.543763832601815, 14.495353140571094, 21.802644068801417, 18.181246031735814, 16.139182322446413, 20.8856537535641, 21.632974546568068, 18.00455893510183, 23.852165264858517, 18.539838978246124, 24.18050850623376, 19.053782246709744, 16.36135047271935, 16.716667072215767, 17.656319241826594, 17.046373787079762, 17.51891667538985, 21.675267738479917, 25.63966650134361, 22.582776184718522, 27.133828946090556, 22.189281900071105, 22.582776184718522, 17.587483776585056, 20.322298876557472, 20.64232851707703, 20.967397891114594, 19.735555861572347, 18.503663694167408, 18.503663694167408, 16.716667072215767, 19.053782246709744, 19.053782246709744, 22.938403163153815, 20.8042283062039, 19.353835657005096, 24.513371642677637, 17.864446345871784, 20.92648590822237, 23.345182011801924, 19.278382202442366, 17.314815113938415, 20.8042283062039, 22.538712245181248, 18.83179895971962, 20.085536923187668, 12.398532576458457, 24.948080241282657, 20.8042283062039, 15.581647041956762, 15.190999662372045, 18.181246031735814, 20.36202965350754, 17.89937193856263, 20.68268496272443, 17.213657767955564, 19.20322291302118, 22.145985754016134, 16.425386987895497, 21.380942759123343, 29.684499285876434, 21.887977204970085, 21.506589466713315, 22.450842133517945, 35.87634357692165, 25.192907336481866, 21.049461966173517, 21.256030110048442, 23.75917448515314, 21.930768935644906, 18.868615759264884, 18.868615759264884, 21.506589466713315, 21.297586362924953, 19.581972547389178, 22.538712245181248, 19.89034374461794, 20.046345644076645, 20.24306974171993, 19.391673055829287, 18.2524054167373, 21.464625403032095, 14.637602771363136, 17.76007792103387, 15.460389702669161, 19.353835657005096, 26.14536361045317, 20.521731031400577, 28.325147094540903, 20.4816886421048, 19.24076585887165, 27.561124577174567, 20.602050815688813, 29.975807060355134, 25.539706914745466, 20.8856537535641, 20.64232851707703, 23.71281513502692, 28.325147094540903, 22.016603538363892, 21.464625403032095, 28.60311491479713, 18.431524745974645, 24.609314266596865, 26.14536361045317, 16.23402543319486, 22.494734284035275, 23.52828054362271, 20.602050815688813, 21.506589466713315, 20.92648590822237, 16.74934871787156, 22.10277408821932, 20.602050815688813, 18.576084986220376, 15.310144149387147, 20.322298876557472, 23.028181534732802, 24.133327041205547, 24.133327041205547, 24.18050850623376, 20.967397891114594, 26.557093177483523, 23.390822626823073, 17.314815113938415, 17.864446345871784, 18.942465433314446, 16.202349296067883, 13.697146137460072, 20.007230835729846, 18.868615759264884, 17.76007792103387, 17.14654807625046, 15.10225016094451, 18.110364070885183, 22.016603538363892, 20.124804822565828, 17.484733434196265, 21.717643615002626, 19.581972547389178, 21.506589466713315, 18.576084986220376, 10.899009079882486, 16.8149038152427, 37.160054292819765, 23.25416777248505, 25.539706914745466, 22.232662691545976, 22.98324851186175, 24.70563239932894, 17.416566918622852, 20.401838105461145, 16.782094257242417, 22.98324851186175, 29.684499285876434, 26.557093177483523, 26.76538304079522, 19.658614220966218, 23.94552000018515, 20.602050815688813, 21.548635571548648, 17.416566918622852, 20.8042283062039, 23.75917448515314, 19.658614220966218, 18.758380733362646, 19.429584428039178, 21.131847230940878, 14.049378009312553, 15.951155153095918, 21.717643615002626, 17.213657767955564, 19.128356641900336, 19.735555861572347, 24.465540688808968, 17.934365811960372, 17.484733434196265, 13.121062323276538, 18.83179895971962, 21.214554942517648, 20.602050815688813, 24.417803063601138, 21.33922385967176, 13.831562592310346, 19.391673055829287, 23.80562446936611, 18.90550453688997, 18.6852487375444, 21.760102337787384, 21.548635571548648, 22.715485556228362, 21.33922385967176, 19.543763832601815, 16.01358620918395, 25.34095533871963, 19.278382202442366, 16.782094257242417, 20.046345644076645, 13.069908149118522, 22.016603538363892, 22.016603538363892, 21.548635571548648, 24.370158584949497, 17.416566918622852, 25.740017318702005, 16.107691244988956, 16.2975636122563, 18.39556080667227, 14.159568646729724, 17.89937193856263, 17.484733434196265, 15.98234019723552, 16.329425896569767, 16.74934871787156, 18.181246031735814, 21.297586362924953, 14.781248363830914, 19.96819234893607, 16.076261613516785, 13.458479789399725, 16.170734966018113, 19.581972547389178, 21.887977204970085, 24.275148340672132, 12.792104248423877, 22.98324851186175, 17.89937193856263, 17.934365811960372, 19.429584428039178, 21.380942759123343, 17.28103011956753, 15.400115355370453, 14.382549407491597, 13.301683750542033, 16.457499171202052, 16.716667072215767, 22.10277408821932, 16.94678474038272, 21.802644068801417, 13.777638440111984, 19.20322291302118, 17.829588900657228, 10.77203177799245, 17.213657767955564, 14.382549407491597, 25.740017318702005, 9.67486465818584, 14.723621737689907, 20.124804822565828, 16.716667072215767, 14.326477161274006, 14.326477161274006, 15.920030957803608, 15.280270706875589, 17.113091407702502, 14.666219776472907, 20.36202965350754, 14.131940228876227, 23.94552000018515, 15.250455553989443, 12.942893525362278, 14.89717913158109, 14.89717913158109, 17.587483776585056, 13.32768898909124, 24.322607071104724, 17.969428099556087, 14.868111474214228, 15.131775567308885, 16.139182322446413, 24.70563239932894, 17.76007792103387, 18.467558995986426, 22.84897480427519, 19.851533330124635, 22.67116267165818, 19.31607208722833, 20.92648590822237, 19.735555861572347, 22.626926270737744, 16.979916273809664, 17.89937193856263, 16.329425896569767, 20.8042283062039, 14.523692042874359, 16.23402543319486, 22.319678873017494, 18.110364070885183, 17.6218678981495, 19.735555861572347, 23.71281513502692, 31.291903031786116, 33.76859572409541, 24.465540688808968, 17.553166745968216, 16.107691244988956, 22.40703562619394, 23.390822626823073, 23.71281513502692, 15.400115355370453, 18.039758452611142, 16.076261613516785, 21.802644068801417, 23.52828054362271, 17.587483776585056, 19.543763832601815, 20.8856537535641, 18.075026786359576, 16.107691244988956, 16.651494963610144, 25.39049775763493, 35.11378365112735, 18.90550453688997, 15.01401915561195, 12.742232560259533, 14.354485905365031, 16.716667072215767, 16.170734966018113, 17.587483776585056, 19.353835657005096, 22.10277408821932, 14.021964597512564, 20.8856537535641, 21.675267738479917, 18.2524054167373, 20.64232851707703, 22.538712245181248, 19.89034374461794, 19.46756991825522, 21.930768935644906, 24.850816893992036, 23.75917448515314, 27.507346790200362, 18.145770440989054, 21.590763877931302, 19.353835657005096, 18.90550453688997, 16.88071548801052, 23.898797049169467, 24.609314266596865, 18.6852487375444, 28.491601937740562, 13.35374506875535, 21.717643615002626, 19.165753221675935, 16.61900425204837, 18.942465433314446, 20.085536923187668, 21.760102337787384, 23.118311288923124, 16.23402543319486, 18.359667040887704, 23.992334296139543, 21.214554942517648, 14.298523068373607, 22.626926270737744, 20.124804822565828, 17.079700020438295, 17.314815113938415, 16.716667072215767, 14.637602771363136, 15.827022278874091, 25.1437504597293, 16.74934871787156, 29.167260806600495, 18.758380733362646, 20.763634702888442, 20.124804822565828, 18.039758452611142, 25.24216031657971, 19.278382202442366, 18.6852487375444, 20.046345644076645, 14.159568646729724, 22.938403163153815, 16.8149038152427, 17.51891667538985, 25.44013703364339, 24.370158584949497, 23.992334296139543, 23.118311288923124, 24.9968545761854, 19.091033031037714, 18.039758452611142, 12.667789349678872, 16.94678474038272, 25.24216031657971, 24.275148340672132, 28.771203256660744, 21.090614371389922, 19.929230034775262, 17.314815113938415, 16.2975636122563, 18.576084986220376, 21.33922385967176, 20.046345644076645, 26.196478674560588, 26.196478674560588, 27.83159468972004, 19.96819234893607, 18.28808948244362, 19.658614220966218, 22.189281900071105, 18.28808948244362, 21.632974546568068, 22.189281900071105, 23.345182011801924, 23.345182011801924, 17.079700020438295, 17.079700020438295, 17.079700020438295, 21.33922385967176, 23.20879380099744, 15.612109685326937, 22.538712245181248, 21.131847230940878, 24.513371642677637, 28.049880592281973, 19.053782246709744, 19.812798643245316, 23.80562446936611, 24.561296107667808, 22.319678873017494, 23.94552000018515, 17.382583384208317, 19.697047472120772, 33.4404292647954, 22.10277408821932, 27.668995768810042, 17.79479946994832, 26.453556883236853, 26.609013186724894, 24.22778221261098, 25.44013703364339, 24.9968545761854, 19.812798643245316, 21.506589466713315, 26.975306541751994, 18.39556080667227, 29.167260806600495, 24.133327041205547, 19.89034374461794, 18.576084986220376, 25.045724266551254, 27.240027749911206, 21.632974546568068, 30.68666283939559, 17.79479946994832, 22.80439145296227, 26.14536361045317, 24.039240115615065, 15.920030957803608, 21.131847230940878, 21.887977204970085, 19.851533330124635, 27.72308958496738, 20.68268496272443, 21.930768935644906, 20.8856537535641, 22.75989509352673, 26.043432497666576, 23.94552000018515, 23.345182011801924, 23.94552000018515, 22.36331459495464, 18.721779026482544, 17.969428099556087, 17.28103011956753, 24.657426302639607, 19.429584428039178, 19.929230034775262, 27.886006392229614, 22.40703562619394, 24.89940107578365, 20.322298876557472, 23.4365524707797, 24.22778221261098, 21.256030110048442, 22.319678873017494, 21.049461966173517, 26.922671813996086, 13.301683750542033, 19.016604146815915, 25.1437504597293, 21.380942759123343, 25.740017318702005, 20.8042283062039, 17.484733434196265, 24.609314266596865, 20.24306974171993, 25.2915095879081, 15.796140293991145, 20.92648590822237, 21.256030110048442, 28.771203256660744, 21.802644068801417, 17.484733434196265, 19.24076585887165, 30.56702687878971, 22.715485556228362, 31.846818916590372, 18.979498589532987, 21.930768935644906, 23.16350836406023, 17.113091407702502, 20.401838105461145, 25.045724266551254, 21.131847230940878, 23.992334296139543, 22.80439145296227, 20.602050815688813, 25.63966650134361, 24.948080241282657, 21.214554942517648, 24.18050850623376, 20.401838105461145, 25.094689498803355, 23.71281513502692, 16.716667072215767, 26.24769367033534, 26.24769367033534, 23.94552000018515, 17.864446345871784, 22.27612829392538, 23.028181534732802, 25.1437504597293, 17.690837939037834, 23.390822626823073, 22.059646737841184, 32.85774546700983, 16.782094257242417, 20.046345644076645, 24.465540688808968, 22.893645317537526, 25.740017318702005, 17.013112580497378, 15.951155153095918, 21.173160702117308, 14.694892728788941, 22.938403163153815, 23.75917448515314, 19.353835657005096, 24.039240115615065, 25.34095533871963, 21.887977204970085, 19.658614220966218, 13.804574186067095, 21.973644325590612, 17.247311047115943, 25.94189877662824, 17.382583384208317, 16.58657693697683, 23.299630451611073, 26.975306541751994, 24.322607071104724, 24.9968545761854, 24.417803063601138, 17.314815113938415, 18.576084986220376, 23.345182011801924, 24.70563239932894, 22.189281900071105, 18.431524745974645, 24.039240115615065, 25.44013703364339, 21.590763877931302, 22.494734284035275, 17.969428099556087, 20.401838105461145, 22.10277408821932, 22.232662691545976, 32.033969127372174, 19.620255961497417, 18.648789727196185, 18.979498589532987, 23.25416777248505, 25.2915095879081, 20.282645623050005, 15.310144149387147, 21.760102337787384, 21.760102337787384, 22.80439145296227, 21.590763877931302, 22.715485556228362, 22.715485556228362, 16.329425896569767, 28.214717811314713, 17.180070153709277, 26.401940203271806, 24.948080241282657, 18.942465433314446, 18.181246031735814, 21.256030110048442, 17.690837939037834, 19.543763832601815, 23.20879380099744, 21.464625403032095, 22.80439145296227, 22.893645317537526, 18.39556080667227, 19.46756991825522, 26.817710263716034, 28.380523708580828, 18.181246031735814, 17.656319241826594, 24.609314266596865, 15.888967492629165, 23.25416777248505, 25.489873356103875, 18.576084986220376, 22.016603538363892, 27.61500750152876, 30.21091022897123, 20.602050815688813, 24.18050850623376, 23.073202403172917, 15.85796463907484, 20.203571081597193, 22.36331459495464, 24.89940107578365, 19.89034374461794, 24.80232751057381, 20.36202965350754, 25.58963789966816, 17.180070153709277, 20.401838105461145, 25.58963789966816, 22.538712245181248, 25.2915095879081, 22.626926270737744, 22.893645317537526, 26.87013978811538, 24.086237637543274, 20.602050815688813, 19.543763832601815, 18.110364070885183, 21.632974546568068, 20.56185170491249, 17.89937193856263, 17.416566918622852, 18.216790978454146, 22.538712245181248, 18.539838978246124, 22.189281900071105, 26.09434828302453, 19.429584428039178, 21.887977204970085, 17.484733434196265, 18.90550453688997, 17.046373787079762, 22.80439145296227, 25.992616060150944, 24.417803063601138, 19.278382202442366, 23.299630451611073, 25.63966650134361, 22.189281900071105, 23.992334296139543, 18.145770440989054, 27.77728915635237, 22.016603538363892, 19.053782246709744, 22.67116267165818, 23.20879380099744, 25.740017318702005, 21.845268970328604, 26.66103470132838, 18.721779026482544, 23.992334296139543, 21.214554942517648, 17.416566918622852, 26.505274475545082, 24.9968545761854, 20.602050815688813, 22.938403163153815, 24.039240115615065, 27.080884906424416, 20.92648590822237, 23.574279122424027, 27.77728915635237, 22.715485556228362, 26.66103470132838, 22.893645317537526, 17.79479946994832, 23.4365524707797, 32.09659661306934, 20.007230835729846, 18.467558995986426, 25.63966650134361, 15.551243837871848, 23.073202403172917, 33.702705802144536, 23.390822626823073, 24.275148340672132, 18.539838978246124, 19.89034374461794, 17.969428099556087, 19.89034374461794, 17.314815113938415, 21.717643615002626, 23.666546242140512, 22.450842133517945, 19.929230034775262, 19.929230034775262, 19.620255961497417, 30.927341318986464, 19.812798643245316, 16.36135047271935, 16.84777751703098, 21.131847230940878, 19.89034374461794, 21.049461966173517, 17.51891667538985, 19.929230034775262, 22.189281900071105, 22.450842133517945, 24.465540688808968, 21.008389858307645, 24.70563239932894, 25.790339917193062, 27.61500750152876, 30.447857334592324, 21.845268970328604, 20.763634702888442, 21.973644325590612, 21.506589466713315, 23.25416777248505, 21.930768935644906, 24.657426302639607, 20.44172438427552, 26.35042423874788, 30.447857334592324, 30.034410770075546, 29.281418237700255, 26.453556883236853, 35.04526897307925, 33.76859572409541, 31.230845679407448, 29.800681131145613, 20.203571081597193, 33.57131140436464, 25.992616060150944, 24.465540688808968, 23.574279122424027, 27.668995768810042, 20.763634702888442, 26.87013978811538, 23.71281513502692, 16.076261613516785, 20.8042283062039, 18.467558995986426, 19.658614220966218, 29.33866439394716, 27.293282919182133, 27.293282919182133, 27.293282919182133, 27.293282919182133, 27.293282919182133, 23.666546242140512, 30.987805310145493, 23.898797049169467, 20.8042283062039, 21.131847230940878, 21.131847230940878, 21.131847230940878, 17.314815113938415, 25.489873356103875, 30.034410770075546, 26.505274475545082, 24.70563239932894, 18.503663694167408, 26.817710263716034, 28.214717811314713, 28.380523708580828, 21.548635571548648, 45.086990684122, 30.866995306309306, 33.90076227087091, 28.049880592281973, 28.940279382223952, 15.490615283867873, 26.609013186724894, 37.01518097066168, 36.65546388385301, 19.20322291302118, 44.5617113320772, 21.845268970328604, 40.57389988868238, 25.2915095879081, 28.883810562338418, 45.88654216244616, 25.44013703364339, 27.240027749911206, 31.660762082694095, 29.975807060355134, 42.438114061533675, 33.11545195869231, 27.40010580820802, 29.858942463791767, 24.086237637543274, 20.521731031400577, 23.20879380099744, 27.080884906424416, 36.228414794400415, 44.04255166984753, 35.38918446958933, 27.080884906424416, 27.028044172168716, 30.806767041912167, 34.56939489431364, 30.447857334592324, 32.22221914035047, 24.850816893992036, 29.975807060355134, 37.8190332109868, 35.25121511458854, 31.048387510438282, 34.300373728940066, 28.883810562338418, 45.529450258314064, 27.4536739354601, 27.83159468972004, 38.79157659527073, 38.79157659527073, 48.84596987082467, 35.04526897307925, 22.016603538363892, 30.62678644311483, 25.24216031657971, 47.435694792893365, 17.28103011956753, 34.03344610288473, 33.63694444585419, 29.975807060355134, 17.864446345871784, 27.72308958496738, 30.21091022897123, 40.49473132867618, 19.24076585887165, 51.1901369093998, 38.33964103892552, 21.887977204970085, 21.506589466713315, 20.521731031400577, 20.521731031400577, 26.09434828302453, 20.521731031400577, 25.992616060150944, 19.735555861572347, 23.028181534732802, 26.2990087931472, 31.846818916590372, 21.760102337787384, 25.539706914745466, 25.992616060150944, 32.22221914035047, 32.22221914035047, 19.851533330124635, 28.883810562338418, 26.66103470132838, 29.74253347911242, 36.44131378072682, 37.01518097066168, 25.489873356103875, 27.561124577174567, 27.561124577174567, 30.987805310145493, 32.9863470466753, 38.41459632502361, 32.22221914035047, 28.436008585761353, 38.33964103892552, 23.345182011801924, 30.26997357328364, 38.56494680401705, 40.65322322588273, 17.934365811960372, 21.42274322042474, 37.524723159601, 32.9863470466753, 31.9714638416083, 29.9173176993003, 36.72712654703974, 47.807737558209425, 34.77254452624438, 47.15856396830611, 29.858942463791767, 35.04526897307925, 30.866995306309306, 38.33964103892552, 35.04526897307925, 45.263456176258764, 47.62135285300038, 24.370158584949497, 47.15856396830611, 36.65546388385301, 24.89940107578365, 53.96206231832688, 41.05217218133816, 27.72308958496738, 55.89290784893326, 24.370158584949497, 35.32013242426123, 40.73270164287187, 15.951155153095918, 27.080884906424416, 23.4365524707797, 40.65322322588273, 48.46584751755162, 52.301824199806006, 33.96703939987107, 32.033969127372174, 26.87013978811538, 36.72712654703974, 21.173160702117308, 16.23402543319486, 36.798929313140206, 25.489873356103875, 29.858942463791767, 29.56877039069624, 47.90120329104557, 34.84052601883783, 30.034410770075546, 29.975807060355134, 25.689792910615846, 37.305494633128845, 36.08717359043372, 43.5294404008871, 24.850816893992036, 23.666546242140512, 32.348333340301785, 24.89940107578365, 38.0412795852805, 48.182698291098816, 42.3553080116137, 26.24769367033534, 31.35307975333966, 35.59715165620933, 33.11545195869231, 19.812798643245316, 18.145770440989054, 31.846818916590372, 30.034410770075546, 27.83159468972004, 26.35042423874788, 22.494734284035275, 36.08717359043372, 28.049880592281973, 35.11378365112735, 38.94340253332466, 28.940279382223952, 38.26483200699776, 35.45837151398557, 40.65322322588273, 58.80452160545304, 40.33685733281881, 22.98324851186175, 18.181246031735814, 21.380942759123343, 31.291903031786116, 40.25815129472595, 59.03467599561125, 28.2698785323965, 27.72308958496738, 26.453556883236853, 21.760102337787384, 22.538712245181248, 28.049880592281973, 38.48969815122384, 30.21091022897123, 26.35042423874788, 45.086990684122, 32.28521466114651, 26.505274475545082, 24.80232751057381, 37.08754689283638, 31.598985005566618, 38.867415430562446, 36.65546388385301, 37.08754689283638, 15.796140293991145, 38.867415430562446, 28.715064341305684, 20.8856537535641, 16.39333746248769, 25.539706914745466, 32.09659661306934, 28.491601937740562, 38.41459632502361, 38.64034257045414, 33.96703939987107, 22.450842133517945, 26.14536361045317, 53.85677052285177, 22.715485556228362, 29.975807060355134, 17.51891667538985, 32.09659661306934, 37.5980852538711, 23.118311288923124, 24.89940107578365, 21.256030110048442, 21.256030110048442, 19.091033031037714, 20.124804822565828, 23.898797049169467, 31.35307975333966, 27.186876493132687, 30.034410770075546, 28.049880592281973, 42.27266353442275, 32.411575418595035, 31.53732846903916, 30.093129052017023, 13.484791569471833, 29.9173176993003, 35.66674528344124, 30.26997357328364, 26.043432497666576, 35.25121511458854, 14.076845015253456, 14.076845015253456, 14.076845015253456, 14.076845015253456, 36.37020880080945, 27.346642204097876, 34.63697922255266, 32.60204446315635, 30.388446900124176, 19.31607208722833, 24.561296107667808, 18.576084986220376, 31.475792237910618, 28.883810562338418, 44.5617113320772, 24.513371642677637, 24.513371642677637, 29.281418237700255, 18.979498589532987, 27.240027749911206, 29.9173176993003, 34.16664924473404, 37.160054292819765, 23.16350836406023, 64.58413226217957, 28.2698785323965, 23.118311288923124, 22.27612829392538, 27.240027749911206, 33.63694444585419, 39.78912962559818, 36.016759615546256, 19.581972547389178, 20.046345644076645, 21.214554942517648, 28.325147094540903, 28.60311491479713, 36.51255777326842, 25.24216031657971, 22.494734284035275, 32.159346537604755, 37.45150421083523, 20.64232851707703, 33.245062173048, 35.04526897307925, 25.94189877662824, 27.668995768810042, 47.99485175252902, 20.046345644076645, 21.930768935644906, 27.346642204097876, 23.52828054362271, 20.56185170491249, 27.293282919182133, 21.008389858307645, 31.475792237910618, 42.02569638966092, 20.92648590822237, 21.380942759123343, 30.26997357328364, 25.740017318702005, 23.028181534732802, 28.214717811314713, 24.657426302639607, 30.151962130172098, 19.429584428039178, 41.132430556936306, 17.382583384208317, 22.938403163153815, 21.464625403032095, 25.39049775763493, 18.6852487375444, 15.04337206739713, 16.425386987895497, 34.03344610288473, 25.740017318702005, 23.20879380099744, 21.214554942517648, 26.2990087931472, 21.930768935644906, 33.05083646319603, 39.32557222442699, 28.325147094540903, 40.57389988868238, 37.160054292819765, 30.329152388418176, 29.56877039069624, 40.33685733281881, 41.943695056893915, 37.160054292819765, 20.4816886421048, 28.940279382223952, 35.59715165620933, 20.124804822565828, 27.61500750152876, 30.388446900124176, 27.346642204097876, 32.28521466114651, 34.84052601883783, 27.668995768810042, 21.33922385967176, 36.016759615546256, 22.319678873017494, 21.008389858307645, 23.898797049169467, 37.37842812826559, 20.203571081597193, 29.396022468352484, 26.043432497666576, 19.812798643245316, 24.609314266596865, 28.380523708580828, 56.55175575986736, 30.56702687878971, 52.40407602298735, 19.391673055829287, 38.56494680401705, 24.657426302639607, 45.7076474882609, 48.84596987082467, 42.60421214369869, 35.04526897307925, 18.075026786359576, 31.660762082694095, 24.9968545761854, 37.8190332109868, 17.382583384208317, 22.75989509352673, 29.56877039069624, 16.61900425204837, 31.53732846903916, 16.2975636122563, 28.2698785323965, 22.538712245181248, 40.57389988868238, 28.049880592281973, 28.380523708580828, 24.086237637543274, 54.173263858709824, 17.829588900657228, 39.556671887793456, 33.375179667439795, 23.20879380099744, 26.2990087931472, 27.507346790200362, 18.83179895971962, 32.033969127372174, 28.214717811314713, 29.684499285876434, 44.82358155812517, 44.82358155812517, 44.82358155812517, 44.82358155812517, 21.548635571548648, 40.41571724385996, 18.2524054167373, 29.74253347911242, 30.447857334592324, 23.852165264858517, 17.864446345871784, 27.028044172168716, 26.35042423874788, 41.45503620813183, 53.43765380953904, 25.689792910615846, 26.609013186724894, 33.96703939987107, 21.131847230940878, 14.076845015253456, 23.71281513502692, 22.494734284035275, 33.83461446305701, 36.08717359043372, 22.582776184718522, 32.79363281328884, 34.300373728940066, 23.992334296139543, 32.53843073806492, 16.202349296067883, 28.380523708580828, 38.94340253332466, 25.34095533871963, 24.133327041205547, 40.97207040739876, 45.79700747446617, 37.160054292819765, 44.82358155812517, 38.56494680401705, 42.52108200006278, 31.846818916590372, 47.90120329104557, 33.50580642730562, 38.48969815122384, 30.447857334592324, 33.57131140436464, 23.52828054362271, 25.891280453627324, 29.33866439394716, 33.4404292647954, 31.784678801853513, 37.45150421083523, 29.684499285876434, 30.56702687878971, 17.45061689198871, 33.90076227087091, 34.908640417532865, 41.2128458403547, 40.25815129472595, 34.16664924473404, 24.70563239932894, 23.20879380099744, 24.609314266596865, 31.475792237910618, 44.99901609609206, 30.56702687878971, 49.133016862271724, 44.5617113320772, 26.09434828302453, 37.01518097066168, 38.115651564714454, 27.028044172168716, 39.32557222442699, 53.22932014095188, 19.96819234893607, 38.33964103892552, 20.322298876557472, 26.2990087931472, 23.118311288923124, 67.41971617548266, 38.79157659527073, 37.37842812826559, 39.86691871057298, 33.63694444585419, 31.10908815096766, 28.940279382223952, 43.275130861102994, 38.94340253332466, 40.41571724385996, 33.375179667439795, 65.8579401324037, 27.995149135337257, 34.908640417532865, 45.17513726542669, 24.322607071104724, 42.60421214369869, 36.016759615546256, 32.85774546700983, 55.89290784893326, 44.21492934326177, 34.4346215959791, 44.21492934326177, 24.561296107667808, 24.086237637543274, 32.72964525738904, 19.353835657005096, 28.940279382223952, 45.086990684122, 38.41459632502361, 48.84596987082467, 38.48969815122384, 35.11378365112735, 45.086990684122, 50.004318665090636, 22.450842133517945, 44.82358155812517, 48.94146538270724, 47.06654728734682, 46.88305220915621, 30.56702687878971, 55.02636863114642, 45.44061233480379, 33.90076227087091, 36.942956250241515, 40.49473132867618, 41.617286632367644, 39.556671887793456, 37.08754689283638, 37.01518097066168, 58.57526450222688, 23.25416777248505, 26.87013978811538, 30.866995306309306, 40.41571724385996, 44.91121316574036, 51.290215346585704, 21.173160702117308, 36.65546388385301, 48.56059987746486, 55.24173574865985, 33.57131140436464, 33.11545195869231, 26.557093177483523, 23.028181534732802, 36.798929313140206, 42.27266353442275, 52.19977189231385, 21.008389858307645, 45.61846186294896, 33.96703939987107, 44.73612093895834, 26.14536361045317, 37.9670527218579, 34.300373728940066, 41.53608219643496, 48.27689708476307, 31.291903031786116, 33.702705802144536, 35.18243227767251, 34.367432111764096, 36.08717359043372, 42.3553080116137, 44.64883097460398, 21.973644325590612, 34.03344610288473, 23.992334296139543, 27.72308958496738, 24.513371642677637, 36.87087245606033, 26.922671813996086, 42.93836112094585, 59.265731184583274, 51.99626426796157, 51.894808174781225, 48.84596987082467, 49.809370301716946, 53.85677052285177, 40.81233544283598, 56.773093073800744, 53.12545808585662, 47.90120329104557, 41.698649825700315, 33.83461446305701, 35.87634357692165, 25.192907336481866, 50.5937537501147, 40.41571724385996, 25.24216031657971, 36.58394105020859, 34.03344610288473, 28.15966472087403, 38.56494680401705, 30.62678644311483, 26.713157919740368, 37.524723159601, 49.037147591523336, 44.21492934326177, 67.1568716116849, 48.56059987746486, 41.53608219643496, 35.806340977539655, 36.51255777326842, 29.56877039069624, 33.1801937796528, 29.975807060355134, 50.6926662381911, 41.943695056893915, 41.943695056893915, 35.18243227767251, 36.942956250241515, 39.095822701298815, 43.275130861102994, 49.42175070681059, 41.45503620813183, 38.26483200699776, 32.411575418595035, 41.132430556936306, 42.10785803778799, 28.214717811314713, 21.380942759123343, 54.598150033144236, 29.22428378123494, 36.87087245606033, 66.37447035867858, 53.22932014095188, 64.83690740886694, 54.067559962798995, 19.658614220966218, 47.34313736903344, 28.883810562338418, 27.995149135337257, 45.79700747446617, 44.64883097460398, 49.61518196998445, 43.78524440968234, 29.453492679719986, 42.438114061533675, 45.086990684122, 33.31005738633129, 35.45837151398557, 47.62135285300038, 63.83170433047907, 67.68358948381629, 36.87087245606033, 52.71203249107963, 59.49769069799599, 47.250760544967605, 66.37447035867858, 26.14536361045317, 50.990565478815945, 54.067559962798995, 48.94146538270724, 57.66713919909174, 41.617286632367644, 32.033969127372174, 56.773093073800744, 54.067559962798995, 62.3530444922685, 54.067559962798995, 42.27266353442275, 53.333385249853684, 46.15619785585283, 59.265731184583274, 59.73055807527518, 39.556671887793456, 49.42175070681059, 66.89505178172818, 51.99626426796157, 45.351947753528236, 59.964336869699544, 30.68666283939559, 43.87084603329354, 60.67117749756472, 53.333385249853684, 27.186876493132687, 34.23344619169951, 47.34313736903344, 62.10995300937982, 52.609179777419676, 41.780172086809024, 34.23344619169951, 32.348333340301785, 43.699809813576366, 48.08868329990042, 58.57526450222688, 48.37128004023317, 50.004318665090636, 53.96206231832688, 67.68358948381629, 57.66713919909174, 61.867809250367884, 49.22907356066477, 37.37842812826559, 53.75168417471723, 61.38635013907805, 59.73055807527518, 38.48969815122384, 88.62154503481175, 52.71203249107963, 63.83170433047907, 81.32370105824417, 44.47476167904283, 49.9067492938035, 67.1568716116849, 67.41971617548266, 62.597087408320306, 50.49503426195374, 58.57526450222688, 70.93179276322707, 47.250760544967605, 72.33079701870139, 96.95226558425567, 77.90333396724647, 74.04607243930377, 42.93836112094585, 37.5980852538711, 40.81233544283598, 51.99626426796157, 64.83690740886694, 50.004318665090636, 32.47494113727571, 27.77728915635237, 26.196478674560588, 45.44061233480379, 40.65322322588273, 42.02569638966092, 45.17513726542669, 43.10641711474324, 48.65553748142455, 20.282645623050005, 36.228414794400415, 54.811841150452466, 43.3597351955915, 44.64883097460398, 56.773093073800744, 24.086237637543274, 52.81508628524973, 48.46584751755162, 46.88305220915621, 25.192907336481866, 35.18243227767251, 41.617286632367644, 29.511075247281195, 33.1801937796528, 45.086990684122, 32.85774546700983, 32.22221914035047, 46.7915731119465, 31.53732846903916, 19.774139536218833, 35.806340977539655, 25.045724266551254, 47.807737558209425, 39.7114923241556, 55.457945789238345, 33.63694444585419, 53.22932014095188, 38.33964103892552, 51.290215346585704, 50.396507397123145, 49.42175070681059, 29.110349096271467, 53.12545808585662, 53.646802873050504, 44.47476167904283, 45.529450258314064, 41.780172086809024, 31.230845679407448, 31.846818916590372, 24.948080241282657, 28.436008585761353, 40.022953418667065, 37.232703447205715, 43.19069160819034, 38.48969815122384, 44.30137101535166, 49.61518196998445, 64.83690740886694, 61.626609520407456, 43.61454191906839, 74.04607243930377, 74.04607243930377, 90.01713130052181, 40.25815129472595, 36.37020880080945, 48.56059987746486, 46.15619785585283, 53.75168417471723, 50.990565478815945, 41.2128458403547, 47.06654728734682, 47.435694792893365, 42.3553080116137, 72.61389175398708, 84.2335880783952, 59.03467599561125, 57.66713919909174, 70.10541234668786, 60.199030648454944, 66.11570082213345, 59.49769069799599, 27.940524471445343, 33.1801937796528, 40.81233544283598, 51.290215346585704, 50.6926662381911, 53.12545808585662, 27.133828946090556, 47.807737558209425, 62.84208548133934, 55.6750020519895, 47.250760544967605, 69.83209724580306, 43.444504934395866, 69.28865955361954, 55.24173574865985, 62.10995300937982, 69.28865955361954, 41.2128458403547, 51.79355004464666, 33.63694444585419, 75.21212841006654, 47.90120329104557, 61.14702744030778, 43.10641711474324, 55.24173574865985, 52.097918711212095, 38.33964103892552, 46.51820540231633, 55.457945789238345, 33.1801937796528, 40.57389988868238, 48.182698291098816, 35.45837151398557, 42.190180314697045, 28.996858600549206, 34.03344610288473, 49.61518196998445, 35.11378365112735, 39.479488162280596, 33.96703939987107, 30.507383918455513, 37.5980852538711, 60.908637772317434, 43.19069160819034, 39.32557222442699, 36.942956250241515, 48.27689708476307, 36.58394105020859, 42.10785803778799, 34.4346215959791, 52.609179777419676, 39.095822701298815, 57.89284200037607, 53.22932014095188, 46.97471015107385, 50.004318665090636, 35.11378365112735, 65.09067188950593, 61.38635013907805, 56.773093073800744, 56.9952966774873, 69.83209724580306, 32.033969127372174, 83.25223705241488, 53.12545808585662, 27.72308958496738, 98.86447768050158, 98.4790416094955, 48.94146538270724, 48.08868329990042, 37.89297069129353, 54.49161709130268, 39.172256347947744, 46.066137010589394, 56.33128135834348, 39.78912962559818, 30.74665629604239, 60.199030648454944, 48.08868329990042, 39.78912962559818, 60.434642992688794, 39.095822701298815, 36.65546388385301, 37.89297069129353, 33.245062173048, 31.598985005566618, 69.01852867010916, 59.49769069799599, 49.42175070681059, 49.42175070681059, 54.173263858709824, 55.24173574865985, 41.132430556936306, 52.40407602298735, 54.811841150452466, 42.77096031537661, 35.38918446958933, 48.75066069158903, 58.34690118774256, 31.291903031786116, 59.265731184583274, 55.6750020519895, 67.9484955630781, 42.93836112094585, 54.067559962798995, 51.894808174781225, 43.699809813576366, 59.964336869699544, 69.28865955361954, 54.49161709130268, 36.228414794400415, 58.57526450222688, 51.290215346585704, 30.74665629604239, 56.773093073800744, 42.68750480955783, 59.964336869699544, 52.40407602298735, 65.8579401324037, 52.19977189231385, 69.55984769951485, 67.9484955630781, 41.698649825700315, 57.89284200037607, 72.33079701870139, 61.626609520407456, 55.6750020519895, 38.56494680401705, 48.46584751755162, 49.22907356066477, 96.57428452170761, 43.699809813576366, 34.908640417532865, 34.908640417532865, 40.49473132867618, 28.436008585761353, 46.51820540231633, 34.63697922255266, 38.94340253332466, 53.02179868836547, 43.3597351955915, 44.99901609609206, 41.780172086809024, 35.66674528344124, 35.66674528344124, 54.811841150452466, 43.5294404008871, 45.7076474882609, 35.18243227767251, 69.83209724580306, 44.64883097460398, 51.894808174781225, 45.351947753528236, 46.70027251047952, 55.457945789238345, 52.9183415530491, 34.501942437892716, 70.65525591042105, 36.72712654703974, 49.42175070681059, 37.5980852538711, 39.172256347947744, 24.039240115615065, 50.79177210350426, 35.32013242426123, 32.79363281328884, 31.10908815096766, 60.67117749756472, 42.93836112094585, 40.022953418667065, 62.84208548133934, 28.325147094540903, 44.12865633779556, 39.78912962559818, 24.657426302639607, 22.145985754016134, 21.214554942517648, 25.39049775763493, 34.501942437892716, 24.657426302639607, 58.57526450222688, 52.81508628524973, 40.892124929554, 51.49095957359226, 34.367432111764096, 42.438114061533675, 26.922671813996086, 42.52108200006278, 25.489873356103875, 53.333385249853684, 19.46756991825522, 36.08717359043372, 56.55175575986736, 50.990565478815945, 40.1011996370117, 41.45503620813183, 43.5294404008871, 40.022953418667065, 30.388446900124176, 54.173263858709824, 56.9952966774873, 24.133327041205547, 36.87087245606033, 51.79355004464666, 32.66578255521719, 44.30137101535166, 34.70469568042362, 39.556671887793456, 26.713157919740368, 31.291903031786116, 22.538712245181248, 52.50652775191775, 40.892124929554, 46.427438201089, 32.09659661306934, 42.854578979512624, 39.94485987582193, 44.64883097460398, 53.542126217761066, 20.64232851707703, 90.7231444115574, 27.995149135337257, 55.24173574865985, 33.76859572409541, 36.016759615546256, 47.250760544967605, 51.49095957359226, 53.542126217761066, 48.37128004023317, 51.99626426796157, 39.01953819342525, 31.90908051733889, 30.447857334592324, 46.51820540231633, 52.19977189231385, 45.61846186294896, 31.72265993608251, 56.773093073800744, 31.53732846903916, 55.02636863114642, 33.63694444585419, 45.79700747446617, 36.299242562272255, 38.867415430562446, 28.049880592281973, 34.300373728940066, 42.10785803778799, 34.97688798216538, 29.800681131145613, 50.1020787877761, 49.42175070681059, 35.45837151398557, 25.891280453627324, 40.17959882934157, 41.2128458403547, 35.38918446958933, 32.47494113727571, 45.263456176258764, 36.016759615546256, 45.351947753528236, 43.61454191906839, 60.67117749756472, 111.15632012308374, 43.78524440968234, 33.11545195869231, 39.556671887793456, 28.491601937740562, 29.975807060355134, 75.80202444429054, 53.22932014095188, 41.617286632367644, 49.133016862271724, 40.97207040739876, 23.80562446936611, 29.22428378123494, 85.2265069597304, 55.02636863114642, 50.29817277977263, 45.351947753528236, 98.09510821132314, 41.780172086809024, 46.246434773093995, 67.41971617548266, 65.8579401324037, 40.65322322588273, 33.702705802144536, 74.91890411002962, 77.59961765106212, 61.867809250367884, 71.76791429061606, 48.08868329990042, 41.617286632367644, 32.348333340301785, 26.975306541751994, 38.79157659527073, 67.41971617548266, 58.34690118774256, 63.3349620664254, 54.067559962798995, 52.40407602298735, 38.71588573814723, 65.09067188950593, 35.52769382137767, 52.609179777419676, 52.9183415530491, 45.79700747446617, 49.42175070681059, 51.39048944047805, 37.37842812826559, 47.90120329104557, 66.37447035867858, 52.40407602298735, 60.67117749756472, 55.457945789238345, 59.964336869699544, 48.08868329990042, 30.62678644311483, 48.75066069158903, 45.17513726542669, 31.784678801853513, 58.57526450222688, 48.46584751755162, 36.44131378072682, 54.49161709130268, 72.04880596519355, 62.84208548133934, 64.08153455750669, 52.9183415530491, 40.49473132867618, 57.89284200037607, 47.250760544967605, 72.04880596519355, 40.25815129472595, 61.626609520407456, 46.427438201089, 62.10995300937982, 55.6750020519895, 69.83209724580306, 48.182698291098816, 56.11166650505244, 58.34690118774256, 61.626609520407456, 58.34690118774256, 39.78912962559818, 60.434642992688794, 57.442316329639176, 56.11166650505244, 40.81233544283598, 69.28865955361954, 54.173263858709824, 85.89494673838159, 65.8579401324037, 53.12545808585662, 60.67117749756472, 52.9183415530491, 62.10995300937982, 53.12545808585662, 32.9863470466753, 61.867809250367884, 57.21836996148928, 105.65258908406659, 51.79355004464666, 40.57389988868238, 31.784678801853513, 44.47476167904283, 33.702705802144536, 51.39048944047805, 98.09510821132314, 33.1801937796528, 54.49161709130268, 88.62154503481175, 45.79700747446617, 69.55984769951485, 67.68358948381629, 72.33079701870139, 44.47476167904283, 49.133016862271724, 60.199030648454944, 67.9484955630781, 71.48811770889668, 47.99485175252902, 53.542126217761066, 68.48142221881122, 66.11570082213345, 56.11166650505244, 42.60421214369869, 60.434642992688794, 69.83209724580306, 78.2082389954826, 52.609179777419676, 46.7915731119465, 60.908637772317434, 39.78912962559818, 71.20941195067294, 69.28865955361954, 78.51433738825796, 59.964336869699544, 44.04255166984753, 56.11166650505244, 67.68358948381629, 60.67117749756472, 66.89505178172818, 52.40407602298735, 51.09025374715126, 29.858942463791767, 56.33128135834348, 49.9067492938035, 40.57389988868238, 45.61846186294896, 56.773093073800744, 58.57526450222688, 36.15772522700737, 41.861853726676614, 51.09025374715126, 51.39048944047805, 74.62682298320091, 48.46584751755162, 29.167260806600495, 26.87013978811538, 30.093129052017023, 43.02230705927991, 69.28865955361954, 36.58394105020859, 47.15856396830611, 39.32557222442699, 50.29817277977263, 36.58394105020859, 34.63697922255266, 45.44061233480379, 33.245062173048, 69.01852867010916, 47.99485175252902, 35.66674528344124, 41.943695056893915, 40.25815129472595, 33.63694444585419, 48.27689708476307, 44.47476167904283, 29.626578330054702, 48.84596987082467, 33.375179667439795, 55.6750020519895, 31.598985005566618, 67.1568716116849, 58.57526450222688, 44.21492934326177, 46.066137010589394, 46.60915005647103, 31.048387510438282, 32.09659661306934, 35.52769382137767, 30.74665629604239, 50.79177210350426, 43.87084603329354, 50.20003003478527, 55.02636863114642, 27.40010580820802, 49.42175070681059, 50.29817277977263, 66.89505178172818, 44.04255166984753, 49.809370301716946, 58.57526450222688, 73.46984133199462, 62.597087408320306, 69.01852867010916, 26.2990087931472, 44.73612093895834, 73.75739416067401, 55.89290784893326, 56.33128135834348, 42.60421214369869, 50.990565478815945, 46.88305220915621, 43.78524440968234, 43.444504934395866, 52.40407602298735, 47.435694792893365, 41.780172086809024, 44.91121316574036, 59.73055807527518, 50.89107172411317, 39.172256347947744, 40.1011996370117, 41.132430556936306, 34.63697922255266, 38.41459632502361, 66.89505178172818, 49.133016862271724, 58.34690118774256, 33.96703939987107, 48.46584751755162, 39.634006510082365, 36.08717359043372, 30.329152388418176, 42.60421214369869, 39.479488162280596, 56.11166650505244, 46.88305220915621, 34.97688798216538, 31.72265993608251, 59.964336869699544, 65.60118435636834, 39.78912962559818, 56.773093073800744, 48.65553748142455, 72.33079701870139, 57.66713919909174, 51.39048944047805, 50.004318665090636, 47.06654728734682, 42.02569638966092, 59.73055807527518, 64.33234259239626, 49.133016862271724, 43.61454191906839, 55.02636863114642, 46.066137010589394, 70.65525591042105, 46.88305220915621, 65.34542957624015, 67.68358948381629, 48.182698291098816, 34.908640417532865, 47.06654728734682, 55.6750020519895, 94.70636877096143, 59.964336869699544, 50.49503426195374, 37.45150421083523, 67.1568716116849, 66.37447035867858, 38.64034257045414, 33.76859572409541, 39.01953819342525, 44.47476167904283, 33.57131140436464, 41.780172086809024, 44.387981683814154, 57.442316329639176, 48.84596987082467, 32.79363281328884, 34.908640417532865, 69.28865955361954, 63.83170433047907, 41.2128458403547, 43.5294404008871, 48.27689708476307, 30.093129052017023, 68.21443845541918, 45.529450258314064, 42.68750480955783, 43.275130861102994, 42.52108200006278, 56.773093073800744, 55.457945789238345, 57.89284200037607, 45.7076474882609, 59.265731184583274, 52.40407602298735, 34.16664924473404, 88.96839995507139, 70.93179276322707, 55.457945789238345, 36.08717359043372, 56.9952966774873, 51.1901369093998, 54.811841150452466, 65.09067188950593, 51.692489491288626, 26.24769367033534, 39.479488162280596, 45.263456176258764, 51.290215346585704, 38.115651564714454, 38.115651564714454, 44.04255166984753, 50.20003003478527, 51.79355004464666, 80.06289393593435, 43.444504934395866, 43.699809813576366, 47.99485175252902, 72.61389175398708, 57.89284200037607, 54.49161709130268, 49.037147591523336, 50.79177210350426, 55.6750020519895, 55.89290784893326, 46.33684810653952, 43.275130861102994, 46.15619785585283, 45.7076474882609, 42.93836112094585, 42.3553080116137, 45.88654216244616, 56.55175575986736, 39.556671887793456, 41.132430556936306, 45.88654216244616, 38.26483200699776, 40.65322322588273, 47.807737558209425, 44.12865633779556, 50.396507397123145, 50.6926662381911, 95.44916002730179, 69.83209724580306, 34.09998263323346, 36.942956250241515, 40.17959882934157, 57.21836996148928, 53.96206231832688, 57.442316329639176, 77.29708541258049, 54.067559962798995, 53.542126217761066, 39.634006510082365, 50.004318665090636, 51.09025374715126, 50.990565478815945, 43.3597351955915, 59.265731184583274, 40.1011996370117, 49.9067492938035, 53.22932014095188, 49.809370301716946, 42.438114061533675, 50.29817277977263, 66.37447035867858, 43.19069160819034, 59.964336869699544, 67.9484955630781, 70.37979717263202, 44.387981683814154, 38.19016894386643, 38.94340253332466, 48.75066069158903, 60.908637772317434, 86.23113005463887, 54.811841150452466, 46.7915731119465, 40.81233544283598, 31.846818916590372, 39.86691871057298, 56.55175575986736, 53.96206231832688, 67.41971617548266, 56.11166650505244, 50.20003003478527, 44.82358155812517, 34.23344619169951, 48.84596987082467, 50.89107172411317, 51.49095957359226, 51.49095957359226, 49.037147591523336, 52.81508628524973, 55.24173574865985, 69.55984769951485, 50.396507397123145, 48.182698291098816, 49.71218131735948, 45.88654216244616, 60.908637772317434, 51.692489491288626, 84.89424029774196, 50.20003003478527, 91.79256012938836, 74.91890411002962, 35.32013242426123, 46.51820540231633, 39.40245503911115, 46.427438201089, 31.846818916590372, 46.60915005647103, 43.699809813576366, 40.33685733281881, 28.1047190510635, 33.1801937796528, 50.20003003478527, 48.27689708476307, 50.6926662381911, 47.250760544967605, 31.10908815096766, 42.3553080116137, 70.37979717263202, 75.80202444429054, 40.17959882934157, 47.52843316962603, 44.73612093895834, 34.4346215959791, 104.01459751366006, 46.33684810653952, 33.50580642730562, 34.56939489431364, 42.93836112094585, 43.699809813576366, 39.556671887793456, 29.281418237700255, 38.115651564714454, 46.97471015107385, 42.190180314697045, 52.609179777419676, 48.65553748142455, 40.022953418667065, 44.387981683814154, 50.6926662381911, 47.807737558209425, 52.81508628524973, 81.641993024771, 38.19016894386643, 54.173263858709824, 51.894808174781225, 43.02230705927991, 43.02230705927991, 74.04607243930377, 42.52108200006278, 56.33128135834348, 43.19069160819034, 62.3530444922685, 51.1901369093998, 39.01953819342525, 79.43983955226133, 48.84596987082467, 37.74523999888851, 65.8579401324037, 43.275130861102994, 47.90120329104557, 44.82358155812517, 59.03467599561125, 66.11570082213345, 48.46584751755162, 43.275130861102994, 33.4404292647954, 28.380523708580828, 41.780172086809024, 31.72265993608251, 30.388446900124176, 33.4404292647954, 40.65322322588273, 75.21212841006654, 45.351947753528236, 49.71218131735948, 52.9183415530491, 69.01852867010916, 54.811841150452466, 44.64883097460398, 63.08804244970428, 47.62135285300038, 55.6750020519895, 65.09067188950593, 72.89809449073893, 39.40245503911115, 48.08868329990042, 37.74523999888851, 56.55175575986736, 47.90120329104557, 46.427438201089, 74.62682298320091, 56.33128135834348, 79.43983955226133, 64.33234259239626, 66.37447035867858, 94.33714363029665, 61.867809250367884, 61.38635013907805, 56.33128135834348, 41.698649825700315, 46.246434773093995, 50.6926662381911, 51.692489491288626, 38.94340253332466, 82.60436300370014, 59.49769069799599, 75.50650035756513, 40.41571724385996, 36.228414794400415, 53.75168417471723, 61.626609520407456, 70.65525591042105, 50.396507397123145, 53.333385249853684, 81.00664999449546, 49.32531805312982, 67.1568716116849, 72.61389175398708, 58.80452160545304, 58.57526450222688, 46.246434773093995, 59.964336869699544, 55.02636863114642, 53.96206231832688, 46.97471015107385, 107.7363996853844, 53.96206231832688, 65.60118435636834, 57.442316329639176, 38.71588573814723, 44.5617113320772, 29.281418237700255, 53.22932014095188, 58.119428177447986, 69.55984769951485, 63.3349620664254, 82.28231911219243, 87.24759532868651, 60.199030648454944, 53.02179868836547, 44.21492934326177, 51.79355004464666, 61.14702744030778, 49.51837188956857, 83.25223705241488, 60.908637772317434, 74.33588057277046, 69.01852867010916, 57.21836996148928, 32.85774546700983, 70.10541234668786, 45.17513726542669, 40.17959882934157, 47.06654728734682, 61.14702744030778, 67.68358948381629, 64.33234259239626, 52.097918711212095, 55.89290784893326, 54.49161709130268, 46.97471015107385, 36.72712654703974, 58.119428177447986, 82.60436300370014, 100.42136666458902, 66.89505178172818, 60.908637772317434, 38.19016894386643, 35.18243227767251, 55.6750020519895, 33.83461446305701, 40.892124929554, 49.61518196998445, 41.37414835829223, 48.56059987746486, 46.427438201089, 47.435694792893365, 59.265731184583274, 70.37979717263202, 47.06654728734682, 39.248839424943355, 31.230845679407448, 43.444504934395866, 48.27689708476307, 45.263456176258764, 35.87634357692165, 51.290215346585704, 50.004318665090636, 69.55984769951485, 47.807737558209425, 43.275130861102994, 70.10541234668786, 71.20941195067294, 52.609179777419676, 46.7915731119465, 73.75739416067401, 56.773093073800744, 67.41971617548266, 56.55175575986736, 39.94485987582193, 60.199030648454944, 47.435694792893365, 59.964336869699544, 59.73055807527518, 60.199030648454944, 67.68358948381629, 77.59961765106212, 53.22932014095188, 49.037147591523336, 60.199030648454944, 84.2335880783952, 102.40200064874995, 72.61389175398708, 72.04880596519355]

tcached_perplexity = []
for i in range(len(test)):
    prompt = 'Question: ' + test['QuestionText'][i] + '\nAnswer: ' + test['MC_Answer'][i] + '\nStudent Explanation: ' + test['StudentExplanation'][i]
    tcached_perplexity.append(scorer.get_perplexity(str(prompt)))

tfidf = feature_extraction.text.TfidfVectorizer(analyzer="word", ngram_range=(1,2), min_df = 4, max_df = 0.97,  stop_words="english", dtype=np.float32,  max_features = 5000)
tfidf.fit(dfall['StudentExplanation'])
fe = [k for k in tfidf.vocabulary_]

def nlpit(df):
    global gqid, gqida, tfidf, fe, cached_perplexity
    df = df.reset_index(drop=True)
    df['inference_sum'] = df['StudentExplanation'].map(lambda x: np.sum([exp[w]['inference'] if w in exp else 0 for w in str(x).lower().split(' ')]))
    df['inference_mean'] = df['StudentExplanation'].map(lambda x: np.mean([exp[w]['inference'] if w in exp else 0 for w in str(x).lower().split(' ')]))
    df['sentiment_sum'] = df['StudentExplanation'].map(lambda x: np.sum([exp[w]['sentiment'] if w in exp else 0.5 for w in str(x).lower().split(' ')]))
    df['sentiment_mean'] = df['StudentExplanation'].map(lambda x: np.mean([exp[w]['sentiment'] if w in exp else 0.5 for w in str(x).lower().split(' ')]))
    df = df.merge(gqid, on=['QuestionId'], how='left')
    df = df.merge(gqida, on=['QuestionId','MC_Answer'], how='left')
    df['rchar_len'] = df['StudentExplanation'].map(len)
    df['-a_rchar_len'] = df['rchar_len'] - df['achar_len']
    df['-mn_rchar_len'] = df['rchar_len'] - df['mnchar_len']
    df['-mx_rchar_len'] = df['rchar_len'] - df['mxchar_len']
    df['rword_len'] = df['StudentExplanation'].map(lambda x: len(str(x).split(' ')))
    df['-a_rword_len'] = df['rword_len'] - df['aword_len']
    df['-mn_rword_len'] = df['rword_len'] - df['mnword_len']
    df['-mx_rword_len'] = df['rword_len'] - df['mxword_len']
    df['Prompt'] = df.apply(lambda r: 'Question: ' + r['QuestionText'] + '\nAnswer: ' + r['MC_Answer'] + '\nStudent Explanation: ' + r['StudentExplanation'], axis=1)
    if len(df)==36696: #hoping train and test size are not the same :)
        df['Perplexity'] = cached_perplexity
    else:
        #df['Perplexity'] = df['Prompt'].progress_map(lambda x: scorer.get_perplexity(str(x))) #need to cache results
        df['Perplexity'] = tcached_perplexity
    df['Perplexity_ScaleC'] = df['Perplexity'] / (df['rchar_len'] + 1)
    df['Perplexity_ScaleW'] = df['Perplexity'] / (df['rword_len'] + 1)
    df['StudentExplanation'] = df.apply(lambda r: 'ANSWER__' + str(r['arank']) + ' ' + r['MC_Answer'] + ' ' + r['StudentExplanation'], axis=1)
    txt = pd.DataFrame(tfidf.transform(df['StudentExplanation']).toarray(), columns=fe)
    df = pd.concat([df, txt], axis=1)
    df = df.fillna(-1)
    df = df.reset_index(drop=True)
    return df

def f_experience(c, s):
    it = {'memory':10_000, 'inference':0.5, 'sentiment':1e-10,}
    exp = {}
    for i in range(len(c)):
        words = set([str(w) for w in str(c[i]).lower().split(' ')])
        for w in words:
            try:
                exp[w]['inference'] += 1
                exp[w]['sentiment'] += s[i]
            except:
                m = [0. for m_ in range(it['memory'])]
                exp[w] = {}
                exp[w]['inference'] = 1
                exp[w]['sentiment'] = s[i]
    for w in exp:
        exp[w]['sentiment'] /= exp[w]['inference'] + it['sentiment']
    return exp

train['Misconception'] = train['Misconception'].fillna('NA')
train['Category:Misconception'] = train['Category'] + ':' + train['Misconception']
le = {k:v for v,k in enumerate(train['Category:Misconception'].value_counts().index.tolist())}
lei = {le[k]:k for k in le}
train['target'] = train['Category:Misconception'].map(le)

def map3(y, pred):
    score = 0
    for i in range(len(y)):
        preds = pred[i].split(' ')
        if y[i] in preds:
            score += (1 / (preds.index(y[i]) + 1))
    return score / len(y)


def gettop3(preds):
    top3 = np.argsort(-preds, axis=1)[:, :3]
    top3 = top3.flatten()
    top3 = np.array([lei[v] for v in top3])
    top3 = top3.reshape(-1,3)
    top3 = [' '.join(r) for r in top3]
    return top3

tcols = ['QuestionId', 'arank', 'Perplexity', 'Perplexity_ScaleW', 'Perplexity_ScaleC', 'inference_sum', 'inference_mean', 
       'sentiment_sum', 'sentiment_mean', 'achar_len', 'mnchar_len',
       'mxchar_len', 'aword_len', 'mnword_len', 'mxword_len',
       'rchar_len', '-a_rchar_len', '-mn_rchar_len', '-mx_rchar_len',
       'rword_len', '-a_rword_len', '-mn_rword_len', '-mx_rword_len'] + fe

#model = ensemble.ExtraTreesClassifier(n_estimators=2500, max_depth=15, n_jobs=-1, random_state=3)
#https://www.kaggle.com/code/codingloading/math-aware-tf-idf-thnks2-arnav
params = {
    'objective': 'multi:softprob',
    'num_class': len(le),
    'eval_metric': 'mlogloss',
    'max_depth': 11,
    'learning_rate': 0.02,
    'subsample': 0.85,
    'colsample_bytree': 0.85,
    'tree_method': 'hist', 
    'device': 'cuda',
    'random_state': 42
}

skf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_preds = np.zeros((len(train), len(le)))
preds = np.zeros((len(test), len(le)))

for fold, (trn_idx, val_idx) in enumerate(skf.split(train, train['Category:Misconception'])):
    print(f"\nFold {fold+1}")

    dfFold = pd.DataFrame(train.values[trn_idx], columns=train.columns)
    exp = f_experience(dfFold['StudentExplanation'].values, dfFold['target'].values)
    #remove stopwords
    for sw in es:
        if sw in exp:
            del exp[sw]

    trainx = nlpit(train)
    testx = nlpit(test)
    X = trainx[tcols]
    Y = trainx.target
    
    dtrain = xgb.DMatrix(X.values[trn_idx], label=Y[trn_idx])
    dvalid = xgb.DMatrix(X.values[val_idx], label=Y[val_idx])

    model = xgb.train(params, dtrain, num_boost_round=5000,
                      evals=[(dvalid, 'valid')],
                      early_stopping_rounds=50,
                      verbose_eval=100)
    oof_preds[val_idx] = model.predict(dvalid, iteration_range=(0, model.best_iteration))
    preds += model.predict(xgb.DMatrix(testx[tcols]), iteration_range=(0, model.best_iteration)) / skf.n_splits
    
#model.fit(train[tcols], train.target)
#preds = model.predict_proba(train[tcols])
train['pred'] = gettop3(oof_preds)
map3(train['Category:Misconception'], train['pred'])

model.feature_names = tcols
#xgb.plot_importance(model, max_num_features = 30)

test['Category:Misconception'] = gettop3(preds)
test[['row_id','Category:Misconception']].to_csv('submission_stoic.csv', index=False)

#Ensemble
from collections import defaultdict

def get_top_k_ensemble(l1, l2, l3, l4, k=3):
    list1, list2, list3, list4 = l1.split('|'), l2.split('|'), l3.split('|'), l4.split('|')
    weights = [4, 4, 4, 4]  # độ tin cậy: list1 > list2 > list3 > list4
    lists = [list1, list2, list3, list4]
    score = defaultdict(int)

    for i, lst in enumerate(lists):
        weight = weights[i]
        for rank, item in enumerate(lst):
            score[item] += (len(lst) - rank) * weight

    # Sắp xếp theo điểm giảm dần
    sorted_items = sorted(score.items(), key=lambda x: -x[1])
    return ' '.join([item for item, _ in sorted_items[:k]])

df1 = pd.read_csv('submission_gemma.csv').rename(columns = {'Category:Misconception':'Category:Misconception_gemma'})
df2 = pd.read_csv('submission_ettin.csv').rename(columns = {'Category:Misconception':'Category:Misconception_ettin'})
df3 = pd.read_csv('submission_modern.csv').rename(columns = {'Category:Misconception':'Category:Misconception_modern'})
df4 = pd.read_csv('submission_stoic.csv').rename(columns = {'Category:Misconception':'Category:Misconception_stoic'})

df = pd.merge(df1, df2, on = 'row_id', how = 'inner')
df = pd.merge(df, df3, on = 'row_id', how = 'inner')
df = pd.merge(df, df4, on = 'row_id', how = 'inner')
df['Category:Misconception'] = df.apply(lambda x: get_top_k_ensemble(x['Category:Misconception_gemma'], x['Category:Misconception_ettin'], x['Category:Misconception_modern'], x['Category:Misconception_stoic']), axis = 1)
df[['row_id', 'Category:Misconception']].to_csv('submission.csv', index = False)

print(pd.read_csv('submission.csv'))

2025-08-02 10:10:31.871400: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754129432.044360      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754129432.099138      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Train shape: (36696, 9) with 65 target classes
   row_id  QuestionId                                       QuestionText  \
0       0       31772  What fraction of the shape is not shaded? Give...   
1       1       31772  What fraction of the shape is not shaded? Give...   
2       2       31772  What fraction of the shape is not shaded? Give...   
3       3       31772  What fraction of the shape is not shaded? Give...   
4       4       31772  What fraction of the shape is not shaded? Give...   

           MC_Answer                                 StudentExplanation  \
0  \( \frac{1}{3} \)                  0ne third is equal to tree nineth   
1  \( \frac{1}{3} \)  1 / 3 because 6 over 9 is 2 thirds and 1 third...   
2  \( \frac{1}{3} \)  1 3rd is half of 3 6th, so it is simplee to un...   
3  \( \frac{1}{3} \)        1 goes into everything and 3 goes into nine   
4  \( \frac{1}{3} \)                    1 out of every 3 isn't coloured   

       Category Misconception           targe

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

(3, 5)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Memory allocated: 17039360
Memory reserved: 8992587776


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(



Fold 1
[0]	valid-mlogloss:3.84687
[100]	valid-mlogloss:0.83981
[200]	valid-mlogloss:0.58515
[300]	valid-mlogloss:0.52753
[400]	valid-mlogloss:0.51165
[500]	valid-mlogloss:0.50925
[533]	valid-mlogloss:0.50965

Fold 2
[0]	valid-mlogloss:3.84370
[100]	valid-mlogloss:0.83606
[200]	valid-mlogloss:0.58132
[300]	valid-mlogloss:0.52450
[400]	valid-mlogloss:0.50971
[500]	valid-mlogloss:0.50752
[533]	valid-mlogloss:0.50813

Fold 3
[0]	valid-mlogloss:3.84638
[100]	valid-mlogloss:0.84716
[200]	valid-mlogloss:0.59143
[300]	valid-mlogloss:0.53361
[400]	valid-mlogloss:0.51856
[500]	valid-mlogloss:0.51715
[505]	valid-mlogloss:0.51724

Fold 4
[0]	valid-mlogloss:3.85207
[100]	valid-mlogloss:0.84827
[200]	valid-mlogloss:0.59111
[300]	valid-mlogloss:0.53151
[400]	valid-mlogloss:0.51661
[500]	valid-mlogloss:0.51488
[515]	valid-mlogloss:0.51502

Fold 5
[0]	valid-mlogloss:3.85211
[100]	valid-mlogloss:0.84192
[200]	valid-mlogloss:0.58276
[300]	valid-mlogloss:0.52332
[400]	valid-mlogloss:0.50703
[500]	valid-m